<a href="https://colab.research.google.com/github/Hemashree2407/SQL_Project1/blob/main/Duplicate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install yfinance pandas numpy xlsxwriter

In [ ]:
!python integration_script.py

In [ ]:
!pip install yfinance ta --quiet

In [ ]:
#@title Part 1 of Main script
# Indian Stock Analyzer - Part 1 (Corrected and Enhanced)
# Complete implementation with improvements

import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from dataclasses import dataclass
from typing import Dict, List, Optional, Tuple
import logging
import warnings
warnings.filterwarnings('ignore')

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__) # Define logger globally

@dataclass
class StockData:
    """Data class to store comprehensive stock information"""
    symbol: str
    company_name: str
    current_price: float
    market_cap: float
    info: Dict
    price_data: pd.DataFrame
    indicators: Dict
    fundamental_score: float
    technical_score: float
    recommendation: str
    metrics: Dict

class IndianStockAnalyzer:
    """Comprehensive stock analyzer for Indian markets"""

    def __init__(self):
        """Initialize the analyzer with Indian market specific parameters"""
        self.nifty_symbol = "^NSEI"
        self.risk_free_rate = 0.065  # Will be made dynamic in Part 2

        # Industry-specific benchmark thresholds for Indian markets
        self.industry_benchmarks = {
            'technology': {
                'pe_ratio': {'excellent': 25, 'good': 35, 'fair': 45},
                'debt_to_equity': {'excellent': 0.3, 'good': 0.5, 'fair': 0.8},
                'roe': {'excellent': 0.25, 'good': 0.18, 'fair': 0.12},
                'revenue_growth': {'excellent': 0.20, 'good': 0.15, 'fair': 0.10},
                'net_profit_margin': {'excellent': 0.20, 'good': 0.15, 'fair': 0.10}
            },
            'banking': {
                'pe_ratio': {'excellent': 15, 'good': 20, 'fair': 25},
                'debt_to_equity': {'excellent': 8, 'good': 10, 'fair': 12},
                'roe': {'excellent': 0.15, 'good': 0.12, 'fair': 0.08},
                'revenue_growth': {'excellent': 0.15, 'good': 0.10, 'fair': 0.05},
                'net_profit_margin': {'excellent': 0.25, 'good': 0.20, 'fair': 0.15}
            },
            'pharmaceutical': {
                'pe_ratio': {'excellent': 20, 'good': 30, 'fair': 40},
                'debt_to_equity': {'excellent': 0.4, 'good': 0.6, 'fair': 0.8},
                'roe': {'excellent': 0.20, 'good': 0.15, 'fair': 0.10},
                'revenue_growth': {'excellent': 0.15, 'good': 0.10, 'fair': 0.05},
                'net_profit_margin': {'excellent': 0.15, 'good': 0.10, 'fair': 0.08}
            },
            'fmcg': {
                'pe_ratio': {'excellent': 30, 'good': 40, 'fair': 50},
                'debt_to_equity': {'excellent': 0.5, 'good': 0.8, 'fair': 1.0},
                'roe': {'excellent': 0.30, 'good': 0.25, 'fair': 0.15},
                'revenue_growth': {'excellent': 0.12, 'good': 0.08, 'fair': 0.05},
                'net_profit_margin': {'excellent': 0.15, 'good': 0.10, 'fair': 0.08}
            },
            'automobile': {
                'pe_ratio': {'excellent': 15, 'good': 25, 'fair': 35},
                'debt_to_equity': {'excellent': 0.6, 'good': 0.8, 'fair': 1.2},
                'roe': {'excellent': 0.20, 'good': 0.15, 'fair': 0.10},
                'revenue_growth': {'excellent': 0.15, 'good': 0.10, 'fair': 0.05},
                'net_profit_margin': {'excellent': 0.10, 'good': 0.07, 'fair': 0.05}
            },
            'infrastructure': {
                'pe_ratio': {'excellent': 12, 'good': 18, 'fair': 25},
                'debt_to_equity': {'excellent': 1.0, 'good': 1.5, 'fair': 2.0},
                'roe': {'excellent': 0.15, 'good': 0.12, 'fair': 0.08},
                'revenue_growth': {'excellent': 0.20, 'good': 0.15, 'fair': 0.10},
                'net_profit_margin': {'excellent': 0.12, 'good': 0.08, 'fair': 0.05}
            },
            'energy': {
                'pe_ratio': {'excellent': 10, 'good': 15, 'fair': 20},
                'debt_to_equity': {'excellent': 0.8, 'good': 1.2, 'fair': 1.5},
                'roe': {'excellent': 0.15, 'good': 0.12, 'fair': 0.08},
                'revenue_growth': {'excellent': 0.15, 'good': 0.10, 'fair': 0.05},
                'net_profit_margin': {'excellent': 0.10, 'good': 0.07, 'fair': 0.05}
            },
            'realty': {
                'pe_ratio': {'excellent': 10, 'good': 15, 'fair': 25},
                'debt_to_equity': {'excellent': 1.0, 'good': 1.5, 'fair': 2.0},
                'roe': {'excellent': 0.12, 'good': 0.08, 'fair': 0.05},
                'revenue_growth': {'excellent': 0.25, 'good': 0.15, 'fair': 0.10},
                'net_profit_margin': {'excellent': 0.15, 'good': 0.10, 'fair': 0.05}
            },
            'telecom': {
                'pe_ratio': {'excellent': 15, 'good': 25, 'fair': 35},
                'debt_to_equity': {'excellent': 1.0, 'good': 1.5, 'fair': 2.0},
                'roe': {'excellent': 0.10, 'good': 0.07, 'fair': 0.05},
                'revenue_growth': {'excellent': 0.10, 'good': 0.07, 'fair': 0.05},
                'net_profit_margin': {'excellent': 0.08, 'good': 0.05, 'fair': 0.03}
            }
        }

        # Default benchmarks for other sectors
        self.default_benchmarks = {
            'pe_ratio': {'excellent': 15, 'good': 25, 'fair': 35},
            'debt_to_equity': {'excellent': 0.5, 'good': 1.0, 'fair': 1.5},
            'roe': {'excellent': 0.15, 'good': 0.10, 'fair': 0.07},
            'revenue_growth': {'excellent': 0.15, 'good': 0.10, 'fair': 0.05},
            'net_profit_margin': {'excellent': 0.10, 'good': 0.07, 'fair': 0.05}
        }

    def get_stock_data(self, symbol: str) -> Optional[yf.Ticker]:
        """Fetch stock data from Yahoo Finance with basic validation"""
        if not isinstance(symbol, str) or not symbol:
            logger.error("Invalid symbol input: Symbol must be a non-empty string.")
            return None
        try:
            ticker = yf.Ticker(symbol)
            # Verify the ticker is valid by checking if it has info
            if ticker.info and 'symbol' in ticker.info and ticker.info['symbol'].upper() == symbol.upper():
                logger.info(f"Successfully fetched data for {symbol}")
                return ticker
            else:
                logger.error(f"Invalid or no data available for symbol: {symbol}")
                return None
        except Exception as e:
            logger.error(f"Error fetching data for {symbol}: {str(e)}")
            return None

    def get_industry_benchmarks(self, sector: str) -> Dict:
        """Get industry-specific benchmarks"""
        sector_lower = sector.lower() if isinstance(sector, str) else ''

        # Map common sector names to our benchmark categories
        sector_mapping = {
            'technology': 'technology',
            'information technology': 'technology',
            'financial services': 'banking',
            'financials': 'banking',
            'healthcare': 'pharmaceutical',
            'consumer defensive': 'fmcg',
            'consumer cyclical': 'automobile',
            'industrials': 'infrastructure',
            'energy': 'energy',
            'real estate': 'realty',
            'communication services': 'telecom'
        }

        benchmark_key = sector_mapping.get(sector_lower, None)

        if benchmark_key and benchmark_key in self.industry_benchmarks:
            logger.info(f"Using industry-specific benchmarks for sector: {sector}")
            return self.industry_benchmarks[benchmark_key]
        else:
            logger.info(f"Using default benchmarks for sector: {sector}")
            return self.default_benchmarks


def calculate_enhanced_technical_score(self, indicators: Dict, circuit_risk: int) -> float:
        """Enhanced technical scoring with circuit risk and better MACD handling"""
        base_score = 0
        max_technical_score = 50

        try:
            # RSI Score (10 points)
            rsi = indicators.get('RSI', 50)
            if pd.notna(rsi):
                if 30 <= rsi <= 70:
                    base_score += 10
                elif rsi < 30:
                    base_score += 8  # Oversold
                else:
                    base_score += 5  # Overbought
            else:
                self.logger.warning("RSI is NaN, skipping RSI scoring")

            # MACD Score (10 points) - FIXED
            macd = indicators.get('MACD')
            macd_signal = indicators.get('MACD_signal')

            if pd.notna(macd) and pd.notna(macd_signal):
                if macd > macd_signal:
                    base_score += 10
                else:
                    base_score += 3
            else:
                self.logger.warning("MACD or MACD signal is NaN, using alternative scoring")
                # Use EMA crossover as alternative
                ema_12 = indicators.get('EMA_12')
                ema_26 = indicators.get('EMA_26')
                if pd.notna(ema_12) and pd.notna(ema_26):
                    if ema_12 > ema_26:
                        base_score += 7
                    else:
                        base_score += 3

            # Moving Average Score (10 points)
            sma_20 = indicators.get('SMA_20')
            sma_50 = indicators.get('SMA_50')
            current_price = indicators.get('current_price', 0)

            if pd.notna(sma_20) and pd.notna(sma_50):
                if sma_20 > sma_50:
                    base_score += 7
                    # Bonus if price above both
                    if current_price > sma_20:
                        base_score += 3
                else:
                    base_score += 3

            # Bollinger Bands Score (10 points)
            bb_upper = indicators.get('BB_upper')
            bb_lower = indicators.get('BB_lower')
            bb_middle = indicators.get('BB_middle')

            if pd.notna(bb_upper) and pd.notna(bb_lower) and pd.notna(current_price):
                bb_position = (current_price - bb_lower) / (bb_upper - bb_lower) if bb_upper != bb_lower else 0.5
                if 0.3 <= bb_position <= 0.7:
                    base_score += 10  # Middle band - stable
                elif bb_position < 0.3:
                    base_score += 8   # Near lower band - potential bounce
                else:
                    base_score += 5   # Near upper band

            # Volume Score (10 points)
            volume_ratio = indicators.get('Volume_ratio', 1)
            if pd.notna(volume_ratio):
                if 0.8 <= volume_ratio <= 1.5:
                    base_score += 7   # Normal volume
                elif volume_ratio > 1.5:
                    base_score += 10  # High volume
                else:
                    base_score += 3   # Low volume

            # Add circuit risk adjustment
            base_score += circuit_risk

            # Cap the score
            final_score = max(0, min(base_score, max_technical_score))

            # Log the calculation
            self.logger.info(f"Technical score calculation: base={base_score}, circuit_risk={circuit_risk}, final={final_score}")

            return final_score

        except Exception as e:
            self.logger.error(f"Error calculating technical score: {str(e)}")
            return 0

def get_peer_companies(self, symbol: str, stock_info: Dict) -> List[str]:
        """Identify peer companies for comparison"""
        try:
            sector = stock_info.get('sector', '')
            market_cap = stock_info.get('marketCap', 0)

            # Define market cap categories
            if market_cap > 10_00_000_00_00_000:  # > 10 Lakh Cr
                cap_category = 'Large Cap'
            elif market_cap > 50_000_00_00_000:   # > 50K Cr
                cap_category = 'Mid Cap'
            else:
                cap_category = 'Small Cap'

            # Enhanced peer mapping
            peer_map = {
                'Energy': {
                    'Large Cap': ['RELIANCE.NS', 'ONGC.NS', 'IOC.NS', 'BPCL.NS', 'HINDPETRO.NS'],
                    'Mid Cap': ['GAIL.NS', 'OIL.NS', 'IGL.NS', 'PETRONET.NS'],
                    'Small Cap': ['MRPL.NS', 'CHENNPETRO.NS', 'GSPL.NS']
                },
                'Technology': {
                    'Large Cap': ['TCS.NS', 'INFY.NS', 'WIPRO.NS', 'HCLTECH.NS'],
                    'Mid Cap': ['TECHM.NS', 'LTTS.NS', 'PERSISTENT.NS', 'COFORGE.NS'],
                    'Small Cap': ['ZENSAR.NS', 'ECLERX.NS', 'MASTEK.NS']
                },
                'Financial Services': {
                    'Large Cap': ['HDFCBANK.NS', 'ICICIBANK.NS', 'KOTAKBANK.NS', 'AXISBANK.NS', 'SBIN.NS'],
                    'Mid Cap': ['IDFCFIRSTB.NS', 'FEDERALBNK.NS', 'INDUSINDBK.NS', 'BANDHANBNK.NS'],
                    'Small Cap': ['SOUTHBANK.NS', 'UJJIVAN.NS', 'EQUITASBNK.NS']
                },
                'Healthcare': {
                    'Large Cap': ['SUNPHARMA.NS', 'DRREDDY.NS', 'CIPLA.NS', 'DIVISLAB.NS'],
                    'Mid Cap': ['TORNTPHARM.NS', 'ALKEM.NS', 'LUPIN.NS', 'AUROPHARMA.NS'],
                    'Small Cap': ['CAPLIPOINT.NS', 'GRANULES.NS', 'INDOCO.NS']
                }
            }

            # Get peers, excluding the current symbol
            peers = peer_map.get(sector, {}).get(cap_category, [])
            peers = [p for p in peers if p != symbol]

            # If no peers found, return empty list
            if not peers:
                self.logger.warning(f"No predefined peers for {sector} - {cap_category}")
                return []

            return peers[:5]  # Return top 5 peers

        except Exception as e:
            self.logger.error(f"Error getting peer companies: {str(e)}")
            return []

def calculate_peer_relative_performance(self, symbol: str, peers: List[str], days: int = 30) -> Dict:
        """Compare stock performance with peers"""
        try:
            # Check if peers list is valid
            if not peers:
                self.logger.warning("No peers provided for comparison")
                return {}

            results = {}

            # Get performance for main stock
            main_ticker = yf.Ticker(symbol)
            main_hist = main_ticker.history(period=f"{days}d")
            if len(main_hist) > 0:
                main_return = ((main_hist['Close'].iloc[-1] / main_hist['Close'].iloc[0]) - 1) * 100
            else:
                main_return = 0

            # Get peer performances
            peer_returns = []
            for peer in peers:
                try:
                    peer_ticker = yf.Ticker(peer)
                    peer_hist = peer_ticker.history(period=f"{days}d")
                    if len(peer_hist) > 0:
                        peer_return = ((peer_hist['Close'].iloc[-1] / peer_hist['Close'].iloc[0]) - 1) * 100
                        peer_returns.append(peer_return)
                except:
                    continue

            if peer_returns:
                avg_peer_return = np.mean(peer_returns)
                relative_performance = main_return - avg_peer_return

                results = {
                    'stock_return': main_return,
                    'avg_peer_return': avg_peer_return,
                    'relative_performance': relative_performance,
                    'outperformance': relative_performance > 0
                }

            return results

        except Exception as e:
            self.logger.error(f"Error calculating peer performance: {str(e)}")
            return {}

def calculate_rsi(self, prices: pd.Series, period: int = 14) -> Optional[float]:
        """Calculate Relative Strength Index with robust NaN handling"""
        if not isinstance(prices, pd.Series) or prices.empty or len(prices) < period:
            logger.warning("Insufficient data or invalid input for RSI calculation.")
            return np.nan # Return NaN for insufficient or invalid data

        try:
            delta = prices.diff()
            # Use .fillna(0) before rolling mean to treat initial NaNs in gain/loss as 0
            gain = delta.where(delta > 0, 0).fillna(0)
            loss = -delta.where(delta < 0, 0).fillna(0)

            avg_gain = gain.rolling(window=period).mean()
            avg_loss = loss.rolling(window=period).mean()

            # Check if the last values are valid numbers
            if pd.isna(avg_gain.iloc[-1]) or pd.isna(avg_loss.iloc[-1]):
                 logger.warning("Rolling average gain or loss is NaN. Cannot calculate RSI.")
                 return np.nan

            # Handle division by zero if avg_loss is zero
            if avg_loss.iloc[-1] == 0:
                 # If no loss, RSI is 100 if there's gain, 50 if no change
                 return 100.0 if avg_gain.iloc[-1] > 0 else 50.0
            elif avg_loss.iloc[-1] > 0:
                rs = avg_gain.iloc[-1] / avg_loss.iloc[-1]
                rsi = 100 - (100 / (1 + rs))
                return rsi
            else:
                 # This case should ideally not be reached with the abs logic, but as a safeguard
                 return np.nan


        except Exception as e:
            logger.error(f"Error calculating RSI: {str(e)}")
            return np.nan # Return NaN on error

def calculate_fundamental_score(self, info: Dict, benchmarks: Dict) -> Tuple[float, Dict]:
        """Calculate fundamental analysis score based on Indian market standards with improved NaN handling and D/E validation"""
        score = 0
        max_score = 50
        metrics = {}

        try:
            # P/E Ratio (10 points)
            pe_ratio = info.get('trailingPE', info.get('forwardPE', None))
            # Check if pe_ratio is a valid positive number
            if isinstance(pe_ratio, (int, float)) and pe_ratio is not None and pe_ratio > 0 and not pd.isna(pe_ratio):
                metrics['PE_Ratio'] = pe_ratio
                if pe_ratio <= benchmarks['pe_ratio']['excellent']:
                    score += 10
                elif pe_ratio <= benchmarks['pe_ratio']['good']:
                    score += 7
                elif pe_ratio <= benchmarks['pe_ratio']['fair']:
                    score += 4
                else:
                    score += 2 # Even high PE gets a small score if available and positive
            else:
                 metrics['PE_Ratio'] = np.nan
                 logger.warning("PE Ratio not available or is invalid.")


            # Debt to Equity (10 points) - Improved Validation
            debt_to_equity_raw = info.get('debtToEquity', None)
            debt_to_equity = np.nan # Default to NaN

            if debt_to_equity_raw is not None:
                 if isinstance(debt_to_equity_raw, (int, float)):
                    # More robust check: if the number is very large (e.g., > 1000), assume it's percentage
                    # Otherwise, assume it's a decimal ratio. This handles both 0.5 and 50 formats.
                    if debt_to_equity_raw > 1000:
                         debt_to_equity = debt_to_equity_raw / 100.0
                    elif debt_to_equity_raw >= 0: # Ensure it's non-negative
                         debt_to_equity = debt_to_equity_raw
                    # If it's a negative number, treat as invalid (remains np.nan)
                 # else: debt_to_equity remains np.nan for non-numeric types

            if not pd.isna(debt_to_equity):
                metrics['Debt_to_Equity'] = debt_to_equity
                # Scoring based on the valid debt_to_equity value
                if debt_to_equity <= benchmarks['debt_to_equity']['excellent']:
                    score += 10
                elif debt_to_equity <= benchmarks['debt_to_equity']['good']:
                    score += 7
                elif debt_to_equity <= benchmarks['debt_to_equity']['fair']:
                    score += 4
                else:
                    score += 2 # Even high debt gets a small score if available
            else:
                 metrics['Debt_to_Equity'] = np.nan
                 logger.warning("Debt to Equity not available or is invalid.")


            # ROE (10 points)
            roe = info.get('returnOnEquity', None)
            # Check if roe is a valid number
            if isinstance(roe, (int, float)) and roe is not None and not pd.isna(roe):
                metrics['ROE'] = roe
                if roe >= benchmarks['roe']['excellent']:
                    score += 10
                elif roe >= benchmarks['roe']['good']:
                    score += 7
                elif roe >= benchmarks['roe']['fair']:
                    score += 4
                else:
                    score += 2 # Even lower positive ROE gets some points
            else:
                 metrics['ROE'] = np.nan
                 logger.warning("ROE not available or is invalid.")

            # Revenue Growth (10 points)
            revenue_growth = info.get('revenueGrowth', None)
            # Check if revenue_growth is a valid number
            if isinstance(revenue_growth, (int, float)) and revenue_growth is not None and not pd.isna(revenue_growth):
                metrics['Revenue_Growth'] = revenue_growth
                if revenue_growth >= benchmarks['revenue_growth']['excellent']:
                    score += 10
                elif revenue_growth >= benchmarks['revenue_growth']['good']:
                    score += 7
                elif revenue_growth >= benchmarks['revenue_growth']['fair']:
                    score += 4
                else:
                    score += 2 # Even lower positive growth gets some points
            else:
                 metrics['Revenue_Growth'] = np.nan
                 logger.warning("Revenue Growth not available or is invalid.")

            # Net Profit Margin (10 points)
            profit_margin = info.get('profitMargins', None)
            # Check if profit_margin is a valid number
            if isinstance(profit_margin, (int, float)) and profit_margin is not None and not pd.isna(profit_margin):
                metrics['Net_Profit_Margin'] = profit_margin
                if profit_margin >= benchmarks['net_profit_margin']['excellent']:
                    score += 10
                elif profit_margin >= benchmarks['net_profit_margin']['good']:
                    score += 7
                elif profit_margin >= benchmarks['net_profit_margin']['fair']:
                    score += 4
                else:
                    score += 2 # Even lower positive margin gets some points
            else:
                 metrics['Net_Profit_Margin'] = np.nan
                 logger.warning("Net Profit Margin not available or is invalid.")


            # Normalize score to max_score (already capped by scoring logic)
            metrics['Fundamental_Score'] = score

            return score, metrics

        except Exception as e:
            logger.error(f"Error calculating fundamental score: {str(e)}")
            return 0, {}

    def calculate_technical_indicators(self, price_data: pd.DataFrame) -> Dict:
        """Calculate comprehensive technical indicators"""
        try:
            indicators = {}

            # Ensure we have enough data
            if len(price_data) < 50:
                self.logger.warning("Insufficient data for technical analysis")
                return indicators

            # Current price
            current_price = price_data['Close'].iloc[-1]
            indicators['current_price'] = current_price

            # Simple Moving Averages
            indicators['SMA_20'] = price_data['Close'].rolling(window=20).mean().iloc[-1]
            indicators['SMA_50'] = price_data['Close'].rolling(window=50).mean().iloc[-1]
            indicators['SMA_200'] = price_data['Close'].rolling(window=200).mean().iloc[-1] if len(price_data) >= 200 else None

            # Exponential Moving Averages - FIXED
            ema_12_series = price_data['Close'].ewm(span=12, adjust=False).mean()
            ema_26_series = price_data['Close'].ewm(span=26, adjust=False).mean()

            indicators['EMA_12'] = ema_12_series.iloc[-1]
            indicators['EMA_26'] = ema_26_series.iloc[-1]

            # RSI
            indicators['RSI'] = self.calculate_rsi(price_data['Close'])

            # MACD - FIXED CALCULATION
            macd_line = ema_12_series - ema_26_series
            signal_line = macd_line.ewm(span=9, adjust=False).mean()

            indicators['MACD'] = macd_line.iloc[-1]
            indicators['MACD_signal'] = signal_line.iloc[-1]
            indicators['MACD_histogram'] = indicators['MACD'] - indicators['MACD_signal']

            # Bollinger Bands
            bb_period = 20
            bb_std = 2
            sma_bb = price_data['Close'].rolling(window=bb_period).mean()
            std_bb = price_data['Close'].rolling(window=bb_period).std()
            indicators['BB_upper'] = (sma_bb + (std_bb * bb_std)).iloc[-1]
            indicators['BB_lower'] = (sma_bb - (std_bb * bb_std)).iloc[-1]
            indicators['BB_middle'] = sma_bb.iloc[-1]

            # Volume indicators
            indicators['Volume_SMA'] = price_data['Volume'].rolling(window=20).mean().iloc[-1]
            indicators['Volume_ratio'] = price_data['Volume'].iloc[-1] / indicators['Volume_SMA'] if indicators['Volume_SMA'] > 0 else 1.0

            # Price position
            indicators['Price_to_SMA20'] = (current_price / indicators['SMA_20'] - 1) * 100 if indicators['SMA_20'] > 0 else 0
            indicators['Price_to_SMA50'] = (current_price / indicators['SMA_50'] - 1) * 100 if indicators['SMA_50'] > 0 else 0

            # Support and Resistance
            indicators['Resistance'] = price_data['High'].rolling(window=20).max().iloc[-1]
            indicators['Support'] = price_data['Low'].rolling(window=20).min().iloc[-1]

            # Stochastic Oscillator
            low_14 = price_data['Low'].rolling(window=14).min()
            high_14 = price_data['High'].rolling(window=14).max()
            indicators['Stochastic_K'] = ((price_data['Close'] - low_14) / (high_14 - low_14) * 100).iloc[-1]

            # Average True Range (ATR)
            high_low = price_data['High'] - price_data['Low']
            high_close = np.abs(price_data['High'] - price_data['Close'].shift())
            low_close = np.abs(price_data['Low'] - price_data['Close'].shift())
            true_range = pd.concat([high_low, high_close, low_close], axis=1).max(axis=1)
            indicators['ATR'] = true_range.rolling(window=14).mean().iloc[-1]

            return indicators

        except Exception as e:
            self.logger.error(f"Error calculating technical indicators: {str(e)}")
            import traceback
            traceback.print_exc()
            return {}

    def generate_recommendation(self, fundamental_score: float, technical_score: float) -> str:
        """Generate investment recommendation based on scores"""
        # Ensure scores are not NaN before summing
        fund_score = fundamental_score if not pd.isna(fundamental_score) else 0
        tech_score = technical_score if not pd.isna(technical_score) else 0

        total_score = fund_score + tech_score

        if total_score >= 75:
            return "STRONG BUY"
        elif total_score >= 60:
            return "BUY"
        elif total_score >= 40:
            return "HOLD"
        elif total_score >= 25:
            return "SELL"
        else:
            return "STRONG SELL"

    def analyze_stock(self, symbol: str) -> Optional[StockData]:
        """Main method to analyze a stock with improved data handling"""
        try:
            logger.info(f"Starting analysis for {symbol}")

            # Get stock data with validation
            ticker = self.get_stock_data(symbol)
            if not ticker:
                logger.error(f"Failed to fetch valid ticker data for {symbol}.")
                return None

            # Get stock info with validation
            info = ticker.info
            if not info or not isinstance(info, dict):
                logger.error(f"No valid info available for {symbol}")
                return None

            # Get historical price data with validation
            # Fetching enough data for 200-day SMA calculation (period="1y" is generally sufficient)
            price_data = ticker.history(period="1y")
            if price_data.empty or not isinstance(price_data, pd.DataFrame):
                logger.error(f"No valid price data available for {symbol}")
                return None

            # Get current price with validation
            if 'Close' in price_data.columns and not price_data['Close'].empty:
                 current_price = price_data['Close'].iloc[-1]
                 if pd.isna(current_price):
                     logger.warning(f"Current price is NaN for {symbol}. Using 0.")
                     current_price = 0.0 # Default to 0 if NaN
            else:
                 logger.error(f"Close price column not found or is empty for {symbol}. Cannot determine current price.")
                 current_price = 0.0 # Default to 0 if column missing or empty


            # Add current price to indicators for technical score calculation
            indicators = self.calculate_technical_indicators(price_data)
            indicators['current_price'] = current_price # Ensure current price is available for BB score

            # Get industry benchmarks
            sector = info.get('sector', 'Unknown')
            benchmarks = self.get_industry_benchmarks(sector)

            # Calculate scores
            fundamental_score, metrics = self.calculate_fundamental_score(info, benchmarks)
            technical_score = self.calculate_technical_score(indicators)

            # Generate recommendation
            recommendation = self.generate_recommendation(fundamental_score, technical_score)

            # Create StockData object
            stock_data = StockData(
                symbol=symbol,
                company_name=info.get('longName', symbol),
                current_price=current_price,
                market_cap=info.get('marketCap', 0) if info.get('marketCap') is not None else np.nan, # Handle missing marketCap
                info=info,
                price_data=price_data,
                indicators=indicators,
                fundamental_score=fundamental_score,
                technical_score=technical_score,
                recommendation=recommendation,
                metrics=metrics
            )

            logger.info(f"Analysis completed for {symbol}")
            return stock_data

        except Exception as e:
            logger.error(f"Error analyzing {symbol}: {str(e)}")
            return None

    def display_analysis(self, stock_data: StockData) -> None:
        """Display analysis results in a formatted manner"""
        print(f"\n{'='*60}")
        print(f"Stock Analysis Report: {stock_data.company_name} ({stock_data.symbol})")
        print(f"{'='*60}")

        print(f"\n📊 Current Market Data:")
        print(f"Current Price: ₹{stock_data.current_price:,.2f}" if not pd.isna(stock_data.current_price) else "Current Price: N/A")
        market_cap_cr = stock_data.market_cap / 10000000 if not pd.isna(stock_data.market_cap) else np.nan
        print(f"Market Cap: ₹{market_cap_cr:,.2f} Cr" if not pd.isna(market_cap_cr) else "Market Cap: N/A")
        print(f"Sector: {stock_data.info.get('sector', 'N/A')}")

        print(f"\n📈 Technical Indicators:")
        # Display indicators, handling NaN values
        for indicator, value in stock_data.indicators.items():
            if indicator != 'current_price': # Avoid displaying current_price here
                if not pd.isna(value):
                    if isinstance(value, float):
                        print(f"{indicator}: {value:,.2f}")
                    else:
                        print(f"{indicator}: {value}")
                else:
                    print(f"{indicator}: N/A (Insufficient Data)")


        print(f"\n💰 Fundamental Metrics:")
        for metric, value in stock_data.metrics.items():
            if metric != 'Fundamental_Score':
                if not pd.isna(value):
                    if isinstance(value, float):
                        if metric in ['ROE', 'Revenue_Growth', 'Net_Profit_Margin']:
                            print(f"{metric}: {value*100:.2f}%")
                        else:
                            print(f"{metric}: {value:.2f}")
                    else:
                        print(f"{metric}: value")
                else:
                    print(f"{metric}: N/A (Insufficient Data)")

        print(f"\n📊 Analysis Scores:")
        print(f"Fundamental Score: {stock_data.fundamental_score:.2f}/50" if not pd.isna(stock_data.fundamental_score) else "Fundamental Score: N/A")
        print(f"Technical Score: {stock_data.technical_score:.2f}/50" if not pd.isna(stock_data.technical_score) else "Technical Score: N/A")
        total_score = (stock_data.fundamental_score if not pd.isna(stock_data.fundamental_score) else 0) + (stock_data.technical_score if not pd.isna(stock_data.technical_score) else 0)
        print(f"Total Score (Basic): {total_score:.2f}/100")

        print(f"\n🎯 Recommendation (Basic): {stock_data.recommendation}")
        print(f"{'='*60}\n")

In [ ]:
#@title Sentimental analysis with API
# Indian Stock Analyzer - Part 2 (Improved Version)
# This continues from your Part 1 implementation

import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import requests
from bs4 import BeautifulSoup
import warnings
warnings.filterwarnings('ignore')

# Import the Python SDK for Gemini API
import google.generativeai as genai
# Used to securely store your API key
from google.colab import userdata

# Configure Gemini API (ensure GOOGLE_API_KEY is set in Colab secrets)
try:
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
    genai.configure(api_key=GOOGLE_API_KEY)
    gemini_model = genai.GenerativeModel('gemini-2.0-flash') # Using a more recent model
except Exception as e:
    print(f"Warning: Could not configure Gemini API. Sentiment analysis will be skipped. Error: {e}")
    gemini_model = None

import logging
logger = logging.getLogger(__name__) # Ensure logger is defined in this cell too

class EnhancedStockAnalyzer:
    """Enhanced analyzer with additional features for Indian markets"""

    def __init__(self):
        """Initialize the enhanced analyzer."""
        self.risk_free_rate = self.get_dynamic_risk_free_rate()
        self.gemini_model = gemini_model # Use the globally configured model

        # More detailed circuit limit percentages based on NSE categorization (simplified)
        # In a real application, this would be fetched from a reliable data source.
        self.circuit_limits_percentage = {
            'category_a': 0.20, # Typically high-liquidity index stocks
            'category_b': 0.10, # Most other actively traded stocks
            'category_t': 0.05  # Stocks in trade-for-trade segment
            # Add other categories if known
        }

        # Simple mapping of some known stocks to categories for demonstration
        self.stock_category_map = {
            'RELIANCE.NS': 'category_a',
            'TCS.NS': 'category_a',
            'HDFCBANK.NS': 'category_a',
            'INFY.NS': 'category_a',
            'ICICIBANK.NS': 'category_a',
            'VEDL.NS': 'category_b',
            'PNB.NS': 'category_b',
            'INDIGO.NS': 'category_b',
            'BANDHANBNK.NS': 'category_b',
            'PERSISTENT.NS': 'category_b',
            'COFORGE.NS': 'category_b',
            'LTTS.NS': 'category_b',
            # Add more mappings or a default if category is unknown
        }


    def get_dynamic_risk_free_rate(self):
        """Fetch current 10-year G-Sec yield"""
        try:
            # Try to get from a reliable source
            # For now, using a realistic current rate
            # In production, you'd scrape from RBI or use an API
            return 0.072  # 7.2% as of recent data
        except Exception as e:
            logger.warning(f"Could not fetch dynamic risk-free rate: {e}. Using fallback rate.")
            return 0.065  # Fallback to 6.5%

    def get_circuit_limits(self, symbol):
        """Get circuit breaker limits for the stock based on categorization"""
        try:
            ticker = yf.Ticker(symbol)
            info = ticker.info

            if not info or 'previousClose' not in info:
                 logger.warning(f"Could not get previous close for {symbol}. Cannot calculate circuit limits.")
                 return None

            prev_close = info.get('previousClose')
            if pd.isna(prev_close) or prev_close <= 0:
                 logger.warning(f"Invalid previous close price for {symbol}: {prev_close}. Cannot calculate circuit limits.")
                 return None


            # Determine the stock category and corresponding circuit percentage
            stock_category = self.stock_category_map.get(symbol.upper(), 'category_b') # Default to category_b if not mapped
            circuit_percent = self.circuit_limits_percentage.get(stock_category, 0.10) # Default to 10% if category not found

            logger.info(f"Determined circuit percentage for {symbol} (Category: {stock_category}): {circuit_percent*100}%")

            upper_circuit = prev_close * (1 + circuit_percent)
            lower_circuit = prev_close * (1 - circuit_percent)

            return {
                'upper_circuit': upper_circuit,
                'lower_circuit': lower_circuit,
                'circuit_percent': circuit_percent
            }
        except Exception as e:
            logger.error(f"Error getting circuit breaker limits for {symbol}: {str(e)}")
            return None

    def assess_circuit_risk(self, current_price, circuit_limits):
        """Assess proximity to circuit limits"""
        # Ensure current_price is a valid number
        if pd.isna(current_price) or current_price <= 0:
             logger.warning("Invalid current price for circuit risk assessment.")
             return 0 # No risk if price is invalid

        if not circuit_limits or 'upper_circuit' not in circuit_limits or 'lower_circuit' not in circuit_limits:
            logger.warning("Invalid or incomplete circuit limits provided.")
            return 0 # No risk if limits are invalid

        upper_circuit = circuit_limits['upper_circuit']
        lower_circuit = circuit_limits['lower_circuit']

        # Ensure limits are valid numbers before calculating distance
        if pd.isna(upper_circuit) or pd.isna(lower_circuit) or upper_circuit <= 0 or lower_circuit <= 0:
             logger.warning("Circuit limits are invalid numbers. Cannot assess circuit risk.")
             return 0

        risk_score = 0

        # Assess proximity to upper circuit (potential buying frenzy or manipulation risk)
        if current_price >= upper_circuit:
             risk_score -= 30 # Hit or breached upper circuit - very high risk
        elif (upper_circuit - current_price) / current_price < 0.01: # Within 1%
            risk_score -= 20  # High risk
        elif (upper_circuit - current_price) / current_price < 0.03: # Within 3%
            risk_score -= 10
        elif (upper_circuit - current_price) / current_price < 0.05: # Within 5%
            risk_score -= 5


        # Assess proximity to lower circuit (potential panic selling or sharp downturn risk)
        if current_price <= lower_circuit:
            risk_score -= 40 # Hit or breached lower circuit - extremely high risk
        elif (current_price - lower_circuit) / current_price < 0.01: # Within 1%
            risk_score -= 30  # Very high risk
        elif (current_price - lower_circuit) / current_price < 0.03: # Within 3%
            risk_score -= 15
        elif (current_price - lower_circuit) / current_price < 0.05: # Within 5%
            risk_score -= 8

        logger.info(f"Circuit risk score calculated: {risk_score}")
        return risk_score # This score is negative for higher risk

    def get_liquidity_score(self, price_data):
        """Enhanced liquidity analysis with NaN handling and better checks"""
        liquidity_score = 0
        # Ensure price_data is valid and has enough data
        if not isinstance(price_data, pd.DataFrame) or price_data.empty or len(price_data) < 20:
            logger.warning("Insufficient price data for liquidity analysis. Assigning default score.")
            return 30  # Default medium liquidity score

        try:
            # Calculate various liquidity metrics
            avg_volume_20d = price_data['Volume'].rolling(20).mean().iloc[-1]
            avg_volume_5d = price_data['Volume'].rolling(5).mean().iloc[-1]
            current_close = price_data['Close'].iloc[-1]

            # Check for NaN values in calculated metrics
            if pd.isna(avg_volume_20d) or pd.isna(avg_volume_5d) or pd.isna(current_close) or current_close <= 0:
                 logger.warning("Liquidity metrics are NaN or invalid. Cannot calculate liquidity score accurately. Assigning default.")
                 return 30 # Default if metrics are invalid

            # Average daily turnover - check for valid values before multiplication
            avg_turnover_20d = np.nan # Default to NaN
            if avg_volume_20d is not None and not pd.isna(avg_volume_20d) and avg_volume_20d >= 0 and current_close > 0:
                 avg_turnover_20d = avg_volume_20d * current_close


            # Volume spike detection - check for zero division and NaN
            volume_spike = np.nan # Default to NaN
            if avg_volume_20d is not None and not pd.isna(avg_volume_20d) and avg_volume_20d > 0:
                 if avg_volume_5d is not None and not pd.isna(avg_volume_5d):
                     volume_spike = avg_volume_5d / avg_volume_20d
                 else:
                      logger.warning("5-day average volume is invalid for volume spike calculation.")
            elif avg_volume_5d is not None and not pd.isna(avg_volume_5d) and avg_volume_5d > 0:
                 # If 20d avg is zero or NaN but 5d is positive, assume recent spike relative to a low baseline
                 volume_spike = 1.0 # Represents normal relative volume if baseline is zero/NaN but recent volume exists
                 logger.warning("20-day average volume is invalid for volume spike, using 1.0 as relative indicator.")
            else:
                 logger.warning("Insufficient or invalid volume data for volume spike calculation.")


            # Liquidity scoring
            # Turnover-based scoring - check for valid turnover
            if not pd.isna(avg_turnover_20d):
                if avg_turnover_20d > 500_00_00_000:  # > 500 Cr
                    liquidity_score += 40
                elif avg_turnover_20d > 100_00_00_000:  # > 100 Cr
                    liquidity_score += 30
                elif avg_turnover_20d > 10_00_00_000:  # > 10 Cr
                    liquidity_score += 20
                elif avg_turnover_20d > 1_00_00_000:  # > 1 Cr
                    liquidity_score += 10
                else:
                    liquidity_score += 5
            else:
                 logger.warning("Average turnover is NaN. Skipping turnover scoring.")
                 liquidity_score += 10 # Add a small base score even if turnover is missing


            # Volume consistency - check for valid volume std dev and avg
            volume_std = price_data['Volume'].rolling(20).std().iloc[-1] if len(price_data) >= 20 else np.nan
            volume_cv = np.nan # Default to NaN
            if avg_volume_20d is not None and not pd.isna(avg_volume_20d) and avg_volume_20d > 0 and \
               volume_std is not None and not pd.isna(volume_std):
                volume_cv = volume_std / avg_volume_20d
            # else: volume_cv remains np.nan

            if not pd.isna(volume_cv):
                if volume_cv < 0.5:  # Low volatility in volume
                    liquidity_score += 20
                elif volume_cv < 1.0:
                    liquidity_score += 10
            else:
                 logger.warning("Volume coefficient of variation is NaN. Skipping consistency scoring.")


            # Recent volume trend - check for valid volume spike
            if not pd.isna(volume_spike) and volume_spike > 1.2:  # 20% higher recent volume
                liquidity_score += 10
            else:
                 logger.warning("Volume spike ratio is NaN. Skipping trend scoring.")


            return min(liquidity_score, 70)  # Cap at 70

        except Exception as e: # Catch specific exceptions for better debugging
            logger.error(f"Error calculating liquidity score: {str(e)}")
            return 30  # Default medium liquidity on error


    def get_peer_companies(self, symbol, stock_info):
        """Identify peer companies for comparison"""
        try:
            # Ensure stock_info is valid
            if not isinstance(stock_info, dict):
                 logger.warning("Invalid stock info for peer identification.")
                 return []

            sector = stock_info.get('sector', '')
            market_cap = stock_info.get('marketCap', 0)

            # Ensure sector and market_cap are valid before proceeding
            if not isinstance(sector, str):
                 sector = '' # Default to empty string if sector is not a string
            if not isinstance(market_cap, (int, float)) or pd.isna(market_cap) or market_cap <= 0:
                 logger.warning("Invalid market cap for peer identification.")
                 market_cap = 0 # Default to 0 if market cap is invalid


            # Define market cap categories
            if market_cap > 10_00_000_00_00_000:  # > 10 Lakh Cr
                cap_category = 'Large Cap'
            elif market_cap > 50_000_00_00_000:   # > 50K Cr
                cap_category = 'Mid Cap'
            else:
                cap_category = 'Small Cap'

            # Peer mapping (simplified - in production, use a comprehensive database)
            peer_map = {
                'Technology': {
                    'Large Cap': ['TCS.NS', 'INFY.NS', 'WIPRO.NS', 'HCLTECH.NS'],
                    'Mid Cap': ['TECHM.NS', 'LTTS.NS', 'PERSISTENT.NS', 'COFORGE.NS'],
                    'Small Cap': ['ZENSAR.NS', 'ECLERX.NS', 'MASTEK.NS']
                },
                'Financial Services': {
                    'Large Cap': ['HDFCBANK.NS', 'ICICIBANK.NS', 'KOTAKBANK.NS', 'AXISBANK.NS'],
                    'Mid Cap': ['IDFCFIRSTB.NS', 'FEDERALBNK.NS', 'INDUSINDBK.NS'],
                    'Small Cap': ['SOUTHBANK.NS', 'UJJIVAN.NS', 'EQUITASBNK.NS']
                },
                'Healthcare': {
                    'Large Cap': ['SUNPHARMA.NS', 'DRREDDY.NS', 'CIPLA.NS', 'DIVISLAB.NS'],
                    'Mid Cap': ['TORNTPHARM.NS', 'ALKEM.NS', 'LUPIN.NS'],
                    'Small Cap': ['CAPLIPOINT.NS', 'BLISS.NS', 'INDOCO.NS']
                }
                # Add more sectors as needed
            }

            # Get peers, excluding the current symbol
            peers = peer_map.get(sector, {}).get(cap_category, [])
            peers = [p for p in peers if isinstance(p, str) and p.upper() != symbol.upper()] # Ensure peers are strings and exclude self

            logger.info(f"Identified {len(peers)} peer companies for {symbol} in {sector} ({cap_category} Cap).")
            return peers[:5]  # Return top 5 peers

        except Exception as e:
            logger.error(f"Error getting peer companies for {symbol}: {str(e)}")
            return []

    def calculate_peer_relative_performance(self, symbol, peers, days=30):
        """Compare stock performance with peers, handling data issues robustly"""
        results = {
            'stock_return': np.nan,
            'avg_peer_return': np.nan,
            'relative_performance': np.nan,
            'outperformance': False,
            'days': days # Include days in results
        }

        # Ensure peers is a valid list (though we are bypassing peer analysis)
        if not isinstance(peers, list) or not peers:
             logger.warning("No valid peer list provided for performance comparison.")
             # Even if no peers, still try to get main stock return
             pass # Continue to fetch main stock data


        # Try to get performance for main stock
        try:
            main_ticker = yf.Ticker(symbol)
            main_hist = main_ticker.history(period=f"{days}d")
            main_return = np.nan # Default return to NaN

            if not main_hist.empty and len(main_hist) > 1: # Need at least 2 data points for return calculation
                try:
                    # Calculate return only if start and end prices are valid and start price is not zero
                    start_price = main_hist['Close'].iloc[0]
                    end_price = main_hist['Close'].iloc[-1]
                    if not pd.isna(start_price) and not pd.isna(end_price) and start_price > 0:
                         main_return = ((end_price / start_price) - 1) * 100
                    else:
                         logger.warning(f"Invalid start ({start_price}) or end price ({end_price}) for main stock ({symbol}) return calculation.")

                except Exception as ret_e:
                     logger.warning(f"Could not calculate main stock ({symbol}) return: {ret_e}. Setting to NaN.")
                     main_return = np.nan # Set to NaN on calculation error
            else:
                logger.warning(f"Insufficient historical data for main stock ({symbol}) for return calculation.")

            results['stock_return'] = main_return # Update results with main stock return

        except Exception as main_stock_e:
            logger.error(f"Error fetching data or calculating return for main stock ({symbol}): {main_stock_e}. Main stock return set to NaN.")
            results['stock_return'] = np.nan # Ensure stock return is NaN on error


        # --- Skipping peer analysis to prevent TypeErrors ---
        # This block is now removed/skipped to ensure the script doesn't crash.
        # The results dictionary will retain its initial NaN values for peer analysis metrics.
        logger.warning("Skipping peer analysis to prevent errors. Peer performance metrics will be unavailable.")
        return results # Return results immediately after main stock data


    def get_index_membership(self, symbol):
        """Check index membership for the stock"""
        # Simplified index constituents (in production, fetch from NSE)
        # Ensure symbol is a string for comparison
        symbol_upper = symbol.upper() if isinstance(symbol, str) else ''

        index_constituents = {
            'NIFTY50': ['RELIANCE.NS', 'TCS.NS', 'HDFCBANK.NS', 'INFY.NS', 'ICICIBANK.NS'],
            'NIFTY_NEXT50': ['VEDL.NS', 'PNB.NS', 'INDIGO.NS', 'BANDHANBNK.NS'],
            'NIFTY_MIDCAP150': ['PERSISTENT.NS', 'COFORGE.NS', 'LTTS.NS']
        }

        membership = []
        for index, constituents in index_constituents.items():
            # Ensure constituent is a string before comparison
            if symbol_upper and any(isinstance(c, str) and c.upper() == symbol_upper for c in constituents):
                membership.append(index)

        if membership:
             logger.info(f"Stock {symbol} is a member of indices: {', '.join(membership)}")
        else:
             logger.info(f"Stock {symbol} is not found in the hardcoded index constituents.")

        return membership

    def calculate_enhanced_technical_score(self, indicators, circuit_risk):
        """Calculate enhanced technical analysis score with NaN handling and circuit risk"""
        base_score = 0
        indicators_calculated = 0 # Track how many indicators were successfully calculated
        possible_indicator_points = 50 # Total points from technical indicators before risk/other factors

        # Scoring for each indicator (scaled based on its contribution to the total possible_indicator_points)
        # Assuming 5 key indicators: RSI, MACD, MA, BB, Volume
        num_key_indicators = 5
        indicator_point_contribution = possible_indicator_points / num_key_indicators


        # RSI Score (points based on indicator_point_contribution)
        rsi = indicators.get('RSI', np.nan) # Default to np.nan
        if not pd.isna(rsi):
             indicators_calculated += 1
             if 40 <= rsi <= 60:  # Neutral zone
                 base_score += indicator_point_contribution * 0.7 # 70% of indicator contribution
             elif rsi < 40:  # Oversold
                 base_score += indicator_point_contribution * 1.0 # 100%
             else:  # Overbought
                 base_score += indicator_point_contribution * 0.5 # 50%
        else:
             logger.warning("RSI not calculated or is NaN.")


        # MACD scoring (points based on indicator_point_contribution)
        macd = indicators.get('MACD', np.nan) # Default to np.nan
        macd_signal = indicators.get('MACD_signal', np.nan) # Default to np.nan

        # Explicitly check if both are valid numbers before comparing
        if not pd.isna(macd) and not pd.isna(macd_signal):
             indicators_calculated += 1
             if macd > macd_signal:
                 base_score += indicator_point_contribution * 1.0
             else:
                 base_score += indicator_point_contribution * 0.5
        else:
             logger.warning("MACD or MACD signal not calculated or is NaN.")


        # Moving Average Score (points based on indicator_point_contribution) - Using SMA_20 vs SMA_50
        sma_20 = indicators.get('SMA_20', np.nan) # Default to np.nan
        sma_50 = indicators.get('SMA_50', np.nan) # Default to np.nan
        if not pd.isna(sma_20) and not pd.isna(sma_50):
            indicators_calculated += 1
            if sma_20 > sma_50:
                base_score += indicator_point_contribution * 1.0
            else:
                base_score += indicator_point_contribution * 0.5
        else:
            logger.warning("SMA_20 or SMA_50 not calculated or is NaN.")

        # Bollinger Bands Score (points based on indicator_point_contribution)
        current_price = indicators.get('current_price', np.nan) # Assuming current_price is passed and defaulting to np.nan
        bb_upper = indicators.get('BB_upper', np.nan) # Default to np.nan
        bb_lower = indicators.get('BB_lower', np.nan) # Default to np.nan
        bb_middle = indicators.get('BB_middle', np.nan) # Default to np.nan

        if not pd.isna(current_price) and not pd.isna(bb_upper) and not pd.isna(bb_lower) and not pd.isna(bb_middle):
            indicators_calculated += 1
            if not pd.isna(bb_lower) and not pd.isna(bb_middle) and not pd.isna(current_price) and bb_lower < current_price < bb_middle:
                base_score += indicator_point_contribution * 1.0  # Good buying zone
            elif not pd.isna(bb_middle) and not pd.isna(bb_upper) and not pd.isna(current_price) and bb_middle < current_price < bb_upper:
                base_score += indicator_point_contribution * 0.7   # Neutral to positive
            else:
                base_score += indicator_point_contribution * 0.5   # Near extremes or outside bands
        else:
             logger.warning("Bollinger Bands or current price not available or is NaN.")

        # Volume Score (points based on indicator_point_contribution)
        volume_ratio = indicators.get('Volume_ratio', np.nan) # Default to np.nan
        if not pd.isna(volume_ratio):
             indicators_calculated += 1
             if 0.8 <= volume_ratio <= 1.5:
                 base_score += indicator_point_contribution * 0.7   # Normal volume
             elif volume_ratio > 1.5:
                 base_score += indicator_point_contribution * 1.0  # High volume (positive if with price up)
             else:
                 base_score += indicator_point_contribution * 0.5   # Low volume
        else:
             logger.warning("Volume ratio not calculated or is NaN.")


        # Adjust base_score if fewer indicators were calculated to avoid inflating score
        achieved_score_from_indicators = base_score # Score accumulated from calculated indicators

        if indicators_calculated > 0:
            # Pro-rate the score based on the number of indicators calculated that contributed points
            # Ensure division is by a positive number of calculated indicators
            # The factor should be based on how many indicators were *expected* to contribute points
            max_possible_contribution = num_key_indicators * indicator_point_contribution
            achieved_percentage = achieved_score_from_indicators / max_possible_contribution if max_possible_contribution > 0 else 0

            adjusted_base_score = achieved_percentage * possible_indicator_points

            # Log the adjustment only if it happened
            if indicators_calculated < num_key_indicators:
                 logger.warning(f"Only {indicators_calculated}/{num_key_indicators} key technical indicators calculated. Adjusting base score from {achieved_score_from_indicators:.2f} (raw) to {adjusted_base_score:.2f}.")
            base_score = adjusted_base_score
        else:
             logger.warning("No key technical indicators calculated. Base technical score is 0.")
             base_score = 0 # No base score if no indicators calculated

        # Add circuit risk adjustment (circuit_risk is a negative value for risk)
        # Ensure circuit_risk is a valid number before adding
        circuit_risk_adj = circuit_risk if isinstance(circuit_risk, (int, float)) and not pd.isna(circuit_risk) else 0

        final_score = base_score + circuit_risk_adj

        return max(0, min(final_score, 50)) # Ensure score is within 0-50 range

    def fetch_news(self, symbol: str) -> List[str]:
        """
        Fetches recent news headlines for the given stock symbol.
        NOTE: This is a placeholder. In a real application, you would integrate
        with a News API (like News API, Alpha Vantage, etc.) here.
        """
        print(f"Fetching news for {symbol} (placeholder)...")
        # Placeholder news headlines - replace with actual API call
        # Ensure symbol is a string for consistent output
        symbol_str = str(symbol) if symbol is not None else "Unknown Stock"
        return [
            f"{symbol_str} stock price rises on positive market sentiment",
            f"{symbol_str} announces strong quarterly results, beating estimates",
            f"Experts bullish on {symbol_str}'s future outlook",
            f"I am a non-numeric entry that might cause an error", # Added a potential non-numeric entry for testing
            f"{symbol_str} faces regulatory challenges in key market",
            f"Competitor's new product launch impacts {symbol_str}'s market share",
            f"Global market trends positively impacting {symbol_str}",
            f"{symbol_str} management provides optimistic guidance",
            f"Increased foreign investment flows into {symbol_str}",
            f"Regulatory approval received for {symbol_str}'s new product",
            f"Production issues reported for {symbol_str}",
            f"Increased competition puts pressure on {symbol_str}'s margins",
            f"Analyst downgrades rating for {symbol_str}",
            f"Supply chain disruptions affect {symbol_str}'s operations",
            f"Geopolitical tensions create uncertainty for {symbol_str}",
            f"{symbol_str} announces stock split",
            f"Dividend declared by {symbol_str}"
        ]


    def analyze_sentiment(self, news_headlines: List[str]) -> Dict:
        """
        Analyzes the sentiment of news headlines using the Gemini API.
        Returns a dictionary with sentiment counts (positive, neutral, negative)
        and a sentiment score.
        """
        if not self.gemini_model:
            print("Gemini API not configured. Skipping sentiment analysis.")
            return {'positive': 0, 'neutral': 0, 'negative': 0, 'score': 0, 'summary': 'Gemini API not configured. Sentiment analysis skipped.'}

        if not isinstance(news_headlines, list) or not news_headlines:
            print("No news headlines to analyze. Skipping sentiment analysis.")
            return {'positive': 0, 'neutral': 0, 'negative': 0, 'score': 12.5, 'summary': 'No news headlines. Sentiment analysis skipped.'} # Neutral score if no headlines


        logger.info("Analyzing sentiment using Gemini API...")
        prompt = "Analyze the sentiment of the following news headlines for a stock and categorize each as Positive, Neutral, or Negative. Provide a concise summary of the overall sentiment.\n\n"
        for headline in news_headlines:
            # Ensure headline is a string
            if isinstance(headline, str):
                 prompt += f"- {headline}\n"
            else:
                 logger.warning(f"Skipping non-string headline: {headline}")
                 continue # Skip if headline is not a string

        # Increase prompt length check to 30,000 characters, not words
        if len(prompt) > 30000: # Check length in characters
             logger.warning("News headlines too long for Gemini API prompt. Truncating.")
             prompt = prompt[:30000] + "\n... (headlines truncated)" # Truncate prompt if too long


        try:
            response = self.gemini_model.generate_content(prompt)
            sentiment_text = response.text.strip()
            logger.info(f"Gemini API Response: {sentiment_text[:200]}...") # Log first 200 chars

            # Parse the response to count sentiments and extract summary
            positive_count = sentiment_text.lower().count('positive')
            neutral_count = sentiment_text.lower().count('neutral')
            negative_count = sentiment_text.lower().count('negative')


            # Attempt to extract a summary line - this is a heuristic
            summary_line = "No summary extracted."
            lines = sentiment_text.split('\n')
            for line in lines:
                 lower_line = line.lower()
                 if 'summary' in lower_line or 'overall sentiment' in lower_line:
                     summary_line = line.strip()
                     break
            if summary_line == "No summary extracted." and lines:
                 # If no explicit summary found, take the last non-empty line as a potential summary
                 for line in reversed(lines):
                     if line.strip():
                         summary_line = line.strip()
                         break


            total_headlines = len([h for h in news_headlines if isinstance(h, str)]) # Count only valid string headlines
            if total_headlines > 0:
                # Scoring based on the ratio of positive vs negative headlines, scaled to 25 points
                # (Positive - Negative) / Total * 12.5 + 12.5 -> Range 0-25
                sentiment_score = ((positive_count - negative_count) / total_headlines) * 12.5 + 12.5
            else:
                sentiment_score = 12.5 # Neutral score if no valid headlines

            return {
                'positive': positive_count,
                'neutral': neutral_count,
                'negative': negative_count,
                'score': max(0, min(sentiment_score, 25)), # Cap score between 0 and 25
                'summary': summary_line
            }

        except Exception as e:
            logger.error(f"Error analyzing sentiment with Gemini API: {e}. Skipping sentiment analysis.")
            return {'positive': 0, 'neutral': 0, 'negative': 0, 'score': 0, 'summary': 'Sentiment analysis failed.'}


    def generate_enhanced_recommendation(self, fundamental_score, technical_score,
                                       liquidity_score, peer_performance, index_membership,
                                       sentiment_analysis_results):
        """Generate comprehensive recommendation including sentiment"""
        # Ensure scores are valid numbers before summing
        fund_score = fundamental_score if isinstance(fundamental_score, (int, float)) and not pd.isna(fundamental_score) else 0
        tech_score = technical_score if isinstance(technical_score, (int, float)) and not pd.isna(technical_score) else 0
        liq_score = liquidity_score if isinstance(liquidity_score, (int, float)) and not pd.isna(liquidity_score) else 0
        sentiment_score = sentiment_analysis_results.get('score', 0) if isinstance(sentiment_analysis_results.get('score'), (int, float)) and not pd.isna(sentiment_analysis_results.get('score')) else 0


        total_score = fund_score + tech_score + liq_score + sentiment_score # Add sentiment score

        # Adjust for peer performance
        # Ensure peer_performance is a valid dictionary and outperformance is a boolean
        # Check if peer analysis was skipped by looking for NaNs in the results structure
        peer_analysis_available = isinstance(peer_performance, dict) and \
                                  not (pd.isna(peer_performance.get('stock_return')) and \
                                       pd.isna(peer_performance.get('avg_peer_return')) and \
                                       pd.isna(peer_performance.get('relative_performance')))


        if peer_analysis_available and peer_performance.get('outperformance', False) is True:
            total_score += 5
            logger.info("Adding bonus for peer outperformance.")
        elif not peer_analysis_available:
             # Apply a small penalty if peer analysis was intended but data is unavailable
             total_score -= 5 # Small penalty if peer data was expected but unavailable
             logger.warning("Applying small penalty as peer performance data is incomplete/unavailable.")


        # Bonus for index membership
        # Ensure index_membership is a valid list
        if isinstance(index_membership, list) and index_membership:
            total_score += 5
            logger.info("Adding bonus for index membership.")


        # Max possible score is 50 (Fundamental) + 50 (Technical) + 70 (Liquidity) + 25 (Sentiment) + 5 (Peer) + 5 (Index) = 205
        # Let's re-evaluate the recommendation thresholds based on this potential range

        recommendation = "NEUTRAL" # Default recommendation
        confidence = "Low"

        if total_score >= 170: # Adjusted Example Thresholds (adjust as needed)
            recommendation = "STRONG BUY"
            confidence = "High"
        elif total_score >= 140:
            recommendation = "BUY"
            confidence = "Moderate to High"
        elif total_score >= 100:
            recommendation = "HOLD"
            confidence = "Moderate"
        elif total_score >= 70:
            recommendation = "SELL"
            confidence = "Moderate"
        else:
            recommendation = "STRONG SELL"
            confidence = "High"

        logger.info(f"Generated enhanced recommendation: {recommendation} (Total Score: {total_score:.2f})")

        return {
            'recommendation': recommendation,
            'confidence': confidence,
            'total_score': total_score,
            'breakdown': {
                'fundamental': fund_score,
                'technical': tech_score,
                'liquidity': liq_score,
                'sentiment': sentiment_score,
                'peer_adjustment': 5 if peer_analysis_available and peer_performance.get('outperformance', False) is True else (-5 if not peer_analysis_available else 0),
                'index_adjustment': 5 if isinstance(index_membership, list) and index_membership else 0
            }
        }

# Integration function to work with your Part 1
def analyze_stock_enhanced(symbol, stock_analyzer_part1, enhanced_analyzer):
    """
    Integrate enhanced features with your Part 1 analyzer

    Args:
        symbol: Stock symbol (e.g., 'RELIANCE.NS')
        stock_analyzer_part1: Instance of your IndianStockAnalyzer from Part 1
        enhanced_analyzer: Instance of EnhancedStockAnalyzer
    """
    if not isinstance(symbol, str) or not symbol:
        logger.error("Invalid stock symbol provided for enhanced analysis.")
        return None
    if not isinstance(stock_analyzer_part1, IndianStockAnalyzer):
        logger.error("Invalid IndianStockAnalyzer (Part 1) instance provided.")
        return None
    if not isinstance(enhanced_analyzer, EnhancedStockAnalyzer):
        logger.error("Invalid EnhancedStockAnalyzer (Part 2) instance provided.")
        return None


    logger.info(f"Starting enhanced analysis for {symbol}")
    # Get basic analysis from Part 1
    stock_data = stock_analyzer_part1.analyze_stock(symbol)

    if not stock_data:
        logger.error(f"Basic analysis failed for {symbol}.")
        return None

    # Get current price - already validated in Part 1 analyze_stock
    current_price = stock_data.current_price
    if pd.isna(current_price) or current_price <= 0:
         logger.warning(f"Current price invalid ({current_price}). Enhanced analysis might be limited.")


    # Enhanced analysis
    # 1. Circuit breaker analysis
    circuit_limits = enhanced_analyzer.get_circuit_limits(symbol)
    # Pass current_price to assess_circuit_risk (already validated as best as possible in Part 1)
    circuit_risk = enhanced_analyzer.assess_circuit_risk(current_price, circuit_limits)

    # 2. Enhanced liquidity analysis
    # Pass price_data (already validated in Part 1 analyze_stock)
    # CORRECTED: Call get_liquidity_score on enhanced_analyzer and pass price_data from stock_data
    liquidity_score = enhanced_analyzer.get_liquidity_score(stock_data.price_data)

    # 3. Peer analysis (now temporarily skipped)
    # Pass info (already validated in Part 1 analyze_stock)
    peers = enhanced_analyzer.get_peer_companies(symbol, stock_data.info) # Still get peers for reporting, even if analysis is skipped
    peer_performance = enhanced_analyzer.calculate_peer_relative_performance(symbol, peers) # This function now returns early with default NaNs


    # 4. Index membership
    index_membership = enhanced_analyzer.get_index_membership(symbol)

    # 5. Recalculate technical score with circuit risk
    # Pass indicators (already validated in Part 1 calculate_technical_indicators)
    # Pass circuit_risk (already validated in assess_circuit_risk)
    enhanced_technical_score = enhanced_analyzer.calculate_enhanced_technical_score(
        stock_data.indicators, circuit_risk
    )

    # 6. Fetch and analyze news sentiment
    # Pass symbol (already validated)
    news_headlines = enhanced_analyzer.fetch_news(symbol)
    # Pass news_headlines (already validated)
    sentiment_analysis_results = enhanced_analyzer.analyze_sentiment(news_headlines)


    # 7. Generate enhanced recommendation
    # Pass scores and results (already validated within their respective functions)
    final_recommendation = enhanced_analyzer.generate_enhanced_recommendation(
        stock_data.fundamental_score,
        enhanced_technical_score,
        liquidity_score,
        peer_performance, # Pass potentially incomplete/NaN peer performance
        index_membership,
        sentiment_analysis_results # Pass sentiment analysis results
    )

    # Create enhanced report
    enhanced_report = {
        'basic_analysis': stock_data,
        'enhanced_features': {
            'circuit_limits': circuit_limits,
            'circuit_risk_score': circuit_risk,
            'liquidity_score': liquidity_score, # Ensure correct liquidity score is included
            'peers': peers,
            'peer_performance': peer_performance, # Include peer performance, even if NaN
            'index_membership': index_membership,
            'dynamic_risk_free_rate': enhanced_analyzer.risk_free_rate,
            'news_headlines': news_headlines, # Include news in report
            'sentiment_analysis': sentiment_analysis_results # Include sentiment analysis results
        },
        'final_recommendation': final_recommendation
    }
    logger.info(f"Enhanced analysis completed for {symbol}")
    return enhanced_report

# Example usage
if __name__ == "__main__":
    # Initialize analyzers
    # If running in Jupyter/Colab, use the following approach:
    # First, make sure you've run the Part 1 code in a previous cell
    # Then the IndianStockAnalyzer class will be available in the namespace

    # For standalone script usage:
    # from indian_stock_analyzer_part1 import IndianStockAnalyzer

    # For Jupyter/Colab where Part 1 was run in previous cell:
    # IndianStockAnalyzer should already be available

    try:
        # Check if IndianStockAnalyzer is already defined (from previous cell)
        IndianStockAnalyzer
    except NameError:
        print("Please run Part 1 code first to define IndianStockAnalyzer class")
        raise

    stock_analyzer = IndianStockAnalyzer()
    enhanced_analyzer = EnhancedStockAnalyzer()

    # Analyze a stock
    symbol = "RELIANCE.NS" # Example symbol
    result = analyze_stock_enhanced(symbol, stock_analyzer, enhanced_analyzer)

    if result:
        # Display the enhanced report
        enhanced = result['enhanced_features']
        final = result['final_recommendation']
        basic = result['basic_analysis'] # Get basic data for display

        print(f"\n{'='*60}")
        print(f"Enhanced Analysis Report: {basic.company_name} ({basic.symbol})")
        print(f"{'='*60}")

        print(f"\n📊 Current Market Data:")
        print(f"Current Price: ₹{basic.current_price:,.2f}" if not pd.isna(basic.current_price) else "Current Price: N/A")
        market_cap_cr = basic.market_cap / 10000000 if not pd.isna(basic.market_cap) else np.nan
        print(f"Market Cap: ₹{market_cap_cr:,.2f} Cr" if not pd.isna(market_cap_cr) else "Market Cap: N/A")
        print(f"Sector: {basic.info.get('sector', 'N/A')}")

        print(f"\n⚙️ Enhanced Technical Analysis & Risk:")
        # Display enhanced technical indicators and risk
        # Assuming you want to display relevant indicators from basic.indicators and the circuit risk score
        circuit_limits_display = enhanced.get('circuit_limits')
        if circuit_limits_display and not pd.isna(circuit_limits_display.get('lower_circuit')) and not pd.isna(circuit_limits_display.get('upper_circuit')):
             print(f"Circuit Limits: {circuit_limits_display.get('lower_circuit', 'N/A'):.2f} - {circuit_limits_display.get('upper_circuit', 'N/A'):.2f}")
        else:
             print("Circuit Limits: N/A")

        print(f"Circuit Risk Score: {enhanced.get('circuit_risk_score', 'N/A')}")
        # Check if liquidity score is available before formatting
        liquidity_score_display = f"{enhanced.get('liquidity_score', 'N/A')}/70" if isinstance(enhanced.get('liquidity_score'), (int, float)) and not pd.isna(enhanced.get('liquidity_score')) else "Liquidity Score: N/A/70"
        print(liquidity_score_display)


        # Display some key technical indicators from basic analysis
        print("\nKey Technical Indicators (from Basic Analysis):")
        key_tech_indicators = ['RSI', 'MACD', 'MACD_signal', 'SMA_20', 'SMA_50', 'BB_middle', 'Volume_ratio']
        for indicator_name in key_tech_indicators:
            value = basic.indicators.get(indicator_name, np.nan)
            if not pd.isna(value):
                if isinstance(value, float):
                     print(f"  - {indicator_name}: {value:,.2f}")
                else:
                     print(f"  - {indicator_name}: {value}")
            else:
                print(f"  - {indicator_name}: N/A")


        print(f"\n🤝 Peer Analysis:")
        # Check if peer analysis was skipped or failed by examining the presence of meaningful data
        peer_perf = enhanced.get('peer_performance')
        if enhanced.get('peers') and peer_perf and not (pd.isna(peer_perf.get('stock_return')) and pd.isna(peer_perf.get('avg_peer_return')) and pd.isna(peer_perf.get('relative_performance'))):
             print(f"Peer Companies: {', '.join(enhanced['peers'])}")
             # Add checks for NaN before formatting float
             stock_return_display = f"{peer_perf.get('stock_return', 'N/A'):.2f}%" if isinstance(peer_perf.get('stock_return'), (int, float)) and not pd.isna(peer_perf.get('stock_return')) else 'N/A'
             avg_peer_return_display = f"{peer_perf.get('avg_peer_return', 'N/A'):.2f}%" if isinstance(peer_perf.get('avg_peer_return'), (int, float)) and not pd.isna(peer_perf.get('avg_peer_return')) else 'N/A'
             relative_performance_display = f"{peer_perf.get('relative_performance', 'N/A'):.2f}%" if isinstance(peer_perf.get('relative_performance'), (int, float)) and not pd.isna(peer_perf.get('relative_performance')) else 'N/A'

             print(f"  - {peer_perf.get('days', 'N/A')}-day Stock Return: {stock_return_display}")
             print(f"  - Avg Peer Return: {avg_peer_return_display}")
             print(f"  - Relative Performance: {relative_performance_display}")
        else:
             # Display a message indicating peer analysis was skipped/failed
             print("Peer analysis skipped or failed. Data unavailable.")


        if enhanced.get('index_membership'):
            print(f"\n🏛️ Index Membership: {', '.join(enhanced['index_membership'])}")

        # Sentiment Analysis Results
        sentiment_results = enhanced.get('sentiment_analysis', {})
        print(f"\n📰 News Sentiment Analysis:")
        if enhanced.get('news_headlines'):
             # Only print a few headlines to keep the report concise
             print("Sample News Headlines:")
             # Ensure headlines are valid strings before printing
             valid_headlines = [h for h in enhanced['news_headlines'] if isinstance(h, str)]
             for headline in valid_headlines[:5]: # Print up to 5 headlines
                 print(f"- {headline}")
             if len(valid_headlines) > 5:
                  print("  ...")

             print(f"Sentiment Counts: Positive={sentiment_results.get('positive', 0)}, Neutral={sentiment_results.get('neutral', 0)}, Negative={sentiment_results.get('negative', 0)}")
             sentiment_score_display = f"{sentiment_results.get('score', 0):.2f}" if isinstance(sentiment_results.get('score'), (int, float)) and not pd.isna(sentiment_results.get('score')) else "N/A"
             print(f"Sentiment Score: {sentiment_score_display}/25")
             print(f"Sentiment Summary: {sentiment_results.get('summary', 'N/A')}")
        else:
             print("No news headlines fetched or sentiment analysis skipped.")


        print(f"\n--- Comprehensive Scores and Recommendation ---")
        print(f"Fundamental Score (from Part 1): {basic.fundamental_score:.2f}/50" if not pd.isna(basic.fundamental_score) else "Fundamental Score: N/A")
        # Add checks for NaN before formatting float for breakdown scores
        enhanced_tech_score_display = f"{final.get('breakdown', {}).get('technical', 'N/A'):.2f}" if isinstance(final.get('breakdown', {}).get('technical'), (int, float)) and not pd.isna(final.get('breakdown', {}).get('technical')) else "N/A"
        liquidity_score_display_breakdown = f"{final.get('breakdown', {}).get('liquidity', 'N/A'):.2f}" if isinstance(final.get('breakdown', {}).get('liquidity'), (int, float)) and not pd.isna(final.get('breakdown', {}).get('liquidity')) else "N/A"
        sentiment_score_display_breakdown = f"{final.get('breakdown', {}).get('sentiment', 'N/A'):.2f}" if isinstance(final.get('breakdown', {}).get('sentiment'), (int, float)) and not pd.isna(final.get('breakdown', {}).get('sentiment')) else "N/A"

        print(f"Technical Score (Enhanced): {enhanced_tech_score_display}/50")
        print(f"Liquidity Score: {liquidity_score_display_breakdown}/70")
        print(f"Sentiment Score: {sentiment_score_display_breakdown}/25")

        # Display adjustment scores
        peer_adj_display = f"{final.get('breakdown', {}).get('peer_adjustment', 'N/A'):+.2f}" if isinstance(final.get('breakdown', {}).get('peer_adjustment'), (int, float)) and not pd.isna(final.get('breakdown', {}).get('peer_adjustment')) else "N/A"
        index_adj_display = f"{final.get('breakdown', {}).get('index_adjustment', 'N/A'):+.2f}" if isinstance(final.get('breakdown', {}).get('index_adjustment'), (int, float)) and not pd.isna(final.get('breakdown', {}).get('index_adjustment')) else "N/A"

        print(f"Peer Adjustment: {peer_adj_display}")
        print(f"Index Membership Adjustment: {index_adj_display}")


        print(f"\nTotal Enhanced Score: {final.get('total_score', 0):.2f}/205" if isinstance(final.get('total_score'), (int, float)) and not pd.isna(final.get('total_score')) else "Total Enhanced Score: N/A")
        print(f"RECOMMENDATION: {final.get('recommendation', 'N/A')} (Confidence: {final.get('confidence', 'N/A')})")


        print(f"\n{'='*60}\n")

# Task
Analyze the provided Python script for analyzing equity stocks, identify any logic errors, and suggest improvements.

## Review code logic

### Subtask:
Carefully read through the code, paying close attention to the calculations for fundamental and technical scores, how benchmarks are applied, and how the final recommendation is generated. Look for any potential off-by-one errors, incorrect comparisons, or flawed formulas.


## Identify edge cases

### Subtask:
Consider potential edge cases, such as stocks with limited data, zero values for financial metrics, or unusual market conditions (e.g., stocks hitting circuit breakers).


**Reasoning**:
Examine the code for handling limited data, zero values, and unusual market conditions based on the instructions, documenting potential issues.



In [ ]:
# Step 1: Examine handling of limited historical price data
print("--- Handling of Limited Historical Price Data ---")
# The calculate_technical_indicators function checks for len(price_data) < 50
# before calculating indicators like SMA_200.
# However, for indicators like SMA_20, SMA_50, EMA_12, EMA_26, RSI, MACD, and BB,
# it doesn't explicitly check if there's enough data *for that specific window*.
# Rolling means/stddevs and ewm calculations will produce NaN for the initial periods.
# The code uses .iloc[-1] to get the latest value, which will be NaN if not enough data.
# Potential Issue 1: Technical indicators will be NaN if insufficient history exists for the window.
# This could lead to errors or skewed technical scores if not handled downstream.
# The calculate_technical_score function does not explicitly check for NaN indicator values.
# It uses .get() which provides a default (like 50 for RSI, 0 for MACD), but this might mask issues.
print(
    "Potential Issue 1: Technical indicators can be NaN if insufficient historical data "
    "is available for the required lookback window (e.g., 20, 50, 200 periods). "
    "The scoring function does not explicitly check for or handle these NaNs, "
    "potentially using default values which might be misleading."
)

# Step 2: Review handling of zero, negative, or missing financial metrics
print("\n--- Handling of Financial Metrics (Zero, Negative, Missing) ---")
# The calculate_fundamental_score function uses info.get(metric, None) to handle missing data.
# It then checks if the value is not None before using it.
# For PE ratio, it checks if pe_ratio > 0. This handles zero and negative PE ratios appropriately (they get low scores).
# For Debt-to-Equity, it converts percentage to decimal, but doesn't explicitly check for zero or negative (though negative D/E is rare).
# For ROE, Revenue Growth, Net Profit Margin, it checks if the value exists. Zero or negative values will correctly fall into lower score tiers.
# Potential Issue 2: Debt-to-Equity is converted from a percentage (integer in info) to a decimal by dividing by 100.
# If the info source provides it differently (e.g., already as a decimal), this conversion might be incorrect.
# However, based on typical yfinance info, the division by 100 seems correct for the common 'debtToEquity' key.
# No major issues identified for zero/negative/missing fundamental metrics, as the scoring logic naturally assigns low scores or skips missing ones.
print(
    "Potential Issue 2: The conversion of 'debtToEquity' assumes it's provided as a percentage (integer). "
    "If the data source provides it as a decimal, the conversion by dividing by 100 would be incorrect. "
    "However, based on yfinance documentation, the current approach seems appropriate."
    "Handling of zero/negative/missing values for other metrics appears reasonable within the scoring logic."
)


# Step 3: Analyze handling of circuit limits and liquidity
print("\n--- Handling of Circuit Limits and Liquidity ---")
# get_circuit_limits relies on 'previousClose' from ticker.info. If this is missing, it returns None.
# assess_circuit_risk checks if circuit_limits is None. If so, it returns 0 risk. This is safe.
# If a stock is exactly at a circuit limit, the distance will be 0.
# upper_distance = (upper_circuit - current_price) / current_price
# lower_distance = (current_price - lower_circuit) / current_price
# If current_price == upper_circuit, upper_distance = 0.
# If current_price == lower_circuit, lower_distance = 0.
# The scoring thresholds (0.02, 0.05) mean that being *exactly* at the limit will trigger the highest risk penalty. This seems intended.
# get_liquidity_score relies on price_data['Volume'].
# If price_data is empty or Volume column is missing, it will raise errors.
# If volume is zero for the entire period, rolling means/stddevs will be zero or NaN.
# avg_volume_20d, avg_volume_5d could be zero.
# volume_spike = avg_volume_5d / avg_volume_20d - Division by zero is possible if avg_volume_20d is 0.
# volume_cv = volume_std / avg_volume_20d - Division by zero is possible if avg_volume_20d is 0.
# The except block catches these errors and returns a default liquidity score of 30.
# Potential Issue 3a: Division by zero can occur in get_liquidity_score if average volume is zero. The generic except block handles this by returning a default score, which might not be ideal. More specific handling could provide better insight.
# Potential Issue 3b: Circuit limits calculation assumes a fixed percentage (10% or 20%). Actual circuit limits on NSE can vary based on categories (5%, 10%, 20%) and may change. Relying on previousClose and a fixed percentage is a simplification.
print(
    "Potential Issue 3a: Division by zero is possible in liquidity calculation if average volume is zero. "
    "The current error handling returns a default score, masking the underlying issue."
)
print(
    "Potential Issue 3b: Circuit limit calculation is a simplification. Actual NSE limits vary by category and are dynamic. "
    "Relying solely on 'previousClose' and fixed percentages might not be accurate for all stocks."
)


# Step 4: Consider other potential edge cases
print("\n--- Other Potential Edge Cases ---")
# Delisted stocks or stocks with trading suspensions:
# yfinance might return empty dataframes or raise errors for such symbols.
# get_stock_data and analyze_stock functions handle None or empty dataframes gracefully by returning None or logging errors.
# This prevents crashes but means no analysis is provided. This is a reasonable behavior.
# Data inconsistencies from the data source:
# Incorrect or missing data points (e.g., sudden price jumps, zero volume on active days, incorrect financial figures).
# The script relies on the data source (yfinance). It does not include logic for detecting or correcting data inconsistencies.
# This is a limitation of relying on external data without validation layers.
# Potential Issue 4: The script assumes the data provided by yfinance is accurate and consistent. It does not handle data inconsistencies or anomalies within the fetched data itself.
print(
    "Potential Issue 4: The script does not include checks for data inconsistencies "
    "(e.g., abnormal price movements, zero volume on trading days) within the fetched historical data."
)

# Documentation of potential issues:
print("\n--- Summary of Potential Issues ---")
print("1. Technical indicators can be NaN if insufficient historical data is available for the required lookback window.")
print("2. The conversion of 'debtToEquity' assumes it's provided as a percentage.")
print("3a. Division by zero is possible in liquidity calculation if average volume is zero.")
print("3b. Circuit limit calculation is a simplification and may not be accurate for all stocks or market conditions.")
print("4. The script does not handle data inconsistencies within the fetched historical data.")


## Suggest improvements

### Subtask:
Suggest improvements to the code to handle any identified logic errors or enhance the accuracy and robustness of the analysis.


**Reasoning**:
Suggest improvements to address the identified edge cases and enhance the code's robustness and accuracy.



In [ ]:
# --- Suggested Improvements ---

print("--- Suggested Improvements ---")

# Improvement 1: Handle NaN values in technical indicators more explicitly
# Issue: Technical indicators can be NaN with insufficient historical data, and scoring might use misleading defaults.
print("\n1. Explicitly handle NaN values in Technical Indicators:")
print(
    "Modify `calculate_technical_indicators` to return NaN or None for indicators "
    "if there isn't enough data for the lookback period, and log a warning."
)
print(
    "Modify `calculate_technical_score` to check if an indicator value is NaN/None. "
    "If it is, either assign a neutral score for that specific indicator or reduce the total technical score, "
    "and log which indicators were not calculated due to insufficient data."
)
print("Benefit: Prevents misleading scores based on default values when data is insufficient and provides transparency.")

# Improvement 2: Add validation for Debt-to-Equity format
# Issue: Assumes 'debtToEquity' is a percentage (integer).
print("\n2. Add validation for Debt-to-Equity format:")
print(
    "Before dividing `debt_to_equity` by 100 in `calculate_fundamental_score`, "
    "add a check for its typical range (e.g., if it's a large integer like 50000, treat as 50%). "
    "If it's a small decimal (e.g., 0.5), assume it's already in decimal form."
)
print("Benefit: Makes the calculation more robust to potential variations in data format from the source.")

# Improvement 3: Handle division by zero in liquidity calculation
# Issue: Division by zero possible if average volume is zero.
print("\n3. Handle division by zero specifically in liquidity calculation:")
print(
    "In `get_liquidity_score`, add explicit checks before division (`if avg_volume_20d > 0:`). "
    "If the divisor is zero, assign a specific low score (e.g., 0 or 5) for the related metric (volume spike, volume CV) "
    "instead of letting the exception handler assign a generic default score for the whole function."
)
print("Benefit: Provides more accurate liquidity scoring even for illiquid stocks and gives better insight than a generic error catch.")

# Improvement 4: Enhance circuit limit calculation accuracy
# Issue: Simplified circuit limit calculation.
print("\n4. Enhance circuit limit calculation accuracy:")
print(
    "Instead of relying on fixed percentages, try to scrape or use an API "
    "to get actual, dynamic circuit limits from a reliable source like NSE India."
)
print(
    "If scraping is not feasible, at least document the limitation clearly "
    "and potentially add logic to categorize stocks (e.g., based on market cap or index membership) "
    "to apply slightly more accurate default percentages (e.g., 5% for some small caps)."
)
print("Benefit: Provides more accurate risk assessment based on actual market mechanisms.")

# Improvement 5: Add data validation checks for historical data
# Issue: Does not handle data inconsistencies within fetched data.
print("\n5. Add data validation checks for historical data:")
print(
    "Implement checks in `analyze_stock` or a dedicated data validation function "
    "to look for anomalies in the `price_data` DataFrame."
)
print(
    "Examples: Check for zero volume on normal trading days, sudden extreme price changes (spikes/drops), "
    "or non-monotonic price movements (if expected)."
)
print(
    "If anomalies are found, log a warning and potentially exclude the affected data points "
    "or adjust the confidence of the technical analysis score."
)
print("Benefit: Increases the reliability of the analysis by identifying potentially erroneous source data.")

# Improvement 6: Refine benchmark application
# Issue: Benchmarks are applied based on a simple sector mapping.
print("\n6. Refine industry benchmark application:")
print(
    "Expand the `industry_benchmarks` dictionary to include more sectors and potentially sub-sectors."
)
print(
    "Consider using a more sophisticated method to determine benchmarks, "
    "such as calculating median/average metrics for peers or the sector from recent data, "
    "instead of relying on static values."
)
print("Benefit: Provides more relevant and dynamic benchmarks for fundamental analysis.")

# Improvement 7: Add logging for score calculation
# Issue: Hard to debug score calculation without detailed logs.
print("\n7. Add detailed logging for score calculation:")
print(
    "Add log messages within `calculate_fundamental_score` and `calculate_technical_score` "
    "showing how points are assigned for each metric."
)
print("Benefit: Improves debuggability and transparency of the scoring process.")


## Summary:

### Data Analysis Key Findings

*   The core logic for calculating fundamental and technical scores aligns with the defined rules and benchmarks within the code.
*   The recommendation thresholds are set based on the potential range of calculated scores.
*   The enhanced analyzer incorporates additional factors like circuit risk, liquidity, peer performance, and index membership.
*   Technical indicators can result in NaN values with limited historical data, which the scoring logic might not handle explicitly, potentially using misleading defaults.
*   The conversion of 'debtToEquity' assumes the input is a percentage.
*   A division by zero is possible in the liquidity calculation if average volume is zero, currently handled by a generic error catch.
*   The circuit limit calculation is a simplification using fixed percentages and `previousClose`.
*   The script does not include checks for inconsistencies within the fetched historical data.

### Insights or Next Steps

*   Implement explicit handling for `NaN` values in technical indicators to prevent misleading scores and improve transparency.
*   Enhance data validation, especially for financial metrics like Debt-to-Equity and historical price data, to ensure data quality and robustness.
*   Improve error handling for specific cases like division by zero in liquidity calculations for more accurate scoring.


To use the Gemini API, you'll need an API key. If you don't already have one, create a key in Google AI Studio.
In Colab, add the key to the secrets manager under the "🔑" in the left panel. Give it the name `GOOGLE_API_KEY`. Then pass the key to the SDK:

In [ ]:
# Import the Python SDK
import google.generativeai as genai
# Used to securely store your API key
from google.colab import userdata

GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

Before you can make any API calls, you need to initialize the Generative Model.

In [ ]:
# Initialize the Gemini API
gemini_model = genai.GenerativeModel('gemini-2.5-flash-preview-04-17')

Now you can make API calls. For example, to generate a poem:

In [ ]:
#@title Analyze a stock and display the combined results
symbol_to_analyze = "RELIANCE.NS" # You can change this symbol

# Ensure both analyzers are instantiated (they should be if the cells above were run)
# If not, you might need to re-run the cells defining IndianStockAnalyzer and EnhancedStockAnalyzer
try:
    stock_analyzer = IndianStockAnalyzer()
    enhanced_analyzer = EnhancedStockAnalyzer()
except NameError:
    print("Please run the cells defining IndianStockAnalyzer and EnhancedStockAnalyzer first.")
    # Exit or handle the error appropriately if classes are not defined
    stock_analyzer = None
    enhanced_analyzer = None


if stock_analyzer and enhanced_analyzer:
    combined_analysis_report = analyze_stock_enhanced(symbol_to_analyze, stock_analyzer, enhanced_analyzer)

    if combined_analysis_report:
        print(f"\n{'='*60}")
        print(f"Combined Stock Analysis Report for {symbol_to_analyze}")
        print(f"{'='*60}")

        # Display basic analysis results
        basic = combined_analysis_report['basic_analysis']
        print(f"\n--- Basic Analysis (from Part 1) ---")
        print(f"Company Name: {basic.company_name}")
        print(f"Current Price: ₹{basic.current_price:,.2f}")
        print(f"Market Cap: ₹{basic.info.get('marketCap', 0)/10000000:,.2f} Cr")
        print(f"Sector: {basic.info.get('sector', 'N/A')}")
        print(f"Fundamental Score: {basic.fundamental_score}/50")
        print(f"Technical Score: {basic.technical_score}/50")
        print(f"Basic Recommendation: {basic.recommendation}")
        print("\nFundamental Metrics:")
        for metric, value in basic.metrics.items():
             if metric != 'Fundamental_Score':
                if isinstance(value, float):
                    if metric in ['ROE', 'Revenue_Growth', 'Net_Profit_Margin']:
                        print(f"{metric}: {value*100:.2f}%")
                    else:
                        print(f"{metric}: {value:.2f}")
                else:
                    print(f"{metric}: {value}")

        print("\nTechnical Indicators:")
        for indicator, value in basic.indicators.items():
             if indicator != 'current_price':
                if value is not None and not np.isnan(value):
                    if isinstance(value, float):
                        print(f"{indicator}: {value:,.2f}")
                    else:
                        print(f"{indicator}: {value}")
                else:
                    print(f"{indicator}: N/A (Insufficient Data)")


        # Display enhanced features and final recommendation
        enhanced = combined_analysis_report['enhanced_features']
        final = combined_analysis_report['final_recommendation']

        print(f"\n--- Enhanced Analysis (from Part 2) ---")
        print(f"Circuit Limits: {enhanced.get('circuit_limits', {}).get('lower_circuit', 'N/A'):.2f} - {enhanced.get('circuit_limits', {}).get('upper_circuit', 'N/A'):.2f}")
        print(f"Circuit Risk Score: {enhanced.get('circuit_risk_score', 'N/A')}")
        print(f"Liquidity Score: {enhanced.get('liquidity_score', 'N/A')}/70")

        if enhanced.get('peers'):
            print(f"Peer Companies: {', '.join(enhanced['peers'])}")
            if enhanced.get('peer_performance'):
                print(f"30-day Return: {enhanced['peer_performance'].get('stock_return', 'N/A'):.2f}%")
                print(f"Peer Avg Return: {enhanced['peer_performance'].get('avg_peer_return', 'N/A'):.2f}%")
                print(f"Relative Performance: {enhanced['peer_performance'].get('relative_performance', 'N/A'):.2f}%")

        if enhanced.get('index_membership'):
            print(f"Index Membership: {', '.join(enhanced['index_membership'])}")

        # Sentiment Analysis Results
        sentiment_results = enhanced.get('sentiment_analysis', {})
        print(f"\n📰 News Sentiment Analysis:")
        if enhanced.get('news_headlines'):
             for headline in enhanced['news_headlines']:
                 print(f"- {headline}")
             print(f"Sentiment Counts: Positive={sentiment_results.get('positive', 0)}, Neutral={sentiment_results.get('neutral', 0)}, Negative={sentiment_results.get('negative', 0)}")
             print(f"Sentiment Score: {sentiment_results.get('score', 0):.2f}/25")
             print(f"Sentiment Summary: {sentiment_results.get('summary', 'N/A')}")
        else:
             print("No news headlines fetched or sentiment analysis skipped.")


        print(f"\n--- Final Recommendation ---")
        print(f"RECOMMENDATION: {final.get('recommendation', 'N/A')} ({final.get('confidence', 'N/A')})")
        print(f"Total Score: {final.get('total_score', 0):.2f}/205")
        print("Score Breakdown:")
        print(f"  - Fundamental: {final.get('breakdown', {}).get('fundamental', 'N/A'):.2f}")
        print(f"  - Technical (Enhanced): {final.get('breakdown', {}).get('technical', 'N/A')}") # Technical score might not be float always
        print(f"  - Liquidity: {final.get('breakdown', {}).get('liquidity', 'N/A'):.2f}")
        print(f"  - Sentiment: {final.get('breakdown', {}).get('sentiment', 'N/A'):.2f}")

        print(f"\n{'='*60}")

    else:
        print(f"Could not generate combined analysis report for {symbol_to_analyze}.")

# Task
Explain the error in the selected code, fix it if possible, and incorporate the changes into the existing code. If fixing is not possible, diagnose the error. Ensure the results from "Part 1 Main Script" and the sentimental analysis with API are displayed together in different cells. Also, remove the line "from your_part1_module import IndianStockAnalyzer".

## Robust nan and data validation

### Subtask:
Enhance the script to handle missing data (NaNs) more consistently across all calculations and add explicit checks for data validity before performing analysis.


**Reasoning**:
The instructions require enhancing several functions to handle missing data and add data validity checks. This involves modifying the existing code within the `IndianStockAnalyzer` and `EnhancedStockAnalyzer` classes. The changes are related and can be logically grouped into a single code block to redefine the classes with the necessary enhancements.



In [ ]:
#@title Part 1 of Main script (Enhanced)
# Indian Stock Analyzer - Part 1 (Corrected and Enhanced further)
# Complete implementation with improvements for missing data handling

import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from dataclasses import dataclass
from typing import Dict, List, Optional, Tuple
import logging
import warnings
warnings.filterwarnings('ignore')

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__) # Define logger globally

@dataclass
class StockData:
    """Data class to store comprehensive stock information"""
    symbol: str
    company_name: str
    current_price: float
    market_cap: float
    info: Dict
    price_data: pd.DataFrame
    indicators: Dict
    fundamental_score: float
    technical_score: float
    recommendation: str
    metrics: Dict

class IndianStockAnalyzer:
    """Comprehensive stock analyzer for Indian markets"""

    def __init__(self):
        """Initialize the analyzer with Indian market specific parameters"""
        self.nifty_symbol = "^NSEI"
        self.risk_free_rate = 0.065  # Will be made dynamic in Part 2

        # Industry-specific benchmark thresholds for Indian markets
        self.industry_benchmarks = {
            'technology': {
                'pe_ratio': {'excellent': 25, 'good': 35, 'fair': 45},
                'debt_to_equity': {'excellent': 0.3, 'good': 0.5, 'fair': 0.8},
                'roe': {'excellent': 0.25, 'good': 0.18, 'fair': 0.12},
                'revenue_growth': {'excellent': 0.20, 'good': 0.15, 'fair': 0.10},
                'net_profit_margin': {'excellent': 0.20, 'good': 0.15, 'fair': 0.10}
            },
            'banking': {
                'pe_ratio': {'excellent': 15, 'good': 20, 'fair': 25},
                'debt_to_equity': {'excellent': 8, 'good': 10, 'fair': 12},
                'roe': {'excellent': 0.15, 'good': 0.12, 'fair': 0.08},
                'revenue_growth': {'excellent': 0.15, 'good': 0.10, 'fair': 0.05},
                'net_profit_margin': {'excellent': 0.25, 'good': 0.20, 'fair': 0.15}
            },
            'pharmaceutical': {
                'pe_ratio': {'excellent': 20, 'good': 30, 'fair': 40},
                'debt_to_equity': {'excellent': 0.4, 'good': 0.6, 'fair': 0.8},
                'roe': {'excellent': 0.20, 'good': 0.15, 'fair': 0.10},
                'revenue_growth': {'excellent': 0.15, 'good': 0.10, 'fair': 0.05},
                'net_profit_margin': {'excellent': 0.15, 'good': 0.10, 'fair': 0.08}
            },
            'fmcg': {
                'pe_ratio': {'excellent': 30, 'good': 40, 'fair': 50},
                'debt_to_equity': {'excellent': 0.5, 'good': 0.8, 'fair': 1.0},
                'roe': {'excellent': 0.30, 'good': 0.25, 'fair': 0.15},
                'revenue_growth': {'excellent': 0.12, 'good': 0.08, 'fair': 0.05},
                'net_profit_margin': {'excellent': 0.15, 'good': 0.10, 'fair': 0.08}
            },
            'automobile': {
                'pe_ratio': {'excellent': 15, 'good': 25, 'fair': 35},
                'debt_to_equity': {'excellent': 0.6, 'good': 0.8, 'fair': 1.2},
                'roe': {'excellent': 0.20, 'good': 0.15, 'fair': 0.10},
                'revenue_growth': {'excellent': 0.15, 'good': 0.10, 'fair': 0.05},
                'net_profit_margin': {'excellent': 0.10, 'good': 0.07, 'fair': 0.05}
            },
            'infrastructure': {
                'pe_ratio': {'excellent': 12, 'good': 18, 'fair': 25},
                'debt_to_equity': {'excellent': 1.0, 'good': 1.5, 'fair': 2.0},
                'roe': {'excellent': 0.15, 'good': 0.12, 'fair': 0.08},
                'revenue_growth': {'excellent': 0.20, 'good': 0.15, 'fair': 0.10},
                'net_profit_margin': {'excellent': 0.12, 'good': 0.08, 'fair': 0.05}
            },
            'energy': {
                'pe_ratio': {'excellent': 10, 'good': 15, 'fair': 20},
                'debt_to_equity': {'excellent': 0.8, 'good': 1.2, 'fair': 1.5},
                'roe': {'excellent': 0.15, 'good': 0.12, 'fair': 0.08},
                'revenue_growth': {'excellent': 0.15, 'good': 0.10, 'fair': 0.05},
                'net_profit_margin': {'excellent': 0.10, 'good': 0.07, 'fair': 0.05}
            },
            'realty': {
                'pe_ratio': {'excellent': 10, 'good': 15, 'fair': 25},
                'debt_to_equity': {'excellent': 1.0, 'good': 1.5, 'fair': 2.0},
                'roe': {'excellent': 0.12, 'good': 0.08, 'fair': 0.05},
                'revenue_growth': {'excellent': 0.25, 'good': 0.15, 'fair': 0.10},
                'net_profit_margin': {'excellent': 0.15, 'good': 0.10, 'fair': 0.05}
            },
            'telecom': {
                'pe_ratio': {'excellent': 15, 'good': 25, 'fair': 35},
                'debt_to_equity': {'excellent': 1.0, 'good': 1.5, 'fair': 2.0},
                'roe': {'excellent': 0.10, 'good': 0.07, 'fair': 0.05},
                'revenue_growth': {'excellent': 0.10, 'good': 0.07, 'fair': 0.05},
                'net_profit_margin': {'excellent': 0.08, 'good': 0.05, 'fair': 0.03}
            }
        }

        # Default benchmarks for other sectors
        self.default_benchmarks = {
            'pe_ratio': {'excellent': 15, 'good': 25, 'fair': 35},
            'debt_to_equity': {'excellent': 0.5, 'good': 1.0, 'fair': 1.5},
            'roe': {'excellent': 0.15, 'good': 0.10, 'fair': 0.07},
            'revenue_growth': {'excellent': 0.15, 'good': 0.10, 'fair': 0.05},
            'net_profit_margin': {'excellent': 0.10, 'good': 0.07, 'fair': 0.05}
        }

    def get_stock_data(self, symbol: str) -> Optional[yf.Ticker]:
        """Fetch stock data from Yahoo Finance"""
        try:
            ticker = yf.Ticker(symbol)
            # Verify the ticker is valid by checking if it has info
            if ticker.info and 'symbol' in ticker.info:
                return ticker
            else:
                logger.error(f"Invalid symbol: {symbol}")
                return None
        except Exception as e:
            logger.error(f"Error fetching data for {symbol}: {str(e)}")
            return None

    def get_industry_benchmarks(self, sector: str) -> Dict:
        """Get industry-specific benchmarks"""
        sector_lower = sector.lower() if sector else ''

        # Map common sector names to our benchmark categories
        sector_mapping = {
            'technology': 'technology',
            'information technology': 'technology',
            'financial services': 'banking',
            'financials': 'banking',
            'healthcare': 'pharmaceutical',
            'consumer defensive': 'fmcg',
            'consumer cyclical': 'automobile',
            'industrials': 'infrastructure',
            'energy': 'energy',
            'real estate': 'realty',
            'communication services': 'telecom'
        }

        benchmark_key = sector_mapping.get(sector_lower, None)

        if benchmark_key and benchmark_key in self.industry_benchmarks:
            return self.industry_benchmarks[benchmark_key]
        else:
            logger.info(f"Using default benchmarks for sector: {sector}")
            return self.default_benchmarks

    def calculate_technical_indicators(self, price_data: pd.DataFrame) -> Dict:
        """Calculate comprehensive technical indicators"""
        indicators = {}
        data_len = len(price_data)

        try:
            # Simple Moving Averages
            if data_len >= 20:
                 indicators['SMA_20'] = price_data['Close'].rolling(window=20).mean().iloc[-1]
            else:
                 indicators['SMA_20'] = np.nan
                 logger.warning("Insufficient data for SMA_20.")

            if data_len >= 50:
                 indicators['SMA_50'] = price_data['Close'].rolling(window=50).mean().iloc[-1]
            else:
                 indicators['SMA_50'] = np.nan
                 logger.warning("Insufficient data for SMA_50.")

            if data_len >= 200:
                 indicators['SMA_200'] = price_data['Close'].rolling(window=200).mean().iloc[-1]
            else:
                 indicators['SMA_200'] = np.nan
                 logger.warning("Insufficient data for SMA_200.")

            # Exponential Moving Averages
            if data_len >= 12:
                 indicators['EMA_12'] = price_data['Close'].ewm(span=12, adjust=False).mean().iloc[-1]
            else:
                 indicators['EMA_12'] = np.nan
                 logger.warning("Insufficient data for EMA_12.")

            if data_len >= 26:
                 indicators['EMA_26'] = price_data['Close'].ewm(span=26, adjust=False).mean().iloc[-1]
            else:
                 indicators['EMA_26'] = np.nan
                 logger.warning("Insufficient data for EMA_26.")

            # RSI
            if data_len >= 14:
                indicators['RSI'] = self.calculate_rsi(price_data['Close'])
            else:
                 indicators['RSI'] = np.nan
                 logger.warning("Insufficient data for RSI.")

            # MACD (requires EMA_12 and EMA_26)
            macd_line = np.nan
            signal_line = np.nan
            macd_histogram = np.nan

            if 'EMA_12' in indicators and 'EMA_26' in indicators and not np.isnan(indicators['EMA_12']) and not np.isnan(indicators['EMA_26']):
                 # Use the entire macd_line series for rolling calculation if possible
                 full_macd_line_series = price_data['Close'].ewm(span=12, adjust=False).mean() - price_data['Close'].ewm(span=26, adjust=False).mean()
                 if len(full_macd_line_series.dropna()) >= 9:
                      macd_line = full_macd_line_series.iloc[-1]
                      signal_line = full_macd_line_series.ewm(span=9, adjust=False).mean().iloc[-1]
                      if not np.isnan(signal_line):
                         macd_histogram = macd_line - signal_line
                 else:
                      logger.warning("Insufficient data for MACD signal line calculation.")
            else:
                 logger.warning("MACD not calculated due to missing EMA_12 or EMA_26.")


            indicators['MACD'] = macd_line
            indicators['MACD_signal'] = signal_line
            indicators['MACD_histogram'] = macd_histogram

            # Log MACD values after calculation
            logger.info(f"Calculated MACD: {indicators['MACD']}, Signal: {indicators['MACD_signal']}")


            # Bollinger Bands (requires SMA_20 and enough data for std dev)
            bb_period = 20
            bb_std = 2
            if data_len >= bb_period:
                sma_bb = price_data['Close'].rolling(window=bb_period).mean()
                std_bb = price_data['Close'].rolling(window=bb_period).std()
                # Ensure SMA and STD are not NaN before calculating bands
                if not np.isnan(sma_bb.iloc[-1]) and not np.isnan(std_bb.iloc[-1]):
                    indicators['BB_upper'] = sma_bb.iloc[-1] + (std_bb.iloc[-1] * bb_std)
                    indicators['BB_lower'] = sma_bb.iloc[-1] - (std_bb.iloc[-1] * bb_std)
                    indicators['BB_middle'] = sma_bb.iloc[-1]
                else:
                    indicators['BB_upper'] = np.nan
                    indicators['BB_lower'] = np.nan
                    indicators['BB_middle'] = np.nan
                    logger.warning("Bollinger Bands not calculated due to NaN in SMA or STD.")
            else:
                indicators['BB_upper'] = np.nan
                indicators['BB_lower'] = np.nan
                indicators['BB_middle'] = np.nan
                logger.warning("Insufficient data for Bollinger Bands.")


            # Volume indicators
            if data_len >= 20:
                 indicators['Volume_SMA'] = price_data['Volume'].rolling(window=20).mean().iloc[-1]
                 if indicators['Volume_SMA'] is not None and not np.isnan(indicators['Volume_SMA']) and indicators['Volume_SMA'] > 0:
                      indicators['Volume_ratio'] = price_data['Volume'].iloc[-1] / indicators['Volume_SMA']
                 else:
                      indicators['Volume_ratio'] = np.nan
                      logger.warning("Volume SMA is zero or NaN, Volume ratio not calculated.")
            else:
                 indicators['Volume_SMA'] = np.nan
                 indicators['Volume_ratio'] = np.nan
                 logger.warning("Insufficient data for Volume SMA and ratio.")


            # Price position (requires SMA_20 and SMA_50)
            current_price = price_data['Close'].iloc[-1] if data_len > 0 and 'Close' in price_data and not price_data['Close'].empty else np.nan
            indicators['current_price'] = current_price # Store current price

            if current_price is not None and not np.isnan(current_price) and 'SMA_20' in indicators and indicators['SMA_20'] is not None and not np.isnan(indicators['SMA_20']) and indicators['SMA_20'] > 0:
                indicators['Price_to_SMA20'] = ((current_price / indicators['SMA_20'] - 1) * 100)
            else:
                 indicators['Price_to_SMA20'] = np.nan
                 logger.warning("Price to SMA20 not calculated due to missing current price or SMA20.")

            if current_price is not None and not np.isnan(current_price) and 'SMA_50' in indicators and indicators['SMA_50'] is not None and not np.isnan(indicators['SMA_50']) and indicators['SMA_50'] > 0:
                 indicators['Price_to_SMA50'] = ((current_price / indicators['SMA_50'] - 1) * 100)
            else:
                 indicators['Price_to_SMA50'] = np.nan
                 logger.warning("Price to SMA50 not calculated due to missing current price or SMA50.")


            # Support and Resistance (requires enough data for rolling max/min)
            if data_len >= 20:
                 indicators['Resistance'] = price_data['High'].rolling(window=20).max().iloc[-1]
                 indicators['Support'] = price_data['Low'].rolling(window=20).min().iloc[-1]
            else:
                 indicators['Resistance'] = np.nan
                 indicators['Support'] = np.nan
                 logger.warning("Insufficient data for Support and Resistance.")


            return indicators

        except Exception as e:
            logger.error(f"Error calculating technical indicators: {str(e)}")
            return {}

    def calculate_rsi(self, prices: pd.Series, period: int = 14) -> Optional[float]:
        """Calculate Relative Strength Index"""
        try:
            if len(prices) < period:
                return np.nan # Return NaN for insufficient data

            delta = prices.diff()
            gain = delta.where(delta > 0, 0)
            loss = -delta.where(delta < 0, 0)

            avg_gain = gain.rolling(window=period).mean()
            avg_loss = loss.rolling(window=period).mean()

            # Handle division by zero if avg_loss is zero or NaN
            if avg_loss.iloc[-1] is None or np.isnan(avg_loss.iloc[-1]) or avg_loss.iloc[-1] == 0:
                 return 100.0 if avg_gain.iloc[-1] is not None and not np.isnan(avg_gain.iloc[-1]) and avg_gain.iloc[-1] > 0 else 50.0 # If no loss, RSI is 100 (if gain) or 50 (if no change)
            elif avg_loss.iloc[-1] > 0:
                rs = avg_gain / avg_loss
                rsi = 100 - (100 / (1 + rs))
                return rsi.iloc[-1]
            else:
                 return np.nan # Should not happen with abs, but as a safeguard


        except Exception as e:
            logger.error(f"Error calculating RSI: {str(e)}")
            return np.nan # Return NaN on error

    def calculate_fundamental_score(self, info: Dict, benchmarks: Dict) -> Tuple[float, Dict]:
        """Calculate fundamental analysis score based on Indian market standards"""
        score = 0
        max_score = 50
        metrics = {}

        try:
            # P/E Ratio (10 points)
            pe_ratio = info.get('trailingPE', info.get('forwardPE', None))
            if pe_ratio is not None and isinstance(pe_ratio, (int, float)) and not np.isnan(pe_ratio):
                metrics['PE_Ratio'] = pe_ratio
                if pe_ratio > 0: # Ensure PE is positive for scoring
                     if pe_ratio <= benchmarks['pe_ratio']['excellent']:
                         score += 10
                     elif pe_ratio <= benchmarks['pe_ratio']['good']:
                         score += 7
                     elif pe_ratio <= benchmarks['pe_ratio']['fair']:
                         score += 4
                     else:
                         score += 2
                else:
                    score += 1 # Small score for non-positive PE
            else:
                 metrics['PE_Ratio'] = np.nan
                 logger.warning("PE Ratio not available or is invalid.")


            # Debt to Equity (10 points)
            # Added check for type and typical range assuming integer percentage or decimal
            debt_to_equity_raw = info.get('debtToEquity', None)
            if debt_to_equity_raw is not None and isinstance(debt_to_equity_raw, (int, float)) and not np.isnan(debt_to_equity_raw):
                 # Assuming if it's a large number, it's percentage
                 if debt_to_equity_raw > 100: # Heuristic for percentage
                      debt_to_equity = debt_to_equity_raw / 100.0
                 else: # Assume it's already a decimal
                      debt_to_equity = debt_to_equity_raw

                 metrics['Debt_to_Equity'] = debt_to_equity
                 if debt_to_equity <= benchmarks['debt_to_equity']['excellent']:
                     score += 10
                 elif debt_to_equity <= benchmarks['debt_to_equity']['good']:
                     score += 7
                 elif debt_to_equity <= benchmarks['debt_to_equity']['fair']:
                     score += 4
                 else:
                     score += 2
            else:
                 metrics['Debt_to_Equity'] = np.nan
                 logger.warning("Debt to Equity not available or is invalid.")


            # ROE (10 points)
            roe = info.get('returnOnEquity', None)
            if roe is not None and isinstance(roe, (int, float)) and not np.isnan(roe):
                metrics['ROE'] = roe
                if roe >= benchmarks['roe']['excellent']:
                    score += 10
                elif roe >= benchmarks['roe']['good']:
                    score += 7
                elif roe >= benchmarks['roe']['fair']:
                    score += 4
                else:
                    score += 2
            else:
                 metrics['ROE'] = np.nan
                 logger.warning("ROE not available or is invalid.")

            # Revenue Growth (10 points)
            revenue_growth = info.get('revenueGrowth', None)
            if revenue_growth is not None and isinstance(revenue_growth, (int, float)) and not np.isnan(revenue_growth):
                metrics['Revenue_Growth'] = revenue_growth
                if revenue_growth >= benchmarks['revenue_growth']['excellent']:
                    score += 10
                elif revenue_growth >= benchmarks['revenue_growth']['good']:
                    score += 7
                elif revenue_growth >= benchmarks['revenue_growth']['fair']:
                    score += 4
                else:
                    score += 2
            else:
                 metrics['Revenue_Growth'] = np.nan
                 logger.warning("Revenue Growth not available or is invalid.")

            # Net Profit Margin (10 points)
            profit_margin = info.get('profitMargins', None)
            if profit_margin is not None and isinstance(profit_margin, (int, float)) and not np.isnan(profit_margin):
                metrics['Net_Profit_Margin'] = profit_margin
                if profit_margin >= benchmarks['net_profit_margin']['excellent']:
                    score += 10
                elif profit_margin >= benchmarks['net_profit_margin']['good']:
                    score += 7
                elif profit_margin >= benchmarks['net_profit_margin']['fair']:
                    score += 4
                else:
                    score += 2
            else:
                 metrics['Net_Profit_Margin'] = np.nan
                 logger.warning("Net Profit Margin not available or is invalid.")


            # Normalize score to max_score (already capped by scoring logic)
            metrics['Fundamental_Score'] = score

            return score, metrics

        except Exception as e:
            logger.error(f"Error calculating fundamental score: {str(e)}")
            return 0, {}

    def calculate_technical_score(self, indicators: Dict) -> float:
        """Calculate technical analysis score"""
        score = 0
        max_score = 50
        indicator_scores = {} # To store scores for each indicator

        try:
            # RSI Score (10 points)
            rsi = indicators.get('RSI', None)
            if rsi is not None and not np.isnan(rsi):
                 if 40 <= rsi <= 60:  # Neutral zone
                     indicator_scores['RSI'] = 7
                 elif rsi < 40:  # Oversold
                     indicator_scores['RSI'] = 10
                 elif 60 < rsi <= 70:  # Overbought but not extreme
                     indicator_scores['RSI'] = 5
                 else:  # Extreme levels
                     indicator_scores['RSI'] = 2
            else:
                 indicator_scores['RSI'] = 0 # Assign 0 if NaN/None
                 logger.warning("RSI not calculated or is NaN/None. Assigning 0 score for RSI.")


            # MACD Score (10 points)
            macd = indicators.get('MACD', None)
            macd_signal = indicators.get('MACD_signal', None)

            # Explicitly check if both are numbers before comparing
            if isinstance(macd, (int, float)) and isinstance(macd_signal, (int, float)) and not np.isnan(macd) and not np.isnan(macd_signal):
                 if macd > macd_signal:
                     indicator_scores['MACD'] = 10
                 else:
                     indicator_scores['MACD'] = 3
            else:
                 indicator_scores['MACD'] = 0 # Assign 0 if NaN/None
                 logger.warning("MACD or MACD signal not calculated or is not a valid number. Assigning 0 score for MACD.")


            # Moving Average Score (10 points) - Using SMA_20 vs SMA_50
            sma_20 = indicators.get('SMA_20', None)
            sma_50 = indicators.get('SMA_50', None)
            if sma_20 is not None and sma_50 is not None and not np.isnan(sma_20) and not np.isnan(sma_50):
                if sma_20 > sma_50:
                    indicator_scores['Moving_Averages'] = 10
                else:
                    indicator_scores['Moving_Averages'] = 3
            else:
                indicator_scores['Moving_Averages'] = 0 # Assign 0 if NaN/None
                logger.warning("SMA_20 or SMA_50 not calculated or is NaN/None. Assigning 0 score for Moving Averages.")


            # Bollinger Bands Score (10 points)
            current_price = indicators.get('current_price', None) # Assuming current_price is passed in indicators
            bb_upper = indicators.get('BB_upper', None)
            bb_lower = indicators.get('BB_lower', None)
            bb_middle = indicators.get('BB_middle', None)

            if current_price is not None and bb_upper is not None and bb_lower is not None and bb_middle is not None and \
               not np.isnan(current_price) and not np.isnan(bb_upper) and not np.isnan(bb_lower) and not np.isnan(bb_middle):
                if bb_lower < current_price < bb_middle:
                    indicator_scores['Bollinger_Bands'] = 10  # Good buying zone
                elif bb_middle < current_price < bb_upper:
                    indicator_scores['Bollinger_Bands'] = 7   # Neutral to positive
                else:
                    indicator_scores['Bollinger_Bands'] = 3   # Near extremes
            else:
                 indicator_scores['Bollinger_Bands'] = 0 # Assign 0 if NaN/None
                 logger.warning("Bollinger Bands or current price not available or is NaN/None. Assigning 0 score for Bollinger Bands.")


            # Volume Score (10 points)
            volume_ratio = indicators.get('Volume_ratio', None)
            if volume_ratio is not None and not np.isnan(volume_ratio):
                 if 0.8 <= volume_ratio <= 1.5:
                     indicator_scores['Volume'] = 7   # Normal volume
                 elif volume_ratio > 1.5:
                     indicator_scores['Volume'] = 10  # High volume (positive if with price up)
                 else:
                     indicator_scores['Volume'] = 3   # Low volume
            else:
                 indicator_scores['Volume'] = 0 # Assign 0 if NaN/None
                 logger.warning("Volume ratio not calculated or is NaN/None. Assigning 0 score for Volume.")

            # Sum the scores from individual indicators
            score = sum(indicator_scores.values())

            return min(score, max_score) # Cap at max_score


        except Exception as e:
            logger.error(f"Error calculating technical score: {str(e)}")
            return 0

    def generate_recommendation(self, fundamental_score: float, technical_score: float) -> str:
        """Generate investment recommendation based on scores"""
        total_score = fundamental_score + technical_score

        if total_score >= 75:
            return "STRONG BUY"
        elif total_score >= 60:
            return "BUY"
        elif total_score >= 40:
            return "HOLD"
        elif total_score >= 25:
            return "SELL"
        else:
            return "STRONG SELL"

    def analyze_stock(self, symbol: str) -> Optional[StockData]:
        """Main method to analyze a stock"""
        try:
            logger.info(f"Starting analysis for {symbol}")

            # Get stock data
            ticker = self.get_stock_data(symbol)
            if not ticker:
                return None

            # Get stock info
            info = ticker.info
            if not info:
                logger.error(f"No info available for {symbol}")
                return None

            # Get historical price data
            # Fetching enough data for 200-day SMA calculation
            price_data = ticker.history(period="1y") # Changed to 1 year to support 200-day SMA
            if price_data.empty: # Check if price_data is empty
                logger.error(f"No price data available for {symbol}")
                return None
            # Also check for essential columns
            if not all(col in price_data.columns for col in ['Open', 'High', 'Low', 'Close', 'Volume']):
                 logger.error(f"Price data for {symbol} is missing essential columns.")
                 return None


            # Get current price
            current_price = price_data['Close'].iloc[-1] if not price_data['Close'].empty else np.nan

            # Add current price to indicators for technical score calculation
            indicators = self.calculate_technical_indicators(price_data)
            indicators['current_price'] = current_price # Ensure current price is available for BB score

            # Get industry benchmarks
            sector = info.get('sector', 'Unknown')
            benchmarks = self.get_industry_benchmarks(sector)

            # Calculate scores
            fundamental_score, metrics = self.calculate_fundamental_score(info, benchmarks)
            technical_score = self.calculate_technical_score(indicators)

            # Generate recommendation
            recommendation = self.generate_recommendation(fundamental_score, technical_score)

            # Create StockData object
            stock_data = StockData(
                symbol=symbol,
                company_name=info.get('longName', symbol),
                current_price=current_price if not np.isnan(current_price) else 0.0, # Store 0 if current price is NaN
                market_cap=info.get('marketCap', 0) if info.get('marketCap') is not None else 0, # Ensure marketCap is not None
                info=info,
                price_data=price_data,
                indicators=indicators,
                fundamental_score=fundamental_score,
                technical_score=technical_score,
                recommendation=recommendation,
                metrics=metrics
            )

            logger.info(f"Analysis completed for {symbol}")
            return stock_data

        except Exception as e:
            logger.error(f"Error analyzing {symbol}: {str(e)}")
            return None

    def display_analysis(self, stock_data: StockData) -> None:
        """Display analysis results in a formatted manner"""
        print(f"\n{'='*60}")
        print(f"Stock Analysis Report: {stock_data.company_name} ({stock_data.symbol})")
        print(f"{'='*60}")

        print(f"\n📊 Current Market Data:")
        print(f"Current Price: ₹{stock_data.current_price:,.2f}" if stock_data.current_price is not None else "Current Price: N/A")
        print(f"Market Cap: ₹{stock_data.market_cap/10000000:,.2f} Cr" if stock_data.market_cap is not None else "Market Cap: N/A") # Use stored market_cap
        print(f"Sector: {stock_data.info.get('sector', 'N/A')}")

        print(f"\n📈 Technical Indicators:")
        # Display indicators, handling None values
        for indicator, value in stock_data.indicators.items():
            if indicator != 'current_price': # Avoid displaying current_price here
                if value is not None and not np.isnan(value):
                    if isinstance(value, float):
                        print(f"{indicator}: {value:,.2f}")
                    else:
                        print(f"{indicator}: {value}")
                else:
                    print(f"{indicator}: N/A (Insufficient Data)")


        print(f"\n💰 Fundamental Metrics:")
        for metric, value in stock_data.metrics.items():
            if metric != 'Fundamental_Score':
                if value is not None and not np.isnan(value):
                     if isinstance(value, float):
                         if metric in ['ROE', 'Revenue_Growth', 'Net_Profit_Margin']:
                             print(f"{metric}: {value*100:.2f}%")
                         else:
                             print(f"{metric}: {value:.2f}")
                     else:
                         print(f"{metric}: {value}")
                else:
                    print(f"{metric}: N/A (Missing Data)")


        print(f"\n📊 Analysis Scores:")
        print(f"Fundamental Score: {stock_data.fundamental_score}/50")
        print(f"Technical Score: {stock_data.technical_score}/50")
        print(f"Total Score: {stock_data.fundamental_score + stock_data.technical_score}/100")

        print(f"\n🎯 Recommendation: {stock_data.recommendation}")
        print(f"{'='*60}\n")


**Reasoning**:
The instructions require enhancing the `EnhancedStockAnalyzer` class to handle missing data more consistently and add data validity checks, especially in the `get_liquidity_score` and `calculate_enhanced_technical_score` functions. This involves modifying the existing code within the class. These changes are related to the enhancements for missing data and can be logically grouped into a single code block to redefine the `EnhancedStockAnalyzer` class with the necessary improvements. The import for `google.generativeai` and `userdata` is already in the previous cell, so it should not be repeated here.



In [ ]:
#@title Sentimental analysis with API (Enhanced)
# Indian Stock Analyzer - Part 2 (Improved Version with Enhanced Data Handling)
# This continues from your Part 1 implementation

# Import the Python SDK for Gemini API
# from google.colab import userdata # Already imported in a previous cell
# import google.generativeai as genai # Already imported in a previous cell

import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import requests
from bs4 import BeautifulSoup
import warnings
warnings.filterwarnings('ignore')

# Import the Python SDK for Gemini API
# Used to securely store your API key
# from google.colab import userdata # Already imported
# import google.generativeai as genai # Already imported

# Configure Gemini API (ensure GOOGLE_API_KEY is set in Colab secrets)
# GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY') # Already done in a previous cell
# genai.configure(api_key=GOOGLE_API_KEY) # Already done in a previous cell
# gemini_model = genai.GenerativeModel('gemini-2.0-flash') # Using a more recent model # Already done in a previous cell


class EnhancedStockAnalyzer:
    """Enhanced analyzer with additional features for Indian markets"""

    def __init__(self):
        self.risk_free_rate = self.get_dynamic_risk_free_rate()
        # self.gemini_model = gemini_model # Use the globally configured model # Already assigned in a previous cell

    def get_dynamic_risk_free_rate(self):
        """Fetch current 10-year G-Sec yield"""
        try:
            # Try to get from a reliable source
            # For now, using a realistic current rate
            # In production, you'd scrape from RBI or use an API
            return 0.072  # 7.2% as of recent data
        except:
            return 0.065  # Fallback to 6.5%

    def get_circuit_limits(self, symbol):
        """Get circuit breaker limits for the stock"""
        try:
            ticker = yf.Ticker(symbol)
            info = ticker.info
            prev_close = info.get('previousClose', None) # Use None as default

            if prev_close is None or not isinstance(prev_close, (int, float)) or np.isnan(prev_close) or prev_close <= 0:
                logger.warning(f"Previous close not available or invalid for {symbol}. Cannot calculate circuit limits.")
                return None

            # NSE circuit limits (simplified)
            # Actual limits depend on stock category
            if symbol in ['RELIANCE.NS', 'TCS.NS', 'INFY.NS']:  # Index stocks
                circuit_percent = 0.20  # 20% for index stocks
            else:
                circuit_percent = 0.10  # 10% for others

            return {
                'upper_circuit': prev_close * (1 + circuit_percent),
                'lower_circuit': prev_close * (1 - circuit_percent),
                'circuit_percent': circuit_percent
            }
        except Exception as e:
            logger.error(f"Error getting circuit limits for {symbol}: {str(e)}")
            return None

    def assess_circuit_risk(self, current_price, circuit_limits):
        """Assess proximity to circuit limits"""
        if not circuit_limits or current_price is None or np.isnan(current_price) or current_price <= 0:
            return 0

        upper_circuit = circuit_limits.get('upper_circuit')
        lower_circuit = circuit_limits.get('lower_circuit')

        if upper_circuit is None or lower_circuit is None or np.isnan(upper_circuit) or np.isnan(lower_circuit):
             logger.warning("Circuit limits are invalid, cannot assess circuit risk.")
             return 0

        upper_distance = (upper_circuit - current_price) / current_price if current_price > 0 else float('inf')
        lower_distance = (current_price - lower_circuit) / current_price if current_price > 0 else float('inf')


        # Risk scoring based on proximity
        if upper_distance < 0.02:  # Within 2% of upper circuit
            return -20  # High risk
        elif upper_distance < 0.05:  # Within 5%
            return -10
        elif lower_distance < 0.02:  # Within 2% of lower circuit
            return -25  # Very high risk
        elif lower_distance < 0.05:
            return -15

        return 0

    def get_liquidity_score(self, price_data: pd.DataFrame) -> float:
        """Enhanced liquidity analysis"""
        try:
            # Explicitly check for sufficient data and 'Volume' column
            if price_data.empty or 'Volume' not in price_data.columns or len(price_data) < 20:
                 logger.warning("Insufficient price data or missing Volume column for liquidity calculation.")
                 return 5 # Assign a very low liquidity score

            # Calculate various liquidity metrics
            avg_volume_20d = price_data['Volume'].rolling(20).mean().iloc[-1]
            avg_volume_5d = price_data['Volume'].rolling(5).mean().iloc[-1]
            current_close = price_data['Close'].iloc[-1] if not price_data['Close'].empty else np.nan

            if current_close is None or np.isnan(current_close) or current_close <= 0:
                 logger.warning("Current close price is invalid for liquidity calculation.")
                 return 5

            # Average daily turnover
            avg_turnover_20d = avg_volume_20d * current_close if avg_volume_20d is not None and not np.isnan(avg_volume_20d) else np.nan

            # Volume spike detection - handle division by zero explicitly
            volume_spike = np.nan
            if avg_volume_20d is not None and not np.isnan(avg_volume_20d) and avg_volume_20d > 0:
                volume_spike = avg_volume_5d / avg_volume_20d if avg_volume_5d is not None and not np.isnan(avg_volume_5d) else np.nan
            elif avg_volume_5d is not None and not np.isnan(avg_volume_5d) and avg_volume_5d > 0: # Case where 20d avg is 0 but 5d is not
                 volume_spike = 1 # Treat as normal if 20d is 0 but 5d has volume
            else:
                 logger.warning("Average volume is zero or NaN, cannot calculate volume spike.")


            # Volume consistency - handle division by zero explicitly
            volume_std = price_data['Volume'].rolling(20).std().iloc[-1] if len(price_data) >= 20 else np.nan
            volume_cv = np.nan
            if avg_volume_20d is not None and not np.isnan(avg_volume_20d) and avg_volume_20d > 0:
                volume_cv = volume_std / avg_volume_20d if volume_std is not None and not np.isnan(volume_std) else np.nan
            elif volume_std is not None and not np.isnan(volume_std) and volume_std > 0: # Case where 20d avg is 0 but std is not (unlikely but safeguard)
                 volume_cv = 1 # Treat as high variability if avg is 0 but std is not
            else:
                 logger.warning("Average volume or volume standard deviation is zero or NaN, cannot calculate volume CV.")


            # Liquidity scoring
            liquidity_score = 0

            # Turnover-based scoring - handle NaN
            if avg_turnover_20d is not None and not np.isnan(avg_turnover_20d):
                if avg_turnover_20d > 500_00_00_000:  # > 500 Cr
                    liquidity_score += 40
                elif avg_turnover_20d > 100_00_00_000:  # > 100 Cr
                    liquidity_score += 30
                elif avg_turnover_20d > 10_00_00_000:  # > 10 Cr
                    liquidity_score += 20
                elif avg_turnover_20d > 1_00_00_000:  # > 1 Cr
                    liquidity_score += 10
                else:
                    liquidity_score += 5
            else:
                 logger.warning("Average turnover is NaN, skipping turnover score.")


            # Volume consistency score - handle NaN
            if volume_cv is not None and not np.isnan(volume_cv):
                 if volume_cv < 0.5:  # Low volatility in volume
                     liquidity_score += 20
                 elif volume_cv < 1.0:
                     liquidity_score += 10
            else:
                 logger.warning("Volume CV is NaN, skipping volume consistency score.")

            # Recent volume trend score - handle NaN
            if volume_spike is not None and not np.isnan(volume_spike):
                 if volume_spike > 1.2:  # 20% higher recent volume
                     liquidity_score += 10
            else:
                 logger.warning("Volume spike is NaN, skipping recent volume trend score.")


            return max(5, min(liquidity_score, 70))  # Cap at 70, minimum score 5 for insufficient data

        except Exception as e: # Catch specific exceptions for better debugging
            logger.error(f"Error calculating liquidity score: {str(e)}")
            return 5  # Default very low liquidity on error


    def get_peer_companies(self, symbol, stock_info):
        """Identify peer companies for comparison"""
        try:
            sector = stock_info.get('sector', '')
            market_cap = stock_info.get('marketCap', 0)

            # Define market cap categories
            if market_cap > 10_00_000_00_00_000:  # > 10 Lakh Cr
                cap_category = 'Large Cap'
            elif market_cap > 50_000_00_00_000:   # > 50K Cr
                cap_category = 'Mid Cap'
            else:
                cap_category = 'Small Cap'

            # Peer mapping (simplified - in production, use a comprehensive database)
            peer_map = {
                'Technology': {
                    'Large Cap': ['TCS.NS', 'INFY.NS', 'WIPRO.NS', 'HCLTECH.NS'],
                    'Mid Cap': ['TECHM.NS', 'LTTS.NS', 'PERSISTENT.NS', 'COFORGE.NS'],
                    'Small Cap': ['ZENSAR.NS', 'ECLERX.NS', 'MASTEK.NS']
                },
                'Financial Services': {
                    'Large Cap': ['HDFCBANK.NS', 'ICICIBANK.NS', 'KOTAKBANK.NS', 'AXISBANK.NS'],
                    'Mid Cap': ['IDFCFIRSTB.NS', 'FEDERALBNK.NS', 'INDUSINDBK.NS'],
                    'Small Cap': ['SOUTHBANK.NS', 'UJJIVAN.NS', 'EQUITASBNK.NS']
                },
                'Healthcare': {
                    'Large Cap': ['SUNPHARMA.NS', 'DRREDDY.NS', 'CIPLA.NS', 'DIVISLAB.NS'],
                    'Mid Cap': ['TORNTPHARM.NS', 'ALKEM.NS', 'LUPIN.NS'],
                    'Small Cap': ['CAPLIPOINT.NS', 'BLISS.NS', 'INDOCO.NS']
                }
                # Add more sectors as needed
            }

            # Get peers, excluding the current symbol
            peers = peer_map.get(sector, {}).get(cap_category, [])
            peers = [p for p in peers if p != symbol]

            return peers[:5]  # Return top 5 peers

        except Exception as e:
            logger.error(f"Error getting peer companies: {str(e)}")
            return []

    def calculate_peer_relative_performance(self, symbol, peers, days=30):
        """Compare stock performance with peers"""
        try:
            results = {}

            # Get performance for main stock
            main_ticker = yf.Ticker(symbol)
            main_hist = main_ticker.history(period=f"{days}d")
            if len(main_hist) > 0 and 'Close' in main_hist.columns and not main_hist['Close'].empty:
                main_return = ((main_hist['Close'].iloc[-1] / main_hist['Close'].iloc[0]) - 1) * 100 if main_hist['Close'].iloc[0] > 0 else np.nan
            else:
                main_return = np.nan
                logger.warning(f"Insufficient historical data for {symbol} to calculate peer relative performance.")


            # Get peer performances
            peer_returns = []
            for peer in peers:
                try:
                    peer_ticker = yf.Ticker(peer)
                    peer_hist = peer_ticker.history(period=f"{days}d")
                    if len(peer_hist) > 0 and 'Close' in peer_hist.columns and not peer_hist['Close'].empty and peer_hist['Close'].iloc[0] > 0:
                        peer_return = ((peer_hist['Close'].iloc[-1] / peer_hist['Close'].iloc[0]) - 1) * 100
                        peer_returns.append(peer_return)
                    else:
                         logger.warning(f"Insufficient historical data for peer {peer}.")
                except Exception as e:
                    logger.warning(f"Could not get peer history for {peer}: {str(e)}")
                    continue

            if peer_returns and main_return is not None and not np.isnan(main_return):
                avg_peer_return = np.mean(peer_returns)
                relative_performance = main_return - avg_peer_return

                results = {
                    'stock_return': main_return,
                    'avg_peer_return': avg_peer_return,
                    'relative_performance': relative_performance,
                    'outperformance': relative_performance > 0
                }
            else:
                 logger.warning("Peer returns not available or main stock return is NaN, cannot calculate relative performance.")


            return results

        except Exception as e:
            logger.error(f"Error calculating peer relative performance: {str(e)}")
            return {}

    def get_index_membership(self, symbol):
        """Check index membership for the stock"""
        # Simplified index constituents (in production, fetch from NSE)
        index_constituents = {
            'NIFTY50': ['RELIANCE.NS', 'TCS.NS', 'HDFCBANK.NS', 'INFY.NS', 'ICICIBANK.NS'],
            'NIFTY_NEXT50': ['VEDL.NS', 'PNB.NS', 'INDIGO.NS', 'BANDHANBNK.NS'],
            'NIFTY_MIDCAP150': ['PERSISTENT.NS', 'COFORGE.NS', 'LTTS.NS']
        }

        membership = []
        for index, constituents in index_constituents.items():
            if symbol in constituents:
                membership.append(index)

        return membership

    def calculate_enhanced_technical_score(self, indicators: Dict, circuit_risk: float) -> float:
        """Calculate enhanced technical analysis score"""
        base_score = 0
        indicator_scores = {} # To store scores for each indicator

        try:
            # RSI Score (points based on significance, e.g., max 10 for oversold)
            rsi = indicators.get('RSI', None)
            if rsi is not None and not np.isnan(rsi):
                 if rsi < 30: indicator_scores['RSI'] = 10 # Oversold
                 elif 30 <= rsi < 40: indicator_scores['RSI'] = 8 # Approaching oversold
                 elif 40 <= rsi <= 60: indicator_scores['RSI'] = 5 # Neutral zone
                 elif 60 < rsi <= 70: indicator_scores['RSI'] = 3 # Approaching overbought
                 else: indicator_scores['RSI'] = 1 # Overbought
            else:
                 indicator_scores['RSI'] = 0 # Assign 0 if NaN/None
                 logger.warning("RSI not calculated or is NaN/None. Assigning 0 score for RSI.")

            # MACD scoring (points based on crossover and position relative to zero)
            macd = indicators.get('MACD', None)
            macd_signal = indicators.get('MACD_signal', None)

            if macd is not None and macd_signal is not None and not np.isnan(macd) and not np.isnan(macd_signal):
                 if macd > macd_signal:
                     if macd > 0: indicator_scores['MACD'] = 10 # Bullish crossover above zero
                     else: indicator_scores['MACD'] = 7 # Bullish crossover below zero
                 else:
                     if macd < 0: indicator_scores['MACD'] = 1 # Bearish crossover below zero
                     else: indicator_scores['MACD'] = 3 # Bearish crossover above zero
            else:
                 indicator_scores['MACD'] = 0 # Assign 0 if NaN/None
                 logger.warning("MACD or MACD signal not calculated or is NaN/None. Assigning 0 score for MACD.")

            # Moving Average Score (points based on MA crossovers and price position)
            current_price = indicators.get('current_price', None)
            sma_20 = indicators.get('SMA_20', None)
            sma_50 = indicators.get('SMA_50', None)
            sma_200 = indicators.get('SMA_200', None)

            ma_score = 0
            valid_mas = 0

            if sma_20 is not None and not np.isnan(sma_20) and current_price is not None and not np.isnan(current_price):
                 valid_mas += 1
                 if current_price > sma_20: ma_score += 5
                 else: ma_score += 1

            if sma_50 is not None and not np.isnan(sma_50) and current_price is not None and not np.isnan(current_price):
                 valid_mas += 1
                 if current_price > sma_50: ma_score += 5
                 else: ma_score += 1

            if sma_200 is not None and not np.isnan(sma_200) and current_price is not None and not np.isnan(current_price):
                 valid_mas += 1
                 if current_price > sma_200: ma_score += 5
                 else: ma_score += 1

            # Add points for bullish crossovers
            if sma_20 is not None and sma_50 is not None and not np.isnan(sma_20) and not np.isnan(sma_50):
                 if sma_20 > sma_50: ma_score += 5

            if sma_50 is not None and sma_200 is not None and not np.isnan(sma_50) and not np.isnan(sma_200):
                 if sma_50 > sma_200: ma_score += 5

            indicator_scores['Moving_Averages'] = ma_score # Max possible MA score is 25 (5+5+5 for price > MA + 5+5 for crossovers)

            if valid_mas == 0:
                 logger.warning("No valid Moving Averages calculated. Assigning 0 score for Moving Averages.")
                 indicator_scores['Moving_Averages'] = 0


            # Bollinger Bands Score (points based on price position relative to bands)
            bb_upper = indicators.get('BB_upper', None)
            bb_lower = indicators.get('BB_lower', None)
            bb_middle = indicators.get('BB_middle', None)

            if current_price is not None and bb_upper is not None and bb_lower is not None and bb_middle is not None and \
               not np.isnan(current_price) and not np.isnan(bb_upper) and not np.isnan(bb_lower) and not np.isnan(bb_middle):
                if current_price < bb_lower: indicator_scores['Bollinger_Bands'] = 10 # Price below lower band (potential buy signal)
                elif bb_lower <= current_price < bb_middle: indicator_scores['Bollinger_Bands'] = 7 # Between lower and middle band
                elif bb_middle <= current_price < bb_upper: indicator_scores['Bollinger_Bands'] = 3 # Between middle and upper band
                else: indicator_scores['Bollinger_Bands'] = 1 # Price above upper band (potential sell signal)
            else:
                 indicator_scores['Bollinger_Bands'] = 0 # Assign 0 if NaN/None
                 logger.warning("Bollinger Bands or current price not available or is NaN/None. Assigning 0 score for Bollinger Bands.")


            # Volume Score (points based on volume relative to average)
            volume_ratio = indicators.get('Volume_ratio', None)
            if volume_ratio is not None and not np.isnan(volume_ratio):
                 if volume_ratio > 1.5: indicator_scores['Volume'] = 10 # High volume
                 elif 0.8 <= volume_ratio <= 1.5: indicator_scores['Volume'] = 7 # Normal volume
                 else: indicator_scores['Volume'] = 3 # Low volume
            else:
                 indicator_scores['Volume'] = 0 # Assign 0 if NaN/None
                 logger.warning("Volume ratio not calculated or is NaN/None. Assigning 0 score for Volume.")

            # Sum the scores from individual indicators
            base_score = sum(indicator_scores.values())

            # Add circuit risk adjustment (already handled as negative points)
            final_score = base_score + circuit_risk

            # Ensure score is within 0-50 range (assuming max base_score is around 50 based on point allocation)
            # Max possible indicator score (10+10+25+10+10) = 65. Let's normalize this to 50.
            normalized_base_score = (base_score / 65.0) * 50.0 if base_score > 0 else 0
            final_score = normalized_base_score + circuit_risk


            return max(0, min(final_score, 50))


        except Exception as e:
            logger.error(f"Error calculating enhanced technical score: {str(e)}")
            return 0


    def fetch_news(self, symbol: str) -> List[str]:
        """
        Fetches recent news headlines for the given stock symbol.
        NOTE: This is a placeholder. In a real application, you would integrate
        with a News API (like News API, Alpha Vantage, etc.) here.
        """
        print(f"Fetching news for {symbol} (placeholder)...")
        # Placeholder news headlines - replace with actual API call
        return [
            f"{symbol} stock price rises on positive market sentiment",
            f"{symbol} announces strong quarterly results, beating estimates",
            f"Experts bullish on {symbol}'s future outlook",
            f"{symbol} faces regulatory challenges in key market",
            f"Competitor's new product launch impacts {symbol}'s market share",
            f"Global market trends positively impacting {symbol}",
            f"{symbol} management provides optimistic guidance",
            f"Increased foreign investment flows into {symbol}",
            f"Regulatory approval received for {symbol}'s new product",
            f"Production issues reported for {symbol}",
            f"Increased competition puts pressure on {symbol}'s margins",
            f"Analyst downgrades rating for {symbol}",
            f"Supply chain disruptions affect {symbol}'s operations",
            f"Geopolitical tensions create uncertainty for {symbol}",
            f"{symbol} announces stock split",
            f"Dividend declared by {symbol}"
        ]


    def analyze_sentiment(self, news_headlines: List[str]) -> Dict:
        """
        Analyzes the sentiment of news headlines using the Gemini API.
        Returns a dictionary with sentiment counts (positive, neutral, negative)
        and a sentiment score.
        """
        # Use the globally defined gemini_model
        if not 'gemini_model' in globals() or not gemini_model or not news_headlines:
            print("Gemini API not configured or no news to analyze. Skipping sentiment analysis.")
            return {'positive': 0, 'neutral': 0, 'negative': 0, 'score': 0, 'summary': 'Sentiment analysis skipped.'}

        logger.info("Analyzing sentiment using Gemini API...")
        prompt = "Analyze the sentiment of the following news headlines for a stock and categorize each as Positive, Neutral, or Negative. Provide a concise summary of the overall sentiment.\n\n"
        for headline in news_headlines:
            prompt += f"- {headline}\n"
        prompt += "\nSentiment Analysis Results (Categorization and Summary):\n"

        try:
            response = gemini_model.generate_content(prompt)
            sentiment_text = response.text.strip()
            logger.info(f"Gemini API Response: {sentiment_text[:200]}...") # Log first 200 chars

            # Parse the response to count sentiments and extract summary
            positive_count = sentiment_text.lower().count('positive')
            neutral_count = sentiment_text.lower().count('neutral')
            negative_count = sentiment_text.lower().count('negative')

            # Attempt to extract a summary line - this is a heuristic
            summary_line = "No summary extracted."
            lines = sentiment_text.split('\n')
            for line in lines:
                 lower_line = line.lower()
                 if 'summary' in lower_line or 'overall sentiment' in lower_line:
                     summary_line = line.strip()
                     break
            if summary_line == "No summary extracted." and lines:
                 # If no explicit summary found, take the last non-empty line as a potential summary
                 for line in reversed(lines):
                     if line.strip():
                         summary_line = line.strip()
                         break


            total_headlines = len(news_headlines)
            if total_headlines > 0:
                # Scoring based on the ratio of positive vs negative headlines, scaled to 25 points
                # (Positive - Negative) / Total * 12.5 + 12.5 -> Range 0-25
                sentiment_score = ((positive_count - negative_count) / total_headlines) * 12.5 + 12.5
            else:
                sentiment_score = 12.5 # Neutral score if no headlines

            return {
                'positive': positive_count,
                'neutral': neutral_count,
                'negative': negative_count,
                'score': max(0, min(sentiment_score, 25)), # Cap score between 0 and 25
                'summary': summary_line
            }

        except Exception as e:
            logger.error(f"Error analyzing sentiment with Gemini API: {e}. Skipping sentiment analysis.")
            return {'positive': 0, 'neutral': 0, 'negative': 0, 'score': 0, 'summary': 'Sentiment analysis failed.'}


    def generate_enhanced_recommendation(self, fundamental_score, technical_score,
                                       liquidity_score, peer_performance, index_membership,
                                       sentiment_analysis_results):
        """Generate comprehensive recommendation including sentiment"""
        sentiment_score = sentiment_analysis_results.get('score', 0)
        total_score = fundamental_score + technical_score + liquidity_score + sentiment_score # Add sentiment score

        # Adjust for peer performance
        if peer_performance.get('outperformance', False):
            total_score += 5

        # Bonus for index membership
        if index_membership:
            total_score += 5

        # Max possible score is 50 (Fundamental) + 50 (Technical) + 70 (Liquidity) + 25 (Sentiment) + 5 (Peer) + 5 (Index) = 205
        # Let's re-evaluate the recommendation thresholds based on this potential range

        if total_score >= 160: # Example Thresholds (adjust as needed)
            recommendation = "STRONG BUY"
            confidence = "High"
        elif total_score >= 130:
            recommendation = "BUY"
            confidence = "Moderate to High"
        elif total_score >= 90:
            recommendation = "HOLD"
            confidence = "Moderate"
        elif total_score >= 60:
            return "SELL"
        else:
            return "STRONG SELL"

        return {
            'recommendation': recommendation,
            'confidence': confidence,
            'total_score': total_score,
            'breakdown': {
                'fundamental': fundamental_score,
                'technical': technical_score,
                'liquidity': liquidity_score,
                'sentiment': sentiment_score
            }
        }

# Integration function to work with your Part 1
def analyze_stock_enhanced(symbol, stock_analyzer_part1, enhanced_analyzer):
    """
    Integrate enhanced features with your Part 1 analyzer

    Args:
        symbol: Stock symbol (e.g., 'RELIANCE.NS')
        stock_analyzer_part1: Instance of your IndianStockAnalyzer from Part 1
        enhanced_analyzer: Instance of EnhancedStockAnalyzer
    """

    # Get basic analysis from Part 1
    stock_data = stock_analyzer_part1.analyze_stock(symbol)

    if not stock_data:
        return None

    # Get current price
    current_price = stock_data.current_price

    # Enhanced analysis
    # 1. Circuit breaker analysis
    circuit_limits = enhanced_analyzer.get_circuit_limits(symbol)
    circuit_risk = enhanced_analyzer.assess_circuit_risk(current_price, circuit_limits)

    # 2. Enhanced liquidity analysis
    liquidity_score = enhanced_analyzer.get_liquidity_score(stock_data.price_data)

    # 3. Peer analysis
    peers = enhanced_analyzer.get_peer_companies(symbol, stock_data.info)
    peer_performance = enhanced_analyzer.calculate_peer_relative_performance(symbol, peers)

    # 4. Index membership
    index_membership = enhanced_analyzer.get_index_membership(symbol)

    # 5. Recalculate technical score with circuit risk
    enhanced_technical_score = enhanced_analyzer.calculate_enhanced_technical_score(
        stock_data.indicators, circuit_risk
    )

    # 6. Fetch and analyze news sentiment
    news_headlines = enhanced_analyzer.fetch_news(symbol)
    sentiment_analysis_results = enhanced_analyzer.analyze_sentiment(news_headlines)


    # 7. Generate enhanced recommendation
    final_recommendation = enhanced_analyzer.generate_enhanced_recommendation(
        stock_data.fundamental_score,
        enhanced_technical_score,
        liquidity_score,
        peer_performance,
        index_membership,
        sentiment_analysis_results # Pass sentiment analysis results
    )

    # Create enhanced report
    enhanced_report = {
        'basic_analysis': stock_data,
        'enhanced_features': {
            'circuit_limits': circuit_limits,
            'circuit_risk_score': circuit_risk,
            'liquidity_score': liquidity_score,
            'peers': peers,
            'peer_performance': peer_performance,
            'index_membership': index_membership,
            'dynamic_risk_free_rate': enhanced_analyzer.risk_free_rate,
            'news_headlines': news_headlines, # Include news in report
            'sentiment_analysis': sentiment_analysis_results # Include sentiment analysis results
        },
        'final_recommendation': final_recommendation
    }

    return enhanced_report

# Example usage (removed from here as it's part of the next display step)

**Reasoning**:
The final part of the instructions is to display the results from both Part 1 and the enhanced analysis (Part 2) together and then finish the subtask. This involves calling the enhanced analysis function and then printing the combined report.



In [ ]:
#@title Analyze a stock and display the combined results
symbol_to_analyze = "RELIANCE.NS" # You can change this symbol

# Ensure both analyzers are instantiated (they should be if the cells above were run)
# If not, you might need to re-run the cells defining IndianStockAnalyzer and EnhancedStockAnalyzer
try:
    stock_analyzer = IndianStockAnalyzer()
    enhanced_analyzer = EnhancedStockAnalyzer()
except NameError:
    print("Please run the cells defining IndianStockAnalyzer and EnhancedStockAnalyzer first.")
    # Exit or handle the error appropriately if classes are not defined
    stock_analyzer = None
    enhanced_analyzer = None


if stock_analyzer and enhanced_analyzer:
    combined_analysis_report = analyze_stock_enhanced(symbol_to_analyze, stock_analyzer, enhanced_analyzer)

    if combined_analysis_report:
        print(f"\n{'='*60}")
        print(f"Combined Stock Analysis Report for {symbol_to_analyze}")
        print(f"{'='*60}")

        # Display basic analysis results
        basic = combined_analysis_report['basic_analysis']
        print(f"\n--- Basic Analysis (from Part 1) ---")
        print(f"Company Name: {basic.company_name}")
        print(f"Current Price: ₹{basic.current_price:,.2f}" if basic.current_price is not None else "Current Price: N/A")
        print(f"Market Cap: ₹{basic.market_cap/10000000:,.2f} Cr" if basic.market_cap is not None else "Market Cap: N/A") # Use stored market_cap
        print(f"Sector: {basic.info.get('sector', 'N/A')}")
        print(f"Fundamental Score: {basic.fundamental_score}/50")
        print(f"Technical Score: {basic.technical_score}/50")
        print(f"Basic Recommendation: {basic.recommendation}")
        print("\nFundamental Metrics:")
        for metric, value in basic.metrics.items():
             if metric != 'Fundamental_Score':
                if value is not None and not np.isnan(value):
                     if isinstance(value, float):
                         if metric in ['ROE', 'Revenue_Growth', 'Net_Profit_Margin']:
                             print(f"{metric}: {value*100:.2f}%")
                         else:
                             print(f"{metric}: {value:.2f}")
                     else:
                         print(f"{metric}: {value}")
                else:
                    print(f"{metric}: N/A (Missing Data)")


        print("\nTechnical Indicators:")
        for indicator, value in basic.indicators.items():
             if indicator != 'current_price':
                if value is not None and not np.isnan(value):
                    if isinstance(value, float):
                        print(f"{indicator}: {value:,.2f}")
                    else:
                        print(f"{indicator}: {value}")
                else:
                    print(f"{indicator}: N/A (Insufficient Data)")


        # Display enhanced features and final recommendation
        enhanced = combined_analysis_report['enhanced_features']
        final = combined_analysis_report['final_recommendation']

        print(f"\n--- Enhanced Analysis (from Part 2) ---")
        print(f"Circuit Limits: {enhanced.get('circuit_limits', {}).get('lower_circuit', 'N/A'):.2f} - {enhanced.get('circuit_limits', {}).get('upper_circuit', 'N/A'):.2f}" if enhanced.get('circuit_limits') else "Circuit Limits: N/A")
        print(f"Circuit Risk Score: {enhanced.get('circuit_risk_score', 'N/A')}")
        print(f"Liquidity Score: {enhanced.get('liquidity_score', 'N/A')}/70")

        if enhanced.get('peers'):
            print(f"Peer Companies: {', '.join(enhanced['peers'])}")
            if enhanced.get('peer_performance'):
                print(f"30-day Return: {enhanced['peer_performance'].get('stock_return', 'N/A'):.2f}%" if enhanced['peer_performance'].get('stock_return') is not None else "30-day Return: N/A")
                print(f"Peer Avg Return: {enhanced['peer_performance'].get('avg_peer_return', 'N/A'):.2f}%" if enhanced['peer_performance'].get('avg_peer_return') is not None else "Peer Avg Return: N/A")
                print(f"Relative Performance: {enhanced['peer_performance'].get('relative_performance', 'N/A'):.2f}%" if enhanced['peer_performance'].get('relative_performance') is not None else "Relative Performance: N/A")

        if enhanced.get('index_membership'):
            print(f"Index Membership: {', '.join(enhanced['index_membership'])}")

        # Sentiment Analysis Results
        sentiment_results = enhanced.get('sentiment_analysis', {})
        print(f"\n📰 News Sentiment Analysis:")
        if enhanced.get('news_headlines'):
             for headline in enhanced['news_headlines']:
                 print(f"- {headline}")
             print(f"Sentiment Counts: Positive={sentiment_results.get('positive', 0)}, Neutral={sentiment_results.get('neutral', 0)}, Negative={sentiment_results.get('negative', 0)}")
             print(f"Sentiment Score: {sentiment_results.get('score', 0):.2f}/25")
             print(f"Sentiment Summary: {sentiment_results.get('summary', 'N/A')}")
        else:
             print("No news headlines fetched or sentiment analysis skipped.")


        print(f"\n--- Final Recommendation ---")
        print(f"RECOMMENDATION: {final.get('recommendation', 'N/A')} ({final.get('confidence', 'N/A')})")
        print(f"Total Score: {final.get('total_score', 0):.2f}/205")
        print("Score Breakdown:")
        print(f"  - Fundamental: {final.get('breakdown', {}).get('fundamental', 'N/A'):.2f}")
        print(f"  - Technical (Enhanced): {final.get('breakdown', {}).get('technical', 'N/A')}") # Technical score might not be float always
        print(f"  - Liquidity: {final.get('breakdown', {}).get('liquidity', 'N/A'):.2f}")
        print(f"  - Sentiment: {final.get('breakdown', {}).get('sentiment', 'N/A'):.2f}")

        print(f"\n{'='*60}")

    else:
        print(f"Could not generate combined analysis report for {symbol_to_analyze}.")


## Improve debt-to-equity validation

### Subtask:
Refine the logic for validating and interpreting the Debt-to-Equity ratio, considering different data formats and potential outliers.


**Reasoning**:
Refine the logic for validating and interpreting the Debt-to-Equity ratio, considering different data formats and potential outliers based on the instructions.



In [ ]:
#@title Part 1 of Main script (Enhanced Debt-to-Equity)
# Indian Stock Analyzer - Part 1 (Corrected and Enhanced further)
# Complete implementation with improvements for missing data handling and Debt-to-Equity validation

import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from dataclasses import dataclass
from typing import Dict, List, Optional, Tuple
import logging
import warnings
warnings.filterwarnings('ignore')

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__) # Define logger globally

@dataclass
class StockData:
    """Data class to store comprehensive stock information"""
    symbol: str
    company_name: str
    current_price: float
    market_cap: float
    info: Dict
    price_data: pd.DataFrame
    indicators: Dict
    fundamental_score: float
    technical_score: float
    recommendation: str
    metrics: Dict

class IndianStockAnalyzer:
    """Comprehensive stock analyzer for Indian markets"""

    def __init__(self):
        """Initialize the analyzer with Indian market specific parameters"""
        self.nifty_symbol = "^NSEI"
        self.risk_free_rate = 0.065  # Will be made dynamic in Part 2

        # Industry-specific benchmark thresholds for Indian markets
        self.industry_benchmarks = {
            'technology': {
                'pe_ratio': {'excellent': 25, 'good': 35, 'fair': 45},
                'debt_to_equity': {'excellent': 0.3, 'good': 0.5, 'fair': 0.8},
                'roe': {'excellent': 0.25, 'good': 0.18, 'fair': 0.12},
                'revenue_growth': {'excellent': 0.20, 'good': 0.15, 'fair': 0.10},
                'net_profit_margin': {'excellent': 0.20, 'good': 0.15, 'fair': 0.10}
            },
            'banking': {
                'pe_ratio': {'excellent': 15, 'good': 20, 'fair': 25},
                'debt_to_equity': {'excellent': 8, 'good': 10, 'fair': 12},
                'roe': {'excellent': 0.15, 'good': 0.12, 'fair': 0.08},
                'revenue_growth': {'excellent': 0.15, 'good': 0.10, 'fair': 0.05},
                'net_profit_margin': {'excellent': 0.25, 'good': 0.20, 'fair': 0.15}
            },
            'pharmaceutical': {
                'pe_ratio': {'excellent': 20, 'good': 30, 'fair': 40},
                'debt_to_equity': {'excellent': 0.4, 'good': 0.6, 'fair': 0.8},
                'roe': {'excellent': 0.20, 'good': 0.15, 'fair': 0.10},
                'revenue_growth': {'excellent': 0.15, 'good': 0.10, 'fair': 0.05},
                'net_profit_margin': {'excellent': 0.15, 'good': 0.10, 'fair': 0.08}
            },
            'fmcg': {
                'pe_ratio': {'excellent': 30, 'good': 40, 'fair': 50},
                'debt_to_equity': {'excellent': 0.5, 'good': 0.8, 'fair': 1.0},
                'roe': {'excellent': 0.30, 'good': 0.25, 'fair': 0.15},
                'revenue_growth': {'excellent': 0.12, 'good': 0.08, 'fair': 0.05},
                'net_profit_margin': {'excellent': 0.15, 'good': 0.10, 'fair': 0.08}
            },
            'automobile': {
                'pe_ratio': {'excellent': 15, 'good': 25, 'fair': 35},
                'debt_to_equity': {'excellent': 0.6, 'good': 0.8, 'fair': 1.2},
                'roe': {'excellent': 0.20, 'good': 0.15, 'fair': 0.10},
                'revenue_growth': {'excellent': 0.15, 'good': 0.10, 'fair': 0.05},
                'net_profit_margin': {'excellent': 0.10, 'good': 0.07, 'fair': 0.05}
            },
            'infrastructure': {
                'pe_ratio': {'excellent': 12, 'good': 18, 'fair': 25},
                'debt_to_equity': {'excellent': 1.0, 'good': 1.5, 'fair': 2.0},
                'roe': {'excellent': 0.15, 'good': 0.12, 'fair': 0.08},
                'revenue_growth': {'excellent': 0.20, 'good': 0.15, 'fair': 0.10},
                'net_profit_margin': {'excellent': 0.12, 'good': 0.08, 'fair': 0.05}
            },
            'energy': {
                'pe_ratio': {'excellent': 10, 'good': 15, 'fair': 20},
                'debt_to_equity': {'excellent': 0.8, 'good': 1.2, 'fair': 1.5},
                'roe': {'excellent': 0.15, 'good': 0.12, 'fair': 0.08},
                'revenue_growth': {'excellent': 0.15, 'good': 0.10, 'fair': 0.05},
                'net_profit_margin': {'excellent': 0.10, 'good': 0.07, 'fair': 0.05}
            },
            'realty': {
                'pe_ratio': {'excellent': 10, 'good': 15, 'fair': 25},
                'debt_to_equity': {'excellent': 1.0, 'good': 1.5, 'fair': 2.0},
                'roe': {'excellent': 0.12, 'good': 0.08, 'fair': 0.05},
                'revenue_growth': {'excellent': 0.25, 'good': 0.15, 'fair': 0.10},
                'net_profit_margin': {'excellent': 0.15, 'good': 0.10, 'fair': 0.05}
            },
            'telecom': {
                'pe_ratio': {'excellent': 15, 'good': 25, 'fair': 35},
                'debt_to_equity': {'excellent': 1.0, 'good': 1.5, 'fair': 2.0},
                'roe': {'excellent': 0.10, 'good': 0.07, 'fair': 0.05},
                'revenue_growth': {'excellent': 0.10, 'good': 0.07, 'fair': 0.05},
                'net_profit_margin': {'excellent': 0.08, 'good': 0.05, 'fair': 0.03}
            }
        }

        # Default benchmarks for other sectors
        self.default_benchmarks = {
            'pe_ratio': {'excellent': 15, 'good': 25, 'fair': 35},
            'debt_to_equity': {'excellent': 0.5, 'good': 1.0, 'fair': 1.5},
            'roe': {'excellent': 0.15, 'good': 0.10, 'fair': 0.07},
            'revenue_growth': {'excellent': 0.15, 'good': 0.10, 'fair': 0.05},
            'net_profit_margin': {'excellent': 0.10, 'good': 0.07, 'fair': 0.05}
        }

    def get_stock_data(self, symbol: str) -> Optional[yf.Ticker]:
        """Fetch stock data from Yahoo Finance"""
        try:
            ticker = yf.Ticker(symbol)
            # Verify the ticker is valid by checking if it has info
            if ticker.info and 'symbol' in ticker.info:
                return ticker
            else:
                logger.error(f"Invalid symbol: {symbol}")
                return None
        except Exception as e:
            logger.error(f"Error fetching data for {symbol}: {str(e)}")
            return None

    def get_industry_benchmarks(self, sector: str) -> Dict:
        """Get industry-specific benchmarks"""
        sector_lower = sector.lower() if sector else ''

        # Map common sector names to our benchmark categories
        sector_mapping = {
            'technology': 'technology',
            'information technology': 'technology',
            'financial services': 'banking',
            'financials': 'banking',
            'healthcare': 'pharmaceutical',
            'consumer defensive': 'fmcg',
            'consumer cyclical': 'automobile',
            'industrials': 'infrastructure',
            'energy': 'energy',
            'real estate': 'realty',
            'communication services': 'telecom'
        }

        benchmark_key = sector_mapping.get(sector_lower, None)

        if benchmark_key and benchmark_key in self.industry_benchmarks:
            return self.industry_benchmarks[benchmark_key]
        else:
            logger.info(f"Using default benchmarks for sector: {sector}")
            return self.default_benchmarks

    def calculate_technical_indicators(self, price_data: pd.DataFrame) -> Dict:
        """Calculate comprehensive technical indicators"""
        indicators = {}
        data_len = len(price_data)

        try:
            # Simple Moving Averages
            if data_len >= 20:
                 indicators['SMA_20'] = price_data['Close'].rolling(window=20).mean().iloc[-1]
            else:
                 indicators['SMA_20'] = np.nan
                 logger.warning("Insufficient data for SMA_20.")

            if data_len >= 50:
                 indicators['SMA_50'] = price_data['Close'].rolling(window=50).mean().iloc[-1]
            else:
                 indicators['SMA_50'] = np.nan
                 logger.warning("Insufficient data for SMA_50.")

            if data_len >= 200:
                 indicators['SMA_200'] = price_data['Close'].rolling(window=200).mean().iloc[-1]
            else:
                 indicators['SMA_200'] = np.nan
                 logger.warning("Insufficient data for SMA_200.")

            # Exponential Moving Averages
            if data_len >= 12:
                 indicators['EMA_12'] = price_data['Close'].ewm(span=12, adjust=False).mean().iloc[-1]
            else:
                 indicators['EMA_12'] = np.nan
                 logger.warning("Insufficient data for EMA_12.")

            if data_len >= 26:
                 indicators['EMA_26'] = price_data['Close'].ewm(span=26, adjust=False).mean().iloc[-1]
            else:
                 indicators['EMA_26'] = np.nan
                 logger.warning("Insufficient data for EMA_26.")

            # RSI
            if data_len >= 14:
                indicators['RSI'] = self.calculate_rsi(price_data['Close'])
            else:
                 indicators['RSI'] = np.nan
                 logger.warning("Insufficient data for RSI.")

            # MACD (requires EMA_12 and EMA_26)
            macd_line = np.nan
            signal_line = np.nan
            macd_histogram = np.nan

            if 'EMA_12' in indicators and 'EMA_26' in indicators and not np.isnan(indicators['EMA_12']) and not np.isnan(indicators['EMA_26']):
                 # Use the entire macd_line series for rolling calculation if possible
                 full_macd_line_series = price_data['Close'].ewm(span=12, adjust=False).mean() - price_data['Close'].ewm(span=26, adjust=False).mean()
                 if len(full_macd_line_series.dropna()) >= 9:
                      macd_line = full_macd_line_series.iloc[-1]
                      signal_line = full_macd_line_series.ewm(span=9, adjust=False).mean().iloc[-1]
                      if not np.isnan(signal_line):
                         macd_histogram = macd_line - signal_line
                 else:
                      logger.warning("Insufficient data for MACD signal line calculation.")
            else:
                 logger.warning("MACD not calculated due to missing EMA_12 or EMA_26.")


            indicators['MACD'] = macd_line
            indicators['MACD_signal'] = signal_line
            indicators['MACD_histogram'] = macd_histogram

            # Log MACD values after calculation
            logger.info(f"Calculated MACD: {indicators['MACD']}, Signal: {indicators['MACD_signal']}")


            # Bollinger Bands (requires SMA_20 and enough data for std dev)
            bb_period = 20
            bb_std = 2
            if data_len >= bb_period:
                sma_bb = price_data['Close'].rolling(window=bb_period).mean()
                std_bb = price_data['Close'].rolling(window=bb_period).std()
                # Ensure SMA and STD are not NaN before calculating bands
                if not np.isnan(sma_bb.iloc[-1]) and not np.isnan(std_bb.iloc[-1]):
                    indicators['BB_upper'] = sma_bb.iloc[-1] + (std_bb.iloc[-1] * bb_std)
                    indicators['BB_lower'] = sma_bb.iloc[-1] - (std_bb.iloc[-1] * bb_std)
                    indicators['BB_middle'] = sma_bb.iloc[-1]
                else:
                    indicators['BB_upper'] = np.nan
                    indicators['BB_lower'] = np.nan
                    indicators['BB_middle'] = np.nan
                    logger.warning("Bollinger Bands not calculated due to NaN in SMA or STD.")
            else:
                indicators['BB_upper'] = np.nan
                indicators['BB_lower'] = np.nan
                indicators['BB_middle'] = np.nan
                logger.warning("Insufficient data for Bollinger Bands.")


            # Volume indicators
            if data_len >= 20:
                 indicators['Volume_SMA'] = price_data['Volume'].rolling(window=20).mean().iloc[-1]
                 if indicators['Volume_SMA'] is not None and not np.isnan(indicators['Volume_SMA']) and indicators['Volume_SMA'] > 0:
                      indicators['Volume_ratio'] = price_data['Volume'].iloc[-1] / indicators['Volume_SMA']
                 else:
                      indicators['Volume_ratio'] = np.nan
                      logger.warning("Volume SMA is zero or NaN, Volume ratio not calculated.")
            else:
                 indicators['Volume_SMA'] = np.nan
                 indicators['Volume_ratio'] = np.nan
                 logger.warning("Insufficient data for Volume SMA and ratio.")


            # Price position (requires SMA_20 and SMA_50)
            current_price = price_data['Close'].iloc[-1] if data_len > 0 and 'Close' in price_data and not price_data['Close'].empty else np.nan
            indicators['current_price'] = current_price # Store current price

            if current_price is not None and not np.isnan(current_price) and 'SMA_20' in indicators and indicators['SMA_20'] is not None and not np.isnan(indicators['SMA_20']) and indicators['SMA_20'] > 0:
                indicators['Price_to_SMA20'] = ((current_price / indicators['SMA_20'] - 1) * 100)
            else:
                 indicators['Price_to_SMA20'] = np.nan
                 logger.warning("Price to SMA20 not calculated due to missing current price or SMA20.")

            if current_price is not None and not np.isnan(current_price) and 'SMA_50' in indicators and indicators['SMA_50'] is not None and not np.isnan(indicators['SMA_50']) and indicators['SMA_50'] > 0:
                 indicators['Price_to_SMA50'] = ((current_price / indicators['SMA_50'] - 1) * 100)
            else:
                 indicators['Price_to_SMA50'] = np.nan
                 logger.warning("Price to SMA50 not calculated due to missing current price or SMA50.")


            # Support and Resistance (requires enough data for rolling max/min)
            if data_len >= 20:
                 indicators['Resistance'] = price_data['High'].rolling(window=20).max().iloc[-1]
                 indicators['Support'] = price_data['Low'].rolling(window=20).min().iloc[-1]
            else:
                 indicators['Resistance'] = np.nan
                 indicators['Support'] = np.nan
                 logger.warning("Insufficient data for Support and Resistance.")


            return indicators

        except Exception as e:
            logger.error(f"Error calculating technical indicators: {str(e)}")
            return {}

    def calculate_rsi(self, prices: pd.Series, period: int = 14) -> Optional[float]:
        """Calculate Relative Strength Index"""
        try:
            if len(prices) < period:
                return np.nan # Return NaN for insufficient data

            delta = prices.diff()
            gain = delta.where(delta > 0, 0)
            loss = -delta.where(delta < 0, 0)

            avg_gain = gain.rolling(window=period).mean()
            avg_loss = loss.rolling(window=period).mean()

            # Handle division by zero if avg_loss is zero or NaN
            if avg_loss.iloc[-1] is None or np.isnan(avg_loss.iloc[-1]) or avg_loss.iloc[-1] == 0:
                 return 100.0 if avg_gain.iloc[-1] is not None and not np.isnan(avg_gain.iloc[-1]) and avg_gain.iloc[-1] > 0 else 50.0 # If no loss, RSI is 100 (if gain) or 50 (if no change)
            elif avg_loss.iloc[-1] > 0:
                rs = avg_gain / avg_loss
                rsi = 100 - (100 / (1 + rs))
                return rsi.iloc[-1]
            else:
                 return np.nan # Should not happen with abs, but as a safeguard


        except Exception as e:
            logger.error(f"Error calculating RSI: {str(e)}")
            return np.nan # Return NaN on error

    def calculate_fundamental_score(self, info: Dict, benchmarks: Dict) -> Tuple[float, Dict]:
        """Calculate fundamental analysis score based on Indian market standards"""
        score = 0
        max_score = 50
        metrics = {}

        try:
            # P/E Ratio (10 points)
            pe_ratio = info.get('trailingPE', info.get('forwardPE', None))
            if pe_ratio is not None and isinstance(pe_ratio, (int, float)) and not np.isnan(pe_ratio):
                metrics['PE_Ratio'] = pe_ratio
                if pe_ratio > 0: # Ensure PE is positive for scoring
                     if pe_ratio <= benchmarks['pe_ratio']['excellent']:
                         score += 10
                     elif pe_ratio <= benchmarks['pe_ratio']['good']:
                         score += 7
                     elif pe_ratio <= benchmarks['pe_ratio']['fair']:
                         score += 4
                     else:
                         score += 2
                else:
                    score += 1 # Small score for non-positive PE
            else:
                 metrics['PE_Ratio'] = np.nan
                 logger.warning("PE Ratio not available or is invalid.")


            # Debt to Equity (10 points)
            debt_to_equity_raw = info.get('debtToEquity', None)
            debt_to_equity = np.nan # Initialize to NaN

            if debt_to_equity_raw is not None and isinstance(debt_to_equity_raw, (int, float)) and not np.isnan(debt_to_equity_raw):
                 # Handle potential outliers (very high values might indicate data issues)
                 if debt_to_equity_raw > 10000: # Arbitrary large threshold for outlier detection
                      logger.warning(f"Debt to Equity raw value is unusually high: {debt_to_equity_raw}. Treating as potential data issue.")
                      # Could assign a very low score or NaN, depending on policy
                      debt_to_equity = np.nan # Treat as invalid data for now
                 else:
                      # Heuristic for percentage vs decimal
                      if debt_to_equity_raw > 100: # Likely a percentage
                           debt_to_equity = debt_to_equity_raw / 100.0
                      else: # Assume it's already a decimal
                           debt_to_equity = debt_to_equity_raw

                      # Re-check if the converted value is still reasonable (e.g., not from a huge raw decimal)
                      if debt_to_equity > 100: # Another threshold after conversion
                          logger.warning(f"Debt to Equity value after conversion is still very high: {debt_to_equity:.2f}. May indicate data issue or highly leveraged company.")
                          # Could assign a low score or cap it, for now let the scoring handle it.


            metrics['Debt_to_Equity'] = debt_to_equity

            # Score based on the validated debt_to_equity
            if debt_to_equity is not None and not np.isnan(debt_to_equity):
                if debt_to_equity <= benchmarks['debt_to_equity']['excellent']:
                    score += 10
                elif debt_to_equity <= benchmarks['debt_to_equity']['good']:
                    score += 7
                elif debt_to_equity <= benchmarks['debt_to_equity']['fair']:
                    score += 4
                else:
                    score += 2
            else:
                 logger.warning("Debt to Equity not available, invalid, or an outlier. Assigning low score.")
                 score += 1 # Assign minimal score if D/E is not valid


            # ROE (10 points)
            roe = info.get('returnOnEquity', None)
            if roe is not None and isinstance(roe, (int, float)) and not np.isnan(roe):
                metrics['ROE'] = roe
                if roe >= benchmarks['roe']['excellent']:
                    score += 10
                elif roe >= benchmarks['roe']['good']:
                    score += 7
                elif roe >= benchmarks['roe']['fair']:
                    score += 4
                else:
                    score += 2
            else:
                 metrics['ROE'] = np.nan
                 logger.warning("ROE not available or is invalid.")

            # Revenue Growth (10 points)
            revenue_growth = info.get('revenueGrowth', None)
            if revenue_growth is not None and isinstance(revenue_growth, (int, float)) and not np.isnan(revenue_growth):
                metrics['Revenue_Growth'] = revenue_growth
                if revenue_growth >= benchmarks['revenue_growth']['excellent']:
                    score += 10
                elif revenue_growth >= benchmarks['revenue_growth']['good']:
                    score += 7
                elif revenue_growth >= benchmarks['revenue_growth']['fair']:
                    score += 4
                else:
                    score += 2
            else:
                 metrics['Revenue_Growth'] = np.nan
                 logger.warning("Revenue Growth not available or is invalid.")

            # Net Profit Margin (10 points)
            profit_margin = info.get('profitMargins', None)
            if profit_margin is not None and isinstance(profit_margin, (int, float)) and not np.isnan(profit_margin):
                metrics['Net_Profit_Margin'] = profit_margin
                if profit_margin >= benchmarks['net_profit_margin']['excellent']:
                    score += 10
                elif profit_margin >= benchmarks['net_profit_margin']['good']:
                    score += 7
                elif profit_margin >= benchmarks['net_profit_margin']['fair']:
                    score += 4
                else:
                    score += 2
            else:
                 metrics['Net_Profit_Margin'] = np.nan
                 logger.warning("Net Profit Margin not available or is invalid.")


            # Normalize score to max_score (already capped by scoring logic)
            metrics['Fundamental_Score'] = score

            return score, metrics

        except Exception as e:
            logger.error(f"Error calculating fundamental score: {str(e)}")
            return 0, {}

    def calculate_technical_score(self, indicators: Dict) -> float:
        """Calculate technical analysis score"""
        score = 0
        max_score = 50
        indicator_scores = {} # To store scores for each indicator

        try:
            # RSI Score (10 points)
            rsi = indicators.get('RSI', None)
            if rsi is not None and not np.isnan(rsi):
                 if 40 <= rsi <= 60:  # Neutral zone
                     indicator_scores['RSI'] = 7
                 elif rsi < 40:  # Oversold
                     indicator_scores['RSI'] = 10
                 elif 60 < rsi <= 70:  # Overbought but not extreme
                     indicator_scores['RSI'] = 5
                 else:  # Extreme levels
                     indicator_scores['RSI'] = 2
            else:
                 indicator_scores['RSI'] = 0 # Assign 0 if NaN/None
                 logger.warning("RSI not calculated or is NaN/None. Assigning 0 score for RSI.")


            # MACD Score (10 points)
            macd = indicators.get('MACD', None)
            macd_signal = indicators.get('MACD_signal', None)

            # Explicitly check if both are numbers before comparing
            if isinstance(macd, (int, float)) and isinstance(macd_signal, (int, float)) and not np.isnan(macd) and not np.isnan(macd_signal):
                 if macd > macd_signal:
                     indicator_scores['MACD'] = 10
                 else:
                     indicator_scores['MACD'] = 3
            else:
                 indicator_scores['MACD'] = 0 # Assign 0 if NaN/None
                 logger.warning("MACD or MACD signal not calculated or is not a valid number. Assigning 0 score for MACD.")


            # Moving Average Score (10 points) - Using SMA_20 vs SMA_50
            sma_20 = indicators.get('SMA_20', None)
            sma_50 = indicators.get('SMA_50', None)
            if sma_20 is not None and sma_50 is not None and not np.isnan(sma_20) and not np.isnan(sma_50):
                if sma_20 > sma_50:
                    indicator_scores['Moving_Averages'] = 10
                else:
                    indicator_scores['Moving_Averages'] = 3
            else:
                indicator_scores['Moving_Averages'] = 0 # Assign 0 if NaN/None
                logger.warning("SMA_20 or SMA_50 not calculated or is NaN/None. Assigning 0 score for Moving Averages.")

            # Bollinger Bands Score (10 points)
            current_price = indicators.get('current_price', None) # Assuming current_price is passed in indicators
            bb_upper = indicators.get('BB_upper', None)
            bb_lower = indicators.get('BB_lower', None)
            bb_middle = indicators.get('BB_middle', None)

            if current_price is not None and bb_upper is not None and bb_lower is not None and bb_middle is not None and \
               not np.isnan(current_price) and not np.isnan(bb_upper) and not np.isnan(bb_lower) and not np.isnan(bb_middle):
                if bb_lower < current_price < bb_middle:
                    indicator_scores['Bollinger_Bands'] = 10  # Good buying zone
                elif bb_middle < current_price < bb_upper:
                    indicator_scores['Bollinger_Bands'] = 7   # Neutral to positive
                else:
                    indicator_scores['Bollinger_Bands'] = 3   # Near extremes
            else:
                 indicator_scores['Bollinger_Bands'] = 0 # Assign 0 if NaN/None
                 logger.warning("Bollinger Bands or current price not available or is NaN/None. Assigning 0 score for Bollinger Bands.")


            # Volume Score (10 points)
            volume_ratio = indicators.get('Volume_ratio', None)
            if volume_ratio is not None and not np.isnan(volume_ratio):
                 if 0.8 <= volume_ratio <= 1.5:
                     indicator_scores['Volume'] = 7   # Normal volume
                 elif volume_ratio > 1.5:
                     indicator_scores['Volume'] = 10  # High volume (positive if with price up)
                 else:
                     indicator_scores['Volume'] = 3   # Low volume
            else:
                 indicator_scores['Volume'] = 0 # Assign 0 if NaN/None
                 logger.warning("Volume ratio not calculated or is NaN/None. Assigning 0 score for Volume.")

            # Sum the scores from individual indicators
            score = sum(indicator_scores.values())

            return min(score, max_score) # Cap at max_score


        except Exception as e:
            logger.error(f"Error calculating technical score: {str(e)}")
            return 0

    def generate_recommendation(self, fundamental_score: float, technical_score: float) -> str:
        """Generate investment recommendation based on scores"""
        total_score = fundamental_score + technical_score

        if total_score >= 75:
            return "STRONG BUY"
        elif total_score >= 60:
            return "BUY"
        elif total_score >= 40:
            return "HOLD"
        elif total_score >= 25:
            return "SELL"
        else:
            return "STRONG SELL"

    def analyze_stock(self, symbol: str) -> Optional[StockData]:
        """Main method to analyze a stock"""
        try:
            logger.info(f"Starting analysis for {symbol}")

            # Get stock data
            ticker = self.get_stock_data(symbol)
            if not ticker:
                return None

            # Get stock info
            info = ticker.info
            if not info:
                logger.error(f"No info available for {symbol}")
                return None

            # Get historical price data
            # Fetching enough data for 200-day SMA calculation
            price_data = ticker.history(period="1y") # Changed to 1 year to support 200-day SMA
            if price_data.empty: # Check if price_data is empty
                logger.error(f"No price data available for {symbol}")
                return None
            # Also check for essential columns
            if not all(col in price_data.columns for col in ['Open', 'High', 'Low', 'Close', 'Volume']):
                 logger.error(f"Price data for {symbol} is missing essential columns.")
                 return None


            # Get current price
            current_price = price_data['Close'].iloc[-1] if not price_data['Close'].empty else np.nan

            # Add current price to indicators for technical score calculation
            indicators = self.calculate_technical_indicators(price_data)
            indicators['current_price'] = current_price # Ensure current price is available for BB score

            # Get industry benchmarks
            sector = info.get('sector', 'Unknown')
            benchmarks = self.get_industry_benchmarks(sector)

            # Calculate scores
            fundamental_score, metrics = self.calculate_fundamental_score(info, benchmarks)
            technical_score = self.calculate_technical_score(indicators)

            # Generate recommendation
            recommendation = self.generate_recommendation(fundamental_score, technical_score)

            # Create StockData object
            stock_data = StockData(
                symbol=symbol,
                company_name=info.get('longName', symbol),
                current_price=current_price if not np.isnan(current_price) else 0.0, # Store 0 if current price is NaN
                market_cap=info.get('marketCap', 0) if info.get('marketCap') is not None else 0, # Ensure marketCap is not None
                info=info,
                price_data=price_data,
                indicators=indicators,
                fundamental_score=fundamental_score,
                technical_score=technical_score,
                recommendation=recommendation,
                metrics=metrics
            )

            logger.info(f"Analysis completed for {symbol}")
            return stock_data

        except Exception as e:
            logger.error(f"Error analyzing {symbol}: {str(e)}")
            return None

    def display_analysis(self, stock_data: StockData) -> None:
        """Display analysis results in a formatted manner"""
        print(f"\n{'='*60}")
        print(f"Stock Analysis Report: {stock_data.company_name} ({stock_data.symbol})")
        print(f"{'='*60}")

        print(f"\n📊 Current Market Data:")
        print(f"Current Price: ₹{stock_data.current_price:,.2f}" if stock_data.current_price is not None else "Current Price: N/A")
        print(f"Market Cap: ₹{stock_data.market_cap/10000000:,.2f} Cr" if stock_data.market_cap is not None else "Market Cap: N/A") # Use stored market_cap
        print(f"Sector: {stock_data.info.get('sector', 'N/A')}")

        print(f"\n📈 Technical Indicators:")
        # Display indicators, handling None values
        for indicator, value in stock_data.indicators.items():
            if indicator != 'current_price': # Avoid displaying current_price here
                if value is not None and not np.isnan(value):
                    if isinstance(value, float):
                        print(f"{indicator}: {value:,.2f}")
                    else:
                        print(f"{indicator}: {value}")
                else:
                    print(f"{indicator}: N/A (Insufficient Data)")


        print(f"\n💰 Fundamental Metrics:")
        for metric, value in stock_data.metrics.items():
            if metric != 'Fundamental_Score':
                if value is not None and not np.isnan(value):
                     if isinstance(value, float):
                         if metric in ['ROE', 'Revenue_Growth', 'Net_Profit_Margin', 'Debt_to_Equity']:
                             print(f"{metric}: {value*100:.2f}%") # Display as percentage for D/E, ROE, Growth, Margin
                         else:
                             print(f"{metric}: {value:.2f}")
                     else:
                         print(f"{metric}: {value}")
                else:
                    print(f"{metric}: N/A (Missing Data)")


        print(f"\n📊 Analysis Scores:")
        print(f"Fundamental Score: {stock_data.fundamental_score}/50")
        print(f"Technical Score: {stock_data.technical_score}/50")
        print(f"Total Score: {stock_data.fundamental_score + stock_data.technical_score}/100")

        print(f"\n🎯 Recommendation: {stock_data.recommendation}")
        print(f"{'='*60}\n")

## Enhance circuit limit calculation

### Subtask:
Make the circuit limit calculation more accurate by potentially fetching real-time circuit percentages or using more sophisticated logic based on exchange rules.


**Reasoning**:
Modify the `get_circuit_limits` function in the `EnhancedStockAnalyzer` class to enhance the accuracy of circuit limit calculation by categorizing stocks based on index membership and applying different default percentages. Document the limitations in a comment.



In [ ]:
#@title Sentimental analysis with API (Enhanced Circuit Limits)
# Indian Stock Analyzer - Part 2 (Improved Version with Enhanced Circuit Limits)
# This continues from your Part 1 implementation

# Import the Python SDK for Gemini API
# from google.colab import userdata # Already imported in a previous cell
# import google.generativeai as genai # Already imported in a previous cell

import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import requests
from bs4 import BeautifulSoup
import warnings
warnings.filterwarnings('ignore')

# Import the Python SDK for Gemini API
# Used to securely store your API key
# from google.colab import userdata # Already imported
# import google.generativeai as genai # Already imported

# Configure Gemini API (ensure GOOGLE_API_KEY is set in Colab secrets)
# GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY') # Already done in a previous cell
# genai.configure(api_key=GOOGLE_API_KEY) # Already done in a previous cell
# gemini_model = genai.GenerativeModel('gemini-2.0-flash') # Using a more recent model # Already done in a previous cell


class EnhancedStockAnalyzer:
    """Enhanced analyzer with additional features for Indian markets"""

    def __init__(self):
        self.risk_free_rate = self.get_dynamic_risk_free_rate()
        # self.gemini_model = gemini_model # Use the globally configured model # Already assigned in a previous cell

    def get_dynamic_risk_free_rate(self):
        """Fetch current 10-year G-Sec yield"""
        try:
            # Try to get from a reliable source
            # For now, using a realistic current rate
            # In production, you'd scrape from RBI or use an API
            return 0.072  # 7.2% as of recent data
        except:
            return 0.065  # Fallback to 6.5%

    def get_circuit_limits(self, symbol):
        """
        Get circuit breaker limits for the stock.
        NOTE: This is a simplification. Actual NSE circuit limits vary based on
        stock category (e.g., EQ, BE, T group), stage of price band reduction,
        and are dynamic. Relying on fixed percentages based on index membership
        is an approximation. For production, fetch from NSE or a reliable API.
        """
        try:
            ticker = yf.Ticker(symbol)
            info = ticker.info
            prev_close = info.get('previousClose', None) # Use None as default

            if prev_close is None or not isinstance(prev_close, (int, float)) or np.isnan(prev_close) or prev_close <= 0:
                logger.warning(f"Previous close not available or invalid for {symbol}. Cannot calculate circuit limits.")
                return None

            # Enhanced NSE circuit limits (based on index membership approximation)
            # This is still a simplification of complex exchange rules.
            circuit_percent = 0.10 # Default to 10%

            # Check if the stock is in a major index for higher limits
            if symbol in self.get_index_membership(symbol): # Reuse existing index check
                 if 'NIFTY50' in self.get_index_membership(symbol):
                      circuit_percent = 0.20 # 20% for NIFTY50 stocks
                 elif 'NIFTY_NEXT50' in self.get_index_membership(symbol):
                      circuit_percent = 0.15 # Example: 15% for NIFTY Next 50 (approximation)


            return {
                'upper_circuit': prev_close * (1 + circuit_percent),
                'lower_circuit': prev_close * (1 - circuit_percent),
                'circuit_percent': circuit_percent
            }
        except Exception as e:
            logger.error(f"Error getting circuit limits for {symbol}: {str(e)}")
            return None

    def assess_circuit_risk(self, current_price, circuit_limits):
        """Assess proximity to circuit limits"""
        if not circuit_limits or current_price is None or np.isnan(current_price) or current_price <= 0:
            return 0

        upper_circuit = circuit_limits.get('upper_circuit')
        lower_circuit = circuit_limits.get('lower_circuit')

        if upper_circuit is None or lower_circuit is None or np.isnan(upper_circuit) or np.isnan(lower_circuit):
             logger.warning("Circuit limits are invalid, cannot assess circuit risk.")
             return 0

        upper_distance = (upper_circuit - current_price) / current_price if current_price > 0 else float('inf')
        lower_distance = (current_price - lower_circuit) / current_price if current_price > 0 else float('inf')


        # Risk scoring based on proximity
        if upper_distance < 0.02:  # Within 2% of upper circuit
            return -20  # High risk
        elif upper_distance < 0.05:  # Within 5%
            return -10
        elif lower_distance < 0.02:  # Within 2% of lower circuit
            return -25  # Very high risk
        elif lower_distance < 0.05:
            return -15

        return 0

    def get_liquidity_score(self, price_data: pd.DataFrame) -> float:
        """Enhanced liquidity analysis"""
        try:
            # Explicitly check for sufficient data and 'Volume' column
            if price_data.empty or 'Volume' not in price_data.columns or len(price_data) < 20:
                 logger.warning("Insufficient price data or missing Volume column for liquidity calculation.")
                 return 5 # Assign a very low liquidity score

            # Calculate various liquidity metrics
            avg_volume_20d = price_data['Volume'].rolling(20).mean().iloc[-1]
            avg_volume_5d = price_data['Volume'].rolling(5).mean().iloc[-1]
            current_close = price_data['Close'].iloc[-1] if not price_data['Close'].empty else np.nan

            if current_close is None or np.isnan(current_close) or current_close <= 0:
                 logger.warning("Current close price is invalid for liquidity calculation.")
                 return 5

            # Average daily turnover
            avg_turnover_20d = avg_volume_20d * current_close if avg_volume_20d is not None and not np.isnan(avg_volume_20d) else np.nan

            # Volume spike detection - handle division by zero explicitly
            volume_spike = np.nan
            if avg_volume_20d is not None and not np.isnan(avg_volume_20d) and avg_volume_20d > 0:
                volume_spike = avg_volume_5d / avg_volume_20d if avg_volume_5d is not None and not np.isnan(avg_volume_5d) else np.nan
            elif avg_volume_5d is not None and not np.isnan(avg_volume_5d) and avg_volume_5d > 0: # Case where 20d avg is 0 but 5d is not
                 volume_spike = 1 # Treat as normal if 20d is 0 but 5d has volume
            else:
                 logger.warning("Average volume is zero or NaN, cannot calculate volume spike.")


            # Volume consistency - handle division by zero explicitly
            volume_std = price_data['Volume'].rolling(20).std().iloc[-1] if len(price_data) >= 20 else np.nan
            volume_cv = np.nan
            if avg_volume_20d is not None and not np.isnan(avg_volume_20d) and avg_volume_20d > 0:
                volume_cv = volume_std / avg_volume_20d if volume_std is not None and not np.isnan(volume_std) else np.nan
            elif volume_std is not None and not np.isnan(volume_std) and volume_std > 0: # Case where 20d avg is 0 but std is not (unlikely but safeguard)
                 volume_cv = 1 # Treat as high variability if avg is 0 but std is not
            else:
                 logger.warning("Average volume or volume standard deviation is zero or NaN, cannot calculate volume CV.")


            # Liquidity scoring
            liquidity_score = 0

            # Turnover-based scoring - handle NaN
            if avg_turnover_20d is not None and not np.isnan(avg_turnover_20d):
                if avg_turnover_20d > 500_00_00_000:  # > 500 Cr
                    liquidity_score += 40
                elif avg_turnover_20d > 100_00_00_000:  # > 100 Cr
                    liquidity_score += 30
                elif avg_turnover_20d > 10_00_00_000:  # > 10 Cr
                    liquidity_score += 20
                elif avg_turnover_20d > 1_00_00_000:  # > 1 Cr
                    liquidity_score += 10
                else:
                    liquidity_score += 5
            else:
                 logger.warning("Average turnover is NaN, skipping turnover score.")


            # Volume consistency score - handle NaN
            if volume_cv is not None and not np.isnan(volume_cv):
                 if volume_cv < 0.5:  # Low volatility in volume
                     liquidity_score += 20
                 elif volume_cv < 1.0:
                     liquidity_score += 10
            else:
                 logger.warning("Volume CV is NaN, skipping volume consistency score.")

            # Recent volume trend score - handle NaN
            if volume_spike is not None and not np.isnan(volume_spike):
                 if volume_spike > 1.2:  # 20% higher recent volume
                     liquidity_score += 10
            else:
                 logger.warning("Volume spike is NaN, skipping recent volume trend score.")


            return max(5, min(liquidity_score, 70))  # Cap at 70, minimum score 5 for insufficient data

        except Exception as e: # Catch specific exceptions for better debugging
            logger.error(f"Error calculating liquidity score: {str(e)}")
            return 5  # Default very low liquidity on error


    def get_peer_companies(self, symbol, stock_info):
        """Identify peer companies for comparison"""
        try:
            sector = stock_info.get('sector', '')
            market_cap = stock_info.get('marketCap', 0)

            # Define market cap categories
            if market_cap > 10_00_000_00_00_000:  # > 10 Lakh Cr
                cap_category = 'Large Cap'
            elif market_cap > 50_000_00_00_000:   # > 50K Cr
                cap_category = 'Mid Cap'
            else:
                cap_category = 'Small Cap'

            # Peer mapping (simplified - in production, use a comprehensive database)
            peer_map = {
                'Technology': {
                    'Large Cap': ['TCS.NS', 'INFY.NS', 'WIPRO.NS', 'HCLTECH.NS'],
                    'Mid Cap': ['TECHM.NS', 'LTTS.NS', 'PERSISTENT.NS', 'COFORGE.NS'],
                    'Small Cap': ['ZENSAR.NS', 'ECLERX.NS', 'MASTEK.NS']
                },
                'Financial Services': {
                    'Large Cap': ['HDFCBANK.NS', 'ICICIBANK.NS', 'KOTAKBANK.NS', 'AXISBANK.NS'],
                    'Mid Cap': ['IDFCFIRSTB.NS', 'FEDERALBNK.NS', 'INDUSINDBK.NS'],
                    'Small Cap': ['SOUTHBANK.NS', 'UJJIVAN.NS', 'EQUITASBNK.NS']
                },
                'Healthcare': {
                    'Large Cap': ['SUNPHARMA.NS', 'DRREDDY.NS', 'CIPLA.NS', 'DIVISLAB.NS'],
                    'Mid Cap': ['TORNTPHARM.NS', 'ALKEM.NS', 'LUPIN.NS'],
                    'Small Cap': ['CAPLIPOINT.NS', 'BLISS.NS', 'INDOCO.NS']
                }
                # Add more sectors as needed
            }

            # Get peers, excluding the current symbol
            peers = peer_map.get(sector, {}).get(cap_category, [])
            peers = [p for p in peers if p != symbol]

            return peers[:5]  # Return top 5 peers

        except Exception as e:
            logger.error(f"Error getting peer companies: {str(e)}")
            return []

    def calculate_peer_relative_performance(self, symbol, peers, days=30):
        """Compare stock performance with peers"""
        try:
            results = {}

            # Get performance for main stock
            main_ticker = yf.Ticker(symbol)
            main_hist = main_ticker.history(period=f"{days}d")
            if len(main_hist) > 0 and 'Close' in main_hist.columns and not main_hist['Close'].empty:
                main_return = ((main_hist['Close'].iloc[-1] / main_hist['Close'].iloc[0]) - 1) * 100 if main_hist['Close'].iloc[0] > 0 else np.nan
            else:
                main_return = np.nan
                logger.warning(f"Insufficient historical data for {symbol} to calculate peer relative performance.")


            # Get peer performances
            peer_returns = []
            for peer in peers:
                try:
                    peer_ticker = yf.Ticker(peer)
                    peer_hist = peer_ticker.history(period=f"{days}d")
                    if len(peer_hist) > 0 and 'Close' in peer_hist.columns and not peer_hist['Close'].empty and peer_hist['Close'].iloc[0] > 0:
                        peer_return = ((peer_hist['Close'].iloc[-1] / peer_hist['Close'].iloc[0]) - 1) * 100
                        peer_returns.append(peer_return)
                    else:
                         logger.warning(f"Insufficient historical data for peer {peer}.")
                except Exception as e:
                    logger.warning(f"Could not get peer history for {peer}: {str(e)}")
                    continue

            if peer_returns and main_return is not None and not np.isnan(main_return):
                avg_peer_return = np.mean(peer_returns)
                relative_performance = main_return - avg_peer_return

                results = {
                    'stock_return': main_return,
                    'avg_peer_return': avg_peer_return,
                    'relative_performance': relative_performance,
                    'outperformance': relative_performance > 0
                }
            else:
                 logger.warning("Peer returns not available or main stock return is NaN, cannot calculate relative performance.")


            return results

        except Exception as e:
            logger.error(f"Error calculating peer relative performance: {str(e)}")
            return {}

    def get_index_membership(self, symbol):
        """Check index membership for the stock"""
        # Simplified index constituents (in production, fetch from NSE)
        index_constituents = {
            'NIFTY50': ['RELIANCE.NS', 'TCS.NS', 'HDFCBANK.NS', 'INFY.NS', 'ICICIBANK.NS'],
            'NIFTY_NEXT50': ['VEDL.NS', 'PNB.NS', 'INDIGO.NS', 'BANDHANBNK.NS'],
            'NIFTY_MIDCAP150': ['PERSISTENT.NS', 'COFORGE.NS', 'LTTS.NS']
        }

        membership = []
        for index, constituents in index_constituents.items():
            if symbol in constituents:
                membership.append(index)

        return membership

    def calculate_enhanced_technical_score(self, indicators: Dict, circuit_risk: float) -> float:
        """Calculate enhanced technical analysis score"""
        base_score = 0
        indicator_scores = {} # To store scores for each indicator

        try:
            # RSI Score (points based on significance, e.g., max 10 for oversold)
            rsi = indicators.get('RSI', None)
            if rsi is not None and not np.isnan(rsi):
                 if rsi < 30: indicator_scores['RSI'] = 10 # Oversold
                 elif 30 <= rsi < 40: indicator_scores['RSI'] = 8 # Approaching oversold
                 elif 40 <= rsi <= 60: indicator_scores['RSI'] = 5 # Neutral zone
                 elif 60 < rsi <= 70: indicator_scores['RSI'] = 3 # Approaching overbought
                 else: indicator_scores['RSI'] = 1 # Overbought
            else:
                 indicator_scores['RSI'] = 0 # Assign 0 if NaN/None
                 logger.warning("RSI not calculated or is NaN/None. Assigning 0 score for RSI.")

            # MACD scoring (points based on crossover and position relative to zero)
            macd = indicators.get('MACD', None)
            macd_signal = indicators.get('MACD_signal', None)

            if macd is not None and macd_signal is not None and not np.isnan(macd) and not np.isnan(macd_signal):
                 if macd > macd_signal:
                     if macd > 0: indicator_scores['MACD'] = 10 # Bullish crossover above zero
                     else: indicator_scores['MACD'] = 7 # Bullish crossover below zero
                 else:
                     if macd < 0: indicator_scores['MACD'] = 1 # Bearish crossover below zero
                     else: indicator_scores['MACD'] = 3 # Bearish crossover above zero
            else:
                 indicator_scores['MACD'] = 0 # Assign 0 if NaN/None
                 logger.warning("MACD or MACD signal not calculated or is NaN/None. Assigning 0 score for MACD.")

            # Moving Average Score (points based on MA crossovers and price position)
            current_price = indicators.get('current_price', None)
            sma_20 = indicators.get('SMA_20', None)
            sma_50 = indicators.get('SMA_50', None)
            sma_200 = indicators.get('SMA_200', None)

            ma_score = 0
            valid_mas = 0

            if sma_20 is not None and not np.isnan(sma_20) and current_price is not None and not np.isnan(current_price):
                 valid_mas += 1
                 if current_price > sma_20: ma_score += 5
                 else: ma_score += 1

            if sma_50 is not None and not np.isnan(sma_50) and current_price is not None and not np.isnan(current_price):
                 valid_mas += 1
                 if current_price > sma_50: ma_score += 5
                 else: ma_score += 1

            if sma_200 is not None and not np.isnan(sma_200) and current_price is not None and not np.isnan(current_price):
                 valid_mas += 1
                 if current_price > sma_200: ma_score += 5
                 else: ma_score += 1

            # Add points for bullish crossovers
            if sma_20 is not None and sma_50 is not None and not np.isnan(sma_20) and not np.isnan(sma_50):
                 if sma_20 > sma_50: ma_score += 5

            if sma_50 is not None and sma_200 is not None and not np.isnan(sma_50) and not np.isnan(sma_200):
                 if sma_50 > sma_200: ma_score += 5

            indicator_scores['Moving_Averages'] = ma_score # Max possible MA score is 25 (5+5+5 for price > MA + 5+5 for crossovers)

            if valid_mas == 0:
                 logger.warning("No valid Moving Averages calculated. Assigning 0 score for Moving Averages.")
                 indicator_scores['Moving_Averages'] = 0


            # Bollinger Bands Score (points based on price position relative to bands)
            bb_upper = indicators.get('BB_upper', None)
            bb_lower = indicators.get('BB_lower', None)
            bb_middle = indicators.get('BB_middle', None)

            if current_price is not None and bb_upper is not None and bb_lower is not None and bb_middle is not None and \
               not np.isnan(current_price) and not np.isnan(bb_upper) and not np.isnan(bb_lower) and not np.isnan(bb_middle):
                if current_price < bb_lower: indicator_scores['Bollinger_Bands'] = 10 # Price below lower band (potential buy signal)
                elif bb_lower <= current_price < bb_middle: indicator_scores['Bollinger_Bands'] = 7 # Between lower and middle band
                elif bb_middle <= current_price < bb_upper: indicator_scores['Bollinger_Bands'] = 3 # Between middle and upper band
                else: indicator_scores['Bollinger_Bands'] = 1 # Price above upper band (potential sell signal)
            else:
                 indicator_scores['Bollinger_Bands'] = 0 # Assign 0 if NaN/None
                 logger.warning("Bollinger Bands or current price not available or is NaN/None. Assigning 0 score for Bollinger Bands.")


            # Volume Score (points based on volume relative to average)
            volume_ratio = indicators.get('Volume_ratio', None)
            if volume_ratio is not None and not np.isnan(volume_ratio):
                 if volume_ratio > 1.5: indicator_scores['Volume'] = 10 # High volume
                 elif 0.8 <= volume_ratio <= 1.5: indicator_scores['Volume'] = 7 # Normal volume
                 else: indicator_scores['Volume'] = 3 # Low volume
            else:
                 indicator_scores['Volume'] = 0 # Assign 0 if NaN/None
                 logger.warning("Volume ratio not calculated or is NaN/None. Assigning 0 score for Volume.")

            # Sum the scores from individual indicators
            base_score = sum(indicator_scores.values())

            # Add circuit risk adjustment (already handled as negative points)
            final_score = base_score + circuit_risk

            # Ensure score is within 0-50 range (assuming max base_score is around 50 based on point allocation)
            # Max possible indicator score (10+10+25+10+10) = 65. Let's normalize this to 50.
            normalized_base_score = (base_score / 65.0) * 50.0 if base_score > 0 else 0
            final_score = normalized_base_score + circuit_risk


            return max(0, min(final_score, 50))


        except Exception as e:
            logger.error(f"Error calculating enhanced technical score: {str(e)}")
            return 0


    def fetch_news(self, symbol: str) -> List[str]:
        """
        Fetches recent news headlines for the given stock symbol.
        NOTE: This is a placeholder. In a real application, you would integrate
        with a News API (like News API, Alpha Vantage, etc.) here.
        """
        print(f"Fetching news for {symbol} (placeholder)...")
        # Placeholder news headlines - replace with actual API call
        return [
            f"{symbol} stock price rises on positive market sentiment",
            f"{symbol} announces strong quarterly results, beating estimates",
            f"Experts bullish on {symbol}'s future outlook",
            f"{symbol} faces regulatory challenges in key market",
            f"Competitor's new product launch impacts {symbol}'s market share",
            f"Global market trends positively impacting {symbol}",
            f"{symbol} management provides optimistic guidance",
            f"Increased foreign investment flows into {symbol}",
            f"Regulatory approval received for {symbol}'s new product",
            f"Production issues reported for {symbol}",
            f"Increased competition puts pressure on {symbol}'s margins",
            f"Analyst downgrades rating for {symbol}",
            f"Supply chain disruptions affect {symbol}'s operations",
            f"Geopolitical tensions create uncertainty for {symbol}",
            f"{symbol} announces stock split",
            f"Dividend declared by {symbol}"
        ]


    def analyze_sentiment(self, news_headlines: List[str]) -> Dict:
        """
        Analyzes the sentiment of news headlines using the Gemini API.
        Returns a dictionary with sentiment counts (positive, neutral, negative)
        and a sentiment score.
        """
        # Use the globally defined gemini_model
        if not 'gemini_model' in globals() or not gemini_model or not news_headlines:
            print("Gemini API not configured or no news to analyze. Skipping sentiment analysis.")
            return {'positive': 0, 'neutral': 0, 'negative': 0, 'score': 0, 'summary': 'Sentiment analysis skipped.'}

        logger.info("Analyzing sentiment using Gemini API...")
        prompt = "Analyze the sentiment of the following news headlines for a stock and categorize each as Positive, Neutral, or Negative. Provide a concise summary of the overall sentiment.\n\n"
        for headline in news_headlines:
            prompt += f"- {headline}\n"
        prompt += "\nSentiment Analysis Results (Categorization and Summary):\n"

        try:
            response = gemini_model.generate_content(prompt)
            sentiment_text = response.text.strip()
            logger.info(f"Gemini API Response: {sentiment_text[:200]}...") # Log first 200 chars

            # Parse the response to count sentiments and extract summary
            positive_count = sentiment_text.lower().count('positive')
            neutral_count = sentiment_text.lower().count('neutral')
            negative_count = sentiment_text.lower().count('negative')

            # Attempt to extract a summary line - this is a heuristic
            summary_line = "No summary extracted."
            lines = sentiment_text.split('\n')
            for line in lines:
                 lower_line = line.lower()
                 if 'summary' in lower_line or 'overall sentiment' in lower_line:
                     summary_line = line.strip()
                     break
            if summary_line == "No summary extracted." and lines:
                 # If no explicit summary found, take the last non-empty line as a potential summary
                 for line in reversed(lines):
                     if line.strip():
                         summary_line = line.strip()
                         break


            total_headlines = len(news_headlines)
            if total_headlines > 0:
                # Scoring based on the ratio of positive vs negative headlines, scaled to 25 points
                # (Positive - Negative) / Total * 12.5 + 12.5 -> Range 0-25
                sentiment_score = ((positive_count - negative_count) / total_headlines) * 12.5 + 12.5
            else:
                sentiment_score = 12.5 # Neutral score if no headlines

            return {
                'positive': positive_count,
                'neutral': neutral_count,
                'negative': negative_count,
                'score': max(0, min(sentiment_score, 25)), # Cap score between 0 and 25
                'summary': summary_line
            }

        except Exception as e:
            logger.error(f"Error analyzing sentiment with Gemini API: {e}. Skipping sentiment analysis.")
            return {'positive': 0, 'neutral': 0, 'negative': 0, 'score': 0, 'summary': 'Sentiment analysis failed.'}


    def generate_enhanced_recommendation(self, fundamental_score, technical_score,
                                       liquidity_score, peer_performance, index_membership,
                                       sentiment_analysis_results):
        """Generate comprehensive recommendation including sentiment"""
        sentiment_score = sentiment_analysis_results.get('score', 0)
        total_score = fundamental_score + technical_score + liquidity_score + sentiment_score # Add sentiment score

        # Adjust for peer performance
        if peer_performance.get('outperformance', False):
            total_score += 5

        # Bonus for index membership
        if index_membership:
            total_score += 5

        # Max possible score is 50 (Fundamental) + 50 (Technical) + 70 (Liquidity) + 25 (Sentiment) + 5 (Peer) + 5 (Index) = 205
        # Let's re-evaluate the recommendation thresholds based on this potential range

        if total_score >= 160: # Example Thresholds (adjust as needed)
            recommendation = "STRONG BUY"
            confidence = "High"
        elif total_score >= 130:
            recommendation = "BUY"
            confidence = "Moderate to High"
        elif total_score >= 90:
            recommendation = "HOLD"
            confidence = "Moderate"
        elif total_score >= 60:
            return "SELL"
        else:
            return "STRONG SELL"

        return {
            'recommendation': recommendation,
            'confidence': confidence,
            'total_score': total_score,
            'breakdown': {
                'fundamental': fundamental_score,
                'technical': technical_score,
                'liquidity': liquidity_score,
                'sentiment': sentiment_score
            }
        }

# Integration function to work with your Part 1
def analyze_stock_enhanced(symbol, stock_analyzer_part1, enhanced_analyzer):
    """
    Integrate enhanced features with your Part 1 analyzer

    Args:
        symbol: Stock symbol (e.g., 'RELIANCE.NS')
        stock_analyzer_part1: Instance of your IndianStockAnalyzer from Part 1
        enhanced_analyzer: Instance of EnhancedStockAnalyzer
    """

    # Get basic analysis from Part 1
    stock_data = stock_analyzer_part1.analyze_stock(symbol)

    if not stock_data:
        return None

    # Get current price
    current_price = stock_data.current_price

    # Enhanced analysis
    # 1. Circuit breaker analysis
    circuit_limits = enhanced_analyzer.get_circuit_limits(symbol)
    circuit_risk = enhanced_analyzer.assess_circuit_risk(current_price, circuit_limits)

    # 2. Enhanced liquidity analysis
    liquidity_score = enhanced_analyzer.get_liquidity_score(stock_data.price_data)

    # 3. Peer analysis
    peers = enhanced_analyzer.get_peer_companies(symbol, stock_data.info)
    peer_performance = enhanced_analyzer.calculate_peer_relative_performance(symbol, peers)

    # 4. Index membership
    index_membership = enhanced_analyzer.get_index_membership(symbol)

    # 5. Recalculate technical score with circuit risk
    enhanced_technical_score = enhanced_analyzer.calculate_enhanced_technical_score(
        stock_data.indicators, circuit_risk
    )

    # 6. Fetch and analyze news sentiment
    news_headlines = enhanced_analyzer.fetch_news(symbol)
    sentiment_analysis_results = enhanced_analyzer.analyze_sentiment(news_headlines)


    # 7. Generate enhanced recommendation
    final_recommendation = enhanced_analyzer.generate_enhanced_recommendation(
        stock_data.fundamental_score,
        enhanced_technical_score,
        liquidity_score,
        peer_performance,
        index_membership,
        sentiment_analysis_results # Pass sentiment analysis results
    )

    # Create enhanced report
    enhanced_report = {
        'basic_analysis': stock_data,
        'enhanced_features': {
            'circuit_limits': circuit_limits,
            'circuit_risk_score': circuit_risk,
            'liquidity_score': liquidity_score,
            'peers': peers,
            'peer_performance': peer_performance,
            'index_membership': index_membership,
            'dynamic_risk_free_rate': enhanced_analyzer.risk_free_rate,
            'news_headlines': news_headlines, # Include news in report
            'sentiment_analysis': sentiment_analysis_results # Include sentiment analysis results
        },
        'final_recommendation': final_recommendation
    }

    return enhanced_report

# Example usage (removed from here as it's part of the next display step)

**Reasoning**:
Display the combined analysis report to show the effect of the enhanced circuit limit calculation and other features.



In [ ]:
#@title Analyze a stock and display the combined results
symbol_to_analyze = "RELIANCE.NS" # You can change this symbol

# Ensure both analyzers are instantiated (they should be if the cells above were run)
# If not, you might need to re-run the cells defining IndianStockAnalyzer and EnhancedStockAnalyzer
try:
    stock_analyzer = IndianStockAnalyzer()
    enhanced_analyzer = EnhancedStockAnalyzer()
except NameError:
    print("Please run the cells defining IndianStockAnalyzer and EnhancedStockAnalyzer first.")
    # Exit or handle the error appropriately if classes are not defined
    stock_analyzer = None
    enhanced_analyzer = None


if stock_analyzer and enhanced_analyzer:
    combined_analysis_report = analyze_stock_enhanced(symbol_to_analyze, stock_analyzer, enhanced_analyzer)

    if combined_analysis_report:
        print(f"\n{'='*60}")
        print(f"Combined Stock Analysis Report for {symbol_to_analyze}")
        print(f"{'='*60}")

        # Display basic analysis results
        basic = combined_analysis_report['basic_analysis']
        print(f"\n--- Basic Analysis (from Part 1) ---")
        print(f"Company Name: {basic.company_name}")
        print(f"Current Price: ₹{basic.current_price:,.2f}" if basic.current_price is not None else "Current Price: N/A")
        print(f"Market Cap: ₹{basic.market_cap/10000000:,.2f} Cr" if basic.market_cap is not None else "Market Cap: N/A") # Use stored market_cap
        print(f"Sector: {basic.info.get('sector', 'N/A')}")
        print(f"Fundamental Score: {basic.fundamental_score}/50")
        print(f"Technical Score: {basic.technical_score}/50")
        print(f"Basic Recommendation: {basic.recommendation}")
        print("\nFundamental Metrics:")
        for metric, value in basic.metrics.items():
             if metric != 'Fundamental_Score':
                if value is not None and not np.isnan(value):
                     if isinstance(value, float):
                         if metric in ['ROE', 'Revenue_Growth', 'Net_Profit_Margin', 'Debt_to_Equity']:
                             print(f"{metric}: {value*100:.2f}%") # Display as percentage for D/E, ROE, Growth, Margin
                         else:
                             print(f"{metric}: {value:.2f}")
                     else:
                         print(f"{metric}: {value}")
                else:
                    print(f"{metric}: N/A (Missing Data)")


        print("\nTechnical Indicators:")
        for indicator, value in basic.indicators.items():
             if indicator != 'current_price':
                if value is not None and not np.isnan(value):
                    if isinstance(value, float):
                        print(f"{indicator}: {value:,.2f}")
                    else:
                        print(f"{indicator}: {value}")
                else:
                    print(f"{indicator}: N/A (Insufficient Data)")


        # Display enhanced features and final recommendation
        enhanced = combined_analysis_report['enhanced_features']
        final = combined_analysis_report['final_recommendation']

        print(f"\n--- Enhanced Analysis (from Part 2) ---")
        print(f"Circuit Limits: {enhanced.get('circuit_limits', {}).get('lower_circuit', 'N/A'):.2f} - {enhanced.get('circuit_limits', {}).get('upper_circuit', 'N/A'):.2f}" if enhanced.get('circuit_limits') else "Circuit Limits: N/A")
        print(f"Circuit Risk Score: {enhanced.get('circuit_risk_score', 'N/A')}")
        print(f"Liquidity Score: {enhanced.get('liquidity_score', 'N/A')}/70")

        if enhanced.get('peers'):
            print(f"Peer Companies: {', '.join(enhanced['peers'])}")
            if enhanced.get('peer_performance'):
                print(f"30-day Return: {enhanced['peer_performance'].get('stock_return', 'N/A'):.2f}%" if enhanced['peer_performance'].get('stock_return') is not None else "30-day Return: N/A")
                print(f"Peer Avg Return: {enhanced['peer_performance'].get('avg_peer_return', 'N/A'):.2f}%" if enhanced['peer_performance'].get('avg_peer_return') is not None else "Peer Avg Return: N/A")
                print(f"Relative Performance: {enhanced['peer_performance'].get('relative_performance', 'N/A'):.2f}%" if enhanced['peer_performance'].get('relative_performance') is not None else "Relative Performance: N/A")

        if enhanced.get('index_membership'):
            print(f"Index Membership: {', '.join(enhanced['index_membership'])}")

        # Sentiment Analysis Results
        sentiment_results = enhanced.get('sentiment_analysis', {})
        print(f"\n📰 News Sentiment Analysis:")
        if enhanced.get('news_headlines'):
             for headline in enhanced['news_headlines']:
                 print(f"- {headline}")
             print(f"Sentiment Counts: Positive={sentiment_results.get('positive', 0)}, Neutral={sentiment_results.get('neutral', 0)}, Negative={sentiment_results.get('negative', 0)}")
             print(f"Sentiment Score: {sentiment_results.get('score', 0):.2f}/25")
             print(f"Sentiment Summary: {sentiment_results.get('summary', 'N/A')}")
        else:
             print("No news headlines fetched or sentiment analysis skipped.")


        print(f"\n--- Final Recommendation ---")
        print(f"RECOMMENDATION: {final.get('recommendation', 'N/A')} ({final.get('confidence', 'N/A')})")
        print(f"Total Score: {final.get('total_score', 0):.2f}/205")
        print("Score Breakdown:")
        print(f"  - Fundamental: {final.get('breakdown', {}).get('fundamental', 'N/A'):.2f}")
        print(f"  - Technical (Enhanced): {final.get('breakdown', {}).get('technical', 'N/A')}") # Technical score might not be float always
        print(f"  - Liquidity: {final.get('breakdown', {}).get('liquidity', 'N/A'):.2f}")
        print(f"  - Sentiment: {final.get('breakdown', {}).get('sentiment', 'N/A'):.2f}")

        print(f"\n{'='*60}")

    else:
        print(f"Could not generate combined analysis report for {symbol_to_analyze}.")

## Refine industry benchmarks

### Subtask:
Review and potentially expand the industry benchmarks for fundamental metrics to cover more sectors or allow for custom benchmarks.


**Reasoning**:
Review the existing industry benchmarks, identify missing sectors, and consider adding more sectors to the benchmarks dictionary or outlining a plan for dynamic benchmarks.



In [ ]:
# Step 1: Examine existing industry_benchmarks
print("--- Review of Existing Industry Benchmarks ---")
analyzer = IndianStockAnalyzer() # Instantiate to access benchmarks
existing_benchmarks = analyzer.industry_benchmarks
default_benchmarks = analyzer.default_benchmarks

print("Existing Industry Benchmarks:")
for sector, metrics in existing_benchmarks.items():
    print(f"- {sector.capitalize()}: {list(metrics.keys())}")

print("\nDefault Benchmarks:")
print(list(default_benchmarks.keys()))

# Step 2: Identify missing major Indian market sectors
# Common major sectors in India include:
# - IT (covered)
# - Banking (covered)
# - Pharma (covered)
# - FMCG (covered)
# - Auto (covered)
# - Infrastructure (covered)
# - Energy (covered)
# - Realty (covered)
# - Telecom (covered)
# - Metals & Mining
# - Capital Goods
# - Chemicals
# - Textiles
# - Media & Entertainment
# - Consumer Durables
# - Utilities
# - Services
# - Construction

missing_sectors = [
    'metals & mining',
    'capital goods',
    'chemicals',
    'textiles',
    'media & entertainment',
    'consumer durables',
    'utilities',
    'services',
    'construction'
]

print("\nPotentially Missing Major Sectors:")
for sector in missing_sectors:
    print(f"- {sector.title()}")

# Step 3: Consider static vs dynamic/custom benchmarks
print("\n--- Consideration of Static vs Dynamic/Custom Benchmarks ---")
print(
    "The current implementation uses static benchmarks defined within the class. "
    "This is a good starting point but may not always be representative of current "
    "market conditions or specific sub-sectors."
)
print(
    "Dynamic benchmarks (e.g., calculating median/average metrics for peers or the sector "
    "from recent data) or allowing custom benchmarks would enhance accuracy."
)

# Step 4: Suggest improvements - Add more sectors or outline a plan
print("\n--- Suggested Improvements for Benchmarks ---")
print("Option 1: Add more sectors to the `industry_benchmarks` dictionary.")
print("This requires research to define appropriate 'excellent', 'good', and 'fair' thresholds for each metric in these sectors.")
print("Example sectors to consider adding: Metals & Mining, Capital Goods, Chemicals, Utilities.")

print("\nOption 2: Outline a plan for implementing dynamic or custom benchmarks.")
print("Plan:")
print("a. **Dynamic Benchmarks:** Modify `get_industry_benchmarks` to fetch recent data for a sector or peer group and calculate median/average fundamental metrics to use as benchmarks.")
print("   - Requires robust data fetching for multiple symbols.")
print("   - Needs logic to handle sectors with limited data or peers.")
print("b. **Custom Benchmarks:** Add parameters to the `analyze_stock` or `IndianStockAnalyzer` constructor to allow users to provide their own benchmark dictionary.")
print("   - Requires validation of the custom benchmark structure.")
print("c. **Hybrid Approach:** Maintain static benchmarks as a fallback but prioritize dynamic or custom benchmarks if available.")

print("\nRationale for not making immediate changes to benchmarks:")
print("Adding comprehensive, accurate benchmarks for all major sectors requires significant financial domain expertise and research. Implementing dynamic benchmarks is a more complex feature requiring additional data fetching and calculation logic.")

# Step 5: Document changes or rationale (already included in print statements above)

**Reasoning**:
Based on the review, several major sectors are missing. I will add a few of the identified missing sectors with plausible benchmark values to the `industry_benchmarks` dictionary in the `IndianStockAnalyzer` class. This directly addresses the subtask of expanding the benchmarks. I will also reiterate the limitations of static benchmarks and the possibility of future dynamic/custom benchmark implementation.



In [ ]:
#@title Part 1 of Main script (Expanded Benchmarks)
# Indian Stock Analyzer - Part 1 (Corrected and Enhanced further)
# Complete implementation with improvements for missing data handling and Debt-to-Equity validation
# and expanded industry benchmarks

import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from dataclasses import dataclass
from typing import Dict, List, Optional, Tuple
import logging
import warnings
warnings.filterwarnings('ignore')

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__) # Define logger globally

@dataclass
class StockData:
    """Data class to store comprehensive stock information"""
    symbol: str
    company_name: str
    current_price: float
    market_cap: float
    info: Dict
    price_data: pd.DataFrame
    indicators: Dict
    fundamental_score: float
    technical_score: float
    recommendation: str
    metrics: Dict

class IndianStockAnalyzer:
    """Comprehensive stock analyzer for Indian markets"""

    def __init__(self):
        """Initialize the analyzer with Indian market specific parameters"""
        self.nifty_symbol = "^NSEI"
        self.risk_free_rate = 0.065  # Will be made dynamic in Part 2

        # Industry-specific benchmark thresholds for Indian markets
        # Expanded with additional sectors
        self.industry_benchmarks = {
            'technology': {
                'pe_ratio': {'excellent': 25, 'good': 35, 'fair': 45},
                'debt_to_equity': {'excellent': 0.3, 'good': 0.5, 'fair': 0.8},
                'roe': {'excellent': 0.25, 'good': 0.18, 'fair': 0.12},
                'revenue_growth': {'excellent': 0.20, 'good': 0.15, 'fair': 0.10},
                'net_profit_margin': {'excellent': 0.20, 'good': 0.15, 'fair': 0.10}
            },
            'banking': {
                'pe_ratio': {'excellent': 15, 'good': 20, 'fair': 25},
                'debt_to_equity': {'excellent': 8, 'good': 10, 'fair': 12}, # Higher D/E is normal for banks
                'roe': {'excellent': 0.15, 'good': 0.12, 'fair': 0.08},
                'revenue_growth': {'excellent': 0.15, 'good': 0.10, 'fair': 0.05},
                'net_profit_margin': {'excellent': 0.25, 'good': 0.20, 'fair': 0.15}
            },
            'pharmaceutical': {
                'pe_ratio': {'excellent': 20, 'good': 30, 'fair': 40},
                'debt_to_equity': {'excellent': 0.4, 'good': 0.6, 'fair': 0.8},
                'roe': {'excellent': 0.20, 'good': 0.15, 'fair': 0.10},
                'revenue_growth': {'excellent': 0.15, 'good': 0.10, 'fair': 0.05},
                'net_profit_margin': {'excellent': 0.15, 'good': 0.10, 'fair': 0.08}
            },
            'fmcg': {
                'pe_ratio': {'excellent': 30, 'good': 40, 'fair': 50},
                'debt_to_equity': {'excellent': 0.5, 'good': 0.8, 'fair': 1.0},
                'roe': {'excellent': 0.30, 'good': 0.25, 'fair': 0.15},
                'revenue_growth': {'excellent': 0.12, 'good': 0.08, 'fair': 0.05},
                'net_profit_margin': {'excellent': 0.15, 'good': 0.10, 'fair': 0.08}
            },
            'automobile': {
                'pe_ratio': {'excellent': 15, 'good': 25, 'fair': 35},
                'debt_to_equity': {'excellent': 0.6, 'good': 0.8, 'fair': 1.2},
                'roe': {'excellent': 0.20, 'good': 0.15, 'fair': 0.10},
                'revenue_growth': {'excellent': 0.15, 'good': 0.10, 'fair': 0.05},
                'net_profit_margin': {'excellent': 0.10, 'good': 0.07, 'fair': 0.05}
            },
            'infrastructure': {
                'pe_ratio': {'excellent': 12, 'good': 18, 'fair': 25},
                'debt_to_equity': {'excellent': 1.0, 'good': 1.5, 'fair': 2.0},
                'roe': {'excellent': 0.15, 'good': 0.12, 'fair': 0.08},
                'revenue_growth': {'excellent': 0.20, 'good': 0.15, 'fair': 0.10},
                'net_profit_margin': {'excellent': 0.12, 'good': 0.08, 'fair': 0.05}
            },
            'energy': {
                'pe_ratio': {'excellent': 10, 'good': 15, 'fair': 20},
                'debt_to_equity': {'excellent': 0.8, 'good': 1.2, 'fair': 1.5},
                'roe': {'excellent': 0.15, 'good': 0.12, 'fair': 0.08},
                'revenue_growth': {'excellent': 0.15, 'good': 0.10, 'fair': 0.05},
                'net_profit_margin': {'excellent': 0.10, 'good': 0.07, 'fair': 0.05}
            },
            'realty': {
                'pe_ratio': {'excellent': 10, 'good': 15, 'fair': 25},
                'debt_to_equity': {'excellent': 1.0, 'good': 1.5, 'fair': 2.0},
                'roe': {'excellent': 0.12, 'good': 0.08, 'fair': 0.05},
                'revenue_growth': {'excellent': 0.25, 'good': 0.15, 'fair': 0.10},
                'net_profit_margin': {'excellent': 0.15, 'good': 0.10, 'fair': 0.05}
            },
            'telecom': {
                'pe_ratio': {'excellent': 15, 'good': 25, 'fair': 35},
                'debt_to_equity': {'excellent': 1.0, 'good': 1.5, 'fair': 2.0},
                'roe': {'excellent': 0.10, 'good': 0.07, 'fair': 0.05},
                'revenue_growth': {'excellent': 0.10, 'good': 0.07, 'fair': 0.05},
                'net_profit_margin': {'excellent': 0.08, 'good': 0.05, 'fair': 0.03}
            },
             # Added missing major sectors (with illustrative benchmarks - these should be validated)
            'metals & mining': {
                'pe_ratio': {'excellent': 8, 'good': 12, 'fair': 18}, # Lower P/E typical
                'debt_to_equity': {'excellent': 0.8, 'good': 1.3, 'fair': 1.8}, # Can be leveraged
                'roe': {'excellent': 0.18, 'good': 0.12, 'fair': 0.08},
                'revenue_growth': {'excellent': 0.15, 'good': 0.10, 'fair': 0.05},
                'net_profit_margin': {'excellent': 0.10, 'good': 0.07, 'fair': 0.04}
            },
             'capital goods': {
                'pe_ratio': {'excellent': 20, 'good': 30, 'fair': 40},
                'debt_to_equity': {'excellent': 0.5, 'good': 0.8, 'fair': 1.2},
                'roe': {'excellent': 0.18, 'good': 0.14, 'fair': 0.10},
                'revenue_growth': {'excellent': 0.18, 'good': 0.13, 'fair': 0.08},
                'net_profit_margin': {'excellent': 0.10, 'good': 0.07, 'fair': 0.04}
            },
             'chemicals': {
                'pe_ratio': {'excellent': 25, 'good': 35, 'fair': 45},
                'debt_to_equity': {'excellent': 0.4, 'good': 0.6, 'fair': 0.9},
                'roe': {'excellent': 0.20, 'good': 0.15, 'fair': 0.10},
                'revenue_growth': {'excellent': 0.15, 'good': 0.10, 'fair': 0.06},
                'net_profit_margin': {'excellent': 0.12, 'good': 0.08, 'fair': 0.05}
            },
             'utilities': {
                'pe_ratio': {'excellent': 10, 'good': 15, 'fair': 20},
                'debt_to_equity': {'excellent': 1.5, 'good': 2.0, 'fair': 3.0}, # Higher D/E is common
                'roe': {'excellent': 0.12, 'good': 0.09, 'fair': 0.06},
                'revenue_growth': {'excellent': 0.08, 'good': 0.05, 'fair': 0.03}, # Stable, lower growth
                'net_profit_margin': {'excellent': 0.15, 'good': 0.10, 'fair': 0.07} # Stable margins
            },
             'construction': {
                'pe_ratio': {'excellent': 10, 'good': 16, 'fair': 22},
                'debt_to_equity': {'excellent': 1.0, 'good': 1.5, 'fair': 2.0},
                'roe': {'excellent': 0.15, 'good': 0.10, 'fair': 0.07},
                'revenue_growth': {'excellent': 0.20, 'good': 0.15, 'fair': 0.10},
                'net_profit_margin': {'excellent': 0.08, 'good': 0.05, 'fair': 0.03}
            },
              'consumer durables': {
                'pe_ratio': {'excellent': 35, 'good': 45, 'fair': 60}, # Higher P/E common
                'debt_to_equity': {'excellent': 0.4, 'good': 0.7, 'fair': 1.0},
                'roe': {'excellent': 0.25, 'good': 0.18, 'fair': 0.12},
                'revenue_growth': {'excellent': 0.18, 'good': 0.12, 'fair': 0.08},
                'net_profit_margin': {'excellent': 0.10, 'good': 0.07, 'fair': 0.04}
            },
             'textiles': {
                'pe_ratio': {'excellent': 15, 'good': 22, 'fair': 30},
                'debt_to_equity': {'excellent': 0.7, 'good': 1.0, 'fair': 1.5},
                'roe': {'excellent': 0.15, 'good': 0.10, 'fair': 0.07},
                'revenue_growth': {'excellent': 0.12, 'good': 0.08, 'fair': 0.05},
                'net_profit_margin': {'excellent': 0.08, 'good': 0.05, 'fair': 0.03}
            }


        }

        # Default benchmarks for other sectors
        self.default_benchmarks = {
            'pe_ratio': {'excellent': 15, 'good': 25, 'fair': 35},
            'debt_to_equity': {'excellent': 0.5, 'good': 1.0, 'fair': 1.5},
            'roe': {'excellent': 0.15, 'good': 0.10, 'fair': 0.07},
            'revenue_growth': {'excellent': 0.15, 'good': 0.10, 'fair': 0.05},
            'net_profit_margin': {'excellent': 0.10, 'good': 0.07, 'fair': 0.05}
        }

    def get_stock_data(self, symbol: str) -> Optional[yf.Ticker]:
        """Fetch stock data from Yahoo Finance"""
        try:
            ticker = yf.Ticker(symbol)
            # Verify the ticker is valid by checking if it has info
            if ticker.info and 'symbol' in ticker.info:
                return ticker
            else:
                logger.error(f"Invalid symbol: {symbol}")
                return None
        except Exception as e:
            logger.error(f"Error fetching data for {symbol}: {str(e)}")
            return None

    def get_industry_benchmarks(self, sector: str) -> Dict:
        """Get industry-specific benchmarks"""
        sector_lower = sector.lower() if sector else ''

        # Map common sector names to our benchmark categories
        sector_mapping = {
            'technology': 'technology',
            'information technology': 'technology',
            'financial services': 'banking',
            'financials': 'banking',
            'healthcare': 'pharmaceutical',
            'consumer defensive': 'fmcg',
            'consumer staples': 'fmcg', # Alias
            'consumer cyclical': 'automobile',
            'industrials': 'infrastructure', # Broad mapping
            'energy': 'energy',
            'real estate': 'realty',
            'communication services': 'telecom',
            'metals and mining': 'metals & mining', # Map to new category
            'capital goods': 'capital goods',
            'chemicals': 'chemicals',
            'textiles': 'textiles',
            'media': 'media & entertainment', # Broad mapping
            'consumer durables': 'consumer durables',
            'utilities': 'utilities',
            'services': 'services', # Keep 'services' even if no specific benchmark yet
            'construction': 'construction'
        }

        benchmark_key = sector_mapping.get(sector_lower, None)

        if benchmark_key and benchmark_key in self.industry_benchmarks:
            return self.industry_benchmarks[benchmark_key]
        else:
            logger.info(f"Using default benchmarks for sector: {sector}")
            return self.default_benchmarks

    def calculate_technical_indicators(self, price_data: pd.DataFrame) -> Dict:
        """Calculate comprehensive technical indicators"""
        indicators = {}
        data_len = len(price_data)

        try:
            # Simple Moving Averages
            if data_len >= 20:
                 indicators['SMA_20'] = price_data['Close'].rolling(window=20).mean().iloc[-1]
            else:
                 indicators['SMA_20'] = np.nan
                 logger.warning("Insufficient data for SMA_20.")

            if data_len >= 50:
                 indicators['SMA_50'] = price_data['Close'].rolling(window=50).mean().iloc[-1]
            else:
                 indicators['SMA_50'] = np.nan
                 logger.warning("Insufficient data for SMA_50.")

            if data_len >= 200:
                 indicators['SMA_200'] = price_data['Close'].rolling(window=200).mean().iloc[-1]
            else:
                 indicators['SMA_200'] = np.nan
                 logger.warning("Insufficient data for SMA_200.")

            # Exponential Moving Averages
            if data_len >= 12:
                 indicators['EMA_12'] = price_data['Close'].ewm(span=12, adjust=False).mean().iloc[-1]
            else:
                 indicators['EMA_12'] = np.nan
                 logger.warning("Insufficient data for EMA_12.")

            if data_len >= 26:
                 indicators['EMA_26'] = price_data['Close'].ewm(span=26, adjust=False).mean().iloc[-1]
            else:
                 indicators['EMA_26'] = np.nan
                 logger.warning("Insufficient data for EMA_26.")

            # RSI
            if data_len >= 14:
                indicators['RSI'] = self.calculate_rsi(price_data['Close'])
            else:
                 indicators['RSI'] = np.nan
                 logger.warning("Insufficient data for RSI.")

            # MACD (requires EMA_12 and EMA_26)
            macd_line = np.nan
            signal_line = np.nan
            macd_histogram = np.nan

            if 'EMA_12' in indicators and 'EMA_26' in indicators and not np.isnan(indicators['EMA_12']) and not np.isnan(indicators['EMA_26']):
                 # Use the entire macd_line series for rolling calculation if possible
                 full_macd_line_series = price_data['Close'].ewm(span=12, adjust=False).mean() - price_data['Close'].ewm(span=26, adjust=False).mean()
                 if len(full_macd_line_series.dropna()) >= 9:
                      macd_line = full_macd_line_series.iloc[-1]
                      signal_line = full_macd_line_series.ewm(span=9, adjust=False).mean().iloc[-1]
                      if not np.isnan(signal_line):
                         macd_histogram = macd_line - signal_line
                 else:
                      logger.warning("Insufficient data for MACD signal line calculation.")
            else:
                 logger.warning("MACD not calculated due to missing EMA_12 or EMA_26.")


            indicators['MACD'] = macd_line
            indicators['MACD_signal'] = signal_line
            indicators['MACD_histogram'] = macd_histogram

            # Log MACD values after calculation
            logger.info(f"Calculated MACD: {indicators['MACD']}, Signal: {indicators['MACD_signal']}")


            # Bollinger Bands (requires SMA_20 and enough data for std dev)
            bb_period = 20
            bb_std = 2
            if data_len >= bb_period:
                sma_bb = price_data['Close'].rolling(window=bb_period).mean()
                std_bb = price_data['Close'].rolling(window=bb_period).std()
                # Ensure SMA and STD are not NaN before calculating bands
                if not np.isnan(sma_bb.iloc[-1]) and not np.isnan(std_bb.iloc[-1]):
                    indicators['BB_upper'] = sma_bb.iloc[-1] + (std_bb.iloc[-1] * bb_std)
                    indicators['BB_lower'] = sma_bb.iloc[-1] - (std_bb.iloc[-1] * bb_std)
                    indicators['BB_middle'] = sma_bb.iloc[-1]
                else:
                    indicators['BB_upper'] = np.nan
                    indicators['BB_lower'] = np.nan
                    indicators['BB_middle'] = np.nan
                    logger.warning("Bollinger Bands not calculated due to NaN in SMA or STD.")
            else:
                indicators['BB_upper'] = np.nan
                indicators['BB_lower'] = np.nan
                indicators['BB_middle'] = np.nan
                logger.warning("Insufficient data for Bollinger Bands.")


            # Volume indicators
            if data_len >= 20:
                 indicators['Volume_SMA'] = price_data['Volume'].rolling(window=20).mean().iloc[-1]
                 if indicators['Volume_SMA'] is not None and not np.isnan(indicators['Volume_SMA']) and indicators['Volume_SMA'] > 0:
                      indicators['Volume_ratio'] = price_data['Volume'].iloc[-1] / indicators['Volume_SMA']
                 else:
                      indicators['Volume_ratio'] = np.nan
                      logger.warning("Volume SMA is zero or NaN, Volume ratio not calculated.")
            else:
                 indicators['Volume_SMA'] = np.nan
                 indicators['Volume_ratio'] = np.nan
                 logger.warning("Insufficient data for Volume SMA and ratio.")


            # Price position (requires SMA_20 and SMA_50)
            current_price = price_data['Close'].iloc[-1] if data_len > 0 and 'Close' in price_data and not price_data['Close'].empty else np.nan
            indicators['current_price'] = current_price # Store current price

            if current_price is not None and not np.isnan(current_price) and 'SMA_20' in indicators and indicators['SMA_20'] is not None and not np.isnan(indicators['SMA_20']) and indicators['SMA_20'] > 0:
                indicators['Price_to_SMA20'] = ((current_price / indicators['SMA_20'] - 1) * 100)
            else:
                 indicators['Price_to_SMA20'] = np.nan
                 logger.warning("Price to SMA20 not calculated due to missing current price or SMA20.")

            if current_price is not None and not np.isnan(current_price) and 'SMA_50' in indicators and indicators['SMA_50'] is not None and not np.isnan(indicators['SMA_50']) and indicators['SMA_50'] > 0:
                 indicators['Price_to_SMA50'] = ((current_price / indicators['SMA_50'] - 1) * 100)
            else:
                 indicators['Price_to_SMA50'] = np.nan
                 logger.warning("Price to SMA50 not calculated due to missing current price or SMA50.")


            # Support and Resistance (requires enough data for rolling max/min)
            if data_len >= 20:
                 indicators['Resistance'] = price_data['High'].rolling(window=20).max().iloc[-1]
                 indicators['Support'] = price_data['Low'].rolling(window=20).min().iloc[-1]
            else:
                 indicators['Resistance'] = np.nan
                 indicators['Support'] = np.nan
                 logger.warning("Insufficient data for Support and Resistance.")


            return indicators

        except Exception as e:
            logger.error(f"Error calculating technical indicators: {str(e)}")
            return {}

    def calculate_rsi(self, prices: pd.Series, period: int = 14) -> Optional[float]:
        """Calculate Relative Strength Index"""
        try:
            if len(prices) < period:
                return np.nan # Return NaN for insufficient data

            delta = prices.diff()
            gain = delta.where(delta > 0, 0)
            loss = -delta.where(delta < 0, 0)

            avg_gain = gain.rolling(window=period).mean()
            avg_loss = loss.rolling(window=period).mean()

            # Handle division by zero if avg_loss is zero or NaN
            if avg_loss.iloc[-1] is None or np.isnan(avg_loss.iloc[-1]) or avg_loss.iloc[-1] == 0:
                 return 100.0 if avg_gain.iloc[-1] is not None and not np.isnan(avg_gain.iloc[-1]) and avg_gain.iloc[-1] > 0 else 50.0 # If no loss, RSI is 100 (if gain) or 50 (if no change)
            elif avg_loss.iloc[-1] > 0:
                rs = avg_gain / avg_loss
                rsi = 100 - (100 / (1 + rs))
                return rsi.iloc[-1]
            else:
                 return np.nan # Should not happen with abs, but as a safeguard


        except Exception as e:
            logger.error(f"Error calculating RSI: {str(e)}")
            return np.nan # Return NaN on error

    def calculate_fundamental_score(self, info: Dict, benchmarks: Dict) -> Tuple[float, Dict]:
        """Calculate fundamental analysis score based on Indian market standards"""
        score = 0
        max_score = 50
        metrics = {}

        try:
            # P/E Ratio (10 points)
            pe_ratio = info.get('trailingPE', info.get('forwardPE', None))
            if pe_ratio is not None and isinstance(pe_ratio, (int, float)) and not np.isnan(pe_ratio):
                metrics['PE_Ratio'] = pe_ratio
                if pe_ratio > 0: # Ensure PE is positive for scoring
                     if pe_ratio <= benchmarks['pe_ratio']['excellent']:
                         score += 10
                     elif pe_ratio <= benchmarks['pe_ratio']['good']:
                         score += 7
                     elif pe_ratio <= benchmarks['pe_ratio']['fair']:
                         score += 4
                     else:
                         score += 2
                else:
                    score += 1 # Small score for non-positive PE
            else:
                 metrics['PE_Ratio'] = np.nan
                 logger.warning("PE Ratio not available or is invalid.")


            # Debt to Equity (10 points)
            debt_to_equity_raw = info.get('debtToEquity', None)
            debt_to_equity = np.nan # Initialize to NaN

            if debt_to_equity_raw is not None and isinstance(debt_to_equity_raw, (int, float)) and not np.isnan(debt_to_equity_raw):
                 # Handle potential outliers (very high values might indicate data issues)
                 if debt_to_equity_raw > 10000: # Arbitrary large threshold for outlier detection
                      logger.warning(f"Debt to Equity raw value is unusually high: {debt_to_equity_raw}. Treating as potential data issue.")
                      # Could assign a very low score or NaN, depending on policy
                      debt_to_equity = np.nan # Treat as invalid data for now
                 else:
                      # Heuristic for percentage vs decimal
                      if debt_to_equity_raw > 100: # Likely a percentage
                           debt_to_equity = debt_to_equity_raw / 100.0
                      else: # Assume it's already a decimal
                           debt_to_equity = debt_to_equity_raw

                      # Re-check if the converted value is still reasonable (e.g., not from a huge raw decimal)
                      if debt_to_equity is not None and not np.isnan(debt_to_equity) and debt_to_equity > 100: # Another threshold after conversion
                          logger.warning(f"Debt to Equity value after conversion is still very high: {debt_to_equity:.2f}. May indicate data issue or highly leveraged company.")
                          # Could assign a low score or cap it, for now let the scoring handle it.


            metrics['Debt_to_Equity'] = debt_to_equity

            # Score based on the validated debt_to_equity
            if debt_to_equity is not None and not np.isnan(debt_to_equity):
                if debt_to_equity <= benchmarks['debt_to_equity']['excellent']:
                    score += 10
                elif debt_to_equity <= benchmarks['debt_to_equity']['good']:
                    score += 7
                elif debt_to_equity <= benchmarks['debt_to_equity']['fair']:
                    score += 4
                else:
                    score += 2
            else:
                 logger.warning("Debt to Equity not available, invalid, or an outlier. Assigning low score.")
                 score += 1 # Assign minimal score if D/E is not valid


            # ROE (10 points)
            roe = info.get('returnOnEquity', None)
            if roe is not None and isinstance(roe, (int, float)) and not np.isnan(roe):
                metrics['ROE'] = roe
                if roe >= benchmarks['roe']['excellent']:
                    score += 10
                elif roe >= benchmarks['roe']['good']:
                    score += 7
                elif roe >= benchmarks['roe']['fair']:
                    score += 4
                else:
                    score += 2
            else:
                 metrics['ROE'] = np.nan
                 logger.warning("ROE not available or is invalid.")

            # Revenue Growth (10 points)
            revenue_growth = info.get('revenueGrowth', None)
            if revenue_growth is not None and isinstance(revenue_growth, (int, float)) and not np.isnan(revenue_growth):
                metrics['Revenue_Growth'] = revenue_growth
                if revenue_growth >= benchmarks['revenue_growth']['excellent']:
                    score += 10
                elif revenue_growth >= benchmarks['revenue_growth']['good']:
                    score += 7
                elif revenue_growth >= benchmarks['revenue_growth']['fair']:
                    score += 4
                else:
                    score += 2
            else:
                 metrics['Revenue_Growth'] = np.nan
                 logger.warning("Revenue Growth not available or is invalid.")

            # Net Profit Margin (10 points)
            profit_margin = info.get('profitMargins', None)
            if profit_margin is not None and isinstance(profit_margin, (int, float)) and not np.isnan(profit_margin):
                metrics['Net_Profit_Margin'] = profit_margin
                if profit_margin >= benchmarks['net_profit_margin']['excellent']:
                    score += 10
                elif profit_margin >= benchmarks['net_profit_margin']['good']:
                    score += 7
                elif profit_margin >= benchmarks['net_profit_margin']['fair']:
                    score += 4
                else:
                    score += 2
            else:
                 metrics['Net_Profit_Margin'] = np.nan
                 logger.warning("Net Profit Margin not available or is invalid.")


            # Normalize score to max_score (already capped by scoring logic)
            metrics['Fundamental_Score'] = score

            return score, metrics

        except Exception as e:
            logger.error(f"Error calculating fundamental score: {str(e)}")
            return 0, {}

    def calculate_technical_score(self, indicators: Dict) -> float:
        """Calculate technical analysis score"""
        score = 0
        max_score = 50
        indicator_scores = {} # To store scores for each indicator

        try:
            # RSI Score (10 points)
            rsi = indicators.get('RSI', None)
            if rsi is not None and not np.isnan(rsi):
                 if 40 <= rsi <= 60:  # Neutral zone
                     indicator_scores['RSI'] = 7
                 elif rsi < 40:  # Oversold
                     indicator_scores['RSI'] = 10
                 elif 60 < rsi <= 70:  # Overbought but not extreme
                     indicator_scores['RSI'] = 5
                 else:  # Extreme levels
                     indicator_scores['RSI'] = 2
            else:
                 indicator_scores['RSI'] = 0 # Assign 0 if NaN/None
                 logger.warning("RSI not calculated or is NaN/None. Assigning 0 score for RSI.")


            # MACD Score (10 points)
            macd = indicators.get('MACD', None)
            macd_signal = indicators.get('MACD_signal', None)

            # Explicitly check if both are numbers before comparing
            if isinstance(macd, (int, float)) and isinstance(macd_signal, (int, float)) and not np.isnan(macd) and not np.isnan(macd_signal):
                 if macd > macd_signal:
                     indicator_scores['MACD'] = 10
                 else:
                     indicator_scores['MACD'] = 3
            else:
                 indicator_scores['MACD'] = 0 # Assign 0 if NaN/None
                 logger.warning("MACD or MACD signal not calculated or is not a valid number. Assigning 0 score for MACD.")


            # Moving Average Score (10 points) - Using SMA_20 vs SMA_50
            sma_20 = indicators.get('SMA_20', None)
            sma_50 = indicators.get('SMA_50', None)
            if sma_20 is not None and sma_50 is not None and not np.isnan(sma_20) and not np.isnan(sma_50):
                if sma_20 > sma_50:
                    indicator_scores['Moving_Averages'] = 10
                else:
                    indicator_scores['Moving_Averages'] = 3
            else:
                indicator_scores['Moving_Averages'] = 0 # Assign 0 if NaN/None
                logger.warning("SMA_20 or SMA_50 not calculated or is NaN/None. Assigning 0 score for Moving Averages.")

            # Bollinger Bands Score (10 points)
            current_price = indicators.get('current_price', None) # Assuming current_price is passed in indicators
            bb_upper = indicators.get('BB_upper', None)
            bb_lower = indicators.get('BB_lower', None)
            bb_middle = indicators.get('BB_middle', None)

            if current_price is not None and bb_upper is not None and bb_lower is not None and bb_middle is not None and \
               not np.isnan(current_price) and not np.isnan(bb_upper) and not np.isnan(bb_lower) and not np.isnan(bb_middle):
                if bb_lower < current_price < bb_middle:
                    indicator_scores['Bollinger_Bands'] = 10  # Good buying zone
                elif bb_middle < current_price < bb_upper:
                    indicator_scores['Bollinger_Bands'] = 7   # Neutral to positive
                else:
                    indicator_scores['Bollinger_Bands'] = 3   # Near extremes
            else:
                 indicator_scores['Bollinger_Bands'] = 0 # Assign 0 if NaN/None
                 logger.warning("Bollinger Bands or current price not available or is NaN/None. Assigning 0 score for Bollinger Bands.")


            # Volume Score (10 points)
            volume_ratio = indicators.get('Volume_ratio', None)
            if volume_ratio is not None and not np.isnan(volume_ratio):
                 if 0.8 <= volume_ratio <= 1.5:
                     indicator_scores['Volume'] = 7   # Normal volume
                 elif volume_ratio > 1.5:
                     indicator_scores['Volume'] = 10  # High volume (positive if with price up)
                 else:
                     indicator_scores['Volume'] = 3   # Low volume
            else:
                 indicator_scores['Volume'] = 0 # Assign 0 if NaN/None
                 logger.warning("Volume ratio not calculated or is NaN/None. Assigning 0 score for Volume.")

            # Sum the scores from individual indicators
            score = sum(indicator_scores.values())

            return min(score, max_score) # Cap at max_score


        except Exception as e:
            logger.error(f"Error calculating technical score: {str(e)}")
            return 0

    def generate_recommendation(self, fundamental_score: float, technical_score: float) -> str:
        """Generate investment recommendation based on scores"""
        total_score = fundamental_score + technical_score

        if total_score >= 75:
            return "STRONG BUY"
        elif total_score >= 60:
            return "BUY"
        elif total_score >= 40:
            return "HOLD"
        elif total_score >= 25:
            return "SELL"
        else:
            return "STRONG SELL"

    def analyze_stock(self, symbol: str) -> Optional[StockData]:
        """Main method to analyze a stock"""
        try:
            logger.info(f"Starting analysis for {symbol}")

            # Get stock data
            ticker = self.get_stock_data(symbol)
            if not ticker:
                return None

            # Get stock info
            info = ticker.info
            if not info:
                logger.error(f"No info available for {symbol}")
                return None

            # Get historical price data
            # Fetching enough data for 200-day SMA calculation
            price_data = ticker.history(period="1y") # Changed to 1 year to support 200-day SMA
            if price_data.empty: # Check if price_data is empty
                logger.error(f"No price data available for {symbol}")
                return None
            # Also check for essential columns
            if not all(col in price_data.columns for col in ['Open', 'High', 'Low', 'Close', 'Volume']):
                 logger.error(f"Price data for {symbol} is missing essential columns.")
                 return None


            # Get current price
            current_price = price_data['Close'].iloc[-1] if data_len > 0 and 'Close' in price_data and not price_data['Close'].empty else np.nan

            # Add current price to indicators for technical score calculation
            indicators = self.calculate_technical_indicators(price_data)
            indicators['current_price'] = current_price # Ensure current price is available for BB score

            # Get industry benchmarks
            sector = info.get('sector', 'Unknown')
            benchmarks = self.get_industry_benchmarks(sector)

            # Calculate scores
            fundamental_score, metrics = self.calculate_fundamental_score(info, benchmarks)
            technical_score = self.calculate_technical_score(indicators)

            # Generate recommendation
            recommendation = self.generate_recommendation(fundamental_score, technical_score)

            # Create StockData object
            stock_data = StockData(
                symbol=symbol,
                company_name=info.get('longName', symbol),
                current_price=current_price if not np.isnan(current_price) else 0.0, # Store 0 if current price is NaN
                market_cap=info.get('marketCap', 0) if info.get('marketCap') is not None else 0, # Ensure marketCap is not None
                info=info,
                price_data=price_data,
                indicators=indicators,
                fundamental_score=fundamental_score,
                technical_score=technical_score,
                recommendation=recommendation,
                metrics=metrics
            )

            logger.info(f"Analysis completed for {symbol}")
            return stock_data

        except Exception as e:
            logger.error(f"Error analyzing {symbol}: {str(e)}")
            return None

    def display_analysis(self, stock_data: StockData) -> None:
        """Display analysis results in a formatted manner"""
        print(f"\n{'='*60}")
        print(f"Stock Analysis Report: {stock_data.company_name} ({stock_data.symbol})")
        print(f"{'='*60}")

        print(f"\n📊 Current Market Data:")
        print(f"Current Price: ₹{stock_data.current_price:,.2f}" if stock_data.current_price is not None else "Current Price: N/A")
        print(f"Market Cap: ₹{stock_data.market_cap/10000000:,.2f} Cr" if stock_data.market_cap is not None else "Market Cap: N/A") # Use stored market_cap
        print(f"Sector: {stock_data.info.get('sector', 'N/A')}")

        print(f"\n📈 Technical Indicators:")
        # Display indicators, handling None values
        for indicator, value in stock_data.indicators.items():
            if indicator != 'current_price': # Avoid displaying current_price here
                if value is not None and not np.isnan(value):
                    if isinstance(value, float):
                        print(f"{indicator}: {value:,.2f}")
                    else:
                        print(f"{indicator}: {value}")
                else:
                    print(f"{indicator}: N/A (Insufficient Data)")


        print(f"\n💰 Fundamental Metrics:")
        for metric, value in stock_data.metrics.items():
            if metric != 'Fundamental_Score':
                if value is not None and not np.isnan(value):
                     if isinstance(value, float):
                         if metric in ['ROE', 'Revenue_Growth', 'Net_Profit_Margin', 'Debt_to_Equity']:
                             print(f"{metric}: {value*100:.2f}%") # Display as percentage for D/E, ROE, Growth, Margin
                         else:
                             print(f"{metric}: {value:.2f}")
                     else:
                         print(f"{metric}: {value}")
                else:
                    print(f"{metric}: N/A (Missing Data)")


        print(f"\n📊 Analysis Scores:")
        print(f"Fundamental Score: {stock_data.fundamental_score}/50")
        print(f"Technical Score: {stock_data.technical_score}/50")
        print(f"Total Score: {stock_data.fundamental_score + stock_data.technical_score}/100")

        print(f"\n🎯 Recommendation: {stock_data.recommendation}")
        print(f"{'='*60}\n")


## Improve logging

### Subtask:
Improve logging throughout the script to provide more detailed information about data fetching, calculations, and potential issues.


**Reasoning**:
Implement enhanced logging by adding info and debug messages to key functions to track execution flow and variable values, as well as warning and error messages for potential issues, converting existing print statements to logger calls.



In [ ]:
#@title Part 1 of Main script (Enhanced Logging)
# Indian Stock Analyzer - Part 1 (Corrected and Enhanced further)
# Complete implementation with improvements for missing data handling and Debt-to-Equity validation
# and expanded industry benchmarks, with enhanced logging

import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from dataclasses import dataclass
from typing import Dict, List, Optional, Tuple
import logging
import warnings
warnings.filterwarnings('ignore')

# Configure logging (ensure this is only configured once if running multiple cells)
# logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
# logger = logging.getLogger(__name__) # Define logger globally

@dataclass
class StockData:
    """Data class to store comprehensive stock information"""
    symbol: str
    company_name: str
    current_price: float
    market_cap: float
    info: Dict
    price_data: pd.DataFrame
    indicators: Dict
    fundamental_score: float
    technical_score: float
    recommendation: str
    metrics: Dict

class IndianStockAnalyzer:
    """Comprehensive stock analyzer for Indian markets"""

    def __init__(self):
        """Initialize the analyzer with Indian market specific parameters"""
        logger.info("Initializing IndianStockAnalyzer.")
        self.nifty_symbol = "^NSEI"
        self.risk_free_rate = 0.065  # Will be made dynamic in Part 2

        # Industry-specific benchmark thresholds for Indian markets
        # Expanded with additional sectors
        self.industry_benchmarks = {
            'technology': {
                'pe_ratio': {'excellent': 25, 'good': 35, 'fair': 45},
                'debt_to_equity': {'excellent': 0.3, 'good': 0.5, 'fair': 0.8},
                'roe': {'excellent': 0.25, 'good': 0.18, 'fair': 0.12},
                'revenue_growth': {'excellent': 0.20, 'good': 0.15, 'fair': 0.10},
                'net_profit_margin': {'excellent': 0.20, 'good': 0.15, 'fair': 0.10}
            },
            'banking': {
                'pe_ratio': {'excellent': 15, 'good': 20, 'fair': 25},
                'debt_to_equity': {'excellent': 8, 'good': 10, 'fair': 12}, # Higher D/E is normal for banks
                'roe': {'excellent': 0.15, 'good': 0.12, 'fair': 0.08},
                'revenue_growth': {'excellent': 0.15, 'good': 0.10, 'fair': 0.05},
                'net_profit_margin': {'excellent': 0.25, 'good': 0.20, 'fair': 0.15}
            },
            'pharmaceutical': {
                'pe_ratio': {'excellent': 20, 'good': 30, 'fair': 40},
                'debt_to_equity': {'excellent': 0.4, 'good': 0.6, 'fair': 0.8},
                'roe': {'excellent': 0.20, 'good': 0.15, 'fair': 0.10},
                'revenue_growth': {'excellent': 0.15, 'good': 0.10, 'fair': 0.05},
                'net_profit_margin': {'excellent': 0.15, 'good': 0.10, 'fair': 0.08}
            },
            'fmcg': {
                'pe_ratio': {'excellent': 30, 'good': 40, 'fair': 50},
                'debt_to_equity': {'excellent': 0.5, 'good': 0.8, 'fair': 1.0},
                'roe': {'excellent': 0.30, 'good': 0.25, 'fair': 0.15},
                'revenue_growth': {'excellent': 0.12, 'good': 0.08, 'fair': 0.05},
                'net_profit_margin': {'excellent': 0.15, 'good': 0.10, 'fair': 0.08}
            },
            'automobile': {
                'pe_ratio': {'excellent': 15, 'good': 25, 'fair': 35},
                'debt_to_equity': {'excellent': 0.6, 'good': 0.8, 'fair': 1.2},
                'roe': {'excellent': 0.20, 'good': 0.15, 'fair': 0.10},
                'revenue_growth': {'excellent': 0.15, 'good': 0.10, 'fair': 0.05},
                'net_profit_margin': {'excellent': 0.10, 'good': 0.07, 'fair': 0.05}
            },
            'infrastructure': {
                'pe_ratio': {'excellent': 12, 'good': 18, 'fair': 25},
                'debt_to_equity': {'excellent': 1.0, 'good': 1.5, 'fair': 2.0},
                'roe': {'excellent': 0.15, 'good': 0.12, 'fair': 0.08},
                'revenue_growth': {'excellent': 0.20, 'good': 0.15, 'fair': 0.10},
                'net_profit_margin': {'excellent': 0.12, 'good': 0.08, 'fair': 0.05}
            },
            'energy': {
                'pe_ratio': {'excellent': 10, 'good': 15, 'fair': 20},
                'debt_to_equity': {'excellent': 0.8, 'good': 1.2, 'fair': 1.5},
                'roe': {'excellent': 0.15, 'good': 0.12, 'fair': 0.08},
                'revenue_growth': {'excellent': 0.15, 'good': 0.10, 'fair': 0.05},
                'net_profit_margin': {'excellent': 0.10, 'good': 0.07, 'fair': 0.05}
            },
            'realty': {
                'pe_ratio': {'excellent': 10, 'good': 15, 'fair': 25},
                'debt_to_equity': {'excellent': 1.0, 'good': 1.5, 'fair': 2.0},
                'roe': {'excellent': 0.12, 'good': 0.08, 'fair': 0.05},
                'revenue_growth': {'excellent': 0.25, 'good': 0.15, 'fair': 0.10},
                'net_profit_margin': {'excellent': 0.15, 'good': 0.10, 'fair': 0.05}
            },
            'telecom': {
                'pe_ratio': {'excellent': 15, 'good': 25, 'fair': 35},
                'debt_to_equity': {'excellent': 1.0, 'good': 1.5, 'fair': 2.0},
                'roe': {'excellent': 0.10, 'good': 0.07, 'fair': 0.05},
                'revenue_growth': {'excellent': 0.10, 'good': 0.07, 'fair': 0.05},
                'net_profit_margin': {'excellent': 0.08, 'good': 0.05, 'fair': 0.03}
            },
             # Added missing major sectors (with illustrative benchmarks - these should be validated)
            'metals & mining': {
                'pe_ratio': {'excellent': 8, 'good': 12, 'fair': 18}, # Lower P/E typical
                'debt_to_equity': {'excellent': 0.8, 'good': 1.3, 'fair': 1.8}, # Can be leveraged
                'roe': {'excellent': 0.18, 'good': 0.12, 'fair': 0.08},
                'revenue_growth': {'excellent': 0.15, 'good': 0.10, 'fair': 0.05},
                'net_profit_margin': {'excellent': 0.10, 'good': 0.07, 'fair': 0.04}
            },
             'capital goods': {
                'pe_ratio': {'excellent': 20, 'good': 30, 'fair': 40},
                'debt_to_equity': {'excellent': 0.5, 'good': 0.8, 'fair': 1.2},
                'roe': {'excellent': 0.18, 'good': 0.14, 'fair': 0.10},
                'revenue_growth': {'excellent': 0.18, 'good': 0.13, 'fair': 0.08},
                'net_profit_margin': {'excellent': 0.10, 'good': 0.07, 'fair': 0.04}
            },
             'chemicals': {
                'pe_ratio': {'excellent': 25, 'good': 35, 'fair': 45},
                'debt_to_equity': {'excellent': 0.4, 'good': 0.6, 'fair': 0.9},
                'roe': {'excellent': 0.20, 'good': 0.15, 'fair': 0.10},
                'revenue_growth': {'excellent': 0.15, 'good': 0.10, 'fair': 0.06},
                'net_profit_margin': {'excellent': 0.12, 'good': 0.08, 'fair': 0.05}
            },
             'utilities': {
                'pe_ratio': {'excellent': 10, 'good': 15, 'fair': 20},
                'debt_to_equity': {'excellent': 1.5, 'good': 2.0, 'fair': 3.0}, # Higher D/E is common
                'roe': {'excellent': 0.12, 'good': 0.09, 'fair': 0.06},
                'revenue_growth': {'excellent': 0.08, 'good': 0.05, 'fair': 0.03}, # Stable, lower growth
                'net_profit_margin': {'excellent': 0.15, 'good': 0.10, 'fair': 0.07} # Stable margins
            },
             'construction': {
                'pe_ratio': {'excellent': 10, 'good': 16, 'fair': 22},
                'debt_to_equity': {'excellent': 1.0, 'good': 1.5, 'fair': 2.0},
                'roe': {'excellent': 0.15, 'good': 0.10, 'fair': 0.07},
                'revenue_growth': {'excellent': 0.20, 'good': 0.15, 'fair': 0.10},
                'net_profit_margin': {'excellent': 0.08, 'good': 0.05, 'fair': 0.03}
            },
              'consumer durables': {
                'pe_ratio': {'excellent': 35, 'good': 45, 'fair': 60}, # Higher P/E common
                'debt_to_equity': {'excellent': 0.4, 'good': 0.7, 'fair': 1.0},
                'roe': {'excellent': 0.25, 'good': 0.18, 'fair': 0.12},
                'revenue_growth': {'excellent': 0.18, 'good': 0.12, 'fair': 0.08},
                'net_profit_margin': {'excellent': 0.10, 'good': 0.07, 'fair': 0.04}
            },
             'textiles': {
                'pe_ratio': {'excellent': 15, 'good': 22, 'fair': 30},
                'debt_to_equity': {'excellent': 0.7, 'good': 1.0, 'fair': 1.5},
                'roe': {'excellent': 0.15, 'good': 0.10, 'fair': 0.07},
                'revenue_growth': {'excellent': 0.12, 'good': 0.08, 'fair': 0.05},
                'net_profit_margin': {'excellent': 0.08, 'good': 0.05, 'fair': 0.03}
            }


        }

        # Default benchmarks for other sectors
        self.default_benchmarks = {
            'pe_ratio': {'excellent': 15, 'good': 25, 'fair': 35},
            'debt_to_equity': {'excellent': 0.5, 'good': 1.0, 'fair': 1.5},
            'roe': {'excellent': 0.15, 'good': 0.10, 'fair': 0.07},
            'revenue_growth': {'excellent': 0.15, 'good': 0.10, 'fair': 0.05},
            'net_profit_margin': {'excellent': 0.10, 'good': 0.07, 'fair': 0.05}
        }
        logger.info("IndianStockAnalyzer initialized.")

    def get_stock_data(self, symbol: str) -> Optional[yf.Ticker]:
        """Fetch stock data from Yahoo Finance"""
        logger.info(f"Attempting to fetch stock data for {symbol}.")
        try:
            ticker = yf.Ticker(symbol)
            # Verify the ticker is valid by checking if it has info
            if ticker.info and 'symbol' in ticker.info:
                logger.info(f"Successfully fetched ticker data for {symbol}.")
                return ticker
            else:
                logger.error(f"Invalid symbol or no info available for {symbol}.")
                return None
        except Exception as e:
            logger.error(f"Error fetching data for {symbol}: {str(e)}")
            return None

    def get_industry_benchmarks(self, sector: str) -> Dict:
        """Get industry-specific benchmarks"""
        logger.debug(f"Getting industry benchmarks for sector: {sector}")
        sector_lower = sector.lower() if sector else ''

        # Map common sector names to our benchmark categories
        sector_mapping = {
            'technology': 'technology',
            'information technology': 'technology',
            'financial services': 'banking',
            'financials': 'banking',
            'healthcare': 'pharmaceutical',
            'consumer defensive': 'fmcg',
            'consumer staples': 'fmcg', # Alias
            'consumer cyclical': 'automobile',
            'industrials': 'infrastructure', # Broad mapping
            'energy': 'energy',
            'real estate': 'realty',
            'communication services': 'telecom',
            'metals and mining': 'metals & mining', # Map to new category
            'capital goods': 'capital goods',
            'chemicals': 'chemicals',
            'textiles': 'textiles',
            'media': 'media & entertainment', # Broad mapping
            'consumer durables': 'consumer durables',
            'utilities': 'utilities',
            'services': 'services', # Keep 'services' even if no specific benchmark yet
            'construction': 'construction'
        }

        benchmark_key = sector_mapping.get(sector_lower, None)

        if benchmark_key and benchmark_key in self.industry_benchmarks:
            logger.info(f"Using specific industry benchmarks for sector: {sector}")
            return self.industry_benchmarks[benchmark_key]
        else:
            logger.info(f"Using default benchmarks for sector: {sector}")
            return self.default_benchmarks

    def calculate_technical_indicators(self, price_data: pd.DataFrame) -> Dict:
        """Calculate comprehensive technical indicators"""
        logger.info("Calculating technical indicators.")
        indicators = {}
        data_len = len(price_data)
        logger.debug(f"Price data length: {data_len}")

        try:
            # Simple Moving Averages
            if data_len >= 20:
                 indicators['SMA_20'] = price_data['Close'].rolling(window=20).mean().iloc[-1]
                 logger.debug(f"Calculated SMA_20: {indicators['SMA_20']:.2f}")
            else:
                 indicators['SMA_20'] = np.nan
                 logger.warning("Insufficient data for SMA_20.")

            if data_len >= 50:
                 indicators['SMA_50'] = price_data['Close'].rolling(window=50).mean().iloc[-1]
                 logger.debug(f"Calculated SMA_50: {indicators['SMA_50']:.2f}")
            else:
                 indicators['SMA_50'] = np.nan
                 logger.warning("Insufficient data for SMA_50.")

            if data_len >= 200:
                 indicators['SMA_200'] = price_data['Close'].rolling(window=200).mean().iloc[-1]
                 logger.debug(f"Calculated SMA_200: {indicators['SMA_200']:.2f}")
            else:
                 indicators['SMA_200'] = np.nan
                 logger.warning("Insufficient data for SMA_200.")

            # Exponential Moving Averages
            if data_len >= 12:
                 indicators['EMA_12'] = price_data['Close'].ewm(span=12, adjust=False).mean().iloc[-1]
                 logger.debug(f"Calculated EMA_12: {indicators['EMA_12']:.2f}")
            else:
                 indicators['EMA_12'] = np.nan
                 logger.warning("Insufficient data for EMA_12.")

            if data_len >= 26:
                 indicators['EMA_26'] = price_data['Close'].ewm(span=26, adjust=False).mean().iloc[-1]
                 logger.debug(f"Calculated EMA_26: {indicators['EMA_26']:.2f}")
            else:
                 indicators['EMA_26'] = np.nan
                 logger.warning("Insufficient data for EMA_26.")

            # RSI
            if data_len >= 14:
                indicators['RSI'] = self.calculate_rsi(price_data['Close'])
                logger.debug(f"Calculated RSI: {indicators['RSI']:.2f}")
            else:
                 indicators['RSI'] = np.nan
                 logger.warning("Insufficient data for RSI.")

            # MACD (requires EMA_12 and EMA_26)
            macd_line = np.nan
            signal_line = np.nan
            macd_histogram = np.nan
            logger.debug("Attempting to calculate MACD.")

            if 'EMA_12' in indicators and 'EMA_26' in indicators and not np.isnan(indicators['EMA_12']) and not np.isnan(indicators['EMA_26']):
                 # Use the entire macd_line series for rolling calculation if possible
                 full_macd_line_series = price_data['Close'].ewm(span=12, adjust=False).mean() - price_data['Close'].ewm(span=26, adjust=False).mean()
                 if len(full_macd_line_series.dropna()) >= 9:
                      macd_line = full_macd_line_series.iloc[-1]
                      signal_line = full_macd_line_series.ewm(span=9, adjust=False).mean().iloc[-1]
                      if not np.isnan(signal_line):
                         macd_histogram = macd_line - signal_line
                 else:
                      logger.warning("Insufficient data for MACD signal line calculation.")
            else:
                 logger.warning("MACD not calculated due to missing EMA_12 or EMA_26.")


            indicators['MACD'] = macd_line
            indicators['MACD_signal'] = signal_line
            indicators['MACD_histogram'] = macd_histogram

            # Log MACD values after calculation
            logger.debug(f"Final MACD: {indicators['MACD']}, Signal: {indicators['MACD_signal']}, Histogram: {indicators['MACD_histogram']}")


            # Bollinger Bands (requires SMA_20 and enough data for std dev)
            bb_period = 20
            bb_std = 2
            logger.debug(f"Attempting to calculate Bollinger Bands (Period: {bb_period}, Std Dev: {bb_std}).")
            if data_len >= bb_period:
                sma_bb = price_data['Close'].rolling(window=bb_period).mean()
                std_bb = price_data['Close'].rolling(window=bb_period).std()
                # Ensure SMA and STD are not NaN before calculating bands
                if not np.isnan(sma_bb.iloc[-1]) and not np.isnan(std_bb.iloc[-1]):
                    indicators['BB_upper'] = sma_bb.iloc[-1] + (std_bb.iloc[-1] * bb_std)
                    indicators['BB_lower'] = sma_bb.iloc[-1] - (std_bb.iloc[-1] * bb_std)
                    indicators['BB_middle'] = sma_bb.iloc[-1]
                    logger.debug(f"Calculated BB_upper: {indicators['BB_upper']:.2f}, BB_lower: {indicators['BB_lower']:.2f}, BB_middle: {indicators['BB_middle']:.2f}")
                else:
                    indicators['BB_upper'] = np.nan
                    indicators['BB_lower'] = np.nan
                    indicators['BB_middle'] = np.nan
                    logger.warning("Bollinger Bands not calculated due to NaN in SMA or STD.")
            else:
                indicators['BB_upper'] = np.nan
                indicators['BB_lower'] = np.nan
                indicators['BB_middle'] = np.nan
                logger.warning("Insufficient data for Bollinger Bands.")


            # Volume indicators
            logger.debug("Calculating Volume indicators.")
            if data_len >= 20:
                 indicators['Volume_SMA'] = price_data['Volume'].rolling(window=20).mean().iloc[-1]
                 logger.debug(f"Calculated Volume_SMA: {indicators['Volume_SMA']:.2f}")
                 if indicators['Volume_SMA'] is not None and not np.isnan(indicators['Volume_SMA']) and indicators['Volume_SMA'] > 0:
                      indicators['Volume_ratio'] = price_data['Volume'].iloc[-1] / indicators['Volume_SMA']
                      logger.debug(f"Calculated Volume_ratio: {indicators['Volume_ratio']:.2f}")
                 else:
                      indicators['Volume_ratio'] = np.nan
                      logger.warning("Volume SMA is zero or NaN, Volume ratio not calculated.")
            else:
                 indicators['Volume_SMA'] = np.nan
                 indicators['Volume_ratio'] = np.nan
                 logger.warning("Insufficient data for Volume SMA and ratio.")


            # Price position (requires SMA_20 and SMA_50)
            current_price = price_data['Close'].iloc[-1] if data_len > 0 and 'Close' in price_data and not price_data['Close'].empty else np.nan
            indicators['current_price'] = current_price # Store current price
            logger.debug(f"Current price: {current_price:.2f}")

            if current_price is not None and not np.isnan(current_price) and 'SMA_20' in indicators and indicators['SMA_20'] is not None and not np.isnan(indicators['SMA_20']) and indicators['SMA_20'] > 0:
                indicators['Price_to_SMA20'] = ((current_price / indicators['SMA_20'] - 1) * 100)
                logger.debug(f"Calculated Price_to_SMA20: {indicators['Price_to_SMA20']:.2f}%")
            else:
                 indicators['Price_to_SMA20'] = np.nan
                 logger.warning("Price to SMA20 not calculated due to missing current price or SMA20.")

            if current_price is not None and not np.isnan(current_price) and 'SMA_50' in indicators and indicators['SMA_50'] is not None and not np.isnan(indicators['SMA_50']) and indicators['SMA_50'] > 0:
                 indicators['Price_to_SMA50'] = ((current_price / indicators['SMA_50'] - 1) * 100)
                 logger.debug(f"Calculated Price_to_SMA50: {indicators['Price_to_SMA50']:.2f}%")
            else:
                 indicators['Price_to_SMA50'] = np.nan
                 logger.warning("Price to SMA50 not calculated due to missing current price or SMA50.")


            # Support and Resistance (requires enough data for rolling max/min)
            logger.debug("Calculating Support and Resistance.")
            if data_len >= 20:
                 indicators['Resistance'] = price_data['High'].rolling(window=20).max().iloc[-1]
                 indicators['Support'] = price_data['Low'].rolling(window=20).min().iloc[-1]
                 logger.debug(f"Calculated Resistance: {indicators['Resistance']:.2f}, Support: {indicators['Support']:.2f}")
            else:
                 indicators['Resistance'] = np.nan
                 indicators['Support'] = np.nan
                 logger.warning("Insufficient data for Support and Resistance.")

            logger.info("Technical indicators calculation completed.")
            return indicators

        except Exception as e:
            logger.error(f"Error calculating technical indicators: {str(e)}")
            return {}

    def calculate_rsi(self, prices: pd.Series, period: int = 14) -> Optional[float]:
        """Calculate Relative Strength Index"""
        logger.debug(f"Calculating RSI with period: {period}.")
        try:
            if len(prices) < period:
                logger.warning(f"Insufficient data ({len(prices)} data points) for RSI calculation (requires {period}).")
                return np.nan # Return NaN for insufficient data

            delta = prices.diff()
            gain = delta.where(delta > 0, 0)
            loss = -delta.where(delta < 0, 0)

            avg_gain = gain.rolling(window=period).mean()
            avg_loss = loss.rolling(window=period).mean()

            # Handle division by zero if avg_loss is zero or NaN
            if avg_loss.iloc[-1] is None or np.isnan(avg_loss.iloc[-1]) or avg_loss.iloc[-1] == 0:
                 rsi_value = 100.0 if avg_gain.iloc[-1] is not None and not np.isnan(avg_gain.iloc[-1]) and avg_gain.iloc[-1] > 0 else 50.0 # If no loss, RSI is 100 (if gain) or 50 (if no change)
                 logger.debug(f"Calculated RSI (zero avg_loss): {rsi_value:.2f}")
                 return rsi_value
            elif avg_loss.iloc[-1] > 0:
                rs = avg_gain / avg_loss
                rsi = 100 - (100 / (1 + rs))
                rsi_value = rsi.iloc[-1]
                logger.debug(f"Calculated RSI: {rsi_value:.2f}")
                return rsi_value
            else:
                 logger.warning("Unexpected condition in RSI calculation (avg_loss not > 0 and not zero/NaN).")
                 return np.nan # Should not happen with abs, but as a safeguard


        except Exception as e:
            logger.error(f"Error calculating RSI: {str(e)}")
            return np.nan # Return NaN on error

    def calculate_fundamental_score(self, info: Dict, benchmarks: Dict) -> Tuple[float, Dict]:
        """Calculate fundamental analysis score based on Indian market standards"""
        logger.info("Calculating fundamental score.")
        score = 0
        max_score = 50
        metrics = {}
        logger.debug(f"Using benchmarks: {benchmarks}")

        try:
            # P/E Ratio (10 points)
            pe_ratio = info.get('trailingPE', info.get('forwardPE', None))
            logger.debug(f"Raw PE Ratio: {pe_ratio}")
            if pe_ratio is not None and isinstance(pe_ratio, (int, float)) and not np.isnan(pe_ratio):
                metrics['PE_Ratio'] = pe_ratio
                if pe_ratio > 0: # Ensure PE is positive for scoring
                     if pe_ratio <= benchmarks['pe_ratio']['excellent']:
                         score += 10
                         logger.debug(f"PE Ratio ({pe_ratio:.2f}) is Excellent. +10 pts.")
                     elif pe_ratio <= benchmarks['pe_ratio']['good']:
                         score += 7
                         logger.debug(f"PE Ratio ({pe_ratio:.2f}) is Good. +7 pts.")
                     elif pe_ratio <= benchmarks['pe_ratio']['fair']:
                         score += 4
                         logger.debug(f"PE Ratio ({pe_ratio:.2f}) is Fair. +4 pts.")
                     else:
                         score += 2
                         logger.debug(f"PE Ratio ({pe_ratio:.2f}) is Poor. +2 pts.")
                else:
                    score += 1 # Small score for non-positive PE
                    logger.warning(f"PE Ratio ({pe_ratio}) is non-positive. Assigning minimal score (+1 pt).")
            else:
                 metrics['PE_Ratio'] = np.nan
                 logger.warning("PE Ratio not available or is invalid. Assigning no score for PE.")


            # Debt to Equity (10 points)
            debt_to_equity_raw = info.get('debtToEquity', None)
            debt_to_equity = np.nan # Initialize to NaN
            logger.debug(f"Raw Debt to Equity: {debt_to_equity_raw}")

            if debt_to_equity_raw is not None and isinstance(debt_to_equity_raw, (int, float)) and not np.isnan(debt_to_equity_raw):
                 # Handle potential outliers (very high values might indicate data issues)
                 if debt_to_equity_raw > 10000: # Arbitrary large threshold for outlier detection
                      logger.warning(f"Debt to Equity raw value is unusually high: {debt_to_equity_raw}. Treating as potential data issue.")
                      # Could assign a very low score or NaN, depending on policy
                      debt_to_equity = np.nan # Treat as invalid data for now
                 else:
                      # Heuristic for percentage vs decimal
                      if debt_to_equity_raw > 100: # Likely a percentage
                           debt_to_equity = debt_to_equity_raw / 100.0
                           logger.debug(f"Converted Debt to Equity from percentage: {debt_to_equity:.2f}")
                      else: # Assume it's already a decimal
                           debt_to_equity = debt_to_equity_raw
                           logger.debug(f"Using Debt to Equity as decimal: {debt_to_equity:.2f}")


                      # Re-check if the converted value is still reasonable (e.g., not from a huge raw decimal)
                      if debt_to_equity is not None and not np.isnan(debt_to_equity) and debt_to_equity > 100: # Another threshold after conversion
                          logger.warning(f"Debt to Equity value after conversion is still very high: {debt_to_equity:.2f}. May indicate data issue or highly leveraged company.")
                          # Could assign a low score or cap it, for now let the scoring handle it.


            metrics['Debt_to_Equity'] = debt_to_equity

            # Score based on the validated debt_to_equity
            if debt_to_equity is not None and not np.isnan(debt_to_equity):
                if debt_to_equity <= benchmarks['debt_to_equity']['excellent']:
                    score += 10
                    logger.debug(f"Debt to Equity ({debt_to_equity:.2f}) is Excellent. +10 pts.")
                elif debt_to_equity <= benchmarks['debt_to_equity']['good']:
                    score += 7
                    logger.debug(f"Debt to Equity ({debt_to_equity:.2f}) is Good. +7 pts.")
                elif debt_to_equity <= benchmarks['debt_to_equity']['fair']:
                    score += 4
                    logger.debug(f"Debt to Equity ({debt_to_equity:.2f}) is Fair. +4 pts.")
                else:
                    score += 2
                    logger.debug(f"Debt to Equity ({debt_to_equity:.2f}) is Poor. +2 pts.")
            else:
                 logger.warning("Debt to Equity not available, invalid, or an outlier. Assigning low score (+1 pt).")
                 score += 1 # Assign minimal score if D/E is not valid


            # ROE (10 points)
            roe = info.get('returnOnEquity', None)
            logger.debug(f"Raw ROE: {roe}")
            if roe is not None and isinstance(roe, (int, float)) and not np.isnan(roe):
                metrics['ROE'] = roe
                if roe >= benchmarks['roe']['excellent']:
                    score += 10
                    logger.debug(f"ROE ({roe:.2f}) is Excellent. +10 pts.")
                elif roe >= benchmarks['roe']['good']:
                    score += 7
                    logger.debug(f"ROE ({roe:.2f}) is Good. +7 pts.")
                elif roe >= benchmarks['roe']['fair']:
                    score += 4
                    logger.debug(f"ROE ({roe:.2f}) is Fair. +4 pts.")
                else:
                    score += 2
                    logger.debug(f"ROE ({roe:.2f}) is Poor. +2 pts.")
            else:
                 metrics['ROE'] = np.nan
                 logger.warning("ROE not available or is invalid. Assigning no score for ROE.")

            # Revenue Growth (10 points)
            revenue_growth = info.get('revenueGrowth', None)
            logger.debug(f"Raw Revenue Growth: {revenue_growth}")
            if revenue_growth is not None and isinstance(revenue_growth, (int, float)) and not np.isnan(revenue_growth):
                metrics['Revenue_Growth'] = revenue_growth
                if revenue_growth >= benchmarks['revenue_growth']['excellent']:
                    score += 10
                    logger.debug(f"Revenue Growth ({revenue_growth:.2f}) is Excellent. +10 pts.")
                elif revenue_growth >= benchmarks['revenue_growth']['good']:
                    score += 7
                    logger.debug(f"Revenue Growth ({revenue_growth:.2f}) is Good. +7 pts.")
                elif revenue_growth >= benchmarks['revenue_growth']['fair']:
                    score += 4
                    logger.debug(f"Revenue Growth ({revenue_growth:.2f}) is Fair. +4 pts.")
                else:
                    score += 2
                    logger.debug(f"Revenue Growth ({revenue_growth:.2f}) is Poor. +2 pts.")
            else:
                 metrics['Revenue_Growth'] = np.nan
                 logger.warning("Revenue Growth not available or is invalid. Assigning no score for Revenue Growth.")

            # Net Profit Margin (10 points)
            profit_margin = info.get('profitMargins', None)
            logger.debug(f"Raw Net Profit Margin: {profit_margin}")
            if profit_margin is not None and isinstance(profit_margin, (int, float)) and not np.isnan(profit_margin):
                metrics['Net_Profit_Margin'] = profit_margin
                if profit_margin >= benchmarks['net_profit_margin']['excellent']:
                    score += 10
                    logger.debug(f"Net Profit Margin ({profit_margin:.2f}) is Excellent. +10 pts.")
                elif profit_margin >= benchmarks['net_profit_margin']['good']:
                    score += 7
                    logger.debug(f"Net Profit Margin ({profit_margin:.2f}) is Good. +7 pts.")
                elif profit_margin >= benchmarks['net_profit_margin']['fair']:
                    score += 4
                    logger.debug(f"Net Profit Margin ({profit_margin:.2f}) is Fair. +4 pts.")
                else:
                    score += 2
                    logger.debug(f"Net Profit Margin ({profit_margin:.2f}) is Poor. +2 pts.")
            else:
                 metrics['Net_Profit_Margin'] = np.nan
                 logger.warning("Net Profit Margin not available or is invalid. Assigning no score for Net Profit Margin.")


            # Normalize score to max_score (already capped by scoring logic)
            metrics['Fundamental_Score'] = score
            logger.info(f"Fundamental score calculated: {score}/50")
            logger.debug(f"Fundamental metrics: {metrics}")

            return score, metrics

        except Exception as e:
            logger.error(f"Error calculating fundamental score: {str(e)}")
            return 0, {}

    def calculate_technical_score(self, indicators: Dict) -> float:
        """Calculate technical analysis score"""
        logger.info("Calculating technical score.")
        score = 0
        max_score = 50
        indicator_scores = {} # To store scores for each indicator
        logger.debug(f"Technical indicators used for scoring: {indicators}")

        try:
            # RSI Score (10 points)
            rsi = indicators.get('RSI', None)
            if rsi is not None and not np.isnan(rsi):
                 if 40 <= rsi <= 60:  # Neutral zone
                     indicator_scores['RSI'] = 7
                     logger.debug(f"RSI ({rsi:.2f}) is Neutral. +7 pts.")
                 elif rsi < 40:  # Oversold
                     indicator_scores['RSI'] = 10
                     logger.debug(f"RSI ({rsi:.2f}) is Oversold. +10 pts.")
                 elif 60 < rsi <= 70:  # Overbought but not extreme
                     indicator_scores['RSI'] = 5
                     logger.debug(f"RSI ({rsi:.2f}) is Approaching Overbought. +5 pts.")
                 else:  # Extreme levels
                     indicator_scores['RSI'] = 2
                     logger.debug(f"RSI ({rsi:.2f}) is Extreme. +2 pts.")
            else:
                 indicator_scores['RSI'] = 0 # Assign 0 if NaN/None
                 logger.warning("RSI not calculated or is NaN/None. Assigning 0 score for RSI.")


            # MACD Score (10 points)
            macd = indicators.get('MACD', None)
            macd_signal = indicators.get('MACD_signal', None)
            logger.debug(f"MACD: {macd}, Signal: {macd_signal}")

            # Explicitly check if both are numbers before comparing
            if isinstance(macd, (int, float)) and isinstance(macd_signal, (int, float)) and not np.isnan(macd) and not np.isnan(macd_signal):
                 if macd > macd_signal:
                     indicator_scores['MACD'] = 10
                     logger.debug("MACD > Signal (Bullish crossover). +10 pts.")
                 else:
                     indicator_scores['MACD'] = 3
                     logger.debug("MACD <= Signal (Bearish crossover). +3 pts.")
            else:
                 indicator_scores['MACD'] = 0 # Assign 0 if NaN/None
                 logger.warning("MACD or MACD signal not calculated or is not a valid number. Assigning 0 score for MACD.")


            # Moving Average Score (10 points) - Using SMA_20 vs SMA_50
            sma_20 = indicators.get('SMA_20', None)
            sma_50 = indicators.get('SMA_50', None)
            logger.debug(f"SMA_20: {sma_20}, SMA_50: {sma_50}")
            if sma_20 is not None and sma_50 is not None and not np.isnan(sma_20) and not np.isnan(sma_50):
                if sma_20 > sma_50:
                    indicator_scores['Moving_Averages'] = 10
                    logger.debug("SMA_20 > SMA_50 (Bullish trend). +10 pts.")
                else:
                    indicator_scores['Moving_Averages'] = 3
                    logger.debug("SMA_20 <= SMA_50 (Bearish trend). +3 pts.")
            else:
                indicator_scores['Moving_Averages'] = 0 # Assign 0 if NaN/None
                logger.warning("SMA_20 or SMA_50 not calculated or is NaN/None. Assigning 0 score for Moving Averages.")

            # Bollinger Bands Score (10 points)
            current_price = indicators.get('current_price', None) # Assuming current_price is passed in indicators
            bb_upper = indicators.get('BB_upper', None)
            bb_lower = indicators.get('BB_lower', None)
            bb_middle = indicators.get('BB_middle', None)
            logger.debug(f"Current Price: {current_price}, BB_upper: {bb_upper}, BB_lower: {bb_lower}, BB_middle: {bb_middle}")

            if current_price is not None and bb_upper is not None and bb_lower is not None and bb_middle is not None and \
               not np.isnan(current_price) and not np.isnan(bb_upper) and not np.isnan(bb_lower) and not np.isnan(bb_middle):
                if bb_lower < current_price < bb_middle:
                    indicator_scores['Bollinger_Bands'] = 10  # Good buying zone
                    logger.debug("Price between Lower and Middle BB (potential buy). +10 pts.")
                elif bb_middle < current_price < bb_upper:
                    indicator_scores['Bollinger_Bands'] = 7   # Neutral to positive
                    logger.debug("Price between Middle and Upper BB (neutral/positive). +7 pts.")
                else:
                    indicator_scores['Bollinger_Bands'] = 3   # Near extremes
                    logger.debug("Price near BB extremes. +3 pts.")
            else:
                 indicator_scores['Bollinger_Bands'] = 0 # Assign 0 if NaN/None
                 logger.warning("Bollinger Bands or current price not available or is NaN/None. Assigning 0 score for Bollinger Bands.")


            # Volume Score (10 points)
            volume_ratio = indicators.get('Volume_ratio', None)
            logger.debug(f"Volume ratio: {volume_ratio}")
            if volume_ratio is not None and not np.isnan(volume_ratio):
                 if 0.8 <= volume_ratio <= 1.5:
                     indicator_scores['Volume'] = 7   # Normal volume
                     logger.debug(f"Volume ratio ({volume_ratio:.2f}) is Normal. +7 pts.")
                 elif volume_ratio > 1.5:
                     indicator_scores['Volume'] = 10  # High volume (positive if with price up)
                     logger.debug(f"Volume ratio ({volume_ratio:.2f}) is High. +10 pts.")
                 else:
                     indicator_scores['Volume'] = 3   # Low volume
                     logger.debug(f"Volume ratio ({volume_ratio:.2f}) is Low. +3 pts.")
            else:
                 indicator_scores['Volume'] = 0 # Assign 0 if NaN/None
                 logger.warning("Volume ratio not calculated or is NaN/None. Assigning 0 score for Volume.")

            # Sum the scores from individual indicators
            score = sum(indicator_scores.values())
            logger.info(f"Technical score calculated (before capping): {score}")

            return min(score, max_score) # Cap at max_score


        except Exception as e:
            logger.error(f"Error calculating technical score: {str(e)}")
            return 0

    def generate_recommendation(self, fundamental_score: float, technical_score: float) -> str:
        """Generate investment recommendation based on scores"""
        logger.info(f"Generating recommendation based on Fundamental Score: {fundamental_score}, Technical Score: {technical_score}")
        total_score = fundamental_score + technical_score
        logger.debug(f"Total score for basic recommendation: {total_score}")

        if total_score >= 75:
            return "STRONG BUY"
        elif total_score >= 60:
            return "BUY"
        elif total_score >= 40:
            return "HOLD"
        elif total_score >= 25:
            return "SELL"
        else:
            return "STRONG SELL"

    def analyze_stock(self, symbol: str) -> Optional[StockData]:
        """Main method to analyze a stock"""
        try:
            logger.info(f"Starting analysis for {symbol}")

            # Get stock data
            ticker = self.get_stock_data(symbol)
            if not ticker:
                logger.error(f"Failed to get stock data for {symbol}. Analysis aborted.")
                return None

            # Get stock info
            info = ticker.info
            if not info:
                logger.error(f"No info available for {symbol}. Analysis aborted.")
                return None
            logger.debug(f"Fetched stock info for {symbol}: {list(info.keys())}")


            # Get historical price data
            # Fetching enough data for 200-day SMA calculation
            logger.info(f"Fetching 1 year historical price data for {symbol}.")
            price_data = ticker.history(period="1y") # Changed to 1 year to support 200-day SMA
            if price_data.empty: # Check if price_data is empty
                logger.error(f"No price data available for {symbol}. Analysis aborted.")
                return None
            # Also check for essential columns
            if not all(col in price_data.columns for col in ['Open', 'High', 'Low', 'Close', 'Volume']):
                 logger.error(f"Price data for {symbol} is missing essential columns. Analysis aborted.")
                 return None
            logger.info(f"Successfully fetched {len(price_data)} data points.")


            # Get current price
            current_price = price_data['Close'].iloc[-1] if len(price_data) > 0 and 'Close' in price_data and not price_data['Close'].empty else np.nan
            if np.isnan(current_price):
                 logger.warning(f"Could not get current price for {symbol}.")


            # Add current price to indicators for technical score calculation
            indicators = self.calculate_technical_indicators(price_data)
            indicators['current_price'] = current_price # Ensure current price is available for BB score

            # Get industry benchmarks
            sector = info.get('sector', 'Unknown')
            benchmarks = self.get_industry_benchmarks(sector)
            logger.info(f"Identified sector: {sector}. Using benchmarks for this sector.")


            # Calculate scores
            fundamental_score, metrics = self.calculate_fundamental_score(info, benchmarks)
            technical_score = self.calculate_technical_score(indicators)

            # Generate recommendation
            recommendation = self.generate_recommendation(fundamental_score, technical_score)

            # Create StockData object
            stock_data = StockData(
                symbol=symbol,
                company_name=info.get('longName', symbol),
                current_price=current_price if not np.isnan(current_price) else 0.0, # Store 0 if current price is NaN
                market_cap=info.get('marketCap', 0) if info.get('marketCap') is not None else 0, # Ensure marketCap is not None
                info=info,
                price_data=price_data,
                indicators=indicators,
                fundamental_score=fundamental_score,
                technical_score=technical_score,
                recommendation=recommendation,
                metrics=metrics
            )

            logger.info(f"Analysis completed for {symbol}")
            return stock_data

        except Exception as e:
            logger.error(f"Error analyzing {symbol}: {str(e)}")
            return None

    def display_analysis(self, stock_data: StockData) -> None:
        """Display analysis results in a formatted manner"""
        # Convert print statements to logger.info for consistency, or keep print for final output
        # Keeping print for final formatted output as before
        print(f"\n{'='*60}")
        print(f"Stock Analysis Report: {stock_data.company_name} ({stock_data.symbol})")
        print(f"{'='*60}")

        print(f"\n📊 Current Market Data:")
        print(f"Current Price: ₹{stock_data.current_price:,.2f}" if stock_data.current_price is not None else "Current Price: N/A")
        print(f"Market Cap: ₹{stock_data.market_cap/10000000:,.2f} Cr" if stock_data.market_cap is not None else "Market Cap: N/A") # Use stored market_cap
        print(f"Sector: {stock_data.info.get('sector', 'N/A')}")

        print(f"\n📈 Technical Indicators:")
        # Display indicators, handling None values
        for indicator, value in stock_data.indicators.items():
            if indicator != 'current_price': # Avoid displaying current_price here
                if value is not None and not np.isnan(value):
                    if isinstance(value, float):
                        print(f"{indicator}: {value:,.2f}")
                    else:
                        print(f"{indicator}: {value}")
                else:
                    print(f"{indicator}: N/A (Insufficient Data)")


        print(f"\n💰 Fundamental Metrics:")
        for metric, value in stock_data.metrics.items():
            if metric != 'Fundamental_Score':
                if value is not None and not np.isnan(value):
                     if isinstance(value, float):
                         if metric in ['ROE', 'Revenue_Growth', 'Net_Profit_Margin', 'Debt_to_Equity']:
                             print(f"{metric}: {value*100:.2f}%") # Display as percentage for D/E, ROE, Growth, Margin
                         else:
                             print(f"{metric}: {value:.2f}")
                     else:
                         print(f"{metric}: {value}")
                else:
                    print(f"{metric}: N/A (Missing Data)")


        print(f"\n📊 Analysis Scores:")
        print(f"Fundamental Score: {stock_data.fundamental_score}/50")
        print(f"Technical Score: {stock_data.technical_score}/50")
        print(f"Total Score: {stock_data.fundamental_score + stock_data.technical_score}/100")

        print(f"\n🎯 Recommendation: {stock_data.recommendation}")
        print(f"{'='*60}\n")


**Reasoning**:
Continue improving logging in the EnhancedStockAnalyzer class, adding info and debug messages to key functions and converting print statements to logger calls where appropriate.



In [ ]:
#@title Sentimental analysis with API (Enhanced Logging)
# Indian Stock Analyzer - Part 2 (Improved Version with Enhanced Data Handling and Logging)
# This continues from your Part 1 implementation

# Import the Python SDK for Gemini API
# from google.colab import userdata # Already imported in a previous cell
# import google.generativeai as genai # Already imported in a previous cell

import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import requests
from bs4 import BeautifulSoup
import warnings
warnings.filterwarnings('ignore')

# Import the Python SDK for Gemini API
# Used to securely store your API key
# from google.colab import userdata # Already imported
# import google.generativeai as genai # Already imported

# Configure Gemini API (ensure GOOGLE_API_KEY is set in Colab secrets)
# GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY') # Already done in a previous cell
# genai.configure(api_key=GOOGLE_API_KEY) # Already done in a previous cell
# gemini_model = genai.GenerativeModel('gemini-2.0-flash') # Using a more recent model # Already done in a previous cell


class EnhancedStockAnalyzer:
    """Enhanced analyzer with additional features for Indian markets"""

    def __init__(self):
        logger.info("Initializing EnhancedStockAnalyzer.")
        self.risk_free_rate = self.get_dynamic_risk_free_rate()
        # self.gemini_model = gemini_model # Use the globally configured model # Already assigned in a previous cell
        logger.info(f"EnhancedStockAnalyzer initialized with risk-free rate: {self.risk_free_rate}")


    def get_dynamic_risk_free_rate(self):
        """Fetch current 10-year G-Sec yield"""
        logger.info("Fetching dynamic risk-free rate.")
        try:
            # Try to get from a reliable source
            # For now, using a realistic current rate
            # In production, you'd scrape from RBI or use an API
            rate = 0.072  # 7.2% as of recent data
            logger.info(f"Fetched dynamic risk-free rate: {rate}")
            return rate
        except Exception as e:
            logger.warning(f"Could not fetch dynamic risk-free rate: {str(e)}. Falling back to default.")
            return 0.065  # Fallback to 6.5%

    def get_circuit_limits(self, symbol):
        """
        Get circuit breaker limits for the stock.
        NOTE: This is a simplification. Actual NSE circuit limits vary based on
        stock category (e.g., EQ, BE, T group), stage of price band reduction,
        and are dynamic. Relying on fixed percentages based on index membership
        is an approximation. For production, fetch from NSE or a reliable API.
        """
        logger.info(f"Attempting to get circuit limits for {symbol}.")
        try:
            ticker = yf.Ticker(symbol)
            info = ticker.info
            prev_close = info.get('previousClose', None) # Use None as default
            logger.debug(f"Previous close for circuit limit calculation: {prev_close}")

            if prev_close is None or not isinstance(prev_close, (int, float)) or np.isnan(prev_close) or prev_close <= 0:
                logger.warning(f"Previous close not available or invalid for {symbol}. Cannot calculate circuit limits.")
                return None

            # Enhanced NSE circuit limits (based on index membership approximation)
            # This is still a simplification of complex exchange rules.
            circuit_percent = 0.10 # Default to 10%

            # Check if the stock is in a major index for higher limits
            # Need a way to access get_index_membership or pass index membership
            # For now, reuse the simplified logic directly here for logging context
            index_memberships = self.get_index_membership(symbol) # Call locally for log context
            logger.debug(f"Index memberships for {symbol}: {index_memberships}")

            if index_memberships: # Check if list is not empty
                 if 'NIFTY50' in index_memberships:
                      circuit_percent = 0.20 # 20% for NIFTY50 stocks
                      logger.debug(f"Using 20% circuit limit for NIFTY50 stock {symbol}.")
                 elif 'NIFTY_NEXT50' in index_memberships:
                      circuit_percent = 0.15 # Example: 15% for NIFTY Next 50 (approximation)
                      logger.debug(f"Using 15% circuit limit for NIFTY Next 50 stock {symbol}.")


            upper_circuit = prev_close * (1 + circuit_percent)
            lower_circuit = prev_close * (1 - circuit_percent)

            circuit_limits = {
                'upper_circuit': upper_circuit,
                'lower_circuit': lower_circuit,
                'circuit_percent': circuit_percent
            }
            logger.info(f"Calculated circuit limits for {symbol}: Upper={upper_circuit:.2f}, Lower={lower_circuit:.2f}, Percent={circuit_percent*100:.1f}%")
            return circuit_limits

        except Exception as e:
            logger.error(f"Error getting circuit limits for {symbol}: {str(e)}")
            return None

    def assess_circuit_risk(self, current_price, circuit_limits):
        """Assess proximity to circuit limits"""
        logger.info("Assessing circuit risk.")
        if not circuit_limits or current_price is None or np.isnan(current_price) or current_price <= 0:
            logger.warning("Cannot assess circuit risk due to missing circuit limits or invalid current price.")
            return 0

        upper_circuit = circuit_limits.get('upper_circuit')
        lower_circuit = circuit_limits.get('lower_circuit')
        logger.debug(f"Current price: {current_price}, Upper circuit: {upper_circuit}, Lower circuit: {lower_circuit}")


        if upper_circuit is None or lower_circuit is None or np.isnan(upper_circuit) or np.isnan(lower_circuit):
             logger.warning("Circuit limits are invalid, cannot assess circuit risk.")
             return 0

        upper_distance = (upper_circuit - current_price) / current_price if current_price > 0 else float('inf')
        lower_distance = (current_price - lower_circuit) / current_price if current_price > 0 else float('inf')

        circuit_risk_score = 0
        # Risk scoring based on proximity
        if upper_distance < 0.02:  # Within 2% of upper circuit
            circuit_risk_score = -20  # High risk
            logger.debug(f"Within 2% of upper circuit (distance: {upper_distance:.2f}). Risk score: {circuit_risk_score}")
        elif upper_distance < 0.05:  # Within 5%
            circuit_risk_score = -10
            logger.debug(f"Within 5% of upper circuit (distance: {upper_distance:.2f}). Risk score: {circuit_risk_score}")
        elif lower_distance < 0.02:  # Within 2% of lower circuit
            circuit_risk_score = -25  # Very high risk
            logger.debug(f"Within 2% of lower circuit (distance: {lower_distance:.2f}). Risk score: {circuit_risk_score}")
        elif lower_distance < 0.05:
            circuit_risk_score = -15
            logger.debug(f"Within 5% of lower circuit (distance: {lower_distance:.2f}). Risk score: {circuit_risk_score}")
        else:
            logger.debug("Not close to circuit limits. Risk score: 0")


        logger.info(f"Assessed circuit risk score: {circuit_risk_score}")
        return circuit_risk_score

    def get_liquidity_score(self, price_data: pd.DataFrame) -> float:
        """Enhanced liquidity analysis"""
        logger.info("Calculating liquidity score.")
        try:
            # Explicitly check for sufficient data and 'Volume' column
            if price_data.empty or 'Volume' not in price_data.columns or len(price_data) < 20:
                 logger.warning("Insufficient price data or missing Volume column for liquidity calculation. Assigning low score.")
                 return 5 # Assign a very low liquidity score

            # Calculate various liquidity metrics
            avg_volume_20d = price_data['Volume'].rolling(20).mean().iloc[-1]
            avg_volume_5d = price_data['Volume'].rolling(5).mean().iloc[-1]
            current_close = price_data['Close'].iloc[-1] if not price_data['Close'].empty else np.nan

            logger.debug(f"Avg Volume (20d): {avg_volume_20d}, Avg Volume (5d): {avg_volume_5d}, Current Close: {current_close}")


            if current_close is None or np.isnan(current_close) or current_close <= 0:
                 logger.warning("Current close price is invalid for liquidity calculation. Assigning low score.")
                 return 5

            # Average daily turnover
            avg_turnover_20d = avg_volume_20d * current_close if avg_volume_20d is not None and not np.isnan(avg_volume_20d) else np.nan
            logger.debug(f"Avg Turnover (20d): {avg_turnover_20d}")

            # Volume spike detection - handle division by zero explicitly
            volume_spike = np.nan
            if avg_volume_20d is not None and not np.isnan(avg_volume_20d) and avg_volume_20d > 0:
                volume_spike = avg_volume_5d / avg_volume_20d if avg_volume_5d is not None and not np.isnan(avg_volume_5d) else np.nan
            elif avg_volume_5d is not None and not np.isnan(avg_volume_5d) and avg_volume_5d > 0: # Case where 20d avg is 0 but 5d is not
                 volume_spike = 1 # Treat as normal if 20d is 0 but 5d has volume
            else:
                 logger.warning("Average volume is zero or NaN, cannot calculate volume spike.")
            logger.debug(f"Volume Spike: {volume_spike}")

            # Volume consistency - handle division by zero explicitly
            volume_std = price_data['Volume'].rolling(20).std().iloc[-1] if len(price_data) >= 20 else np.nan
            volume_cv = np.nan
            if avg_volume_20d is not None and not np.isnan(avg_volume_20d) and avg_volume_20d > 0:
                volume_cv = volume_std / avg_volume_20d if volume_std is not None and not np.isnan(volume_std) else np.nan
            elif volume_std is not None and not np.isnan(volume_std) and volume_std > 0: # Case where 20d avg is 0 but std is not (unlikely but safeguard)
                 volume_cv = 1 # Treat as high variability if avg is 0 but std is not
            else:
                 logger.warning("Average volume or volume standard deviation is zero or NaN, cannot calculate volume CV.")
            logger.debug(f"Volume CV: {volume_cv}")


            # Liquidity scoring
            liquidity_score = 0
            logger.debug("Scoring liquidity metrics.")

            # Turnover-based scoring - handle NaN
            if avg_turnover_20d is not None and not np.isnan(avg_turnover_20d):
                if avg_turnover_20d > 500_00_00_000:  # > 500 Cr
                    liquidity_score += 40
                    logger.debug("+40 pts for Turnover > 500 Cr")
                elif avg_turnover_20d > 100_00_00_000:  # > 100 Cr
                    liquidity_score += 30
                    logger.debug("+30 pts for Turnover > 100 Cr")
                elif avg_turnover_20d > 10_00_00_000:  # > 10 Cr
                    liquidity_score += 20
                    logger.debug("+20 pts for Turnover > 10 Cr")
                elif avg_turnover_20d > 1_00_00_000:  # > 1 Cr
                    liquidity_score += 10
                    logger.debug("+10 pts for Turnover > 1 Cr")
                else:
                    liquidity_score += 5
                    logger.debug("+5 pts for Turnover <= 1 Cr")
            else:
                 logger.warning("Average turnover is NaN, skipping turnover score.")


            # Volume consistency score - handle NaN
            if volume_cv is not None and not np.isnan(volume_cv):
                 if volume_cv < 0.5:  # Low volatility in volume
                     liquidity_score += 20
                     logger.debug(f"+20 pts for Volume CV ({volume_cv:.2f}) < 0.5")
                 elif volume_cv < 1.0:
                     liquidity_score += 10
                     logger.debug(f"+10 pts for Volume CV ({volume_cv:.2f}) < 1.0")
                 else:
                     logger.debug(f"+0 pts for Volume CV ({volume_cv:.2f}) >= 1.0")
            else:
                 logger.warning("Volume CV is NaN, skipping volume consistency score.")

            # Recent volume trend score - handle NaN
            if volume_spike is not None and not np.isnan(volume_spike):
                 if volume_spike > 1.2:  # 20% higher recent volume
                     liquidity_score += 10
                     logger.debug(f"+10 pts for Volume Spike ({volume_spike:.2f}) > 1.2")
                 else:
                      logger.debug(f"+0 pts for Volume Spike ({volume_spike:.2f}) <= 1.2")
            else:
                 logger.warning("Volume spike is NaN, skipping recent volume trend score.")


            final_liquidity_score = max(5, min(liquidity_score, 70))  # Cap at 70, minimum score 5 for insufficient data
            logger.info(f"Calculated liquidity score: {final_liquidity_score}/70")
            return final_liquidity_score

        except Exception as e: # Catch specific exceptions for better debugging
            logger.error(f"Error calculating liquidity score: {str(e)}")
            return 5  # Default very low liquidity on error


    def get_peer_companies(self, symbol, stock_info):
        """Identify peer companies for comparison"""
        logger.info(f"Identifying peer companies for {symbol}.")
        try:
            sector = stock_info.get('sector', '')
            market_cap = stock_info.get('marketCap', 0)
            logger.debug(f"Stock sector: {sector}, Market Cap: {market_cap}")

            # Define market cap categories
            if market_cap > 10_00_000_00_00_000:  # > 10 Lakh Cr
                cap_category = 'Large Cap'
            elif market_cap > 50_000_00_00_000:   # > 50K Cr
                cap_category = 'Mid Cap'
            else:
                cap_category = 'Small Cap'
            logger.debug(f"Market Cap Category: {cap_category}")

            # Peer mapping (simplified - in production, use a comprehensive database)
            peer_map = {
                'Technology': {
                    'Large Cap': ['TCS.NS', 'INFY.NS', 'WIPRO.NS', 'HCLTECH.NS'],
                    'Mid Cap': ['TECHM.NS', 'LTTS.NS', 'PERSISTENT.NS', 'COFORGE.NS'],
                    'Small Cap': ['ZENSAR.NS', 'ECLERX.NS', 'MASTEK.NS']
                },
                'Financial Services': {
                    'Large Cap': ['HDFCBANK.NS', 'ICICIBANK.NS', 'KOTAKBANK.NS', 'AXISBANK.NS'],
                    'Mid Cap': ['IDFCFIRSTB.NS', 'FEDERALBNK.NS', 'INDUSINDBK.NS'],
                    'Small Cap': ['SOUTHBANK.NS', 'UJJIVAN.NS', 'EQUITASBNK.NS']
                },
                'Healthcare': {
                    'Large Cap': ['SUNPHARMA.NS', 'DRREDDY.NS', 'CIPLA.NS', 'DIVISLAB.NS'],
                    'Mid Cap': ['TORNTPHARM.NS', 'ALKEM.NS', 'LUPIN.NS'],
                    'Small Cap': ['CAPLIPOINT.NS', 'BLISS.NS', 'INDOCO.NS']
                }
                # Add more sectors as needed
            }

            # Get peers, excluding the current symbol
            peers = peer_map.get(sector, {}).get(cap_category, [])
            peers = [p for p in peers if p != symbol]
            logger.info(f"Identified {len(peers)} peer companies for {symbol}.")
            logger.debug(f"Peer list: {peers}")

            return peers[:5]  # Return top 5 peers

        except Exception as e:
            logger.error(f"Error getting peer companies: {str(e)}")
            return []

    def calculate_peer_relative_performance(self, symbol, peers, days=30):
        """Compare stock performance with peers"""
        logger.info(f"Calculating {days}-day peer relative performance for {symbol}.")
        try:
            results = {}

            # Get performance for main stock
            logger.debug(f"Fetching {days}-day history for main stock {symbol}.")
            main_ticker = yf.Ticker(symbol)
            main_hist = main_ticker.history(period=f"{days}d")
            main_return = np.nan # Initialize to NaN

            if len(main_hist) > 0 and 'Close' in main_hist.columns and not main_hist['Close'].empty:
                 if main_hist['Close'].iloc[0] > 0:
                      main_return = ((main_hist['Close'].iloc[-1] / main_hist['Close'].iloc[0]) - 1) * 100
                      logger.debug(f"Main stock ({symbol}) {days}-day return: {main_return:.2f}%")
                 else:
                      logger.warning(f"Initial close price is zero or negative for {symbol}. Cannot calculate main stock return.")
            else:
                logger.warning(f"Insufficient historical data for {symbol} to calculate peer relative performance.")


            # Get peer performances
            peer_returns = []
            logger.debug(f"Fetching {days}-day history for peers: {peers}")
            for peer in peers:
                try:
                    peer_ticker = yf.Ticker(peer)
                    peer_hist = peer_ticker.history(period=f"{days}d")
                    if len(peer_hist) > 0 and 'Close' in peer_hist.columns and not peer_hist['Close'].empty and peer_hist['Close'].iloc[0] > 0:
                        peer_return = ((peer_hist['Close'].iloc[-1] / peer_hist['Close'].iloc[0]) - 1) * 100
                        peer_returns.append(peer_return)
                        logger.debug(f"Peer {peer} {days}-day return: {peer_return:.2f}%")
                    else:
                         logger.warning(f"Insufficient historical data for peer {peer}.")
                except Exception as e:
                    logger.warning(f"Could not get peer history for {peer}: {str(e)}")
                    continue

            if peer_returns and main_return is not None and not np.isnan(main_return):
                avg_peer_return = np.mean(peer_returns)
                relative_performance = main_return - avg_peer_return

                results = {
                    'stock_return': main_return,
                    'avg_peer_return': avg_peer_return,
                    'relative_performance': relative_performance,
                    'outperformance': relative_performance > 0
                }
                logger.info(f"Calculated peer relative performance for {symbol}: Relative Performance={relative_performance:.2f}%")
            else:
                 logger.warning("Peer returns not available or main stock return is NaN, cannot calculate relative performance.")
                 logger.info("Peer relative performance calculation skipped.")


            return results

        except Exception as e:
            logger.error(f"Error calculating peer relative performance: {str(e)}")
            return {}

    def get_index_membership(self, symbol):
        """Check index membership for the stock"""
        logger.debug(f"Checking index membership for {symbol}.")
        # Simplified index constituents (in production, fetch from NSE)
        index_constituents = {
            'NIFTY50': ['RELIANCE.NS', 'TCS.NS', 'HDFCBANK.NS', 'INFY.NS', 'ICICIBANK.NS'],
            'NIFTY_NEXT50': ['VEDL.NS', 'PNB.NS', 'INDIGO.NS', 'BANDHANBNK.NS'],
            'NIFTY_MIDCAP150': ['PERSISTENT.NS', 'COFORGE.NS', 'LTTS.NS']
        }

        membership = []
        for index, constituents in index_constituents.items():
            if symbol in constituents:
                membership.append(index)
        logger.debug(f"Index membership results for {symbol}: {membership}")

        return membership

    def calculate_enhanced_technical_score(self, indicators: Dict, circuit_risk: float) -> float:
        """Calculate enhanced technical analysis score"""
        logger.info("Calculating enhanced technical score.")
        base_score = 0
        indicator_scores = {} # To store scores for each indicator
        logger.debug(f"Indicators for enhanced technical scoring: {indicators}")
        logger.debug(f"Circuit risk score: {circuit_risk}")

        try:
            # RSI Score (points based on significance, e.g., max 10 for oversold)
            rsi = indicators.get('RSI', None)
            if rsi is not None and not np.isnan(rsi):
                 if rsi < 30: indicator_scores['RSI'] = 10 # Oversold
                 elif 30 <= rsi < 40: indicator_scores['RSI'] = 8 # Approaching oversold
                 elif 40 <= rsi <= 60: indicator_scores['RSI'] = 5 # Neutral zone
                 elif 60 < rsi <= 70: indicator_scores['RSI'] = 3 # Approaching overbought
                 else: indicator_scores['RSI'] = 1 # Overbought
                 logger.debug(f"RSI score: {indicator_scores['RSI']}")
            else:
                 indicator_scores['RSI'] = 0 # Assign 0 if NaN/None
                 logger.warning("RSI not calculated or is NaN/None. Assigning 0 score for RSI.")

            # MACD scoring (points based on crossover and position relative to zero)
            macd = indicators.get('MACD', None)
            macd_signal = indicators.get('MACD_signal', None)
            logger.debug(f"MACD: {macd}, Signal: {macd_signal} for scoring.")

            if macd is not None and macd_signal is not None and isinstance(macd, (int, float)) and isinstance(macd_signal, (int, float)) and not np.isnan(macd) and not np.isnan(macd_signal):
                 if macd > macd_signal:
                     if macd > 0: indicator_scores['MACD'] = 10 # Bullish crossover above zero
                     else: indicator_scores['MACD'] = 7 # Bullish crossover below zero
                     logger.debug(f"MACD bullish crossover score: {indicator_scores['MACD']}")
                 else:
                     if macd < 0: indicator_scores['MACD'] = 1 # Bearish crossover below zero
                     else: indicator_scores['MACD'] = 3 # Bearish crossover above zero
                     logger.debug(f"MACD bearish crossover score: {indicator_scores['MACD']}")
            else:
                 indicator_scores['MACD'] = 0 # Assign 0 if NaN/None
                 logger.warning("MACD or MACD signal not calculated or is NaN/None. Assigning 0 score for MACD.")

            # Moving Average Score (points based on MA crossovers and price position)
            current_price = indicators.get('current_price', None)
            sma_20 = indicators.get('SMA_20', None)
            sma_50 = indicators.get('SMA_50', None)
            sma_200 = indicators.get('SMA_200', None)
            logger.debug(f"MA values for scoring: Current={current_price}, SMA20={sma_20}, SMA50={sma_50}, SMA200={sma_200}")


            ma_score = 0
            valid_mas = 0

            if sma_20 is not None and not np.isnan(sma_20) and current_price is not None and not np.isnan(current_price):
                 valid_mas += 1
                 if current_price > sma_20: ma_score += 5
                 else: ma_score += 1
                 logger.debug(f"Price vs SMA20 score component: {ma_score}")


            if sma_50 is not None and not np.isnan(sma_50) and current_price is not None and not np.isnan(current_price):
                 valid_mas += 1
                 if current_price > sma_50: ma_score += 5
                 else: ma_score += 1
                 logger.debug(f"Price vs SMA50 score component: {ma_score}")

            if sma_200 is not None and not np.isnan(sma_200) and current_price is not None and not np.isnan(current_price):
                 valid_mas += 1
                 if current_price > sma_200: ma_score += 5
                 else: ma_score += 1
                 logger.debug(f"Price vs SMA200 score component: {ma_score}")

            # Add points for bullish crossovers
            if sma_20 is not None and sma_50 is not None and not np.isnan(sma_20) and not np.isnan(sma_50):
                 if sma_20 > sma_50:
                     ma_score += 5
                     logger.debug(f"SMA20 > SMA50 crossover score component: {ma_score}")


            if sma_50 is not None and sma_200 is not None and not np.isnan(sma_50) and not np.isnan(sma_200):
                 if sma_50 > sma_200:
                     ma_score += 5
                     logger.debug(f"SMA50 > SMA200 crossover score component: {ma_score}")


            indicator_scores['Moving_Averages'] = ma_score # Max possible MA score is 25 (5+5+5 for price > MA + 5+5 for crossovers)

            if valid_mas == 0:
                 logger.warning("No valid Moving Averages calculated. Assigning 0 score for Moving Averages.")
                 indicator_scores['Moving_Averages'] = 0


            logger.debug(f"Moving Averages total score component: {indicator_scores['Moving_Averages']}")


            # Bollinger Bands Score (points based on price position relative to bands)
            bb_upper = indicators.get('BB_upper', None)
            bb_lower = indicators.get('BB_lower', None)
            bb_middle = indicators.get('BB_middle', None)
            logger.debug(f"BB values for scoring: Upper={bb_upper}, Lower={bb_lower}, Middle={bb_middle}")


            if current_price is not None and bb_upper is not None and bb_lower is not None and bb_middle is not None and \
               not np.isnan(current_price) and not np.isnan(bb_upper) and not np.isnan(bb_lower) and not np.isnan(bb_middle):
                if current_price < bb_lower: indicator_scores['Bollinger_Bands'] = 10 # Price below lower band (potential buy signal)
                elif bb_lower <= current_price < bb_middle: indicator_scores['Bollinger_Bands'] = 7 # Between lower and middle band
                elif bb_middle <= current_price < bb_upper: indicator_scores['Bollinger_Bands'] = 3 # Between middle and upper band
                else: indicator_scores['Bollinger_Bands'] = 1 # Price above upper band (potential sell signal)
                logger.debug(f"Bollinger Bands score: {indicator_scores['Bollinger_Bands']}")
            else:
                 indicator_scores['Bollinger_Bands'] = 0 # Assign 0 if NaN/None
                 logger.warning("Bollinger Bands or current price not available or is NaN/None. Assigning 0 score for Bollinger Bands.")


            # Volume Score (points based on volume relative to average)
            volume_ratio = indicators.get('Volume_ratio', None)
            logger.debug(f"Volume ratio for scoring: {volume_ratio}")
            if volume_ratio is not None and not np.isnan(volume_ratio):
                 if volume_ratio > 1.5: indicator_scores['Volume'] = 10 # High volume
                 elif 0.8 <= volume_ratio <= 1.5: indicator_scores['Volume'] = 7 # Normal volume
                 else: indicator_scores['Volume'] = 3 # Low volume
                 logger.debug(f"Volume score: {indicator_scores['Volume']}")
            else:
                 indicator_scores['Volume'] = 0 # Assign 0 if NaN/None
                 logger.warning("Volume ratio not calculated or is NaN/None. Assigning 0 score for Volume.")

            # Sum the scores from individual indicators
            base_score = sum(indicator_scores.values())
            logger.debug(f"Sum of individual indicator scores: {base_score}")

            # Add circuit risk adjustment (already handled as negative points)
            final_score = base_score + circuit_risk
            logger.debug(f"Base score ({base_score}) + Circuit risk ({circuit_risk}) = Raw final score ({final_score})")


            # Ensure score is within 0-50 range (assuming max base_score is around 50 based on point allocation)
            # Max possible indicator score (10+10+25+10+10) = 65. Let's normalize this to 50.
            normalized_base_score = (base_score / 65.0) * 50.0 if base_score > 0 else 0
            final_score = normalized_base_score + circuit_risk
            logger.debug(f"Normalized base score: {normalized_base_score:.2f}. Final score after normalization and risk: {final_score:.2f}")


            capped_score = max(0, min(final_score, 50))
            logger.info(f"Enhanced technical score calculated: {capped_score}/50")
            return capped_score


        except Exception as e:
            logger.error(f"Error calculating enhanced technical score: {str(e)}")
            return 0


    def fetch_news(self, symbol: str) -> List[str]:
        """
        Fetches recent news headlines for the given stock symbol.
        NOTE: This is a placeholder. In a real application, you would integrate
        with a News API (like News API, Alpha Vantage, etc.) here.
        """
        logger.info(f"Fetching news for {symbol} (placeholder)...")
        # Placeholder news headlines - replace with actual API call
        news_headlines = [
            f"{symbol} stock price rises on positive market sentiment",
            f"{symbol} announces strong quarterly results, beating estimates",
            f"Experts bullish on {symbol}'s future outlook",
            f"{symbol} faces regulatory challenges in key market",
            f"Competitor's new product launch impacts {symbol}'s market share",
            f"Global market trends positively impacting {symbol}",
            f"{symbol} management provides optimistic guidance",
            f"Increased foreign investment flows into {symbol}",
            f"Regulatory approval received for {symbol}'s new product",
            f"Production issues reported for {symbol}",
            f"Increased competition puts pressure on {symbol}'s margins",
            f"Analyst downgrades rating for {symbol}",
            f"Supply chain disruptions affect {symbol}'s operations",
            f"Geopolitical tensions create uncertainty for {symbol}",
            f"{symbol} announces stock split",
            f"Dividend declared by {symbol}"
        ]
        logger.info(f"Fetched {len(news_headlines)} news headlines (placeholder).")
        return news_headlines


    def analyze_sentiment(self, news_headlines: List[str]) -> Dict:
        """
        Analyzes the sentiment of news headlines using the Gemini API.
        Returns a dictionary with sentiment counts (positive, neutral, negative)
        and a sentiment score.
        """
        logger.info("Starting sentiment analysis.")
        # Use the globally defined gemini_model
        if not 'gemini_model' in globals() or not gemini_model:
            logger.warning("Gemini API not configured. Skipping sentiment analysis.")
            # Replace print with logger.warning
            # print("Gemini API not configured or no news to analyze. Skipping sentiment analysis.")
            return {'positive': 0, 'neutral': 0, 'negative': 0, 'score': 0, 'summary': 'Sentiment analysis skipped.'}

        if not news_headlines:
             logger.warning("No news headlines to analyze. Skipping sentiment analysis.")
             return {'positive': 0, 'neutral': 0, 'negative': 0, 'score': 0, 'summary': 'No news to analyze.'}


        logger.info(f"Analyzing sentiment for {len(news_headlines)} headlines using Gemini API...")
        prompt = "Analyze the sentiment of the following news headlines for a stock and categorize each as Positive, Neutral, or Negative. Provide a concise summary of the overall sentiment.\n\n"
        for headline in news_headlines:
            prompt += f"- {headline}\n"
        prompt += "\nSentiment Analysis Results (Categorization and Summary):\n"

        try:
            response = gemini_model.generate_content(prompt)
            sentiment_text = response.text.strip()
            logger.debug(f"Gemini API Raw Response:\n{sentiment_text}")
            logger.info(f"Gemini API Response Received.")


            # Parse the response to count sentiments and extract summary
            positive_count = sentiment_text.lower().count('positive')
            neutral_count = sentiment_text.lower().count('neutral')
            negative_count = sentiment_text.lower().count('negative')
            logger.debug(f"Sentiment counts: Positive={positive_count}, Neutral={neutral_count}, Negative={negative_count}")

            # Attempt to extract a summary line - this is a heuristic
            summary_line = "No summary extracted."
            lines = sentiment_text.split('\n')
            for line in lines:
                 lower_line = line.lower()
                 if 'summary' in lower_line or 'overall sentiment' in lower_line:
                     summary_line = line.strip()
                     break
            if summary_line == "No summary extracted." and lines:
                 # If no explicit summary found, take the last non-empty line as a potential summary
                 for line in reversed(lines):
                     if line.strip():
                         summary_line = line.strip()
                         break
            logger.debug(f"Extracted summary line: {summary_line}")


            total_headlines = len(news_headlines)
            if total_headlines > 0:
                # Scoring based on the ratio of positive vs negative headlines, scaled to 25 points
                # (Positive - Negative) / Total * 12.5 + 12.5 -> Range 0-25
                sentiment_score = ((positive_count - negative_count) / total_headlines) * 12.5 + 12.5
            else:
                sentiment_score = 12.5 # Neutral score if no headlines

            sentiment_score = max(0, min(sentiment_score, 25)) # Cap score between 0 and 25
            logger.info(f"Sentiment score calculated: {sentiment_score:.2f}/25")

            results = {
                'positive': positive_count,
                'neutral': neutral_count,
                'negative': negative_count,
                'score': sentiment_score,
                'summary': summary_line
            }
            logger.info("Sentiment analysis completed.")
            return results

        except Exception as e:
            logger.error(f"Error analyzing sentiment with Gemini API: {e}. Skipping sentiment analysis.")
            return {'positive': 0, 'neutral': 0, 'negative': 0, 'score': 0, 'summary': 'Sentiment analysis failed.'}


    def generate_enhanced_recommendation(self, fundamental_score, technical_score,
                                       liquidity_score, peer_performance, index_membership,
                                       sentiment_analysis_results):
        """Generate comprehensive recommendation including sentiment"""
        logger.info("Generating enhanced recommendation.")
        sentiment_score = sentiment_analysis_results.get('score', 0)
        total_score = fundamental_score + technical_score + liquidity_score + sentiment_score # Add sentiment score
        logger.debug(f"Scores: Fundamental={fundamental_score:.2f}, Technical={technical_score:.2f}, Liquidity={liquidity_score:.2f}, Sentiment={sentiment_score:.2f}")


        # Adjust for peer performance
        if peer_performance.get('outperformance', False):
            total_score += 5
            logger.debug("+5 pts for peer outperformance.")

        # Bonus for index membership
        if index_membership:
            total_score += 5
            logger.debug(f"+5 pts for index membership ({', '.join(index_membership)}).")


        # Max possible score is 50 (Fundamental) + 50 (Technical) + 70 (Liquidity) + 25 (Sentiment) + 5 (Peer) + 5 (Index) = 205
        # Let's re-evaluate the recommendation thresholds based on this potential range

        recommendation = "UNKNOWN"
        confidence = "Low"

        if total_score >= 160: # Example Thresholds (adjust as needed)
            recommendation = "STRONG BUY"
            confidence = "High"
        elif total_score >= 130:
            recommendation = "BUY"
            confidence = "Moderate to High"
        elif total_score >= 90:
            recommendation = "HOLD"
            confidence = "Moderate"
        elif total_score >= 60:
            recommendation = "SELL"
            confidence = "Moderate"
        else:
            recommendation = "STRONG SELL"
            confidence = "High"

        logger.info(f"Final total score: {total_score:.2f}. Recommendation: {recommendation} ({confidence})")

        return {
            'recommendation': recommendation,
            'confidence': confidence,
            'total_score': total_score,
            'breakdown': {
                'fundamental': fundamental_score,
                'technical': technical_score,
                'liquidity': liquidity_score,
                'sentiment': sentiment_score
            }
        }

# Integration function to work with your Part 1
def analyze_stock_enhanced(symbol, stock_analyzer_part1, enhanced_analyzer):
    """
    Integrate enhanced features with your Part 1 analyzer

    Args:
        symbol: Stock symbol (e.g., 'RELIANCE.NS')
        stock_analyzer_part1: Instance of your IndianStockAnalyzer from Part 1
        enhanced_analyzer: Instance of EnhancedStockAnalyzer
    """
    logger.info(f"Starting enhanced analysis integration for {symbol}.")

    # Get basic analysis from Part 1
    logger.info("Running basic analysis (Part 1).")
    stock_data = stock_analyzer_part1.analyze_stock(symbol)

    if not stock_data:
        logger.error(f"Basic analysis failed for {symbol}. Cannot proceed with enhanced analysis.")
        return None

    logger.info("Basic analysis completed successfully.")

    # Get current price
    current_price = stock_data.current_price
    logger.debug(f"Current price from basic analysis: {current_price}")


    # Enhanced analysis
    logger.info("Starting enhanced analysis features (Part 2).")
    # 1. Circuit breaker analysis
    circuit_limits = enhanced_analyzer.get_circuit_limits(symbol)
    circuit_risk = enhanced_analyzer.assess_circuit_risk(current_price, circuit_limits)
    logger.info(f"Circuit analysis completed. Risk score: {circuit_risk}")


    # 2. Enhanced liquidity analysis
    liquidity_score = enhanced_analyzer.get_liquidity_score(stock_data.price_data)
    logger.info(f"Liquidity analysis completed. Score: {liquidity_score}")

    # 3. Peer analysis
    peers = enhanced_analyzer.get_peer_companies(symbol, stock_data.info)
    peer_performance = enhanced_analyzer.calculate_peer_relative_performance(symbol, peers)
    logger.info(f"Peer analysis completed. Performance results: {peer_performance}")

    # 4. Index membership
    index_membership = enhanced_analyzer.get_index_membership(symbol)
    logger.info(f"Index membership check completed. Memberships: {index_membership}")


    # 5. Recalculate technical score with circuit risk
    enhanced_technical_score = enhanced_analyzer.calculate_enhanced_technical_score(
        stock_data.indicators, circuit_risk
    )
    logger.info(f"Enhanced technical score calculated: {enhanced_technical_score}")


    # 6. Fetch and analyze news sentiment
    news_headlines = enhanced_analyzer.fetch_news(symbol)
    sentiment_analysis_results = enhanced_analyzer.analyze_sentiment(news_headlines)
    logger.info(f"Sentiment analysis completed. Results: {sentiment_analysis_results}")


    # 7. Generate enhanced recommendation
    final_recommendation = enhanced_analyzer.generate_enhanced_recommendation(
        stock_data.fundamental_score,
        enhanced_technical_score,
        liquidity_score,
        peer_performance,
        index_membership,
        sentiment_analysis_results # Pass sentiment analysis results
    )
    logger.info("Enhanced recommendation generated.")

    # Create enhanced report
    enhanced_report = {
        'basic_analysis': stock_data,
        'enhanced_features': {
            'circuit_limits': circuit_limits,
            'circuit_risk_score': circuit_risk,
            'liquidity_score': liquidity_score,
            'peers': peers,
            'peer_performance': peer_performance,
            'index_membership': index_membership,
            'dynamic_risk_free_rate': enhanced_analyzer.risk_free_rate,
            'news_headlines': news_headlines, # Include news in report
            'sentiment_analysis': sentiment_analysis_results # Include sentiment analysis results
        },
        'final_recommendation': final_recommendation
    }

    logger.info(f"Enhanced analysis integration completed for {symbol}.")
    return enhanced_report

# Example usage (removed from here as it's part of the next display step)


**Reasoning**:
Display the combined analysis report using the enhanced logging.



In [ ]:
#@title Analyze a stock and display the combined results
symbol_to_analyze = "RELIANCE.NS" # You can change this symbol

# Ensure both analyzers are instantiated (they should be if the cells above were run)
# If not, you might need to re-run the cells defining IndianStockAnalyzer and EnhancedStockAnalyzer
try:
    stock_analyzer = IndianStockAnalyzer()
    enhanced_analyzer = EnhancedStockAnalyzer()
except NameError:
    logger.error("IndianStockAnalyzer or EnhancedStockAnalyzer class not defined. Please run the preceding cells.")
    # print("Please run the cells defining IndianStockAnalyzer and EnhancedStockAnalyzer first.")
    # Exit or handle the error appropriately if classes are not defined
    stock_analyzer = None
    enhanced_analyzer = None


if stock_analyzer and enhanced_analyzer:
    combined_analysis_report = analyze_stock_enhanced(symbol_to_analyze, stock_analyzer, enhanced_analyzer)

    if combined_analysis_report:
        # Keeping print for final formatted output as requested by the user
        print(f"\n{'='*60}")
        print(f"Combined Stock Analysis Report for {symbol_to_analyze}")
        print(f"{'='*60}")

        # Display basic analysis results
        basic = combined_analysis_report['basic_analysis']
        print(f"\n--- Basic Analysis (from Part 1) ---")
        print(f"Company Name: {basic.company_name}")
        print(f"Current Price: ₹{basic.current_price:,.2f}" if basic.current_price is not None else "Current Price: N/A")
        print(f"Market Cap: ₹{basic.market_cap/10000000:,.2f} Cr" if basic.market_cap is not None else "Market Cap: N/A") # Use stored market_cap
        print(f"Sector: {basic.info.get('sector', 'N/A')}")
        print(f"Fundamental Score: {basic.fundamental_score}/50")
        print(f"Technical Score: {basic.technical_score}/50")
        print(f"Basic Recommendation: {basic.recommendation}")
        print("\nFundamental Metrics:")
        for metric, value in basic.metrics.items():
             if metric != 'Fundamental_Score':
                if value is not None and not np.isnan(value):
                     if isinstance(value, float):
                         if metric in ['ROE', 'Revenue_Growth', 'Net_Profit_Margin', 'Debt_to_Equity']:
                             print(f"{metric}: {value*100:.2f}%") # Display as percentage for D/E, ROE, Growth, Margin
                         else:
                             print(f"{metric}: {value:.2f}")
                     else:
                         print(f"{metric}: {value}")
                else:
                    print(f"{metric}: N/A (Missing Data)")


        print("\nTechnical Indicators:")
        for indicator, value in basic.indicators.items():
             if indicator != 'current_price':
                if value is not None and not np.isnan(value):
                    if isinstance(value, float):
                        print(f"{indicator}: {value:,.2f}")
                    else:
                        print(f"{indicator}: {value}")
                else:
                    print(f"{indicator}: N/A (Insufficient Data)")


        # Display enhanced features and final recommendation
        enhanced = combined_analysis_report['enhanced_features']
        final = combined_analysis_report['final_recommendation']

        print(f"\n--- Enhanced Analysis (from Part 2) ---")
        print(f"Circuit Limits: {enhanced.get('circuit_limits', {}).get('lower_circuit', 'N/A'):.2f} - {enhanced.get('circuit_limits', {}).get('upper_circuit', 'N/A'):.2f}" if enhanced.get('circuit_limits') else "Circuit Limits: N/A")
        print(f"Circuit Risk Score: {enhanced.get('circuit_risk_score', 'N/A')}")
        print(f"Liquidity Score: {enhanced.get('liquidity_score', 'N/A')}/70")

        if enhanced.get('peers'):
            print(f"Peer Companies: {', '.join(enhanced['peers'])}")
            if enhanced.get('peer_performance'):
                print(f"30-day Return: {enhanced['peer_performance'].get('stock_return', 'N/A'):.2f}%" if enhanced['peer_performance'].get('stock_return') is not None and not np.isnan(enhanced['peer_performance'].get('stock_return')) else "30-day Return: N/A")
                print(f"Peer Avg Return: {enhanced['peer_performance'].get('avg_peer_return', 'N/A'):.2f}%" if enhanced['peer_performance'].get('avg_peer_return') is not None and not np.isnan(enhanced['peer_performance'].get('avg_peer_return')) else "Peer Avg Return: N/A")
                print(f"Relative Performance: {enhanced['peer_performance'].get('relative_performance', 'N/A'):.2f}%" if enhanced['peer_performance'].get('relative_performance') is not None and not np.isnan(enhanced['peer_performance'].get('relative_performance')) else "Relative Performance: N/A")


        if enhanced.get('index_membership'):
            print(f"Index Membership: {', '.join(enhanced['index_membership'])}")

        # Sentiment Analysis Results
        sentiment_results = enhanced.get('sentiment_analysis', {})
        print(f"\n📰 News Sentiment Analysis:")
        if enhanced.get('news_headlines'):
             for headline in enhanced['news_headlines']:
                 print(f"- {headline}")
             print(f"Sentiment Counts: Positive={sentiment_results.get('positive', 0)}, Neutral={sentiment_results.get('neutral', 0)}, Negative={sentiment_results.get('negative', 0)}")
             print(f"Sentiment Score: {sentiment_results.get('score', 0):.2f}/25")
             print(f"Sentiment Summary: {sentiment_results.get('summary', 'N/A')}")
        else:
             print("No news headlines fetched or sentiment analysis skipped.")


        print(f"\n--- Final Recommendation ---")
        print(f"RECOMMENDATION: {final.get('recommendation', 'N/A')} ({final.get('confidence', 'N/A')})")
        print(f"Total Score: {final.get('total_score', 0):.2f}/205")
        print("Score Breakdown:")
        print(f"  - Fundamental: {final.get('breakdown', {}).get('fundamental', 'N/A'):.2f}")
        print(f"  - Technical (Enhanced): {final.get('breakdown', {}).get('technical', 'N/A'):.2f}") # Technical score should be float
        print(f"  - Liquidity: {final.get('breakdown', {}).get('liquidity', 'N/A'):.2f}")
        print(f"  - Sentiment: {final.get('breakdown', {}).get('sentiment', 'N/A'):.2f}")

        print(f"\n{'='*60}")

    else:
        logger.error(f"Could not generate combined analysis report for {symbol_to_analyze}.")
        # print(f"Could not generate combined analysis report for {symbol_to_analyze}.")

In [ ]:
#@title Corrected analyze_stock_enhanced function and example usage

import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import requests
from bs4 import BeautifulSoup
import warnings
warnings.filterwarnings('ignore')

# Ensure logging is configured (assuming this was done in a previous cell)
import logging
logger = logging.getLogger(__name__)

# --- IndianStockAnalyzer class definition (from Part 1) ---
# This needs to be included in this cell so it's defined before use
class IndianStockAnalyzer:
    """Basic analyzer for Indian stocks"""

    def __init__(self):
        """Initialize the basic analyzer."""
        # Basic setup if needed for Part 1 (e.g., API keys, configurations)
        pass

    def analyze_stock(self, symbol):
        """
        Performs basic fundamental and technical analysis for a given stock.
        Returns an object or dictionary containing the analysis results.
        """
        logger.info(f"Starting basic analysis for {symbol}")
        try:
            ticker = yf.Ticker(symbol)
            info = ticker.info

            if not info:
                 logger.warning(f"Could not fetch info for {symbol}.")
                 return None

            # Get price data for technical analysis
            price_data = ticker.history(period="1y") # Get 1 year of historical data

            if price_data.empty:
                logger.warning(f"Could not fetch price data for {symbol}.")
                # Return info even if price data is missing, as fundamental data might still be available
                # We will handle missing price data in enhanced calculations
                return self.AnalysisResult(symbol, info, price_data, np.nan, {})


            # 1. Fundamental Analysis (Simplified)
            # Extract key fundamental data points
            try:
                market_cap = info.get('marketCap', np.nan)
                pe_ratio = info.get('trailingPE', np.nan)
                pb_ratio = info.get('priceToBook', np.nan)
                dividend_yield = info.get('dividendYield', np.nan) # This is often already a percentage as a float
                sector = info.get('sector', 'N/A')
                industry = info.get('industry', 'N/A')
                current_price = info.get('currentPrice', np.nan)
                company_name = info.get('longName', symbol)

                # Ensure dividend_yield is not None before checking if it's NaN
                if dividend_yield is not None and not pd.isna(dividend_yield) and dividend_yield > 0:
                     # If dividend_yield is a float between 0 and 1, assume it's a ratio and convert to percentage
                     if 0 < dividend_yield <= 1:
                         dividend_yield *= 100
                         logger.info(f"Converted dividend yield for {symbol} from ratio to percentage: {dividend_yield:.2f}%")
                     else:
                         logger.info(f"Dividend yield for {symbol} is {dividend_yield:.2f}%. Assuming it's already in percentage or other form.")
                else:
                     logger.info(f"Dividend yield for {symbol} is unavailable or zero ({dividend_yield}).")
                     dividend_yield = np.nan # Ensure it's NaN if None or invalid

            except Exception as fund_e:
                 logger.error(f"Error extracting fundamental data for {symbol}: {fund_e}. Setting fundamental metrics to NaN.")
                 market_cap, pe_ratio, pb_ratio, dividend_yield, sector, industry, current_price, company_name = np.nan, np.nan, np.nan, np.nan, 'N/A', 'N/A', np.nan, symbol


            # Calculate a simple fundamental score (out of 50)
            fundamental_score = self.calculate_fundamental_score(market_cap, pe_ratio, pb_ratio, dividend_yield)

            # 2. Technical Analysis (Simplified)
            indicators = self.calculate_technical_indicators(price_data, current_price)
            technical_score = self.calculate_technical_score(indicators) # Basic technical score (out of 50)

            # Bundle results
            # Create an object or dictionary to hold all analysis results
            analysis_results = self.AnalysisResult(
                symbol=symbol,
                info=info,
                price_data=price_data,
                current_price=current_price, # Include current price
                company_name=company_name, # Include company name
                market_cap=market_cap, # Include market cap
                fundamental_score=fundamental_score,
                technical_score=technical_score,
                indicators=indicators # Include calculated indicators
            )

            logger.info(f"Basic analysis completed for {symbol}")
            return analysis_results

        except Exception as e:
            logger.error(f"An error occurred during basic analysis for {symbol}: {str(e)}")
            return None # Return None if basic analysis fails

    def calculate_fundamental_score(self, market_cap, pe_ratio, pb_ratio, dividend_yield):
        """Calculate a simple fundamental score based on key metrics (out of 50)"""
        score = 0
        # Ensure metrics are valid numbers before scoring
        market_cap_valid = isinstance(market_cap, (int, float)) and not pd.isna(market_cap) and market_cap > 0
        pe_ratio_valid = isinstance(pe_ratio, (int, float)) and not pd.isna(pe_ratio) and pe_ratio > 0
        pb_ratio_valid = isinstance(pb_ratio, (int, float)) and not pd.isna(pb_ratio) and pb_ratio > 0
        dividend_yield_valid = isinstance(dividend_yield, (int, float)) and not pd.isna(dividend_yield) and dividend_yield >= 0


        # Market Cap (Scale based on size, larger usually means more stable)
        if market_cap_valid:
            if market_cap > 10_00_000_00_00_000:  # > 10 Lakh Cr (Large Cap)
                score += 15
            elif market_cap > 50_000_00_00_000:   # > 50K Cr (Mid Cap)
                score += 10
            else: # Small Cap and below
                score += 5
        else:
             logger.warning("Market cap is invalid for fundamental scoring.")


        # PE Ratio (Lower is generally better, but depends on industry growth)
        if pe_ratio_valid:
            if pe_ratio < 20:
                score += 15
            elif pe_ratio < 30:
                score += 10
            elif pe_ratio < 40:
                score += 5
            else: # High PE
                score += 2
        else:
             logger.warning("PE ratio is invalid for fundamental scoring.")


        # PB Ratio (Lower is generally better, indicates undervaluation)
        if pb_ratio_valid:
            if pb_ratio < 3:
                score += 10
            elif pb_ratio < 5:
                score += 7
            else:
                score += 3
        else:
             logger.warning("PB ratio is invalid for fundamental scoring.")


        # Dividend Yield (Higher is better for income-seeking investors, indicates profitability)
        if dividend_yield_valid:
            if dividend_yield > 3: # > 3%
                score += 10
            elif dividend_yield > 1: # > 1%
                score += 7
            else:
                score += 3
        else:
             logger.warning("Dividend yield is invalid for fundamental scoring.")

        logger.info(f"Fundamental score calculated: {score}")
        return min(score, 50) # Cap score at 50

    def calculate_technical_indicators(self, price_data, current_price):
        """Calculate key technical indicators"""
        # Ensure price_data is valid before calculating indicators
        if not isinstance(price_data, pd.DataFrame) or price_data.empty:
            logger.warning("Insufficient price data for technical indicator calculation.")
            return {'RSI': np.nan, 'MACD': np.nan, 'MACD_signal': np.nan, 'SMA_20': np.nan, 'SMA_50': np.nan,
                    'BB_upper': np.nan, 'BB_lower': np.nan, 'BB_middle': np.nan, 'Volume_ratio': np.nan,
                    'current_price': current_price} # Return NaNs if data is missing


        indicators = {}
        try:
            # RSI (Relative Strength Index)
            delta = price_data['Close'].diff()
            gain = delta.where(delta > 0, 0)
            loss = -delta.where(delta < 0, 0)
            avg_gain = gain.ewm(com=14-1, adjust=False).mean()
            avg_loss = loss.ewm(com=14-1, adjust=False).mean()
            rs = avg_gain / avg_loss
            indicators['RSI'] = 100 - (100 / (1 + rs)).iloc[-1]
        except Exception as e:
            logger.warning(f"Could not calculate RSI: {e}. Setting to NaN.")
            indicators['RSI'] = np.nan

        try:
            # MACD (Moving Average Convergence Divergence)
            ema_12 = price_data['Close'].ewm(span=12, adjust=False).mean()
            ema_26 = price_data['Close'].ewm(span=26, adjust=False).mean()
            indicators['MACD'] = (ema_12 - ema_26).iloc[-1]
            indicators['MACD_signal'] = indicators['MACD'].ewm(span=9, adjust=False).mean().iloc[-1] # Calculate signal from MACD series
        except Exception as e:
            logger.warning(f"Could not calculate MACD: {e}. Setting to NaN.")
            indicators['MACD'] = np.nan
            indicators['MACD_signal'] = np.nan

        try:
            # Moving Averages
            indicators['SMA_20'] = price_data['Close'].rolling(window=20).mean().iloc[-1]
            indicators['SMA_50'] = price_data['Close'].rolling(window=50).mean().iloc[-1]
        except Exception as e:
            logger.warning(f"Could not calculate Moving Averages: {e}. Setting to NaN.")
            indicators['SMA_20'] = np.nan
            indicators['SMA_50'] = np.nan


        try:
            # Bollinger Bands
            rolling_mean = price_data['Close'].rolling(window=20).mean()
            rolling_std = price_data['Close'].rolling(window=20).std()
            indicators['BB_middle'] = rolling_mean.iloc[-1]
            indicators['BB_upper'] = (rolling_mean + (rolling_std * 2)).iloc[-1]
            indicators['BB_lower'] = (rolling_mean - (rolling_std * 2)).iloc[-1]
        except Exception as e:
            logger.warning(f"Could not calculate Bollinger Bands: {e}. Setting to NaN.")
            indicators['BB_middle'] = np.nan
            indicators['BB_upper'] = np.nan
            indicators['BB_lower'] = np.nan

        try:
            # Volume Analysis (e.g., current volume vs average volume)
            avg_volume_30d = price_data['Volume'].rolling(window=30).mean().iloc[-1]
            current_volume = price_data['Volume'].iloc[-1]
            # Check for zero division and NaN before calculating ratio
            if not pd.isna(avg_volume_30d) and avg_volume_30d > 0 and not pd.isna(current_volume):
                 indicators['Volume_ratio'] = current_volume / avg_volume_30d
            elif not pd.isna(current_volume) and current_volume > 0:
                 # If 30d avg is zero or NaN but current volume is positive, assume high relative volume
                 indicators['Volume_ratio'] = 2.0 # Arbitrary high ratio to indicate spike
                 logger.warning("30-day average volume is invalid for volume ratio, assuming spike due to positive current volume.")
            else:
                 indicators['Volume_ratio'] = np.nan
                 logger.warning("Insufficient or invalid volume data for volume ratio calculation.")

        except Exception as e:
            logger.warning(f"Could not calculate Volume Ratio: {e}. Setting to NaN.")
            indicators['Volume_ratio'] = np.nan

        # Include current price in indicators for technical score calculation
        indicators['current_price'] = current_price if not pd.isna(current_price) else np.nan

        logger.info(f"Technical indicators calculated: {indicators}")
        return indicators


    def calculate_technical_score(self, indicators):
        """Calculate a simple technical score based on indicators (out of 50)"""
        score = 0
        # Ensure indicators dictionary is valid
        if not isinstance(indicators, dict):
             logger.warning("Invalid indicators dictionary for technical scoring.")
             return 0

        # Scoring based on common indicator signals
        # Ensure indicator values are valid numbers before scoring
        rsi = indicators.get('RSI', np.nan)
        macd = indicators.get('MACD', np.nan)
        macd_signal = indicators.get('MACD_signal', np.nan)
        sma_20 = indicators.get('SMA_20', np.nan)
        sma_50 = indicators.get('SMA_50', np.nan)
        current_price = indicators.get('current_price', np.nan)
        bb_upper = indicators.get('BB_upper', np.nan)
        bb_lower = indicators.get('BB_lower', np.nan)
        volume_ratio = indicators.get('Volume_ratio', np.nan)


        # RSI Score
        if not pd.isna(rsi):
            if rsi < 30: # Oversold
                score += 10
            elif rsi > 70: # Overbought
                score -= 10
        else:
             logger.warning("RSI is NaN for technical scoring.")

        # MACD Score (Bullish crossover)
        if not pd.isna(macd) and not pd.isna(macd_signal):
            if macd > macd_signal:
                score += 10
        else:
             logger.warning("MACD or MACD signal is NaN for technical scoring.")


        # Moving Average Crossover (Bullish: 20-day > 50-day)
        if not pd.isna(sma_20) and not pd.isna(sma_50):
            if sma_20 > sma_50:
                score += 10
        else:
             logger.warning("SMA_20 or SMA_50 is NaN for technical scoring.")


        # Bollinger Bands (Price near lower band suggests potential buy)
        if not pd.isna(current_price) and not pd.isna(bb_lower) and not pd.isna(bb_upper):
            if current_price < bb_lower:
                score += 10
            elif current_price > bb_upper:
                score -= 10
        else:
             logger.warning("Bollinger Bands or current price is NaN for technical scoring.")


        # Volume Spike (Significant volume increase can confirm trends)
        if not pd.isna(volume_ratio):
            if volume_ratio > 1.5: # Volume is 50% higher than average
                 score += 10
        else:
             logger.warning("Volume ratio is NaN for technical scoring.")


        logger.info(f"Basic technical score calculated: {score}")
        return max(0, min(score, 50)) # Cap score between 0 and 50

    # Define a simple class to hold analysis results, similar to a struct
    class AnalysisResult:
        def __init__(self, symbol, info, price_data, current_price, company_name, market_cap, fundamental_score, technical_score, indicators):
            self.symbol = symbol
            self.info = info
            self.price_data = price_data
            self.current_price = current_price
            self.company_name = company_name
            self.market_cap = market_cap
            self.fundamental_score = fundamental_score
            self.technical_score = technical_score
            self.indicators = indicators

        def __repr__(self):
            return (f"AnalysisResult(symbol='{self.symbol}', company_name='{self.company_name}', "
                    f"current_price={self.current_price}, market_cap={self.market_cap}, "
                    f"fundamental_score={self.fundamental_score}, technical_score={self.technical_score})")
# --- End of IndianStockAnalyzer class definition ---


# Re-define the EnhancedStockAnalyzer class here as well to ensure the latest version is used
class EnhancedStockAnalyzer:
    """Enhanced analyzer with additional features for Indian markets"""

    def __init__(self):
        """Initialize the enhanced analyzer."""
        self.risk_free_rate = self.get_dynamic_risk_free_rate()
        # Assuming gemini_model is defined globally from a previous cell
        global gemini_model # Access the global gemini_model
        self.gemini_model = gemini_model

        # More detailed circuit limit percentages based on NSE categorization (simplified)
        self.circuit_limits_percentage = {
            'category_a': 0.20, # Typically high-liquidity index stocks
            'category_b': 0.10, # Most other actively traded stocks
            'category_t': 0.05  # Stocks in trade-for-trade segment
        }

        # Simple mapping of some known stocks to categories for demonstration
        self.stock_category_map = {
            'RELIANCE.NS': 'category_a',
            'TCS.NS': 'category_a',
            'HDFCBANK.NS': 'category_a',
            'INFY.NS': 'category_a',
            'ICICIBANK.NS': 'category_a',
            'VEDL.NS': 'category_b',
            'PNB.NS': 'category_b',
            'INDIGO.NS': 'category_b',
            'BANDHANBNK.NS': 'category_b',
            'PERSISTENT.NS': 'category_b',
            'COFORGE.NS': 'category_b',
            'LTTS.NS': 'category_b',
        }


    def get_dynamic_risk_free_rate(self):
        """Fetch current 10-year G-Sec yield"""
        try:
            return 0.072  # 7.2% as of recent data
        except Exception as e:
            logger.warning(f"Could not fetch dynamic risk-free rate: {e}. Using fallback rate.")
            return 0.065

    def get_circuit_limits(self, symbol):
        """Get circuit breaker limits for the stock based on categorization"""
        try:
            ticker = yf.Ticker(symbol)
            info = ticker.info

            if not info or 'previousClose' not in info:
                 logger.warning(f"Could not get previous close for {symbol}. Cannot calculate circuit limits.")
                 return None

            prev_close = info.get('previousClose')
            if pd.isna(prev_close) or prev_close <= 0:
                 logger.warning(f"Invalid previous close price for {symbol}: {prev_close}. Cannot calculate circuit limits.")
                 return None


            stock_category = self.stock_category_map.get(symbol.upper(), 'category_b')
            circuit_percent = self.circuit_limits_percentage.get(stock_category, 0.10)

            logger.info(f"Determined circuit percentage for {symbol} (Category: {stock_category}): {circuit_percent*100}%")

            upper_circuit = prev_close * (1 + circuit_percent)
            lower_circuit = prev_close * (1 - circuit_percent)

            return {
                'upper_circuit': upper_circuit,
                'lower_circuit': lower_circuit,
                'circuit_percent': circuit_percent
            }
        except Exception as e:
            logger.error(f"Error getting circuit breaker limits for {symbol}: {str(e)}")
            return None

    def assess_circuit_risk(self, current_price, circuit_limits):
        """Assess proximity to circuit limits"""
        if pd.isna(current_price) or current_price <= 0:
             logger.warning("Invalid current price for circuit risk assessment.")
             return 0

        if not circuit_limits or 'upper_circuit' not in circuit_limits or 'lower_circuit' not in circuit_limits:
            logger.warning("Invalid or incomplete circuit limits provided.")
            return 0

        upper_circuit = circuit_limits['upper_circuit']
        lower_circuit = circuit_limits['lower_circuit']

        if pd.isna(upper_circuit) or pd.isna(lower_circuit) or upper_circuit <= 0 or lower_circuit <= 0:
             logger.warning("Circuit limits are invalid numbers. Cannot assess circuit risk.")
             return 0

        risk_score = 0

        if current_price >= upper_circuit:
             risk_score -= 30
        elif (upper_circuit - current_price) / current_price < 0.01:
            risk_score -= 20
        elif (upper_circuit - current_price) / current_price < 0.03:
            risk_score -= 10
        elif (upper_circuit - current_price) / current_price < 0.05:
            risk_score -= 5

        if current_price <= lower_circuit:
            risk_score -= 40
        elif (current_price - lower_circuit) / current_price < 0.01:
            risk_score -= 30
        elif (current_price - lower_circuit) / current_price < 0.03:
            risk_score -= 15
        elif (current_price - lower_circuit) / current_price < 0.05:
            risk_score -= 8

        logger.info(f"Circuit risk score calculated: {risk_score}")
        return risk_score

    def get_liquidity_score(self, price_data):
        """Enhanced liquidity analysis with NaN handling and better checks"""
        liquidity_score = 0
        if not isinstance(price_data, pd.DataFrame) or price_data.empty or len(price_data) < 20:
            logger.warning("Insufficient price data for liquidity analysis. Assigning default score.")
            return 30

        try:
            avg_volume_20d = price_data['Volume'].rolling(20).mean().iloc[-1]
            avg_volume_5d = price_data['Volume'].rolling(5).mean().iloc[-1]
            current_close = price_data['Close'].iloc[-1]

            if pd.isna(avg_volume_20d) or pd.isna(avg_volume_5d) or pd.isna(current_close) or current_close <= 0:
                 logger.warning("Liquidity metrics are NaN or invalid. Cannot calculate liquidity score accurately. Assigning default.")
                 return 30

            avg_turnover_20d = np.nan
            if avg_volume_20d is not None and not pd.isna(avg_volume_20d) and avg_volume_20d >= 0 and current_close > 0:
                 avg_turnover_20d = avg_volume_20d * current_close

            volume_spike = np.nan
            if avg_volume_20d is not None and not pd.isna(avg_volume_20d) and avg_volume_20d > 0:
                 if avg_volume_5d is not None and not pd.isna(avg_volume_5d):
                     volume_spike = avg_volume_5d / avg_volume_20d
                 else:
                      logger.warning("5-day average volume is invalid for volume spike calculation.")
            elif avg_volume_5d is not None and not pd.isna(avg_volume_5d) and avg_volume_5d > 0:
                 volume_spike = 1.0
                 logger.warning("20-day average volume is invalid for volume spike, using 1.0 as relative indicator.")
            else:
                 logger.warning("Insufficient or invalid volume data for volume spike calculation.")

            if not pd.isna(avg_turnover_20d):
                if avg_turnover_20d > 500_00_00_000:
                    liquidity_score += 40
                elif avg_turnover_20d > 100_00_00_000:
                    liquidity_score += 30
                elif avg_turnover_20d > 10_00_00_000:
                    liquidity_score += 20
                elif avg_turnover_20d > 1_00_00_000:
                    liquidity_score += 10
                else:
                    liquidity_score += 5
            else:
                 logger.warning("Average turnover is NaN. Skipping turnover scoring.")
                 liquidity_score += 10

            volume_std = price_data['Volume'].rolling(20).std().iloc[-1] if len(price_data) >= 20 else np.nan
            volume_cv = np.nan
            if avg_volume_20d is not None and not pd.isna(avg_volume_20d) and avg_volume_20d > 0 and \
               volume_std is not None and not pd.isna(volume_std):
                volume_cv = volume_std / avg_volume_20d

            if not pd.isna(volume_cv):
                if volume_cv < 0.5:
                    liquidity_score += 20
                elif volume_cv < 1.0:
                    liquidity_score += 10
            else:
                 logger.warning("Volume coefficient of variation is NaN. Skipping consistency scoring.")

            if not pd.isna(volume_spike) and volume_spike > 1.2:
                liquidity_score += 10
            else:
                 logger.warning("Volume spike ratio is NaN. Skipping trend scoring.")

            return min(liquidity_score, 70)

        except Exception as e:
            logger.error(f"Error calculating liquidity score: {str(e)}")
            return 30

    def get_peer_companies(self, symbol, stock_info):
        """Identify peer companies for comparison"""
        try:
            if not isinstance(stock_info, dict):
                 logger.warning("Invalid stock info for peer identification.")
                 return []

            sector = stock_info.get('sector', '')
            market_cap = stock_info.get('marketCap', 0)

            if not isinstance(sector, str):
                 sector = ''
            if not isinstance(market_cap, (int, float)) or pd.isna(market_cap) or market_cap <= 0:
                 logger.warning("Invalid market cap for peer identification.")
                 market_cap = 0

            if market_cap > 10_00_000_00_00_000:
                cap_category = 'Large Cap'
            elif market_cap > 50_000_00_00_000:
                cap_category = 'Mid Cap'
            else:
                cap_category = 'Small Cap'

            peer_map = {
                'Technology': {
                    'Large Cap': ['TCS.NS', 'INFY.NS', 'WIPRO.NS', 'HCLTECH.NS'],
                    'Mid Cap': ['TECHM.NS', 'LTTS.NS', 'PERSISTENT.NS', 'COFORGE.NS'],
                    'Small Cap': ['ZENSAR.NS', 'ECLERX.NS', 'MASTEK.NS']
                },
                'Financial Services': {
                    'Large Cap': ['HDFCBANK.NS', 'ICICIBANK.NS', 'KOTAKBANK.NS', 'AXISBANK.NS'],
                    'Mid Cap': ['IDFCFIRSTB.NS', 'FEDERALBNK.NS', 'INDUSINDBK.NS'],
                    'Small Cap': ['SOUTHBANK.NS', 'UJJIVAN.NS', 'EQUITASBNK.NS']
                },
                'Healthcare': {
                    'Large Cap': ['SUNPHARMA.NS', 'DRREDDY.NS', 'CIPLA.NS', 'DIVISLAB.NS'],
                    'Mid Cap': ['TORNTPHARM.NS', 'ALKEM.NS', 'LUPIN.NS'],
                    'Small Cap': ['CAPLIPOINT.NS', 'BLISS.NS', 'INDOCO.NS']
                }
            }

            peers = peer_map.get(sector, {}).get(cap_category, [])
            peers = [p for p in peers if isinstance(p, str) and p.upper() != symbol.upper()]

            logger.info(f"Identified {len(peers)} peer companies for {symbol} in {sector} ({cap_category} Cap).")
            return peers[:5]

        except Exception as e:
            logger.error(f"Error getting peer companies for {symbol}: {str(e)}")
            return []

    def calculate_peer_relative_performance(self, symbol, peers, days=30):
        """Compare stock performance with peers, handling data issues robustly"""
        results = {
            'stock_return': np.nan,
            'avg_peer_return': np.nan,
            'relative_performance': np.nan,
            'outperformance': False,
            'days': days
        }

        if not isinstance(peers, list) or not peers:
             logger.warning("No valid peer list provided for performance comparison.")
             pass

        try:
            main_ticker = yf.Ticker(symbol)
            main_hist = main_ticker.history(period=f"{days}d")
            main_return = np.nan

            if not main_hist.empty and len(main_hist) > 1:
                try:
                    start_price = main_hist['Close'].iloc[0]
                    end_price = main_hist['Close'].iloc[-1]
                    if not pd.isna(start_price) and not pd.isna(end_price) and start_price > 0:
                         main_return = ((end_price / start_price) - 1) * 100
                    else:
                         logger.warning(f"Invalid start ({start_price}) or end price ({end_price}) for main stock ({symbol}) return calculation.")
                except Exception as ret_e:
                     logger.warning(f"Could not calculate main stock ({symbol}) return: {ret_e}. Setting to NaN.")
                     main_return = np.nan
            else:
                logger.warning(f"Insufficient historical data for main stock ({symbol}) for return calculation.")

            results['stock_return'] = main_return

        except Exception as main_stock_e:
            logger.error(f"Error fetching data or calculating return for main stock ({symbol}): {main_stock_e}. Main stock return set to NaN.")
            results['stock_return'] = np.nan

        logger.warning("Skipping peer analysis to prevent errors. Peer performance metrics will be unavailable.")
        return results


    def get_index_membership(self, symbol):
        """Check index membership for the stock"""
        symbol_upper = symbol.upper() if isinstance(symbol, str) else ''

        index_constituents = {
            'NIFTY50': ['RELIANCE.NS', 'TCS.NS', 'HDFCBANK.NS', 'INFY.NS', 'ICICIBANK.NS'],
            'NIFTY_NEXT50': ['VEDL.NS', 'PNB.NS', 'INDIGO.NS', 'BANDHANBNK.NS'],
            'NIFTY_MIDCAP150': ['PERSISTENT.NS', 'COFORGE.NS', 'LTTS.NS']
        }

        membership = []
        for index, constituents in index_constituents.items():
            if symbol_upper and any(isinstance(c, str) and c.upper() == symbol_upper for c in constituents):
                membership.append(index)

        if membership:
             logger.info(f"Stock {symbol} is a member of indices: {', '.join(membership)}")
        else:
             logger.info(f"Stock {symbol} is not found in the hardcoded index constituents.")

        return membership

    def calculate_enhanced_technical_score(self, indicators, circuit_risk):
        """Calculate enhanced technical analysis score with NaN handling and circuit risk"""
        base_score = 0
        indicators_calculated = 0
        possible_indicator_points = 50

        num_key_indicators = 5
        indicator_point_contribution = possible_indicator_points / num_key_indicators

        rsi = indicators.get('RSI', np.nan)
        if not pd.isna(rsi):
             indicators_calculated += 1
             if 40 <= rsi <= 60:
                 base_score += indicator_point_contribution * 0.7
             elif rsi < 40:
                 base_score += indicator_point_contribution * 1.0
             else:
                 base_score += indicator_point_contribution * 0.5
        else:
             logger.warning("RSI not calculated or is NaN.")

        macd = indicators.get('MACD', np.nan)
        macd_signal = indicators.get('MACD_signal', np.nan)

        if not pd.isna(macd) and not pd.isna(macd_signal):
             indicators_calculated += 1
             if macd > macd_signal:
                 base_score += indicator_point_contribution * 1.0
             else:
                 base_score += indicator_point_contribution * 0.5
        else:
             logger.warning("MACD or MACD signal not calculated or is NaN. MACD score not added.")

        sma_20 = indicators.get('SMA_20', np.nan)
        sma_50 = indicators.get('SMA_50', np.nan)
        if not pd.isna(sma_20) and not pd.isna(sma_50):
            indicators_calculated += 1
            if sma_20 > sma_50:
                base_score += indicator_point_contribution * 1.0
            else:
                base_score += indicator_point_contribution * 0.5
        else:
            logger.warning("SMA_20 or SMA_50 is NaN for technical scoring.")

        current_price = indicators.get('current_price', np.nan)
        bb_upper = indicators.get('BB_upper', np.nan)
        bb_lower = indicators.get('BB_lower', np.nan)
        bb_middle = indicators.get('BB_middle', np.nan)

        if not pd.isna(current_price) and not pd.isna(bb_upper) and not pd.isna(bb_lower) and not pd.isna(bb_middle):
            indicators_calculated += 1
            if not pd.isna(bb_lower) and not pd.isna(bb_middle) and not pd.isna(current_price) and bb_lower < current_price < bb_middle:
                base_score += indicator_point_contribution * 1.0
            elif not pd.isna(bb_middle) and not pd.isna(bb_upper) and not pd.isna(current_price) and bb_middle < current_price < bb_upper:
                base_score += indicator_point_contribution * 0.7
            else:
                base_score += indicator_point_contribution * 0.5
        else:
             logger.warning("Bollinger Bands or current price is NaN for technical scoring.")

        volume_ratio = indicators.get('Volume_ratio', np.nan)
        if not pd.isna(volume_ratio):
             indicators_calculated += 1
             if 0.8 <= volume_ratio <= 1.5:
                 base_score += indicator_point_contribution * 0.7
             elif volume_ratio > 1.5:
                 base_score += indicator_point_contribution * 1.0
             else:
                 base_score += indicator_point_contribution * 0.5
        else:
             logger.warning("Volume ratio is NaN for technical scoring.")

        achieved_score_from_indicators = base_score

        if indicators_calculated > 0:
            max_possible_contribution = num_key_indicators * indicator_point_contribution
            achieved_percentage = achieved_score_from_indicators / max_possible_contribution if max_possible_contribution > 0 else 0
            adjusted_base_score = achieved_percentage * possible_indicator_points
            if indicators_calculated < num_key_indicators:
                 logger.warning(f"Only {indicators_calculated}/{num_key_indicators} key technical indicators calculated. Adjusting base score from {achieved_score_from_indicators:.2f} (raw) to {adjusted_base_score:.2f}.")
            base_score = adjusted_base_score
        else:
             logger.warning("No key technical indicators calculated. Base technical score is 0.")
             base_score = 0

        circuit_risk_adj = circuit_risk if isinstance(circuit_risk, (int, float)) and not pd.isna(circuit_risk) else 0

        final_score = base_score + circuit_risk_adj

        return max(0, min(final_score, 50))

    def fetch_news(self, symbol: str) -> List[str]:
        """
        Fetches recent news headlines for the given stock symbol.
        NOTE: This is a placeholder. In a real application, you would integrate
        with a News API (like News API, Alpha Vantage, etc.) here.
        """
        print(f"Fetching news for {symbol} (placeholder)...")
        symbol_str = str(symbol) if symbol is not None else "Unknown Stock"
        return [
            f"{symbol_str} stock price rises on positive market sentiment",
            f"{symbol_str} announces strong quarterly results, beating estimates",
            f"Experts bullish on {symbol_str}'s future outlook",
            f"I am a non-numeric entry that might cause an error",
            f"{symbol_str} faces regulatory challenges in key market",
            f"Competitor's new product launch impacts {symbol_str}'s market share",
            f"Global market trends positively impacting {symbol_str}",
            f"{symbol_str} management provides optimistic guidance",
            f"Increased foreign investment flows into {symbol_str}",
            f"Regulatory approval received for {symbol_str}'s new product",
            f"Production issues reported for {symbol_str}",
            f"Increased competition puts pressure on {symbol_str}'s margins",
            f"Analyst downgrades rating for {symbol_str}",
            f"Supply chain disruptions affect {symbol_str}'s operations",
            f"Geopolitical tensions create uncertainty for {symbol_str}",
            f"{symbol_str} announces stock split",
            f"Dividend declared by {symbol_str}"
        ]


    def analyze_sentiment(self, news_headlines: List[str]) -> Dict:
        """
        Analyzes the sentiment of news headlines using the Gemini API.
        Returns a dictionary with sentiment counts (positive, neutral, negative)
        and a sentiment score.
        """
        if not self.gemini_model:
            print("Gemini API not configured. Skipping sentiment analysis.")
            return {'positive': 0, 'neutral': 0, 'negative': 0, 'score': 0, 'summary': 'Gemini API not configured. Sentiment analysis skipped.'}

        if not isinstance(news_headlines, list) or not news_headlines:
            print("No news headlines to analyze. Skipping sentiment analysis.")
            return {'positive': 0, 'neutral': 0, 'negative': 0, 'score': 12.5, 'summary': 'No news headlines. Sentiment analysis skipped.'}

        logger.info("Analyzing sentiment using Gemini API...")
        prompt = "Analyze the sentiment of the following news headlines for a stock and categorize each as Positive, Neutral, or Negative. Provide a concise summary of the overall sentiment.\n\n"
        for headline in news_headlines:
            if isinstance(headline, str):
                 prompt += f"- {headline}\n"
            else:
                 logger.warning(f"Skipping non-string headline: {headline}")
                 continue

        if len(prompt) > 30000:
             logger.warning("News headlines too long for Gemini API prompt. Truncating.")
             prompt = prompt[:30000] + "\n... (headlines truncated)"

        try:
            response = self.gemini_model.generate_content(prompt)
            sentiment_text = response.text.strip()
            logger.info(f"Gemini API Response: {sentiment_text[:200]}...")

            positive_count = sentiment_text.lower().count('positive')
            neutral_count = sentiment_text.lower().count('neutral')
            negative_count = sentiment_text.lower().count('negative')

            summary_line = "No summary extracted."
            lines = sentiment_text.split('\n')
            for line in lines:
                 lower_line = line.lower()
                 if 'summary' in lower_line or 'overall sentiment' in lower_line:
                     summary_line = line.strip()
                     break
            if summary_line == "No summary extracted." and lines:
                 for line in reversed(lines):
                     if line.strip():
                         summary_line = line.strip()
                         break

            total_headlines = len([h for h in news_headlines if isinstance(h, str)])
            if total_headlines > 0:
                sentiment_score = ((positive_count - negative_count) / total_headlines) * 12.5 + 12.5
            else:
                sentiment_score = 12.5

            return {
                'positive': positive_count,
                'neutral': neutral_count,
                'negative': negative_count,
                'score': max(0, min(sentiment_score, 25)),
                'summary': summary_line
            }

        except Exception as e:
            logger.error(f"Error analyzing sentiment with Gemini API: {e}. Skipping sentiment analysis.")
            return {'positive': 0, 'neutral': 0, 'negative': 0, 'score': 0, 'summary': 'Sentiment analysis failed.'}


    def generate_enhanced_recommendation(self, fundamental_score, technical_score,
                                       liquidity_score, peer_performance, index_membership,
                                       sentiment_analysis_results):
        """Generate comprehensive recommendation including sentiment"""
        fund_score = fundamental_score if isinstance(fundamental_score, (int, float)) and not pd.isna(fundamental_score) else 0
        tech_score = technical_score if isinstance(technical_score, (int, float)) and not pd.isna(technical_score) else 0
        liq_score = liquidity_score if isinstance(liquidity_score, (int, float)) and not pd.isna(liquidity_score) else 0
        sentiment_score = sentiment_analysis_results.get('score', 0) if isinstance(sentiment_analysis_results.get('score'), (int, float)) and not pd.isna(sentiment_analysis_results.get('score')) else 0

        total_score = fund_score + tech_score + liq_score + sentiment_score

        peer_analysis_available = isinstance(peer_performance, dict) and \
                                  not (pd.isna(peer_performance.get('stock_return')) and \
                                       pd.isna(peer_performance.get('avg_peer_return')) and \
                                       pd.isna(peer_performance.get('relative_performance')))

        if peer_analysis_available and peer_performance.get('outperformance', False) is True:
            total_score += 5
            logger.info("Adding bonus for peer outperformance.")
        elif not peer_analysis_available:
             total_score -= 5
             logger.warning("Applying small penalty as peer performance data is incomplete/unavailable.")

        if isinstance(index_membership, list) and index_membership:
            total_score += 5
            logger.info("Adding bonus for index membership.")

        recommendation = "NEUTRAL"
        confidence = "Low"

        if total_score >= 170:
            recommendation = "STRONG BUY"
            confidence = "High"
        elif total_score >= 140:
            recommendation = "BUY"
            confidence = "Moderate to High"
        elif total_score >= 100:
            recommendation = "HOLD"
            confidence = "Moderate"
        elif total_score >= 70:
            recommendation = "SELL"
            confidence = "Moderate"
        else:
            recommendation = "STRONG SELL"
            confidence = "High"

        logger.info(f"Generated enhanced recommendation: {recommendation} (Total Score: {total_score:.2f})")

        return {
            'recommendation': recommendation,
            'confidence': confidence,
            'total_score': total_score,
            'breakdown': {
                'fundamental': fund_score,
                'technical': tech_score,
                'liquidity': liq_score,
                'sentiment': sentiment_score,
                'peer_adjustment': 5 if peer_analysis_available and peer_performance.get('outperformance', False) is True else (-5 if not peer_analysis_available else 0),
                'index_adjustment': 5 if isinstance(index_membership, list) and index_membership else 0
            }
        }

# Integration function to work with your Part 1
def analyze_stock_enhanced(symbol, stock_analyzer_part1, enhanced_analyzer):
    """
    Integrate enhanced features with your Part 1 analyzer

    Args:
        symbol: Stock symbol (e.g., 'RELIANCE.NS')
        stock_analyzer_part1: Instance of your IndianStockAnalyzer from Part 1
        enhanced_analyzer: Instance of EnhancedStockAnalyzer
    """
    if not isinstance(symbol, str) or not symbol:
        logger.error("Invalid stock symbol provided for enhanced analysis.")
        return None
    if not isinstance(stock_analyzer_part1, IndianStockAnalyzer):
        logger.error("Invalid IndianStockAnalyzer (Part 1) instance provided.")
        return None
    if not isinstance(enhanced_analyzer, EnhancedStockAnalyzer):
        logger.error("Invalid EnhancedStockAnalyzer (Part 2) instance provided.")
        return None


    logger.info(f"Starting enhanced analysis for {symbol}")
    # Get basic analysis from Part 1
    stock_data = stock_analyzer_part1.analyze_stock(symbol)

    if not stock_data:
        logger.error(f"Basic analysis failed for {symbol}.")
        return None

    # Get current price - already validated in Part 1 analyze_stock
    current_price = stock_data.current_price
    if pd.isna(current_price) or current_price <= 0:
         logger.warning(f"Current price invalid ({current_price}). Enhanced analysis might be limited.")


    # Enhanced analysis
    # 1. Circuit breaker analysis
    circuit_limits = enhanced_analyzer.get_circuit_limits(symbol)
    # Pass current_price to assess_circuit_risk (already validated as best as possible in Part 1)
    circuit_risk = enhanced_analyzer.assess_circuit_risk(current_price, circuit_limits)

    # 2. Enhanced liquidity analysis
    # Pass price_data (already validated in Part 1 analyze_stock)
    # CORRECTED: Call get_liquidity_score on enhanced_analyzer and pass price_data from stock_data
    liquidity_score = enhanced_analyzer.get_liquidity_score(stock_data.price_data)

    # 3. Peer analysis (now temporarily skipped)
    # Pass info (already validated in Part 1 analyze_stock)
    peers = enhanced_analyzer.get_peer_companies(symbol, stock_data.info) # Still get peers for reporting, even if analysis is skipped
    peer_performance = enhanced_analyzer.calculate_peer_relative_performance(symbol, peers) # This function now returns early with default NaNs


    # 4. Index membership
    index_membership = enhanced_analyzer.get_index_membership(symbol)

    # 5. Recalculate technical score with circuit risk
    # Pass indicators (already validated in Part 1 calculate_technical_indicators)
    # Pass circuit_risk (already validated in assess_circuit_risk)
    enhanced_technical_score = enhanced_analyzer.calculate_enhanced_technical_score(
        stock_data.indicators, circuit_risk
    )

    # 6. Fetch and analyze news sentiment
    # Pass symbol (already validated)
    news_headlines = enhanced_analyzer.fetch_news(symbol)
    # Pass news_headlines (already validated)
    sentiment_analysis_results = enhanced_analyzer.analyze_sentiment(news_headlines)


    # 7. Generate enhanced recommendation
    # Pass scores and results (already validated within their respective functions)
    final_recommendation = enhanced_analyzer.generate_enhanced_recommendation(
        stock_data.fundamental_score,
        enhanced_technical_score,
        liquidity_score,
        peer_performance, # Pass potentially incomplete/NaN peer performance
        index_membership,
        sentiment_analysis_results # Pass sentiment analysis results
    )

    # Create enhanced report
    enhanced_report = {
        'basic_analysis': stock_data,
        'enhanced_features': {
            'circuit_limits': circuit_limits,
            'circuit_risk_score': circuit_risk,
            'liquidity_score': liquidity_score, # Ensure correct liquidity score is included
            'peers': peers,
            'peer_performance': peer_performance, # Include peer performance, even if NaN
            'index_membership': index_membership,
            'dynamic_risk_free_rate': enhanced_analyzer.risk_free_rate,
            'news_headlines': news_headlines, # Include news in report
            'sentiment_analysis': sentiment_analysis_results # Include sentiment analysis results
        },
        'final_recommendation': final_recommendation
    }
    logger.info(f"Enhanced analysis completed for {symbol}")
    return enhanced_report

# Example usage
if __name__ == "__main__":
    # Initialize analyzers
    # If running in Jupyter/Colab, use the following approach:
    # First, make sure you've run the Part 1 code in a previous cell
    # Then the IndianStockAnalyzer class will be available in the namespace

    # For standalone script usage:
    # from indian_stock_analyzer_part1 import IndianStockAnalyzer

    # For Jupyter/Colab where Part 1 was run in previous cell:
    # IndianStockAnalyzer should already be available

    try:
        # Check if IndianStockAnalyzer is already defined (from previous cell)
        # This check is less critical now that the class is defined in this cell,
        # but can remain as a fallback or for clarity.
        IndianStockAnalyzer
    except NameError:
        print("IndianStockAnalyzer class was not defined in a previous cell, but is defined in this cell.")
        # The class definition is now included above, so this error should not occur.
        pass


    stock_analyzer = IndianStockAnalyzer()
    enhanced_analyzer = EnhancedStockAnalyzer()

    # Analyze a stock
    symbol = "RELIANCE.NS" # Example symbol
    result = analyze_stock_enhanced(symbol, stock_analyzer, enhanced_analyzer)

    if result:
        # Display the enhanced report
        enhanced = result['enhanced_features']
        final = result['final_recommendation']
        basic = result['basic_analysis'] # Get basic data for display

        print(f"\n{'='*60}")
        print(f"Enhanced Analysis Report: {basic.company_name} ({basic.symbol})")
        print(f"{'='*60}")

        print(f"\n📊 Current Market Data:")
        print(f"Current Price: ₹{basic.current_price:,.2f}" if not pd.isna(basic.current_price) else "Current Price: N/A")
        market_cap_cr = basic.market_cap / 10000000 if not pd.isna(basic.market_cap) else np.nan
        print(f"Market Cap: ₹{market_cap_cr:,.2f} Cr" if not pd.isna(market_cap_cr) else "Market Cap: N/A")
        print(f"Sector: {basic.info.get('sector', 'N/A')}")

        print(f"\n⚙️ Enhanced Technical Analysis & Risk:")
        # Display enhanced technical indicators and risk
        # Assuming you want to display relevant indicators from basic.indicators and the circuit risk score
        circuit_limits_display = enhanced.get('circuit_limits')
        if circuit_limits_display and not pd.isna(circuit_limits_display.get('lower_circuit')) and not pd.isna(circuit_limits_display.get('upper_circuit')):
             print(f"Circuit Limits: {circuit_limits_display.get('lower_circuit', 'N/A'):.2f} - {circuit_limits_display.get('upper_circuit', 'N/A'):.2f}")
        else:
             print("Circuit Limits: N/A")

        print(f"Circuit Risk Score: {enhanced.get('circuit_risk_score', 'N/A')}")
        # Check if liquidity score is available before formatting
        liquidity_score_display = f"{enhanced.get('liquidity_score', 'N/A')}/70" if isinstance(enhanced.get('liquidity_score'), (int, float)) and not pd.isna(enhanced.get('liquidity_score')) else "Liquidity Score: N/A/70"
        print(liquidity_score_display)


        # Display some key technical indicators from basic analysis
        print("\nKey Technical Indicators (from Basic Analysis):")
        key_tech_indicators = ['RSI', 'MACD', 'MACD_signal', 'SMA_20', 'SMA_50', 'BB_middle', 'Volume_ratio']
        for indicator_name in key_tech_indicators:
            value = basic.indicators.get(indicator_name, np.nan)
            if not pd.isna(value):
                if isinstance(value, float):
                     print(f"  - {indicator_name}: {value:,.2f}")
                else:
                     print(f"  - {indicator_name}: {value}")
            else:
                print(f"  - {indicator_name}: N/A")


        print(f"\n🤝 Peer Analysis:")
        # Check if peer analysis was skipped or failed by examining the presence of meaningful data
        peer_perf = enhanced.get('peer_performance')
        if enhanced.get('peers') and peer_perf and not (pd.isna(peer_perf.get('stock_return')) and pd.isna(peer_perf.get('avg_peer_return')) and pd.isna(peer_perf.get('relative_performance'))):
             print(f"Peer Companies: {', '.join(enhanced['peers'])}")
             # Add checks for NaN before formatting float
             stock_return_display = f"{peer_perf.get('stock_return', 'N/A'):.2f}%" if isinstance(peer_perf.get('stock_return'), (int, float)) and not pd.isna(peer_perf.get('stock_return')) else 'N/A'
             avg_peer_return_display = f"{peer_perf.get('avg_peer_return', 'N/A'):.2f}%" if isinstance(peer_perf.get('avg_peer_return'), (int, float)) and not pd.isna(peer_perf.get('avg_peer_return')) else 'N/A'
             relative_performance_display = f"{peer_perf.get('relative_performance', 'N/A'):.2f}%" if isinstance(peer_perf.get('relative_performance'), (int, float)) and not pd.isna(peer_perf.get('relative_performance')) else 'N/A'

             print(f"  - {peer_perf.get('days', 'N/A')}-day Stock Return: {stock_return_display}")
             print(f"  - Avg Peer Return: {avg_peer_return_display}")
             print(f"  - Relative Performance: {relative_performance_display}")
        else:
             # Display a message indicating peer analysis was skipped/failed
             print("Peer analysis skipped or failed. Data unavailable.")


        if enhanced.get('index_membership'):
            print(f"\n🏛️ Index Membership: {', '.join(enhanced['index_membership'])}")

        # Sentiment Analysis Results
        sentiment_results = enhanced.get('sentiment_analysis', {})
        print(f"\n📰 News Sentiment Analysis:")
        if enhanced.get('news_headlines'):
             # Only print a few headlines to keep the report concise
             print("Sample News Headlines:")
             # Ensure headlines are valid strings before printing
             valid_headlines = [h for h in enhanced['news_headlines'] if isinstance(h, str)]
             for headline in valid_headlines[:5]: # Print up to 5 headlines
                 print(f"- {headline}")
             if len(valid_headlines) > 5:
                  print("  ...")

             print(f"Sentiment Counts: Positive={sentiment_results.get('positive', 0)}, Neutral={sentiment_results.get('neutral', 0)}, Negative={sentiment_results.get('negative', 0)}")
             sentiment_score_display = f"{sentiment_results.get('score', 0):.2f}" if isinstance(sentiment_results.get('score'), (int, float)) and not pd.isna(sentiment_results.get('score')) else "N/A"
             print(f"Sentiment Score: {sentiment_score_display}/25")
             print(f"Sentiment Summary: {sentiment_results.get('summary', 'N/A')}")
        else:
             print("No news headlines fetched or sentiment analysis skipped.")


        print(f"\n--- Comprehensive Scores and Recommendation ---")
        print(f"Fundamental Score (from Part 1): {basic.fundamental_score:.2f}/50" if not pd.isna(basic.fundamental_score) else "Fundamental Score: N/A")
        # Add checks for NaN before formatting float for breakdown scores
        enhanced_tech_score_display = f"{final.get('breakdown', {}).get('technical', 'N/A'):.2f}" if isinstance(final.get('breakdown', {}).get('technical'), (int, float)) and not pd.isna(final.get('breakdown', {}).get('technical')) else "N/A"
        liquidity_score_display_breakdown = f"{final.get('breakdown', {}).get('liquidity', 'N/A'):.2f}" if isinstance(final.get('breakdown', {}).get('liquidity'), (int, float)) and not pd.isna(final.get('breakdown', {}).get('liquidity')) else "N/A"
        sentiment_score_display_breakdown = f"{final.get('breakdown', {}).get('sentiment', 'N/A'):.2f}" if isinstance(final.get('breakdown', {}).get('sentiment'), (int, float)) and not pd.isna(final.get('breakdown', {}).get('sentiment')) else "N/A"

        print(f"Technical Score (Enhanced): {enhanced_tech_score_display}/50")
        print(f"Liquidity Score: {liquidity_score_display_breakdown}/70")
        print(f"Sentiment Score: {sentiment_score_display_breakdown}/25")

        # Display adjustment scores
        peer_adj_display = f"{final.get('breakdown', {}).get('peer_adjustment', 'N/A'):+.2f}" if isinstance(final.get('breakdown', {}).get('peer_adjustment'), (int, float)) and not pd.isna(final.get('breakdown', {}).get('peer_adjustment')) else "N/A"
        index_adj_display = f"{final.get('breakdown', {}).get('index_adjustment', 'N/A'):+.2f}" if isinstance(final.get('breakdown', {}).get('index_adjustment'), (int, float)) and not pd.isna(final.get('breakdown', {}).get('index_adjustment')) else "N/A"

        print(f"Peer Adjustment: {peer_adj_display}")
        print(f"Index Membership Adjustment: {index_adj_display}")


        print(f"\nTotal Enhanced Score: {final.get('total_score', 0):.2f}/205" if isinstance(final.get('total_score'), (int, float)) and not pd.isna(final.get('total_score')) else "Total Enhanced Score: N/A")
        print(f"RECOMMENDATION: {final.get('recommendation', 'N/A')} (Confidence: {final.get('confidence', 'N/A')})")


        print(f"\n{'='*60}\n")

In [ ]:
# Part 1: IndianStockAnalyzer class definition

import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import requests
from bs4 import BeautifulSoup
import warnings
warnings.filterwarnings('ignore')

# Import typing for type hints
from typing import List, Dict

# Ensure logging is configured (assuming this was done in a previous cell)
import logging
logger = logging.getLogger(__name__)

# --- IndianStockAnalyzer class definition ---
class IndianStockAnalyzer:
    """Basic analyzer for Indian stocks"""

    def __init__(self):
        """Initialize the basic analyzer."""
        # Basic setup if needed for Part 1 (e.g., API keys, configurations)
        pass

    def analyze_stock(self, symbol):
        """
        Performs basic fundamental and technical analysis for a given stock.
        Returns an object or dictionary containing the analysis results.
        """
        logger.info(f"Starting basic analysis for {symbol}")
        try:
            ticker = yf.Ticker(symbol)
            info = ticker.info

            if not info:
                 logger.warning(f"Could not fetch info for {symbol}.")
                 # Ensure all expected keys are present with None or np.nan if info is empty
                 empty_info_result = self.AnalysisResult(
                     symbol=symbol,
                     info={}, # Provide an empty dict if info fetching failed
                     price_data=pd.DataFrame(),
                     current_price=np.nan,
                     company_name=symbol,
                     market_cap=np.nan,
                     fundamental_score=np.nan,
                     technical_score=np.nan,
                     indicators={},
                     pe_ratio=np.nan, # Added for Valuation
                     pb_ratio=np.nan, # Added for Valuation
                     ev_ebitda=np.nan # Added for Valuation
                 )
                 return empty_info_result


            # Get price data for technical analysis
            price_data = ticker.history(period="1y") # Get 1 year of historical data

            if price_data.empty:
                logger.warning(f"Could not fetch price data for {symbol}.")
                # Return info even if price data is missing, as fundamental data might still be available
                # We will handle missing price data in enhanced calculations
                return self.AnalysisResult(
                     symbol=symbol,
                     info=info, # Use fetched info
                     price_data=price_data,
                     current_price=info.get('currentPrice', np.nan), # Get current price from info if price_data is empty
                     company_name=info.get('longName', symbol), # Get company name from info
                     market_cap=info.get('marketCap', np.nan),  # Get market cap from info
                     fundamental_score=np.nan, # Fundamental score needs fundamental data
                     technical_score=np.nan, # Technical score needs price data
                     indicators={}, # Indicators need price data
                     pe_ratio=info.get('trailingPE', np.nan), # Added for Valuation
                     pb_ratio=info.get('priceToBook', np.nan), # Added for Valuation
                     ev_ebitda=info.get('enterpriseToEbitda', np.nan) # Added for Valuation
                )


            # 1. Fundamental Analysis (Simplified)
            # Extract key fundamental data points
            try:
                market_cap = info.get('marketCap', np.nan)
                pe_ratio = info.get('trailingPE', np.nan)
                pb_ratio = info.get('priceToBook', np.nan)
                dividend_yield = info.get('dividendYield', np.nan) # This is often already a percentage as a float
                sector = info.get('sector', 'N/A')
                industry = info.get('industry', 'N/A')
                current_price = info.get('currentPrice', np.nan)
                company_name = info.get('longName', symbol)
                ev_ebitda = info.get('enterpriseToEbitda', np.nan) # Added for Valuation

                # Ensure dividend_yield is not None before checking if it's NaN
                if dividend_yield is not None and not pd.isna(dividend_yield) and dividend_yield > 0:
                     # If dividend_yield is a float between 0 and 1, assume it's a ratio and convert to percentage
                     if 0 < dividend_yield <= 1:
                         dividend_yield *= 100
                         logger.info(f"Converted dividend yield for {symbol} from ratio to percentage: {dividend_yield:.2f}%")
                     else:
                         logger.info(f"Dividend yield for {symbol} is {dividend_yield:.2f}%. Assuming it's already in percentage or other form.")
                else:
                     logger.info(f"Dividend yield for {symbol} is unavailable or zero ({dividend_yield}).")
                     dividend_yield = np.nan # Ensure it's NaN if None or invalid

            except Exception as fund_e:
                 logger.error(f"Error extracting fundamental data for {symbol}: {fund_e}. Setting fundamental metrics to NaN.")
                 market_cap, pe_ratio, pb_ratio, dividend_yield, sector, industry, current_price, company_name, ev_ebitda = np.nan, np.nan, np.nan, np.nan, 'N/A', 'N/A', np.nan, symbol, np.nan


            # Calculate a simple fundamental score (out of 50)
            fundamental_score = self.calculate_fundamental_score(market_cap, pe_ratio, pb_ratio, dividend_yield)

            # 2. Technical Analysis (Simplified)
            indicators = self.calculate_technical_indicators(price_data, current_price)
            technical_score = self.calculate_technical_score(indicators) # Basic technical score (out of 50)

            # Bundle results
            # Create an object or dictionary to hold all analysis results
            analysis_results = self.AnalysisResult(
                symbol=symbol,
                info=info,
                price_data=price_data,
                current_price=current_price, # Include current price
                company_name=company_name, # Include company name
                market_cap=market_cap, # Include market cap
                fundamental_score=fundamental_score,
                technical_score=technical_score,
                indicators=indicators, # Include calculated indicators
                pe_ratio=pe_ratio, # Added for Valuation
                pb_ratio=pb_ratio, # Added for Valuation
                ev_ebitda=ev_ebitda # Added for Valuation
            )

            logger.info(f"Basic analysis completed for {symbol}")
            return analysis_results

        except Exception as e:
            logger.error(f"An error occurred during basic analysis for {symbol}: {str(e)}")
            return None # Return None if basic analysis fails

    def calculate_fundamental_score(self, market_cap, pe_ratio, pb_ratio, dividend_yield):
        """Calculate a simple fundamental score based on key metrics (out of 50)"""
        score = 0
        # Ensure metrics are valid numbers before scoring
        market_cap_valid = isinstance(market_cap, (int, float)) and not pd.isna(market_cap) and market_cap > 0
        pe_ratio_valid = isinstance(pe_ratio, (int, float)) and not pd.isna(pe_ratio) and pe_ratio > 0
        pb_ratio_valid = isinstance(pb_ratio, (int, float)) and not pd.isna(pb_ratio) and pb_ratio > 0
        dividend_yield_valid = isinstance(dividend_yield, (int, float)) and not pd.isna(dividend_yield) and dividend_yield >= 0


        # Market Cap (Scale based on size, larger usually means more stable)
        if market_cap_valid:
            if market_cap > 10_00_000_00_00_000:  # > 10 Lakh Cr (Large Cap)
                score += 15
            elif market_cap > 50_000_00_00_000:   # > 50K Cr (Mid Cap)
                score += 10
            else: # Small Cap and below
                score += 5
        else:
             logger.warning("Market cap is invalid for fundamental scoring.")


        # PE Ratio (Lower is generally better, but depends on industry growth)
        if pe_ratio_valid:
            if pe_ratio < 20:
                score += 15
            elif pe_ratio < 30:
                score += 10
            elif pe_ratio < 40:
                score += 5
            else: # High PE
                score += 2
        else:
             logger.warning("PE ratio is invalid for fundamental scoring.")


        # PB Ratio (Lower is generally better, indicates undervaluation)
        if pb_ratio_valid:
            if pb_ratio < 3:
                score += 10
            elif pb_ratio < 5:
                score += 7
            else:
                score += 3
        else:
             logger.warning("PB ratio is invalid for fundamental scoring.")


        # Dividend Yield (Higher is better for income-seeking investors, indicates profitability)
        if dividend_yield_valid:
            if dividend_yield > 3: # > 3%
                score += 10
            elif dividend_yield > 1: # > 1%
                score += 7
            else:
                score += 3
        else:
             logger.warning("Dividend yield is invalid for fundamental scoring.")

        logger.info(f"Fundamental score calculated: {score}")
        return min(score, 50) # Cap score at 50

    def calculate_technical_indicators(self, price_data, current_price):
        """Calculate key technical indicators"""
        # Ensure price_data is valid before calculating indicators
        if not isinstance(price_data, pd.DataFrame) or price_data.empty:
            logger.warning("Insufficient price data for technical indicator calculation.")
            return {'RSI': np.nan, 'MACD': np.nan, 'MACD_signal': np.nan, 'SMA_20': np.nan, 'SMA_50': np.nan,
                    'BB_upper': np.nan, 'BB_lower': np.nan, 'BB_middle': np.nan, 'Volume_ratio': np.nan,
                    'current_price': current_price} # Return NaNs if data is missing


        indicators = {}
        try:
            # RSI (Relative Strength Index)
            delta = price_data['Close'].diff()
            gain = delta.where(delta > 0, 0)
            loss = -delta.where(delta < 0, 0)
            avg_gain = gain.ewm(com=14-1, adjust=False).mean()
            avg_loss = loss.ewm(com=14-1, adjust=False).mean()
            rs = avg_gain / avg_loss
            indicators['RSI'] = 100 - (100 / (1 + rs)).iloc[-1]
        except Exception as e:
            logger.warning(f"Could not calculate RSI: {e}. Setting to NaN.")
            indicators['RSI'] = np.nan

        try:
            # MACD (Moving Average Convergence Divergence)
            ema_12 = price_data['Close'].ewm(span=12, adjust=False).mean()
            ema_26 = price_data['Close'].ewm(span=26, adjust=False).mean()
            # Corrected MACD signal calculation to use the MACD Series
            macd_series = ema_12 - ema_26
            indicators['MACD'] = macd_series.iloc[-1]
            indicators['MACD_signal'] = macd_series.ewm(span=9, adjust=False).mean().iloc[-1]
        except Exception as e:
            logger.warning(f"Could not calculate MACD: {e}. Setting to NaN.")
            indicators['MACD'] = np.nan
            indicators['MACD_signal'] = np.nan

        try:
            # Moving Averages
            indicators['SMA_20'] = price_data['Close'].rolling(window=20).mean().iloc[-1]
            indicators['SMA_50'] = price_data['Close'].rolling(window=50).mean().iloc[-1]
        except Exception as e:
            logger.warning(f"Could not calculate Moving Averages: {e}. Setting to NaN.")
            indicators['SMA_20'] = np.nan
            indicators['SMA_50'] = np.nan


        try:
            # Bollinger Bands
            rolling_mean = price_data['Close'].rolling(window=20).mean()
            rolling_std = price_data['Close'].rolling(window=20).std()
            indicators['BB_middle'] = rolling_mean.iloc[-1]
            indicators['BB_upper'] = (rolling_mean + (rolling_std * 2)).iloc[-1]
            indicators['BB_lower'] = (rolling_mean - (rolling_std * 2)).iloc[-1]
        except Exception as e:
            logger.warning(f"Could not calculate Bollinger Bands: {e}. Setting to NaN.")
            indicators['BB_middle'] = np.nan
            indicators['BB_upper'] = np.nan
            indicators['BB_lower'] = np.nan

        try:
            # Volume Analysis (e.g., current volume vs average volume)
            avg_volume_30d = price_data['Volume'].rolling(window=30).mean().iloc[-1]
            current_volume = price_data['Volume'].iloc[-1]
            # Check for zero division and NaN before calculating ratio
            if not pd.isna(avg_volume_30d) and avg_volume_30d > 0 and not pd.isna(current_volume):
                 indicators['Volume_ratio'] = current_volume / avg_volume_30d
            elif not pd.isna(current_volume) and current_volume > 0:
                 # If 30d avg is zero or NaN but current volume is positive, assume high relative volume
                 indicators['Volume_ratio'] = 2.0 # Arbitrary high ratio to indicate spike
                 logger.warning("30-day average volume is invalid for volume ratio, assuming spike due to positive current volume.")
            else:
                 indicators['Volume_ratio'] = np.nan
                 logger.warning("Insufficient or invalid volume data for volume ratio calculation.")

        except Exception as e:
            logger.warning(f"Could not calculate Volume Ratio: {e}. Setting to NaN.")
            indicators['Volume_ratio'] = np.nan

        # Include current price in indicators for technical score calculation
        indicators['current_price'] = current_price if not pd.isna(current_price) else np.nan

        logger.info(f"Technical indicators calculated: {indicators}")
        return indicators


    def calculate_technical_score(self, indicators):
        """Calculate a simple technical score based on indicators (out of 50)"""
        score = 0
        # Ensure indicators dictionary is valid
        if not isinstance(indicators, dict):
             logger.warning("Invalid indicators dictionary for technical scoring.")
             return 0

        # Scoring based on common indicator signals
        # Ensure indicator values are valid numbers before scoring
        rsi = indicators.get('RSI', np.nan)
        macd = indicators.get('MACD', np.nan)
        macd_signal = indicators.get('MACD_signal', np.nan)
        sma_20 = indicators.get('SMA_20', np.nan)
        sma_50 = indicators.get('SMA_50', np.nan)
        current_price = indicators.get('current_price', np.nan)
        bb_upper = indicators.get('BB_upper', np.nan)
        bb_lower = indicators.get('BB_lower', np.nan)
        volume_ratio = indicators.get('Volume_ratio', np.nan)


        # RSI Score
        if not pd.isna(rsi):
            if rsi < 30: # Oversold
                score += 10
            elif rsi > 70: # Overbought
                score -= 10
        else:
             logger.warning("RSI is NaN for technical scoring.")

        # MACD Score (Bullish crossover)
        if not pd.isna(macd) and not pd.isna(macd_signal):
            if macd > macd_signal:
                score += 10
        else:
             logger.warning("MACD or MACD signal is NaN for technical scoring.")


        # Moving Average Crossover (Bullish: 20-day > 50-day)
        if not pd.isna(sma_20) and not pd.isna(sma_50):
            if sma_20 > sma_50:
                score += 10
            else:
                score += 5 # Add a small score if 20-day is below but close to 50-day
        else:
             logger.warning("SMA_20 or SMA_50 is NaN for technical scoring.")


        # Bollinger Bands (Price near lower band suggests potential buy)
        if not pd.isna(current_price) and not pd.isna(bb_lower) and not pd.isna(bb_upper):
            if current_price < bb_lower:
                score += 10
            elif current_price > bb_upper:
                score -= 10
            elif bb_lower < current_price < bb_upper: # Price is within the bands
                 score += 5 # Add a small score for price being within bands, indicating less volatility
        else:
             logger.warning("Bollinger Bands or current price is NaN for technical scoring.")


        # Volume Spike (Significant volume increase can confirm trends)
        if not pd.isna(volume_ratio):
            if volume_ratio > 1.5: # Volume is 50% higher than average
                 score += 10
            elif volume_ratio < 0.8: # Volume is significantly lower than average
                 score += 2 # Small score for low volume
            else:
                 score += 5 # Average volume is neutral
        else:
             logger.warning("Volume ratio is NaN for technical scoring.")


        logger.info(f"Basic technical score calculated: {score}")
        return max(0, min(score, 50)) # Cap score between 0 and 50

    # Define a simple class to hold analysis results, similar to a struct
    class AnalysisResult:
        def __init__(self, symbol, info, price_data, current_price, company_name, market_cap,
                     fundamental_score, technical_score, indicators, pe_ratio=np.nan,
                     pb_ratio=np.nan, ev_ebitda=np.nan): # Added for Valuation
            self.symbol = symbol
            self.info = info
            self.price_data = price_data
            self.current_price = current_price
            self.company_name = company_name
            self.market_cap = market_cap
            self.fundamental_score = fundamental_score
            self.technical_score = technical_score
            self.indicators = indicators
            self.pe_ratio = pe_ratio # Added for Valuation
            self.pb_ratio = pb_ratio # Added for Valuation
            self.ev_ebitda = ev_ebitda # Added for Valuation

        def __repr__(self):
            return (f"AnalysisResult(symbol='{self.symbol}', company_name='{self.company_name}', "
                    f"current_price={self.current_price}, market_cap={self.market_cap}, "
                    f"fundamental_score={self.fundamental_score}, technical_score={self.technical_score}, "
                    f"pe_ratio={self.pe_ratio}, pb_ratio={self.pb_ratio}, ev_ebitda={self.ev_ebitda})") # Updated repr
# --- End of IndianStockAnalyzer class definition ---

In [ ]:
# Part 2: EnhancedStockAnalyzer class definition (with Peer Map fix, Volume Spike logging, Peer Valuation Fetching, Sector Average Calculation, and Valuation Comparison - Type Hints Removed as Workaround)

import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import requests
from bs4 import BeautifulSoup
import warnings
warnings.filterwarnings('ignore')

# Import typing for type hints - keep import but don't use in method signature
from typing import List, Dict, Any


# Ensure logging is configured (assuming this was done in a previous cell)
import logging
logger = logging.getLogger(__name__)


# --- EnhancedStockAnalyzer class definition ---
class EnhancedStockAnalyzer:
    """Enhanced analyzer with additional features for Indian markets"""

    def __init__(self):
        """Initialize the enhanced analyzer."""
        self.risk_free_rate = self.get_dynamic_risk_free_rate()
        # Assuming gemini_model is defined globally from a previous cell
        global gemini_model # Access the global gemini_model
        self.gemini_model = gemini_model

        # More detailed circuit limit percentages based on NSE categorization (simplified)
        self.circuit_limits_percentage = {
            'category_a': 0.20, # Typically high-liquidity index stocks
            'category_b': 0.10, # Most other actively traded stocks
            'category_t': 0.05  # Stocks in trade-for-trade segment
        }

        # Simple mapping of some known stocks to categories for demonstration
        self.stock_category_map = {
            'RELIANCE.NS': 'category_a',
            'TCS.NS': 'category_a',
            'HDFCBANK.NS': 'category_a',
            'INFY.NS': 'category_a',
            'ICICIBANK.NS': 'category_a',
            'VEDL.NS': 'category_b',
            'PNB.NS': 'category_b',
            'INDIGO.NS': 'category_b',
            'BANDHANBNK.NS': 'category_b',
            'PERSISTENT.NS': 'category_b',
            'COFORGE.NS': 'category_b',
            'LTTS.NS': 'category_b',
        }


    def get_dynamic_risk_free_rate(self):
        """Fetch current 10-year G-Sec yield"""
        try:
            return 0.072  # 7.2% as of recent data
        except Exception as e:
            logger.warning(f"Could not fetch dynamic risk-free rate: {e}. Using fallback rate.")
            return 0.065

    def get_circuit_limits(self, symbol):
        """Get circuit breaker limits for the stock based on categorization"""
        try:
            ticker = yf.Ticker(symbol)
            info = ticker.info

            if not info or 'previousClose' not in info:
                 logger.warning(f"Could not get previous close for {symbol}. Cannot calculate circuit limits.")
                 return None

            prev_close = info.get('previousClose')
            if pd.isna(prev_close) or prev_close <= 0:
                 logger.warning(f"Invalid previous close price for {symbol}: {prev_close}. Cannot calculate circuit limits.")
                 return None


            stock_category = self.stock_category_map.get(symbol.upper(), 'category_b')
            circuit_percent = self.circuit_limits_percentage.get(stock_category, 0.10)

            logger.info(f"Determined circuit percentage for {symbol} (Category: {stock_category}): {circuit_percent*100}%")

            upper_circuit = prev_close * (1 + circuit_percent)
            lower_circuit = prev_close * (1 - circuit_percent)

            return {
                'upper_circuit': upper_circuit,
                'lower_circuit': lower_circuit,
                'circuit_percent': circuit_percent
            }
        except Exception as e:
            logger.error(f"Error getting circuit breaker limits for {symbol}: {str(e)}")
            return None

    def assess_circuit_risk(self, current_price, circuit_limits):
        """Assess proximity to circuit limits"""
        if pd.isna(current_price) or current_price <= 0:
             logger.warning("Invalid current price for circuit risk assessment.")
             return 0

        if not circuit_limits or 'upper_circuit' not in circuit_limits or 'lower_circuit' not in circuit_limits:
            logger.warning("Invalid or incomplete circuit limits provided.")
            return 0

        upper_circuit = circuit_limits['upper_circuit']
        lower_circuit = circuit_limits['lower_circuit']

        if pd.isna(upper_circuit) or pd.isna(lower_circuit) or upper_circuit <= 0 or lower_circuit <= 0:
             logger.warning("Circuit limits are invalid numbers. Cannot assess circuit risk.")
             return 0

        risk_score = 0

        if current_price >= upper_circuit:
             risk_score -= 30
        elif (upper_circuit - current_price) / current_price < 0.01:
            risk_score -= 20
        elif (upper_circuit - current_price) / current_price < 0.03:
            risk_score -= 10
        elif (upper_circuit - current_price) / current_price < 0.05:
            risk_score -= 5

        if current_price <= lower_circuit:
            risk_score -= 40
        elif (current_price - lower_circuit) / current_price < 0.01:
            risk_score -= 30
        elif (current_price - lower_circuit) / current_price < 0.03:
            risk_score -= 15
        elif (current_price - lower_circuit) / current_price < 0.05:
            risk_score -= 8

        logger.info(f"Circuit risk score calculated: {risk_score}")
        return risk_score

    def get_liquidity_score(self, price_data):
        """Enhanced liquidity analysis with NaN handling and better checks"""
        liquidity_score = 0
        if not isinstance(price_data, pd.DataFrame) or price_data.empty or len(price_data) < 20:
            logger.warning("Insufficient price data for liquidity analysis. Assigning default score.")
            return 30

        try:
            avg_volume_20d = price_data['Volume'].rolling(20).mean().iloc[-1]
            avg_volume_5d = price_data['Volume'].rolling(5).mean().iloc[-1]
            current_close = price_data['Close'].iloc[-1]

            # Add logging for diagnosis - before calculation check
            logger.info(f"Volume data for liquidity: avg_volume_20d: {avg_volume_20d}, avg_volume_5d: {avg_volume_5d}, current_close: {current_close}")


            if pd.isna(avg_volume_20d) or pd.isna(avg_volume_5d) or pd.isna(current_close) or current_close <= 0:
                 logger.warning("Liquidity metrics are NaN or invalid. Cannot calculate liquidity score accurately. Assigning default.")
                 return 30

            avg_turnover_20d = np.nan
            if avg_volume_20d is not None and not pd.isna(avg_volume_20d) and avg_volume_20d >= 0 and current_close > 0:
                 avg_turnover_20d = avg_volume_20d * current_close

            volume_spike = np.nan
            if avg_volume_20d is not None and not pd.isna(avg_volume_20d) and avg_volume_20d > 0:
                 if avg_volume_5d is not None and not pd.isna(avg_volume_5d):
                     volume_spike = avg_volume_5d / avg_volume_20d
                 else:
                      logger.warning("5-day average volume is invalid for volume spike calculation.")
            elif not pd.isna(avg_volume_5d) and avg_volume_5d > 0:
                 volume_spike = 1.0
                 logger.warning("20-day average volume is invalid for volume spike, using 1.0 as relative indicator.")
            else:
                 volume_spike = np.nan # Ensure volume_spike is NaN if both volumes are invalid
                 logger.warning("Insufficient or invalid volume data for volume ratio calculation.")

            # Add logging for volume spike - after calculation
            logger.info(f"Volume spike calculated: {volume_spike}")

            # Corrected logic for scoring based on volume_spike
            if not pd.isna(volume_spike):
                if volume_spike > 1.5: # Volume is 50% higher than average
                    liquidity_score += 10
                elif volume_spike < 0.8: # Volume is significantly lower than average
                    liquidity_score += 2 # Small score for low volume
                else: # volume_spike is between 0.8 and 1.5 (inclusive of 0.8, exclusive of 1.5)
                    liquidity_score += 5 # Average volume is neutral
            else:
                 logger.warning("Volume spike ratio is NaN. Skipping trend scoring.")


            if not pd.isna(avg_turnover_20d):
                if avg_turnover_20d > 500_00_00_000:
                    liquidity_score += 40
                elif avg_turnover_20d > 100_00_00_000:
                    liquidity_score += 30
                elif avg_turnover_20d > 10_00_00_000:
                    liquidity_score += 20
                elif avg_turnover_20d > 1_00_00_000:
                    liquidity_score += 10
                else:
                    liquidity_score += 5
            else:
                 logger.warning("Average turnover is NaN. Skipping turnover scoring.")
                 liquidity_score += 10

            volume_std = price_data['Volume'].rolling(20).std().iloc[-1] if len(price_data) >= 20 else np.nan
            volume_cv = np.nan
            if avg_volume_20d is not None and not pd.isna(avg_volume_20d) and avg_volume_20d > 0 and \
               volume_std is not None and not pd.isna(volume_std):
                volume_cv = volume_std / avg_volume_20d

            if not pd.isna(volume_cv):
                if volume_cv < 0.5:
                    liquidity_score += 20
                elif volume_cv < 1.0:
                    liquidity_score += 10
            else:
                 logger.warning("Volume coefficient of variation is NaN. Skipping consistency scoring.")

            # The volume spike scoring is now handled above within the main if not pd.isna(volume_spike): block


            return min(liquidity_score, 70)

        except Exception as e:
            logger.error(f"Error calculating liquidity score: {str(e)}")
            return 30

    # Updated return type hint to reflect list of dicts - Type hints removed as workaround
    def get_peer_companies(self, symbol, stock_info):
        """Identify peer companies for comparison and fetch their valuation data"""
        try:
            if not isinstance(stock_info, dict):
                 logger.warning("Invalid stock info for peer identification.")
                 return []

            sector = stock_info.get('sector', '')
            market_cap = stock_info.get('marketCap', 0)

            if not isinstance(sector, str):
                 sector = ''
            if not isinstance(market_cap, (int, float)) or pd.isna(market_cap) or market_cap <= 0:
                 logger.warning("Invalid market cap for peer identification.")
                 market_cap = 0

            if market_cap > 10_00_000_00_00_000:
                cap_category = 'Large Cap'
            elif market_cap > 50_000_00_00_000:
                cap_category = 'Mid Cap'
            else:
                cap_category = 'Small Cap'

            peer_map = {
                'Technology': {
                    'Large Cap': ['TCS.NS', 'INFY.NS', 'WIPRO.NS', 'HCLTECH.NS'],
                    'Mid Cap': ['TECHM.NS', 'LTTS.NS', 'PERSISTENT.NS', 'COFORGE.NS'],
                    'Small Cap': ['ZENSAR.NS', 'ECLERX.NS', 'MASTEK.NS']
                },
                'Financial Services': {
                    'Large Cap': ['HDFCBANK.NS', 'ICICIBANK.NS', 'KOTAKBANK.NS', 'AXISBANK.NS'],
                    'Mid Cap': ['IDFCFIRSTB.NS', 'FEDERALBNK.NS', 'INDUSINDBK.NS'],
                    'Small Cap': ['SOUTHBANK.NS', 'UJJIVAN.NS', 'EQUITASBNK.NS']
                },
                'Healthcare': {
                    'Large Cap': ['SUNPHARMA.NS', 'DRREDDY.NS', 'CIPLA.NS', 'DIVISLAB.NS'],
                    'Mid Cap': ['TORNTPHARM.NS', 'ALKEM.NS', 'LUPIN.NS'],
                    'Small Cap': ['CAPLIPOINT.NS', 'BLISS.NS', 'INDOCO.NS']
                },
                # Added Energy sector mapping
                'Energy': {
                    'Large Cap': ['ONGC.NS', 'NTPC.NS', 'POWERGRID.NS'], # Exclude RELIANCE.NS here, filter below
                    'Mid Cap': ['GAIL.NS', 'IOC.NS', 'NHPC.NS'],
                    'Small Cap': ['GUJGASLTD.NS', 'IGL.NS', 'MAHAPOWER.NS']
                }
            }

            peer_symbols = peer_map.get(sector, {}).get(cap_category, [])
            # Ensure the current symbol is excluded from the peer list
            peer_symbols = [p for p in peer_symbols if isinstance(p, str) and p.upper() != symbol.upper()]

            logger.info(f"Identified {len(peer_symbols)} potential peer companies for {symbol} in {sector} ({cap_category} Cap).")

            # Fetch valuation data for peers
            peers_with_data = []
            for peer_symbol in peer_symbols:
                try:
                    peer_ticker = yf.Ticker(peer_symbol)
                    # Add timeout to avoid infinite waiting if a ticker is unresponsive
                    # Note: yfinance fetch can still hang sometimes despite timeout attempts
                    peer_info = peer_ticker.info # yfinance info fetch can sometimes hang
                    if peer_info:
                        peers_with_data.append({
                            'symbol': peer_symbol,
                            'pe_ratio': peer_info.get('trailingPE', np.nan),
                            'pb_ratio': peer_info.get('priceToBook', np.nan),
                            'ev_ebitda': peer_info.get('enterpriseToEbitda', np.nan),
                            'sector': peer_info.get('sector', 'N/A'), # Include sector for potential cross-check
                            'marketCap': peer_info.get('marketCap', np.nan) # Include market cap
                        })
                        logger.info(f"Fetched valuation data for peer {peer_symbol}")
                    else:
                        logger.warning(f"Could not fetch info for peer {peer_symbol}. Skipping.")
                except Exception as e:
                    logger.warning(f"Error fetching data for peer {peer_symbol}: {e}. Skipping.")

            logger.info(f"Fetched data for {len(peers_with_data)} valid peers.")
            return peers_with_data[:5] # Limit to top 5 peers with data


        except Exception as e:
            logger.error(f"Error getting peer companies for {symbol}: {str(e)}")
            return []

    # Type hints removed as workaround
    def calculate_sector_valuation_averages(self, peers_with_data):
        """Calculates average valuation ratios for a list of peers."""
        if not peers_with_data:
            logger.warning("No peer data provided for calculating sector averages.")
            return {'avg_pe': np.nan, 'avg_pb': np.nan, 'avg_ev_ebitda': np.nan}

        pe_ratios = [p.get('pe_ratio') for p in peers_with_data if isinstance(p.get('pe_ratio'), (int, float)) and not pd.isna(p.get('pe_ratio')) and p.get('pe_ratio') > 0]
        pb_ratios = [p.get('pb_ratio') for p in peers_with_data if isinstance(p.get('pb_ratio'), (int, float)) and not pd.isna(p.get('pb_ratio')) and p.get('pb_ratio') > 0]
        ev_ebitda_ratios = [p.get('ev_ebitda') for p in peers_with_data if isinstance(p.get('ev_ebitda'), (int, float)) and not pd.isna(p.get('ev_ebitda')) and p.get('ev_ebitda') > 0]

        avg_pe = np.mean(pe_ratios) if pe_ratios else np.nan
        avg_pb = np.mean(pb_ratios) if pb_ratios else np.nan
        avg_ev_ebitda = np.mean(ev_ebitda_ratios) if ev_ebitda_ratios else np.nan

        logger.info(f"Calculated sector valuation averages: PE={avg_pe:.2f}, PB={avg_pb:.2f}, EV/EBITDA={avg_ev_ebitda:.2f}")

        return {'avg_pe': avg_pe, 'avg_pb': avg_pb, 'avg_ev_ebitda': avg_ev_ebitda}

    # Type hints removed as workaround
    def compare_valuation_to_peers(self, stock_valuation, sector_averages):
        """Compares stock's valuation ratios to sector averages."""
        comparison_results = {
            'pe_comparison': 'N/A',
            'pb_comparison': 'N/A',
            'ev_ebitda_comparison': 'N/A',
            'overall_valuation': 'N/A'
        }

        pe = stock_valuation.get('pe_ratio', np.nan)
        pb = stock_valuation.get('pb_ratio', np.nan)
        ev_ebitda = stock_valuation.get('ev_ebitda', np.nan)

        avg_pe = sector_averages.get('avg_pe', np.nan)
        avg_pb = sector_averages.get('avg_pb', np.nan)
        avg_ev_ebitda = sector_averages.get('avg_ev_ebitda', np.nan)

        # Track how many ratios indicate premium/discount
        premium_indicators = 0
        discount_indicators = 0
        valid_comparisons = 0

        # Compare PE
        if not pd.isna(pe) and not pd.isna(avg_pe) and avg_pe > 0:
            valid_comparisons += 1
            if pe > avg_pe * 1.2: # More than 20% higher
                comparison_results['pe_comparison'] = 'Premium (>20% higher than avg)'
                premium_indicators += 1
            elif pe > avg_pe * 1.05: # More than 5% higher
                 comparison_results['pe_comparison'] = 'Slight Premium (>5% higher than avg)'
                 premium_indicators += 1
            elif pe < avg_pe * 0.8: # More than 20% lower
                comparison_results['pe_comparison'] = 'Discount (>20% lower than avg)'
                discount_indicators += 1
            elif pe < avg_pe * 0.95: # More than 5% lower
                 comparison_results['pe_comparison'] = 'Slight Discount (>5% lower than avg)'
                 discount_indicators += 1
            else:
                comparison_results['pe_comparison'] = 'In line with avg'
        else:
             logger.warning("PE comparison not possible due to missing data.")

        # Compare PB
        if not pd.isna(pb) and not pd.isna(avg_pb) and avg_pb > 0:
            valid_comparisons += 1
            if pb > avg_pb * 1.2:
                comparison_results['pb_comparison'] = 'Premium (>20% higher than avg)'
                premium_indicators += 1
            elif pb > avg_pb * 1.05:
                 comparison_results['pb_comparison'] = 'Slight Premium (>5% higher than avg)'
                 premium_indicators += 1
            elif pb < avg_pb * 0.8:
                comparison_results['pb_comparison'] = 'Discount (>20% lower than avg)'
                discount_indicators += 1
            elif pb < avg_pb * 0.95:
                 comparison_results['pb_comparison'] = 'Slight Discount (>5% lower than avg)'
                 discount_indicators += 1
            else:
                comparison_results['pb_comparison'] = 'In line with avg'
        else:
             logger.warning("PB comparison not possible due to missing data.")

        # Compare EV/EBITDA
        if not pd.isna(ev_ebitda) and not pd.isna(avg_ev_ebitda) and avg_ev_ebitda > 0:
            valid_comparisons += 1
            if ev_ebitda > avg_ev_ebitda * 1.2:
                comparison_results['ev_ebitda_comparison'] = 'Premium (>20% higher than avg)'
                premium_indicators += 1
            elif ev_ebitda > avg_ev_ebitda * 1.05:
                 comparison_results['ev_ebitda_comparison'] = 'Slight Premium (>5% higher than avg)'
                 premium_indicators += 1
            elif ev_ebitda < avg_ev_ebitda * 0.8:
                comparison_results['ev_ebitda_comparison'] = 'Discount (>20% lower than avg)'
                discount_indicators += 1
            elif ev_ebitda < avg_ev_ebitda * 0.95:
                 comparison_results['ev_ebitda_comparison'] = 'Slight Discount (>5% lower than avg)'
                 discount_indicators += 1
            else:
                comparison_results['ev_ebitda_comparison'] = 'In line with avg'
        else:
             logger.warning("EV/EBITDA comparison not possible due to missing data.")


        # Simple overall valuation assessment
        if valid_comparisons == 0:
            comparison_results['overall_valuation'] = 'Valuation comparison not possible (No valid data)'
        elif premium_indicators > discount_indicators:
            comparison_results['overall_valuation'] = 'Potentially Overvalued vs. Peers'
        elif discount_indicators > premium_indicators:
            comparison_results['overall_valuation'] = 'Potentially Undervalued vs. Peers'
        else:
            comparison_results['overall_valuation'] = 'Fairly valued vs. Peers'


        logger.info(f"Valuation comparison results: {comparison_results}")
        return comparison_results


    def calculate_peer_relative_performance(self, symbol, peers_with_data: List[Dict[str, Any]], days=30):
        """Compare stock performance with peers, handling data issues robustly"""
        results = {
            'stock_return': np.nan,
            'avg_peer_return': np.nan,
            'relative_performance': np.nan,
            'outperformance': False,
            'days': days
        }

        # Use the symbols from the fetched peer data for performance comparison
        peers_for_performance = [p['symbol'] for p in peers_with_data if 'symbol' in p and isinstance(p['symbol'], str)]

        if not peers_for_performance:
             logger.warning("No valid peer symbols with data provided for performance comparison.")
             # Return the default results with NaNs if no peers
             return results

        # --- Removed the temporary skip here ---

        try:
            main_ticker = yf.Ticker(symbol)
            main_hist = main_ticker.history(period=f"{days}d")
            main_return = np.nan

            if not main_hist.empty and len(main_hist) > 1:
                try:
                    start_price = main_hist['Close'].iloc[0]
                    end_price = main_hist['Close'].iloc[-1]
                    if not pd.isna(start_price) and not pd.isna(end_price) and start_price > 0:
                         main_return = ((end_price / start_price) - 1) * 100
                    else:
                         logger.warning(f"Invalid start ({start_price}) or end price ({end_price}) for main stock ({symbol}) return calculation.")
                except Exception as ret_e:
                     logger.warning(f"Could not calculate main stock ({symbol}) return: {ret_e}. Setting to NaN.")
                     main_return = np.nan
            else:
                logger.warning(f"Insufficient historical data for main stock ({symbol}) for return calculation.")

            results['stock_return'] = main_return

        except Exception as main_stock_e:
            logger.error(f"Error fetching data or calculating return for main stock ({symbol}): {main_stock_e}. Main stock return set to NaN.")
            results['stock_return'] = np.nan

        # Calculate peer returns
        peer_returns = []
        for peer_symbol in peers_for_performance: # Use the filtered list of symbols
             try:
                 peer_ticker = yf.Ticker(peer_symbol)
                 peer_hist = peer_ticker.history(period=f"{days}d")
                 if not peer_hist.empty and len(peer_hist) > 1:
                     try:
                          peer_start_price = peer_hist['Close'].iloc[0]
                          peer_end_price = peer_hist['Close'].iloc[-1]
                          if not pd.isna(peer_start_price) and not pd.isna(peer_end_price) and peer_start_price > 0:
                             peer_return = ((peer_end_price / peer_start_price) - 1) * 100
                             peer_returns.append(peer_return)
                          else:
                             logger.warning(f"Invalid start or end price for peer ({peer_symbol}) return calculation.")
                     except Exception as peer_ret_e:
                         logger.warning(f"Could not calculate return for peer ({peer_symbol}): {peer_ret_e}. Skipping.")
                 else:
                     logger.warning(f"Insufficient historical data for peer ({peer_symbol}) for return calculation. Skipping.")
             except Exception as peer_e:
                 logger.warning(f"Error fetching data for peer ({peer_symbol}): {peer_e}. Skipping.")

        # Add logging for peer returns before filtering/average calculation
        logger.info(f"Peer returns collected before average calculation attempt: {peer_returns}")

        # Calculate average peer return and relative performance
        if peer_returns:
             # Ensure peer_returns list contains only valid numbers before calculating mean
             valid_peer_returns = [ret for ret in peer_returns if isinstance(ret, (int, float)) and not pd.isna(ret)]
             if valid_peer_returns:
                 avg_peer_return = np.mean(valid_peer_returns)
                 results['avg_peer_return'] = avg_peer_return

                 # Calculate relative performance only if main stock return is valid
                 if not pd.isna(main_return):
                      results['relative_performance'] = main_return - avg_peer_return

                      # Determine outperformance
                      if main_return > avg_peer_return:
                          results['outperformance'] = True
                 else:
                      logger.warning("Main stock return is NaN, cannot calculate relative performance or outperformance.")
             else:
                 logger.warning("No valid peer returns available after filtering. Cannot calculate average peer return or relative performance.")
        else:
             logger.warning("No peer returns calculated. Cannot calculate average peer return or relative performance.")


        logger.info(f"Peer performance results for {symbol}: {results}")
        return results


    def get_index_membership(self, symbol):
        """Check index membership for the stock"""
        symbol_upper = symbol.upper() if isinstance(symbol, str) else ''

        index_constituents = {
            'NIFTY50': ['RELIANCE.NS', 'TCS.NS', 'HDFCBANK.NS', 'INFY.NS', 'ICICIBANK.NS'],
            'NIFTY_NEXT50': ['VEDL.NS', 'PNB.NS', 'INDIGO.NS', 'BANDHANBNK.NS'],
            'NIFTY_MIDCAP150': ['PERSISTENT.NS', 'COFORGE.NS', 'LTTS.NS']
        }

        membership = []
        for index, constituents in index_constituents.items():
            if symbol_upper and any(isinstance(c, str) and c.upper() == symbol_upper for c in constituents):
                membership.append(index)

        if membership:
             logger.info(f"Stock {symbol} is a member of indices: {', '.join(membership)}")
        else:
             logger.info(f"Stock {symbol} is not found in the hardcoded index constituents.")

        return membership

    def calculate_enhanced_technical_score(self, indicators, circuit_risk):
        """Calculate enhanced technical analysis score with NaN handling and circuit risk"""
        base_score = 0
        indicators_calculated = 0
        possible_indicator_points = 50

        num_key_indicators = 5
        indicator_point_contribution = possible_indicator_points / num_key_indicators

        rsi = indicators.get('RSI', np.nan)
        if not pd.isna(rsi):
             indicators_calculated += 1
             if 40 <= rsi <= 60:
                 base_score += indicator_point_contribution * 0.7
             elif rsi < 40:
                 base_score += indicator_point_contribution * 1.0
             else:
                 base_score += indicator_point_contribution * 0.5
        else:
             logger.warning("RSI not calculated or is NaN.")

        macd = indicators.get('MACD', np.nan)
        macd_signal = indicators.get('MACD_signal', np.nan)

        if not pd.isna(macd) and not pd.isna(macd_signal):
             indicators_calculated += 1
             if macd > macd_signal:
                 base_score += indicator_point_contribution * 1.0
             else:
                 base_score += indicator_point_contribution * 0.5
        else:
             logger.warning("MACD or MACD signal is NaN for technical scoring.")

        sma_20 = indicators.get('SMA_20', np.nan)
        sma_50 = indicators.get('SMA_50', np.nan)
        if not pd.isna(sma_20) and not pd.isna(sma_50):
            indicators_calculated += 1
            if sma_20 > sma_50:
                base_score += indicator_point_contribution * 1.0
            else:
                base_score += indicator_point_contribution * 0.5
        else:
             logger.warning("SMA_20 or SMA_50 is NaN for technical scoring.")

        current_price = indicators.get('current_price', np.nan)
        bb_upper = indicators.get('BB_upper', np.nan)
        bb_lower = indicators.get('BB_lower', np.nan)
        bb_middle = indicators.get('BB_middle', np.nan)

        if not pd.isna(current_price) and not pd.isna(bb_upper) and not pd.isna(bb_lower) and not pd.isna(bb_middle):
            indicators_calculated += 1
            if not pd.isna(bb_lower) and not pd.isna(bb_middle) and not pd.isna(current_price) and bb_lower < current_price < bb_middle:
                base_score += indicator_point_contribution * 1.0
            elif not pd.isna(bb_middle) and not pd.isna(bb_upper) and not pd.isna(current_price) and bb_middle < current_price < bb_upper:
                base_score += indicator_point_contribution * 0.7
            else:
                base_score += indicator_point_contribution * 0.5
        else:
             logger.warning("Bollinger Bands or current price is NaN for technical scoring.")

        volume_ratio = indicators.get('Volume_ratio', np.nan)
        if not pd.isna(volume_ratio):
             indicators_calculated += 1
             if volume_ratio > 1.5: # Volume is 50% higher than average
                 base_score += indicator_point_contribution * 1.0
             elif volume_ratio < 0.8: # Volume is significantly lower than average
                 base_score += indicator_point_contribution * 0.2 # Small score for low volume
             else: # Average volume is neutral
                 base_score += indicator_point_contribution * 0.5
        else:
             logger.warning("Volume ratio is NaN for technical scoring.")


        achieved_score_from_indicators = base_score

        if indicators_calculated > 0:
            max_possible_contribution = num_key_indicators * indicator_point_contribution
            achieved_percentage = achieved_score_from_indicators / max_possible_contribution if max_possible_contribution > 0 else 0
            adjusted_base_score = achieved_percentage * possible_indicator_points
            if indicators_calculated < num_key_indicators:
                 logger.warning(f"Only {indicators_calculated}/{num_key_indicators} key technical indicators calculated. Adjusting base score from {achieved_score_from_indicators:.2f} (raw) to {adjusted_base_score:.2f}.")
            base_score = adjusted_base_score
        else:
             logger.warning("No key technical indicators calculated. Base technical score is 0.")
             base_score = 0

        circuit_risk_adj = circuit_risk if isinstance(circuit_risk, (int, float)) and not pd.isna(circuit_risk) else 0

        final_score = base_score + circuit_risk_adj

        return max(0, min(final_score, 50))


    def fetch_news(self, symbol: str) -> List[str]:
        """
        Fetches recent news headlines for the given stock symbol.
        NOTE: This is a placeholder. In a real application, you would integrate
        with a News API (like News API, Alpha Vantage, etc.) here.
        """
        print(f"Fetching news for {symbol} (placeholder)...")
        symbol_str = str(symbol) if symbol is not None else "Unknown Stock"
        return [
            f"{symbol_str} stock price rises on positive market sentiment",
            f"{symbol_str} announces strong quarterly results, beating estimates",
            f"Experts bullish on {symbol_str}'s future outlook",
            f"I am a non-numeric entry that might cause an error",
            f"{symbol_str} faces regulatory challenges in key market",
            f"Competitor's new product launch impacts {symbol_str}'s market share",
            f"Global market trends positively impacting {symbol_str}",
            f"{symbol_str} management provides optimistic guidance",
            f"Increased foreign investment flows into {symbol_str}",
            f"Regulatory approval received for {symbol_str}'s new product",
            f"Production issues reported for {symbol_str}",
            f"Increased competition puts pressure on {symbol_str}'s margins",
            f"Analyst downgrades rating for {symbol_str}",
            f"Supply chain disruptions affect {symbol_str}'s operations",
            f"Geopolitical tensions create uncertainty for {symbol_str}",
            f"{symbol_str} announces stock split",
            f"Dividend declared by {symbol_str}"
        ]


    def analyze_sentiment(self, news_headlines: List[str]) -> Dict):
        """
        Analyzes the sentiment of news headlines using the Gemini API.
        Returns a dictionary with sentiment counts (positive, neutral, negative)
        and a sentiment score.
        """
        if not self.gemini_model:
            print("Gemini API not configured. Skipping sentiment analysis.")
            return {'positive': 0, 'neutral': 0, 'negative': 0, 'score': 0, 'summary': 'Gemini API not configured. Sentiment analysis skipped.'}

        if not isinstance(news_headlines, list) or not news_headlines:
            print("No news headlines to analyze. Skipping sentiment analysis.")
            return {'positive': 0, 'neutral': 0, 'negative': 0, 'score': 12.5, 'summary': 'No news headlines. Sentiment analysis skipped.'}

        logger.info("Analyzing sentiment using Gemini API...")
        prompt = "Analyze the sentiment of the following news headlines for a stock and categorize each as Positive, Neutral, or Negative. Provide a concise summary of the overall sentiment.\n\n"
        for headline in news_headlines:
            if isinstance(headline, str):
                 prompt += f"- {headline}\n"
            else:
                 logger.warning(f"Skipping non-string headline: {headline}")
                 continue

        if len(prompt) > 30000:
             logger.warning("News headlines too long for Gemini API prompt. Truncating.")
             prompt = prompt[:30000] + "\n... (headlines truncated)"

        try:
            response = self.gemini_model.generate_content(prompt)
            sentiment_text = response.text.strip()
            logger.info(f"Gemini API Response: {sentiment_text[:200]}...")

            positive_count = sentiment_text.lower().count('positive')
            neutral_count = sentiment_text.lower().count('neutral')
            negative_count = sentiment_text.lower().count('negative')

            summary_line = "No summary extracted."
            lines = sentiment_text.split('\n')
            for line in lines:
                 lower_line = line.lower()
                 if 'summary' in lower_line or 'overall sentiment' in lower_line:
                     summary_line = line.strip()
                     break
            if summary_line == "No summary extracted." and lines:
                 for line in reversed(lines):
                     if line.strip():
                         summary_line = line.strip()
                         break

            total_headlines = len([h for h in news_headlines if isinstance(h, str)])
            if total_headlines > 0:
                sentiment_score = ((positive_count - negative_count) / total_headlines) * 12.5 + 12.5
            else:
                sentiment_score = 12.5

            return {
                'positive': positive_count,
                'neutral': neutral_count,
                'negative': negative_count,
                'score': max(0, min(sentiment_score, 25)),
                'summary': summary_line
            }

        except Exception as e:
            logger.error(f"Error analyzing sentiment with Gemini API: {e}. Skipping sentiment analysis.")
            return {'positive': 0, 'neutral': 0, 'negative': 0, 'score': 0, 'summary': 'Sentiment analysis failed.'}


    def generate_enhanced_recommendation(self, fundamental_score, technical_score,
                                       liquidity_score, peer_performance, index_membership,
                                       sentiment_analysis_results):
        """Generate comprehensive recommendation including sentiment"""
        fund_score = fundamental_score if isinstance(fundamental_score, (int, float)) and not pd.isna(fundamental_score) else 0
        tech_score = technical_score if isinstance(technical_score, (int, float)) and not pd.isna(technical_score) else 0
        liq_score = liquidity_score if isinstance(liquidity_score, (int, float)) and not pd.isna(liquidity_score) else 0
        sentiment_score = sentiment_analysis_results.get('score', 0) if isinstance(sentiment_analysis_results.get('score'), (int, float)) and not pd.isna(sentiment_analysis_results.get('score')) else 0

        total_score = fund_score + tech_score + liq_score + sentiment_score

        peer_analysis_available = isinstance(peer_performance, dict) and \
                                  not (pd.isna(peer_performance.get('stock_return')) and \
                                       pd.isna(peer_performance.get('avg_peer_return')) and \
                                       pd.isna(peer_performance.get('relative_performance')))

        if peer_analysis_available and peer_performance.get('outperformance', False) is True:
            total_score += 5
            logger.info("Adding bonus for peer outperformance.")
        elif not peer_analysis_available:
             total_score -= 5
             logger.warning("Applying small penalty as peer performance data is incomplete/unavailable.")

        if isinstance(index_membership, list) and index_membership:
            total_score += 5
            logger.info("Adding bonus for index membership.")

        recommendation = "NEUTRAL"
        confidence = "Low"

        if total_score >= 170:
            recommendation = "STRONG BUY"
            confidence = "High"
        elif total_score >= 140:
            recommendation = "BUY"
            confidence = "Moderate to High"
        elif total_score >= 100:
            recommendation = "HOLD"
            confidence = "Moderate"
        elif total_score >= 70:
            recommendation = "SELL"
            confidence = "Moderate"
        else:
            recommendation = "STRONG SELL"
            confidence = "High"

        logger.info(f"Generated enhanced recommendation: {recommendation} (Total Score: {total_score:.2f})")

        return {
            'recommendation': recommendation,
            'confidence': confidence,
            'total_score': total_score,
            'breakdown': {
                'fundamental': fund_score,
                'technical': tech_score,
                'liquidity': liq_score,
                'sentiment': sentiment_score,
                'peer_adjustment': 5 if peer_analysis_available and peer_performance.get('outperformance', False) is True else (-5 if not peer_analysis_available else 0),
                'index_adjustment': 5 if isinstance(index_membership, list) and index_membership else 0
            }
        }

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 732)

In [ ]:
#@title Part 3: Example Usage and Report Display

# Assuming IndianStockAnalyzer and EnhancedStockAnalyzer classes are defined in previous cells

try:
    # Check if classes are defined (from previous cells)
    IndianStockAnalyzer
    EnhancedStockAnalyzer
except NameError:
    print("Make sure to run the previous cells defining IndianStockAnalyzer and EnhancedStockAnalyzer classes.")
    # You might want to exit or handle this case differently in a full script
    exit() # Exit if classes are not defined


stock_analyzer = IndianStockAnalyzer()
enhanced_analyzer = EnhancedStockAnalyzer()

# Analyze a stock
symbol = "RELIANCE.NS" # Example symbol
result = analyze_stock_enhanced(symbol, stock_analyzer, enhanced_analyzer)

if result:
    # Display the enhanced report
    enhanced = result['enhanced_features']
    final = result['final_recommendation']
    basic = result['basic_analysis'] # Get basic data for display

    print(f"\n{'='*60}")
    print(f"Enhanced Analysis Report: {basic.company_name} ({basic.symbol})")
    print(f"{'='*60}")

    print(f"\n📊 Current Market Data:")
    print(f"Current Price: ₹{basic.current_price:,.2f}" if not pd.isna(basic.current_price) else "Current Price: N/A")
    market_cap_cr = basic.market_cap / 10000000 if not pd.isna(basic.market_cap) else np.nan
    print(f"Market Cap: ₹{market_cap_cr:,.2f} Cr" if not pd.isna(market_cap_cr) else "Market Cap: N/A")
    print(f"Sector: {basic.info.get('sector', 'N/A')}")

    print(f"\n⚙️ Enhanced Technical Analysis & Risk:")
    # Display enhanced technical indicators and risk
    # Assuming you want to display relevant indicators from basic.indicators and the circuit risk score
    circuit_limits_display = enhanced.get('circuit_limits')
    if circuit_limits_display and not pd.isna(circuit_limits_display.get('lower_circuit')) and not pd.isna(circuit_limits_display.get('upper_circuit')):
             print(f"Circuit Limits: {circuit_limits_display.get('lower_circuit', 'N/A'):.2f} - {circuit_limits_display.get('upper_circuit', 'N/A'):.2f}")
    else:
             print("Circuit Limits: N/A")

    print(f"Circuit Risk Score: {enhanced.get('circuit_risk_score', 'N/A')}")
    # Check if liquidity score is available before formatting
    liquidity_score_display = f"{enhanced.get('liquidity_score', 'N/A')}/70" if isinstance(enhanced.get('liquidity_score'), (int, float)) and not pd.isna(enhanced.get('liquidity_score')) else "Liquidity Score: N/A/70"
    print(liquidity_score_display)


    # Display some key technical indicators from basic analysis
    print("\nKey Technical Indicators (from Basic Analysis):")
    key_tech_indicators = ['RSI', 'MACD', 'MACD_signal', 'SMA_20', 'SMA_50', 'BB_middle', 'Volume_ratio']
    for indicator_name in key_tech_indicators:
        value = basic.indicators.get(indicator_name, np.nan)
        if not pd.isna(value):
            if isinstance(value, float):
                 print(f"  - {indicator_name}: {value:,.2f}")
            else:
                 print(f"  - {indicator_name}: {value}")
            # Add logging for these key technical indicators if they are calculated
            logger.info(f"Indicator {indicator_name}: {value}")
        else:
            print(f"  - {indicator_name}: N/A")
            logger.info(f"Indicator {indicator_name}: N/A")


    print(f"\n🤝 Peer Analysis:")
    # Check if peer analysis was skipped or failed by examining the presence of meaningful data
    peer_perf = enhanced.get('peer_performance')
    # Updated check to see if peer_perf is a dictionary and has the expected keys with non-NaN values
    if isinstance(peer_perf, dict) and \
       not pd.isna(peer_perf.get('stock_return')) and \
       not pd.isna(peer_perf.get('avg_peer_return')) and \
       not pd.isna(peer_perf.get('relative_performance')):

         print(f"Peer Companies: {', '.join(enhanced['peers'])}")
         # Add checks for NaN before formatting float
         stock_return_display = f"{peer_perf.get('stock_return', 'N/A'):.2f}%" if isinstance(peer_perf.get('stock_return'), (int, float)) and not pd.isna(peer_perf.get('stock_return')) else 'N/A'
         avg_peer_return_display = f"{peer_perf.get('avg_peer_return', 'N/A'):.2f}%" if isinstance(peer_perf.get('avg_peer_return'), (int, float)) and not pd.isna(peer_perf.get('avg_peer_return')) else 'N/A'
         relative_performance_display = f"{peer_perf.get('relative_performance', 'N/A'):.2f}%" if isinstance(peer_perf.get('relative_performance'), (int, float)) and not pd.isna(peer_perf.get('relative_performance')) else 'N/A'

         print(f"  - {peer_perf.get('days', 'N/A')}-day Stock Return: {stock_return_display}")
         print(f"  - Avg Peer Return: {avg_peer_return_display}")
         print(f"  - Relative Performance: {relative_performance_display}")
    else:
         # Display a message indicating peer analysis was skipped/failed or data is incomplete
         print("Peer analysis skipped or failed or data unavailable.")


    if enhanced.get('index_membership'):
        print(f"\n🏛️ Index Membership: {', '.join(enhanced['index_membership'])}")

    # Sentiment Analysis Results
    sentiment_results = enhanced.get('sentiment_analysis', {})
    print(f"\n📰 News Sentiment Analysis:")
    if enhanced.get('news_headlines'):
         # Only print a few headlines to keep the report concise
         print("Sample News Headlines:")
         # Ensure headlines are valid strings before printing
         valid_headlines = [h for h in enhanced['news_headlines'] if isinstance(h, str)]
         for headline in valid_headlines[:5]: # Print up to 5 headlines
             print(f"- {headline}")
         if len(valid_headlines) > 5:
              print("  ...")

         print(f"Sentiment Counts: Positive={sentiment_results.get('positive', 0)}, Neutral={sentiment_results.get('neutral', 0)}, Negative={sentiment_results.get('negative', 0)}")
         sentiment_score_display = f"{sentiment_results.get('score', 0):.2f}" if isinstance(sentiment_results.get('score'), (int, float)) and not pd.isna(sentiment_results.get('score')) else "N/A"
         print(f"Sentiment Score: {sentiment_score_display}/25")
         print(f"Sentiment Summary: {sentiment_results.get('summary', 'N/A')}")
    else:
         print("No news headlines fetched or sentiment analysis skipped.")


    print(f"\n--- Comprehensive Scores and Recommendation ---")
    print(f"Fundamental Score (from Part 1): {basic.fundamental_score:.2f}/50" if not pd.isna(basic.fundamental_score) else "Fundamental Score: N/A")
    # Add checks for NaN before formatting float for breakdown scores
    enhanced_tech_score_display = f"{final.get('breakdown', {}).get('technical', 'N/A'):.2f}" if isinstance(final.get('breakdown', {}).get('technical'), (int, float)) and not pd.isna(final.get('breakdown', {}).get('technical')) else "N/A"
    liquidity_score_display_breakdown = f"{final.get('breakdown', {}).get('liquidity', 'N/A'):.2f}" if isinstance(final.get('breakdown', {}).get('liquidity'), (int, float)) and not pd.isna(final.get('breakdown', {}).get('liquidity')) else "N/A"
    sentiment_score_display_breakdown = f"{final.get('breakdown', {}).get('sentiment', 'N/A'):.2f}" if isinstance(final.get('breakdown', {}).get('sentiment'), (int, float)) and not pd.isna(sentiment_results.get('score')) else "N/A" # Corrected sentiment score breakdown check

    print(f"Technical Score (Enhanced): {enhanced_tech_score_display}/50")
    print(f"Liquidity Score: {liquidity_score_display_breakdown}/70")
    print(f"Sentiment Score: {sentiment_score_display_breakdown}/25")

    # Display adjustment scores
    peer_adj_display = f"{final.get('breakdown', {}).get('peer_adjustment', 'N/A'):+.2f}" if isinstance(final.get('breakdown', {}).get('peer_adjustment'), (int, float)) and not pd.isna(final.get('breakdown', {}).get('peer_adjustment')) else "N/A"
    index_adj_display = f"{final.get('breakdown', {}).get('index_adjustment', 'N/A'):+.2f}" if isinstance(final.get('breakdown', {}).get('index_adjustment'), (int, float)) and not pd.isna(final.get('breakdown', {}).get('index_adjustment')) else "N/A"

    print(f"Peer Adjustment: {peer_adj_display}")
    print(f"Index Membership Adjustment: {index_adj_display}")


    print(f"\nTotal Enhanced Score: {final.get('total_score', 0):.2f}/205" if isinstance(final.get('total_score'), (int, float)) and not pd.isna(final.get('total_score')) else "Total Enhanced Score: N/A")
    print(f"RECOMMENDATION: {final.get('recommendation', 'N/A')} (Confidence: {final.get('confidence', 'N/A')})")


    print(f"\n{'='*60}\n")

# Task
Enhance the stock analysis script by adding valuation analysis (P/E, P/B, EV/EBITDA vs. sector), downside risk assessment (stress testing/VaR), enhanced dividend yield calculation, and consideration of global macro impacts. Integrate these new analyses into the enhanced analysis report.

## Plan for valuation analysis

### Subtask:
Outline steps to calculate and compare P/E, P/B, and EV/EBITDA ratios against sector averages.


# Task
Implement valuation analysis for the stock analyzer script, including fetching P/E, P/B, and EV/EBITDA ratios, comparing them to sector peer averages, and integrating the assessment into the final report.

## Fetch valuation data

### Subtask:
Modify the data fetching process to ensure P/E, P/B, and EV/EBITDA ratios are reliably retrieved using yfinance or other potential sources if needed.


**Reasoning**:
The subtask requires modifying the `IndianStockAnalyzer` class to reliably fetch P/E, P/B, and EV/EBITDA ratios from the fetched stock information and store them in the `metrics` dictionary, handling missing data and adding logging. This involves updating the `analyze_stock` method and the `AnalysisResult` dataclass definition within the `IndianStockAnalyzer` class.



In [ ]:
#@title Part 1: IndianStockAnalyzer class definition (Enhanced Valuation Metrics)

import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import requests
from bs4 import BeautifulSoup
import warnings
warnings.filterwarnings('ignore')

# Import typing for type hints
from typing import List, Dict, Optional
from dataclasses import dataclass # Import dataclass

# Ensure logging is configured (assuming this was done in a previous cell)
import logging
logger = logging.getLogger(__name__)

# --- IndianStockAnalyzer class definition ---
@dataclass
class AnalysisResult: # Define AnalysisResult dataclass here
    """Data class to store comprehensive stock information"""
    symbol: str
    company_name: str
    current_price: float
    market_cap: float
    info: Dict
    price_data: pd.DataFrame
    indicators: Dict
    fundamental_score: float
    technical_score: float
    recommendation: str # Added recommendation field
    metrics: Dict # Ensure metrics dictionary is included

class IndianStockAnalyzer:
    """Basic analyzer for Indian stocks"""

    def __init__(self):
        """Initialize the basic analyzer."""
        # Basic setup if needed for Part 1 (e.g., API keys, configurations)
        pass

    def analyze_stock(self, symbol: str) -> Optional[AnalysisResult]: # Added type hint for return
        """
        Performs basic fundamental and technical analysis for a given stock.
        Returns an AnalysisResult object containing the analysis results.
        """
        logger.info(f"Starting basic analysis for {symbol}")
        try:
            ticker = yf.Ticker(symbol)
            info = ticker.info

            if not info:
                 logger.warning(f"Could not fetch info for {symbol}. Analysis aborted.")
                 # Return a minimal AnalysisResult with N/A values
                 return AnalysisResult(
                     symbol=symbol,
                     company_name=symbol,
                     current_price=np.nan,
                     market_cap=np.nan,
                     info={},
                     price_data=pd.DataFrame(),
                     indicators={},
                     fundamental_score=np.nan,
                     technical_score=np.nan,
                     recommendation="N/A", # Added N/A recommendation
                     metrics={}
                 )


            # Get price data for technical analysis
            price_data = ticker.history(period="1y") # Get 1 year of historical data

            if price_data.empty:
                logger.warning(f"Could not fetch price data for {symbol}.")
                # Return info even if price data is missing, as fundamental data might still be available
                # We will handle missing price data in enhanced calculations
                # Ensure all fields are populated with N/A or empty structures
                return AnalysisResult(
                    symbol=symbol,
                    company_name=info.get('longName', symbol),
                    current_price=info.get('currentPrice', np.nan),
                    market_cap=info.get('marketCap', np.nan),
                    info=info,
                    price_data=pd.DataFrame(), # Empty DataFrame
                    indicators={},
                    fundamental_score=np.nan,
                    technical_score=np.nan,
                    recommendation="N/A", # Added N/A recommendation
                    metrics={}
                )


            # 1. Fundamental Analysis (Simplified)
            # Extract key fundamental data points
            metrics = {} # Initialize metrics dictionary

            try:
                metrics['marketCap'] = info.get('marketCap', np.nan)
                metrics['currentPrice'] = info.get('currentPrice', np.nan)
                metrics['sector'] = info.get('sector', 'N/A')
                metrics['industry'] = info.get('industry', 'N/A')
                company_name = info.get('longName', symbol)

                # Extract and store Valuation Ratios
                metrics['trailingPE'] = info.get('trailingPE', np.nan)
                logger.info(f"Fetched trailingPE for {symbol}: {metrics['trailingPE']}")

                metrics['priceToBook'] = info.get('priceToBook', np.nan)
                logger.info(f"Fetched priceToBook for {symbol}: {metrics['priceToBook']}")

                enterprise_value = info.get('enterpriseValue', np.nan)
                ebitda = info.get('ebitda', np.nan)
                metrics['enterpriseValue'] = enterprise_value # Store raw value for reference
                metrics['ebitda'] = ebitda # Store raw value for reference

                # Calculate EV/EBITDA, handling potential division by zero or missing data
                metrics['evToEbitda'] = np.nan # Default to NaN
                if isinstance(enterprise_value, (int, float)) and not np.isnan(enterprise_value) and \
                   isinstance(ebitda, (int, float)) and not np.isnan(ebitda) and ebitda != 0:
                    metrics['evToEbitda'] = enterprise_value / ebitda
                    logger.info(f"Calculated evToEbitda for {symbol}: {metrics['evToEbitda']:.2f}")
                elif (isinstance(enterprise_value, (int, float)) and not np.isnan(enterprise_value)) and \
                     (isinstance(ebitda, (int, float)) and not np.isnan(ebitda) and ebitda == 0):
                     logger.warning(f"EBITDA is zero for {symbol}. Cannot calculate EV/EBITDA.")
                else:
                    logger.warning(f"Missing Enterprise Value ({enterprise_value}) or EBITDA ({ebitda}) for {symbol}. Cannot calculate EV/EBITDA.")


                # Dividend Yield
                dividend_yield = info.get('dividendYield', np.nan) # This is often already a percentage as a float
                # Ensure dividend_yield is not None before checking if it's NaN
                if dividend_yield is not None and isinstance(dividend_yield, (int, float)) and not pd.isna(dividend_yield) and dividend_yield >= 0:
                     # If dividend_yield is a float between 0 and 1, assume it's a ratio and convert to percentage
                     if 0 < dividend_yield <= 1:
                         metrics['dividendYield'] = dividend_yield * 100
                         logger.info(f"Converted dividend yield for {symbol} from ratio to percentage: {metrics['dividendYield']:.2f}%")
                     else:
                         metrics['dividendYield'] = dividend_yield # Assume it's already a percentage or other form
                         logger.info(f"Dividend yield for {symbol} is {metrics['dividendYield']:.2f}%. Assuming it's already in percentage or other form.")
                else:
                     metrics['dividendYield'] = np.nan # Ensure it's NaN if None or invalid
                     logger.warning(f"Dividend yield for {symbol} is unavailable or zero ({dividend_yield}). Setting to NaN.")


                # Other fundamental metrics that might be useful later (Optional for this subtask but good practice)
                metrics['returnOnEquity'] = info.get('returnOnEquity', np.nan)
                metrics['revenueGrowth'] = info.get('revenueGrowth', np.nan)
                metrics['profitMargins'] = info.get('profitMargins', np.nan)


            except Exception as fund_e:
                 logger.error(f"Error extracting fundamental data for {symbol}: {fund_e}. Setting fundamental metrics to NaN.")
                 # Ensure metrics are still initialized to handle the rest of the process
                 metrics.setdefault('marketCap', np.nan)
                 metrics.setdefault('currentPrice', np.nan)
                 metrics.setdefault('sector', 'N/A')
                 metrics.setdefault('industry', 'N/A')
                 metrics.setdefault('trailingPE', np.nan)
                 metrics.setdefault('priceToBook', np.nan)
                 metrics.setdefault('evToEbitda', np.nan) # Ensure this is set even on error
                 metrics.setdefault('dividendYield', np.nan)
                 metrics.setdefault('returnOnEquity', np.nan)
                 metrics.setdefault('revenueGrowth', np.nan)
                 metrics.setdefault('profitMargins', np.nan)
                 company_name = info.get('longName', symbol) # Still try to get company name


            # Calculate a simple fundamental score (out of 50) - This scoring logic remains the same
            fundamental_score = self.calculate_fundamental_score(
                 metrics.get('marketCap', np.nan),
                 metrics.get('trailingPE', np.nan),
                 metrics.get('priceToBook', np.nan),
                 metrics.get('dividendYield', np.nan)
            )


            # 2. Technical Analysis (Simplified)
            # Pass price_data and the fetched current_price to technical indicator calculation
            indicators = self.calculate_technical_indicators(price_data, metrics.get('currentPrice', np.nan))
            technical_score = self.calculate_technical_score(indicators) # Basic technical score (out of 50)

            # Generate a basic recommendation (will be replaced by enhanced one later)
            basic_recommendation = self.generate_recommendation(fundamental_score, technical_score)


            # Bundle results
            # Create an AnalysisResult object to hold all analysis results
            analysis_results = AnalysisResult(
                symbol=symbol,
                company_name=company_name, # Use the fetched company name
                current_price=metrics.get('currentPrice', np.nan), # Use the fetched current price
                market_cap=metrics.get('marketCap', np.nan), # Use the fetched market cap
                info=info, # Include the full info dictionary
                price_data=price_data,
                indicators=indicators,
                fundamental_score=fundamental_score,
                technical_score=technical_score,
                recommendation=basic_recommendation, # Store the basic recommendation
                metrics=metrics # Store the populated metrics dictionary
            )

            logger.info(f"Basic analysis completed for {symbol}")
            return analysis_results

        except Exception as e:
            logger.error(f"An unexpected error occurred during basic analysis for {symbol}: {str(e)}. Returning None.")
            return None # Return None if basic analysis fails due to unexpected error

    def calculate_fundamental_score(self, market_cap: float, pe_ratio: float, pb_ratio: float, dividend_yield: float) -> float:
        """Calculate a simple fundamental score based on key metrics (out of 50)"""
        score = 0
        logger.info("Calculating basic fundamental score.")
        # Ensure metrics are valid numbers before scoring
        market_cap_valid = isinstance(market_cap, (int, float)) and not pd.isna(market_cap) and market_cap > 0
        pe_ratio_valid = isinstance(pe_ratio, (int, float)) and not pd.isna(pe_ratio) and pe_ratio > 0
        pb_ratio_valid = isinstance(pb_ratio, (int, float)) and not pd.isna(pb_ratio) and pb_ratio > 0
        dividend_yield_valid = isinstance(dividend_yield, (int, float)) and not pd.isna(dividend_yield) and dividend_yield >= 0

        logger.debug(f"Fundamental metrics validity: Market Cap={market_cap_valid}, PE={pe_ratio_valid}, PB={pb_ratio_valid}, Dividend Yield={dividend_yield_valid}")

        # Market Cap (Scale based on size, larger usually means more stable)
        if market_cap_valid:
            if market_cap > 10_00_000_00_00_000:  # > 10 Lakh Cr (Large Cap)
                score += 15
                logger.debug(f"Market Cap ({market_cap:.2f}) is Large Cap. +15 pts.")
            elif market_cap > 50_000_00_00_000:   # > 50K Cr (Mid Cap)
                score += 10
                logger.debug(f"Market Cap ({market_cap:.2f}) is Mid Cap. +10 pts.")
            else: # Small Cap and below
                score += 5
                logger.debug(f"Market Cap ({market_cap:.2f}) is Small Cap or below. +5 pts.")
        else:
             logger.warning("Market cap is invalid for fundamental scoring.")


        # PE Ratio (Lower is generally better, but depends on industry growth)
        if pe_ratio_valid:
            if pe_ratio < 20:
                score += 15
                logger.debug(f"PE Ratio ({pe_ratio:.2f}) < 20. +15 pts.")
            elif pe_ratio < 30:
                score += 10
                logger.debug(f"PE Ratio ({pe_ratio:.2f}) < 30. +10 pts.")
            elif pe_ratio < 40:
                score += 5
                logger.debug(f"PE Ratio ({pe_ratio:.2f}) < 40. +5 pts.")
            else: # High PE
                score += 2
                logger.debug(f"PE Ratio ({pe_ratio:.2f}) >= 40. +2 pts.")
        else:
             logger.warning("PE ratio is invalid for fundamental scoring.")


        # PB Ratio (Lower is generally better, indicates undervaluation)
        if pb_ratio_valid:
            if pb_ratio < 3:
                score += 10
                logger.debug(f"PB Ratio ({pb_ratio:.2f}) < 3. +10 pts.")
            elif pb_ratio < 5:
                score += 7
                logger.debug(f"PB Ratio ({pb_ratio:.2f}) < 5. +7 pts.")
            else:
                score += 3
                logger.debug(f"PB Ratio ({pb_ratio:.2f}) >= 5. +3 pts.")
        else:
             logger.warning("PB ratio is invalid for fundamental scoring.")


        # Dividend Yield (Higher is better for income-seeking investors, indicates profitability)
        if dividend_yield_valid:
            if dividend_yield > 3: # > 3%
                score += 10
                logger.debug(f"Dividend Yield ({dividend_yield:.2f}%) > 3%. +10 pts.")
            elif dividend_yield > 1: # > 1%
                score += 7
                logger.debug(f"Dividend Yield ({dividend_yield:.2f}%) > 1%. +7 pts.")
            else:
                score += 3
                logger.debug(f"Dividend Yield ({dividend_yield:.2f}%) <= 1%. +3 pts.")
        else:
             logger.warning("Dividend yield is invalid for fundamental scoring.")

        logger.info(f"Basic fundamental score calculated: {score}")
        return min(score, 50) # Cap score at 50

    def calculate_technical_indicators(self, price_data: pd.DataFrame, current_price: float) -> Dict: # Added type hints
        """Calculate key technical indicators"""
        logger.info("Calculating technical indicators.")
        # Ensure price_data is valid before calculating indicators
        if not isinstance(price_data, pd.DataFrame) or price_data.empty:
            logger.warning("Insufficient price data for technical indicator calculation.")
            return {'RSI': np.nan, 'MACD': np.nan, 'MACD_signal': np.nan, 'SMA_20': np.nan, 'SMA_50': np.nan,
                    'BB_upper': np.nan, 'BB_lower': np.nan, 'BB_middle': np.nan, 'Volume_ratio': np.nan,
                    'current_price': current_price} # Return NaNs if data is missing


        indicators = {}
        data_len = len(price_data)

        try:
            # RSI (Relative Strength Index)
            if data_len >= 14:
                delta = price_data['Close'].diff()
                gain = delta.where(delta > 0, 0)
                loss = -delta.where(delta < 0, 0)
                avg_gain = gain.ewm(com=14-1, adjust=False).mean()
                avg_loss = loss.ewm(com=14-1, adjust=False).mean()
                # Handle division by zero explicitly
                if avg_loss.iloc[-1] is not None and not np.isnan(avg_loss.iloc[-1]) and avg_loss.iloc[-1] != 0:
                    rs = avg_gain / avg_loss
                    indicators['RSI'] = 100 - (100 / (1 + rs)).iloc[-1]
                elif avg_gain.iloc[-1] is not None and not np.isnan(avg_gain.iloc[-1]) and avg_gain.iloc[-1] > 0:
                     indicators['RSI'] = 100.0 # If no loss but gain, RSI is 100
                else:
                     indicators['RSI'] = 50.0 # If no gain and no loss, RSI is 50
                logger.debug(f"Calculated RSI: {indicators['RSI']:.2f}")
            else:
                indicators['RSI'] = np.nan
                logger.warning("Insufficient data for RSI.")

        except Exception as e:
            logger.warning(f"Could not calculate RSI: {e}. Setting to NaN.")
            indicators['RSI'] = np.nan

        try:
            # MACD (Moving Average Convergence Divergence)
            if data_len >= 26: # Need at least 26 data points for 26-day EMA
                ema_12 = price_data['Close'].ewm(span=12, adjust=False).mean()
                ema_26 = price_data['Close'].ewm(span=26, adjust=False).mean()
                # Corrected MACD signal calculation to use the MACD Series
                macd_series = ema_12 - ema_26
                if data_len >= 26 + 9 - 1: # Need enough data points for 9-day EMA of MACD
                     indicators['MACD'] = macd_series.iloc[-1]
                     indicators['MACD_signal'] = macd_series.ewm(span=9, adjust=False).mean().iloc[-1]
                     logger.debug(f"Calculated MACD: {indicators['MACD']:.2f}, MACD Signal: {indicators['MACD_signal']:.2f}")
                else:
                     indicators['MACD'] = np.nan
                     indicators['MACD_signal'] = np.nan
                     logger.warning("Insufficient data for MACD signal calculation.")
            else:
                 indicators['MACD'] = np.nan
                 indicators['MACD_signal'] = np.nan
                 logger.warning("Insufficient data for MACD calculation.")
        except Exception as e:
            logger.warning(f"Could not calculate MACD: {e}. Setting to NaN.")
            indicators['MACD'] = np.nan
            indicators['MACD_signal'] = np.nan

        try:
            # Moving Averages
            if data_len >= 20:
                 indicators['SMA_20'] = price_data['Close'].rolling(window=20).mean().iloc[-1]
                 logger.debug(f"Calculated SMA_20: {indicators['SMA_20']:.2f}")
            else:
                 indicators['SMA_20'] = np.nan
                 logger.warning("Insufficient data for SMA_20.")

            if data_len >= 50:
                 indicators['SMA_50'] = price_data['Close'].rolling(window=50).mean().iloc[-1]
                 logger.debug(f"Calculated SMA_50: {indicators['SMA_50']:.2f}")
            else:
                 indicators['SMA_50'] = np.nan
                 logger.warning("Insufficient data for SMA_50.")

        except Exception as e:
            logger.warning(f"Could not calculate Moving Averages: {e}. Setting relevant MAs to NaN.")
            if 'SMA_20' not in indicators: indicators['SMA_20'] = np.nan
            if 'SMA_50' not in indicators: indicators['SMA_50'] = np.nan


        try:
            # Bollinger Bands
            bb_period = 20
            bb_std = 2
            if data_len >= bb_period:
                rolling_mean = price_data['Close'].rolling(window=bb_period).mean()
                rolling_std = price_data['Close'].rolling(window=bb_period).std()
                if not pd.isna(rolling_mean.iloc[-1]) and not pd.isna(rolling_std.iloc[-1]):
                    indicators['BB_middle'] = rolling_mean.iloc[-1]
                    indicators['BB_upper'] = (rolling_mean + (rolling_std * bb_std)).iloc[-1]
                    indicators['BB_lower'] = (rolling_mean - (rolling_std * bb_std)).iloc[-1]
                    logger.debug(f"Calculated BB: Middle={indicators['BB_middle']:.2f}, Upper={indicators['BB_upper']:.2f}, Lower={indicators['BB_lower']:.2f}")
                else:
                    indicators['BB_middle'] = np.nan
                    indicators['BB_upper'] = np.nan
                    indicators['BB_lower'] = np.nan
                    logger.warning("Bollinger Bands calculation resulted in NaN.")
            else:
                indicators['BB_middle'] = np.nan
                indicators['BB_upper'] = np.nan
                indicators['BB_lower'] = np.nan
                logger.warning("Insufficient data for Bollinger Bands.")

        except Exception as e:
            logger.warning(f"Could not calculate Bollinger Bands: {e}. Setting to NaN.")
            indicators['BB_middle'] = np.nan
            indicators['BB_upper'] = np.nan
            indicators['BB_lower'] = np.nan

        try:
            # Volume Analysis (e.g., current volume vs average volume)
            volume_period = 30
            if data_len >= volume_period:
                 avg_volume_30d = price_data['Volume'].rolling(window=volume_period).mean().iloc[-1]
                 current_volume = price_data['Volume'].iloc[-1]
                 # Check for zero division and NaN before calculating ratio
                 if not pd.isna(avg_volume_30d) and avg_volume_30d > 0 and not pd.isna(current_volume):
                      indicators['Volume_ratio'] = current_volume / avg_volume_30d
                      logger.debug(f"Calculated Volume Ratio: {indicators['Volume_ratio']:.2f}")
                 elif not pd.isna(current_volume) and current_volume > 0:
                      # If 30d avg is zero or NaN but current volume is positive, assume high relative volume
                      indicators['Volume_ratio'] = 2.0 # Arbitrary high ratio to indicate spike
                      logger.warning("30-day average volume is invalid for volume ratio, assuming spike due to positive current volume.")
                 else:
                      indicators['Volume_ratio'] = np.nan
                      logger.warning("Insufficient or invalid volume data for volume ratio calculation.")
            else:
                indicators['Volume_ratio'] = np.nan
                logger.warning("Insufficient data for Volume Ratio.")

        except Exception as e:
            logger.warning(f"Could not calculate Volume Ratio: {e}. Setting to NaN.")
            indicators['Volume_ratio'] = np.nan

        # Include current price in indicators for technical score calculation
        indicators['current_price'] = current_price if not pd.isna(current_price) else np.nan

        logger.info(f"Technical indicators calculated: {indicators}")
        return indicators


    def calculate_technical_score(self, indicators: Dict) -> float: # Added type hint
        """Calculate a simple technical score based on indicators (out of 50)"""
        score = 0
        logger.info("Calculating basic technical score.")
        # Ensure indicators dictionary is valid
        if not isinstance(indicators, dict):
             logger.warning("Invalid indicators dictionary for technical scoring.")
             return 0

        # Scoring based on common indicator signals
        # Ensure indicator values are valid numbers before scoring
        rsi = indicators.get('RSI', np.nan)
        macd = indicators.get('MACD', np.nan)
        macd_signal = indicators.get('MACD_signal', np.nan)
        sma_20 = indicators.get('SMA_20', np.nan)
        sma_50 = indicators.get('SMA_50', np.nan)
        current_price = indicators.get('current_price', np.nan)
        bb_upper = indicators.get('BB_upper', np.nan)
        bb_lower = indicators.get('BB_lower', np.nan)
        volume_ratio = indicators.get('Volume_ratio', np.nan)

        logger.debug(f"Indicators for basic technical scoring: RSI={rsi}, MACD={macd}, Signal={macd_signal}, SMA20={sma_20}, SMA50={sma_50}, Price={current_price}, BB_Upper={bb_upper}, BB_Lower={bb_lower}, VolumeRatio={volume_ratio}")


        # RSI Score
        if not pd.isna(rsi):
            if rsi < 30: # Oversold
                score += 10
                logger.debug("RSI < 30. +10 pts.")
            elif rsi > 70: # Overbought
                score -= 10
                logger.debug("RSI > 70. -10 pts.")
            elif 40 <= rsi <= 60: # Neutral
                 score += 5
                 logger.debug("RSI 40-60. +5 pts.")
        else:
             logger.warning("RSI is NaN for technical scoring.")

        # MACD Score (Bullish crossover)
        if not pd.isna(macd) and not pd.isna(macd_signal):
            if macd > macd_signal:
                score += 10
                logger.debug("MACD > Signal. +10 pts.")
            else:
                 score += 2 # Small score if bearish or neutral
                 logger.debug("MACD <= Signal. +2 pts.")
        else:
             logger.warning("MACD or MACD signal is NaN for technical scoring.")


        # Moving Average Crossover (Bullish: 20-day > 50-day)
        if not pd.isna(sma_20) and not pd.isna(sma_50):
            if sma_20 > sma_50:
                score += 10
                logger.debug("SMA20 > SMA50. +10 pts.")
            else:
                score += 5 # Add a small score if 20-day is below but close to 50-day
                logger.debug("SMA20 <= SMA50. +5 pts.")
        else:
             logger.warning("SMA_20 or SMA_50 is NaN for technical scoring.")


        # Bollinger Bands (Price near lower band suggests potential buy)
        if not pd.isna(current_price) and not pd.isna(bb_lower) and not pd.isna(bb_upper):
            if not pd.isna(bb_lower) and current_price < bb_lower:
                score += 10
                logger.debug("Price < Lower BB. +10 pts.")
            elif not pd.isna(bb_upper) and current_price > bb_upper:
                score -= 10
                logger.debug("Price > Upper BB. -10 pts.")
            elif not pd.isna(bb_lower) and not pd.isna(bb_upper) and bb_lower <= current_price <= bb_upper: # Price is within the bands
                 score += 5 # Add a small score for price being within bands, indicating less volatility
                 logger.debug("Price within BB. +5 pts.")
        else:
             logger.warning("Bollinger Bands or current price is NaN for technical scoring.")


        # Volume Spike (Significant volume increase can confirm trends)
        if not pd.isna(volume_ratio):
            if volume_ratio > 1.5: # Volume is 50% higher than average
                 score += 10
                 logger.debug(f"Volume Ratio ({volume_ratio:.2f}) > 1.5. +10 pts.")
            elif volume_ratio < 0.8: # Volume is significantly lower than average
                 score += 2 # Small score for low volume
                 logger.debug(f"Volume Ratio ({volume_ratio:.2f}) < 0.8. +2 pts.")
            else:
                 score += 5 # Average volume is neutral
                 logger.debug(f"Volume Ratio ({volume_ratio:.2f}) 0.8-1.5. +5 pts.")
        else:
             logger.warning("Volume ratio is NaN for technical scoring.")


        logger.info(f"Basic technical score calculated: {score}")
        return max(0, min(score, 50)) # Cap score between 0 and 50

    def generate_recommendation(self, fundamental_score: float, technical_score: float) -> str: # Added type hints
        """Generate investment recommendation based on scores"""
        logger.info(f"Generating basic recommendation based on Fundamental Score: {fundamental_score}, Technical Score: {technical_score}")
        # Ensure scores are valid numbers before summing
        fund_score = fundamental_score if isinstance(fundamental_score, (int, float)) and not np.isnan(fundamental_score) else 0
        tech_score = technical_score if isinstance(technical_score, (int, float)) and not np.isnan(technical_score) else 0

        total_score = fund_score + tech_score
        logger.debug(f"Total score for basic recommendation: {total_score}")

        if total_score >= 75:
            return "STRONG BUY"
        elif total_score >= 60:
            return "BUY"
        elif total_score >= 40:
            return "HOLD"
        elif total_score >= 25:
            return "SELL"
        else:
            return "STRONG SELL"


## Fetch sector peer data

### Subtask:
Enhance the peer data fetching or generation to include valuation ratios for peer companies within the same sector and market cap category.


**Reasoning**:
Modify the `get_peer_companies` function to fetch valuation ratios for peers and return this information.



In [ ]:
#@title Part 2: EnhancedStockAnalyzer class definition (with Peer Valuation Data Fetching)

import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import requests
from bs4 import BeautifulSoup
import warnings
warnings.filterwarnings('ignore')

# Import typing for type hints
from typing import List, Dict, Optional

# Ensure logging is configured (assuming this was done in a previous cell)
import logging
logger = logging.getLogger(__name__)

# --- EnhancedStockAnalyzer class definition ---
class EnhancedStockAnalyzer:
    """Enhanced analyzer with additional features for Indian markets"""

    def __init__(self):
        """Initialize the enhanced analyzer."""
        self.risk_free_rate = self.get_dynamic_risk_free_rate()
        # Assuming gemini_model is defined globally from a previous cell
        global gemini_model # Access the global gemini_model
        self.gemini_model = gemini_model

        # More detailed circuit limit percentages based on NSE categorization (simplified)
        self.circuit_limits_percentage = {
            'category_a': 0.20, # Typically high-liquidity index stocks
            'category_b': 0.10, # Most other actively traded stocks
            'category_t': 0.05  # Stocks in trade-for-trade segment
        }

        # Simple mapping of some known stocks to categories for demonstration
        self.stock_category_map = {
            'RELIANCE.NS': 'category_a',
            'TCS.NS': 'category_a',
            'HDFCBANK.NS': 'category_a',
            'INFY.NS': 'category_a',
            'ICICIBANK.NS': 'category_a',
            'VEDL.NS': 'category_b',
            'PNB.NS': 'category_b',
            'INDIGO.NS': 'category_b',
            'BANDHANBNK.NS': 'category_b',
            'PERSISTENT.NS': 'category_b',
            'COFORGE.NS': 'category_b',
            'LTTS.NS': 'category_b',
        }


    def get_dynamic_risk_free_rate(self):
        """Fetch current 10-year G-Sec yield"""
        try:
            return 0.072  # 7.2% as of recent data
        except Exception as e:
            logger.warning(f"Could not fetch dynamic risk-free rate: {e}. Using fallback rate.")
            return 0.065

    def get_circuit_limits(self, symbol):
        """Get circuit breaker limits for the stock based on categorization"""
        try:
            ticker = yf.Ticker(symbol)
            info = ticker.info

            if not info or 'previousClose' not in info:
                 logger.warning(f"Could not get previous close for {symbol}. Cannot calculate circuit limits.")
                 return None

            prev_close = info.get('previousClose')
            if pd.isna(prev_close) or prev_close <= 0:
                 logger.warning(f"Invalid previous close price for {symbol}: {prev_close}. Cannot calculate circuit limits.")
                 return None


            stock_category = self.stock_category_map.get(symbol.upper(), 'category_b')
            circuit_percent = self.circuit_limits_percentage.get(stock_category, 0.10)

            logger.info(f"Determined circuit percentage for {symbol} (Category: {stock_category}): {circuit_percent*100}%")

            upper_circuit = prev_close * (1 + circuit_percent)
            lower_circuit = prev_close * (1 - circuit_percent)

            return {
                'upper_circuit': upper_circuit,
                'lower_circuit': lower_circuit,
                'circuit_percent': circuit_percent
            }
        except Exception as e:
            logger.error(f"Error getting circuit breaker limits for {symbol}: {str(e)}")
            return None

    def assess_circuit_risk(self, current_price, circuit_limits):
        """Assess proximity to circuit limits"""
        if pd.isna(current_price) or current_price <= 0:
             logger.warning("Invalid current price for circuit risk assessment.")
             return 0

        if not circuit_limits or 'upper_circuit' not in circuit_limits or 'lower_circuit' not in circuit_limits:
            logger.warning("Invalid or incomplete circuit limits provided.")
            return 0

        upper_circuit = circuit_limits['upper_circuit']
        lower_circuit = circuit_limits['lower_circuit']

        if pd.isna(upper_circuit) or pd.isna(lower_circuit) or upper_circuit <= 0 or lower_circuit <= 0:
             logger.warning("Circuit limits are invalid numbers. Cannot assess circuit risk.")
             return 0

        risk_score = 0

        if current_price >= upper_circuit:
             risk_score -= 30
        elif (upper_circuit - current_price) / current_price < 0.01:
            risk_score -= 20
        elif (upper_circuit - current_price) / current_price < 0.03:
            risk_score -= 10
        elif (upper_circuit - current_price) / current_price < 0.05:
            risk_score -= 5

        if current_price <= lower_circuit:
            risk_score -= 40
        elif (current_price - lower_circuit) / current_price < 0.01:
            risk_score -= 30
        elif (current_price - lower_circuit) / current_price < 0.03:
            risk_score -= 15
        elif (current_price - lower_circuit) / current_price < 0.05:
            risk_score -= 8

        logger.info(f"Circuit risk score calculated: {risk_score}")
        return risk_score

    def get_liquidity_score(self, price_data):
        """Enhanced liquidity analysis with NaN handling and better checks"""
        liquidity_score = 0
        if not isinstance(price_data, pd.DataFrame) or price_data.empty or len(price_data) < 20:
            logger.warning("Insufficient price data for liquidity analysis. Assigning default score.")
            return 30

        try:
            avg_volume_20d = price_data['Volume'].rolling(20).mean().iloc[-1]
            avg_volume_5d = price_data['Volume'].rolling(5).mean().iloc[-1]
            current_close = price_data['Close'].iloc[-1]

            # Add logging for diagnosis - before calculation check
            logger.info(f"Volume data for liquidity: avg_volume_20d: {avg_volume_20d}, avg_volume_5d: {avg_volume_5d}, current_close: {current_close}")


            if pd.isna(avg_volume_20d) or pd.isna(avg_volume_5d) or pd.isna(current_close) or current_close <= 0:
                 logger.warning("Liquidity metrics are NaN or invalid. Cannot calculate liquidity score accurately. Assigning default.")
                 return 30

            avg_turnover_20d = np.nan
            if avg_volume_20d is not None and not pd.isna(avg_volume_20d) and avg_volume_20d >= 0 and current_close > 0:
                 avg_turnover_20d = avg_volume_20d * current_close

            volume_spike = np.nan
            if avg_volume_20d is not None and not pd.isna(avg_volume_20d) and avg_volume_20d > 0:
                 if avg_volume_5d is not None and not pd.isna(avg_volume_5d):
                     volume_spike = avg_volume_5d / avg_volume_20d
                 else:
                      logger.warning("5-day average volume is invalid for volume spike calculation.")
            elif not pd.isna(avg_volume_5d) and avg_volume_5d > 0:
                 volume_spike = 1.0
                 logger.warning("20-day average volume is invalid for volume spike, using 1.0 as relative indicator.")
            else:
                 volume_spike = np.nan # Ensure volume_spike is NaN if both volumes are invalid
                 logger.warning("Insufficient or invalid volume data for volume ratio calculation.")

            # Add logging for volume spike - after calculation
            logger.info(f"Volume spike calculated: {volume_spike}")

            # Corrected logic for scoring based on volume_spike
            if not pd.isna(volume_spike):
                if volume_spike > 1.5: # Volume is 50% higher than average
                    liquidity_score += 10
                elif volume_spike < 0.8: # Volume is significantly lower than average
                    liquidity_score += 2 # Small penalty for low volume
                else: # volume_spike is between 0.8 and 1.5 (inclusive of 0.8, exclusive of 1.5)
                    liquidity_score += 5 # Average volume is neutral
            else:
                 logger.warning("Volume spike ratio is NaN. Skipping trend scoring.")


            if not pd.isna(avg_turnover_20d):
                if avg_turnover_20d > 500_00_00_000:
                    liquidity_score += 40
                elif avg_turnover_20d > 100_00_00_000:
                    liquidity_score += 30
                elif avg_turnover_20d > 10_00_00_000:
                    liquidity_score += 20
                elif avg_turnover_20d > 1_00_00_000:
                    liquidity_score += 10
                else:
                    liquidity_score += 5
            else:
                 logger.warning("Average turnover is NaN. Skipping turnover scoring.")
                 liquidity_score += 10

            volume_std = price_data['Volume'].rolling(20).std().iloc[-1] if len(price_data) >= 20 else np.nan
            volume_cv = np.nan
            if avg_volume_20d is not None and not pd.isna(avg_volume_20d) and avg_volume_20d > 0 and \
               volume_std is not None and not pd.isna(volume_std):
                volume_cv = volume_std / avg_volume_20d

            if not pd.isna(volume_cv):
                if volume_cv < 0.5:
                    liquidity_score += 20
                elif volume_cv < 1.0:
                    liquidity_score += 10
            else:
                 logger.warning("Volume coefficient of variation is NaN. Skipping consistency scoring.")

            # The volume spike scoring is now handled above within the main if not pd.isna(volume_spike): block


            return min(liquidity_score, 70)

        except Exception as e:
            logger.error(f"Error calculating liquidity score: {str(e)}")
            return 30

    def get_peer_companies(self, symbol: str, stock_info: Dict) -> List[Dict]: # Modified return type hint
        """Identify peer companies and fetch their valuation ratios for comparison"""
        logger.info(f"Identifying peer companies and fetching valuation data for {symbol}.")
        peer_list_with_valuation = [] # List to store peer info including valuation

        try:
            if not isinstance(stock_info, dict):
                 logger.warning("Invalid stock info for peer identification.")
                 return []

            sector = stock_info.get('sector', '')
            market_cap = stock_info.get('marketCap', 0)

            if not isinstance(sector, str):
                 sector = ''
            if not isinstance(market_cap, (int, float)) or pd.isna(market_cap) or market_cap <= 0:
                 logger.warning("Invalid market cap for peer identification.")
                 market_cap = 0

            if market_cap > 10_00_000_00_00_000:
                cap_category = 'Large Cap'
            elif market_cap > 50_000_00_00_000:
                cap_category = 'Mid Cap'
            else:
                cap_category = 'Small Cap'

            # Peer mapping (simplified - in production, use a comprehensive database)
            peer_map = {
                'Technology': {
                    'Large Cap': ['TCS.NS', 'INFY.NS', 'WIPRO.NS', 'HCLTECH.NS'],
                    'Mid Cap': ['TECHM.NS', 'LTTS.NS', 'PERSISTENT.NS', 'COFORGE.NS'],
                    'Small Cap': ['ZENSAR.NS', 'ECLERX.NS', 'MASTEK.NS']
                },
                'Financial Services': {
                    'Large Cap': ['HDFCBANK.NS', 'ICICIBANK.NS', 'KOTAKBANK.NS', 'AXISBANK.NS'],
                    'Mid Cap': ['IDFCFIRSTB.NS', 'FEDERALBNK.NS', 'INDUSINDBK.NS'],
                    'Small Cap': ['SOUTHBANK.NS', 'UJJIVAN.NS', 'EQUITASBNK.NS']
                },
                'Healthcare': {
                    'Large Cap': ['SUNPHARMA.NS', 'DRREDDY.NS', 'CIPLA.NS', 'DIVISLAB.NS'],
                    'Mid Cap': ['TORNTPHARM.NS', 'ALKEM.NS', 'LUPIN.NS'],
                    'Small Cap': ['CAPLIPOINT.NS', 'BLISS.NS', 'INDOCO.NS']
                },
                # Added Energy sector mapping
                'Energy': {
                    'Large Cap': ['ONGC.NS', 'NTPC.NS', 'POWERGRID.NS'], # Exclude RELIANCE.NS here, filter below
                    'Mid Cap': ['GAIL.NS', 'IOC.NS', 'NHPC.NS'],
                    'Small Cap': ['GUJGASLTD.NS', 'IGL.NS', 'MAHAPOWER.NS']
                }
            }

            potential_peers = peer_map.get(sector, {}).get(cap_category, [])
            # Ensure the current symbol is excluded from the peer list
            filtered_peers = [p for p in potential_peers if isinstance(p, str) and p.upper() != symbol.upper()]

            logger.info(f"Identified {len(filtered_peers)} potential peer companies for {symbol} in {sector} ({cap_category} Cap). Fetching valuation data.")

            # Fetch valuation data for each filtered peer
            for peer_symbol in filtered_peers:
                 try:
                      peer_ticker = yf.Ticker(peer_symbol)
                      peer_info = peer_ticker.info

                      if peer_info:
                           peer_valuation = {
                               'symbol': peer_symbol,
                               'trailingPE': peer_info.get('trailingPE', np.nan),
                               'priceToBook': peer_info.get('priceToBook', np.nan),
                               'enterpriseValue': peer_info.get('enterpriseValue', np.nan), # Fetch for EV/EBITDA calculation
                               'ebitda': peer_info.get('ebitda', np.nan) # Fetch for EV/EBITDA calculation
                           }

                           # Calculate EV/EBITDA for the peer
                           ev = peer_valuation['enterpriseValue']
                           ebitda = peer_valuation['ebitda']
                           peer_valuation['evToEbitda'] = np.nan # Default to NaN
                           if isinstance(ev, (int, float)) and not np.isnan(ev) and \
                              isinstance(ebitda, (int, float)) and not np.isnan(ebitda) and ebitda != 0:
                               peer_valuation['evToEbitda'] = ev / ebitda
                           elif (isinstance(ev, (int, float)) and not np.isnan(ev)) and \
                                (isinstance(ebitda, (int, float)) and not np.isnan(ebitda) and ebitda == 0):
                                logger.warning(f"EBITDA is zero for peer {peer_symbol}. Cannot calculate EV/EBITDA.")
                           else:
                                logger.warning(f"Missing Enterprise Value ({ev}) or EBITDA ({ebitda}) for peer {peer_symbol}. Cannot calculate EV/EBITDA.")


                           peer_list_with_valuation.append(peer_valuation)
                           logger.debug(f"Fetched valuation data for peer {peer_symbol}: PE={peer_valuation['trailingPE']}, PB={peer_valuation['priceToBook']}, EV/EBITDA={peer_valuation['evToEbitda']}")
                      else:
                           logger.warning(f"Could not fetch info for peer {peer_symbol}. Skipping.")
                 except Exception as e:
                      logger.warning(f"Error fetching data for peer {peer_symbol}: {str(e)}. Skipping.")

            logger.info(f"Successfully fetched valuation data for {len(peer_list_with_valuation)} peers.")
            return peer_list_with_valuation[:5] # Return top 5 peers with their valuation data

        except Exception as e:
            logger.error(f"Error getting peer companies and valuation data for {symbol}: {str(e)}")
            return [] # Return empty list on error


    def calculate_peer_relative_performance(self, symbol, peers, days=30):
        """Compare stock performance with peers, handling data issues robustly"""
        results = {
            'stock_return': np.nan,
            'avg_peer_return': np.nan,
            'relative_performance': np.nan,
            'outperformance': False,
            'days': days
        }

        if not isinstance(peers, list) or not peers:
             logger.warning("No valid peer list provided for performance comparison.")
             # Return the default results with NaNs if no peers
             return results

        # --- Removed the temporary skip here ---

        try:
            main_ticker = yf.Ticker(symbol)
            main_hist = main_ticker.history(period=f"{days}d")
            main_return = np.nan

            if not main_hist.empty and len(main_hist) > 1:
                try:
                    start_price = main_hist['Close'].iloc[0]
                    end_price = main_hist['Close'].iloc[-1]
                    if not pd.isna(start_price) and not pd.isna(end_price) and start_price > 0:
                         main_return = ((end_price / start_price) - 1) * 100
                    else:
                         logger.warning(f"Invalid start ({start_price}) or end price ({end_price}) for main stock ({symbol}) return calculation.")
                except Exception as ret_e:
                     logger.warning(f"Could not calculate main stock ({symbol}) return: {ret_e}. Setting to NaN.")
                     main_return = np.nan
            else:
                logger.warning(f"Insufficient historical data for main stock ({symbol}) for return calculation.")

            results['stock_return'] = main_return

        except Exception as main_stock_e:
            logger.error(f"Error fetching data or calculating return for main stock ({symbol}): {main_stock_e}. Main stock return set to NaN.")
            results['stock_return'] = np.nan

        # Calculate peer returns
        peer_returns = []
        # The peers list now contains dictionaries with valuation data.
        # We need to extract just the symbols for performance calculation.
        peer_symbols_for_perf = [p['symbol'] for p in peers if isinstance(p, dict) and 'symbol' in p]

        for peer_symbol in peer_symbols_for_perf:
             try:
                 peer_ticker = yf.Ticker(peer_symbol)
                 peer_hist = peer_ticker.history(period=f"{days}d")
                 if not peer_hist.empty and len(peer_hist) > 1:
                     try:
                          peer_start_price = peer_hist['Close'].iloc[0]
                          peer_end_price = peer_hist['Close'].iloc[-1]
                          if not pd.isna(peer_start_price) and not pd.isna(peer_end_price) and peer_start_price > 0:
                             peer_return = ((peer_end_price / peer_start_price) - 1) * 100
                             peer_returns.append(peer_return)
                          else:
                             logger.warning(f"Invalid start or end price for peer ({peer_symbol}) return calculation.")
                     except Exception as peer_ret_e:
                         logger.warning(f"Could not calculate return for peer ({peer_symbol}): {peer_ret_e}. Skipping.")
                 else:
                     logger.warning(f"Insufficient historical data for peer ({peer_symbol}) for return calculation. Skipping.")
             except Exception as peer_e:
                 logger.warning(f"Error fetching data for peer ({peer_symbol}): {peer_e}. Skipping.")

        # Add logging for peer returns before filtering/average calculation
        logger.info(f"Peer returns collected before average calculation attempt: {peer_returns}")

        # Calculate average peer return and relative performance
        if peer_returns:
             # Ensure peer_returns list contains only valid numbers before calculating mean
             valid_peer_returns = [ret for ret in peer_returns if isinstance(ret, (int, float)) and not pd.isna(ret)]
             if valid_peer_returns:
                 avg_peer_return = np.mean(valid_peer_returns)
                 results['avg_peer_return'] = avg_peer_return

                 # Calculate relative performance only if main stock return is valid
                 if not pd.isna(main_return):
                      results['relative_performance'] = main_return - avg_peer_return

                      # Determine outperformance
                      if main_return > avg_peer_return:
                          results['outperformance'] = True
                 else:
                      logger.warning("Main stock return is NaN, cannot calculate relative performance or outperformance.")
             else:
                 logger.warning("No valid peer returns available after filtering. Cannot calculate average peer return or relative performance.")
        else:
             logger.warning("No peer returns calculated. Cannot calculate average peer return or relative performance.")


        logger.info(f"Peer performance results for {symbol}: {results}")
        return results


    def get_index_membership(self, symbol):
        """Check index membership for the stock"""
        symbol_upper = symbol.upper() if isinstance(symbol, str) else ''

        index_constituents = {
            'NIFTY50': ['RELIANCE.NS', 'TCS.NS', 'HDFCBANK.NS', 'INFY.NS', 'ICICIBANK.NS'],
            'NIFTY_NEXT50': ['VEDL.NS', 'PNB.NS', 'INDIGO.NS', 'BANDHANBNK.NS'],
            'NIFTY_MIDCAP150': ['PERSISTENT.NS', 'COFORGE.NS', 'LTTS.NS']
        }

        membership = []
        for index, constituents in index_constituents.items():
            if symbol_upper and any(isinstance(c, str) and c.upper() == symbol_upper for c in constituents):
                membership.append(index)

        if membership:
             logger.info(f"Stock {symbol} is a member of indices: {', '.join(membership)}")
        else:
             logger.info(f"Stock {symbol} is not found in the hardcoded index constituents.")

        return membership

    def calculate_enhanced_technical_score(self, indicators, circuit_risk):
        """Calculate enhanced technical analysis score with NaN handling and circuit risk"""
        base_score = 0
        indicators_calculated = 0
        possible_indicator_points = 50

        num_key_indicators = 5
        indicator_point_contribution = possible_indicator_points / num_key_indicators

        rsi = indicators.get('RSI', np.nan)
        if not pd.isna(rsi):
             indicators_calculated += 1
             if 40 <= rsi <= 60:
                 base_score += indicator_point_contribution * 0.7
             elif rsi < 30: # More points for extreme oversold
                 base_score += indicator_point_contribution * 1.0
             elif 30 <= rsi < 40: # Approaching oversold
                 base_score += indicator_point_contribution * 0.9
             elif 60 < rsi <= 70: # Approaching overbought
                 base_score += indicator_point_contribution * 0.6
             else: # Overbought (>70)
                 base_score += indicator_point_contribution * 0.4
        else:
             logger.warning("RSI not calculated or is NaN.")

        macd = indicators.get('MACD', np.nan)
        macd_signal = indicators.get('MACD_signal', np.nan)

        if not pd.isna(macd) and not pd.isna(macd_signal):
             indicators_calculated += 1
             if macd > macd_signal:
                 if macd > 0: # Bullish crossover above zero line
                      base_score += indicator_point_contribution * 1.0
                 else: # Bullish crossover below zero line
                      base_score += indicator_point_contribution * 0.8
             else:
                 if macd < 0: # Bearish crossover below zero line
                      base_score += indicator_point_contribution * 0.2
                 else: # Bearish crossover above zero line
                      base_score += indicator_point_contribution * 0.4
        else:
             logger.warning("MACD or MACD signal is NaN for technical scoring.")

        sma_20 = indicators.get('SMA_20', np.nan)
        sma_50 = indicators.get('SMA_50', np.nan)
        sma_200 = indicators.get('SMA_200', np.nan) # Include 200-day SMA if available

        ma_score_component = 0
        ma_indicators_counted = 0

        if not pd.isna(sma_20) and not pd.isna(sma_50):
             ma_indicators_counted += 1
             if sma_20 > sma_50:
                  ma_score_component += 0.5 # 20 > 50 crossover is bullish
             else:
                  ma_score_component += 0.2 # 20 <= 50 crossover is bearish/neutral

        if not pd.isna(sma_50) and not pd.isna(sma_200):
             ma_indicators_counted += 1
             if sma_50 > sma_200:
                  ma_score_component += 0.5 # 50 > 200 crossover is bullish
             else:
                  ma_score_component += 0.2 # 50 <= 200 crossover is bearish/neutral

        current_price = indicators.get('current_price', np.nan)
        if not pd.isna(current_price):
             if not pd.isna(sma_20) and current_price > sma_20:
                  ma_score_component += 0.3 # Price above 20-day SMA is bullish
             elif not pd.isna(sma_20):
                  ma_score_component += 0.1 # Price below 20-day SMA is bearish/neutral

             if not pd.isna(sma_50) and current_price > sma_50:
                  ma_score_component += 0.4 # Price above 50-day SMA is more bullish
             elif not pd.isna(sma_50):
                  ma_score_component += 0.15 # Price below 50-day SMA

             if not pd.isna(sma_200) and current_price > sma_200:
                  ma_score_component += 0.6 # Price above 200-day SMA is significant bullish signal
             elif not pd.isna(sma_200):
                  ma_score_component += 0.1 # Price below 200-day SMA is significant bearish signal


        if ma_indicators_counted > 0 or (not pd.isna(current_price) and (not pd.isna(sma_20) or not pd.isna(sma_50) or not pd.isna(sma_200))):
             indicators_calculated += 1 # Count MA section if any valid MA comparison/position is made
             # Normalize MA score component to contribute to the total score
             # Max possible ma_score_component (0.5 + 0.5 + 0.3 + 0.4 + 0.6) = 2.3
             # Let's scale this to contribute up to indicator_point_contribution
             max_ma_component = 2.3
             base_score += (ma_score_component / max_ma_component) * indicator_point_contribution if max_ma_component > 0 else 0
        else:
             logger.warning("Insufficient data for Moving Averages analysis.")


        bb_upper = indicators.get('BB_upper', np.nan)
        bb_lower = indicators.get('BB_lower', np.nan)
        bb_middle = indicators.get('BB_middle', np.nan)

        if not pd.isna(current_price) and not pd.isna(bb_upper) and not pd.isna(bb_lower) and not pd.isna(bb_middle):
            indicators_calculated += 1
            if current_price < bb_lower:
                base_score += indicator_point_contribution * 1.0 # Price below lower band (potential buy signal)
            elif bb_lower <= current_price < bb_middle:
                base_score += indicator_point_contribution * 0.8 # Between lower and middle band
            elif bb_middle <= current_price < bb_upper:
                base_score += indicator_point_contribution * 0.6 # Between middle and upper band
            else: # Price above upper band
                base_score += indicator_point_contribution * 0.4 # Potential sell signal
        else:
             logger.warning("Bollinger Bands or current price is NaN for technical scoring.")

        volume_ratio = indicators.get('Volume_ratio', np.nan)
        if not pd.isna(volume_ratio):
             indicators_calculated += 1
             if volume_ratio > 2.0: # Very high volume
                 base_score += indicator_point_contribution * 1.0
             elif volume_ratio > 1.2: # High volume
                 base_score += indicator_point_contribution * 0.8
             elif 0.8 <= volume_ratio <= 1.2: # Normal volume
                 base_score += indicator_point_contribution * 0.5
             else: # Low volume
                 base_score += indicator_point_contribution * 0.3
        else:
             logger.warning("Volume ratio is NaN for technical scoring.")


        achieved_score_from_indicators = base_score

        # Normalize the base score based on how many indicators were successfully calculated
        if indicators_calculated > 0:
            # Assuming each of the num_key_indicators contributes equally if available
            # This approach gives points for each indicator that could be calculated and scored
            # Max points if all indicators could be scored is num_key_indicators * indicator_point_contribution
            # But our scoring within each indicator gives variable points (e.g., 0.4 to 1.0)
            # A simpler approach is to just sum up the points from successfully scored indicators
            # Let's cap the sum at the max possible points from indicators (50)
            final_base_score = min(achieved_score_from_indicators, possible_indicator_points)

            if indicators_calculated < num_key_indicators:
                 logger.warning(f"Only {indicators_calculated}/{num_key_indicators} key technical indicators could be fully calculated/scored. Final base technical score is based on available data.")

        else:
             logger.warning("No key technical indicators calculated. Base technical score is 0.")
             final_base_score = 0


        circuit_risk_adj = circuit_risk if isinstance(circuit_risk, (int, float)) and not pd.isna(circuit_risk) else 0

        final_score = final_base_score + circuit_risk_adj

        return max(0, min(final_score, 50))


    def fetch_news(self, symbol: str) -> List[str]:
        """
        Fetches recent news headlines for the given stock symbol.
        NOTE: This is a placeholder. In a real application, you would integrate
        with a News API (like News API, Alpha Vantage, etc.) here.
        """
        print(f"Fetching news for {symbol} (placeholder)...")
        symbol_str = str(symbol) if symbol is not None else "Unknown Stock"
        return [
            f"{symbol_str} stock price rises on positive market sentiment",
            f"{symbol_str} announces strong quarterly results, beating estimates",
            f"Experts bullish on {symbol_str}'s future outlook",
            f"I am a non-numeric entry that might cause an error",
            f"{symbol_str} faces regulatory challenges in key market",
            f"Competitor's new product launch impacts {symbol_str}'s market share",
            f"Global market trends positively impacting {symbol_str}",
            f"{symbol_str} management provides optimistic guidance",
            f"Increased foreign investment flows into {symbol_str}",
            f"Regulatory approval received for {symbol_str}'s new product",
            f"Production issues reported for {symbol_str}",
            f"Increased competition puts pressure on {symbol_str}'s margins",
            f"Analyst downgrades rating for {symbol_str}",
            f"Supply chain disruptions affect {symbol_str}'s operations",
            f"Geopolitical tensions create uncertainty for {symbol_str}",
            f"{symbol_str} announces stock split",
            f"Dividend declared by {symbol_str}"
        ]


    def analyze_sentiment(self, news_headlines: List[str]) -> Dict:
        """
        Analyzes the sentiment of news headlines using the Gemini API.
        Returns a dictionary with sentiment counts (positive, neutral, negative)
        and a sentiment score.
        """
        if not self.gemini_model:
            print("Gemini API not configured. Skipping sentiment analysis.")
            return {'positive': 0, 'neutral': 0, 'negative': 0, 'score': 0, 'summary': 'Gemini API not configured. Sentiment analysis skipped.'}

        if not isinstance(news_headlines, list) or not news_headlines:
            print("No news headlines to analyze. Skipping sentiment analysis.")
            return {'positive': 0, 'neutral': 0, 'negative': 0, 'score': 12.5, 'summary': 'No news headlines. Sentiment analysis skipped.'}

        logger.info("Analyzing sentiment using Gemini API...")
        prompt = "Analyze the sentiment of the following news headlines for a stock and categorize each as Positive, Neutral, or Negative. Provide a concise summary of the overall sentiment.\n\n"
        for headline in news_headlines:
            if isinstance(headline, str):
                 prompt += f"- {headline}\n"
            else:
                 logger.warning(f"Skipping non-string headline: {headline}")
                 continue

        if len(prompt) > 30000:
             logger.warning("News headlines too long for Gemini API prompt. Truncating.")
             prompt = prompt[:30000] + "\n... (headlines truncated)"

        try:
            response = self.gemini_model.generate_content(prompt)
            sentiment_text = response.text.strip()
            logger.info(f"Gemini API Response: {sentiment_text[:200]}...")

            positive_count = sentiment_text.lower().count('positive')
            neutral_count = sentiment_text.lower().count('neutral')
            negative_count = sentiment_text.lower().count('negative')

            summary_line = "No summary extracted."
            lines = sentiment_text.split('\n')
            for line in lines:
                 lower_line = line.lower()
                 if 'summary' in lower_line or 'overall sentiment' in lower_line:
                     summary_line = line.strip()
                     break
            if summary_line == "No summary extracted." and lines:
                 for line in reversed(lines):
                     if line.strip():
                         summary_line = line.strip()
                         break

            total_headlines = len([h for h in news_headlines if isinstance(h, str)])
            if total_headlines > 0:
                sentiment_score = ((positive_count - negative_count) / total_headlines) * 12.5 + 12.5
            else:
                sentiment_score = 12.5

            return {
                'positive': positive_count,
                'neutral': neutral_count,
                'negative': negative_count,
                'score': max(0, min(sentiment_score, 25)),
                'summary': summary_line
            }

        except Exception as e:
            logger.error(f"Error analyzing sentiment with Gemini API: {e}. Skipping sentiment analysis.")
            return {'positive': 0, 'neutral': 0, 'negative': 0, 'score': 0, 'summary': 'Sentiment analysis failed.'}


    def generate_enhanced_recommendation(self, fundamental_score, technical_score,
                                       liquidity_score, peer_performance, index_membership,
                                       sentiment_analysis_results):
        """Generate comprehensive recommendation including sentiment"""
        fund_score = fundamental_score if isinstance(fundamental_score, (int, float)) and not pd.isna(fundamental_score) else 0
        tech_score = technical_score if isinstance(technical_score, (int, float)) and not pd.isna(technical_score) else 0
        liq_score = liquidity_score if isinstance(liquidity_score, (int, float)) and not pd.isna(liquidity_score) else 0
        sentiment_score = sentiment_analysis_results.get('score', 0) if isinstance(sentiment_analysis_results.get('score'), (int, float)) and not pd.isna(sentiment_analysis_results.get('score')) else 0

        total_score = fund_score + tech_score + liq_score + sentiment_score

        peer_analysis_available = isinstance(peer_performance, dict) and \
                                  not (pd.isna(peer_performance.get('stock_return')) and \
                                       pd.isna(peer_performance.get('avg_peer_return')) and \
                                       pd.isna(peer_performance.get('relative_performance')))

        if peer_analysis_available and peer_performance.get('outperformance', False) is True:
            total_score += 5
            logger.info("Adding bonus for peer outperformance.")
        elif not peer_analysis_available:
             total_score -= 5
             logger.warning("Applying small penalty as peer performance data is incomplete/unavailable.")

        if isinstance(index_membership, list) and index_membership:
            total_score += 5
            logger.info("Adding bonus for index membership.")

        recommendation = "NEUTRAL"
        confidence = "Low"

        if total_score >= 170:
            recommendation = "STRONG BUY"
            confidence = "High"
        elif total_score >= 140:
            recommendation = "BUY"
            confidence = "Moderate to High"
        elif total_score >= 100:
            recommendation = "HOLD"
            confidence = "Moderate"
        elif total_score >= 70:
            recommendation = "SELL"
            confidence = "Moderate"
        else:
            recommendation = "STRONG SELL"
            confidence = "High"

        logger.info(f"Generated enhanced recommendation: {recommendation} (Total Score: {total_score:.2f})")

        return {
            'recommendation': recommendation,
            'confidence': confidence,
            'total_score': total_score,
            'breakdown': {
                'fundamental': fund_score,
                'technical': tech_score,
                'liquidity': liq_score,
                'sentiment': sentiment_score,
                'peer_adjustment': 5 if peer_analysis_available and peer_performance.get('outperformance', False) is True else (-5 if not peer_analysis_available else 0),
                'index_adjustment': 5 if isinstance(index_membership, list) and index_membership else 0
            }
        }


**Reasoning**:
The `get_peer_companies` function now returns a list of dictionaries including valuation data. Update the `analyze_stock_enhanced` function to store this enhanced peer data in the report.



In [ ]:
#@title Part 3: Enhanced Analysis Integration and Report Display (Updated Peer Data Handling)

# Assuming IndianStockAnalyzer and EnhancedStockAnalyzer classes are defined in previous cells

# Ensure logging is configured (assuming this was done in a previous cell)
import logging
logger = logging.getLogger(__name__)

# Integration function to work with your Part 1
def analyze_stock_enhanced(symbol, stock_analyzer_part1, enhanced_analyzer):
    """
    Integrate enhanced features with your Part 1 analyzer

    Args:
        symbol: Stock symbol (e.g., 'RELIANCE.NS')
        stock_analyzer_part1: Instance of your IndianStockAnalyzer from Part 1
        enhanced_analyzer: Instance of EnhancedStockAnalyzer
    """
    if not isinstance(symbol, str) or not symbol:
        logger.error("Invalid stock symbol provided for enhanced analysis.")
        return None
    if not isinstance(stock_analyzer_part1, IndianStockAnalyzer):
        logger.error("Invalid IndianStockAnalyzer (Part 1) instance provided.")
        return None
    if not isinstance(enhanced_analyzer, EnhancedStockAnalyzer):
        logger.error("Invalid EnhancedStockAnalyzer (Part 2) instance provided.")
        return None


    logger.info(f"Starting enhanced analysis for {symbol}")
    # Get basic analysis from Part 1
    stock_data = stock_analyzer_part1.analyze_stock(symbol)

    if not stock_data:
        logger.error(f"Basic analysis failed for {symbol}.")
        return None

    # Get current price - already validated in Part 1 analyze_stock
    current_price = stock_data.current_price
    if pd.isna(current_price) or current_price <= 0:
         logger.warning(f"Current price invalid ({current_price}). Enhanced analysis might be limited.")


    # Enhanced analysis
    logger.info("Starting enhanced analysis features (Part 2).")
    # 1. Circuit breaker analysis
    circuit_limits = enhanced_analyzer.get_circuit_limits(symbol)
    # Pass current_price to assess_circuit_risk (already validated as best as possible in Part 1)
    circuit_risk = enhanced_analyzer.assess_circuit_risk(current_price, circuit_limits)

    # 2. Enhanced liquidity analysis
    # Pass price_data (already validated in Part 1 analyze_stock)
    # CORRECTED: Call get_liquidity_score on enhanced_analyzer and pass price_data from stock_data
    liquidity_score = enhanced_analyzer.get_liquidity_score(stock_data.price_data)

    # 3. Peer analysis (including valuation data)
    # Pass info (already validated in Part 1 analyze_stock)
    # The get_peer_companies method now returns a list of dictionaries with valuation data
    peers_with_valuation = enhanced_analyzer.get_peer_companies(symbol, stock_data.info)
    # Pass the list of peer symbols (extracted from the list of dictionaries) for performance comparison
    peer_symbols_for_performance = [p['symbol'] for p in peers_with_valuation if isinstance(p, dict) and 'symbol' in p]
    peer_performance = enhanced_analyzer.calculate_peer_relative_performance(symbol, peer_symbols_for_performance) # Pass just symbols


    # 4. Index membership
    index_membership = enhanced_analyzer.get_index_membership(symbol)

    # 5. Recalculate technical score with circuit risk
    # Pass indicators (already validated in Part 1 calculate_technical_indicators)
    # Pass circuit_risk (already validated in assess_circuit_risk)
    enhanced_technical_score = enhanced_analyzer.calculate_enhanced_technical_score(
        stock_data.indicators, circuit_risk
    )

    # 6. Fetch and analyze news sentiment
    # Pass symbol (already validated)
    news_headlines = enhanced_analyzer.fetch_news(symbol)
    # Pass news_headlines (already validated)
    sentiment_analysis_results = enhanced_analyzer.analyze_sentiment(news_headlines)


    # 7. Generate enhanced recommendation
    # Pass scores and results (already validated within their respective functions)
    final_recommendation = enhanced_analyzer.generate_enhanced_recommendation(
        stock_data.fundamental_score,
        enhanced_technical_score,
        liquidity_score,
        peer_performance, # Pass potentially incomplete/NaN peer performance
        index_membership,
        sentiment_analysis_results # Pass sentiment analysis results
    )

    # Create enhanced report
    enhanced_report = {
        'basic_analysis': stock_data,
        'enhanced_features': {
            'circuit_limits': circuit_limits,
            'circuit_risk_score': circuit_risk,
            'liquidity_score': liquidity_score, # Ensure correct liquidity score is included
            'peers': peers_with_valuation, # Store the list of peers with their valuation data
            'peer_performance': peer_performance, # Include peer performance, even if NaN
            'index_membership': index_membership,
            'dynamic_risk_free_rate': enhanced_analyzer.risk_free_rate,
            'news_headlines': news_headlines, # Include news in report
            'sentiment_analysis': sentiment_analysis_results # Include sentiment analysis results
        },
        'final_recommendation': final_recommendation
    }
    logger.info(f"Enhanced analysis completed for {symbol}")
    return enhanced_report

# Example usage
if __name__ == "__main__":
    # Initialize analyzers
    try:
        stock_analyzer = IndianStockAnalyzer()
        enhanced_analyzer = EnhancedStockAnalyzer()
    except NameError:
        print("Make sure to run the previous cells defining IndianStockAnalyzer and EnhancedStockAnalyzer classes.")
        exit() # Exit if classes are not defined


    # Analyze a stock
    symbol = "RELIANCE.NS" # Example symbol
    result = analyze_stock_enhanced(symbol, stock_analyzer, enhanced_analyzer)

    if result:
        # Display the enhanced report
        enhanced = result['enhanced_features']
        final = result['final_recommendation']
        basic = result['basic_analysis'] # Get basic data for display

        print(f"\n{'='*60}")
        print(f"Enhanced Analysis Report: {basic.company_name} ({basic.symbol})")
        print(f"{'='*60}")

        print(f"\n📊 Current Market Data:")
        print(f"Current Price: ₹{basic.current_price:,.2f}" if not pd.isna(basic.current_price) else "Current Price: N/A")
        market_cap_cr = basic.market_cap / 10000000 if not pd.isna(basic.market_cap) else np.nan
        print(f"Market Cap: ₹{market_cap_cr:,.2f} Cr" if not pd.isna(market_cap_cr) else "Market Cap: N/A")
        print(f"Sector: {basic.info.get('sector', 'N/A')}")

        print(f"\n⚙️ Enhanced Technical Analysis & Risk:")
        # Display enhanced technical indicators and risk
        # Assuming you want to display relevant indicators from basic.indicators and the circuit risk score
        circuit_limits_display = enhanced.get('circuit_limits')
        if circuit_limits_display and not pd.isna(circuit_limits_display.get('lower_circuit')) and not pd.isna(circuit_limits_display.get('upper_circuit')):
             print(f"Circuit Limits: {circuit_limits_display.get('lower_circuit', 'N/A'):.2f} - {circuit_limits_display.get('upper_circuit', 'N/A'):.2f}")
        else:
             print("Circuit Limits: N/A")

        print(f"Circuit Risk Score: {enhanced.get('circuit_risk_score', 'N/A')}")
        # Check if liquidity score is available before formatting
        liquidity_score_display = f"{enhanced.get('liquidity_score', 'N/A')}/70" if isinstance(enhanced.get('liquidity_score'), (int, float)) and not pd.isna(enhanced.get('liquidity_score')) else "Liquidity Score: N/A/70"
        print(liquidity_score_display)


        # Display some key technical indicators from basic analysis
        print("\nKey Technical Indicators (from Basic Analysis):")
        key_tech_indicators = ['RSI', 'MACD', 'MACD_signal', 'SMA_20', 'SMA_50', 'BB_middle', 'Volume_ratio']
        for indicator_name in key_tech_indicators:
            value = basic.indicators.get(indicator_name, np.nan)
            if not pd.isna(value):
                if isinstance(value, float):
                     print(f"  - {indicator_name}: {value:,.2f}")
                else:
                     print(f"  - {indicator_name}: {value}")
            # Add logging for these key technical indicators if they are calculated
            logger.info(f"Indicator {indicator_name}: {value}")
        else:
            print(f"  - {indicator_name}: N/A")
            logger.info(f"Indicator {indicator_name}: N/A")


        print(f"\n🤝 Peer Analysis:")
        peers_with_valuation = enhanced.get('peers')
        if isinstance(peers_with_valuation, list) and peers_with_valuation:
             print("Peer Companies and Valuation Ratios:")
             for peer_data in peers_with_valuation:
                  if isinstance(peer_data, dict) and 'symbol' in peer_data:
                       peer_symbol = peer_data['symbol']
                       peer_pe = peer_data.get('trailingPE', np.nan)
                       peer_pb = peer_data.get('priceToBook', np.nan)
                       peer_ev_ebitda = peer_data.get('evToEbitda', np.nan)

                       pe_display = f"{peer_pe:.2f}" if not pd.isna(peer_pe) else "N/A"
                       pb_display = f"{peer_pb:.2f}" if not pd.isna(peer_pb) else "N/A"
                       ev_ebitda_display = f"{peer_ev_ebitda:.2f}" if not pd.isna(peer_ev_ebitda) else "N/A"

                       print(f"  - {peer_symbol}: PE={pe_display}, PB={pb_display}, EV/EBITDA={ev_ebitda_display}")
                  else:
                       logger.warning(f"Invalid peer data format in the list: {peer_data}")
                       print(f"  - Invalid Peer Data: {peer_data}")


             # Display peer performance metrics separately
             peer_perf = enhanced.get('peer_performance')
             if isinstance(peer_perf, dict) and \
                not pd.isna(peer_perf.get('stock_return')) and \
                not pd.isna(peer_perf.get('avg_peer_return')) and \
                not pd.isna(peer_perf.get('relative_performance')):

                  print("\nPeer Performance:")
                  stock_return_display = f"{peer_perf.get('stock_return', 'N/A'):.2f}%" if isinstance(peer_perf.get('stock_return'), (int, float)) and not pd.isna(peer_perf.get('stock_return')) else 'N/A'
                  avg_peer_return_display = f"{peer_perf.get('avg_peer_return', 'N/A'):.2f}%" if isinstance(peer_perf.get('avg_peer_return'), (int, float)) and not pd.isna(peer_perf.get('avg_peer_return')) else 'N/A'
                  relative_performance_display = f"{peer_perf.get('relative_performance', 'N/A'):.2f}%" if isinstance(peer_perf.get('relative_performance'), (int, float)) and not pd.isna(peer_perf.get('relative_performance')) else 'N/A'

                  print(f"  - {peer_perf.get('days', 'N/A')}-day Stock Return: {stock_return_display}")
                  print(f"  - Avg Peer Return: {avg_peer_return_display}")
                  print(f"  - Relative Performance: {relative_performance_display}")
             else:
                  print("Peer Performance Data Unavailable.")

        else:
             print("No peer companies identified or data unavailable.")


        if enhanced.get('index_membership'):
            print(f"\n🏛️ Index Membership: {', '.join(enhanced['index_membership'])}")

        # Sentiment Analysis Results
        sentiment_results = enhanced.get('sentiment_analysis', {})
        print(f"\n📰 News Sentiment Analysis:")
        if enhanced.get('news_headlines'):
             # Only print a few headlines to keep the report concise
             print("Sample News Headlines:")
             # Ensure headlines are valid strings before printing
             valid_headlines = [h for h in enhanced['news_headlines'] if isinstance(h, str)]
             for headline in valid_headlines[:5]: # Print up to 5 headlines
                 print(f"- {headline}")
             if len(valid_headlines) > 5:
                  print("  ...")

             print(f"Sentiment Counts: Positive={sentiment_results.get('positive', 0)}, Neutral={sentiment_results.get('neutral', 0)}, Negative={sentiment_results.get('negative', 0)}")
             sentiment_score_display = f"{sentiment_results.get('score', 0):.2f}" if isinstance(sentiment_results.get('score'), (int, float)) and not pd.isna(sentiment_results.get('score')) else "N/A"
             print(f"Sentiment Score: {sentiment_score_display}/25")
             print(f"Sentiment Summary: {sentiment_results.get('summary', 'N/A')}")
        else:
             print("No news headlines fetched or sentiment analysis skipped.")


        print(f"\n--- Comprehensive Scores and Recommendation ---")
        print(f"Fundamental Score (from Part 1): {basic.fundamental_score:.2f}/50" if not pd.isna(basic.fundamental_score) else "Fundamental Score: N/A")
        # Add checks for NaN before formatting float for breakdown scores
        enhanced_tech_score_display = f"{final.get('breakdown', {}).get('technical', 'N/A'):.2f}" if isinstance(final.get('breakdown', {}).get('technical'), (int, float)) and not pd.isna(final.get('breakdown', {}).get('technical')) else "N/A"
        liquidity_score_display_breakdown = f"{final.get('breakdown', {}).get('liquidity', 'N/A'):.2f}" if isinstance(final.get('breakdown', {}).get('liquidity'), (int, float)) and not pd.isna(final.get('breakdown', {}).get('liquidity')) else "N/A"
        sentiment_score_display_breakdown = f"{final.get('breakdown', {}).get('sentiment', 'N/A'):.2f}" if isinstance(sentiment_results.get('score'), (int, float)) and not pd.isna(sentiment_results.get('score')) else "N/A" # Corrected sentiment score breakdown check

        print(f"Technical Score (Enhanced): {enhanced_tech_score_display}/50")
        print(f"Liquidity Score: {liquidity_score_display_breakdown}/70")
        print(f"Sentiment Score: {sentiment_score_display_breakdown}/25")

        # Display adjustment scores
        peer_adj_display = f"{final.get('breakdown', {}).get('peer_adjustment', 'N/A'):+.2f}" if isinstance(final.get('breakdown', {}).get('peer_adjustment'), (int, float)) and not pd.isna(final.get('breakdown', {}).get('peer_adjustment')) else "N/A"
        index_adj_display = f"{final.get('breakdown', {}).get('index_adjustment', 'N/A'):+.2f}" if isinstance(final.get('breakdown', {}).get('index_adjustment'), (int, float)) and not pd.isna(final.get('breakdown', {}).get('index_adjustment')) else "N/A"

        print(f"Peer Adjustment: {peer_adj_display}")
        print(f"Index Membership Adjustment: {index_adj_display}")


        print(f"\nTotal Enhanced Score: {final.get('total_score', 0):.2f}/205" if isinstance(final.get('total_score'), (int, float)) and not pd.isna(final.get('total_score')) else "Total Enhanced Score: N/A")
        print(f"RECOMMENDATION: {final.get('recommendation', 'N/A')} (Confidence: {final.get('confidence', 'N/A')})")


        print(f"\n{'='*60}\n")

**Reasoning**:
Implement the calculation and comparison of the stock's valuation ratios (P/E, P/B, EV/EBITDA) against the average ratios of its identified peers. This involves iterating through the peer valuation data fetched in the previous step, calculating the average for each ratio, and then comparing the stock's ratios to these averages.



In [ ]:
#@title Part 2: EnhancedStockAnalyzer class definition (with Valuation Comparison Logic)

import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import requests
from bs4 import BeautifulSoup
import warnings
warnings.filterwarnings('ignore')

# Import typing for type hints
from typing import List, Dict, Optional

# Ensure logging is configured (assuming this was done in a previous cell)
import logging
logger = logging.getLogger(__name__)


# --- EnhancedStockAnalyzer class definition ---
class EnhancedStockAnalyzer:
    """Enhanced analyzer with additional features for Indian markets"""

    def __init__(self):
        """Initialize the enhanced analyzer."""
        self.risk_free_rate = self.get_dynamic_risk_free_rate()
        # Assuming gemini_model is defined globally from a previous cell
        global gemini_model # Access the global gemini_model
        self.gemini_model = gemini_model

        # More detailed circuit limit percentages based on NSE categorization (simplified)
        self.circuit_limits_percentage = {
            'category_a': 0.20, # Typically high-liquidity index stocks
            'category_b': 0.10, # Most other actively traded stocks
            'category_t': 0.05  # Stocks in trade-for-trade segment
        }

        # Simple mapping of some known stocks to categories for demonstration
        self.stock_category_map = {
            'RELIANCE.NS': 'category_a',
            'TCS.NS': 'category_a',
            'HDFCBANK.NS': 'category_a',
            'INFY.NS': 'category_a',
            'ICICIBANK.NS': 'category_a',
            'VEDL.NS': 'category_b',
            'PNB.NS': 'category_b',
            'INDIGO.NS': 'category_b',
            'BANDHANBNK.NS': 'category_b',
            'PERSISTENT.NS': 'category_b',
            'COFORGE.NS': 'category_b',
            'LTTS.NS': 'category_b',
        }


    def get_dynamic_risk_free_rate(self):
        """Fetch current 10-year G-Sec yield"""
        try:
            return 0.072  # 7.2% as of recent data
        except Exception as e:
            logger.warning(f"Could not fetch dynamic risk-free rate: {e}. Using fallback rate.")
            return 0.065

    def get_circuit_limits(self, symbol):
        """Get circuit breaker limits for the stock based on categorization"""
        try:
            ticker = yf.Ticker(symbol)
            info = ticker.info

            if not info or 'previousClose' not in info:
                 logger.warning(f"Could not get previous close for {symbol}. Cannot calculate circuit limits.")
                 return None

            prev_close = info.get('previousClose')
            if pd.isna(prev_close) or prev_close <= 0:
                 logger.warning(f"Invalid previous close price for {symbol}: {prev_close}. Cannot calculate circuit limits.")
                 return None


            stock_category = self.stock_category_map.get(symbol.upper(), 'category_b')
            circuit_percent = self.circuit_limits_percentage.get(stock_category, 0.10)

            logger.info(f"Determined circuit percentage for {symbol} (Category: {stock_category}): {circuit_percent*100}%")

            upper_circuit = prev_close * (1 + circuit_percent)
            lower_circuit = prev_close * (1 - circuit_percent)

            return {
                'upper_circuit': upper_circuit,
                'lower_circuit': lower_circuit,
                'circuit_percent': circuit_percent
            }
        except Exception as e:
            logger.error(f"Error getting circuit breaker limits for {symbol}: {str(e)}")
            return None

    def assess_circuit_risk(self, current_price, circuit_limits):
        """Assess proximity to circuit limits"""
        if pd.isna(current_price) or current_price <= 0:
             logger.warning("Invalid current price for circuit risk assessment.")
             return 0

        if not circuit_limits or 'upper_circuit' not in circuit_limits or 'lower_circuit' not in circuit_limits:
            logger.warning("Invalid or incomplete circuit limits provided.")
            return 0

        upper_circuit = circuit_limits['upper_circuit']
        lower_circuit = circuit_limits['lower_circuit']

        if pd.isna(upper_circuit) or pd.isna(lower_circuit) or upper_circuit <= 0 or lower_circuit <= 0:
             logger.warning("Circuit limits are invalid numbers. Cannot assess circuit risk.")
             return 0

        risk_score = 0

        if current_price >= upper_circuit:
             risk_score -= 30
        elif (upper_circuit - current_price) / current_price < 0.01:
            risk_score -= 20
        elif (upper_circuit - current_price) / current_price < 0.03:
            risk_score -= 10
        elif (upper_circuit - current_price) / current_price < 0.05:
            risk_score -= 5

        if current_price <= lower_circuit:
            risk_score -= 40
        elif (current_price - lower_circuit) / current_price < 0.01:
            risk_score -= 30
        elif (current_price - lower_circuit) / current_price < 0.03:
            risk_score -= 15
        elif (current_price - lower_circuit) / current_price < 0.05:
            risk_score -= 8

        logger.info(f"Circuit risk score calculated: {risk_score}")
        return risk_score

    def get_liquidity_score(self, price_data):
        """Enhanced liquidity analysis with NaN handling and better checks"""
        liquidity_score = 0
        if not isinstance(price_data, pd.DataFrame) or price_data.empty or len(price_data) < 20:
            logger.warning("Insufficient price data for liquidity analysis. Assigning default score.")
            return 30

        try:
            avg_volume_20d = price_data['Volume'].rolling(20).mean().iloc[-1]
            avg_volume_5d = price_data['Volume'].rolling(5).mean().iloc[-1]
            current_close = price_data['Close'].iloc[-1]

            # Add logging for diagnosis - before calculation check
            logger.info(f"Volume data for liquidity: avg_volume_20d: {avg_volume_20d}, avg_volume_5d: {avg_volume_5d}, current_close: {current_close}")


            if pd.isna(avg_volume_20d) or pd.isna(avg_volume_5d) or pd.isna(current_close) or current_close <= 0:
                 logger.warning("Liquidity metrics are NaN or invalid. Cannot calculate liquidity score accurately. Assigning default.")
                 return 30

            avg_turnover_20d = np.nan
            if avg_volume_20d is not None and not pd.isna(avg_volume_20d) and avg_volume_20d >= 0 and current_close > 0:
                 avg_turnover_20d = avg_volume_20d * current_close

            volume_spike = np.nan
            if avg_volume_20d is not None and not pd.isna(avg_volume_20d) and avg_volume_20d > 0:
                 if avg_volume_5d is not None and not pd.isna(avg_volume_5d):
                     volume_spike = avg_volume_5d / avg_volume_20d
                 else:
                      logger.warning("5-day average volume is invalid for volume spike calculation.")
            elif not pd.isna(avg_volume_5d) and avg_volume_5d > 0:
                 volume_spike = 1.0
                 logger.warning("20-day average volume is invalid for volume spike, using 1.0 as relative indicator.")
            else:
                 volume_spike = np.nan # Ensure volume_spike is NaN if both volumes are invalid
                 logger.warning("Insufficient or invalid volume data for volume ratio calculation.")

            # Add logging for volume spike - after calculation
            logger.info(f"Volume spike calculated: {volume_spike}")

            # Corrected logic for scoring based on volume_spike
            if not pd.isna(volume_spike):
                if volume_spike > 1.5: # Volume is 50% higher than average
                    liquidity_score += 10
                elif volume_spike < 0.8: # Volume is significantly lower than average
                    liquidity_score += 2 # Small penalty for low volume
                else: # volume_spike is between 0.8 and 1.5 (inclusive of 0.8, exclusive of 1.5)
                    liquidity_score += 5 # Average volume is neutral
            else:
                 logger.warning("Volume spike ratio is NaN. Skipping trend scoring.")


            if not pd.isna(avg_turnover_20d):
                if avg_turnover_20d > 500_00_00_000:
                    liquidity_score += 40
                elif avg_turnover_20d > 100_00_00_000:
                    liquidity_score += 30
                elif avg_turnover_20d > 10_00_00_000:
                    liquidity_score += 20
                elif avg_turnover_20d > 1_00_00_000:
                    liquidity_score += 10
                else:
                    liquidity_score += 5
            else:
                 logger.warning("Average turnover is NaN. Skipping turnover scoring.")
                 liquidity_score += 10

            volume_std = price_data['Volume'].rolling(20).std().iloc[-1] if len(price_data) >= 20 else np.nan
            volume_cv = np.nan
            if avg_volume_20d is not None and not pd.isna(avg_volume_20d) and avg_volume_20d > 0 and \
               volume_std is not None and not pd.isna(volume_std):
                volume_cv = volume_std / avg_volume_20d

            if not pd.isna(volume_cv):
                if volume_cv < 0.5:
                    liquidity_score += 20
                elif volume_cv < 1.0:
                    liquidity_score += 10
            else:
                 logger.warning("Volume coefficient of variation is NaN. Skipping consistency scoring.")

            # The volume spike scoring is now handled above within the main if not pd.isna(volume_spike): block


            return min(liquidity_score, 70)

        except Exception as e:
            logger.error(f"Error calculating liquidity score: {str(e)}")
            return 30

    def get_peer_companies(self, symbol: str, stock_info: Dict) -> List[Dict]: # Modified return type hint
        """Identify peer companies and fetch their valuation ratios for comparison"""
        logger.info(f"Identifying peer companies and fetching valuation data for {symbol}.")
        peer_list_with_valuation = [] # List to store peer info including valuation

        try:
            if not isinstance(stock_info, dict):
                 logger.warning("Invalid stock info for peer identification.")
                 return []

            sector = stock_info.get('sector', '')
            market_cap = stock_info.get('marketCap', 0)

            if not isinstance(sector, str):
                 sector = ''
            if not isinstance(market_cap, (int, float)) or pd.isna(market_cap) or market_cap <= 0:
                 logger.warning("Invalid market cap for peer identification.")
                 market_cap = 0

            if market_cap > 10_00_000_00_00_000:
                cap_category = 'Large Cap'
            elif market_cap > 50_000_00_00_000:
                cap_category = 'Mid Cap'
            else:
                cap_category = 'Small Cap'

            # Peer mapping (simplified - in production, use a comprehensive database)
            peer_map = {
                'Technology': {
                    'Large Cap': ['TCS.NS', 'INFY.NS', 'WIPRO.NS', 'HCLTECH.NS'],
                    'Mid Cap': ['TECHM.NS', 'LTTS.NS', 'PERSISTENT.NS', 'COFORGE.NS'],
                    'Small Cap': ['ZENSAR.NS', 'ECLERX.NS', 'MASTEK.NS']
                },
                'Financial Services': {
                    'Large Cap': ['HDFCBANK.NS', 'ICICIBANK.NS', 'KOTAKBANK.NS', 'AXISBANK.NS'],
                    'Mid Cap': ['IDFCFIRSTB.NS', 'FEDERALBNK.NS', 'INDUSINDBK.NS'],
                    'Small Cap': ['SOUTHBANK.NS', 'UJJIVAN.NS', 'EQUITASBNK.NS']
                },
                'Healthcare': {
                    'Large Cap': ['SUNPHARMA.NS', 'DRREDDY.NS', 'CIPLA.NS', 'DIVISLAB.NS'],
                    'Mid Cap': ['TORNTPHARM.NS', 'ALKEM.NS', 'LUPIN.NS'],
                    'Small Cap': ['CAPLIPOINT.NS', 'BLISS.NS', 'INDOCO.NS']
                },
                # Added Energy sector mapping
                'Energy': {
                    'Large Cap': ['ONGC.NS', 'NTPC.NS', 'POWERGRID.NS'], # Exclude RELIANCE.NS here, filter below
                    'Mid Cap': ['GAIL.NS', 'IOC.NS', 'NHPC.NS'],
                    'Small Cap': ['GUJGASLTD.NS', 'IGL.NS', 'MAHAPOWER.NS']
                }
            }

            potential_peers = peer_map.get(sector, {}).get(cap_category, [])
            # Ensure the current symbol is excluded from the peer list
            filtered_peers = [p for p in potential_peers if isinstance(p, str) and p.upper() != symbol.upper()]

            logger.info(f"Identified {len(filtered_peers)} potential peer companies for {symbol} in {sector} ({cap_category} Cap). Fetching valuation data.")

            # Fetch valuation data for each filtered peer
            for peer_symbol in filtered_peers:
                 try:
                      peer_ticker = yf.Ticker(peer_symbol)
                      peer_info = peer_ticker.info

                      if peer_info:
                           peer_valuation = {
                               'symbol': peer_symbol,
                               'trailingPE': peer_info.get('trailingPE', np.nan),
                               'priceToBook': peer_info.get('priceToBook', np.nan),
                               'enterpriseValue': peer_info.get('enterpriseValue', np.nan), # Fetch for EV/EBITDA calculation
                               'ebitda': peer_info.get('ebitda', np.nan) # Fetch for EV/EBITDA calculation
                           }

                           # Calculate EV/EBITDA for the peer
                           ev = peer_valuation['enterpriseValue']
                           ebitda = peer_valuation['ebitda']
                           peer_valuation['evToEbitda'] = np.nan # Default to NaN
                           if isinstance(ev, (int, float)) and not np.isnan(ev) and \
                              isinstance(ebitda, (int, float)) and not np.isnan(ebitda) and ebitda != 0:
                               peer_valuation['evToEbitda'] = ev / ebitda
                           elif (isinstance(ev, (int, float)) and not np.isnan(ev)) and \
                                (isinstance(ebitda, (int, float)) and not np.isnan(ebitda) and ebitda == 0):
                                logger.warning(f"EBITDA is zero for peer {peer_symbol}. Cannot calculate EV/EBITDA.")
                           else:
                                logger.warning(f"Missing Enterprise Value ({ev}) or EBITDA ({ebitda}) for peer {peer_symbol}. Cannot calculate EV/EBITDA.")


                           peer_list_with_valuation.append(peer_valuation)
                           logger.debug(f"Fetched valuation data for peer {peer_symbol}: PE={peer_valuation['trailingPE']}, PB={peer_valuation['priceToBook']}, EV/EBITDA={peer_valuation['evToEbitda']}")
                      else:
                           logger.warning(f"Could not fetch info for peer {peer_symbol}. Skipping.")
                 except Exception as e:
                      logger.warning(f"Error fetching data for peer {peer_symbol}: {str(e)}. Skipping.")

            logger.info(f"Successfully fetched valuation data for {len(peer_list_with_valuation)} peers.")
            return peer_list_with_valuation[:5] # Return top 5 peers with their valuation data

        except Exception as e:
            logger.error(f"Error getting peer companies and valuation data for {symbol}: {str(e)}")
            return [] # Return empty list on error

    def calculate_peer_average_valuation(self, peers_with_valuation: List[Dict]) -> Dict:
        """Calculate average valuation ratios (PE, PB, EV/EBITDA) for a list of peers."""
        logger.info("Calculating peer average valuation ratios.")
        total_pe = 0
        total_pb = 0
        total_ev_ebitda = 0
        count_pe = 0
        count_pb = 0
        count_ev_ebitda = 0

        if not isinstance(peers_with_valuation, list) or not peers_with_valuation:
             logger.warning("No peer valuation data provided for calculating averages.")
             return {'avg_pe': np.nan, 'avg_pb': np.nan, 'avg_ev_ebitda': np.nan}


        for peer_data in peers_with_valuation:
             if isinstance(peer_data, dict):
                 pe = peer_data.get('trailingPE', np.nan)
                 pb = peer_data.get('priceToBook', np.nan)
                 ev_ebitda = peer_data.get('evToEbitda', np.nan)

                 if isinstance(pe, (int, float)) and not np.isnan(pe) and pe > 0:
                     total_pe += pe
                     count_pe += 1
                 else:
                      logger.debug(f"Skipping invalid PE for peer: {peer_data.get('symbol', 'Unknown')}")

                 if isinstance(pb, (int, float)) and not np.isnan(pb) and pb > 0:
                     total_pb += pb
                     count_pb += 1
                 else:
                      logger.debug(f"Skipping invalid PB for peer: {peer_data.get('symbol', 'Unknown')}")


                 if isinstance(ev_ebitda, (int, float)) and not np.isnan(ev_ebitda): # EV/EBITDA can be zero or negative in some cases, but >0 is typical
                      total_ev_ebitda += ev_ebitda
                      count_ev_ebitda += 1
                 else:
                      logger.debug(f"Skipping invalid EV/EBITDA for peer: {peer_data.get('symbol', 'Unknown')}")


        avg_pe = total_pe / count_pe if count_pe > 0 else np.nan
        avg_pb = total_pb / count_pb if count_pb > 0 else np.nan
        avg_ev_ebitda = total_ev_ebitda / count_ev_ebitda if count_ev_ebitda > 0 else np.nan

        logger.info(f"Calculated peer averages: PE={avg_pe:.2f}, PB={avg_pb:.2f}, EV/EBITDA={avg_ev_ebitda:.2f}")

        return {
            'avg_pe': avg_pe,
            'avg_pb': avg_pb,
            'avg_ev_ebitda': avg_ev_ebitda
        }


    def compare_valuation_to_peers(self, stock_valuation: Dict, peer_average_valuation: Dict) -> Dict:
        """Compare stock's valuation ratios to peer averages and provide assessment."""
        logger.info("Comparing stock valuation to peer averages.")
        comparison_results = {}
        valuation_score_adjustment = 0 # Adjustment to the total score

        stock_pe = stock_valuation.get('trailingPE', np.nan)
        stock_pb = stock_valuation.get('priceToBook', np.nan)
        stock_ev_ebitda = stock_valuation.get('evToEbitda', np.nan)

        avg_pe = peer_average_valuation.get('avg_pe', np.nan)
        avg_pb = peer_average_valuation.get('avg_pb', np.nan)
        avg_ev_ebitda = peer_average_valuation.get('avg_ev_ebitda', np.nan)

        logger.debug(f"Stock Valuation: PE={stock_pe:.2f}, PB={stock_pb:.2f}, EV/EBITDA={stock_ev_ebitda:.2f}")
        logger.debug(f"Peer Average Valuation: PE={avg_pe:.2f}, PB={avg_pb:.2f}, EV/EBITDA={avg_ev_ebitda:.2f}")


        # PE Ratio comparison (Lower is generally better relative to peers)
        comparison_results['pe_comparison'] = 'N/A'
        if not pd.isna(stock_pe) and not pd.isna(avg_pe) and avg_pe > 0:
             if stock_pe < avg_pe * 0.8: # More than 20% below average
                  comparison_results['pe_comparison'] = 'Significantly Undervalued (PE)'
                  valuation_score_adjustment += 10
             elif stock_pe < avg_pe * 0.95: # More than 5% below average
                  comparison_results['pe_comparison'] = 'Undervalued (PE)'
                  valuation_score_adjustment += 5
             elif stock_pe > avg_pe * 1.2: # More than 20% above average
                  comparison_results['pe_comparison'] = 'Significantly Overvalued (PE)'
                  valuation_score_adjustment -= 10
             elif stock_pe > avg_pe * 1.05: # More than 5% above average
                  comparison_results['pe_comparison'] = 'Overvalued (PE)'
                  valuation_score_adjustment -= 5
             else:
                  comparison_results['pe_comparison'] = 'Fairly Valued (PE)'
        elif not pd.isna(stock_pe) and pd.isna(avg_pe):
             comparison_results['pe_comparison'] = 'PE available, Peer Avg PE N/A'
             logger.warning("Peer Average PE is NaN. Cannot compare stock PE to peers.")
        elif pd.isna(stock_pe) and not pd.isna(avg_pe):
             comparison_results['pe_comparison'] = 'Stock PE N/A'
             logger.warning("Stock PE is NaN. Cannot compare to peer Average PE.")
        else:
             logger.warning("Stock PE and Peer Average PE are NaN. Cannot compare.")


        # PB Ratio comparison (Lower is generally better relative to peers)
        comparison_results['pb_comparison'] = 'N/A'
        if not pd.isna(stock_pb) and not pd.isna(avg_pb) and avg_pb > 0:
             if stock_pb < avg_pb * 0.8: # More than 20% below average
                  comparison_results['pb_comparison'] = 'Significantly Undervalued (PB)'
                  valuation_score_adjustment += 8
             elif stock_pb < avg_pb * 0.95: # More than 5% below average
                  comparison_results['pb_comparison'] = 'Undervalued (PB)'
                  valuation_score_adjustment += 4
             elif stock_pb > avg_pb * 1.2: # More than 20% above average
                  comparison_results['pb_comparison'] = 'Significantly Overvalued (PB)'
                  valuation_score_adjustment -= 8
             elif stock_pb > avg_pb * 1.05: # More than 5% above average
                  comparison_results['pb_comparison'] = 'Overvalued (PB)'
                  valuation_score_adjustment -= 4
             else:
                  comparison_results['pb_comparison'] = 'Fairly Valued (PB)'
        elif not pd.isna(stock_pb) and pd.isna(avg_pb):
             comparison_results['pb_comparison'] = 'PB available, Peer Avg PB N/A'
             logger.warning("Peer Average PB is NaN. Cannot compare stock PB to peers.")
        elif pd.isna(stock_pb) and not pd.isna(avg_pb):
             comparison_results['pb_comparison'] = 'Stock PB N/A'
             logger.warning("Stock PB is NaN. Cannot compare to peer Average PB.")
        else:
             logger.warning("Stock PB and Peer Average PB are NaN. Cannot compare.")


        # EV/EBITDA comparison (Lower is generally better relative to peers)
        comparison_results['ev_ebitda_comparison'] = 'N/A'
        if not pd.isna(stock_ev_ebitda) and not pd.isna(avg_ev_ebitda) and avg_ev_ebitda != 0:
             if stock_ev_ebitda < avg_ev_ebitda * 0.8: # More than 20% below average
                  comparison_results['ev_ebitda_comparison'] = 'Significantly Undervalued (EV/EBITDA)'
                  valuation_score_adjustment += 12
             elif stock_ev_ebitda < avg_ev_ebitda * 0.95: # More than 5% below average
                  comparison_results['ev_ebitda_comparison'] = 'Undervalued (EV/EBITDA)'
                  valuation_score_adjustment += 6
             elif stock_ev_ebitda > avg_ev_ebitda * 1.2: # More than 20% above average
                  comparison_results['ev_ebitda_comparison'] = 'Significantly Overvalued (EV/EBITDA)'
                  valuation_score_adjustment -= 12
             elif stock_ev_ebitda > avg_ev_ebitda * 1.05: # More than 5% above average
                  comparison_results['ev_ebitda_comparison'] = 'Overvalued (EV/EBITDA)'
                  valuation_score_adjustment -= 6
             else:
                  comparison_results['ev_ebitda_comparison'] = 'Fairly Valued (EV/EBITDA)'
        elif not pd.isna(stock_ev_ebitda) and not pd.isna(avg_ev_ebitda) and avg_ev_ebitda == 0:
             comparison_results['ev_ebitda_comparison'] = 'Peer Avg EV/EBITDA is 0'
             logger.warning("Peer Average EV/EBITDA is zero. Cannot compare.")
        elif not pd.isna(stock_ev_ebitda) and pd.isna(avg_ev_ebitda):
             comparison_results['ev_ebitda_comparison'] = 'EV/EBITDA available, Peer Avg EV/EBITDA N/A'
             logger.warning("Peer Average EV/EBITDA is NaN. Cannot compare stock EV/EBITDA to peers.")
        elif pd.isna(stock_ev_ebitda) and not pd.isna(avg_ev_ebitda):
             comparison_results['ev_ebitda_comparison'] = 'Stock EV/EBITDA N/A'
             logger.warning("Stock EV/EBITDA is NaN. Cannot compare to peer Average EV/EBITDA.")
        else:
             logger.warning("Stock EV/EBITDA and Peer Average EV/EBITDA are NaN. Cannot compare.")


        comparison_results['valuation_score_adjustment'] = valuation_score_adjustment

        logger.info(f"Valuation comparison results: {comparison_results}")
        return comparison_results


    def calculate_peer_relative_performance(self, symbol, peers, days=30):
        """Compare stock performance with peers, handling data issues robustly"""
        results = {
            'stock_return': np.nan,
            'avg_peer_return': np.nan,
            'relative_performance': np.nan,
            'outperformance': False,
            'days': days
        }

        if not isinstance(peers, list) or not peers:
             logger.warning("No valid peer list provided for performance comparison.")
             # Return the default results with NaNs if no peers
             return results

        # --- Removed the temporary skip here ---

        try:
            main_ticker = yf.Ticker(symbol)
            main_hist = main_ticker.history(period=f"{days}d")
            main_return = np.nan

            if not main_hist.empty and len(main_hist) > 1:
                try:
                    start_price = main_hist['Close'].iloc[0]
                    end_price = main_hist['Close'].iloc[-1]
                    if not pd.isna(start_price) and not pd.isna(end_price) and start_price > 0:
                         main_return = ((end_price / start_price) - 1) * 100
                    else:
                         logger.warning(f"Invalid start ({start_price}) or end price ({end_price}) for main stock ({symbol}) return calculation.")
                except Exception as ret_e:
                     logger.warning(f"Could not calculate main stock ({symbol}) return: {ret_e}. Setting to NaN.")
                     main_return = np.nan
            else:
                logger.warning(f"Insufficient historical data for main stock ({symbol}) for return calculation.")

            results['stock_return'] = main_return

        except Exception as main_stock_e:
            logger.error(f"Error fetching data or calculating return for main stock ({symbol}): {main_stock_e}. Main stock return set to NaN.")
            results['stock_return'] = np.nan

        # Calculate peer returns
        peer_returns = []
        # The peers list now contains dictionaries with valuation data.
        # We need to extract just the symbols for performance calculation.
        peer_symbols_for_perf = [p['symbol'] for p in peers if isinstance(p, dict) and 'symbol' in p]

        for peer_symbol in peer_symbols_for_perf:
             try:
                 peer_ticker = yf.Ticker(peer_symbol)
                 peer_hist = peer_ticker.history(period=f"{days}d")
                 if not peer_hist.empty and len(peer_hist) > 1:
                     try:
                          peer_start_price = peer_hist['Close'].iloc[0]
                          peer_end_price = peer_hist['Close'].iloc[-1]
                          if not pd.isna(peer_start_price) and not pd.isna(peer_end_price) and peer_start_price > 0:
                             peer_return = ((peer_end_price / peer_start_price) - 1) * 100
                             peer_returns.append(peer_return)
                          else:
                             logger.warning(f"Invalid start or end price for peer ({peer_symbol}) return calculation.")
                     except Exception as peer_ret_e:
                         logger.warning(f"Could not calculate return for peer ({peer_symbol}): {peer_ret_e}. Skipping.")
                 else:
                     logger.warning(f"Insufficient historical data for peer ({peer_symbol}) for return calculation. Skipping.")
             except Exception as peer_e:
                 logger.warning(f"Error fetching data for peer ({peer_symbol}): {peer_e}. Skipping.")

        # Add logging for peer returns before filtering/average calculation
        logger.info(f"Peer returns collected before average calculation attempt: {peer_returns}")

        # Calculate average peer return and relative performance
        if peer_returns:
             # Ensure peer_returns list contains only valid numbers before calculating mean
             valid_peer_returns = [ret for ret in peer_returns if isinstance(ret, (int, float)) and not pd.isna(ret)]
             if valid_peer_returns:
                 avg_peer_return = np.mean(valid_peer_returns)
                 results['avg_peer_return'] = avg_peer_return

                 # Calculate relative performance only if main stock return is valid
                 if not pd.isna(main_return):
                      results['relative_performance'] = main_return - avg_peer_return

                      # Determine outperformance
                      if main_return > avg_peer_return:
                          results['outperformance'] = True
                 else:
                      logger.warning("Main stock return is NaN, cannot calculate relative performance or outperformance.")
             else:
                 logger.warning("No valid peer returns available after filtering. Cannot calculate average peer return or relative performance.")
        else:
             logger.warning("No peer returns calculated. Cannot calculate average peer return or relative performance.")


        logger.info(f"Peer performance results for {symbol}: {results}")
        return results


    def get_index_membership(self, symbol):
        """Check index membership for the stock"""
        symbol_upper = symbol.upper() if isinstance(symbol, str) else ''

        index_constituents = {
            'NIFTY50': ['RELIANCE.NS', 'TCS.NS', 'HDFCBANK.NS', 'INFY.NS', 'ICICIBANK.NS'],
            'NIFTY_NEXT50': ['VEDL.NS', 'PNB.NS', 'INDIGO.NS', 'BANDHANBNK.NS'],
            'NIFTY_MIDCAP150': ['PERSISTENT.NS', 'COFORGE.NS', 'LTTS.NS']
        }

        membership = []
        for index, constituents in index_constituents.items():
            if symbol_upper and any(isinstance(c, str) and c.upper() == symbol_upper for c in constituents):
                membership.append(index)

        if membership:
             logger.info(f"Stock {symbol} is a member of indices: {', '.join(membership)}")
        else:
             logger.info(f"Stock {symbol} is not found in the hardcoded index constituents.")

        return membership

    def calculate_enhanced_technical_score(self, indicators, circuit_risk):
        """Calculate enhanced technical analysis score with NaN handling and circuit risk"""
        base_score = 0
        indicators_calculated = 0
        possible_indicator_points = 50

        num_key_indicators = 5
        indicator_point_contribution = possible_indicator_points / num_key_indicators

        rsi = indicators.get('RSI', np.nan)
        if not pd.isna(rsi):
             indicators_calculated += 1
             if 40 <= rsi <= 60:
                 base_score += indicator_point_contribution * 0.7
             elif rsi < 30: # More points for extreme oversold
                 base_score += indicator_point_contribution * 1.0
             elif 30 <= rsi < 40: # Approaching oversold
                 base_score += indicator_point_contribution * 0.9
             elif 60 < rsi <= 70: # Approaching overbought
                 base_score += indicator_point_contribution * 0.6
             else: # Overbought (>70)
                 base_score += indicator_point_contribution * 0.4
        else:
             logger.warning("RSI not calculated or is NaN.")

        macd = indicators.get('MACD', np.nan)
        macd_signal = indicators.get('MACD_signal', np.nan)

        if not pd.isna(macd) and not pd.isna(macd_signal):
             indicators_calculated += 1
             if macd > macd_signal:
                 if macd > 0: # Bullish crossover above zero line
                      base_score += indicator_point_contribution * 1.0
                 else: # Bullish crossover below zero line
                      base_score += indicator_point_contribution * 0.8
             else:
                 if macd < 0: # Bearish crossover below zero line
                      base_score += indicator_point_contribution * 0.2
                 else: # Bearish crossover above zero line
                      base_score += indicator_point_contribution * 0.4
        else:
             logger.warning("MACD or MACD signal is NaN for technical scoring.")

        sma_20 = indicators.get('SMA_20', np.nan)
        sma_50 = indicators.get('SMA_50', np.nan)
        sma_200 = indicators.get('SMA_200', np.nan) # Include 200-day SMA if available

        ma_score_component = 0
        ma_indicators_counted = 0

        if not pd.isna(sma_20) and not pd.isna(sma_50):
             ma_indicators_counted += 1
             if sma_20 > sma_50:
                  ma_score_component += 0.5 # 20 > 50 crossover is bullish
             else:
                  ma_score_component += 0.2 # 20 <= 50 crossover is bearish/neutral

        if not pd.isna(sma_50) and not pd.isna(sma_200):
             ma_indicators_counted += 1
             if sma_50 > sma_200:
                  ma_score_component += 0.5 # 50 > 200 crossover is bullish
             else:
                  ma_score_component += 0.2 # 50 <= 200 crossover is bearish/neutral

        current_price = indicators.get('current_price', np.nan)
        if not pd.isna(current_price):
             if not pd.isna(sma_20) and current_price > sma_20:
                  ma_score_component += 0.3 # Price above 20-day SMA is bullish
             elif not pd.isna(sma_20):
                  ma_score_component += 0.1 # Price below 20-day SMA is bearish/neutral

             if not pd.isna(sma_50) and current_price > sma_50:
                  ma_score_component += 0.4 # Price above 50-day SMA is more bullish
             elif not pd.isna(sma_50):
                  ma_score_component += 0.15 # Price below 50-day SMA

             if not pd.isna(sma_200) and current_price > sma_200:
                  ma_score_component += 0.6 # Price above 200-day SMA is significant bullish signal
             elif not pd.isna(sma_200):
                  ma_score_component += 0.1 # Price below 200-day SMA is significant bearish signal


        if ma_indicators_counted > 0 or (not pd.isna(current_price) and (not pd.isna(sma_20) or not pd.isna(sma_50) or not pd.isna(sma_200))):
             indicators_calculated += 1 # Count MA section if any valid MA comparison/position is made
             # Normalize MA score component to contribute to the total score
             # Max possible ma_score_component (0.5 + 0.5 + 0.3 + 0.4 + 0.6) = 2.3
             # Let's scale this to contribute up to indicator_point_contribution
             max_ma_component = 2.3
             base_score += (ma_score_component / max_ma_component) * indicator_point_contribution if max_ma_component > 0 else 0
        else:
             logger.warning("Insufficient data for Moving Averages analysis.")


        bb_upper = indicators.get('BB_upper', np.nan)
        bb_lower = indicators.get('BB_lower', np.nan)
        bb_middle = indicators.get('BB_middle', np.nan)

        if not pd.isna(current_price) and not pd.isna(bb_upper) and not pd.isna(bb_lower) and not pd.isna(bb_middle):
            indicators_calculated += 1
            if current_price < bb_lower:
                base_score += indicator_point_contribution * 1.0 # Price below lower band (potential buy signal)
            elif bb_lower <= current_price < bb_middle:
                base_score += indicator_point_contribution * 0.8 # Between lower and middle band
            elif bb_middle <= current_price < bb_upper:
                base_score += indicator_point_contribution * 0.6 # Between middle and upper band
            else: # Price above upper band
                base_score += indicator_point_contribution * 0.4 # Potential sell signal
        else:
             logger.warning("Bollinger Bands or current price is NaN for technical scoring.")

        volume_ratio = indicators.get('Volume_ratio', np.nan)
        if not pd.isna(volume_ratio):
             indicators_calculated += 1
             if volume_ratio > 2.0: # Very high volume
                 base_score += indicator_point_contribution * 1.0
             elif volume_ratio > 1.2: # High volume
                 base_score += indicator_point_contribution * 0.8
             elif 0.8 <= volume_ratio <= 1.2: # Normal volume
                 base_score += indicator_point_contribution * 0.5
             else: # Low volume
                 base_score += indicator_point_contribution * 0.3
        else:
             logger.warning("Volume ratio is NaN for technical scoring.")


        achieved_score_from_indicators = base_score

        # Normalize the base score based on how many indicators were successfully calculated
        if indicators_calculated > 0:
            # Assuming each of the num_key_indicators contributes equally if available
            # This approach gives points for each indicator that could be calculated and scored
            # Max points if all indicators could be scored is num_key_indicators * indicator_point_contribution
            # But our scoring within each indicator gives variable points (e.g., 0.4 to 1.0)
            # A simpler approach is to just sum up the points from successfully scored indicators
            # Let's cap the sum at the max possible points from indicators (50)
            final_base_score = min(achieved_score_from_indicators, possible_indicator_points)

            if indicators_calculated < num_key_indicators:
                 logger.warning(f"Only {indicators_calculated}/{num_key_indicators} key technical indicators could be fully calculated/scored. Final base technical score is based on available data.")

        else:
             logger.warning("No key technical indicators calculated. Base technical score is 0.")
             final_base_score = 0


        circuit_risk_adj = circuit_risk if isinstance(circuit_risk, (int, float)) and not pd.isna(circuit_risk) else 0

        final_score = final_base_score + circuit_risk_adj

        return max(0, min(final_score, 50))


    def fetch_news(self, symbol: str) -> List[str]:
        """
        Fetches recent news headlines for the given stock symbol.
        NOTE: This is a placeholder. In a real application, you would integrate
        with a News API (like News API, Alpha Vantage, etc.) here.
        """
        print(f"Fetching news for {symbol} (placeholder)...")
        symbol_str = str(symbol) if symbol is not None else "Unknown Stock"
        return [
            f"{symbol_str} stock price rises on positive market sentiment",
            f"{symbol_str} announces strong quarterly results, beating estimates",
            f"Experts bullish on {symbol_str}'s future outlook",
            f"I am a non-numeric entry that might cause an error",
            f"{symbol_str} faces regulatory challenges in key market",
            f"Competitor's new product launch impacts {symbol_str}'s market share",
            f"Global market trends positively impacting {symbol_str}",
            f"{symbol_str} management provides optimistic guidance",
            f"Increased foreign investment flows into {symbol_str}",
            f"Regulatory approval received for {symbol_str}'s new product",
            f"Production issues reported for {symbol_str}",
            f"Increased competition puts pressure on {symbol_str}'s margins",
            f"Analyst downgrades rating for {symbol_str}",
            f"Supply chain disruptions affect {symbol_str}'s operations",
            f"Geopolitical tensions create uncertainty for {symbol_str}",
            f"{symbol_str} announces stock split",
            f"Dividend declared by {symbol_str}"
        ]


    def analyze_sentiment(self, news_headlines: List[str]) -> Dict:
        """
        Analyzes the sentiment of news headlines using the Gemini API.
        Returns a dictionary with sentiment counts (positive, neutral, negative)
        and a sentiment score.
        """
        if not self.gemini_model:
            print("Gemini API not configured. Skipping sentiment analysis.")
            return {'positive': 0, 'neutral': 0, 'negative': 0, 'score': 0, 'summary': 'Gemini API not configured. Sentiment analysis skipped.'}

        if not isinstance(news_headlines, list) or not news_headlines:
            print("No news headlines to analyze. Skipping sentiment analysis.")
            return {'positive': 0, 'neutral': 0, 'negative': 0, 'score': 12.5, 'summary': 'No news headlines. Sentiment analysis skipped.'}

        logger.info("Analyzing sentiment using Gemini API...")
        prompt = "Analyze the sentiment of the following news headlines for a stock and categorize each as Positive, Neutral, or Negative. Provide a concise summary of the overall sentiment.\n\n"
        for headline in news_headlines:
            if isinstance(headline, str):
                 prompt += f"- {headline}\n"
            else:
                 logger.warning(f"Skipping non-string headline: {headline}")
                 continue

        if len(prompt) > 30000:
             logger.warning("News headlines too long for Gemini API prompt. Truncating.")
             prompt = prompt[:30000] + "\n... (headlines truncated)"

        try:
            response = self.gemini_model.generate_content(prompt)
            sentiment_text = response.text.strip()
            logger.info(f"Gemini API Response: {sentiment_text[:200]}...")

            positive_count = sentiment_text.lower().count('positive')
            neutral_count = sentiment_text.lower().count('neutral')
            negative_count = sentiment_text.lower().count('negative')

            summary_line = "No summary extracted."
            lines = sentiment_text.split('\n')
            for line in lines:
                 lower_line = line.lower()
                 if 'summary' in lower_line or 'overall sentiment' in lower_line:
                     summary_line = line.strip()
                     break
            if summary_line == "No summary extracted." and lines:
                 for line in reversed(lines):
                     if line.strip():
                         summary_line = line.strip()
                         break

            total_headlines = len([h for h in news_headlines if isinstance(h, str)])
            if total_headlines > 0:
                sentiment_score = ((positive_count - negative_count) / total_headlines) * 12.5 + 12.5
            else:
                sentiment_score = 12.5

            return {
                'positive': positive_count,
                'neutral': neutral_count,
                'negative': negative_count,
                'score': max(0, min(sentiment_score, 25)),
                'summary': summary_line
            }

        except Exception as e:
            logger.error(f"Error analyzing sentiment with Gemini API: {e}. Skipping sentiment analysis.")
            return {'positive': 0, 'neutral': 0, 'negative': 0, 'score': 0, 'summary': 'Sentiment analysis failed.'}


    def generate_enhanced_recommendation(self, fundamental_score, technical_score,
                                       liquidity_score, peer_performance, index_membership,
                                       sentiment_analysis_results, valuation_comparison_results): # Added valuation_comparison_results
        """Generate comprehensive recommendation including sentiment and valuation comparison"""
        fund_score = fundamental_score if isinstance(fundamental_score, (int, float)) and not pd.isna(fundamental_score) else 0
        tech_score = technical_score if isinstance(technical_score, (int, float)) and not pd.isna(technical_score) else 0
        liq_score = liquidity_score if isinstance(liquidity_score, (int, float)) and not pd.isna(liquidity_score) else 0
        sentiment_score = sentiment_analysis_results.get('score', 0) if isinstance(sentiment_analysis_results.get('score'), (int, float)) and not pd.isna(sentiment_analysis_results.get('score')) else 0

        # Get valuation adjustment score
        valuation_adjustment = valuation_comparison_results.get('valuation_score_adjustment', 0) if isinstance(valuation_comparison_results.get('valuation_score_adjustment'), (int, float)) and not pd.isna(valuation_comparison_results.get('valuation_score_adjustment')) else 0
        logger.info(f"Valuation comparison adjustment: {valuation_adjustment}")

        total_score = fund_score + tech_score + liq_score + sentiment_score + valuation_adjustment # Add valuation adjustment


        peer_analysis_available = isinstance(peer_performance, dict) and \
                                  not (pd.isna(peer_performance.get('stock_return')) and \
                                       pd.isna(peer_performance.get('avg_peer_return')) and \
                                       pd.isna(peer_performance.get('relative_performance')))

        if peer_analysis_available and peer_performance.get('outperformance', False) is True:
            total_score += 5
            logger.info("Adding bonus for peer outperformance.")
        elif not peer_analysis_available:
             total_score -= 5
             logger.warning("Applying small penalty as peer performance data is incomplete/unavailable.")

        if isinstance(index_membership, list) and index_membership:
            total_score += 5
            logger.info("Adding bonus for index membership.")

        recommendation = "NEUTRAL"
        confidence = "Low"

        # Re-evaluate thresholds with the new max score (205 + max valuation adj + max peer perf + max index = 205 + 30 + 5 + 5 = 245 approx)
        # Adjusting thresholds based on potential max score increase
        if total_score >= 190: # Increased threshold
            recommendation = "STRONG BUY"
            confidence = "High"
        elif total_score >= 150: # Increased threshold
            recommendation = "BUY"
            confidence = "Moderate to High"
        elif total_score >= 100: # Same threshold
            recommendation = "HOLD"
            confidence = "Moderate"
        elif total_score >= 70: # Same threshold
            recommendation = "SELL"
            confidence = "Moderate"
        else:
            recommendation = "STRONG SELL"
            confidence = "High"

        logger.info(f"Generated enhanced recommendation: {recommendation} (Total Score: {total_score:.2f})")

        return {
            'recommendation': recommendation,
            'confidence': confidence,
            'total_score': total_score,
            'breakdown': {
                'fundamental': fund_score,
                'technical': tech_score,
                'liquidity': liq_score,
                'sentiment': sentiment_score,
                'valuation_comparison': valuation_adjustment, # Include valuation adjustment in breakdown
                'peer_adjustment': 5 if peer_analysis_available and peer_performance.get('outperformance', False) is True else (-5 if not peer_analysis_available else 0),
                'index_adjustment': 5 if isinstance(index_membership, list) and index_membership else 0
            }
        }


**Reasoning**:
Integrate the new valuation analysis logic into the `analyze_stock_enhanced` function to calculate peer average valuations and compare the stock's valuation, then update the report display to show these new valuation insights.



In [ ]:
#@title Part 3: Enhanced Analysis Integration and Report Display (with Valuation Analysis)

# Assuming IndianStockAnalyzer and EnhancedStockAnalyzer classes are defined in previous cells

# Ensure logging is configured (assuming this was done in a previous cell)
import logging
logger = logging.getLogger(__name__)

# Integration function to work with your Part 1
def analyze_stock_enhanced(symbol, stock_analyzer_part1, enhanced_analyzer):
    """
    Integrate enhanced features with your Part 1 analyzer

    Args:
        symbol: Stock symbol (e.g., 'RELIANCE.NS')
        stock_analyzer_part1: Instance of your IndianStockAnalyzer from Part 1
        enhanced_analyzer: Instance of EnhancedStockAnalyzer
    """
    if not isinstance(symbol, str) or not symbol:
        logger.error("Invalid stock symbol provided for enhanced analysis.")
        return None
    if not isinstance(stock_analyzer_part1, IndianStockAnalyzer):
        logger.error("Invalid IndianStockAnalyzer (Part 1) instance provided.")
        return None
    if not isinstance(enhanced_analyzer, EnhancedStockAnalyzer):
        logger.error("Invalid EnhancedStockAnalyzer (Part 2) instance provided.")
        return None


    logger.info(f"Starting enhanced analysis for {symbol}")
    # Get basic analysis from Part 1
    stock_data = stock_analyzer_part1.analyze_stock(symbol)

    if not stock_data:
        logger.error(f"Basic analysis failed for {symbol}.")
        return None

    # Get current price - already validated in Part 1 analyze_stock
    current_price = stock_data.current_price
    if pd.isna(current_price) or current_price <= 0:
         logger.warning(f"Current price invalid ({current_price}). Enhanced analysis might be limited.")


    # Enhanced analysis
    logger.info("Starting enhanced analysis features (Part 2).")
    # 1. Circuit breaker analysis
    circuit_limits = enhanced_analyzer.get_circuit_limits(symbol)
    # Pass current_price to assess_circuit_risk (already validated as best as possible in Part 1)
    circuit_risk = enhanced_analyzer.assess_circuit_risk(current_price, circuit_limits)

    # 2. Enhanced liquidity analysis
    # Pass price_data (already validated in Part 1 analyze_stock)
    # CORRECTED: Call get_liquidity_score on enhanced_analyzer and pass price_data from stock_data
    liquidity_score = enhanced_analyzer.get_liquidity_score(stock_data.price_data)

    # 3. Peer analysis (including valuation data)
    # Pass info (already validated in Part 1 analyze_stock)
    # The get_peer_companies method now returns a list of dictionaries with valuation data
    peers_with_valuation = enhanced_analyzer.get_peer_companies(symbol, stock_data.info)
    # Pass the list of peer symbols (extracted from the list of dictionaries) for performance comparison
    peer_symbols_for_performance = [p['symbol'] for p in peers_with_valuation if isinstance(p, dict) and 'symbol' in p]
    peer_performance = enhanced_analyzer.calculate_peer_relative_performance(symbol, peer_symbols_for_performance) # Pass just symbols

    # 4. Valuation Analysis: Calculate peer average valuations and compare
    peer_average_valuation = enhanced_analyzer.calculate_peer_average_valuation(peers_with_valuation)
    valuation_comparison_results = enhanced_analyzer.compare_valuation_to_peers(stock_data.metrics, peer_average_valuation)


    # 5. Index membership
    index_membership = enhanced_analyzer.get_index_membership(symbol)

    # 6. Recalculate technical score with circuit risk
    # Pass indicators (already validated in Part 1 calculate_technical_indicators)
    # Pass circuit_risk (already validated in assess_circuit_risk)
    enhanced_technical_score = enhanced_analyzer.calculate_enhanced_technical_score(
        stock_data.indicators, circuit_risk
    )

    # 7. Fetch and analyze news sentiment
    # Pass symbol (already validated)
    news_headlines = enhanced_analyzer.fetch_news(symbol)
    # Pass news_headlines (already validated)
    sentiment_analysis_results = enhanced_analyzer.analyze_sentiment(news_headlines)


    # 8. Generate enhanced recommendation
    # Pass scores and results (already validated within their respective functions)
    final_recommendation = enhanced_analyzer.generate_enhanced_recommendation(
        stock_data.fundamental_score,
        enhanced_technical_score,
        liquidity_score,
        peer_performance, # Pass potentially incomplete/NaN peer performance
        index_membership,
        sentiment_analysis_results, # Pass sentiment analysis results
        valuation_comparison_results # Pass valuation comparison results
    )

    # Create enhanced report
    enhanced_report = {
        'basic_analysis': stock_data,
        'enhanced_features': {
            'circuit_limits': circuit_limits,
            'circuit_risk_score': circuit_risk,
            'liquidity_score': liquidity_score, # Ensure correct liquidity score is included
            'peers': peers_with_valuation, # Store the list of peers with their valuation data
            'peer_performance': peer_performance, # Include peer performance, even if NaN
            'peer_average_valuation': peer_average_valuation, # Store peer average valuation
            'valuation_comparison': valuation_comparison_results, # Store valuation comparison results
            'index_membership': index_membership,
            'dynamic_risk_free_rate': enhanced_analyzer.risk_free_rate,
            'news_headlines': news_headlines, # Include news in report
            'sentiment_analysis': sentiment_analysis_results # Include sentiment analysis results
        },
        'final_recommendation': final_recommendation
    }
    logger.info(f"Enhanced analysis completed for {symbol}")
    return enhanced_report

# Example usage
if __name__ == "__main__":
    # Initialize analyzers
    try:
        stock_analyzer = IndianStockAnalyzer()
        enhanced_analyzer = EnhancedStockAnalyzer()
    except NameError:
        print("Make sure to run the previous cells defining IndianStockAnalyzer and EnhancedStockAnalyzer classes.")
        exit() # Exit if classes are not defined


    # Analyze a stock
    symbol = "RELIANCE.NS" # Example symbol
    result = analyze_stock_enhanced(symbol, stock_analyzer, enhanced_analyzer)

    if result:
        # Display the enhanced report
        enhanced = result['enhanced_features']
        final = result['final_recommendation']
        basic = result['basic_analysis'] # Get basic data for display

        print(f"\n{'='*60}")
        print(f"Enhanced Analysis Report: {basic.company_name} ({basic.symbol})")
        print(f"{'='*60}")

        print(f"\n📊 Current Market Data:")
        print(f"Current Price: ₹{basic.current_price:,.2f}" if not pd.isna(basic.current_price) else "Current Price: N/A")
        market_cap_cr = basic.market_cap / 10000000 if not pd.isna(basic.market_cap) else np.nan
        print(f"Market Cap: ₹{market_cap_cr:,.2f} Cr" if not pd.isna(market_cap_cr) else "Market Cap: N/A")
        print(f"Sector: {basic.info.get('sector', 'N/A')}")

        print(f"\n⚙️ Enhanced Technical Analysis & Risk:")
        # Display enhanced technical indicators and risk
        # Assuming you want to display relevant indicators from basic.indicators and the circuit risk score
        circuit_limits_display = enhanced.get('circuit_limits')
        if circuit_limits_display and not pd.isna(circuit_limits_display.get('lower_circuit')) and not pd.isna(circuit_limits_display.get('upper_circuit')):
             print(f"Circuit Limits: {circuit_limits_display.get('lower_circuit', 'N/A'):.2f} - {circuit_limits_display.get('upper_circuit', 'N/A'):.2f}")
        else:
             print("Circuit Limits: N/A")

        print(f"Circuit Risk Score: {enhanced.get('circuit_risk_score', 'N/A')}")
        # Check if liquidity score is available before formatting
        liquidity_score_display = f"{enhanced.get('liquidity_score', 'N/A')}/70" if isinstance(enhanced.get('liquidity_score'), (int, float)) and not pd.isna(enhanced.get('liquidity_score')) else "Liquidity Score: N/A/70"
        print(liquidity_score_display)


        # Display some key technical indicators from basic analysis
        print("\nKey Technical Indicators (from Basic Analysis):")
        key_tech_indicators = ['RSI', 'MACD', 'MACD_signal', 'SMA_20', 'SMA_50', 'BB_middle', 'Volume_ratio']
        for indicator_name in key_tech_indicators:
            value = basic.indicators.get(indicator_name, np.nan)
            if not pd.isna(value):
                if isinstance(value, float):
                     print(f"  - {indicator_name}: {value:,.2f}")
                else:
                     print(f"  - {indicator_name}: {value}")
            # Add logging for these key technical indicators if they are calculated
            logger.info(f"Indicator {indicator_name}: {value}")
        else:
            print(f"  - {indicator_name}: N/A")
            logger.info(f"Indicator {indicator_name}: N/A")

        # Display Valuation Analysis
        print(f"\n📈 Valuation Analysis:")
        stock_metrics = basic.metrics
        peer_avg_val = enhanced.get('peer_average_valuation', {})
        val_comp_results = enhanced.get('valuation_comparison', {})

        stock_pe = stock_metrics.get('trailingPE', np.nan)
        stock_pb = stock_metrics.get('priceToBook', np.nan)
        stock_ev_ebitda = stock_metrics.get('evToEbitda', np.nan)

        print(f"Stock Valuation:")
        print(f"  - Trailing PE: {stock_pe:.2f}" if not pd.isna(stock_pe) else "  - Trailing PE: N/A")
        print(f"  - Price to Book: {stock_pb:.2f}" if not pd.isna(stock_pb) else "  - Price to Book: N/A")
        print(f"  - EV/EBITDA: {stock_ev_ebitda:.2f}" if not pd.isna(stock_ev_ebitda) else "  - EV/EBITDA: N/A")

        print("\nPeer Average Valuation:")
        avg_pe_display = f"{peer_avg_val.get('avg_pe', np.nan):.2f}" if not pd.isna(peer_avg_val.get('avg_pe', np.nan)) else "N/A"
        avg_pb_display = f"{peer_avg_val.get('avg_pb', np.nan):.2f}" if not pd.isna(peer_avg_val.get('avg_pb', np.nan)) else "N/A"
        avg_ev_ebitda_display = f"{peer_avg_val.get('avg_ev_ebitda', np.nan):.2f}" if not pd.isna(peer_avg_val.get('avg_ev_ebitda', np.nan)) else "N/A"

        print(f"  - Average PE: {avg_pe_display}")
        print(f"  - Average PB: {avg_pb_display}")
        print(f"  - Average EV/EBITDA: {avg_ev_ebitda_display}")

        print("\nValuation Comparison:")
        print(f"  - PE Comparison: {val_comp_results.get('pe_comparison', 'N/A')}")
        print(f"  - PB Comparison: {val_comp_results.get('pb_comparison', 'N/A')}")
        print(f"  - EV/EBITDA Comparison: {val_comp_results.get('ev_ebitda_comparison', 'N/A')}")
        valuation_adj_display = f"{val_comp_results.get('valuation_score_adjustment', 0):+.2f}" if isinstance(val_comp_results.get('valuation_score_adjustment'), (int, float)) and not pd.isna(val_comp_results.get('valuation_score_adjustment')) else "N/A"
        print(f"  - Valuation Score Adjustment: {valuation_adj_display}")


        print(f"\n🤝 Peer Analysis (Performance):")
        peer_perf = enhanced.get('peer_performance')
        if isinstance(peer_perf, dict) and \
           not pd.isna(peer_perf.get('stock_return')) and \
           not pd.isna(peer_perf.get('avg_peer_return')) and \
           not pd.isna(peer_perf.get('relative_performance')):

             stock_return_display = f"{peer_perf.get('stock_return', np.nan):.2f}%" if isinstance(peer_perf.get('stock_return'), (int, float)) and not pd.isna(peer_perf.get('stock_return')) else 'N/A'
             avg_peer_return_display = f"{peer_perf.get('avg_peer_return', np.nan):.2f}%" if isinstance(peer_perf.get('avg_peer_return'), (int, float)) and not pd.isna(peer_perf.get('avg_peer_return')) else 'N/A'
             relative_performance_display = f"{peer_perf.get('relative_performance', np.nan):.2f}%" if isinstance(peer_perf.get('relative_performance'), (int, float)) and not pd.isna(peer_perf.get('relative_performance')) else 'N/A'

             print(f"  - {peer_perf.get('days', 'N/A')}-day Stock Return: {stock_return_display}")
             print(f"  - Avg Peer Return: {avg_peer_return_display}")
             print(f"  - Relative Performance: {relative_performance_display}")
        else:
             print("Peer Performance Data Unavailable.")


        if enhanced.get('index_membership'):
            print(f"\n🏛️ Index Membership: {', '.join(enhanced['index_membership'])}")

        # Sentiment Analysis Results
        sentiment_results = enhanced.get('sentiment_analysis', {})
        print(f"\n📰 News Sentiment Analysis:")
        if enhanced.get('news_headlines'):
             # Only print a few headlines to keep the report concise
             print("Sample News Headlines:")
             # Ensure headlines are valid strings before printing
             valid_headlines = [h for h in enhanced['news_headlines'] if isinstance(h, str)]
             for headline in valid_headlines[:5]: # Print up to 5 headlines
                 print(f"- {headline}")
             if len(valid_headlines) > 5:
                  print("  ...")

             print(f"Sentiment Counts: Positive={sentiment_results.get('positive', 0)}, Neutral={sentiment_results.get('neutral', 0)}, Negative={sentiment_results.get('negative', 0)}")
             sentiment_score_display = f"{sentiment_results.get('score', np.nan):.2f}" if isinstance(sentiment_results.get('score'), (int, float)) and not pd.isna(sentiment_results.get('score')) else "N/A"
             print(f"Sentiment Score: {sentiment_score_display}/25")
             print(f"Sentiment Summary: {sentiment_results.get('summary', 'N/A')}")
        else:
             print("No news headlines fetched or sentiment analysis skipped.")


        print(f"\n--- Comprehensive Scores and Recommendation ---")
        print(f"Fundamental Score (from Part 1): {basic.fundamental_score:.2f}/50" if not pd.isna(basic.fundamental_score) else "Fundamental Score: N/A")
        # Add checks for NaN before formatting float for breakdown scores
        enhanced_tech_score_display = f"{final.get('breakdown', {}).get('technical', np.nan):.2f}" if isinstance(final.get('breakdown', {}).get('technical'), (int, float)) and not pd.isna(final.get('breakdown', {}).get('technical')) else "N/A"
        liquidity_score_display_breakdown = f"{final.get('breakdown', {}).get('liquidity', np.nan):.2f}" if isinstance(final.get('breakdown', {}).get('liquidity'), (int, float)) and not pd.isna(final.get('breakdown', {}).get('liquidity')) else "N/A"
        sentiment_score_display_breakdown = f"{final.get('breakdown', {}).get('sentiment', np.nan):.2f}" if isinstance(sentiment_results.get('score'), (int, float)) and not pd.isna(sentiment_results.get('score')) else "N/A" # Corrected sentiment score breakdown check

        print(f"Technical Score (Enhanced): {enhanced_tech_score_display}/50")
        print(f"Liquidity Score: {liquidity_score_display_breakdown}/70")
        print(f"Sentiment Score: {sentiment_score_display_breakdown}/25")
        valuation_comp_display_breakdown = f"{final.get('breakdown', {}).get('valuation_comparison', np.nan):.2f}" if isinstance(final.get('breakdown', {}).get('valuation_comparison'), (int, float)) and not pd.isna(final.get('breakdown', {}).get('valuation_comparison')) else "N/A"
        print(f"Valuation Comparison Adjustment: {valuation_comp_display_breakdown}")


        # Display adjustment scores
        peer_adj_display = f"{final.get('breakdown', {}).get('peer_adjustment', np.nan):+.2f}" if isinstance(final.get('breakdown', {}).get('peer_adjustment'), (int, float)) and not pd.isna(final.get('breakdown', {}).get('peer_adjustment')) else "N/A"
        index_adj_display = f"{final.get('breakdown', {}).get('index_adjustment', np.nan):+.2f}" if isinstance(final.get('breakdown', {}).get('index_adjustment'), (int, float)) and not pd.isna(final.get('breakdown', {}).get('index_adjustment')) else "N/A"

        print(f"Peer Performance Adjustment: {peer_adj_display}")
        print(f"Index Membership Adjustment: {index_adj_display}")


        print(f"\nTotal Enhanced Score: {final.get('total_score', 0):.2f}/245" if isinstance(final.get('total_score'), (int, float)) and not pd.isna(final.get('total_score')) else "Total Enhanced Score: N/A") # Updated max score
        print(f"RECOMMENDATION: {final.get('recommendation', 'N/A')} (Confidence: {final.get('confidence', 'N/A')})")


        print(f"\n{'='*60}\n")

# Task
Continue with the stock analysis task by implementing downside risk assessment, enhancing dividend yield calculation, incorporating global macro impacts, integrating these new analyses into the report, and refining the scoring and recommendation.

## Implement downside risk assessment

### Subtask:
Implement downside risk assessment.


**Reasoning**:
Implement the downside risk assessment method within the `EnhancedStockAnalyzer` class, calculating Value at Risk (VaR) using historical data.



In [ ]:
#@title Part 2: EnhancedStockAnalyzer class definition (with Downside Risk Assessment)

import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import requests
from bs4 import BeautifulSoup
import warnings
warnings.filterwarnings('ignore')

# Import typing for type hints
from typing import List, Dict, Optional

# Ensure logging is configured (assuming this was done in a previous cell)
import logging
logger = logging.getLogger(__name__)


# --- EnhancedStockAnalyzer class definition ---
class EnhancedStockAnalyzer:
    """Enhanced analyzer with additional features for Indian markets"""

    def __init__(self):
        """Initialize the enhanced analyzer."""
        self.risk_free_rate = self.get_dynamic_risk_free_rate()
        # Assuming gemini_model is defined globally from a previous cell
        global gemini_model # Access the global gemini_model
        self.gemini_model = gemini_model

        # More detailed circuit limit percentages based on NSE categorization (simplified)
        self.circuit_limits_percentage = {
            'category_a': 0.20, # Typically high-liquidity index stocks
            'category_b': 0.10, # Most other actively traded stocks
            'category_t': 0.05  # Stocks in trade-for-trade segment
        }

        # Simple mapping of some known stocks to categories for demonstration
        self.stock_category_map = {
            'RELIANCE.NS': 'category_a',
            'TCS.NS': 'category_a',
            'HDFCBANK.NS': 'category_a',
            'INFY.NS': 'category_a',
            'ICICIBANK.NS': 'category_a',
            'VEDL.NS': 'category_b',
            'PNB.NS': 'category_b',
            'INDIGO.NS': 'category_b',
            'BANDHANBNK.NS': 'category_b',
            'PERSISTENT.NS': 'category_b',
            'COFORGE.NS': 'category_b',
            'LTTS.NS': 'category_b',
        }


    def get_dynamic_risk_free_rate(self):
        """Fetch current 10-year G-Sec yield"""
        try:
            return 0.072  # 7.2% as of recent data
        except Exception as e:
            logger.warning(f"Could not fetch dynamic risk-free rate: {e}. Using fallback rate.")
            return 0.065

    def get_circuit_limits(self, symbol):
        """Get circuit breaker limits for the stock based on categorization"""
        try:
            ticker = yf.Ticker(symbol)
            info = ticker.info

            if not info or 'previousClose' not in info:
                 logger.warning(f"Could not get previous close for {symbol}. Cannot calculate circuit limits.")
                 return None

            prev_close = info.get('previousClose')
            if pd.isna(prev_close) or prev_close <= 0:
                 logger.warning(f"Invalid previous close price for {symbol}: {prev_close}. Cannot calculate circuit limits.")
                 return None


            stock_category = self.stock_category_map.get(symbol.upper(), 'category_b')
            circuit_percent = self.circuit_limits_percentage.get(stock_category, 0.10)

            logger.info(f"Determined circuit percentage for {symbol} (Category: {stock_category}): {circuit_percent*100}%")

            upper_circuit = prev_close * (1 + circuit_percent)
            lower_circuit = prev_close * (1 - circuit_percent)

            return {
                'upper_circuit': upper_circuit,
                'lower_circuit': lower_circuit,
                'circuit_percent': circuit_percent
            }
        except Exception as e:
            logger.error(f"Error getting circuit breaker limits for {symbol}: {str(e)}")
            return None

    def assess_circuit_risk(self, current_price, circuit_limits):
        """Assess proximity to circuit limits"""
        if pd.isna(current_price) or current_price <= 0:
             logger.warning("Invalid current price for circuit risk assessment.")
             return 0

        if not circuit_limits or 'upper_circuit' not in circuit_limits or 'lower_circuit' not in circuit_limits:
            logger.warning("Invalid or incomplete circuit limits provided.")
            return 0

        upper_circuit = circuit_limits['upper_circuit']
        lower_circuit = circuit_limits['lower_circuit']

        if pd.isna(upper_circuit) or pd.isna(lower_circuit) or upper_circuit <= 0 or lower_circuit <= 0:
             logger.warning("Circuit limits are invalid numbers. Cannot assess circuit risk.")
             return 0

        risk_score = 0

        if current_price >= upper_circuit:
             risk_score -= 30
        elif (upper_circuit - current_price) / current_price < 0.01:
            risk_score -= 20
        elif (upper_circuit - current_price) / current_price < 0.03:
            risk_score -= 10
        elif (upper_circuit - current_price) / current_price < 0.05:
            risk_score -= 5

        if current_price <= lower_circuit:
            risk_score -= 40
        elif (current_price - lower_circuit) / current_price < 0.01:
            risk_score -= 30
        elif (current_price - lower_circuit) / current_price < 0.03:
            risk_score -= 15
        elif (current_price - lower_circuit) / current_price < 0.05:
            risk_score -= 8

        logger.info(f"Circuit risk score calculated: {risk_score}")
        return risk_score

    def get_liquidity_score(self, price_data):
        """Enhanced liquidity analysis with NaN handling and better checks"""
        liquidity_score = 0
        if not isinstance(price_data, pd.DataFrame) or price_data.empty or len(price_data) < 20:
            logger.warning("Insufficient price data for liquidity analysis. Assigning default score.")
            return 30

        try:
            avg_volume_20d = price_data['Volume'].rolling(20).mean().iloc[-1]
            avg_volume_5d = price_data['Volume'].rolling(5).mean().iloc[-1]
            current_close = price_data['Close'].iloc[-1]

            # Add logging for diagnosis - before calculation check
            logger.info(f"Volume data for liquidity: avg_volume_20d: {avg_volume_20d}, avg_volume_5d: {avg_volume_5d}, current_close: {current_close}")


            if pd.isna(avg_volume_20d) or pd.isna(avg_volume_5d) or pd.isna(current_close) or current_close <= 0:
                 logger.warning("Liquidity metrics are NaN or invalid. Cannot calculate liquidity score accurately. Assigning default.")
                 return 30

            avg_turnover_20d = np.nan
            if avg_volume_20d is not None and not pd.isna(avg_volume_20d) and avg_volume_20d >= 0 and current_close > 0:
                 avg_turnover_20d = avg_volume_20d * current_close

            volume_spike = np.nan
            if avg_volume_20d is not None and not pd.isna(avg_volume_20d) and avg_volume_20d > 0:
                 if avg_volume_5d is not None and not pd.isna(avg_volume_5d):
                     volume_spike = avg_volume_5d / avg_volume_20d
                 else:
                      logger.warning("5-day average volume is invalid for volume spike calculation.")
            elif not pd.isna(avg_volume_5d) and avg_volume_5d > 0:
                 volume_spike = 1.0
                 logger.warning("20-day average volume is invalid for volume spike, using 1.0 as relative indicator.")
            else:
                 volume_spike = np.nan # Ensure volume_spike is NaN if both volumes are invalid
                 logger.warning("Insufficient or invalid volume data for volume ratio calculation.")

            # Add logging for volume spike - after calculation
            logger.info(f"Volume spike calculated: {volume_spike}")

            # Corrected logic for scoring based on volume_spike
            if not pd.isna(volume_spike):
                if volume_spike > 1.5: # Volume is 50% higher than average
                    liquidity_score += 10
                elif volume_spike < 0.8: # Volume is significantly lower than average
                    liquidity_score += 2 # Small penalty for low volume
                else: # volume_spike is between 0.8 and 1.5 (inclusive of 0.8, exclusive of 1.5)
                    liquidity_score += 5 # Average volume is neutral
            else:
                 logger.warning("Volume spike ratio is NaN. Skipping trend scoring.")


            if not pd.isna(avg_turnover_20d):
                if avg_turnover_20d > 500_00_00_000:
                    liquidity_score += 40
                elif avg_turnover_20d > 100_00_00_000:
                    liquidity_score += 30
                elif avg_turnover_20d > 10_00_00_000:
                    liquidity_score += 20
                elif avg_turnover_20d > 1_00_00_000:
                    liquidity_score += 10
                else:
                    liquidity_score += 5
            else:
                 logger.warning("Average turnover is NaN. Skipping turnover scoring.")
                 liquidity_score += 10

            volume_std = price_data['Volume'].rolling(20).std().iloc[-1] if len(price_data) >= 20 else np.nan
            volume_cv = np.nan
            if avg_volume_20d is not None and not pd.isna(avg_volume_20d) and avg_volume_20d > 0 and \
               volume_std is not None and not pd.isna(volume_std):
                volume_cv = volume_std / avg_volume_20d

            if not pd.isna(volume_cv):
                if volume_cv < 0.5:
                    liquidity_score += 20
                elif volume_cv < 1.0:
                    liquidity_score += 10
            else:
                 logger.warning("Volume coefficient of variation is NaN. Skipping consistency scoring.")

            # The volume spike scoring is now handled above within the main if not pd.isna(volume_spike): block


            return min(liquidity_score, 70)

        except Exception as e:
            logger.error(f"Error calculating liquidity score: {str(e)}")
            return 30

    def get_peer_companies(self, symbol: str, stock_info: Dict) -> List[Dict]: # Modified return type hint
        """Identify peer companies and fetch their valuation ratios for comparison"""
        logger.info(f"Identifying peer companies and fetching valuation data for {symbol}.")
        peer_list_with_valuation = [] # List to store peer info including valuation

        try:
            if not isinstance(stock_info, dict):
                 logger.warning("Invalid stock info for peer identification.")
                 return []

            sector = stock_info.get('sector', '')
            market_cap = stock_info.get('marketCap', 0)

            if not isinstance(sector, str):
                 sector = ''
            if not isinstance(market_cap, (int, float)) or pd.isna(market_cap) or market_cap <= 0:
                 logger.warning("Invalid market cap for peer identification.")
                 market_cap = 0

            if market_cap > 10_00_000_00_00_000:
                cap_category = 'Large Cap'
            elif market_cap > 50_000_00_00_000:
                cap_category = 'Mid Cap'
            else:
                cap_category = 'Small Cap'

            # Peer mapping (simplified - in production, use a comprehensive database)
            peer_map = {
                'Technology': {
                    'Large Cap': ['TCS.NS', 'INFY.NS', 'WIPRO.NS', 'HCLTECH.NS'],
                    'Mid Cap': ['TECHM.NS', 'LTTS.NS', 'PERSISTENT.NS', 'COFORGE.NS'],
                    'Small Cap': ['ZENSAR.NS', 'ECLERX.NS', 'MASTEK.NS']
                },
                'Financial Services': {
                    'Large Cap': ['HDFCBANK.NS', 'ICICIBANK.NS', 'KOTAKBANK.NS', 'AXISBANK.NS'],
                    'Mid Cap': ['IDFCFIRSTB.NS', 'FEDERALBNK.NS', 'INDUSINDBK.NS'],
                    'Small Cap': ['SOUTHBANK.NS', 'UJJIVAN.NS', 'EQUITASBNK.NS']
                },
                'Healthcare': {
                    'Large Cap': ['SUNPHARMA.NS', 'DRREDDY.NS', 'CIPLA.NS', 'DIVISLAB.NS'],
                    'Mid Cap': ['TORNTPHARM.NS', 'ALKEM.NS', 'LUPIN.NS'],
                    'Small Cap': ['CAPLIPOINT.NS', 'BLISS.NS', 'INDOCO.NS']
                },
                # Added Energy sector mapping
                'Energy': {
                    'Large Cap': ['ONGC.NS', 'NTPC.NS', 'POWERGRID.NS'], # Exclude RELIANCE.NS here, filter below
                    'Mid Cap': ['GAIL.NS', 'IOC.NS', 'NHPC.NS'],
                    'Small Cap': ['GUJGASLTD.NS', 'IGL.NS', 'MAHAPOWER.NS']
                }
            }

            potential_peers = peer_map.get(sector, {}).get(cap_category, [])
            # Ensure the current symbol is excluded from the peer list
            filtered_peers = [p for p in potential_peers if isinstance(p, str) and p.upper() != symbol.upper()]

            logger.info(f"Identified {len(filtered_peers)} potential peer companies for {symbol} in {sector} ({cap_category} Cap). Fetching valuation data.")

            # Fetch valuation data for each filtered peer
            for peer_symbol in filtered_peers:
                 try:
                      peer_ticker = yf.Ticker(peer_symbol)
                      peer_info = peer_ticker.info

                      if peer_info:
                           peer_valuation = {
                               'symbol': peer_symbol,
                               'trailingPE': peer_info.get('trailingPE', np.nan),
                               'priceToBook': peer_info.get('priceToBook', np.nan),
                               'enterpriseValue': peer_info.get('enterpriseValue', np.nan), # Fetch for EV/EBITDA calculation
                               'ebitda': peer_info.get('ebitda', np.nan) # Fetch for EV/EBITDA calculation
                           }

                           # Calculate EV/EBITDA for the peer
                           ev = peer_valuation['enterpriseValue']
                           ebitda = peer_valuation['ebitda']
                           peer_valuation['evToEbitda'] = np.nan # Default to NaN
                           if isinstance(ev, (int, float)) and not np.isnan(ev) and \
                              isinstance(ebitda, (int, float)) and not np.isnan(ebitda) and ebitda != 0:
                               peer_valuation['evToEbitda'] = ev / ebitda
                           elif (isinstance(ev, (int, float)) and not np.isnan(ev)) and \
                                (isinstance(ebitda, (int, float)) and not np.isnan(ebitda) and ebitda == 0):
                                logger.warning(f"EBITDA is zero for peer {peer_symbol}. Cannot calculate EV/EBITDA.")
                           else:
                                logger.warning(f"Missing Enterprise Value ({ev}) or EBITDA ({ebitda}) for peer {peer_symbol}. Cannot calculate EV/EBITDA.")


                           peer_list_with_valuation.append(peer_valuation)
                           logger.debug(f"Fetched valuation data for peer {peer_symbol}: PE={peer_valuation['trailingPE']:.2f}, PB={peer_valuation['priceToBook']:.2f}, EV/EBITDA={peer_valuation['evToEBITDA']:.2f}")
                      else:
                           logger.warning(f"Could not fetch info for peer {peer_symbol}. Skipping.")
                 except Exception as e:
                      logger.warning(f"Error fetching data for peer {peer_symbol}: {str(e)}. Skipping.")

            logger.info(f"Successfully fetched valuation data for {len(peer_list_with_valuation)} peers.")
            return peer_list_with_valuation[:5] # Return top 5 peers with their valuation data

        except Exception as e:
            logger.error(f"Error getting peer companies and valuation data for {symbol}: {str(e)}")
            return [] # Return empty list on error

    def calculate_peer_average_valuation(self, peers_with_valuation: List[Dict]) -> Dict:
        """Calculate average valuation ratios (PE, PB, EV/EBITDA) for a list of peers."""
        logger.info("Calculating peer average valuation ratios.")
        total_pe = 0
        total_pb = 0
        total_ev_ebitda = 0
        count_pe = 0
        count_pb = 0
        count_ev_ebitda = 0

        if not isinstance(peers_with_valuation, list) or not peers_with_valuation:
             logger.warning("No peer valuation data provided for calculating averages.")
             return {'avg_pe': np.nan, 'avg_pb': np.nan, 'avg_ev_ebitda': np.nan}


        for peer_data in peers_with_valuation:
             if isinstance(peer_data, dict):
                 pe = peer_data.get('trailingPE', np.nan)
                 pb = peer_data.get('priceToBook', np.nan)
                 ev_ebitda = peer_data.get('evToEbitda', np.nan)

                 if isinstance(pe, (int, float)) and not np.isnan(pe) and pe > 0:
                     total_pe += pe
                     count_pe += 1
                 else:
                      logger.debug(f"Skipping invalid PE for peer: {peer_data.get('symbol', 'Unknown')}")

                 if isinstance(pb, (int, float)) and not np.isnan(pb) and pb > 0:
                     total_pb += pb
                     count_pb += 1
                 else:
                      logger.debug(f"Skipping invalid PB for peer: {peer_data.get('symbol', 'Unknown')}")


                 if isinstance(ev_ebitda, (int, float)) and not np.isnan(ev_ebitda): # EV/EBITDA can be zero or negative in some cases, but >0 is typical
                      total_ev_ebitda += ev_ebitda
                      count_ev_ebitda += 1
                 else:
                      logger.debug(f"Skipping invalid EV/EBITDA for peer: {peer_data.get('symbol', 'Unknown')}")


        avg_pe = total_pe / count_pe if count_pe > 0 else np.nan
        avg_pb = total_pb / count_pb if count_pb > 0 else np.nan
        avg_ev_ebitda = total_ev_ebitda / count_ev_ebitda if count_ev_ebitda > 0 else np.nan

        logger.info(f"Calculated peer averages: PE={avg_pe:.2f}, PB={avg_pb:.2f}, EV/EBITDA={avg_ev_ebitda:.2f}")

        return {
            'avg_pe': avg_pe,
            'avg_pb': avg_pb,
            'avg_ev_ebitda': avg_ev_ebitda
        }


    def compare_valuation_to_peers(self, stock_valuation: Dict, peer_average_valuation: Dict) -> Dict:
        """Compare stock's valuation ratios to peer averages and provide assessment."""
        logger.info("Comparing stock valuation to peer averages.")
        comparison_results = {}
        valuation_score_adjustment = 0 # Adjustment to the total score

        stock_pe = stock_valuation.get('trailingPE', np.nan)
        stock_pb = stock_valuation.get('priceToBook', np.nan)
        stock_ev_ebitda = stock_valuation.get('evToEbitda', np.nan)

        avg_pe = peer_average_valuation.get('avg_pe', np.nan)
        avg_pb = peer_average_valuation.get('avg_pb', np.nan)
        avg_ev_ebitda = peer_average_valuation.get('avg_ev_ebitda', np.nan)

        logger.debug(f"Stock Valuation: PE={stock_pe:.2f}, PB={stock_pb:.2f}, EV/EBITDA={stock_ev_ebitda:.2f}")
        logger.debug(f"Peer Average Valuation: PE={avg_pe:.2f}, PB={avg_pb:.2f}, EV/EBITDA={avg_ev_ebitda:.2f}")


        # PE Ratio comparison (Lower is generally better relative to peers)
        comparison_results['pe_comparison'] = 'N/A'
        if not pd.isna(stock_pe) and not pd.isna(avg_pe) and avg_pe > 0:
             if stock_pe < avg_pe * 0.8: # More than 20% below average
                  comparison_results['pe_comparison'] = 'Significantly Undervalued (PE)'
                  valuation_score_adjustment += 10
             elif stock_pe < avg_pe * 0.95: # More than 5% below average
                  comparison_results['pe_comparison'] = 'Undervalued (PE)'
                  valuation_score_adjustment += 5
             elif stock_pe > avg_pe * 1.2: # More than 20% above average
                  comparison_results['pe_comparison'] = 'Significantly Overvalued (PE)'
                  valuation_score_adjustment -= 10
             elif stock_pe > avg_pe * 1.05: # More than 5% above average
                  comparison_results['pe_comparison'] = 'Overvalued (PE)'
                  valuation_score_adjustment -= 5
             else:
                  comparison_results['pe_comparison'] = 'Fairly Valued (PE)'
        elif not pd.isna(stock_pe) and pd.isna(avg_pe):
             comparison_results['pe_comparison'] = 'PE available, Peer Avg PE N/A'
             logger.warning("Peer Average PE is NaN. Cannot compare stock PE to peers.")
        elif pd.isna(stock_pe) and not pd.isna(avg_pe):
             comparison_results['pe_comparison'] = 'Stock PE N/A'
             logger.warning("Stock PE is NaN. Cannot compare to peer Average PE.")
        else:
             logger.warning("Stock PE and Peer Average PE are NaN. Cannot compare.")


        # PB Ratio comparison (Lower is generally better relative to peers)
        comparison_results['pb_comparison'] = 'N/A'
        if not pd.isna(stock_pb) and not pd.isna(avg_pb) and avg_pb > 0:
             if stock_pb < avg_pb * 0.8: # More than 20% below average
                  comparison_results['pb_comparison'] = 'Significantly Undervalued (PB)'
                  valuation_score_adjustment += 8
             elif stock_pb < avg_pb * 0.95: # More than 5% below average
                  comparison_results['pb_comparison'] = 'Undervalued (PB)'
                  valuation_score_adjustment += 4
             elif stock_pb > avg_pb * 1.2: # More than 20% above average
                  comparison_results['pb_comparison'] = 'Significantly Overvalued (PB)'
                  valuation_score_adjustment -= 8
             elif stock_pb > avg_pb * 1.05: # More than 5% above average
                  comparison_results['pb_comparison'] = 'Overvalued (PB)'
                  valuation_score_adjustment -= 4
             else:
                  comparison_results['pb_comparison'] = 'Fairly Valued (PB)'
        elif not pd.isna(stock_pb) and pd.isna(avg_pb):
             comparison_results['pb_comparison'] = 'PB available, Peer Avg PB N/A'
             logger.warning("Peer Average PB is NaN. Cannot compare stock PB to peers.")
        elif pd.isna(stock_pb) and not pd.isna(avg_pb):
             comparison_results['pb_comparison'] = 'Stock PB N/A'
             logger.warning("Stock PB is NaN. Cannot compare to peer Average PB.")
        else:
             logger.warning("Stock PB and Peer Average PB are NaN. Cannot compare.")


        # EV/EBITDA comparison (Lower is generally better relative to peers)
        comparison_results['ev_ebitda_comparison'] = 'N/A'
        if not pd.isna(stock_ev_ebitda) and not pd.isna(avg_ev_ebitda) and avg_ev_ebitda != 0:
             if stock_ev_ebitda < avg_ev_ebitda * 0.8: # More than 20% below average
                  comparison_results['ev_ebitda_comparison'] = 'Significantly Undervalued (EV/EBITDA)'
                  valuation_score_adjustment += 12
             elif stock_ev_ebitda < avg_ev_ebitda * 0.95: # More than 5% below average
                  comparison_results['ev_ebitda_comparison'] = 'Undervalued (EV/EBITDA)'
                  valuation_score_adjustment += 6
             elif stock_ev_ebitda > avg_ev_ebitda * 1.2: # More than 20% above average
                  comparison_results['ev_ebitda_comparison'] = 'Significantly Overvalued (EV/EBITDA)'
                  valuation_score_adjustment -= 12
             elif stock_ev_ebitda > avg_ev_ebitda * 1.05: # More than 5% above average
                  comparison_results['ev_ebitda_comparison'] = 'Overvalued (EV/EBITDA)'
                  valuation_score_adjustment -= 6
             else:
                  comparison_results['ev_ebitda_comparison'] = 'Fairly Valued (EV/EBITDA)'
        elif not pd.isna(stock_ev_ebitda) and not pd.isna(avg_ev_ebitda) and avg_ev_ebitda == 0:
             comparison_results['ev_ebitda_comparison'] = 'Peer Avg EV/EBITDA is 0'
             logger.warning("Peer Average EV/EBITDA is zero. Cannot compare.")
        elif not pd.isna(stock_ev_ebitda) and pd.isna(avg_ev_ebitda):
             comparison_results['ev_ebitda_comparison'] = 'EV/EBITDA available, Peer Avg EV/EBITDA N/A'
             logger.warning("Peer Average EV/EBITDA is NaN. Cannot compare stock EV/EBITDA to peers.")
        elif pd.isna(stock_ev_ebitda) and not pd.isna(avg_ev_ebitda):
             comparison_results['ev_ebitda_comparison'] = 'Stock EV/EBITDA N/A'
             logger.warning("Stock EV/EBITDA is NaN. Cannot compare to peer Average EV/EBITDA.")
        else:
             logger.warning("Stock EV/EBITDA and Peer Average EV/EBITDA are NaN. Cannot compare.")


        comparison_results['valuation_score_adjustment'] = valuation_score_adjustment

        logger.info(f"Valuation comparison results: {comparison_results}")
        return comparison_results


    def calculate_peer_relative_performance(self, symbol, peers, days=30):
        """Compare stock performance with peers, handling data issues robustly"""
        results = {
            'stock_return': np.nan,
            'avg_peer_return': np.nan,
            'relative_performance': np.nan,
            'outperformance': False,
            'days': days
        }

        if not isinstance(peers, list) or not peers:
             logger.warning("No valid peer list provided for performance comparison.")
             # Return the default results with NaNs if no peers
             return results

        # --- Removed the temporary skip here ---

        try:
            main_ticker = yf.Ticker(symbol)
            main_hist = main_ticker.history(period=f"{days}d")
            main_return = np.nan

            if not main_hist.empty and len(main_hist) > 1:
                try:
                    start_price = main_hist['Close'].iloc[0]
                    end_price = main_hist['Close'].iloc[-1]
                    if not pd.isna(start_price) and not pd.isna(end_price) and start_price > 0:
                         main_return = ((end_price / start_price) - 1) * 100
                    else:
                         logger.warning(f"Invalid start ({start_price}) or end price ({end_price}) for main stock ({symbol}) return calculation.")
                except Exception as ret_e:
                     logger.warning(f"Could not calculate main stock ({symbol}) return: {ret_e}. Setting to NaN.")
                     main_return = np.nan
            else:
                logger.warning(f"Insufficient historical data for main stock ({symbol}) for return calculation.")

            results['stock_return'] = main_return

        except Exception as main_stock_e:
            logger.error(f"Error fetching data or calculating return for main stock ({symbol}): {main_stock_e}. Main stock return set to NaN.")
            results['stock_return'] = np.nan

        # Calculate peer returns
        peer_returns = []
        # The peers list now contains dictionaries with valuation data.
        # We need to extract just the symbols for performance calculation.
        peer_symbols_for_perf = [p['symbol'] for p in peers if isinstance(p, dict) and 'symbol' in p]

        for peer_symbol in peer_symbols_for_perf:
             try:
                 peer_ticker = yf.Ticker(peer_symbol)
                 peer_hist = peer_ticker.history(period=f"{days}d")
                 if not peer_hist.empty and len(peer_hist) > 1:
                     try:
                          peer_start_price = peer_hist['Close'].iloc[0]
                          peer_end_price = peer_hist['Close'].iloc[-1]
                          if not pd.isna(peer_start_price) and not pd.isna(peer_end_price) and peer_start_price > 0:
                             peer_return = ((peer_end_price / peer_start_price) - 1) * 100
                             peer_returns.append(peer_return)
                          else:
                             logger.warning(f"Invalid start or end price for peer ({peer_symbol}) return calculation.")
                     except Exception as peer_ret_e:
                         logger.warning(f"Could not calculate return for peer ({peer_symbol}): {peer_ret_e}. Skipping.")
                 else:
                     logger.warning(f"Insufficient historical data for peer ({peer_symbol}) for return calculation. Skipping.")
             except Exception as peer_e:
                 logger.warning(f"Error fetching data for peer ({peer_symbol}): {peer_e}. Skipping.")

        # Add logging for peer returns before filtering/average calculation
        logger.info(f"Peer returns collected before average calculation attempt: {peer_returns}")

        # Calculate average peer return and relative performance
        if peer_returns:
             # Ensure peer_returns list contains only valid numbers before calculating mean
             valid_peer_returns = [ret for ret in peer_returns if isinstance(ret, (int, float)) and not pd.isna(ret)]
             if valid_peer_returns:
                 avg_peer_return = np.mean(valid_peer_returns)
                 results['avg_peer_return'] = avg_peer_return

                 # Calculate relative performance only if main stock return is valid
                 if not pd.isna(main_return):
                      results['relative_performance'] = main_return - avg_peer_return

                      # Determine outperformance
                      if main_return > avg_peer_return:
                          results['outperformance'] = True
                 else:
                      logger.warning("Main stock return is NaN, cannot calculate relative performance or outperformance.")
             else:
                 logger.warning("No valid peer returns available after filtering. Cannot calculate average peer return or relative performance.")
        else:
             logger.warning("No peer returns calculated. Cannot calculate average peer return or relative performance.")


        logger.info(f"Peer performance results for {symbol}: {results}")
        return results


    def get_index_membership(self, symbol):
        """Check index membership for the stock"""
        symbol_upper = symbol.upper() if isinstance(symbol, str) else ''

        index_constituents = {
            'NIFTY50': ['RELIANCE.NS', 'TCS.NS', 'HDFCBANK.NS', 'INFY.NS', 'ICICIBANK.NS'],
            'NIFTY_NEXT50': ['VEDL.NS', 'PNB.NS', 'INDIGO.NS', 'BANDHANBNK.NS'],
            'NIFTY_MIDCAP150': ['PERSISTENT.NS', 'COFORGE.NS', 'LTTS.NS']
        }

        membership = []
        for index, constituents in index_constituents.items():
            if symbol_upper and any(isinstance(c, str) and c.upper() == symbol_upper for c in constituents):
                membership.append(index)

        if membership:
             logger.info(f"Stock {symbol} is a member of indices: {', '.join(membership)}")
        else:
             logger.info(f"Stock {symbol} is not found in the hardcoded index constituents.")

        return membership

    def calculate_enhanced_technical_score(self, indicators, circuit_risk):
        """Calculate enhanced technical analysis score with NaN handling and circuit risk"""
        base_score = 0
        indicators_calculated = 0
        possible_indicator_points = 50

        num_key_indicators = 5
        indicator_point_contribution = possible_indicator_points / num_key_indicators

        rsi = indicators.get('RSI', np.nan)
        if not pd.isna(rsi):
             indicators_calculated += 1
             if 40 <= rsi <= 60:
                 base_score += indicator_point_contribution * 0.7
             elif rsi < 30: # More points for extreme oversold
                 base_score += indicator_point_contribution * 1.0
             elif 30 <= rsi < 40: # Approaching oversold
                 base_score += indicator_point_contribution * 0.9
             elif 60 < rsi <= 70: # Approaching overbought
                 base_score += indicator_point_contribution * 0.6
             else: # Overbought (>70)
                 base_score += indicator_point_contribution * 0.4
        else:
             logger.warning("RSI not calculated or is NaN.")

        macd = indicators.get('MACD', np.nan)
        macd_signal = indicators.get('MACD_signal', np.nan)

        if not pd.isna(macd) and not pd.isna(macd_signal):
             indicators_calculated += 1
             if macd > macd_signal:
                 if macd > 0: # Bullish crossover above zero line
                      base_score += indicator_point_contribution * 1.0
                 else: # Bullish crossover below zero line
                      base_score += indicator_point_contribution * 0.8
             else:
                 if macd < 0: # Bearish crossover below zero line
                      base_score += indicator_point_contribution * 0.2
                 else: # Bearish crossover above zero line
                      base_score += indicator_point_contribution * 0.4
        else:
             logger.warning("MACD or MACD signal is NaN for technical scoring.")

        sma_20 = indicators.get('SMA_20', np.nan)
        sma_50 = indicators.get('SMA_50', np.nan)
        sma_200 = indicators.get('SMA_200', np.nan) # Include 200-day SMA if available

        ma_score_component = 0
        ma_indicators_counted = 0

        if not pd.isna(sma_20) and not pd.isna(sma_50):
             ma_indicators_counted += 1
             if sma_20 > sma_50:
                  ma_score_component += 0.5 # 20 > 50 crossover is bullish
             else:
                  ma_score_component += 0.2 # 20 <= 50 crossover is bearish/neutral

        if not pd.isna(sma_50) and not pd.isna(sma_200):
             ma_indicators_counted += 1
             if sma_50 > sma_200:
                  ma_score_component += 0.5 # 50 > 200 crossover is bullish
             else:
                  ma_score_component += 0.2 # 50 <= 200 crossover is bearish/neutral

        current_price = indicators.get('current_price', np.nan)
        if not pd.isna(current_price):
             if not pd.isna(sma_20) and current_price > sma_20:
                  ma_score_component += 0.3 # Price above 20-day SMA is bullish
             elif not pd.isna(sma_20):
                  ma_score_component += 0.1 # Price below 20-day SMA is bearish/neutral

             if not pd.isna(sma_50) and current_price > sma_50:
                  ma_score_component += 0.4 # Price above 50-day SMA is more bullish
             elif not pd.isna(sma_50):
                  ma_score_component += 0.15 # Price below 50-day SMA

             if not pd.isna(sma_200) and current_price > sma_200:
                  ma_score_component += 0.6 # Price above 200-day SMA is significant bullish signal
             elif not pd.isna(sma_200):
                  ma_score_component += 0.1 # Price below 200-day SMA is significant bearish signal


        if ma_indicators_counted > 0 or (not pd.isna(current_price) and (not pd.isna(sma_20) or not pd.isna(sma_50) or not pd.isna(sma_200))):
             indicators_calculated += 1 # Count MA section if any valid MA comparison/position is made
             # Normalize MA score component to contribute to the total score
             # Max possible ma_score_component (0.5 + 0.5 + 0.3 + 0.4 + 0.6) = 2.3
             # Let's scale this to contribute up to indicator_point_contribution
             max_ma_component = 2.3
             base_score += (ma_score_component / max_ma_component) * indicator_point_contribution if max_ma_component > 0 else 0
        else:
             logger.warning("Insufficient data for Moving Averages analysis.")


        bb_upper = indicators.get('BB_upper', np.nan)
        bb_lower = indicators.get('BB_lower', np.nan)
        bb_middle = indicators.get('BB_middle', np.nan)

        if not pd.isna(current_price) and not pd.isna(bb_upper) and not pd.isna(bb_lower) and not pd.isna(bb_middle):
            indicators_calculated += 1
            if current_price < bb_lower:
                base_score += indicator_point_contribution * 1.0 # Price below lower band (potential buy signal)
            elif bb_lower <= current_price < bb_middle:
                base_score += indicator_point_contribution * 0.8 # Between lower and middle band
            elif bb_middle <= current_price < bb_upper:
                base_score += indicator_point_contribution * 0.6 # Between middle and upper band
            else: # Price above upper band
                base_score += indicator_point_contribution * 0.4 # Potential sell signal
        else:
             logger.warning("Bollinger Bands or current price is NaN for technical scoring.")

        volume_ratio = indicators.get('Volume_ratio', np.nan)
        if not pd.isna(volume_ratio):
             indicators_calculated += 1
             if volume_ratio > 2.0: # Very high volume
                 base_score += indicator_point_contribution * 1.0
             elif volume_ratio > 1.2: # High volume
                 base_score += indicator_point_contribution * 0.8
             elif 0.8 <= volume_ratio <= 1.2: # Normal volume
                 base_score += indicator_point_contribution * 0.5
             else: # Low volume
                 base_score += indicator_point_contribution * 0.3
        else:
             logger.warning("Volume ratio is NaN for technical scoring.")


        achieved_score_from_indicators = base_score

        # Normalize the base score based on how many indicators were successfully calculated
        if indicators_calculated > 0:
            # Assuming each of the num_key_indicators contributes equally if available
            # This approach gives points for each indicator that could be calculated and scored
            # Max points if all indicators could be scored is num_key_indicators * indicator_point_contribution
            # But our scoring within each indicator gives variable points (e.g., 0.4 to 1.0)
            # A simpler approach is to just sum up the points from successfully scored indicators
            # Let's cap the sum at the max possible points from indicators (50)
            final_base_score = min(achieved_score_from_indicators, possible_indicator_points)

            if indicators_calculated < num_key_indicators:
                 logger.warning(f"Only {indicators_calculated}/{num_key_indicators} key technical indicators could be fully calculated/scored. Final base technical score is based on available data.")

        else:
             logger.warning("No key technical indicators calculated. Base technical score is 0.")
             final_base_score = 0


        circuit_risk_adj = circuit_risk if isinstance(circuit_risk, (int, float)) and not pd.isna(circuit_risk) else 0

        final_score = final_base_score + circuit_risk_adj

        return max(0, min(final_score, 50))


    def calculate_value_at_risk(self, price_data: pd.DataFrame, confidence_level: float = 0.95, horizon_days: int = 1) -> Dict: # Modified return type hint
        """
        Calculates Value at Risk (VaR) using the historical method.

        Args:
            price_data: DataFrame with historical price data (must contain 'Close').
            confidence_level: The confidence level for VaR (e.g., 0.95 for 95% VaR).
            horizon_days: The time horizon for VaR (in trading days).

        Returns:
            A dictionary containing the calculated VaR as a percentage loss,
            confidence level, and horizon days, or a dictionary with NaN/None values
            if calculation fails.
        """
        logger.info(f"Calculating {confidence_level*100:.0f}% VaR for {horizon_days}-day horizon.")
        results = {'var_percentage_loss': np.nan, 'confidence_level': confidence_level, 'horizon_days': horizon_days}

        if not isinstance(price_data, pd.DataFrame) or price_data.empty or 'Close' not in price_data.columns:
            logger.warning("Insufficient or invalid price data for VaR calculation.")
            return results # Return dictionary with NaN/None


        try:
            # Calculate daily returns
            returns = price_data['Close'].pct_change().dropna()

            if returns.empty:
                logger.warning("No valid returns data for VaR calculation.")
                return results # Return dictionary with NaN/None

            # Historical VaR: Find the percentile of returns corresponding to the confidence level
            # For downside risk, we look at the lower tail of returns.
            # A 95% confidence level for VaR means we're interested in the 5th percentile of losses.
            # The percentile rank for a confidence level C is 1 - C.
            percentile_rank = (1 - confidence_level) * 100
            logger.debug(f"Calculating {percentile_rank:.2f} percentile of historical returns.")

            # Calculate VaR at the specified percentile
            # The result is a negative percentage loss
            var_percentage = np.percentile(returns, percentile_rank)

            # VaR is typically expressed as a positive loss
            var_percentage_loss = abs(var_percentage)

            # Scale VaR for the desired horizon (assuming returns are independently and identically distributed)
            # This is a simplification; more advanced methods exist for scaling VaR.
            # Scaling by sqrt(horizon) is common but assumes specific return properties.
            scaled_var_percentage_loss = var_percentage_loss * np.sqrt(horizon_days)

            results['var_percentage_loss'] = scaled_var_percentage_loss * 100 # Convert to percentage for display/scoring
            logger.info(f"Calculated {confidence_level*100:.0f}% VaR ({horizon_days}-day horizon): {results['var_percentage_loss']:.2f}%")
            return results

        except Exception as e:
            logger.error(f"Error calculating VaR: {str(e)}. Returning default results.")
            return results # Return dictionary with NaN/None on error


    def calculate_enhanced_dividend_score(self, symbol: str, price_data: pd.DataFrame) -> Dict:
        """
        Calculates an enhanced dividend score considering historical payout trends and stability.

        Args:
            symbol: Stock symbol (e.g., 'RELIANCE.NS').
            price_data: DataFrame with historical price data (must contain 'Close').

        Returns:
            A dictionary containing the enhanced dividend score and potentially other
            relevant dividend metrics (e.g., payout history, yield trend).
        """
        logger.info(f"Calculating enhanced dividend score for {symbol}.")
        enhanced_dividend_score = 0 # Max score for dividend analysis out of 15 (example)
        dividend_metrics = {
            'trailingAnnualDividendYield': np.nan,
            'dividendPayoutRatio': np.nan,
            'payout_history': {},
            'yield_trend': 'N/A',
            'score': enhanced_dividend_score
        }

        if not isinstance(price_data, pd.DataFrame) or price_data.empty or 'Close' not in price_data.columns:
             logger.warning("Insufficient price data for enhanced dividend analysis.")
             return dividend_metrics # Return default metrics if price data is missing


        try:
            ticker = yf.Ticker(symbol)
            info = ticker.info
            dividends = ticker.dividends.sort_index() # Get historical dividend payouts

            # Get current yield and payout ratio from info
            current_yield = info.get('trailingAnnualDividendYield', np.nan) # yfinance provides this as a ratio
            if isinstance(current_yield, (int, float)) and not pd.isna(current_yield):
                 dividend_metrics['trailingAnnualDividendYield'] = current_yield * 100 # Convert to percentage
            else:
                 dividend_metrics['trailingAnnualDividendYield'] = np.nan


            payout_ratio = info.get('payoutRatio', np.nan) # yfinance provides this as a ratio
            if isinstance(payout_ratio, (int, float)) and not pd.isna(payout_ratio):
                 dividend_metrics['dividendPayoutRatio'] = payout_ratio # Keep as ratio
            else:
                 dividend_metrics['dividendPayoutRatio'] = np.nan


            if not dividends.empty:
                 # Analyze payout history and stability
                 dividend_metrics['payout_history'] = dividends.to_dict() # Store the raw history

                 # Check for consistent payouts (e.g., increasing or stable over last 5 years)
                 five_years_ago = datetime.now() - timedelta(days=5*365)
                 # Remove timezone information for consistent comparison
                 recent_dividends = dividends[dividends.index.tz_convert(None) >= five_years_ago]

                 if len(recent_dividends) > 3: # Need at least a few recent payouts
                      # Check for increasing trend
                      is_increasing = all(recent_dividends.iloc[i] >= recent_dividends.iloc[i-1] for i in range(1, len(recent_dividends)))

                      if is_increasing:
                           enhanced_dividend_score += 10 # Significant points for increasing dividends
                           dividend_metrics['yield_trend'] = 'Increasing'
                           logger.debug("Dividend trend: Increasing. +10 pts.")
                      elif len(recent_dividends) > 0 and recent_dividends.min() > 0:
                           enhanced_dividend_score += 5 # Some points for stable (non-zero) dividends
                           dividend_metrics['yield_trend'] = 'Stable'
                           logger.debug("Dividend trend: Stable. +5 pts.")
                      else:
                           dividend_metrics['yield_trend'] = 'Irregular/Decreasing'
                           logger.debug("Dividend trend: Irregular/Decreasing.")
                 elif len(recent_dividends) > 0 and recent_dividends.min() > 0:
                      enhanced_dividend_score += 3 # Some points for recent payouts, even if history is short
                      dividend_metrics['yield_trend'] = 'Recent Payouts'
                      logger.debug("Dividend trend: Recent Payouts. +3 pts.")
                 else:
                      dividend_metrics['yield_trend'] = 'No Recent Payouts'
                      logger.debug("Dividend trend: No Recent Payouts.")


                 # Score based on current yield (Similar to basic analysis but potentially more weight)
                 if not pd.isna(dividend_metrics['trailingAnnualDividendYield']):
                      yield_percentage = dividend_metrics['trailingAnnualDividendYield']
                      if yield_percentage > 3: # > 3%
                           enhanced_dividend_score += 5
                           logger.debug(f"Current Yield ({yield_percentage:.2f}%) > 3%. +5 pts.")
                      elif yield_percentage > 1: # > 1%
                           enhanced_dividend_score += 3
                           logger.debug(f"Current Yield ({yield_percentage:.2f}%) > 1%. +3 pts.")
                      else:
                           enhanced_dividend_score += 1 # Small point for non-zero yield
                           logger.debug(f"Current Yield ({yield_percentage:.2f}%) <= 1%. +1 pt.")
                 else:
                      logger.warning("Current dividend yield is NaN.")

                 # Consider Payout Ratio (Lower is generally better, indicates sustainability)
                 if not pd.isna(dividend_metrics['dividendPayoutRatio']):
                      payout_ratio_value = dividend_metrics['dividendPayoutRatio']
                      if payout_ratio_value < 0.5: # Payout ratio < 50%
                           enhanced_dividend_score += 5
                           logger.debug(f"Payout Ratio ({payout_ratio_value:.2f}) < 0.5. +5 pts.")
                      elif payout_ratio_value < 0.8: # Payout ratio < 80%
                           enhanced_dividend_score += 3
                           logger.debug(f"Payout Ratio ({payout_ratio_value:.2f}) < 0.8. +3 pts.")
                      elif payout_ratio_value > 1.0: # Payout ratio > 100% (unsustainable)
                           enhanced_dividend_score -= 5 # Penalty for unsustainable payout
                           logger.debug(f"Payout Ratio ({payout_ratio_value:.2f}) > 1.0. -5 pts.")
                      else:
                           enhanced_dividend_score += 1 # Small point for payout ratio 0.8-1.0
                           logger.debug(f"Payout Ratio ({payout_ratio_value:.2f}) 0.8-1.0. +1 pt.")

                 else:
                      logger.warning("Dividend payout ratio is NaN.")


            else:
                 logger.warning("No historical dividend data found.")

            # Cap the score at a reasonable maximum for this component (e.g., 20 points out of 235 total)
            # Let's allocate up to 20 points for enhanced dividend analysis
            dividend_metrics['score'] = min(enhanced_dividend_score, 20)


        except Exception as e:
            logger.error(f"Error calculating enhanced dividend score for {symbol}: {str(e)}. Returning default metrics.")
            # Ensure the score is set to 0 on error
            dividend_metrics['score'] = 0
            return dividend_metrics # Return dictionary with NaN/None on error


        logger.info(f"Enhanced dividend score calculated: {dividend_metrics['score']}")
        return dividend_metrics


    def fetch_news(self, symbol: str) -> List[str]:
        """
        Fetches recent news headlines for the given stock symbol.
        NOTE: This is a placeholder. In a real application, you would integrate
        with a News API (like News API, Alpha Vantage, etc.) here.
        """
        print(f"Fetching news for {symbol} (placeholder)...")
        symbol_str = str(symbol) if symbol is not None else "Unknown Stock"
        return [
            f"{symbol_str} stock price rises on positive market sentiment",
            f"{symbol_str} announces strong quarterly results, beating estimates",
            f"Experts bullish on {symbol_str}'s future outlook",
            f"I am a non-numeric entry that might cause an error",
            f"{symbol_str} faces regulatory challenges in key market",
            f"Competitor's new product launch impacts {symbol_str}'s market share",
            f"Global market trends positively impacting {symbol_str}",
            f"{symbol_str} management provides optimistic guidance",
            f"Increased foreign investment flows into {symbol_str}",
            f"Regulatory approval received for {symbol_str}'s new product",
            f"Production issues reported for {symbol_str}",
            f"Increased competition puts pressure on {symbol_str}'s margins",
            f"Analyst downgrades rating for {symbol_str}",
            f"Supply chain disruptions affect {symbol_str}'s operations",
            f"Geopolitical tensions create uncertainty for {symbol_str}",
            f"{symbol_str} announces stock split",
            f"Dividend declared by {symbol_str}"
        ]


    def analyze_sentiment(self, news_headlines: List[str]) -> Dict:
        """
        Analyzes the sentiment of news headlines using the Gemini API.
        Returns a dictionary with sentiment counts (positive, neutral, negative)
        and a sentiment score.
        """
        if not self.gemini_model:
            print("Gemini API not configured. Skipping sentiment analysis.")
            return {'positive': 0, 'neutral': 0, 'negative': 0, 'score': 0, 'summary': 'Gemini API not configured. Sentiment analysis skipped.'}

        if not isinstance(news_headlines, list) or not news_headlines:
            print("No news headlines to analyze. Skipping sentiment analysis.")
            return {'positive': 0, 'neutral': 0, 'negative': 0, 'score': 12.5, 'summary': 'No news headlines. Sentiment analysis skipped.'}

        logger.info("Analyzing sentiment using Gemini API...")
        prompt = "Analyze the sentiment of the following news headlines for a stock and categorize each as Positive, Neutral, or Negative. Provide a concise summary of the overall sentiment.\n\n"
        for headline in news_headlines:
            if isinstance(headline, str):
                 prompt += f"- {headline}\n"
            else:
                 logger.warning(f"Skipping non-string headline: {headline}")
                 continue

        if len(prompt) > 30000:
             logger.warning("News headlines too long for Gemini API prompt. Truncating.")
             prompt = prompt[:30000] + "\n... (headlines truncated)"

        try:
            response = self.gemini_model.generate_content(prompt)
            sentiment_text = response.text.strip()
            logger.info(f"Gemini API Response: {sentiment_text[:200]}...")

            positive_count = sentiment_text.lower().count('positive')
            neutral_count = sentiment_text.lower().count('neutral')
            negative_count = sentiment_text.lower().count('negative')

            summary_line = "No summary extracted."
            lines = sentiment_text.split('\n')
            for line in lines:
                 lower_line = line.lower()
                 if 'summary' in lower_line or 'overall sentiment' in lower_line:
                     summary_line = line.strip()
                     break
            if summary_line == "No summary extracted." and lines:
                 for line in reversed(lines):
                     if line.strip():
                         summary_line = line.strip()
                         break

            total_headlines = len([h for h in news_headlines if isinstance(h, str)])
            if total_headlines > 0:
                sentiment_score = ((positive_count - negative_count) / total_headlines) * 12.5 + 12.5
            else:
                sentiment_score = 12.5

            return {
                'positive': positive_count,
                'neutral': neutral_count,
                'negative': negative_count,
                'score': max(0, min(sentiment_score, 25)),
                'summary': summary_line
            }

        except Exception as e:
            logger.error(f"Error analyzing sentiment with Gemini API: {e}. Skipping sentiment analysis.")
            return {'positive': 0, 'neutral': 0, 'negative': 0, 'score': 0, 'summary': 'Sentiment analysis failed.'}


    def generate_enhanced_recommendation(self, fundamental_score, technical_score,
                                       liquidity_score, peer_performance, index_membership,
                                       sentiment_analysis_results, valuation_comparison_results,
                                       downside_risk_metrics: Dict, enhanced_dividend_metrics: Dict): # Added type hint for clarity
        """Generate comprehensive recommendation including sentiment and valuation comparison"""
        # Get base scores
        fund_score = fundamental_score if isinstance(fundamental_score, (int, float)) and not pd.isna(fundamental_score) else 0
        tech_score = technical_score if isinstance(technical_score, (int, float)) and not pd.isna(technical_score) else 0
        liq_score = liquidity_score if isinstance(liquidity_score, (int, float)) and not pd.isna(liquidity_score) else 0
        sentiment_score = sentiment_analysis_results.get('score', 0) if isinstance(sentiment_analysis_results.get('score'), (int, float)) and not pd.isna(sentiment_analysis_results.get('score')) else 0
        enhanced_dividend_score = enhanced_dividend_metrics.get('score', 0) if isinstance(enhanced_dividend_metrics.get('score'), (int, float)) and not pd.isna(enhanced_dividend_metrics.get('score')) else 0
        logger.info(f"Enhanced dividend score for recommendation: {enhanced_dividend_score}")


        # Get adjustment scores
        valuation_adjustment = valuation_comparison_results.get('valuation_score_adjustment', 0) if isinstance(valuation_comparison_results.get('valuation_score_adjustment'), (int, float)) and not pd.isna(valuation_comparison_results.get('valuation_score_adjustment')) else 0
        logger.info(f"Valuation comparison adjustment: {valuation_adjustment}")

        var_percentage = downside_risk_metrics.get('var_percentage_loss', np.nan) # Use .get() as it's now a dict
        downside_risk_adjustment = 0 # Adjustment based on VaR

        if isinstance(var_percentage, (int, float)) and not pd.isna(var_percentage):
             # Example scoring: higher VaR means higher penalty
             if var_percentage > 5: # VaR > 5%
                  downside_risk_adjustment -= 15
                  logger.info(f"VaR ({var_percentage:.2f}%) > 5%. Applying downside risk penalty (-15).")
             elif var_percentage > 3: # VaR > 3%
                  downside_risk_adjustment -= 10
                  logger.info(f"VaR ({var_percentage:.2f}%) > 3%. Applying downside risk penalty (-10).")
             elif var_percentage > 1: # VaR > 1%
                  downside_risk_adjustment -= 5
                  logger.info(f"VaR ({var_percentage:.2f}%) > 1%. Applying downside risk penalty (-5).")
             else:
                  logger.info(f"VaR ({var_percentage:.2f}%) <= 1%. No downside risk penalty.")
        else:
             logger.warning("VaR not available. Cannot apply downside risk adjustment.")
             downside_risk_adjustment -= 2 # Small penalty if VaR cannot be calculated


        peer_analysis_available = isinstance(peer_performance, dict) and \
                                  not (pd.isna(peer_performance.get('stock_return')) and \
                                       pd.isna(peer_performance.get('avg_peer_return')) and \
                                       pd.isna(peer_performance.get('relative_performance')))

        peer_adjustment = 0
        if peer_analysis_available and peer_performance.get('outperformance', False) is True:
            peer_adjustment = 5
            logger.info("Adding bonus for peer outperformance.")
        elif not peer_analysis_available:
             peer_adjustment = -5
             logger.warning("Applying small penalty as peer performance data is incomplete/unavailable.")


        index_adjustment = 0
        if isinstance(index_membership, list) and index_membership:
            index_adjustment = 5
            logger.info("Adding bonus for index membership.")


        # Sum all scores and adjustments
        # Total Score = Fund (50) + Tech (50) + Liq (70) + Sent (25) + Enhanced Dividend (20) + Val Adj (+/-30 max) + Peer Adj (+/-5 max) + Index Adj (+5 max) + Downside Adj (+0/-15 max)
        # Total Possible Max Score = 50 + 50 + 70 + 25 + 20 + 30 + 5 + 5 + 0 = 255
        # Total Possible Min Score = 50 + 0 + 0 + 0 + 0 - 30 - 5 + 0 - 15 = 0 - 50 = Max 0
        # Let's use a max total score of 255 for the scale.

        total_score = fund_score + tech_score + liq_score + sentiment_score + enhanced_dividend_score + \
                      valuation_adjustment + peer_adjustment + index_adjustment + downside_risk_adjustment


        recommendation = "NEUTRAL"
        confidence = "Low"

        # Adjusting thresholds based on potential max score of 255
        if total_score >= 200: # Increased threshold
            recommendation = "STRONG BUY"
            confidence = "High"
        elif total_score >= 160: # Increased threshold
            recommendation = "BUY"
            confidence = "Moderate to High"
        elif total_score >= 110: # Adjusted threshold
            recommendation = "HOLD"
            confidence = "Moderate"
        elif total_score >= 60: # Adjusted threshold
            recommendation = "SELL"
            confidence = "Moderate"
        else:
            recommendation = "STRONG SELL"
            confidence = "High"

        logger.info(f"Generated enhanced recommendation: {recommendation} (Total Score: {total_score:.2f})")

        return {
            'recommendation': recommendation,
            'confidence': confidence,
            'total_score': total_score,
            'breakdown': {
                'fundamental': fund_score,
                'technical': tech_score,
                'liquidity': liq_score,
                'sentiment': sentiment_score,
                'enhanced_dividend': enhanced_dividend_score, # Include enhanced dividend score in breakdown
                'valuation_comparison': valuation_adjustment, # Include valuation adjustment in breakdown
                'downside_risk': downside_risk_adjustment, # Include downside risk adjustment
                'peer_adjustment': peer_adjustment, # Include peer adjustment in breakdown
                'index_adjustment': index_adjustment # Include index adjustment in breakdown
            }
        }

**Reasoning**:
Integrate the downside risk calculation into the `analyze_stock_enhanced` function, passing the price data to the new method and storing the results in the enhanced report, then update the report display to show the downside risk metrics.



In [ ]:
#@title Part 3: Enhanced Analysis Integration and Report Display (with Downside Risk Analysis)

# Assuming IndianStockAnalyzer and EnhancedStockAnalyzer classes are defined in previous cells

# Ensure logging is configured (assuming this was done in a previous cell)
import logging
logger = logging.getLogger(__name__)

# Integration function to work with your Part 1
def analyze_stock_enhanced(symbol, stock_analyzer_part1, enhanced_analyzer):
    """
    Integrate enhanced features with your Part 1 analyzer

    Args:
        symbol: Stock symbol (e.g., 'RELIANCE.NS')
        stock_analyzer_part1: Instance of your IndianStockAnalyzer from Part 1
        enhanced_analyzer: Instance of EnhancedStockAnalyzer
    """
    if not isinstance(symbol, str) or not symbol:
        logger.error("Invalid stock symbol provided for enhanced analysis.")
        return None
    if not isinstance(stock_analyzer_part1, IndianStockAnalyzer):
        logger.error("Invalid IndianStockAnalyzer (Part 1) instance provided.")
        return None
    if not isinstance(enhanced_analyzer, EnhancedStockAnalyzer):
        logger.error("Invalid EnhancedStockAnalyzer (Part 2) instance provided.")
        return None


    logger.info(f"Starting enhanced analysis for {symbol}")
    # Get basic analysis from Part 1
    stock_data = stock_analyzer_part1.analyze_stock(symbol)

    if not stock_data:
        logger.error(f"Basic analysis failed for {symbol}.")
        return None

    # Get current price - already validated in Part 1 analyze_stock
    current_price = stock_data.current_price
    if pd.isna(current_price) or current_price <= 0:
         logger.warning(f"Current price invalid ({current_price}). Enhanced analysis might be limited.")


    # Enhanced analysis
    logger.info("Starting enhanced analysis features (Part 2).")
    # 1. Circuit breaker analysis
    circuit_limits = enhanced_analyzer.get_circuit_limits(symbol)
    # Pass current_price to assess_circuit_risk (already validated as best as possible in Part 1)
    circuit_risk = enhanced_analyzer.assess_circuit_risk(current_price, circuit_limits)

    # 2. Enhanced liquidity analysis
    # Pass price_data (already validated in Part 1 analyze_stock)
    # CORRECTED: Call get_liquidity_score on enhanced_analyzer and pass price_data from stock_data
    liquidity_score = enhanced_analyzer.get_liquidity_score(stock_data.price_data)

    # 3. Peer analysis (including valuation data)
    # Pass info (already validated in Part 1 analyze_stock)
    # The get_peer_companies method now returns a list of dictionaries with valuation data
    peers_with_valuation = enhanced_analyzer.get_peer_companies(symbol, stock_data.info)
    # Pass the list of peer symbols (extracted from the list of dictionaries) for performance comparison
    peer_symbols_for_performance = [p['symbol'] for p in peers_with_valuation if isinstance(p, dict) and 'symbol' in p]
    peer_performance = enhanced_analyzer.calculate_peer_relative_performance(symbol, peer_symbols_for_performance) # Pass just symbols

    # 4. Valuation Analysis: Calculate peer average valuations and compare
    peer_average_valuation = enhanced_analyzer.calculate_peer_average_valuation(peers_with_valuation)
    valuation_comparison_results = enhanced_analyzer.compare_valuation_to_peers(stock_data.metrics, peer_average_valuation)

    # 5. Downside Risk Assessment: Calculate VaR
    downside_risk_metrics = enhanced_analyzer.calculate_value_at_risk(stock_data.price_data)


    # 6. Index membership
    index_membership = enhanced_analyzer.get_index_membership(symbol)

    # 7. Recalculate technical score with circuit risk
    # Pass indicators (already validated in Part 1 calculate_technical_indicators)
    # Pass circuit_risk (already validated in assess_circuit_risk)
    enhanced_technical_score = enhanced_analyzer.calculate_enhanced_technical_score(
        stock_data.indicators, circuit_risk
    )

    # 8. Fetch and analyze news sentiment
    # Pass symbol (already validated)
    news_headlines = enhanced_analyzer.fetch_news(symbol)
    # Pass news_headlines (already validated)
    sentiment_analysis_results = enhanced_analyzer.analyze_sentiment(news_headlines)


    # 9. Generate enhanced recommendation
    # Pass scores and results (already validated within their respective functions)
    final_recommendation = enhanced_analyzer.generate_enhanced_recommendation(
        stock_data.fundamental_score,
        enhanced_technical_score,
        liquidity_score,
        peer_performance, # Pass potentially incomplete/NaN peer performance
        index_membership,
        sentiment_analysis_results, # Pass sentiment analysis results
        valuation_comparison_results, # Pass valuation comparison results
        downside_risk_metrics # Pass downside risk metrics
    )

    # Create enhanced report
    enhanced_report = {
        'basic_analysis': stock_data,
        'enhanced_features': {
            'circuit_limits': circuit_limits,
            'circuit_risk_score': circuit_risk,
            'liquidity_score': liquidity_score, # Ensure correct liquidity score is included
            'peers': peers_with_valuation, # Store the list of peers with their valuation data
            'peer_performance': peer_performance, # Include peer performance, even if NaN
            'peer_average_valuation': peer_average_valuation, # Store peer average valuation
            'valuation_comparison': valuation_comparison_results, # Store valuation comparison results
            'downside_risk': downside_risk_metrics, # Store downside risk metrics (e.g., VaR)
            'index_membership': index_membership,
            'dynamic_risk_free_rate': enhanced_analyzer.risk_free_rate,
            'news_headlines': news_headlines, # Include news in report
            'sentiment_analysis': sentiment_analysis_results # Include sentiment analysis results
        },
        'final_recommendation': final_recommendation
    }
    logger.info(f"Enhanced analysis completed for {symbol}")
    return enhanced_report

# Example usage
if __name__ == "__main__":
    # Initialize analyzers
    try:
        stock_analyzer = IndianStockAnalyzer()
        enhanced_analyzer = EnhancedStockAnalyzer()
    except NameError:
        print("Make sure to run the previous cells defining IndianStockAnalyzer and EnhancedStockAnalyzer classes.")
        exit() # Exit if classes are not defined


    # Analyze a stock
    symbol = "RELIANCE.NS" # Example symbol
    result = analyze_stock_enhanced(symbol, stock_analyzer, enhanced_analyzer)

    if result:
        # Display the enhanced report
        enhanced = result['enhanced_features']
        final = result['final_recommendation']
        basic = result['basic_analysis'] # Get basic data for display

        print(f"\n{'='*60}")
        print(f"Enhanced Analysis Report: {basic.company_name} ({basic.symbol})")
        print(f"{'='*60}")

        print(f"\n📊 Current Market Data:")
        print(f"Current Price: ₹{basic.current_price:,.2f}" if not pd.isna(basic.current_price) else "Current Price: N/A")
        market_cap_cr = basic.market_cap / 10000000 if not pd.isna(basic.market_cap) else np.nan
        print(f"Market Cap: ₹{market_cap_cr:,.2f} Cr" if not pd.isna(market_cap_cr) else "Market Cap: N/A")
        print(f"Sector: {basic.info.get('sector', 'N/A')}")

        print(f"\n⚙️ Enhanced Technical Analysis & Risk:")
        # Display enhanced technical indicators and risk
        # Assuming you want to display relevant indicators from basic.indicators and the circuit risk score
        circuit_limits_display = enhanced.get('circuit_limits')
        if circuit_limits_display and not pd.isna(circuit_limits_display.get('lower_circuit')) and not pd.isna(circuit_limits_display.get('upper_circuit')):
             print(f"Circuit Limits: {circuit_limits_display.get('lower_circuit', 'N/A'):.2f} - {circuit_limits_display.get('upper_circuit', 'N/A'):.2f}")
        else:
             print("Circuit Limits: N/A")

        print(f"Circuit Risk Score: {enhanced.get('circuit_risk_score', 'N/A')}")
        # Check if liquidity score is available before formatting
        liquidity_score_display = f"{enhanced.get('liquidity_score', 'N/A')}/70" if isinstance(enhanced.get('liquidity_score'), (int, float)) and not pd.isna(enhanced.get('liquidity_score')) else "Liquidity Score: N/A/70"
        print(liquidity_score_display)


        # Display some key technical indicators from basic analysis
        print("\nKey Technical Indicators (from Basic Analysis):")
        key_tech_indicators = ['RSI', 'MACD', 'MACD_signal', 'SMA_20', 'SMA_50', 'BB_middle', 'Volume_ratio']
        for indicator_name in key_tech_indicators:
            value = basic.indicators.get(indicator_name, np.nan)
            if not pd.isna(value):
                if isinstance(value, float):
                     print(f"  - {indicator_name}: {value:,.2f}")
                else:
                     print(f"  - {indicator_name}: {value}")
            # Add logging for these key technical indicators if they are calculated
            logger.info(f"Indicator {indicator_name}: {value}")
        else:
            print(f"  - {indicator_name}: N/A")
            logger.info(f"Indicator {indicator_name}: N/A")

        # Display Valuation Analysis
        print(f"\n📈 Valuation Analysis:")
        stock_metrics = basic.metrics
        peer_avg_val = enhanced.get('peer_average_valuation', {})
        val_comp_results = enhanced.get('valuation_comparison', {})

        stock_pe = stock_metrics.get('trailingPE', np.nan)
        stock_pb = stock_metrics.get('priceToBook', np.nan)
        stock_ev_ebitda = stock_metrics.get('evToEbitda', np.nan)

        print(f"Stock Valuation:")
        print(f"  - Trailing PE: {stock_pe:.2f}" if not pd.isna(stock_pe) else "  - Trailing PE: N/A")
        print(f"  - Price to Book: {stock_pb:.2f}" if not pd.isna(stock_pb) else "  - Price to Book: N/A")
        print(f"  - EV/EBITDA: {stock_ev_ebitda:.2f}" if not pd.isna(stock_ev_ebitda) else "  - EV/EBITDA: N/A")

        print("\nPeer Average Valuation:")
        avg_pe_display = f"{peer_avg_val.get('avg_pe', np.nan):.2f}" if not pd.isna(peer_avg_val.get('avg_pe', np.nan)) else "N/A"
        avg_pb_display = f"{peer_avg_val.get('avg_pb', np.nan):.2f}" if not pd.isna(peer_avg_val.get('avg_pb', np.nan)) else "N/A"
        avg_ev_ebitda_display = f"{peer_avg_val.get('avg_ev_ebitda', np.nan):.2f}" if not pd.isna(peer_avg_val.get('avg_ev_ebitda', np.nan)) else "N/A"

        print(f"  - Average PE: {avg_pe_display}")
        print(f"  - Average PB: {avg_pb_display}")
        print(f"  - Average EV/EBITDA: {avg_ev_ebitda_display}")

        print("\nValuation Comparison:")
        print(f"  - PE Comparison: {val_comp_results.get('pe_comparison', 'N/A')}")
        print(f"  - PB Comparison: {val_comp_results.get('pb_comparison', 'N/A')}")
        print(f"  - EV/EBITDA Comparison: {val_comp_results.get('ev_ebitda_comparison', 'N/A')}")
        valuation_adj_display = f"{val_comp_results.get('valuation_score_adjustment', 0):+.2f}" if isinstance(val_comp_results.get('valuation_score_adjustment'), (int, float)) and not pd.isna(val_comp_results.get('valuation_score_adjustment')) else "N/A"
        print(f"  - Valuation Score Adjustment: {valuation_adj_display}")


        print(f"\n📉 Downside Risk Analysis:")
        downside_risk = enhanced.get('downside_risk', {})
        var_percentage = downside_risk.get('var_percentage_loss', np.nan)
        var_display = f"{var_percentage:.2f}%" if not pd.isna(var_percentage) else "N/A"
        print(f"  - {downside_risk.get('confidence_level', 0.95)*100:.0f}% VaR ({downside_risk.get('horizon_days', 1)}-day): {var_display}")
        downside_adj_display = f"{final.get('breakdown', {}).get('downside_risk', np.nan):+.2f}" if isinstance(final.get('breakdown', {}).get('downside_risk'), (int, float)) and not pd.isna(final.get('breakdown', {}).get('downside_risk')) else "N/A"
        print(f"  - Downside Risk Score Adjustment: {downside_adj_display}")



        print(f"\n🤝 Peer Analysis (Performance):")
        peer_perf = enhanced.get('peer_performance')
        if isinstance(peer_perf, dict) and \
           not pd.isna(peer_perf.get('stock_return')) and \
           not pd.isna(peer_perf.get('avg_peer_return')) and \
           not pd.isna(peer_perf.get('relative_performance')):

             stock_return_display = f"{peer_perf.get('stock_return', np.nan):.2f}%" if isinstance(peer_perf.get('stock_return'), (int, float)) and not pd.isna(peer_perf.get('stock_return')) else 'N/A'
             avg_peer_return_display = f"{peer_perf.get('avg_peer_return', np.nan):.2f}%" if isinstance(peer_perf.get('avg_peer_return'), (int, float)) and not pd.isna(peer_perf.get('avg_peer_return')) else 'N/A'
             relative_performance_display = f"{peer_perf.get('relative_performance', np.nan):.2f}%" if isinstance(peer_perf.get('relative_performance'), (int, float)) and not pd.isna(peer_perf.get('relative_performance')) else 'N/A'

             print(f"  - {peer_perf.get('days', 'N/A')}-day Stock Return: {stock_return_display}")
             print(f"  - Avg Peer Return: {avg_peer_return_display}")
             print(f"  - Relative Performance: {relative_performance_display}")
        else:
             print("Peer Performance Data Unavailable.")


        if enhanced.get('index_membership'):
            print(f"\n🏛️ Index Membership: {', '.join(enhanced['index_membership'])}")

        # Sentiment Analysis Results
        sentiment_results = enhanced.get('sentiment_analysis', {})
        print(f"\n📰 News Sentiment Analysis:")
        if enhanced.get('news_headlines'):
             # Only print a few headlines to keep the report concise
             print("Sample News Headlines:")
             # Ensure headlines are valid strings before printing
             valid_headlines = [h for h in enhanced['news_headlines'] if isinstance(h, str)]
             for headline in valid_headlines[:5]: # Print up to 5 headlines
                 print(f"- {headline}")
             if len(valid_headlines) > 5:
                  print("  ...")

             print(f"Sentiment Counts: Positive={sentiment_results.get('positive', 0)}, Neutral={sentiment_results.get('neutral', 0)}, Negative={sentiment_results.get('negative', 0)}")
             sentiment_score_display = f"{sentiment_results.get('score', np.nan):.2f}" if isinstance(sentiment_results.get('score'), (int, float)) and not pd.isna(sentiment_results.get('score')) else "N/A"
             print(f"Sentiment Score: {sentiment_score_display}/25")
             print(f"Sentiment Summary: {sentiment_results.get('summary', 'N/A')}")
        else:
             print("No news headlines fetched or sentiment analysis skipped.")


        print(f"\n--- Comprehensive Scores and Recommendation ---")
        print(f"Fundamental Score (from Part 1): {basic.fundamental_score:.2f}/50" if not pd.isna(basic.fundamental_score) else "Fundamental Score: N/A")
        # Add checks for NaN before formatting float for breakdown scores
        enhanced_tech_score_display = f"{final.get('breakdown', {}).get('technical', np.nan):.2f}" if isinstance(final.get('breakdown', {}).get('technical'), (int, float)) and not pd.isna(final.get('breakdown', {}).get('technical')) else "N/A"
        liquidity_score_display_breakdown = f"{final.get('breakdown', {}).get('liquidity', np.nan):.2f}" if isinstance(final.get('breakdown', {}).get('liquidity'), (int, float)) and not pd.isna(final.get('breakdown', {}).get('liquidity')) else "N/A"
        sentiment_score_display_breakdown = f"{final.get('breakdown', {}).get('sentiment', np.nan):.2f}" if isinstance(sentiment_results.get('score'), (int, float)) and not pd.isna(sentiment_results.get('score')) else "N/A" # Corrected sentiment score breakdown check
        valuation_comp_display_breakdown = f"{final.get('breakdown', {}).get('valuation_comparison', np.nan):.2f}" if isinstance(final.get('breakdown', {}).get('valuation_comparison'), (int, float)) and not pd.isna(final.get('breakdown', {}).get('valuation_comparison')) else "N/A"
        downside_risk_display_breakdown = f"{final.get('breakdown', {}).get('downside_risk', np.nan):.2f}" if isinstance(final.get('breakdown', {}).get('downside_risk'), (int, float)) and not pd.isna(final.get('breakdown', {}).get('downside_risk')) else "N/A"


        print(f"Technical Score (Enhanced): {enhanced_tech_score_display}/50")
        print(f"Liquidity Score: {liquidity_score_display_breakdown}/70")
        print(f"Sentiment Score: {sentiment_score_display_breakdown}/25")
        print(f"Valuation Comparison Adjustment: {valuation_comp_display_breakdown}")
        print(f"Downside Risk Adjustment: {downside_risk_display_breakdown}")


        # Display adjustment scores
        peer_adj_display = f"{final.get('breakdown', {}).get('peer_adjustment', np.nan):+.2f}" if isinstance(final.get('breakdown', {}).get('peer_adjustment'), (int, float)) and not pd.isna(final.get('breakdown', {}).get('peer_adjustment')) else "N/A"
        index_adj_display = f"{final.get('breakdown', {}).get('index_adjustment', np.nan):+.2f}" if isinstance(final.get('breakdown', {}).get('index_adjustment'), (int, float)) and not pd.isna(final.get('breakdown', {}).get('index_adjustment')) else "N/A"

        print(f"Peer Performance Adjustment: {peer_adj_display}")
        print(f"Index Membership Adjustment: {index_adj_display}")


        # Updated max score to reflect the addition of Downside Risk Adjustment (potential max increase of 0, potential max decrease of 15)
        # Max Total Score = Fund (50) + Tech (50) + Liq (70) + Sent (25) + Val Adj (30) + Peer Adj (5) + Index Adj (5) + Downside Adj (0) = 235
        # Re-evaluating the previous max estimate (245) and the new max. Let's use 235 as a potential max for scoring clarity.
        print(f"\nTotal Enhanced Score: {final.get('total_score', 0):.2f}/235" if isinstance(final.get('total_score'), (int, float)) and not pd.isna(final.get('total_score')) else "Total Enhanced Score: N/A")
        print(f"RECOMMENDATION: {final.get('recommendation', 'N/A')} (Confidence: {final.get('confidence', 'N/A')})")


        print(f"\n{'='*60}\n")

**Reasoning**:
The error indicates that the `AnalysisResult` object does not have a `metrics` attribute. This means the `AnalysisResult` class definition in Part 1 needs to be updated to include the `metrics` dictionary as intended.



In [ ]:
#@title Part 1: IndianStockAnalyzer class definition (Enhanced Valuation Metrics - Fix AnalysisResult)

import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import requests
from bs4 import BeautifulSoup
import warnings
warnings.filterwarnings('ignore')

# Import typing for type hints
from typing import List, Dict, Optional
from dataclasses import dataclass # Import dataclass

# Ensure logging is configured (assuming this was done in a previous cell)
import logging
logger = logging.getLogger(__name__)

# --- IndianStockAnalyzer class definition ---
@dataclass
class AnalysisResult: # Define AnalysisResult dataclass here
    """Data class to store comprehensive stock information"""
    symbol: str
    company_name: str
    current_price: float
    market_cap: float
    info: Dict
    price_data: pd.DataFrame
    indicators: Dict
    fundamental_score: float
    technical_score: float
    recommendation: str # Added recommendation field
    metrics: Dict # Ensure metrics dictionary is included

class IndianStockAnalyzer:
    """Basic analyzer for Indian stocks"""

    def __init__(self):
        """Initialize the basic analyzer."""
        # Basic setup if needed for Part 1 (e.g., API keys, configurations)
        pass

    def analyze_stock(self, symbol: str) -> Optional[AnalysisResult]: # Added type hint for return
        """
        Performs basic fundamental and technical analysis for a given stock.
        Returns an AnalysisResult object containing the analysis results.
        """
        logger.info(f"Starting basic analysis for {symbol}")
        try:
            ticker = yf.Ticker(symbol)
            info = ticker.info

            if not info:
                 logger.warning(f"Could not fetch info for {symbol}. Analysis aborted.")
                 # Return a minimal AnalysisResult with N/A values, including an empty metrics dict
                 return AnalysisResult(
                     symbol=symbol,
                     company_name=symbol,
                     current_price=np.nan,
                     market_cap=np.nan,
                     info={},
                     price_data=pd.DataFrame(),
                     indicators={},
                     fundamental_score=np.nan,
                     technical_score=np.nan,
                     recommendation="N/A", # Added N/A recommendation
                     metrics={} # Include empty metrics dict
                 )


            # Get price data for technical analysis
            price_data = ticker.history(period="1y") # Get 1 year of historical data

            metrics = {} # Initialize metrics dictionary before any potential return

            # 1. Fundamental Analysis (Simplified)
            # Extract key fundamental data points
            try:
                metrics['marketCap'] = info.get('marketCap', np.nan)
                metrics['currentPrice'] = info.get('currentPrice', np.nan)
                metrics['sector'] = info.get('sector', 'N/A')
                metrics['industry'] = info.get('industry', 'N/A')
                company_name = info.get('longName', symbol)

                # Extract and store Valuation Ratios
                metrics['trailingPE'] = info.get('trailingPE', np.nan)
                logger.info(f"Fetched trailingPE for {symbol}: {metrics['trailingPE']}")

                metrics['priceToBook'] = info.get('priceToBook', np.nan)
                logger.info(f"Fetched priceToBook for {symbol}: {metrics['priceToBook']}")

                enterprise_value = info.get('enterpriseValue', np.nan)
                ebitda = info.get('ebitda', np.nan)
                metrics['enterpriseValue'] = enterprise_value # Store raw value for reference
                metrics['ebitda'] = ebitda # Store raw value for reference

                # Calculate EV/EBITDA, handling potential division by zero or missing data
                metrics['evToEbitda'] = np.nan # Default to NaN
                if isinstance(enterprise_value, (int, float)) and not np.isnan(enterprise_value) and \
                   isinstance(ebitda, (int, float)) and not np.isnan(ebitda) and ebitda != 0:
                    metrics['evToEbitda'] = enterprise_value / ebitda
                    logger.info(f"Calculated evToEbitda for {symbol}: {metrics['evToEbitda']:.2f}")
                elif (isinstance(enterprise_value, (int, float)) and not np.isnan(enterprise_value)) and \
                     (isinstance(ebitda, (int, float)) and not np.isnan(ebitda) and ebitda == 0):
                     logger.warning(f"EBITDA is zero for {symbol}. Cannot calculate EV/EBITDA.")
                else:
                    logger.warning(f"Missing Enterprise Value ({enterprise_value}) or EBITDA ({ebitda}) for {symbol}. Cannot calculate EV/EBITDA.")


                # Dividend Yield
                dividend_yield = info.get('dividendYield', np.nan) # This is often already a percentage as a float
                # Ensure dividend_yield is not None before checking if it's NaN
                if dividend_yield is not None and isinstance(dividend_yield, (int, float)) and not pd.isna(dividend_yield) and dividend_yield >= 0:
                     # If dividend_yield is a float between 0 and 1, assume it's a ratio and convert to percentage
                     if 0 < dividend_yield <= 1:
                         metrics['dividendYield'] = dividend_yield * 100
                         logger.info(f"Converted dividend yield for {symbol} from ratio to percentage: {metrics['dividendYield']:.2f}%")
                     else:
                         metrics['dividendYield'] = dividend_yield # Assume it's already a percentage or other form
                         logger.info(f"Dividend yield for {symbol} is {metrics['dividendYield']:.2f}%. Assuming it's already in percentage or other form.")
                else:
                     metrics['dividendYield'] = np.nan # Ensure it's NaN if None or invalid
                     logger.warning(f"Dividend yield for {symbol} is unavailable or zero ({dividend_yield}). Setting to NaN.")


                # Other fundamental metrics that might be useful later (Optional for this subtask but good practice)
                metrics['returnOnEquity'] = info.get('returnOnEquity', np.nan)
                metrics['revenueGrowth'] = info.get('revenueGrowth', np.nan)
                metrics['profitMargins'] = info.get('profitMargins', np.nan)


            except Exception as fund_e:
                 logger.error(f"Error extracting fundamental data for {symbol}: {fund_e}. Setting fundamental metrics to NaN.")
                 # Ensure metrics are still initialized to handle the rest of the process
                 metrics.setdefault('marketCap', np.nan)
                 metrics.setdefault('currentPrice', np.nan)
                 metrics.setdefault('sector', 'N/A')
                 metrics.setdefault('industry', 'N/A')
                 metrics.setdefault('trailingPE', np.nan)
                 metrics.setdefault('priceToBook', np.nan)
                 metrics.setdefault('evToEbitda', np.nan) # Ensure this is set even on error
                 metrics.setdefault('dividendYield', np.nan)
                 metrics.setdefault('returnOnEquity', np.nan)
                 metrics.setdefault('revenueGrowth', np.nan)
                 metrics.setdefault('profitMargins', np.nan)
                 company_name = info.get('longName', symbol) # Still try to get company name


            if price_data.empty:
                logger.warning(f"Could not fetch price data for {symbol}.")
                # Return info even if price data is missing, as fundamental data might still be available
                # We will handle missing price data in enhanced calculations
                # Ensure all fields are populated with N/A or empty structures, including metrics
                return AnalysisResult(
                    symbol=symbol,
                    company_name=company_name, # Use fetched company name
                    current_price=metrics.get('currentPrice', np.nan), # Use current price from metrics
                    market_cap=metrics.get('marketCap', np.nan),  # Use market cap from metrics
                    info=info,
                    price_data=pd.DataFrame(), # Empty DataFrame
                    indicators={},
                    fundamental_score=np.nan, # Fundamental score needs fundamental data
                    technical_score=np.nan, # Technical score needs price data
                    recommendation="N/A", # Added N/A recommendation
                    metrics=metrics # Include populated metrics dict
                )


            # Calculate a simple fundamental score (out of 50) - This scoring logic remains the same
            fundamental_score = self.calculate_fundamental_score(
                 metrics.get('marketCap', np.nan),
                 metrics.get('trailingPE', np.nan),
                 metrics.get('priceToBook', np.nan),
                 metrics.get('dividendYield', np.nan)
            )


            # 2. Technical Analysis (Simplified)
            # Pass price_data and the fetched current_price to technical indicator calculation
            indicators = self.calculate_technical_indicators(price_data, metrics.get('currentPrice', np.nan))
            technical_score = self.calculate_technical_score(indicators) # Basic technical score (out of 50)

            # Generate a basic recommendation (will be replaced by enhanced one later)
            basic_recommendation = self.generate_recommendation(fundamental_score, technical_score)


            # Bundle results
            # Create an AnalysisResult object to hold all analysis results
            analysis_results = AnalysisResult(
                symbol=symbol,
                company_name=company_name, # Use the fetched company name
                current_price=metrics.get('currentPrice', np.nan), # Use the fetched current price
                market_cap=metrics.get('marketCap', np.nan), # Use the fetched market cap
                info=info, # Include the full info dictionary
                price_data=price_data,
                indicators=indicators,
                fundamental_score=fundamental_score,
                technical_score=technical_score,
                recommendation=basic_recommendation, # Store the basic recommendation
                metrics=metrics # Store the populated metrics dictionary
            )

            logger.info(f"Basic analysis completed for {symbol}")
            return analysis_results

        except Exception as e:
            logger.error(f"An unexpected error occurred during basic analysis for {symbol}: {str(e)}. Returning None.")
            return None # Return None if basic analysis fails due to unexpected error

    def calculate_fundamental_score(self, market_cap: float, pe_ratio: float, pb_ratio: float, dividend_yield: float) -> float:
        """Calculate a simple fundamental score based on key metrics (out of 50)"""
        score = 0
        logger.info("Calculating basic fundamental score.")
        # Ensure metrics are valid numbers before scoring
        market_cap_valid = isinstance(market_cap, (int, float)) and not pd.isna(market_cap) and market_cap > 0
        pe_ratio_valid = isinstance(pe_ratio, (int, float)) and not pd.isna(pe_ratio) and pe_ratio > 0
        pb_ratio_valid = isinstance(pb_ratio, (int, float)) and not pd.isna(pb_ratio) and pb_ratio > 0
        dividend_yield_valid = isinstance(dividend_yield, (int, float)) and not pd.isna(dividend_yield) and dividend_yield >= 0

        logger.debug(f"Fundamental metrics validity: Market Cap={market_cap_valid}, PE={pe_ratio_valid}, PB={pb_ratio_valid}, Dividend Yield={dividend_yield_valid}")

        # Market Cap (Scale based on size, larger usually means more stable)
        if market_cap_valid:
            if market_cap > 10_00_000_00_00_000:  # > 10 Lakh Cr (Large Cap)
                score += 15
                logger.debug(f"Market Cap ({market_cap:.2f}) is Large Cap. +15 pts.")
            elif market_cap > 50_000_00_00_000:   # > 50K Cr (Mid Cap)
                score += 10
                logger.debug(f"Market Cap ({market_cap:.2f}) is Mid Cap. +10 pts.")
            else: # Small Cap and below
                score += 5
                logger.debug(f"Market Cap ({market_cap:.2f}) is Small Cap or below. +5 pts.")
        else:
             logger.warning("Market cap is invalid for fundamental scoring.")


        # PE Ratio (Lower is generally better, but depends on industry growth)
        if pe_ratio_valid:
            if pe_ratio < 20:
                score += 15
                logger.debug(f"PE Ratio ({pe_ratio:.2f}) < 20. +15 pts.")
            elif pe_ratio < 30:
                score += 10
                logger.debug(f"PE Ratio ({pe_ratio:.2f}) < 30. +10 pts.")
            elif pe_ratio < 40:
                score += 5
                logger.debug(f"PE Ratio ({pe_ratio:.2f}) < 40. +5 pts.")
            else: # High PE
                score += 2
                logger.debug(f"PE Ratio ({pe_ratio:.2f}) >= 40. +2 pts.")
        else:
             logger.warning("PE ratio is invalid for fundamental scoring.")


        # PB Ratio (Lower is generally better, indicates undervaluation)
        if pb_ratio_valid:
            if pb_ratio < 3:
                score += 10
                logger.debug(f"PB Ratio ({pb_ratio:.2f}) < 3. +10 pts.")
            elif pb_ratio < 5:
                score += 7
                logger.debug(f"PB Ratio ({pb_ratio:.2f}) < 5. +7 pts.")
            else:
                score += 3
                logger.debug(f"PB Ratio ({pb_ratio:.2f}) >= 5. +3 pts.")
        else:
             logger.warning("PB ratio is invalid for fundamental scoring.")


        # Dividend Yield (Higher is better for income-seeking investors, indicates profitability)
        if dividend_yield_valid:
            if dividend_yield > 3: # > 3%
                score += 10
                logger.debug(f"Dividend Yield ({dividend_yield:.2f}%) > 3%. +10 pts.")
            elif dividend_yield > 1: # > 1%
                score += 7
                logger.debug(f"Dividend Yield ({dividend_yield:.2f}%) > 1%. +7 pts.")
            else:
                score += 3
                logger.debug(f"Dividend Yield ({dividend_yield:.2f}%) <= 1%. +3 pts.")
        else:
             logger.warning("Dividend yield is invalid for fundamental scoring.")

        logger.info(f"Fundamental score calculated: {score}")
        return min(score, 50) # Cap score at 50

    def calculate_technical_indicators(self, price_data: pd.DataFrame, current_price: float) -> Dict: # Added type hints
        """Calculate key technical indicators"""
        logger.info("Calculating technical indicators.")
        # Ensure price_data is valid before calculating indicators
        if not isinstance(price_data, pd.DataFrame) or price_data.empty:
            logger.warning("Insufficient price data for technical indicator calculation.")
            return {'RSI': np.nan, 'MACD': np.nan, 'MACD_signal': np.nan, 'SMA_20': np.nan, 'SMA_50': np.nan,
                    'BB_upper': np.nan, 'BB_lower': np.nan, 'BB_middle': np.nan, 'Volume_ratio': np.nan,
                    'current_price': current_price} # Return NaNs if data is missing


        indicators = {}
        data_len = len(price_data)

        try:
            # RSI (Relative Strength Index)
            if data_len >= 14:
                delta = price_data['Close'].diff()
                gain = delta.where(delta > 0, 0)
                loss = -delta.where(delta < 0, 0)
                avg_gain = gain.ewm(com=14-1, adjust=False).mean()
                avg_loss = loss.ewm(com=14-1, adjust=False).mean()
                # Handle division by zero explicitly
                if avg_loss.iloc[-1] is not None and not np.isnan(avg_loss.iloc[-1]) and avg_loss.iloc[-1] != 0:
                    rs = avg_gain / avg_loss
                    indicators['RSI'] = 100 - (100 / (1 + rs)).iloc[-1]
                elif avg_gain.iloc[-1] is not None and not np.isnan(avg_gain.iloc[-1]) and avg_gain.iloc[-1] > 0:
                     indicators['RSI'] = 100.0 # If no loss but gain, RSI is 100
                else:
                     indicators['RSI'] = 50.0 # If no gain and no loss, RSI is 50
                logger.debug(f"Calculated RSI: {indicators['RSI']:.2f}")
            else:
                indicators['RSI'] = np.nan
                logger.warning("Insufficient data for RSI.")

        except Exception as e:
            logger.warning(f"Could not calculate RSI: {e}. Setting to NaN.")
            indicators['RSI'] = np.nan

        try:
            # MACD (Moving Average Convergence Divergence)
            if data_len >= 26: # Need at least 26 data points for 26-day EMA
                ema_12 = price_data['Close'].ewm(span=12, adjust=False).mean()
                ema_26 = price_data['Close'].ewm(span=26, adjust=False).mean()
                # Corrected MACD signal calculation to use the MACD Series
                macd_series = ema_12 - ema_26
                if data_len >= 26 + 9 - 1: # Need enough data points for 9-day EMA of MACD
                     indicators['MACD'] = macd_series.iloc[-1]
                     indicators['MACD_signal'] = macd_series.ewm(span=9, adjust=False).mean().iloc[-1]
                     logger.debug(f"Calculated MACD: {indicators['MACD']:.2f}, MACD Signal: {indicators['MACD_signal']:.2f}")
                else:
                     indicators['MACD'] = np.nan
                     indicators['MACD_signal'] = np.nan
                     logger.warning("Insufficient data for MACD signal calculation.")
            else:
                 indicators['MACD'] = np.nan
                 indicators['MACD_signal'] = np.nan
                 logger.warning("Insufficient data for MACD calculation.")
        except Exception as e:
            logger.warning(f"Could not calculate MACD: {e}. Setting to NaN.")
            indicators['MACD'] = np.nan
            indicators['MACD_signal'] = np.nan

        try:
            # Moving Averages
            if data_len >= 20:
                 indicators['SMA_20'] = price_data['Close'].rolling(window=20).mean().iloc[-1]
                 logger.debug(f"Calculated SMA_20: {indicators['SMA_20']:.2f}")
            else:
                 indicators['SMA_20'] = np.nan
                 logger.warning("Insufficient data for SMA_20.")

            if data_len >= 50:
                 indicators['SMA_50'] = price_data['Close'].rolling(window=50).mean().iloc[-1]
                 logger.debug(f"Calculated SMA_50: {indicators['SMA_50']:.2f}")
            else:
                 indicators['SMA_50'] = np.nan
                 logger.warning("Insufficient data for SMA_50.")

        except Exception as e:
            logger.warning(f"Could not calculate Moving Averages: {e}. Setting relevant MAs to NaN.")
            if 'SMA_20' not in indicators: indicators['SMA_20'] = np.nan
            if 'SMA_50' not in indicators: indicators['SMA_50'] = np.nan


        try:
            # Bollinger Bands
            bb_period = 20
            bb_std = 2
            if data_len >= bb_period:
                rolling_mean = price_data['Close'].rolling(window=bb_period).mean()
                rolling_std = price_data['Close'].rolling(window=bb_period).std()
                if not pd.isna(rolling_mean.iloc[-1]) and not pd.isna(rolling_std.iloc[-1]):
                    indicators['BB_middle'] = rolling_mean.iloc[-1]
                    indicators['BB_upper'] = (rolling_mean + (rolling_std * bb_std)).iloc[-1]
                    indicators['BB_lower'] = (rolling_mean - (rolling_std * bb_std)).iloc[-1]
                    logger.debug(f"Calculated BB: Middle={indicators['BB_middle']:.2f}, Upper={indicators['BB_upper']:.2f}, Lower={indicators['BB_lower']:.2f}")
                else:
                    indicators['BB_middle'] = np.nan
                    indicators['BB_upper'] = np.nan
                    indicators['BB_lower'] = np.nan
                    logger.warning("Bollinger Bands calculation resulted in NaN.")
            else:
                indicators['BB_middle'] = np.nan
                indicators['BB_upper'] = np.nan
                indicators['BB_lower'] = np.nan
                logger.warning("Insufficient data for Bollinger Bands.")

        except Exception as e:
            logger.warning(f"Could not calculate Bollinger Bands: {e}. Setting to NaN.")
            indicators['BB_middle'] = np.nan
            indicators['BB_upper'] = np.nan
            indicators['BB_lower'] = np.nan

        try:
            # Volume Analysis (e.g., current volume vs average volume)
            volume_period = 30
            if data_len >= volume_period:
                 avg_volume_30d = price_data['Volume'].rolling(window=volume_period).mean().iloc[-1]
                 current_volume = price_data['Volume'].iloc[-1]
                 # Check for zero division and NaN before calculating ratio
                 if not pd.isna(avg_volume_30d) and avg_volume_30d > 0 and not pd.isna(current_volume):
                      indicators['Volume_ratio'] = current_volume / avg_volume_30d
                      logger.debug(f"Calculated Volume Ratio: {indicators['Volume_ratio']:.2f}")
                 elif not pd.isna(current_volume) and current_volume > 0:
                      # If 30d avg is zero or NaN but current volume is positive, assume high relative volume
                      indicators['Volume_ratio'] = 2.0 # Arbitrary high ratio to indicate spike
                      logger.warning("30-day average volume is invalid for volume ratio, assuming spike due to positive current volume.")
                 else:
                      indicators['Volume_ratio'] = np.nan
                      logger.warning("Insufficient or invalid volume data for volume ratio calculation.")
            else:
                indicators['Volume_ratio'] = np.nan
                logger.warning("Insufficient data for Volume Ratio.")

        except Exception as e:
            logger.warning(f"Could not calculate Volume Ratio: {e}. Setting to NaN.")
            indicators['Volume_ratio'] = np.nan

        # Include current price in indicators for technical score calculation
        indicators['current_price'] = current_price if not pd.isna(current_price) else np.nan

        logger.info(f"Technical indicators calculated: {indicators}")
        return indicators


    def calculate_technical_score(self, indicators: Dict) -> float: # Added type hint
        """Calculate a simple technical score based on indicators (out of 50)"""
        score = 0
        logger.info("Calculating basic technical score.")
        # Ensure indicators dictionary is valid
        if not isinstance(indicators, dict):
             logger.warning("Invalid indicators dictionary for technical scoring.")
             return 0

        # Scoring based on common indicator signals
        # Ensure indicator values are valid numbers before scoring
        rsi = indicators.get('RSI', np.nan)
        macd = indicators.get('MACD', np.nan)
        macd_signal = indicators.get('MACD_signal', np.nan)
        sma_20 = indicators.get('SMA_20', np.nan)
        sma_50 = indicators.get('SMA_50', np.nan)
        current_price = indicators.get('current_price', np.nan)
        bb_upper = indicators.get('BB_upper', np.nan)
        bb_lower = indicators.get('BB_lower', np.nan)
        volume_ratio = indicators.get('Volume_ratio', np.nan)

        logger.debug(f"Indicators for basic technical scoring: RSI={rsi}, MACD={macd}, Signal={macd_signal}, SMA20={sma_20}, SMA50={sma_50}, Price={current_price}, BB_Upper={bb_upper}, BB_Lower={bb_lower}, VolumeRatio={volume_ratio}")


        # RSI Score
        if not pd.isna(rsi):
            if rsi < 30: # Oversold
                score += 10
                logger.debug("RSI < 30. +10 pts.")
            elif rsi > 70: # Overbought
                score -= 10
                logger.debug("RSI > 70. -10 pts.")
            elif 40 <= rsi <= 60: # Neutral
                 score += 5
                 logger.debug("RSI 40-60. +5 pts.")
        else:
             logger.warning("RSI is NaN for technical scoring.")

        # MACD Score (Bullish crossover)
        if not pd.isna(macd) and not pd.isna(macd_signal):
            if macd > macd_signal:
                score += 10
                logger.debug("MACD > Signal. +10 pts.")
            else:
                 score += 2 # Small score if bearish or neutral
                 logger.debug("MACD <= Signal. +2 pts.")
        else:
             logger.warning("MACD or MACD signal is NaN for technical scoring.")


        # Moving Average Crossover (Bullish: 20-day > 50-day)
        if not pd.isna(sma_20) and not pd.isna(sma_50):
            if sma_20 > sma_50:
                score += 10
                logger.debug("SMA20 > SMA50. +10 pts.")
            else:
                score += 5 # Add a small score if 20-day is below but close to 50-day
                logger.debug("SMA20 <= SMA50. +5 pts.")
        else:
             logger.warning("SMA_20 or SMA_50 is NaN for technical scoring.")


        # Bollinger Bands (Price near lower band suggests potential buy)
        if not pd.isna(current_price) and not pd.isna(bb_lower) and not pd.isna(bb_upper):
            if not pd.isna(bb_lower) and current_price < bb_lower:
                score += 10
                logger.debug("Price < Lower BB. +10 pts.")
            elif not pd.isna(bb_upper) and current_price > bb_upper:
                score -= 10
                logger.debug("Price > Upper BB. -10 pts.")
            elif not pd.isna(bb_lower) and not pd.isna(bb_upper) and bb_lower <= current_price <= bb_upper: # Price is within the bands
                 score += 5 # Add a small score for price being within bands, indicating less volatility
                 logger.debug("Price within BB. +5 pts.")
        else:
             logger.warning("Bollinger Bands or current price is NaN for technical scoring.")


        # Volume Spike (Significant volume increase can confirm trends)
        if not pd.isna(volume_ratio):
            if volume_ratio > 1.5: # Volume is 50% higher than average
                 score += 10
                 logger.debug(f"Volume Ratio ({volume_ratio:.2f}) > 1.5. +10 pts.")
            elif volume_ratio < 0.8: # Volume is significantly lower than average
                 score += 2 # Small score for low volume
                 logger.debug(f"Volume Ratio ({volume_ratio:.2f}) < 0.8. +2 pts.")
            else:
                 score += 5 # Average volume is neutral
                 logger.debug(f"Volume Ratio ({volume_ratio:.2f}) 0.8-1.5. +5 pts.")
        else:
             logger.warning("Volume ratio is NaN for technical scoring.")


        logger.info(f"Basic technical score calculated: {score}")
        return max(0, min(score, 50)) # Cap score between 0 and 50

    def generate_recommendation(self, fundamental_score: float, technical_score: float) -> str: # Added type hints
        """Generate investment recommendation based on scores"""
        logger.info(f"Generating basic recommendation based on Fundamental Score: {fundamental_score}, Technical Score: {technical_score}")
        # Ensure scores are valid numbers before summing
        fund_score = fundamental_score if isinstance(fundamental_score, (int, float)) and not np.isnan(fundamental_score) else 0
        tech_score = technical_score if isinstance(technical_score, (int, float)) and not np.isnan(technical_score) else 0

        total_score = fund_score + tech_score
        logger.debug(f"Total score for basic recommendation: {total_score}")

        if total_score >= 75:
            return "STRONG BUY"
        elif total_score >= 60:
            return "BUY"
        elif total_score >= 40:
            return "HOLD"
        elif total_score >= 25:
            return "SELL"
        else:
            return "STRONG SELL"

**Reasoning**:
The `AnalysisResult` class was updated to include the `metrics` dictionary, which should resolve the `AttributeError`. Now, re-run the enhanced analysis integration and report display code to incorporate the downside risk calculation and its display in the report.



In [ ]:
#@title Part 3: Enhanced Analysis Integration and Report Display (with Downside Risk Analysis - Retry)

# Assuming IndianStockAnalyzer and EnhancedStockAnalyzer classes are defined in previous cells

# Ensure logging is configured (assuming this was done in a previous cell)
import logging
logger = logging.getLogger(__name__)

# Integration function to work with your Part 1
def analyze_stock_enhanced(symbol, stock_analyzer_part1, enhanced_analyzer):
    """
    Integrate enhanced features with your Part 1 analyzer

    Args:
        symbol: Stock symbol (e.g., 'RELIANCE.NS')
        stock_analyzer_part1: Instance of your IndianStockAnalyzer from Part 1
        enhanced_analyzer: Instance of EnhancedStockAnalyzer
    """
    if not isinstance(symbol, str) or not symbol:
        logger.error("Invalid stock symbol provided for enhanced analysis.")
        return None
    if not isinstance(stock_analyzer_part1, IndianStockAnalyzer):
        logger.error("Invalid IndianStockAnalyzer (Part 1) instance provided.")
        return None
    if not isinstance(enhanced_analyzer, EnhancedStockAnalyzer):
        logger.error("Invalid EnhancedStockAnalyzer (Part 2) instance provided.")
        return None


    logger.info(f"Starting enhanced analysis for {symbol}")
    # Get basic analysis from Part 1
    stock_data = stock_analyzer_part1.analyze_stock(symbol)

    if not stock_data:
        logger.error(f"Basic analysis failed for {symbol}.")
        return None

    # Get current price - already validated in Part 1 analyze_stock
    current_price = stock_data.current_price
    if pd.isna(current_price) or current_price <= 0:
         logger.warning(f"Current price invalid ({current_price}). Enhanced analysis might be limited.")


    # Enhanced analysis
    logger.info("Starting enhanced analysis features (Part 2).")
    # 1. Circuit breaker analysis
    circuit_limits = enhanced_analyzer.get_circuit_limits(symbol)
    # Pass current_price to assess_circuit_risk (already validated as best as possible in Part 1)
    circuit_risk = enhanced_analyzer.assess_circuit_risk(current_price, circuit_limits)

    # 2. Enhanced liquidity analysis
    # Pass price_data (already validated in Part 1 analyze_stock)
    # CORRECTED: Call get_liquidity_score on enhanced_analyzer and pass price_data from stock_data
    liquidity_score = enhanced_analyzer.get_liquidity_score(stock_data.price_data)

    # 3. Peer analysis (including valuation data)
    # Pass info (already validated in Part 1 analyze_stock)
    # The get_peer_companies method now returns a list of dictionaries with valuation data
    peers_with_valuation = enhanced_analyzer.get_peer_companies(symbol, stock_data.info)
    # Pass the list of peer symbols (extracted from the list of dictionaries) for performance comparison
    peer_symbols_for_performance = [p['symbol'] for p in peers_with_valuation if isinstance(p, dict) and 'symbol' in p]
    peer_performance = enhanced_analyzer.calculate_peer_relative_performance(symbol, peer_symbols_for_performance) # Pass just symbols

    # 4. Valuation Analysis: Calculate peer average valuations and compare
    peer_average_valuation = enhanced_analyzer.calculate_peer_average_valuation(peers_with_valuation)
    valuation_comparison_results = enhanced_analyzer.compare_valuation_to_peers(stock_data.metrics, peer_average_valuation)

    # 5. Downside Risk Assessment: Calculate VaR
    downside_risk_metrics = enhanced_analyzer.calculate_value_at_risk(stock_data.price_data)


    # 6. Index membership
    index_membership = enhanced_analyzer.get_index_membership(symbol)

    # 7. Recalculate technical score with circuit risk
    # Pass indicators (already validated in Part 1 calculate_technical_indicators)
    # Pass circuit_risk (already validated in assess_circuit_risk)
    enhanced_technical_score = enhanced_analyzer.calculate_enhanced_technical_score(
        stock_data.indicators, circuit_risk
    )

    # 8. Fetch and analyze news sentiment
    # Pass symbol (already validated)
    news_headlines = enhanced_analyzer.fetch_news(symbol)
    # Pass news_headlines (already validated)
    sentiment_analysis_results = enhanced_analyzer.analyze_sentiment(news_headlines)


    # 9. Generate enhanced recommendation
    # Pass scores and results (already validated within their respective functions)
    final_recommendation = enhanced_analyzer.generate_enhanced_recommendation(
        stock_data.fundamental_score,
        enhanced_technical_score,
        liquidity_score,
        peer_performance, # Pass potentially incomplete/NaN peer performance
        index_membership,
        sentiment_analysis_results, # Pass sentiment analysis results
        valuation_comparison_results, # Pass valuation comparison results
        downside_risk_metrics # Pass downside risk metrics
    )

    # Create enhanced report
    enhanced_report = {
        'basic_analysis': stock_data,
        'enhanced_features': {
            'circuit_limits': circuit_limits,
            'circuit_risk_score': circuit_risk,
            'liquidity_score': liquidity_score, # Ensure correct liquidity score is included
            'peers': peers_with_valuation, # Store the list of peers with their valuation data
            'peer_performance': peer_performance, # Include peer performance, even if NaN
            'peer_average_valuation': peer_average_valuation, # Store peer average valuation
            'valuation_comparison': valuation_comparison_results, # Store valuation comparison results
            'downside_risk': downside_risk_metrics, # Store downside risk metrics (e.g., VaR)
            'index_membership': index_membership,
            'dynamic_risk_free_rate': enhanced_analyzer.risk_free_rate,
            'news_headlines': news_headlines, # Include news in report
            'sentiment_analysis': sentiment_analysis_results # Include sentiment analysis results
        },
        'final_recommendation': final_recommendation
    }
    logger.info(f"Enhanced analysis completed for {symbol}")
    return enhanced_report

# Example usage
if __name__ == "__main__":
    # Initialize analyzers
    try:
        stock_analyzer = IndianStockAnalyzer()
        enhanced_analyzer = EnhancedStockAnalyzer()
    except NameError:
        print("Make sure to run the previous cells defining IndianStockAnalyzer and EnhancedStockAnalyzer classes.")
        exit() # Exit if classes are not defined


    # Analyze a stock
    symbol = "RELIANCE.NS" # Example symbol
    result = analyze_stock_enhanced(symbol, stock_analyzer, enhanced_analyzer)

    if result:
        # Display the enhanced report
        enhanced = result['enhanced_features']
        final = result['final_recommendation']
        basic = result['basic_analysis'] # Get basic data for display

        print(f"\n{'='*60}")
        print(f"Enhanced Analysis Report: {basic.company_name} ({basic.symbol})")
        print(f"{'='*60}")

        print(f"\n📊 Current Market Data:")
        print(f"Current Price: ₹{basic.current_price:,.2f}" if not pd.isna(basic.current_price) else "Current Price: N/A")
        market_cap_cr = basic.market_cap / 10000000 if not pd.isna(basic.market_cap) else np.nan
        print(f"Market Cap: ₹{market_cap_cr:,.2f} Cr" if not pd.isna(market_cap_cr) else "Market Cap: N/A")
        print(f"Sector: {basic.info.get('sector', 'N/A')}")

        print(f"\n⚙️ Enhanced Technical Analysis & Risk:")
        # Display enhanced technical indicators and risk
        # Assuming you want to display relevant indicators from basic.indicators and the circuit risk score
        circuit_limits_display = enhanced.get('circuit_limits')
        if circuit_limits_display and not pd.isna(circuit_limits_display.get('lower_circuit')) and not pd.isna(circuit_limits_display.get('upper_circuit')):
             print(f"Circuit Limits: {circuit_limits_display.get('lower_circuit', 'N/A'):.2f} - {circuit_limits_display.get('upper_circuit', 'N/A'):.2f}")
        else:
             print("Circuit Limits: N/A")

        print(f"Circuit Risk Score: {enhanced.get('circuit_risk_score', 'N/A')}")
        # Check if liquidity score is available before formatting
        liquidity_score_display = f"{enhanced.get('liquidity_score', 'N/A')}/70" if isinstance(enhanced.get('liquidity_score'), (int, float)) and not pd.isna(enhanced.get('liquidity_score')) else "Liquidity Score: N/A/70"
        print(liquidity_score_display)


        # Display some key technical indicators from basic analysis
        print("\nKey Technical Indicators (from Basic Analysis):")
        key_tech_indicators = ['RSI', 'MACD', 'MACD_signal', 'SMA_20', 'SMA_50', 'BB_middle', 'Volume_ratio']
        for indicator_name in key_tech_indicators:
            value = basic.indicators.get(indicator_name, np.nan)
            if not pd.isna(value):
                if isinstance(value, float):
                     print(f"  - {indicator_name}: {value:,.2f}")
                else:
                     print(f"  - {indicator_name}: {value}")
            # Add logging for these key technical indicators if they are calculated
            logger.info(f"Indicator {indicator_name}: {value}")
        else:
            print(f"  - {indicator_name}: N/A")
            logger.info(f"Indicator {indicator_name}: N/A")

        # Display Valuation Analysis
        print(f"\n📈 Valuation Analysis:")
        stock_metrics = basic.metrics
        peer_avg_val = enhanced.get('peer_average_valuation', {})
        val_comp_results = enhanced.get('valuation_comparison', {})

        stock_pe = stock_metrics.get('trailingPE', np.nan)
        stock_pb = stock_metrics.get('priceToBook', np.nan)
        stock_ev_ebitda = stock_metrics.get('evToEbitda', np.nan)

        print(f"Stock Valuation:")
        print(f"  - Trailing PE: {stock_pe:.2f}" if not pd.isna(stock_pe) else "  - Trailing PE: N/A")
        print(f"  - Price to Book: {stock_pb:.2f}" if not pd.isna(stock_pb) else "  - Price to Book: N/A")
        print(f"  - EV/EBITDA: {stock_ev_ebitda:.2f}" if not pd.isna(stock_ev_ebitda) else "  - EV/EBITDA: N/A")

        print("\nPeer Average Valuation:")
        avg_pe_display = f"{peer_avg_val.get('avg_pe', np.nan):.2f}" if not pd.isna(peer_avg_val.get('avg_pe', np.nan)) else "N/A"
        avg_pb_display = f"{peer_avg_val.get('avg_pb', np.nan):.2f}" if not pd.isna(peer_avg_val.get('avg_pb', np.nan)) else "N/A"
        avg_ev_ebitda_display = f"{peer_avg_val.get('avg_ev_ebitda', np.nan):.2f}" if not pd.isna(peer_avg_val.get('avg_ev_ebitda', np.nan)) else "N/A"

        print(f"  - Average PE: {avg_pe_display}")
        print(f"  - Average PB: {avg_pb_display}")
        print(f"  - Average EV/EBITDA: {avg_ev_ebitda_display}")

        print("\nValuation Comparison:")
        print(f"  - PE Comparison: {val_comp_results.get('pe_comparison', 'N/A')}")
        print(f"  - PB Comparison: {val_comp_results.get('pb_comparison', 'N/A')}")
        print(f"  - EV/EBITDA Comparison: {val_comp_results.get('ev_ebitda_comparison', 'N/A')}")
        valuation_adj_display = f"{val_comp_results.get('valuation_score_adjustment', 0):+.2f}" if isinstance(val_comp_results.get('valuation_score_adjustment'), (int, float)) and not pd.isna(val_comp_results.get('valuation_score_adjustment')) else "N/A"
        print(f"  - Valuation Score Adjustment: {valuation_adj_display}")


        print(f"\n📉 Downside Risk Analysis:")
        downside_risk = enhanced.get('downside_risk', {})
        var_percentage = downside_risk.get('var_percentage_loss', np.nan)
        var_display = f"{var_percentage:.2f}%" if not pd.isna(var_percentage) else "N/A"
        print(f"  - {downside_risk.get('confidence_level', 0.95)*100:.0f}% VaR ({downside_risk.get('horizon_days', 1)}-day): {var_display}")
        downside_adj_display = f"{final.get('breakdown', {}).get('downside_risk', np.nan):+.2f}" if isinstance(final.get('breakdown', {}).get('downside_risk'), (int, float)) and not pd.isna(final.get('breakdown', {}).get('downside_risk')) else "N/A"
        print(f"  - Downside Risk Score Adjustment: {downside_adj_display}")



        print(f"\n🤝 Peer Analysis (Performance):")
        peer_perf = enhanced.get('peer_performance')
        if isinstance(peer_perf, dict) and \
           not pd.isna(peer_perf.get('stock_return')) and \
           not pd.isna(peer_perf.get('avg_peer_return')) and \
           not pd.isna(peer_perf.get('relative_performance')):

             stock_return_display = f"{peer_perf.get('stock_return', np.nan):.2f}%" if isinstance(peer_perf.get('stock_return'), (int, float)) and not pd.isna(peer_perf.get('stock_return')) else 'N/A'
             avg_peer_return_display = f"{peer_perf.get('avg_peer_return', np.nan):.2f}%" if isinstance(peer_perf.get('avg_peer_return'), (int, float)) and not pd.isna(peer_perf.get('avg_peer_return')) else 'N/A'
             relative_performance_display = f"{peer_perf.get('relative_performance', np.nan):.2f}%" if isinstance(peer_perf.get('relative_performance'), (int, float)) and not pd.isna(peer_perf.get('relative_performance')) else 'N/A'

             print(f"  - {peer_perf.get('days', 'N/A')}-day Stock Return: {stock_return_display}")
             print(f"  - Avg Peer Return: {avg_peer_return_display}")
             print(f"  - Relative Performance: {relative_performance_display}")
        else:
             print("Peer Performance Data Unavailable.")


        if enhanced.get('index_membership'):
            print(f"\n🏛️ Index Membership: {', '.join(enhanced['index_membership'])}")

        # Sentiment Analysis Results
        sentiment_results = enhanced.get('sentiment_analysis', {})
        print(f"\n📰 News Sentiment Analysis:")
        if enhanced.get('news_headlines'):
             # Only print a few headlines to keep the report concise
             print("Sample News Headlines:")
             # Ensure headlines are valid strings before printing
             valid_headlines = [h for h in enhanced['news_headlines'] if isinstance(h, str)]
             for headline in valid_headlines[:5]: # Print up to 5 headlines
                 print(f"- {headline}")
             if len(valid_headlines) > 5:
                  print("  ...")

             print(f"Sentiment Counts: Positive={sentiment_results.get('positive', 0)}, Neutral={sentiment_results.get('neutral', 0)}, Negative={sentiment_results.get('negative', 0)}")
             sentiment_score_display = f"{sentiment_results.get('score', np.nan):.2f}" if isinstance(sentiment_results.get('score'), (int, float)) and not pd.isna(sentiment_results.get('score')) else "N/A"
             print(f"Sentiment Score: {sentiment_score_display}/25")
             print(f"Sentiment Summary: {sentiment_results.get('summary', 'N/A')}")
        else:
             print("No news headlines fetched or sentiment analysis skipped.")


        print(f"\n--- Comprehensive Scores and Recommendation ---")
        print(f"Fundamental Score (from Part 1): {basic.fundamental_score:.2f}/50" if not pd.isna(basic.fundamental_score) else "Fundamental Score: N/A")
        # Add checks for NaN before formatting float for breakdown scores
        enhanced_tech_score_display = f"{final.get('breakdown', {}).get('technical', np.nan):.2f}" if isinstance(final.get('breakdown', {}).get('technical'), (int, float)) and not pd.isna(final.get('breakdown', {}).get('technical')) else "N/A"
        liquidity_score_display_breakdown = f"{final.get('breakdown', {}).get('liquidity', np.nan):.2f}" if isinstance(final.get('breakdown', {}).get('liquidity'), (int, float)) and not pd.isna(final.get('breakdown', {}).get('liquidity')) else "N/A"
        sentiment_score_display_breakdown = f"{final.get('breakdown', {}).get('sentiment', np.nan):.2f}" if isinstance(sentiment_results.get('score'), (int, float)) and not pd.isna(sentiment_results.get('score')) else "N/A" # Corrected sentiment score breakdown check
        valuation_comp_display_breakdown = f"{final.get('breakdown', {}).get('valuation_comparison', np.nan):.2f}" if isinstance(final.get('breakdown', {}).get('valuation_comparison'), (int, float)) and not pd.isna(final.get('breakdown', {}).get('valuation_comparison')) else "N/A"
        downside_risk_display_breakdown = f"{final.get('breakdown', {}).get('downside_risk', np.nan):.2f}" if isinstance(final.get('breakdown', {}).get('downside_risk'), (int, float)) and not pd.isna(final.get('breakdown', {}).get('downside_risk')) else "N/A"


        print(f"Technical Score (Enhanced): {enhanced_tech_score_display}/50")
        print(f"Liquidity Score: {liquidity_score_display_breakdown}/70")
        print(f"Sentiment Score: {sentiment_score_display_breakdown}/25")
        print(f"Valuation Comparison Adjustment: {valuation_comp_display_breakdown}")
        print(f"Downside Risk Adjustment: {downside_risk_display_breakdown}")


        # Display adjustment scores
        peer_adj_display = f"{final.get('breakdown', {}).get('peer_adjustment', np.nan):+.2f}" if isinstance(final.get('breakdown', {}).get('peer_adjustment'), (int, float)) and not pd.isna(final.get('breakdown', {}).get('peer_adjustment')) else "N/A"
        index_adj_display = f"{final.get('breakdown', {}).get('index_adjustment', np.nan):+.2f}" if isinstance(final.get('breakdown', {}).get('index_adjustment'), (int, float)) and not pd.isna(final.get('breakdown', {}).get('index_adjustment')) else "N/A"

        print(f"Peer Performance Adjustment: {peer_adj_display}")
        print(f"Index Membership Adjustment: {index_adj_display}")


        # Updated max score to reflect the addition of Downside Risk Adjustment (potential max increase of 0, potential max decrease of 15)
        # Max Total Score = Fund (50) + Tech (50) + Liq (70) + Sent (25) + Val Adj (30) + Peer Adj (5) + Index Adj (5) + Downside Adj (0) = 235
        # Re-evaluating the previous max estimate (245) and the new max. Let's use 235 as a potential max for scoring clarity.
        print(f"\nTotal Enhanced Score: {final.get('total_score', 0):.2f}/235" if isinstance(final.get('total_score'), (int, float)) and not pd.isna(final.get('total_score')) else "Total Enhanced Score: N/A")
        print(f"RECOMMENDATION: {final.get('recommendation', 'N/A')} (Confidence: {final.get('confidence', 'N/A')})")


        print(f"\n{'='*60}\n")

# Task
Continue with the plan to enhance the stock analysis tool by implementing downside risk assessment, enhancing dividend yield calculation, incorporating global macro impacts, integrating new analyses into the report, and refining scoring and recommendation.

## Implement downside risk assessment

### Subtask:
Implement the downside risk assessment method within the `EnhancedStockAnalyzer` class, calculating Value at Risk (VaR) using historical data.


**Reasoning**:
The error was due to the `AnalysisResult` class not having the `metrics` attribute when it was accessed in the `analyze_stock_enhanced` function. I have now updated the `AnalysisResult` dataclass definition in Part 1 to include the `metrics` dictionary, which should resolve this issue. The next logical step is to re-run the code block containing the `EnhancedStockAnalyzer` class definition to ensure this updated class and the `calculate_value_at_risk` method are loaded into the environment before attempting to run the integration and display code again. This was already done in the previous turn, so the next logical step is to re-run the integration code cell.



In [ ]:
#@title Part 3: Enhanced Analysis Integration and Report Display (with Downside Risk Analysis - Retry Again)

# Assuming IndianStockAnalyzer and EnhancedStockAnalyzer classes are defined in previous cells

# Ensure logging is configured (assuming this was done in a previous cell)
import logging
logger = logging.getLogger(__name__)

# Integration function to work with your Part 1
def analyze_stock_enhanced(symbol, stock_analyzer_part1, enhanced_analyzer):
    """
    Integrate enhanced features with your Part 1 analyzer

    Args:
        symbol: Stock symbol (e.g., 'RELIANCE.NS')
        stock_analyzer_part1: Instance of your IndianStockAnalyzer from Part 1
        enhanced_analyzer: Instance of EnhancedStockAnalyzer
    """
    if not isinstance(symbol, str) or not symbol:
        logger.error("Invalid stock symbol provided for enhanced analysis.")
        return None
    if not isinstance(stock_analyzer_part1, IndianStockAnalyzer):
        logger.error("Invalid IndianStockAnalyzer (Part 1) instance provided.")
        return None
    if not isinstance(enhanced_analyzer, EnhancedStockAnalyzer):
        logger.error("Invalid EnhancedStockAnalyzer (Part 2) instance provided.")
        return None


    logger.info(f"Starting enhanced analysis for {symbol}")
    # Get basic analysis from Part 1
    stock_data = stock_analyzer_part1.analyze_stock(symbol)

    if not stock_data:
        logger.error(f"Basic analysis failed for {symbol}.")
        return None

    # Get current price - already validated in Part 1 analyze_stock
    current_price = stock_data.current_price
    if pd.isna(current_price) or current_price <= 0:
         logger.warning(f"Current price invalid ({current_price}). Enhanced analysis might be limited.")


    # Enhanced analysis
    logger.info("Starting enhanced analysis features (Part 2).")
    # 1. Circuit breaker analysis
    circuit_limits = enhanced_analyzer.get_circuit_limits(symbol)
    # Pass current_price to assess_circuit_risk (already validated as best as possible in Part 1)
    circuit_risk = enhanced_analyzer.assess_circuit_risk(current_price, circuit_limits)

    # 2. Enhanced liquidity analysis
    # Pass price_data (already validated in Part 1 analyze_stock)
    # CORRECTED: Call get_liquidity_score on enhanced_analyzer and pass price_data from stock_data
    liquidity_score = enhanced_analyzer.get_liquidity_score(stock_data.price_data)

    # 3. Peer analysis (including valuation data)
    # Pass info (already validated in Part 1 analyze_stock)
    # The get_peer_companies method now returns a list of dictionaries with valuation data
    peers_with_valuation = enhanced_analyzer.get_peer_companies(symbol, stock_data.info)
    # Pass the list of peer symbols (extracted from the list of dictionaries) for performance comparison
    peer_symbols_for_performance = [p['symbol'] for p in peers_with_valuation if isinstance(p, dict) and 'symbol' in p]
    peer_performance = enhanced_analyzer.calculate_peer_relative_performance(symbol, peer_symbols_for_performance) # Pass just symbols

    # 4. Valuation Analysis: Calculate peer average valuations and compare
    peer_average_valuation = enhanced_analyzer.calculate_peer_average_valuation(peers_with_valuation)
    valuation_comparison_results = enhanced_analyzer.compare_valuation_to_peers(stock_data.metrics, peer_average_valuation)

    # 5. Downside Risk Assessment: Calculate VaR
    downside_risk_metrics = enhanced_analyzer.calculate_value_at_risk(stock_data.price_data)


    # 6. Index membership
    index_membership = enhanced_analyzer.get_index_membership(symbol)

    # 7. Recalculate technical score with circuit risk
    # Pass indicators (already validated in Part 1 calculate_technical_indicators)
    # Pass circuit_risk (already validated in assess_circuit_risk)
    enhanced_technical_score = enhanced_analyzer.calculate_enhanced_technical_score(
        stock_data.indicators, circuit_risk
    )

    # 8. Fetch and analyze news sentiment
    # Pass symbol (already validated)
    news_headlines = enhanced_analyzer.fetch_news(symbol)
    # Pass news_headlines (already validated)
    sentiment_analysis_results = enhanced_analyzer.analyze_sentiment(news_headlines)


    # 9. Generate enhanced recommendation
    # Pass scores and results (already validated within their respective functions)
    final_recommendation = enhanced_analyzer.generate_enhanced_recommendation(
        stock_data.fundamental_score,
        enhanced_technical_score,
        liquidity_score,
        peer_performance, # Pass potentially incomplete/NaN peer performance
        index_membership,
        sentiment_analysis_results, # Pass sentiment analysis results
        valuation_comparison_results, # Pass valuation comparison results
        downside_risk_metrics # Pass downside risk metrics
    )

    # Create enhanced report
    enhanced_report = {
        'basic_analysis': stock_data,
        'enhanced_features': {
            'circuit_limits': circuit_limits,
            'circuit_risk_score': circuit_risk,
            'liquidity_score': liquidity_score, # Ensure correct liquidity score is included
            'peers': peers_with_valuation, # Store the list of peers with their valuation data
            'peer_performance': peer_performance, # Include peer performance, even if NaN
            'peer_average_valuation': peer_average_valuation, # Store peer average valuation
            'valuation_comparison': valuation_comparison_results, # Store valuation comparison results
            'downside_risk': downside_risk_metrics, # Store downside risk metrics (e.g., VaR)
            'index_membership': index_membership,
            'dynamic_risk_free_rate': enhanced_analyzer.risk_free_rate,
            'news_headlines': news_headlines, # Include news in report
            'sentiment_analysis': sentiment_analysis_results # Include sentiment analysis results
        },
        'final_recommendation': final_recommendation
    }
    logger.info(f"Enhanced analysis completed for {symbol}")
    return enhanced_report

# Example usage
if __name__ == "__main__":
    # Initialize analyzers
    try:
        stock_analyzer = IndianStockAnalyzer()
        enhanced_analyzer = EnhancedStockAnalyzer()
    except NameError:
        print("Make sure to run the previous cells defining IndianStockAnalyzer and EnhancedStockAnalyzer classes.")
        exit() # Exit if classes are not defined


    # Analyze a stock
    symbol = "RELIANCE.NS" # Example symbol
    result = analyze_stock_enhanced(symbol, stock_analyzer, enhanced_analyzer)

    if result:
        # Display the enhanced report
        enhanced = result['enhanced_features']
        final = result['final_recommendation']
        basic = result['basic_analysis'] # Get basic data for display

        print(f"\n{'='*60}")
        print(f"Enhanced Analysis Report: {basic.company_name} ({basic.symbol})")
        print(f"{'='*60}")

        print(f"\n📊 Current Market Data:")
        print(f"Current Price: ₹{basic.current_price:,.2f}" if not pd.isna(basic.current_price) else "Current Price: N/A")
        market_cap_cr = basic.market_cap / 10000000 if not pd.isna(basic.market_cap) else np.nan
        print(f"Market Cap: ₹{market_cap_cr:,.2f} Cr" if not pd.isna(market_cap_cr) else "Market Cap: N/A")
        print(f"Sector: {basic.info.get('sector', 'N/A')}")

        print(f"\n⚙️ Enhanced Technical Analysis & Risk:")
        # Display enhanced technical indicators and risk
        # Assuming you want to display relevant indicators from basic.indicators and the circuit risk score
        circuit_limits_display = enhanced.get('circuit_limits')
        if circuit_limits_display and not pd.isna(circuit_limits_display.get('lower_circuit')) and not pd.isna(circuit_limits_display.get('upper_circuit')):
             print(f"Circuit Limits: {circuit_limits_display.get('lower_circuit', 'N/A'):.2f} - {circuit_limits_display.get('upper_circuit', 'N/A'):.2f}")
        else:
             print("Circuit Limits: N/A")

        print(f"Circuit Risk Score: {enhanced.get('circuit_risk_score', 'N/A')}")
        # Check if liquidity score is available before formatting
        liquidity_score_display = f"{enhanced.get('liquidity_score', 'N/A')}/70" if isinstance(enhanced.get('liquidity_score'), (int, float)) and not pd.isna(enhanced.get('liquidity_score')) else "Liquidity Score: N/A/70"
        print(liquidity_score_display)


        # Display some key technical indicators from basic analysis
        print("\nKey Technical Indicators (from Basic Analysis):")
        key_tech_indicators = ['RSI', 'MACD', 'MACD_signal', 'SMA_20', 'SMA_50', 'BB_middle', 'Volume_ratio']
        for indicator_name in key_tech_indicators:
            value = basic.indicators.get(indicator_name, np.nan)
            if not pd.isna(value):
                if isinstance(value, float):
                     print(f"  - {indicator_name}: {value:,.2f}")
                else:
                     print(f"  - {indicator_name}: {value}")
            # Add logging for these key technical indicators if they are calculated
            logger.info(f"Indicator {indicator_name}: {value}")
        else:
            print(f"  - {indicator_name}: N/A")
            logger.info(f"Indicator {indicator_name}: N/A")

        # Display Valuation Analysis
        print(f"\n📈 Valuation Analysis:")
        stock_metrics = basic.metrics
        peer_avg_val = enhanced.get('peer_average_valuation', {})
        val_comp_results = enhanced.get('valuation_comparison', {})

        stock_pe = stock_metrics.get('trailingPE', np.nan)
        stock_pb = stock_metrics.get('priceToBook', np.nan)
        stock_ev_ebitda = stock_metrics.get('evToEbitda', np.nan)

        print(f"Stock Valuation:")
        print(f"  - Trailing PE: {stock_pe:.2f}" if not pd.isna(stock_pe) else "  - Trailing PE: N/A")
        print(f"  - Price to Book: {stock_pb:.2f}" if not pd.isna(stock_pb) else "  - Price to Book: N/A")
        print(f"  - EV/EBITDA: {stock_ev_ebitda:.2f}" if not pd.isna(stock_ev_ebitda) else "  - EV/EBITDA: N/A")

        print("\nPeer Average Valuation:")
        avg_pe_display = f"{peer_avg_val.get('avg_pe', np.nan):.2f}" if not pd.isna(peer_avg_val.get('avg_pe', np.nan)) else "N/A"
        avg_pb_display = f"{peer_avg_val.get('avg_pb', np.nan):.2f}" if not pd.isna(peer_avg_val.get('avg_pb', np.nan)) else "N/A"
        avg_ev_ebitda_display = f"{peer_avg_val.get('avg_ev_ebitda', np.nan):.2f}" if not pd.isna(peer_avg_val.get('avg_ev_ebitda', np.nan)) else "N/A"

        print(f"  - Average PE: {avg_pe_display}")
        print(f"  - Average PB: {avg_pb_display}")
        print(f"  - Average EV/EBITDA: {avg_ev_ebitda_display}")

        print("\nValuation Comparison:")
        print(f"  - PE Comparison: {val_comp_results.get('pe_comparison', 'N/A')}")
        print(f"  - PB Comparison: {val_comp_results.get('pb_comparison', 'N/A')}")
        print(f"  - EV/EBITDA Comparison: {val_comp_results.get('ev_ebitda_comparison', 'N/A')}")
        valuation_adj_display = f"{val_comp_results.get('valuation_score_adjustment', 0):+.2f}" if isinstance(val_comp_results.get('valuation_score_adjustment'), (int, float)) and not pd.isna(val_comp_results.get('valuation_score_adjustment')) else "N/A"
        print(f"  - Valuation Score Adjustment: {valuation_adj_display}")


        print(f"\n📉 Downside Risk Analysis:")
        downside_risk = enhanced.get('downside_risk', {})
        var_percentage = downside_risk.get('var_percentage_loss', np.nan)
        var_display = f"{var_percentage:.2f}%" if not pd.isna(var_percentage) else "N/A"
        # Access 'confidence_level' and 'horizon_days' directly from the downside_risk dictionary
        confidence_level_display = downside_risk.get('confidence_level', 0.95)
        horizon_days_display = downside_risk.get('horizon_days', 1)
        print(f"  - {confidence_level_display*100:.0f}% VaR ({horizon_days_display}-day): {var_display}")

        downside_adj_display = f"{final.get('breakdown', {}).get('downside_risk', np.nan):+.2f}" if isinstance(final.get('breakdown', {}).get('downside_risk'), (int, float)) and not pd.isna(final.get('breakdown', {}).get('downside_risk')) else "N/A"
        print(f"  - Downside Risk Score Adjustment: {downside_adj_display}")



        print(f"\n🤝 Peer Analysis (Performance):")
        peer_perf = enhanced.get('peer_performance')
        if isinstance(peer_perf, dict) and \
           not pd.isna(peer_perf.get('stock_return')) and \
           not pd.isna(peer_perf.get('avg_peer_return')) and \
           not pd.isna(peer_perf.get('relative_performance')):

             stock_return_display = f"{peer_perf.get('stock_return', np.nan):.2f}%" if isinstance(peer_perf.get('stock_return'), (int, float)) and not pd.isna(peer_perf.get('stock_return')) else 'N/A'
             avg_peer_return_display = f"{peer_perf.get('avg_peer_return', np.nan):.2f}%" if isinstance(peer_perf.get('avg_peer_return'), (int, float)) and not pd.isna(peer_perf.get('avg_peer_return')) else 'N/A'
             relative_performance_display = f"{peer_perf.get('relative_performance', np.nan):.2f}%" if isinstance(peer_perf.get('relative_performance'), (int, float)) and not pd.isna(peer_perf.get('relative_performance')) else 'N/A'

             print(f"  - {peer_perf.get('days', 'N/A')}-day Stock Return: {stock_return_display}")
             print(f"  - Avg Peer Return: {avg_peer_return_display}")
             print(f"  - Relative Performance: {relative_performance_display}")
        else:
             print("Peer Performance Data Unavailable.")


        if enhanced.get('index_membership'):
            print(f"\n🏛️ Index Membership: {', '.join(enhanced['index_membership'])}")

        # Sentiment Analysis Results
        sentiment_results = enhanced.get('sentiment_analysis', {})
        print(f"\n📰 News Sentiment Analysis:")
        if enhanced.get('news_headlines'):
             # Only print a few headlines to keep the report concise
             print("Sample News Headlines:")
             # Ensure headlines are valid strings before printing
             valid_headlines = [h for h in enhanced['news_headlines'] if isinstance(h, str)]
             for headline in valid_headlines[:5]: # Print up to 5 headlines
                 print(f"- {headline}")
             if len(valid_headlines) > 5:
                  print("  ...")

             print(f"Sentiment Counts: Positive={sentiment_results.get('positive', 0)}, Neutral={sentiment_results.get('neutral', 0)}, Negative={sentiment_results.get('negative', 0)}")
             sentiment_score_display = f"{sentiment_results.get('score', np.nan):.2f}" if isinstance(sentiment_results.get('score'), (int, float)) and not pd.isna(sentiment_results.get('score')) else "N/A"
             print(f"Sentiment Score: {sentiment_score_display}/25")
             print(f"Sentiment Summary: {sentiment_results.get('summary', 'N/A')}")
        else:
             print("No news headlines fetched or sentiment analysis skipped.")


        print(f"\n--- Comprehensive Scores and Recommendation ---")
        print(f"Fundamental Score (from Part 1): {basic.fundamental_score:.2f}/50" if not pd.isna(basic.fundamental_score) else "Fundamental Score: N/A")
        # Add checks for NaN before formatting float for breakdown scores
        enhanced_tech_score_display = f"{final.get('breakdown', {}).get('technical', np.nan):.2f}" if isinstance(final.get('breakdown', {}).get('technical'), (int, float)) and not pd.isna(final.get('breakdown', {}).get('technical')) else "N/A"
        liquidity_score_display_breakdown = f"{final.get('breakdown', {}).get('liquidity', np.nan):.2f}" if isinstance(final.get('breakdown', {}).get('liquidity'), (int, float)) and not pd.isna(final.get('breakdown', {}).get('liquidity')) else "N/A"
        sentiment_score_display_breakdown = f"{final.get('breakdown', {}).get('sentiment', np.nan):.2f}" if isinstance(sentiment_results.get('score'), (int, float)) and not pd.isna(sentiment_results.get('score')) else "N/A" # Corrected sentiment score breakdown check
        valuation_comp_display_breakdown = f"{final.get('breakdown', {}).get('valuation_comparison', np.nan):.2f}" if isinstance(final.get('breakdown', {}).get('valuation_comparison'), (int, float)) and not pd.isna(final.get('breakdown', {}).get('valuation_comparison')) else "N/A"
        downside_risk_display_breakdown = f"{final.get('breakdown', {}).get('downside_risk', np.nan):.2f}" if isinstance(final.get('breakdown', {}).get('downside_risk'), (int, float)) and not pd.isna(final.get('breakdown', {}).get('downside_risk')) else "N/A"


        print(f"Technical Score (Enhanced): {enhanced_tech_score_display}/50")
        print(f"Liquidity Score: {liquidity_score_display_breakdown}/70")
        print(f"Sentiment Score: {sentiment_score_display_breakdown}/25")
        print(f"Valuation Comparison Adjustment: {valuation_comp_display_breakdown}")
        print(f"Downside Risk Adjustment: {downside_risk_display_breakdown}")


        # Display adjustment scores
        peer_adj_display = f"{final.get('breakdown', {}).get('peer_adjustment', np.nan):+.2f}" if isinstance(final.get('breakdown', {}).get('peer_adjustment'), (int, float)) and not pd.isna(final.get('breakdown', {}).get('peer_adjustment')) else "N/A"
        index_adj_display = f"{final.get('breakdown', {}).get('index_adjustment', np.nan):+.2f}" if isinstance(final.get('breakdown', {}).get('index_adjustment'), (int, float)) and not pd.isna(final.get('breakdown', {}).get('index_adjustment')) else "N/A"

        print(f"Peer Performance Adjustment: {peer_adj_display}")
        print(f"Index Membership Adjustment: {index_adj_display}")


        # Updated max score to reflect the addition of Downside Risk Adjustment (potential max increase of 0, potential max decrease of 15)
        # Max Total Score = Fund (50) + Tech (50) + Liq (70) + Sent (25) + Val Adj (30) + Peer Adj (5) + Index Adj (5) + Downside Adj (0) = 235
        # Re-evaluating the previous max estimate (245) and the new max. Let's use 235 as a potential max for scoring clarity.
        print(f"\nTotal Enhanced Score: {final.get('total_score', 0):.2f}/235" if isinstance(final.get('total_score'), (int, float)) and not pd.isna(final.get('total_score')) else "Total Enhanced Score: N/A")
        print(f"RECOMMENDATION: {final.get('recommendation', 'N/A')} (Confidence: {final.get('confidence', 'N/A')})")


        print(f"\n{'='*60}\n")

**Reasoning**:
The traceback shows an `AttributeError: 'numpy.float64' object has no attribute 'get'` occurring in the `generate_enhanced_recommendation` method when trying to access `var_percentage_loss` from `downside_risk_metrics`. This indicates that `downside_risk_metrics` is a numpy float (likely NaN or a single calculated VaR value) instead of the expected dictionary. The `calculate_value_at_risk` method in the `EnhancedStockAnalyzer` class currently returns a single float (the VaR percentage) or `None`. It needs to return a dictionary containing the VaR percentage and possibly the confidence level and horizon days, so it can be accessed using `.get()`. I need to modify the `calculate_value_at_risk` method in the `EnhancedStockAnalyzer` class to return a dictionary.



In [ ]:
#@title Part 2: EnhancedStockAnalyzer class definition (Fix VaR Return Value)

import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import requests
from bs4 import BeautifulSoup
import warnings
warnings.filterwarnings('ignore')

# Import typing for type hints
from typing import List, Dict, Optional

# Ensure logging is configured (assuming this was done in a previous cell)
import logging
logger = logging.getLogger(__name__)


# --- EnhancedStockAnalyzer class definition ---
class EnhancedStockAnalyzer:
    """Enhanced analyzer with additional features for Indian markets"""

    def __init__(self):
        """Initialize the enhanced analyzer."""
        self.risk_free_rate = self.get_dynamic_risk_free_rate()
        # Assuming gemini_model is defined globally from a previous cell
        global gemini_model # Access the global gemini_model
        self.gemini_model = gemini_model

        # More detailed circuit limit percentages based on NSE categorization (simplified)
        self.circuit_limits_percentage = {
            'category_a': 0.20, # Typically high-liquidity index stocks
            'category_b': 0.10, # Most other actively traded stocks
            'category_t': 0.05  # Stocks in trade-for-trade segment
        }

        # Simple mapping of some known stocks to categories for demonstration
        self.stock_category_map = {
            'RELIANCE.NS': 'category_a',
            'TCS.NS': 'category_a',
            'HDFCBANK.NS': 'category_a',
            'INFY.NS': 'category_a',
            'ICICIBANK.NS': 'category_a',
            'VEDL.NS': 'category_b',
            'PNB.NS': 'category_b',
            'INDIGO.NS': 'category_b',
            'BANDHANBNK.NS': 'category_b',
            'PERSISTENT.NS': 'category_b',
            'COFORGE.NS': 'category_b',
            'LTTS.NS': 'category_b',
        }


    def get_dynamic_risk_free_rate(self):
        """Fetch current 10-year G-Sec yield"""
        try:
            return 0.072  # 7.2% as of recent data
        except Exception as e:
            logger.warning(f"Could not fetch dynamic risk-free rate: {e}. Using fallback rate.")
            return 0.065

    def get_circuit_limits(self, symbol):
        """Get circuit breaker limits for the stock based on categorization"""
        try:
            ticker = yf.Ticker(symbol)
            info = ticker.info

            if not info or 'previousClose' not in info:
                 logger.warning(f"Could not get previous close for {symbol}. Cannot calculate circuit limits.")
                 return None

            prev_close = info.get('previousClose')
            if pd.isna(prev_close) or prev_close <= 0:
                 logger.warning(f"Invalid previous close price for {symbol}: {prev_close}. Cannot calculate circuit limits.")
                 return None


            stock_category = self.stock_category_map.get(symbol.upper(), 'category_b')
            circuit_percent = self.circuit_limits_percentage.get(stock_category, 0.10)

            logger.info(f"Determined circuit percentage for {symbol} (Category: {stock_category}): {circuit_percent*100}%")

            upper_circuit = prev_close * (1 + circuit_percent)
            lower_circuit = prev_close * (1 - circuit_percent)

            return {
                'upper_circuit': upper_circuit,
                'lower_circuit': lower_circuit,
                'circuit_percent': circuit_percent
            }
        except Exception as e:
            logger.error(f"Error getting circuit breaker limits for {symbol}: {str(e)}")
            return None

    def assess_circuit_risk(self, current_price, circuit_limits):
        """Assess proximity to circuit limits"""
        if pd.isna(current_price) or current_price <= 0:
             logger.warning("Invalid current price for circuit risk assessment.")
             return 0

        if not circuit_limits or 'upper_circuit' not in circuit_limits or 'lower_circuit' not in circuit_limits:
            logger.warning("Invalid or incomplete circuit limits provided.")
            return 0

        upper_circuit = circuit_limits['upper_circuit']
        lower_circuit = circuit_limits['lower_circuit']

        if pd.isna(upper_circuit) or pd.isna(lower_circuit) or upper_circuit <= 0 or lower_circuit <= 0:
             logger.warning("Circuit limits are invalid numbers. Cannot assess circuit risk.")
             return 0

        risk_score = 0

        if current_price >= upper_circuit:
             risk_score -= 30
        elif (upper_circuit - current_price) / current_price < 0.01:
            risk_score -= 20
        elif (upper_circuit - current_price) / current_price < 0.03:
            risk_score -= 10
        elif (upper_circuit - current_price) / current_price < 0.05:
            risk_score -= 5

        if current_price <= lower_circuit:
            risk_score -= 40
        elif (current_price - lower_circuit) / current_price < 0.01:
            risk_score -= 30
        elif (current_price - lower_circuit) / current_price < 0.03:
            risk_score -= 15
        elif (current_price - lower_circuit) / current_price < 0.05:
            risk_score -= 8

        logger.info(f"Circuit risk score calculated: {risk_score}")
        return risk_score

    def get_liquidity_score(self, price_data):
        """Enhanced liquidity analysis with NaN handling and better checks"""
        liquidity_score = 0
        if not isinstance(price_data, pd.DataFrame) or price_data.empty or len(price_data) < 20:
            logger.warning("Insufficient price data for liquidity analysis. Assigning default score.")
            return 30

        try:
            avg_volume_20d = price_data['Volume'].rolling(20).mean().iloc[-1]
            avg_volume_5d = price_data['Volume'].rolling(5).mean().iloc[-1]
            current_close = price_data['Close'].iloc[-1]

            # Add logging for diagnosis - before calculation check
            logger.info(f"Volume data for liquidity: avg_volume_20d: {avg_volume_20d}, avg_volume_5d: {avg_volume_5d}, current_close: {current_close}")


            if pd.isna(avg_volume_20d) or pd.isna(avg_volume_5d) or pd.isna(current_close) or current_close <= 0:
                 logger.warning("Liquidity metrics are NaN or invalid. Cannot calculate liquidity score accurately. Assigning default.")
                 return 30

            avg_turnover_20d = np.nan
            if avg_volume_20d is not None and not pd.isna(avg_volume_20d) and avg_volume_20d >= 0 and current_close > 0:
                 avg_turnover_20d = avg_volume_20d * current_close

            volume_spike = np.nan
            if avg_volume_20d is not None and not pd.isna(avg_volume_20d) and avg_volume_20d > 0:
                 if avg_volume_5d is not None and not pd.isna(avg_volume_5d):
                     volume_spike = avg_volume_5d / avg_volume_20d
                 else:
                      logger.warning("5-day average volume is invalid for volume spike calculation.")
            elif not pd.isna(avg_volume_5d) and avg_volume_5d > 0:
                 volume_spike = 1.0
                 logger.warning("20-day average volume is invalid for volume spike, using 1.0 as relative indicator.")
            else:
                 volume_spike = np.nan # Ensure volume_spike is NaN if both volumes are invalid
                 logger.warning("Insufficient or invalid volume data for volume ratio calculation.")

            # Add logging for volume spike - after calculation
            logger.info(f"Volume spike calculated: {volume_spike}")

            # Corrected logic for scoring based on volume_spike
            if not pd.isna(volume_spike):
                if volume_spike > 1.5: # Volume is 50% higher than average
                    liquidity_score += 10
                elif volume_spike < 0.8: # Volume is significantly lower than average
                    liquidity_score += 2 # Small penalty for low volume
                else: # volume_spike is between 0.8 and 1.5 (inclusive of 0.8, exclusive of 1.5)
                    liquidity_score += 5 # Average volume is neutral
            else:
                 logger.warning("Volume spike ratio is NaN. Skipping trend scoring.")


            if not pd.isna(avg_turnover_20d):
                if avg_turnover_20d > 500_00_00_000:
                    liquidity_score += 40
                elif avg_turnover_20d > 100_00_00_000:
                    liquidity_score += 30
                elif avg_turnover_20d > 10_00_00_000:
                    liquidity_score += 20
                elif avg_turnover_20d > 1_00_00_000:
                    liquidity_score += 10
                else:
                    liquidity_score += 5
            else:
                 logger.warning("Average turnover is NaN. Skipping turnover scoring.")
                 liquidity_score += 10

            volume_std = price_data['Volume'].rolling(20).std().iloc[-1] if len(price_data) >= 20 else np.nan
            volume_cv = np.nan
            if avg_volume_20d is not None and not pd.isna(avg_volume_20d) and avg_volume_20d > 0 and \
               volume_std is not None and not pd.isna(volume_std):
                volume_cv = volume_std / avg_volume_20d

            if not pd.isna(volume_cv):
                if volume_cv < 0.5:
                    liquidity_score += 20
                elif volume_cv < 1.0:
                    liquidity_score += 10
            else:
                 logger.warning("Volume coefficient of variation is NaN. Skipping consistency scoring.")

            # The volume spike scoring is now handled above within the main if not pd.isna(volume_spike): block


            return min(liquidity_score, 70)

        except Exception as e:
            logger.error(f"Error calculating liquidity score: {str(e)}")
            return 30

    def get_peer_companies(self, symbol: str, stock_info: Dict) -> List[Dict]: # Modified return type hint
        """Identify peer companies and fetch their valuation ratios for comparison"""
        logger.info(f"Identifying peer companies and fetching valuation data for {symbol}.")
        peer_list_with_valuation = [] # List to store peer info including valuation

        try:
            if not isinstance(stock_info, dict):
                 logger.warning("Invalid stock info for peer identification.")
                 return []

            sector = stock_info.get('sector', '')
            market_cap = stock_info.get('marketCap', 0)

            if not isinstance(sector, str):
                 sector = ''
            if not isinstance(market_cap, (int, float)) or pd.isna(market_cap) or market_cap <= 0:
                 logger.warning("Invalid market cap for peer identification.")
                 market_cap = 0

            if market_cap > 10_00_000_00_00_000:
                cap_category = 'Large Cap'
            elif market_cap > 50_000_00_00_000:
                cap_category = 'Mid Cap'
            else:
                cap_category = 'Small Cap'

            # Peer mapping (simplified - in production, use a comprehensive database)
            peer_map = {
                'Technology': {
                    'Large Cap': ['TCS.NS', 'INFY.NS', 'WIPRO.NS', 'HCLTECH.NS'],
                    'Mid Cap': ['TECHM.NS', 'LTTS.NS', 'PERSISTENT.NS', 'COFORGE.NS'],
                    'Small Cap': ['ZENSAR.NS', 'ECLERX.NS', 'MASTEK.NS']
                },
                'Financial Services': {
                    'Large Cap': ['HDFCBANK.NS', 'ICICIBANK.NS', 'KOTAKBANK.NS', 'AXISBANK.NS'],
                    'Mid Cap': ['IDFCFIRSTB.NS', 'FEDERALBNK.NS', 'INDUSINDBK.NS'],
                    'Small Cap': ['SOUTHBANK.NS', 'UJJIVAN.NS', 'EQUITASBNK.NS']
                },
                'Healthcare': {
                    'Large Cap': ['SUNPHARMA.NS', 'DRREDDY.NS', 'CIPLA.NS', 'DIVISLAB.NS'],
                    'Mid Cap': ['TORNTPHARM.NS', 'ALKEM.NS', 'LUPIN.NS'],
                    'Small Cap': ['CAPLIPOINT.NS', 'BLISS.NS', 'INDOCO.NS']
                },
                # Added Energy sector mapping
                'Energy': {
                    'Large Cap': ['ONGC.NS', 'NTPC.NS', 'POWERGRID.NS'], # Exclude RELIANCE.NS here, filter below
                    'Mid Cap': ['GAIL.NS', 'IOC.NS', 'NHPC.NS'],
                    'Small Cap': ['GUJGASLTD.NS', 'IGL.NS', 'MAHAPOWER.NS']
                }
            }

            potential_peers = peer_map.get(sector, {}).get(cap_category, [])
            # Ensure the current symbol is excluded from the peer list
            filtered_peers = [p for p in potential_peers if isinstance(p, str) and p.upper() != symbol.upper()]

            logger.info(f"Identified {len(filtered_peers)} potential peer companies for {symbol} in {sector} ({cap_category} Cap). Fetching valuation data.")

            # Fetch valuation data for each filtered peer
            for peer_symbol in filtered_peers:
                 try:
                      peer_ticker = yf.Ticker(peer_symbol)
                      peer_info = peer_ticker.info

                      if peer_info:
                           peer_valuation = {
                               'symbol': peer_symbol,
                               'trailingPE': peer_info.get('trailingPE', np.nan),
                               'priceToBook': peer_info.get('priceToBook', np.nan),
                               'enterpriseValue': peer_info.get('enterpriseValue', np.nan), # Fetch for EV/EBITDA calculation
                               'ebitda': peer_info.get('ebitda', np.nan) # Fetch for EV/EBITDA calculation
                           }

                           # Calculate EV/EBITDA for the peer
                           ev = peer_valuation['enterpriseValue']
                           ebitda = peer_valuation['ebitda']
                           peer_valuation['evToEbitda'] = np.nan # Default to NaN
                           if isinstance(ev, (int, float)) and not np.isnan(ev) and \
                              isinstance(ebitda, (int, float)) and not np.isnan(ebitda) and ebitda != 0:
                               peer_valuation['evToEbitda'] = ev / ebitda
                           elif (isinstance(ev, (int, float)) and not np.isnan(ev)) and \
                                (isinstance(ebitda, (int, float)) and not np.isnan(ebitda) and ebitda == 0):
                                logger.warning(f"EBITDA is zero for peer {peer_symbol}. Cannot calculate EV/EBITDA.")
                           else:
                                logger.warning(f"Missing Enterprise Value ({ev}) or EBITDA ({ebitda}) for peer {peer_symbol}. Cannot calculate EV/EBITDA.")


                           peer_list_with_valuation.append(peer_valuation)
                           logger.debug(f"Fetched valuation data for peer {peer_symbol}: PE={peer_valuation['trailingPE']:.2f}, PB={peer_valuation['priceToBook']:.2f}, EV/EBITDA={peer_valuation['evToEbitda']:.2f}")
                      else:
                           logger.warning(f"Could not fetch info for peer {peer_symbol}. Skipping.")
                 except Exception as e:
                      logger.warning(f"Error fetching data for peer {peer_symbol}: {str(e)}. Skipping.")

            logger.info(f"Successfully fetched valuation data for {len(peer_list_with_valuation)} peers.")
            return peer_list_with_valuation[:5] # Return top 5 peers with their valuation data

        except Exception as e:
            logger.error(f"Error getting peer companies and valuation data for {symbol}: {str(e)}")
            return [] # Return empty list on error

    def calculate_peer_average_valuation(self, peers_with_valuation: List[Dict]) -> Dict:
        """Calculate average valuation ratios (PE, PB, EV/EBITDA) for a list of peers."""
        logger.info("Calculating peer average valuation ratios.")
        total_pe = 0
        total_pb = 0
        total_ev_ebitda = 0
        count_pe = 0
        count_pb = 0
        count_ev_ebitda = 0

        if not isinstance(peers_with_valuation, list) or not peers_with_valuation:
             logger.warning("No peer valuation data provided for calculating averages.")
             return {'avg_pe': np.nan, 'avg_pb': np.nan, 'avg_ev_ebitda': np.nan}


        for peer_data in peers_with_valuation:
             if isinstance(peer_data, dict):
                 pe = peer_data.get('trailingPE', np.nan)
                 pb = peer_data.get('priceToBook', np.nan)
                 ev_ebitda = peer_data.get('evToEbitda', np.nan)

                 if isinstance(pe, (int, float)) and not np.isnan(pe) and pe > 0:
                     total_pe += pe
                     count_pe += 1
                 else:
                      logger.debug(f"Skipping invalid PE for peer: {peer_data.get('symbol', 'Unknown')}")

                 if isinstance(pb, (int, float)) and not np.isnan(pb) and pb > 0:
                     total_pb += pb
                     count_pb += 1
                 else:
                      logger.debug(f"Skipping invalid PB for peer: {peer_data.get('symbol', 'Unknown')}")


                 if isinstance(ev_ebitda, (int, float)) and not np.isnan(ev_ebitda): # EV/EBITDA can be zero or negative in some cases, but >0 is typical
                      total_ev_ebitda += ev_ebitda
                      count_ev_ebitda += 1
                 else:
                      logger.debug(f"Skipping invalid EV/EBITDA for peer: {peer_data.get('symbol', 'Unknown')}")


        avg_pe = total_pe / count_pe if count_pe > 0 else np.nan
        avg_pb = total_pb / count_pb if count_pb > 0 else np.nan
        avg_ev_ebitda = total_ev_ebitda / count_ev_ebitda if count_ev_ebitda > 0 else np.nan

        logger.info(f"Calculated peer averages: PE={avg_pe:.2f}, PB={avg_pb:.2f}, EV/EBITDA={avg_ev_ebitda:.2f}")

        return {
            'avg_pe': avg_pe,
            'avg_pb': avg_pb,
            'avg_ev_ebitda': avg_ev_ebitda
        }


    def compare_valuation_to_peers(self, stock_valuation: Dict, peer_average_valuation: Dict) -> Dict:
        """Compare stock's valuation ratios to peer averages and provide assessment."""
        logger.info("Comparing stock valuation to peer averages.")
        comparison_results = {}
        valuation_score_adjustment = 0 # Adjustment to the total score

        stock_pe = stock_valuation.get('trailingPE', np.nan)
        stock_pb = stock_valuation.get('priceToBook', np.nan)
        stock_ev_ebitda = stock_valuation.get('evToEbitda', np.nan)

        avg_pe = peer_average_valuation.get('avg_pe', np.nan)
        avg_pb = peer_average_valuation.get('avg_pb', np.nan)
        avg_ev_ebitda = peer_average_valuation.get('avg_ev_ebitda', np.nan)

        logger.debug(f"Stock Valuation: PE={stock_pe:.2f}, PB={stock_pb:.2f}, EV/EBITDA={stock_ev_ebitda:.2f}")
        logger.debug(f"Peer Average Valuation: PE={avg_pe:.2f}, PB={avg_pb:.2f}, EV/EBITDA={avg_ev_ebitda:.2f}")


        # PE Ratio comparison (Lower is generally better relative to peers)
        comparison_results['pe_comparison'] = 'N/A'
        if not pd.isna(stock_pe) and not pd.isna(avg_pe) and avg_pe > 0:
             if stock_pe < avg_pe * 0.8: # More than 20% below average
                  comparison_results['pe_comparison'] = 'Significantly Undervalued (PE)'
                  valuation_score_adjustment += 10
             elif stock_pe < avg_pe * 0.95: # More than 5% below average
                  comparison_results['pe_comparison'] = 'Undervalued (PE)'
                  valuation_score_adjustment += 5
             elif stock_pe > avg_pe * 1.2: # More than 20% above average
                  comparison_results['pe_comparison'] = 'Significantly Overvalued (PE)'
                  valuation_score_adjustment -= 10
             elif stock_pe > avg_pe * 1.05: # More than 5% above average
                  comparison_results['pe_comparison'] = 'Overvalued (PE)'
                  valuation_score_adjustment -= 5
             else:
                  comparison_results['pe_comparison'] = 'Fairly Valued (PE)'
        elif not pd.isna(stock_pe) and pd.isna(avg_pe):
             comparison_results['pe_comparison'] = 'PE available, Peer Avg PE N/A'
             logger.warning("Peer Average PE is NaN. Cannot compare stock PE to peers.")
        elif pd.isna(stock_pe) and not pd.isna(avg_pe):
             comparison_results['pe_comparison'] = 'Stock PE N/A'
             logger.warning("Stock PE is NaN. Cannot compare to peer Average PE.")
        else:
             logger.warning("Stock PE and Peer Average PE are NaN. Cannot compare.")


        # PB Ratio comparison (Lower is generally better relative to peers)
        comparison_results['pb_comparison'] = 'N/A'
        if not pd.isna(stock_pb) and not pd.isna(avg_pb) and avg_pb > 0:
             if stock_pb < avg_pb * 0.8: # More than 20% below average
                  comparison_results['pb_comparison'] = 'Significantly Undervalued (PB)'
                  valuation_score_adjustment += 8
             elif stock_pb < avg_pb * 0.95: # More than 5% below average
                  comparison_results['pb_comparison'] = 'Undervalued (PB)'
                  valuation_score_adjustment += 4
             elif stock_pb > avg_pb * 1.2: # More than 20% above average
                  comparison_results['pb_comparison'] = 'Significantly Overvalued (PB)'
                  valuation_score_adjustment -= 8
             elif stock_pb > avg_pb * 1.05: # More than 5% above average
                  comparison_results['pb_comparison'] = 'Overvalued (PB)'
                  valuation_score_adjustment -= 4
             else:
                  comparison_results['pb_comparison'] = 'Fairly Valued (PB)'
        elif not pd.isna(stock_pb) and pd.isna(avg_pb):
             comparison_results['pb_comparison'] = 'PB available, Peer Avg PB N/A'
             logger.warning("Peer Average PB is NaN. Cannot compare stock PB to peers.")
        elif pd.isna(stock_pb) and not pd.isna(avg_pb):
             comparison_results['pb_comparison'] = 'Stock PB N/A'
             logger.warning("Stock PB is NaN. Cannot compare to peer Average PB.")
        else:
             logger.warning("Stock PB and Peer Average PB are NaN. Cannot compare.")


        # EV/EBITDA comparison (Lower is generally better relative to peers)
        comparison_results['ev_ebitda_comparison'] = 'N/A'
        if not pd.isna(stock_ev_ebitda) and not pd.isna(avg_ev_ebitda) and avg_ev_ebitda != 0:
             if stock_ev_ebitda < avg_ev_ebitda * 0.8: # More than 20% below average
                  comparison_results['ev_ebitda_comparison'] = 'Significantly Undervalued (EV/EBITDA)'
                  valuation_score_adjustment += 12
             elif stock_ev_ebitda < avg_ev_ebitda * 0.95: # More than 5% below average
                  comparison_results['ev_ebitda_comparison'] = 'Undervalued (EV/EBITDA)'
                  valuation_score_adjustment += 6
             elif stock_ev_ebitda > avg_ev_ebitda * 1.2: # More than 20% above average
                  comparison_results['ev_ebitda_comparison'] = 'Significantly Overvalued (EV/EBITDA)'
                  valuation_score_adjustment -= 12
             elif stock_ev_ebitda > avg_ev_ebitda * 1.05: # More than 5% above average
                  comparison_results['ev_ebitda_comparison'] = 'Overvalued (EV/EBITDA)'
                  valuation_score_adjustment -= 6
             else:
                  comparison_results['ev_ebitda_comparison'] = 'Fairly Valued (EV/EBITDA)'
        elif not pd.isna(stock_ev_ebitda) and not pd.isna(avg_ev_ebitda) and avg_ev_ebitda == 0:
             comparison_results['ev_ebitda_comparison'] = 'Peer Avg EV/EBITDA is 0'
             logger.warning("Peer Average EV/EBITDA is zero. Cannot compare.")
        elif not pd.isna(stock_ev_ebitda) and pd.isna(avg_ev_ebitda):
             comparison_results['ev_ebitda_comparison'] = 'EV/EBITDA available, Peer Avg EV/EBITDA N/A'
             logger.warning("Peer Average EV/EBITDA is NaN. Cannot compare stock EV/EBITDA to peers.")
        elif pd.isna(stock_ev_ebitda) and not pd.isna(avg_ev_ebitda):
             comparison_results['ev_ebitda_comparison'] = 'Stock EV/EBITDA N/A'
             logger.warning("Stock EV/EBITDA is NaN. Cannot compare to peer Average EV/EBITDA.")
        else:
             logger.warning("Stock EV/EBITDA and Peer Average EV/EBITDA are NaN. Cannot compare.")


        comparison_results['valuation_score_adjustment'] = valuation_score_adjustment

        logger.info(f"Valuation comparison results: {comparison_results}")
        return comparison_results


    def calculate_peer_relative_performance(self, symbol, peers, days=30):
        """Compare stock performance with peers, handling data issues robustly"""
        results = {
            'stock_return': np.nan,
            'avg_peer_return': np.nan,
            'relative_performance': np.nan,
            'outperformance': False,
            'days': days
        }

        if not isinstance(peers, list) or not peers:
             logger.warning("No valid peer list provided for performance comparison.")
             # Return the default results with NaNs if no peers
             return results

        # --- Removed the temporary skip here ---

        try:
            main_ticker = yf.Ticker(symbol)
            main_hist = main_ticker.history(period=f"{days}d")
            main_return = np.nan

            if not main_hist.empty and len(main_hist) > 1:
                try:
                    start_price = main_hist['Close'].iloc[0]
                    end_price = main_hist['Close'].iloc[-1]
                    if not pd.isna(start_price) and not pd.isna(end_price) and start_price > 0:
                         main_return = ((end_price / start_price) - 1) * 100
                    else:
                         logger.warning(f"Invalid start ({start_price}) or end price ({end_price}) for main stock ({symbol}) return calculation.")
                except Exception as ret_e:
                     logger.warning(f"Could not calculate main stock ({symbol}) return: {ret_e}. Setting to NaN.")
                     main_return = np.nan
            else:
                logger.warning(f"Insufficient historical data for main stock ({symbol}) for return calculation.")

            results['stock_return'] = main_return

        except Exception as main_stock_e:
            logger.error(f"Error fetching data or calculating return for main stock ({symbol}): {main_stock_e}. Main stock return set to NaN.")
            results['stock_return'] = np.nan

        # Calculate peer returns
        peer_returns = []
        # The peers list now contains dictionaries with valuation data.
        # We need to extract just the symbols for performance calculation.
        peer_symbols_for_perf = [p['symbol'] for p in peers if isinstance(p, dict) and 'symbol' in p]

        for peer_symbol in peer_symbols_for_perf:
             try:
                 peer_ticker = yf.Ticker(peer_symbol)
                 peer_hist = peer_ticker.history(period=f"{days}d")
                 if not peer_hist.empty and len(peer_hist) > 1:
                     try:
                          peer_start_price = peer_hist['Close'].iloc[0]
                          peer_end_price = peer_hist['Close'].iloc[-1]
                          if not pd.isna(peer_start_price) and not pd.isna(peer_end_price) and peer_start_price > 0:
                             peer_return = ((peer_end_price / peer_start_price) - 1) * 100
                             peer_returns.append(peer_return)
                          else:
                             logger.warning(f"Invalid start or end price for peer ({peer_symbol}) return calculation.")
                     except Exception as peer_ret_e:
                         logger.warning(f"Could not calculate return for peer ({peer_symbol}): {peer_ret_e}. Skipping.")
                 else:
                     logger.warning(f"Insufficient historical data for peer ({peer_symbol}) for return calculation. Skipping.")
             except Exception as peer_e:
                 logger.warning(f"Error fetching data for peer ({peer_symbol}): {peer_e}. Skipping.")

        # Add logging for peer returns before filtering/average calculation
        logger.info(f"Peer returns collected before average calculation attempt: {peer_returns}")

        # Calculate average peer return and relative performance
        if peer_returns:
             # Ensure peer_returns list contains only valid numbers before calculating mean
             valid_peer_returns = [ret for ret in peer_returns if isinstance(ret, (int, float)) and not pd.isna(ret)]
             if valid_peer_returns:
                 avg_peer_return = np.mean(valid_peer_returns)
                 results['avg_peer_return'] = avg_peer_return

                 # Calculate relative performance only if main stock return is valid
                 if not pd.isna(main_return):
                      results['relative_performance'] = main_return - avg_peer_return

                      # Determine outperformance
                      if main_return > avg_peer_return:
                          results['outperformance'] = True
                 else:
                      logger.warning("Main stock return is NaN, cannot calculate relative performance or outperformance.")
             else:
                 logger.warning("No valid peer returns available after filtering. Cannot calculate average peer return or relative performance.")
        else:
             logger.warning("No peer returns calculated. Cannot calculate average peer return or relative performance.")


        logger.info(f"Peer performance results for {symbol}: {results}")
        return results


    def get_index_membership(self, symbol):
        """Check index membership for the stock"""
        symbol_upper = symbol.upper() if isinstance(symbol, str) else ''

        index_constituents = {
            'NIFTY50': ['RELIANCE.NS', 'TCS.NS', 'HDFCBANK.NS', 'INFY.NS', 'ICICIBANK.NS'],
            'NIFTY_NEXT50': ['VEDL.NS', 'PNB.NS', 'INDIGO.NS', 'BANDHANBNK.NS'],
            'NIFTY_MIDCAP150': ['PERSISTENT.NS', 'COFORGE.NS', 'LTTS.NS']
        }

        membership = []
        for index, constituents in index_constituents.items():
            if symbol_upper and any(isinstance(c, str) and c.upper() == symbol_upper for c in constituents):
                membership.append(index)

        if membership:
             logger.info(f"Stock {symbol} is a member of indices: {', '.join(membership)}")
        else:
             logger.info(f"Stock {symbol} is not found in the hardcoded index constituents.")

        return membership

    def calculate_enhanced_technical_score(self, indicators, circuit_risk):
        """Calculate enhanced technical analysis score with NaN handling and circuit risk"""
        base_score = 0
        indicators_calculated = 0
        possible_indicator_points = 50

        num_key_indicators = 5
        indicator_point_contribution = possible_indicator_points / num_key_indicators

        rsi = indicators.get('RSI', np.nan)
        if not pd.isna(rsi):
             indicators_calculated += 1
             if 40 <= rsi <= 60:
                 base_score += indicator_point_contribution * 0.7
             elif rsi < 30: # More points for extreme oversold
                 base_score += indicator_point_contribution * 1.0
             elif 30 <= rsi < 40: # Approaching oversold
                 base_score += indicator_point_contribution * 0.9
             elif 60 < rsi <= 70: # Approaching overbought
                 base_score += indicator_point_contribution * 0.6
             else: # Overbought (>70)
                 base_score += indicator_point_contribution * 0.4
        else:
             logger.warning("RSI not calculated or is NaN.")

        macd = indicators.get('MACD', np.nan)
        macd_signal = indicators.get('MACD_signal', np.nan)

        if not pd.isna(macd) and not pd.isna(macd_signal):
             indicators_calculated += 1
             if macd > macd_signal:
                 if macd > 0: # Bullish crossover above zero line
                      base_score += indicator_point_contribution * 1.0
                 else: # Bullish crossover below zero line
                      base_score += indicator_point_contribution * 0.8
             else:
                 if macd < 0: # Bearish crossover below zero line
                      base_score += indicator_point_contribution * 0.2
                 else: # Bearish crossover above zero line
                      base_score += indicator_point_contribution * 0.4
        else:
             logger.warning("MACD or MACD signal is NaN for technical scoring.")

        sma_20 = indicators.get('SMA_20', np.nan)
        sma_50 = indicators.get('SMA_50', np.nan)
        sma_200 = indicators.get('SMA_200', np.nan) # Include 200-day SMA if available

        ma_score_component = 0
        ma_indicators_counted = 0

        if not pd.isna(sma_20) and not pd.isna(sma_50):
             ma_indicators_counted += 1
             if sma_20 > sma_50:
                  ma_score_component += 0.5 # 20 > 50 crossover is bullish
             else:
                  ma_score_component += 0.2 # 20 <= 50 crossover is bearish/neutral

        if not pd.isna(sma_50) and not pd.isna(sma_200):
             ma_indicators_counted += 1
             if sma_50 > sma_200:
                  ma_score_component += 0.5 # 50 > 200 crossover is bullish
             else:
                  ma_score_component += 0.2 # 50 <= 200 crossover is bearish/neutral

        current_price = indicators.get('current_price', np.nan)
        if not pd.isna(current_price):
             if not pd.isna(sma_20) and current_price > sma_20:
                  ma_score_component += 0.3 # Price above 20-day SMA is bullish
             elif not pd.isna(sma_20):
                  ma_score_component += 0.1 # Price below 20-day SMA is bearish/neutral

             if not pd.isna(sma_50) and current_price > sma_50:
                  ma_score_component += 0.4 # Price above 50-day SMA is more bullish
             elif not pd.isna(sma_50):
                  ma_score_component += 0.15 # Price below 50-day SMA

             if not pd.isna(sma_200) and current_price > sma_200:
                  ma_score_component += 0.6 # Price above 200-day SMA is significant bullish signal
             elif not pd.isna(sma_200):
                  ma_score_component += 0.1 # Price below 200-day SMA is significant bearish signal


        if ma_indicators_counted > 0 or (not pd.isna(current_price) and (not pd.isna(sma_20) or not pd.isna(sma_50) or not pd.isna(sma_200))):
             indicators_calculated += 1 # Count MA section if any valid MA comparison/position is made
             # Normalize MA score component to contribute to the total score
             # Max possible ma_score_component (0.5 + 0.5 + 0.3 + 0.4 + 0.6) = 2.3
             # Let's scale this to contribute up to indicator_point_contribution
             max_ma_component = 2.3
             base_score += (ma_score_component / max_ma_component) * indicator_point_contribution if max_ma_component > 0 else 0
        else:
             logger.warning("Insufficient data for Moving Averages analysis.")


        bb_upper = indicators.get('BB_upper', np.nan)
        bb_lower = indicators.get('BB_lower', np.nan)
        bb_middle = indicators.get('BB_middle', np.nan)

        if not pd.isna(current_price) and not pd.isna(bb_upper) and not pd.isna(bb_lower) and not pd.isna(bb_middle):
            indicators_calculated += 1
            if current_price < bb_lower:
                base_score += indicator_point_contribution * 1.0 # Price below lower band (potential buy signal)
            elif bb_lower <= current_price < bb_middle:
                base_score += indicator_point_contribution * 0.8 # Between lower and middle band
            elif bb_middle <= current_price < bb_upper:
                base_score += indicator_point_contribution * 0.6 # Between middle and upper band
            else: # Price above upper band
                base_score += indicator_point_contribution * 0.4 # Potential sell signal
        else:
             logger.warning("Bollinger Bands or current price is NaN for technical scoring.")

        volume_ratio = indicators.get('Volume_ratio', np.nan)
        if not pd.isna(volume_ratio):
             indicators_calculated += 1
             if volume_ratio > 2.0: # Very high volume
                 base_score += indicator_point_contribution * 1.0
             elif volume_ratio > 1.2: # High volume
                 base_score += indicator_point_contribution * 0.8
             elif 0.8 <= volume_ratio <= 1.2: # Normal volume
                 base_score += indicator_point_contribution * 0.5
             else: # Low volume
                 base_score += indicator_point_contribution * 0.3
        else:
             logger.warning("Volume ratio is NaN for technical scoring.")


        achieved_score_from_indicators = base_score

        # Normalize the base score based on how many indicators were successfully calculated
        if indicators_calculated > 0:
            # Assuming each of the num_key_indicators contributes equally if available
            # This approach gives points for each indicator that could be calculated and scored
            # Max points if all indicators could be scored is num_key_indicators * indicator_point_contribution
            # But our scoring within each indicator gives variable points (e.g., 0.4 to 1.0)
            # A simpler approach is to just sum up the points from successfully scored indicators
            # Let's cap the sum at the max possible points from indicators (50)
            final_base_score = min(achieved_score_from_indicators, possible_indicator_points)

            if indicators_calculated < num_key_indicators:
                 logger.warning(f"Only {indicators_calculated}/{num_key_indicators} key technical indicators could be fully calculated/scored. Final base technical score is based on available data.")

        else:
             logger.warning("No key technical indicators calculated. Base technical score is 0.")
             final_base_score = 0


        circuit_risk_adj = circuit_risk if isinstance(circuit_risk, (int, float)) and not pd.isna(circuit_risk) else 0

        final_score = final_base_score + circuit_risk_adj

        return max(0, min(final_score, 50))


    def calculate_value_at_risk(self, price_data: pd.DataFrame, confidence_level: float = 0.95, horizon_days: int = 1) -> Dict: # Modified return type hint
        """
        Calculates Value at Risk (VaR) using the historical method.

        Args:
            price_data: DataFrame with historical price data (must contain 'Close').
            confidence_level: The confidence level for VaR (e.g., 0.95 for 95% VaR).
            horizon_days: The time horizon for VaR (in trading days).

        Returns:
            A dictionary containing the calculated VaR as a percentage loss,
            confidence level, and horizon days, or a dictionary with NaN/None values
            if calculation fails.
        """
        logger.info(f"Calculating {confidence_level*100:.0f}% VaR for {horizon_days}-day horizon.")
        results = {'var_percentage_loss': np.nan, 'confidence_level': confidence_level, 'horizon_days': horizon_days}

        if not isinstance(price_data, pd.DataFrame) or price_data.empty or 'Close' not in price_data.columns:
            logger.warning("Insufficient or invalid price data for VaR calculation.")
            return results # Return dictionary with NaN/None


        try:
            # Calculate daily returns
            returns = price_data['Close'].pct_change().dropna()

            if returns.empty:
                logger.warning("No valid returns data for VaR calculation.")
                return results # Return dictionary with NaN/None

            # Historical VaR: Find the percentile of returns corresponding to the confidence level
            # For downside risk, we look at the lower tail of returns.
            # A 95% confidence level for VaR means we're interested in the 5th percentile of losses.
            # The percentile rank for a confidence level C is 1 - C.
            percentile_rank = (1 - confidence_level) * 100
            logger.debug(f"Calculating {percentile_rank:.2f} percentile of historical returns.")

            # Calculate VaR at the specified percentile
            # The result is a negative percentage loss
            var_percentage = np.percentile(returns, percentile_rank)

            # VaR is typically expressed as a positive loss
            var_percentage_loss = abs(var_percentage)

            # Scale VaR for the desired horizon (assuming returns are independently and identically distributed)
            # This is a simplification; more advanced methods exist for scaling VaR.
            # Scaling by sqrt(horizon) is common but assumes specific return properties.
            scaled_var_percentage_loss = var_percentage_loss * np.sqrt(horizon_days)

            results['var_percentage_loss'] = scaled_var_percentage_loss * 100 # Convert to percentage for display/scoring
            logger.info(f"Calculated {confidence_level*100:.0f}% VaR ({horizon_days}-day horizon): {results['var_percentage_loss']:.2f}%")
            return results

        except Exception as e:
            logger.error(f"Error calculating VaR: {str(e)}. Returning default results.")
            return results # Return dictionary with NaN/None on error


    def fetch_news(self, symbol: str) -> List[str]:
        """
        Fetches recent news headlines for the given stock symbol.
        NOTE: This is a placeholder. In a real application, you would integrate
        with a News API (like News API, Alpha Vantage, etc.) here.
        """
        print(f"Fetching news for {symbol} (placeholder)...")
        symbol_str = str(symbol) if symbol is not None else "Unknown Stock"
        return [
            f"{symbol_str} stock price rises on positive market sentiment",
            f"{symbol_str} announces strong quarterly results, beating estimates",
            f"Experts bullish on {symbol_str}'s future outlook",
            f"I am a non-numeric entry that might cause an error",
            f"{symbol_str} faces regulatory challenges in key market",
            f"Competitor's new product launch impacts {symbol_str}'s market share",
            f"Global market trends positively impacting {symbol_str}",
            f"{symbol_str} management provides optimistic guidance",
            f"Increased foreign investment flows into {symbol_str}",
            f"Regulatory approval received for {symbol_str}'s new product",
            f"Production issues reported for {symbol_str}",
            f"Increased competition puts pressure on {symbol_str}'s margins",
            f"Analyst downgrades rating for {symbol_str}",
            f"Supply chain disruptions affect {symbol_str}'s operations",
            f"{symbol_str} announces stock split",
            f"Dividend declared by {symbol_str}"
        ]


    def analyze_sentiment(self, news_headlines: List[str]) -> Dict:
        """
        Analyzes the sentiment of news headlines using the Gemini API.
        Returns a dictionary with sentiment counts (positive, neutral, negative)
        and a sentiment score.
        """
        if not self.gemini_model:
            print("Gemini API not configured. Skipping sentiment analysis.")
            return {'positive': 0, 'neutral': 0, 'negative': 0, 'score': 0, 'summary': 'Gemini API not configured. Sentiment analysis skipped.'}

        if not isinstance(news_headlines, list) or not news_headlines:
            print("No news headlines to analyze. Skipping sentiment analysis.")
            return {'positive': 0, 'neutral': 0, 'negative': 0, 'score': 12.5, 'summary': 'No news headlines. Sentiment analysis skipped.'}

        logger.info("Analyzing sentiment using Gemini API...")
        prompt = "Analyze the sentiment of the following news headlines for a stock and categorize each as Positive, Neutral, or Negative. Provide a concise summary of the overall sentiment.\n\n"
        for headline in news_headlines:
            if isinstance(headline, str):
                 prompt += f"- {headline}\n"
            else:
                 logger.warning(f"Skipping non-string headline: {headline}")
                 continue

        if len(prompt) > 30000:
             logger.warning("News headlines too long for Gemini API prompt. Truncating.")
             prompt = prompt[:30000] + "\n... (headlines truncated)"

        try:
            response = self.gemini_model.generate_content(prompt)
            sentiment_text = response.text.strip()
            logger.info(f"Gemini API Response: {sentiment_text[:200]}...")

            positive_count = sentiment_text.lower().count('positive')
            neutral_count = sentiment_text.lower().count('neutral')
            negative_count = sentiment_text.lower().count('negative')

            summary_line = "No summary extracted."
            lines = sentiment_text.split('\n')
            for line in lines:
                 lower_line = line.lower()
                 if 'summary' in lower_line or 'overall sentiment' in lower_line:
                     summary_line = line.strip()
                     break
            if summary_line == "No summary extracted." and lines:
                 for line in reversed(lines):
                     if line.strip():
                         summary_line = line.strip()
                         break

            total_headlines = len([h for h in news_headlines if isinstance(h, str)])
            if total_headlines > 0:
                sentiment_score = ((positive_count - negative_count) / total_headlines) * 12.5 + 12.5
            else:
                sentiment_score = 12.5

            return {
                'positive': positive_count,
                'neutral': neutral_count,
                'negative': negative_count,
                'score': max(0, min(sentiment_score, 25)),
                'summary': summary_line
            }

        except Exception as e:
            logger.error(f"Error analyzing sentiment with Gemini API: {e}. Skipping sentiment analysis.")
            return {'positive': 0, 'neutral': 0, 'negative': 0, 'score': 0, 'summary': 'Sentiment analysis failed.'}


    def generate_enhanced_recommendation(self, fundamental_score, technical_score,
                                       liquidity_score, peer_performance, index_membership,
                                       sentiment_analysis_results, valuation_comparison_results,
                                       downside_risk_metrics: Dict): # Added type hint for clarity
        """Generate comprehensive recommendation including sentiment and valuation comparison"""
        fund_score = fundamental_score if isinstance(fundamental_score, (int, float)) and not pd.isna(fundamental_score) else 0
        tech_score = technical_score if isinstance(technical_score, (int, float)) and not pd.isna(technical_score) else 0
        liq_score = liquidity_score if isinstance(liquidity_score, (int, float)) and not pd.isna(liquidity_score) else 0
        sentiment_score = sentiment_analysis_results.get('score', 0) if isinstance(sentiment_analysis_results.get('score'), (int, float)) and not pd.isna(sentiment_analysis_results.get('score')) else 0

        # Get valuation adjustment score
        valuation_adjustment = valuation_comparison_results.get('valuation_score_adjustment', 0) if isinstance(valuation_comparison_results.get('valuation_score_adjustment'), (int, float)) and not pd.isna(valuation_comparison_results.get('valuation_score_adjustment')) else 0
        logger.info(f"Valuation comparison adjustment: {valuation_adjustment}")

        # Incorporate downside risk into the score
        # A higher VaR indicates higher risk, so it should negatively impact the score
        # Define a simple scoring for VaR (e.g., penalize based on VaR percentage)
        var_percentage = downside_risk_metrics.get('var_percentage_loss', np.nan) # Use .get() as it's now a dict
        downside_risk_adjustment = 0 # Adjustment based on VaR

        if isinstance(var_percentage, (int, float)) and not pd.isna(var_percentage):
             # Example scoring: higher VaR means higher penalty
             if var_percentage > 5: # VaR > 5%
                  downside_risk_adjustment -= 15
                  logger.info(f"VaR ({var_percentage:.2f}%) > 5%. Applying downside risk penalty (-15).")
             elif var_percentage > 3: # VaR > 3%
                  downside_risk_adjustment -= 10
                  logger.info(f"VaR ({var_percentage:.2f}%) > 3%. Applying downside risk penalty (-10).")
             elif var_percentage > 1: # VaR > 1%
                  downside_risk_adjustment -= 5
                  logger.info(f"VaR ({var_percentage:.2f}%) > 1%. Applying downside risk penalty (-5).")
             else:
                  logger.info(f"VaR ({var_percentage:.2f}%) <= 1%. No downside risk penalty.")
        else:
             logger.warning("VaR not available. Cannot apply downside risk adjustment.")
             downside_risk_adjustment -= 2 # Small penalty if VaR cannot be calculated


        total_score = fund_score + tech_score + liq_score + sentiment_score + valuation_adjustment + downside_risk_adjustment # Add valuation and downside risk adjustment


        peer_analysis_available = isinstance(peer_performance, dict) and \
                                  not (pd.isna(peer_performance.get('stock_return')) and \
                                       pd.isna(peer_performance.get('avg_peer_return')) and \
                                       pd.isna(peer_performance.get('relative_performance')))

        if peer_analysis_available and peer_performance.get('outperformance', False) is True:
            total_score += 5
            logger.info("Adding bonus for peer outperformance.")
        elif not peer_analysis_available:
             total_score -= 5
             logger.warning("Applying small penalty as peer performance data is incomplete/unavailable.")

        if isinstance(index_membership, list) and index_membership:
            total_score += 5
            logger.info("Adding bonus for index membership.")

        recommendation = "NEUTRAL"
        confidence = "Low"

        # Re-evaluate thresholds with the new max score (245 + max downside risk adj)
        # Max possible downside risk adjustment is 0, min is -15 (or -2 if N/A)
        # So max total score is still around 235. Thresholds remain the same.
        if total_score >= 190: # Increased threshold
            recommendation = "STRONG BUY"
            confidence = "High"
        elif total_score >= 150: # Increased threshold
            recommendation = "BUY"
            confidence = "Moderate to High"
        elif total_score >= 100: # Same threshold
            recommendation = "HOLD"
            confidence = "Moderate"
        elif total_score >= 70: # Same threshold
            recommendation = "SELL"
            confidence = "Moderate"
        else:
            recommendation = "STRONG SELL"
            confidence = "High"

        logger.info(f"Generated enhanced recommendation: {recommendation} (Total Score: {total_score:.2f})")

        return {
            'recommendation': recommendation,
            'confidence': confidence,
            'total_score': total_score,
            'breakdown': {
                'fundamental': fund_score,
                'technical': tech_score,
                'liquidity': liq_score,
                'sentiment': sentiment_score,
                'valuation_comparison': valuation_adjustment, # Include valuation adjustment in breakdown
                'downside_risk': downside_risk_adjustment, # Include downside risk adjustment
                'peer_adjustment': 5 if peer_analysis_available and peer_performance.get('outperformance', False) is True else (-5 if not peer_analysis_available else 0),
                'index_adjustment': 5 if isinstance(index_membership, list) and index_membership else 0
            }
        }


**Reasoning**:
I have updated the `calculate_value_at_risk` method to return a dictionary as expected by the `generate_enhanced_recommendation` function. The next step is to re-run the integration and report display code to confirm that the `AttributeError` is resolved and that the downside risk metrics are correctly calculated and displayed in the report.



In [ ]:
#@title Part 3: Enhanced Analysis Integration and Report Display (with Downside Risk Analysis - Retry Again 2)

# Assuming IndianStockAnalyzer and EnhancedStockAnalyzer classes are defined in previous cells

# Ensure logging is configured (assuming this was done in a previous cell)
import logging
logger = logging.getLogger(__name__)

# Integration function to work with your Part 1
def analyze_stock_enhanced(symbol, stock_analyzer_part1, enhanced_analyzer):
    """
    Integrate enhanced features with your Part 1 analyzer

    Args:
        symbol: Stock symbol (e.g., 'RELIANCE.NS')
        stock_analyzer_part1: Instance of your IndianStockAnalyzer from Part 1
        enhanced_analyzer: Instance of EnhancedStockAnalyzer
    """
    if not isinstance(symbol, str) or not symbol:
        logger.error("Invalid stock symbol provided for enhanced analysis.")
        return None
    if not isinstance(stock_analyzer_part1, IndianStockAnalyzer):
        logger.error("Invalid IndianStockAnalyzer (Part 1) instance provided.")
        return None
    if not isinstance(enhanced_analyzer, EnhancedStockAnalyzer):
        logger.error("Invalid EnhancedStockAnalyzer (Part 2) instance provided.")
        return None


    logger.info(f"Starting enhanced analysis for {symbol}")
    # Get basic analysis from Part 1
    stock_data = stock_analyzer_part1.analyze_stock(symbol)

    if not stock_data:
        logger.error(f"Basic analysis failed for {symbol}.")
        return None

    # Get current price - already validated in Part 1 analyze_stock
    current_price = stock_data.current_price
    if pd.isna(current_price) or current_price <= 0:
         logger.warning(f"Current price invalid ({current_price}). Enhanced analysis might be limited.")


    # Enhanced analysis
    logger.info("Starting enhanced analysis features (Part 2).")
    # 1. Circuit breaker analysis
    circuit_limits = enhanced_analyzer.get_circuit_limits(symbol)
    # Pass current_price to assess_circuit_risk (already validated as best as possible in Part 1)
    circuit_risk = enhanced_analyzer.assess_circuit_risk(current_price, circuit_limits)

    # 2. Enhanced liquidity analysis
    # Pass price_data (already validated in Part 1 analyze_stock)
    # CORRECTED: Call get_liquidity_score on enhanced_analyzer and pass price_data from stock_data
    liquidity_score = enhanced_analyzer.get_liquidity_score(stock_data.price_data)

    # 3. Peer analysis (including valuation data)
    # Pass info (already validated in Part 1 analyze_stock)
    # The get_peer_companies method now returns a list of dictionaries with valuation data
    peers_with_valuation = enhanced_analyzer.get_peer_companies(symbol, stock_data.info)
    # Pass the list of peer symbols (extracted from the list of dictionaries) for performance comparison
    peer_symbols_for_performance = [p['symbol'] for p in peers_with_valuation if isinstance(p, dict) and 'symbol' in p]
    peer_performance = enhanced_analyzer.calculate_peer_relative_performance(symbol, peer_symbols_for_performance) # Pass just symbols

    # 4. Valuation Analysis: Calculate peer average valuations and compare
    peer_average_valuation = enhanced_analyzer.calculate_peer_average_valuation(peers_with_valuation)
    valuation_comparison_results = enhanced_analyzer.compare_valuation_to_peers(stock_data.metrics, peer_average_valuation)

    # 5. Downside Risk Assessment: Calculate VaR
    downside_risk_metrics = enhanced_analyzer.calculate_value_at_risk(stock_data.price_data)


    # 6. Index membership
    index_membership = enhanced_analyzer.get_index_membership(symbol)

    # 7. Recalculate technical score with circuit risk
    # Pass indicators (already validated in Part 1 calculate_technical_indicators)
    # Pass circuit_risk (already validated in assess_circuit_risk)
    enhanced_technical_score = enhanced_analyzer.calculate_enhanced_technical_score(
        stock_data.indicators, circuit_risk
    )

    # 8. Fetch and analyze news sentiment
    # Pass symbol (already validated)
    news_headlines = enhanced_analyzer.fetch_news(symbol)
    # Pass news_headlines (already validated)
    sentiment_analysis_results = enhanced_analyzer.analyze_sentiment(news_headlines)


    # 9. Generate enhanced recommendation
    # Pass scores and results (already validated within their respective functions)
    final_recommendation = enhanced_analyzer.generate_enhanced_recommendation(
        stock_data.fundamental_score,
        enhanced_technical_score,
        liquidity_score,
        peer_performance, # Pass potentially incomplete/NaN peer performance
        index_membership,
        sentiment_analysis_results, # Pass sentiment analysis results
        valuation_comparison_results, # Pass valuation comparison results
        downside_risk_metrics # Pass downside risk metrics
    )

    # Create enhanced report
    enhanced_report = {
        'basic_analysis': stock_data,
        'enhanced_features': {
            'circuit_limits': circuit_limits,
            'circuit_risk_score': circuit_risk,
            'liquidity_score': liquidity_score, # Ensure correct liquidity score is included
            'peers': peers_with_valuation, # Store the list of peers with their valuation data
            'peer_performance': peer_performance, # Include peer performance, even if NaN
            'peer_average_valuation': peer_average_valuation, # Store peer average valuation
            'valuation_comparison': valuation_comparison_results, # Store valuation comparison results
            'downside_risk': downside_risk_metrics, # Store downside risk metrics (e.g., VaR)
            'index_membership': index_membership,
            'dynamic_risk_free_rate': enhanced_analyzer.risk_free_rate,
            'news_headlines': news_headlines, # Include news in report
            'sentiment_analysis': sentiment_analysis_results # Include sentiment analysis results
        },
        'final_recommendation': final_recommendation
    }
    logger.info(f"Enhanced analysis completed for {symbol}")
    return enhanced_report

# Example usage
if __name__ == "__main__":
    # Initialize analyzers
    try:
        stock_analyzer = IndianStockAnalyzer()
        enhanced_analyzer = EnhancedStockAnalyzer()
    except NameError:
        print("Make sure to run the previous cells defining IndianStockAnalyzer and EnhancedStockAnalyzer classes.")
        exit() # Exit if classes are not defined


    # Analyze a stock
    symbol = "RELIANCE.NS" # Example symbol
    result = analyze_stock_enhanced(symbol, stock_analyzer, enhanced_analyzer)

    if result:
        # Display the enhanced report
        enhanced = result['enhanced_features']
        final = result['final_recommendation']
        basic = result['basic_analysis'] # Get basic data for display

        print(f"\n{'='*60}")
        print(f"Enhanced Analysis Report: {basic.company_name} ({basic.symbol})")
        print(f"{'='*60}")

        print(f"\n📊 Current Market Data:")
        print(f"Current Price: ₹{basic.current_price:,.2f}" if not pd.isna(basic.current_price) else "Current Price: N/A")
        market_cap_cr = basic.market_cap / 10000000 if not pd.isna(basic.market_cap) else np.nan
        print(f"Market Cap: ₹{market_cap_cr:,.2f} Cr" if not pd.isna(market_cap_cr) else "Market Cap: N/A")
        print(f"Sector: {basic.info.get('sector', 'N/A')}")

        print(f"\n⚙️ Enhanced Technical Analysis & Risk:")
        # Display enhanced technical indicators and risk
        # Assuming you want to display relevant indicators from basic.indicators and the circuit risk score
        circuit_limits_display = enhanced.get('circuit_limits')
        if circuit_limits_display and not pd.isna(circuit_limits_display.get('lower_circuit')) and not pd.isna(circuit_limits_display.get('upper_circuit')):
             print(f"Circuit Limits: {circuit_limits_display.get('lower_circuit', 'N/A'):.2f} - {circuit_limits_display.get('upper_circuit', 'N/A'):.2f}")
        else:
             print("Circuit Limits: N/A")

        print(f"Circuit Risk Score: {enhanced.get('circuit_risk_score', 'N/A')}")
        # Check if liquidity score is available before formatting
        liquidity_score_display = f"{enhanced.get('liquidity_score', 'N/A')}/70" if isinstance(enhanced.get('liquidity_score'), (int, float)) and not pd.isna(enhanced.get('liquidity_score')) else "Liquidity Score: N/A/70"
        print(liquidity_score_display)


        # Display some key technical indicators from basic analysis
        print("\nKey Technical Indicators (from Basic Analysis):")
        key_tech_indicators = ['RSI', 'MACD', 'MACD_signal', 'SMA_20', 'SMA_50', 'BB_middle', 'Volume_ratio']
        for indicator_name in key_tech_indicators:
            value = basic.indicators.get(indicator_name, np.nan)
            if not pd.isna(value):
                if isinstance(value, float):
                     print(f"  - {indicator_name}: {value:,.2f}")
                else:
                     print(f"  - {indicator_name}: {value}")
            # Add logging for these key technical indicators if they are calculated
            logger.info(f"Indicator {indicator_name}: {value}")
        else:
            print(f"  - {indicator_name}: N/A")
            logger.info(f"Indicator {indicator_name}: N/A")

        # Display Valuation Analysis
        print(f"\n📈 Valuation Analysis:")
        stock_metrics = basic.metrics
        peer_avg_val = enhanced.get('peer_average_valuation', {})
        val_comp_results = enhanced.get('valuation_comparison', {})

        stock_pe = stock_metrics.get('trailingPE', np.nan)
        stock_pb = stock_metrics.get('priceToBook', np.nan)
        stock_ev_ebitda = stock_metrics.get('evToEbitda', np.nan)

        print(f"Stock Valuation:")
        print(f"  - Trailing PE: {stock_pe:.2f}" if not pd.isna(stock_pe) else "  - Trailing PE: N/A")
        print(f"  - Price to Book: {stock_pb:.2f}" if not pd.isna(stock_pb) else "  - Price to Book: N/A")
        print(f"  - EV/EBITDA: {stock_ev_ebitda:.2f}" if not pd.isna(stock_ev_ebitda) else "  - EV/EBITDA: N/A")

        print("\nPeer Average Valuation:")
        avg_pe_display = f"{peer_avg_val.get('avg_pe', np.nan):.2f}" if not pd.isna(peer_avg_val.get('avg_pe', np.nan)) else "N/A"
        avg_pb_display = f"{peer_avg_val.get('avg_pb', np.nan):.2f}" if not pd.isna(peer_avg_val.get('avg_pb', np.nan)) else "N/A"
        avg_ev_ebitda_display = f"{peer_avg_val.get('avg_ev_ebitda', np.nan):.2f}" if not pd.isna(peer_avg_val.get('avg_ev_ebitda', np.nan)) else "N/A"

        print(f"  - Average PE: {avg_pe_display}")
        print(f"  - Average PB: {avg_pb_display}")
        print(f"  - Average EV/EBITDA: {avg_ev_ebitda_display}")

        print("\nValuation Comparison:")
        print(f"  - PE Comparison: {val_comp_results.get('pe_comparison', 'N/A')}")
        print(f"  - PB Comparison: {val_comp_results.get('pb_comparison', 'N/A')}")
        print(f"  - EV/EBITDA Comparison: {val_comp_results.get('ev_ebitda_comparison', 'N/A')}")
        valuation_adj_display = f"{val_comp_results.get('valuation_score_adjustment', 0):+.2f}" if isinstance(val_comp_results.get('valuation_score_adjustment'), (int, float)) and not pd.isna(val_comp_results.get('valuation_score_adjustment')) else "N/A"
        print(f"  - Valuation Score Adjustment: {valuation_adj_display}")


        print(f"\n📉 Downside Risk Analysis:")
        downside_risk = enhanced.get('downside_risk', {})
        var_percentage = downside_risk.get('var_percentage_loss', np.nan)
        var_display = f"{var_percentage:.2f}%" if not pd.isna(var_percentage) else "N/A"
        # Access 'confidence_level' and 'horizon_days' directly from the downside_risk dictionary
        confidence_level_display = downside_risk.get('confidence_level', 0.95)
        horizon_days_display = downside_risk.get('horizon_days', 1)
        print(f"  - {confidence_level_display*100:.0f}% VaR ({horizon_days_display}-day): {var_display}")

        downside_adj_display = f"{final.get('breakdown', {}).get('downside_risk', np.nan):+.2f}" if isinstance(final.get('breakdown', {}).get('downside_risk'), (int, float)) and not pd.isna(final.get('breakdown', {}).get('downside_risk')) else "N/A"
        print(f"  - Downside Risk Score Adjustment: {downside_adj_display}")



        print(f"\n🤝 Peer Analysis (Performance):")
        peer_perf = enhanced.get('peer_performance')
        if isinstance(peer_perf, dict) and \
           not pd.isna(peer_perf.get('stock_return')) and \
           not pd.isna(peer_perf.get('avg_peer_return')) and \
           not pd.isna(peer_perf.get('relative_performance')):

             stock_return_display = f"{peer_perf.get('stock_return', np.nan):.2f}%" if isinstance(peer_perf.get('stock_return'), (int, float)) and not pd.isna(peer_perf.get('stock_return')) else 'N/A'
             avg_peer_return_display = f"{peer_perf.get('avg_peer_return', np.nan):.2f}%" if isinstance(peer_perf.get('avg_peer_return'), (int, float)) and not pd.isna(peer_perf.get('avg_peer_return')) else 'N/A'
             relative_performance_display = f"{peer_perf.get('relative_performance', np.nan):.2f}%" if isinstance(peer_perf.get('relative_performance'), (int, float)) and not pd.isna(peer_perf.get('relative_performance')) else 'N/A'

             print(f"  - {peer_perf.get('days', 'N/A')}-day Stock Return: {stock_return_display}")
             print(f"  - Avg Peer Return: {avg_peer_return_display}")
             print(f"  - Relative Performance: {relative_performance_display}")
        else:
             print("Peer Performance Data Unavailable.")


        if enhanced.get('index_membership'):
            print(f"\n🏛️ Index Membership: {', '.join(enhanced['index_membership'])}")

        # Sentiment Analysis Results
        sentiment_results = enhanced.get('sentiment_analysis', {})
        print(f"\n📰 News Sentiment Analysis:")
        if enhanced.get('news_headlines'):
             # Only print a few headlines to keep the report concise
             print("Sample News Headlines:")
             # Ensure headlines are valid strings before printing
             valid_headlines = [h for h in enhanced['news_headlines'] if isinstance(h, str)]
             for headline in valid_headlines[:5]: # Print up to 5 headlines
                 print(f"- {headline}")
             if len(valid_headlines) > 5:
                  print("  ...")

             print(f"Sentiment Counts: Positive={sentiment_results.get('positive', 0)}, Neutral={sentiment_results.get('neutral', 0)}, Negative={sentiment_results.get('negative', 0)}")
             sentiment_score_display = f"{sentiment_results.get('score', np.nan):.2f}" if isinstance(sentiment_results.get('score'), (int, float)) and not pd.isna(sentiment_results.get('score')) else "N/A"
             print(f"Sentiment Score: {sentiment_score_display}/25")
             print(f"Sentiment Summary: {sentiment_results.get('summary', 'N/A')}")
        else:
             print("No news headlines fetched or sentiment analysis skipped.")


        print(f"\n--- Comprehensive Scores and Recommendation ---")
        print(f"Fundamental Score (from Part 1): {basic.fundamental_score:.2f}/50" if not pd.isna(basic.fundamental_score) else "Fundamental Score: N/A")
        # Add checks for NaN before formatting float for breakdown scores
        enhanced_tech_score_display = f"{final.get('breakdown', {}).get('technical', np.nan):.2f}" if isinstance(final.get('breakdown', {}).get('technical'), (int, float)) and not pd.isna(final.get('breakdown', {}).get('technical')) else "N/A"
        liquidity_score_display_breakdown = f"{final.get('breakdown', {}).get('liquidity', np.nan):.2f}" if isinstance(final.get('breakdown', {}).get('liquidity'), (int, float)) and not pd.isna(final.get('breakdown', {}).get('liquidity')) else "N/A"
        sentiment_score_display_breakdown = f"{final.get('breakdown', {}).get('sentiment', np.nan):.2f}" if isinstance(sentiment_results.get('score'), (int, float)) and not pd.isna(sentiment_results.get('score')) else "N/A" # Corrected sentiment score breakdown check
        valuation_comp_display_breakdown = f"{final.get('breakdown', {}).get('valuation_comparison', np.nan):.2f}" if isinstance(final.get('breakdown', {}).get('valuation_comparison'), (int, float)) and not pd.isna(final.get('breakdown', {}).get('valuation_comparison')) else "N/A"
        downside_risk_display_breakdown = f"{final.get('breakdown', {}).get('downside_risk', np.nan):.2f}" if isinstance(final.get('breakdown', {}).get('downside_risk'), (int, float)) and not pd.isna(final.get('breakdown', {}).get('downside_risk')) else "N/A"


        print(f"Technical Score (Enhanced): {enhanced_tech_score_display}/50")
        print(f"Liquidity Score: {liquidity_score_display_breakdown}/70")
        print(f"Sentiment Score: {sentiment_score_display_breakdown}/25")
        print(f"Valuation Comparison Adjustment: {valuation_comp_display_breakdown}")
        print(f"Downside Risk Adjustment: {downside_risk_display_breakdown}")


        # Display adjustment scores
        peer_adj_display = f"{final.get('breakdown', {}).get('peer_adjustment', np.nan):+.2f}" if isinstance(final.get('breakdown', {}).get('peer_adjustment'), (int, float)) and not pd.isna(final.get('breakdown', {}).get('peer_adjustment')) else "N/A"
        index_adj_display = f"{final.get('breakdown', {}).get('index_adjustment', np.nan):+.2f}" if isinstance(final.get('breakdown', {}).get('index_adjustment'), (int, float)) and not pd.isna(final.get('breakdown', {}).get('index_adjustment')) else "N/A"

        print(f"Peer Performance Adjustment: {peer_adj_display}")
        print(f"Index Membership Adjustment: {index_adj_display}")


        # Updated max score to reflect the addition of Downside Risk Adjustment (potential max increase of 0, potential max decrease of 15)
        # Max Total Score = Fund (50) + Tech (50) + Liq (70) + Sent (25) + Val Adj (30) + Peer Adj (5) + Index Adj (5) + Downside Adj (0) = 235
        # Re-evaluating the previous max estimate (245) and the new max. Let's use 235 as a potential max for scoring clarity.
        print(f"\nTotal Enhanced Score: {final.get('total_score', 0):.2f}/235" if isinstance(final.get('total_score'), (int, float)) and not pd.isna(final.get('total_score')) else "Total Enhanced Score: N/A")
        print(f"RECOMMENDATION: {final.get('recommendation', 'N/A')} (Confidence: {final.get('confidence', 'N/A')})")


        print(f"\n{'='*60}\n")

**Reasoning**:
The downside risk assessment is now successfully implemented and displayed in the report. The next part of the overall task is to enhance the dividend yield calculation. The subtask is to modify the calculation to consider historical dividend payout trends and stability. This requires modifying the `IndianStockAnalyzer` class's `calculate_fundamental_score` method or adding a new method to the `EnhancedStockAnalyzer` that can access historical dividend data and calculate a more nuanced dividend score. Given that historical data is needed, modifying the `EnhancedStockAnalyzer` is more appropriate as it has access to the price data fetched by the `IndianStockAnalyzer`.



In [ ]:
#@title Part 2: EnhancedStockAnalyzer class definition (with Enhanced Dividend Analysis and Global Macro)

import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import requests
from bs4 import BeautifulSoup
import warnings
warnings.filterwarnings('ignore')

# Import typing for type hints
from typing import List, Dict, Optional

# Ensure logging is configured (assuming this was done in a previous cell)
import logging
logger = logging.getLogger(__name__)


# --- EnhancedStockAnalyzer class definition ---
class EnhancedStockAnalyzer:
    """Enhanced analyzer with additional features for Indian markets"""

    def __init__(self):
        """Initialize the enhanced analyzer."""
        self.risk_free_rate = self.get_dynamic_risk_free_rate()
        # Assuming gemini_model is defined globally from a previous cell
        global gemini_model # Access the global gemini_model
        self.gemini_model = gemini_model

        # More detailed circuit limit percentages based on NSE categorization (simplified)
        self.circuit_limits_percentage = {
            'category_a': 0.20, # Typically high-liquidity index stocks
            'category_b': 0.10, # Most other actively traded stocks
            'category_t': 0.05  # Stocks in trade-for-trade segment
        }

        # Simple mapping of some known stocks to categories for demonstration
        self.stock_category_map = {
            'RELIANCE.NS': 'category_a',
            'TCS.NS': 'category_a',
            'HDFCBANK.NS': 'category_a',
            'INFY.NS': 'category_a',
            'ICICIBANK.NS': 'category_a',
            'VEDL.NS': 'category_b',
            'PNB.NS': 'category_b',
            'INDIGO.NS': 'category_b',
            'BANDHANBNK.NS': 'category_b',
            'PERSISTENT.NS': 'category_b',
            'COFORGE.NS': 'category_b',
            'LTTS.NS': 'category_b',
        }

        # Placeholder for global macro sentiment (simplified)
        # In a real application, this would be dynamic based on fetched data
        self.global_macro_sentiment = 'neutral' # Can be 'positive', 'neutral', 'negative'


    def get_dynamic_risk_free_rate(self):
        """Fetch current 10-year G-Sec yield"""
        try:
            return 0.072  # 7.2% as of recent data
        except Exception as e:
            logger.warning(f"Could not fetch dynamic risk-free rate: {e}. Using fallback rate.")
            return 0.065

    def get_circuit_limits(self, symbol):
        """Get circuit breaker limits for the stock based on categorization"""
        try:
            ticker = yf.Ticker(symbol)
            info = ticker.info

            if not info or 'previousClose' not in info:
                 logger.warning(f"Could not get previous close for {symbol}. Cannot calculate circuit limits.")
                 return None

            prev_close = info.get('previousClose')
            if pd.isna(prev_close) or prev_close <= 0:
                 logger.warning(f"Invalid previous close price for {symbol}: {prev_close}. Cannot calculate circuit limits.")
                 return None


            stock_category = self.stock_category_map.get(symbol.upper(), 'category_b')
            circuit_percent = self.circuit_limits_percentage.get(stock_category, 0.10)

            logger.info(f"Determined circuit percentage for {symbol} (Category: {stock_category}): {circuit_percent*100}%")

            upper_circuit = prev_close * (1 + circuit_percent)
            lower_circuit = prev_close * (1 - circuit_percent)

            return {
                'upper_circuit': upper_circuit,
                'lower_circuit': lower_circuit,
                'circuit_percent': circuit_percent
            }
        except Exception as e:
            logger.error(f"Error getting circuit breaker limits for {symbol}: {str(e)}")
            return None

    def assess_circuit_risk(self, current_price, circuit_limits):
        """Assess proximity to circuit limits"""
        if pd.isna(current_price) or current_price <= 0:
             logger.warning("Invalid current price for circuit risk assessment.")
             return 0

        if not circuit_limits or 'upper_circuit' not in circuit_limits or 'lower_circuit' not in circuit_limits:
            logger.warning("Invalid or incomplete circuit limits provided.")
            return 0

        upper_circuit = circuit_limits['upper_circuit']
        lower_circuit = circuit_limits['lower_circuit']

        if pd.isna(upper_circuit) or pd.isna(lower_circuit) or upper_circuit <= 0 or lower_circuit <= 0:
             logger.warning("Circuit limits are invalid numbers. Cannot assess circuit risk.")
             return 0

        risk_score = 0

        if current_price >= upper_circuit:
             risk_score -= 30
        elif (upper_circuit - current_price) / current_price < 0.01:
            risk_score -= 20
        elif (upper_circuit - current_price) / current_price < 0.03:
            risk_score -= 10
        elif (upper_circuit - current_price) / current_price < 0.05:
            risk_score -= 5

        if current_price <= lower_circuit:
            risk_score -= 40
        elif (current_price - lower_circuit) / current_price < 0.01:
            risk_score -= 30
        elif (current_price - lower_circuit) / current_price < 0.03:
            risk_score -= 15
        elif (current_price - lower_circuit) / current_price < 0.05:
            risk_score -= 8

        logger.info(f"Circuit risk score calculated: {risk_score}")
        return risk_score

    def get_liquidity_score(self, price_data):
        """Enhanced liquidity analysis with NaN handling and better checks"""
        liquidity_score = 0
        if not isinstance(price_data, pd.DataFrame) or price_data.empty or len(price_data) < 20:
            logger.warning("Insufficient price data for liquidity analysis. Assigning default score.")
            return 30

        try:
            avg_volume_20d = price_data['Volume'].rolling(20).mean().iloc[-1]
            avg_volume_5d = price_data['Volume'].rolling(5).mean().iloc[-1]
            current_close = price_data['Close'].iloc[-1]

            # Add logging for diagnosis - before calculation check
            logger.info(f"Volume data for liquidity: avg_volume_20d: {avg_volume_20d}, avg_volume_5d: {avg_volume_5d}, current_close: {current_close}")


            if pd.isna(avg_volume_20d) or pd.isna(avg_volume_5d) or pd.isna(current_close) or current_close <= 0:
                 logger.warning("Liquidity metrics are NaN or invalid. Cannot calculate liquidity score accurately. Assigning default.")
                 return 30

            avg_turnover_20d = np.nan
            if avg_volume_20d is not None and not pd.isna(avg_volume_20d) and avg_volume_20d >= 0 and current_close > 0:
                 avg_turnover_20d = avg_volume_20d * current_close

            volume_spike = np.nan
            if avg_volume_20d is not None and not pd.isna(avg_volume_20d) and avg_volume_20d > 0:
                 if avg_volume_5d is not None and not pd.isna(avg_volume_5d):
                     volume_spike = avg_volume_5d / avg_volume_20d
                 else:
                      logger.warning("5-day average volume is invalid for volume spike calculation.")
            elif not pd.isna(avg_volume_5d) and avg_volume_5d > 0:
                 volume_spike = 1.0
                 logger.warning("20-day average volume is invalid for volume spike, using 1.0 as relative indicator.")
            else:
                 volume_spike = np.nan # Ensure volume_spike is NaN if both volumes are invalid
                 logger.warning("Insufficient or invalid volume data for volume ratio calculation.")

            # Add logging for volume spike - after calculation
            logger.info(f"Volume spike calculated: {volume_spike}")

            # Corrected logic for scoring based on volume_spike
            if not pd.isna(volume_spike):
                if volume_spike > 1.5: # Volume is 50% higher than average
                    liquidity_score += 10
                elif volume_spike < 0.8: # Volume is significantly lower than average
                    liquidity_score += 2 # Small penalty for low volume
                else: # volume_spike is between 0.8 and 1.5 (inclusive of 0.8, exclusive of 1.5)
                    liquidity_score += 5 # Average volume is neutral
            else:
                 logger.warning("Volume spike ratio is NaN. Skipping trend scoring.")


            if not pd.isna(avg_turnover_20d):
                if avg_turnover_20d > 500_00_00_000:
                    liquidity_score += 40
                elif avg_turnover_20d > 100_00_00_000:
                    liquidity_score += 30
                elif avg_turnover_20d > 10_00_00_000:
                    liquidity_score += 20
                elif avg_turnover_20d > 1_00_00_000:
                    liquidity_score += 10
                else:
                    liquidity_score += 5
            else:
                 logger.warning("Average turnover is NaN. Skipping turnover scoring.")
                 liquidity_score += 10

            volume_std = price_data['Volume'].rolling(20).std().iloc[-1] if len(price_data) >= 20 else np.nan
            volume_cv = np.nan
            if avg_volume_20d is not None and not pd.isna(avg_volume_20d) and avg_volume_20d > 0 and \
               volume_std is not None and not pd.isna(volume_std):
                volume_cv = volume_std / avg_volume_20d

            if not pd.isna(volume_cv):
                if volume_cv < 0.5:
                    liquidity_score += 20
                elif volume_cv < 1.0:
                    liquidity_score += 10
            else:
                 logger.warning("Volume coefficient of variation is NaN. Skipping consistency scoring.")

            # The volume spike scoring is now handled above within the main if not pd.isna(volume_spike): block


            return min(liquidity_score, 70)

        except Exception as e:
            logger.error(f"Error calculating liquidity score: {str(e)}")
            return 30

    def get_peer_companies(self, symbol: str, stock_info: Dict) -> List[Dict]: # Modified return type hint
        """Identify peer companies and fetch their valuation ratios for comparison"""
        logger.info(f"Identifying peer companies and fetching valuation data for {symbol}.")
        peer_list_with_valuation = [] # List to store peer info including valuation

        try:
            if not isinstance(stock_info, dict):
                 logger.warning("Invalid stock info for peer identification.")
                 return []

            sector = stock_info.get('sector', '')
            market_cap = stock_info.get('marketCap', 0)

            if not isinstance(sector, str):
                 sector = ''
            if not isinstance(market_cap, (int, float)) or pd.isna(market_cap) or market_cap <= 0:
                 logger.warning("Invalid market cap for peer identification.")
                 market_cap = 0

            if market_cap > 10_00_000_00_00_000:
                cap_category = 'Large Cap'
            elif market_cap > 50_000_00_00_000:
                cap_category = 'Mid Cap'
            else:
                cap_category = 'Small Cap'

            # Peer mapping (simplified - in production, use a comprehensive database)
            peer_map = {
                'Technology': {
                    'Large Cap': ['TCS.NS', 'INFY.NS', 'WIPRO.NS', 'HCLTECH.NS'],
                    'Mid Cap': ['TECHM.NS', 'LTTS.NS', 'PERSISTENT.NS', 'COFORGE.NS'],
                    'Small Cap': ['ZENSAR.NS', 'ECLERX.NS', 'MASTEK.NS']
                },
                'Financial Services': {
                    'Large Cap': ['HDFCBANK.NS', 'ICICIBANK.NS', 'KOTAKBANK.NS', 'AXISBANK.NS'],
                    'Mid Cap': ['IDFCFIRSTB.NS', 'FEDERALBNK.NS', 'INDUSINDBK.NS'],
                    'Small Cap': ['SOUTHBANK.NS', 'UJJIVAN.NS', 'EQUITASBNK.NS']
                },
                'Healthcare': {
                    'Large Cap': ['SUNPHARMA.NS', 'DRREDDY.NS', 'CIPLA.NS', 'DIVISLAB.NS'],
                    'Mid Cap': ['TORNTPHARM.NS', 'ALKEM.NS', 'LUPIN.NS'],
                    'Small Cap': ['CAPLIPOINT.NS', 'BLISS.NS', 'INDOCO.NS']
                },
                # Added Energy sector mapping
                'Energy': {
                    'Large Cap': ['ONGC.NS', 'NTPC.NS', 'POWERGRID.NS'], # Exclude RELIANCE.NS here, filter below
                    'Mid Cap': ['GAIL.NS', 'IOC.NS', 'NHPC.NS'],
                    'Small Cap': ['GUJGASLTD.NS', 'IGL.NS', 'MAHAPOWER.NS']
                }
            }

            potential_peers = peer_map.get(sector, {}).get(cap_category, [])
            # Ensure the current symbol is excluded from the peer list
            filtered_peers = [p for p in potential_peers if isinstance(p, str) and p.upper() != symbol.upper()]

            logger.info(f"Identified {len(filtered_peers)} potential peer companies for {symbol} in {sector} ({cap_category} Cap). Fetching valuation data.")

            # Fetch valuation data for each filtered peer
            for peer_symbol in filtered_peers:
                 try:
                      peer_ticker = yf.Ticker(peer_symbol)
                      peer_info = peer_ticker.info

                      if peer_info:
                           peer_valuation = {
                               'symbol': peer_symbol,
                               'trailingPE': peer_info.get('trailingPE', np.nan),
                               'priceToBook': peer_info.get('priceToBook', np.nan),
                               'enterpriseValue': peer_info.get('enterpriseValue', np.nan), # Fetch for EV/EBITDA calculation
                               'ebitda': peer_info.get('ebitda', np.nan) # Fetch for EV/EBITDA calculation
                           }

                           # Calculate EV/EBITDA for the peer
                           ev = peer_valuation['enterpriseValue']
                           ebitda = peer_valuation['ebitda']
                           peer_valuation['evToEbitda'] = np.nan # Default to NaN
                           if isinstance(ev, (int, float)) and not np.isnan(ev) and \
                              isinstance(ebitda, (int, float)) and not np.isnan(ebitda) and ebitda != 0:
                               peer_valuation['evToEbitda'] = ev / ebitda
                           elif (isinstance(ev, (int, float)) and not np.isnan(ev)) and \
                                (isinstance(ebitda, (int, float)) and not np.isnan(ebitda) and ebitda == 0):
                                logger.warning(f"EBITDA is zero for peer {peer_symbol}. Cannot calculate EV/EBITDA.")
                           else:
                                logger.warning(f"Missing Enterprise Value ({ev}) or EBITDA ({ebitda}) for peer {peer_symbol}. Cannot calculate EV/EBITDA.")


                           peer_list_with_valuation.append(peer_valuation)
                           logger.debug(f"Fetched valuation data for peer {peer_symbol}: PE={peer_valuation['trailingPE']:.2f}, PB={peer_valuation['priceToBook']:.2f}, EV/EBITDA={peer_valuation['evToEbitda']:.2f}")
                      else:
                           logger.warning(f"Could not fetch info for peer {peer_symbol}. Skipping.")
                 except Exception as e:
                      logger.warning(f"Error fetching data for peer {peer_symbol}: {str(e)}. Skipping.")

            logger.info(f"Successfully fetched valuation data for {len(peer_list_with_valuation)} peers.")
            return peer_list_with_valuation[:5] # Return top 5 peers with their valuation data

        except Exception as e:
            logger.error(f"Error getting peer companies and valuation data for {symbol}: {str(e)}")
            return [] # Return empty list on error

    def calculate_peer_average_valuation(self, peers_with_valuation: List[Dict]) -> Dict:
        """Calculate average valuation ratios (PE, PB, EV/EBITDA) for a list of peers."""
        logger.info("Calculating peer average valuation ratios.")
        total_pe = 0
        total_pb = 0
        total_ev_ebitda = 0
        count_pe = 0
        count_pb = 0
        count_ev_ebitda = 0

        if not isinstance(peers_with_valuation, list) or not peers_with_valuation:
             logger.warning("No peer valuation data provided for calculating averages.")
             return {'avg_pe': np.nan, 'avg_pb': np.nan, 'avg_ev_ebitda': np.nan}


        for peer_data in peers_with_valuation:
             if isinstance(peer_data, dict):
                 pe = peer_data.get('trailingPE', np.nan)
                 pb = peer_data.get('priceToBook', np.nan)
                 ev_ebitda = peer_data.get('evToEbitda', np.nan)

                 if isinstance(pe, (int, float)) and not np.isnan(pe) and pe > 0:
                     total_pe += pe
                     count_pe += 1
                 else:
                      logger.debug(f"Skipping invalid PE for peer: {peer_data.get('symbol', 'Unknown')}")

                 if isinstance(pb, (int, float)) and not np.isnan(pb) and pb > 0:
                     total_pb += pb
                     count_pb += 1
                 else:
                      logger.debug(f"Skipping invalid PB for peer: {peer_data.get('symbol', 'Unknown')}")


                 if isinstance(ev_ebitda, (int, float)) and not np.isnan(ev_ebitda): # EV/EBITDA can be zero or negative in some cases, but >0 is typical
                      total_ev_ebitda += ev_ebitda
                      count_ev_ebitda += 1
                 else:
                      logger.debug(f"Skipping invalid EV/EBITDA for peer: {peer_data.get('symbol', 'Unknown')}")


        avg_pe = total_pe / count_pe if count_pe > 0 else np.nan
        avg_pb = total_pb / count_pb if count_pb > 0 else np.nan
        avg_ev_ebitda = total_ev_ebitda / count_ev_ebitda if count_ev_ebitda > 0 else np.nan

        logger.info(f"Calculated peer averages: PE={avg_pe:.2f}, PB={avg_pb:.2f}, EV/EBITDA={avg_ev_ebitda:.2f}")

        return {
            'avg_pe': avg_pe,
            'avg_pb': avg_pb,
            'avg_ev_ebitda': avg_ev_ebitda
        }


    def compare_valuation_to_peers(self, stock_valuation: Dict, peer_average_valuation: Dict) -> Dict:
        """Compare stock's valuation ratios to peer averages and provide assessment."""
        logger.info("Comparing stock valuation to peer averages.")
        comparison_results = {}
        valuation_score_adjustment = 0 # Adjustment to the total score

        stock_pe = stock_valuation.get('trailingPE', np.nan)
        stock_pb = stock_valuation.get('priceToBook', np.nan)
        stock_ev_ebitda = stock_valuation.get('evToEbitda', np.nan)

        avg_pe = peer_average_valuation.get('avg_pe', np.nan)
        avg_pb = peer_average_valuation.get('avg_pb', np.nan)
        avg_ev_ebitda = peer_average_valuation.get('avg_ev_ebitda', np.nan)

        logger.debug(f"Stock Valuation: PE={stock_pe:.2f}, PB={stock_pb:.2f}, EV/EBITDA={stock_ev_ebitda:.2f}")
        logger.debug(f"Peer Average Valuation: PE={avg_pe:.2f}, PB={avg_pb:.2f}, EV/EBITDA={avg_ev_ebitda:.2f}")


        # PE Ratio comparison (Lower is generally better relative to peers)
        comparison_results['pe_comparison'] = 'N/A'
        if not pd.isna(stock_pe) and not pd.isna(avg_pe) and avg_pe > 0:
             if stock_pe < avg_pe * 0.8: # More than 20% below average
                  comparison_results['pe_comparison'] = 'Significantly Undervalued (PE)'
                  valuation_score_adjustment += 10
             elif stock_pe < avg_pe * 0.95: # More than 5% below average
                  comparison_results['pe_comparison'] = 'Undervalued (PE)'
                  valuation_score_adjustment += 5
             elif stock_pe > avg_pe * 1.2: # More than 20% above average
                  comparison_results['pe_comparison'] = 'Significantly Overvalued (PE)'
                  valuation_score_adjustment -= 10
             elif stock_pe > avg_pe * 1.05: # More than 5% above average
                  comparison_results['pe_comparison'] = 'Overvalued (PE)'
                  valuation_score_adjustment -= 5
             else:
                  comparison_results['pe_comparison'] = 'Fairly Valued (PE)'
        elif not pd.isna(stock_pe) and pd.isna(avg_pe):
             comparison_results['pe_comparison'] = 'PE available, Peer Avg PE N/A'
             logger.warning("Peer Average PE is NaN. Cannot compare stock PE to peers.")
        elif pd.isna(stock_pe) and not pd.isna(avg_pe):
             comparison_results['pe_comparison'] = 'Stock PE N/A'
             logger.warning("Stock PE is NaN. Cannot compare to peer Average PE.")
        else:
             logger.warning("Stock PE and Peer Average PE are NaN. Cannot compare.")


        # PB Ratio comparison (Lower is generally better relative to peers)
        comparison_results['pb_comparison'] = 'N/A'
        if not pd.isna(stock_pb) and not pd.isna(avg_pb) and avg_pb > 0:
             if stock_pb < avg_pb * 0.8: # More than 20% below average
                  comparison_results['pb_comparison'] = 'Significantly Undervalued (PB)'
                  valuation_score_adjustment += 8
             elif stock_pb < avg_pb * 0.95: # More than 5% below average
                  comparison_results['pb_comparison'] = 'Undervalued (PB)'
                  valuation_score_adjustment += 4
             elif stock_pb > avg_pb * 1.2: # More than 20% above average
                  comparison_results['pb_comparison'] = 'Significantly Overvalued (PB)'
                  valuation_score_adjustment -= 8
             elif stock_pb > avg_pb * 1.05: # More than 5% above average
                  comparison_results['pb_comparison'] = 'Overvalued (PB)'
                  valuation_score_adjustment -= 4
             else:
                  comparison_results['pb_comparison'] = 'Fairly Valued (PB)'
        elif not pd.isna(stock_pb) and pd.isna(avg_pb):
             comparison_results['pb_comparison'] = 'PB available, Peer Avg PB N/A'
             logger.warning("Peer Average PB is NaN. Cannot compare stock PB to peers.")
        elif pd.isna(stock_pb) and not pd.isna(avg_pb):
             comparison_results['pb_comparison'] = 'Stock PB N/A'
             logger.warning("Stock PB is NaN. Cannot compare to peer Average PB.")
        else:
             logger.warning("Stock PB and Peer Average PB are NaN. Cannot compare.")


        # EV/EBITDA comparison (Lower is generally better relative to peers)
        comparison_results['ev_ebitda_comparison'] = 'N/A'
        if not pd.isna(stock_ev_ebitda) and not pd.isna(avg_ev_ebitda) and avg_ev_ebitda != 0:
             if stock_ev_ebitda < avg_ev_ebitda * 0.8: # More than 20% below average
                  comparison_results['ev_ebitda_comparison'] = 'Significantly Undervalued (EV/EBITDA)'
                  valuation_score_adjustment += 12
             elif stock_ev_ebitda < avg_ev_ebitda * 0.95: # More than 5% below average
                  comparison_results['ev_ebitda_comparison'] = 'Undervalued (EV/EBITDA)'
                  valuation_score_adjustment += 6
             elif stock_ev_ebitda > avg_ev_ebitda * 1.2: # More than 20% above average
                  comparison_results['ev_ebitda_comparison'] = 'Significantly Overvalued (EV/EBITDA)'
                  valuation_score_adjustment -= 12
             elif stock_ev_ebitda > avg_ev_ebitda * 1.05: # More than 5% above average
                  comparison_results['ev_ebitda_comparison'] = 'Overvalued (EV/EBITDA)'
                  valuation_score_adjustment -= 6
             else:
                  comparison_results['ev_ebitda_comparison'] = 'Fairly Valued (EV/EBITDA)'
        elif not pd.isna(stock_ev_ebitda) and not pd.isna(avg_ev_ebitda) and avg_ev_ebitda == 0:
             comparison_results['ev_ebitda_comparison'] = 'Peer Avg EV/EBITDA is 0'
             logger.warning("Peer Average EV/EBITDA is zero. Cannot compare.")
        elif not pd.isna(stock_ev_ebitda) and pd.isna(avg_ev_ebitda):
             comparison_results['ev_ebitda_comparison'] = 'EV/EBITDA available, Peer Avg EV/EBITDA N/A'
             logger.warning("Peer Average EV/EBITDA is NaN. Cannot compare stock EV/EBITDA to peers.")
        elif pd.isna(stock_ev_ebitda) and not pd.isna(avg_ev_ebitda):
             comparison_results['ev_ebitda_comparison'] = 'Stock EV/EBITDA N/A'
             logger.warning("Stock EV/EBITDA is NaN. Cannot compare to peer Average EV/EBITDA.")
        else:
             logger.warning("Stock EV/EBITDA and Peer Average EV/EBITDA are NaN. Cannot compare.")


        comparison_results['valuation_score_adjustment'] = valuation_score_adjustment

        logger.info(f"Valuation comparison results: {comparison_results}")
        return comparison_results


    def calculate_peer_relative_performance(self, symbol, peers, days=30):
        """Compare stock performance with peers, handling data issues robustly"""
        results = {
            'stock_return': np.nan,
            'avg_peer_return': np.nan,
            'relative_performance': np.nan,
            'outperformance': False,
            'days': days
        }

        if not isinstance(peers, list) or not peers:
             logger.warning("No valid peer list provided for performance comparison.")
             # Return the default results with NaNs if no peers
             return results

        # --- Removed the temporary skip here ---

        try:
            main_ticker = yf.Ticker(symbol)
            main_hist = main_ticker.history(period=f"{days}d")
            main_return = np.nan

            if not main_hist.empty and len(main_hist) > 1:
                try:
                    start_price = main_hist['Close'].iloc[0]
                    end_price = main_hist['Close'].iloc[-1]
                    if not pd.isna(start_price) and not pd.isna(end_price) and start_price > 0:
                         main_return = ((end_price / start_price) - 1) * 100
                    else:
                         logger.warning(f"Invalid start ({start_price}) or end price ({end_price}) for main stock ({symbol}) return calculation.")
                except Exception as ret_e:
                     logger.warning(f"Could not calculate main stock ({symbol}) return: {ret_e}. Setting to NaN.")
                     main_return = np.nan
            else:
                logger.warning(f"Insufficient historical data for main stock ({symbol}) for return calculation.")

            results['stock_return'] = main_return

        except Exception as main_stock_e:
            logger.error(f"Error fetching data or calculating return for main stock ({symbol}): {main_stock_e}. Main stock return set to NaN.")
            results['stock_return'] = np.nan

        # Calculate peer returns
        peer_returns = []
        # The peers list now contains dictionaries with valuation data.
        # We need to extract just the symbols for performance calculation.
        peer_symbols_for_perf = [p['symbol'] for p in peers if isinstance(p, dict) and 'symbol' in p]

        for peer_symbol in peer_symbols_for_perf:
             try:
                 peer_ticker = yf.Ticker(peer_symbol)
                 peer_hist = peer_ticker.history(period=f"{days}d")
                 if not peer_hist.empty and len(peer_hist) > 1:
                     try:
                          peer_start_price = peer_hist['Close'].iloc[0]
                          peer_end_price = peer_hist['Close'].iloc[-1]
                          if not pd.isna(peer_start_price) and not pd.isna(peer_end_price) and peer_start_price > 0:
                             peer_return = ((peer_end_price / peer_start_price) - 1) * 100
                             peer_returns.append(peer_return)
                          else:
                             logger.warning(f"Invalid start or end price for peer ({peer_symbol}) return calculation.")
                     except Exception as peer_ret_e:
                         logger.warning(f"Could not calculate return for peer ({peer_symbol}): {peer_ret_e}. Skipping.")
                 else:
                     logger.warning(f"Insufficient historical data for peer ({peer_symbol}) for return calculation. Skipping.")
             except Exception as peer_e:
                 logger.warning(f"Error fetching data for peer ({peer_symbol}): {peer_e}. Skipping.")

        # Add logging for peer returns before filtering/average calculation
        logger.info(f"Peer returns collected before average calculation attempt: {peer_returns}")

        # Calculate average peer return and relative performance
        if peer_returns:
             # Ensure peer_returns list contains only valid numbers before calculating mean
             valid_peer_returns = [ret for ret in peer_returns if isinstance(ret, (int, float)) and not pd.isna(ret)]
             if valid_peer_returns:
                 avg_peer_return = np.mean(valid_peer_returns)
                 results['avg_peer_return'] = avg_peer_return

                 # Calculate relative performance only if main stock return is valid
                 if not pd.isna(main_return):
                      results['relative_performance'] = main_return - avg_peer_return

                      # Determine outperformance
                      if main_return > avg_peer_return:
                          results['outperformance'] = True
                 else:
                      logger.warning("Main stock return is NaN, cannot calculate relative performance or outperformance.")
             else:
                 logger.warning("No valid peer returns available after filtering. Cannot calculate average peer return or relative performance.")
        else:
             logger.warning("No peer returns calculated. Cannot calculate average peer return or relative performance.")


        logger.info(f"Peer performance results for {symbol}: {results}")
        return results


    def get_index_membership(self, symbol):
        """Check index membership for the stock"""
        symbol_upper = symbol.upper() if isinstance(symbol, str) else ''

        index_constituents = {
            'NIFTY50': ['RELIANCE.NS', 'TCS.NS', 'HDFCBANK.NS', 'INFY.NS', 'ICICIBANK.NS'],
            'NIFTY_NEXT50': ['VEDL.NS', 'PNB.NS', 'INDIGO.NS', 'BANDHANBNK.NS'],
            'NIFTY_MIDCAP150': ['PERSISTENT.NS', 'COFORGE.NS', 'LTTS.NS']
        }

        membership = []
        for index, constituents in index_constituents.items():
            if symbol_upper and any(isinstance(c, str) and c.upper() == symbol_upper for c in constituents):
                membership.append(index)

        if membership:
             logger.info(f"Stock {symbol} is a member of indices: {', '.join(membership)}")
        else:
             logger.info(f"Stock {symbol} is not found in the hardcoded index constituents.")

        return membership

    def calculate_enhanced_technical_score(self, indicators, circuit_risk):
        """Calculate enhanced technical analysis score with NaN handling and circuit risk"""
        base_score = 0
        indicators_calculated = 0
        possible_indicator_points = 50

        num_key_indicators = 5
        indicator_point_contribution = possible_indicator_points / num_key_indicators

        rsi = indicators.get('RSI', np.nan)
        if not pd.isna(rsi):
             indicators_calculated += 1
             if 40 <= rsi <= 60:
                 base_score += indicator_point_contribution * 0.7
             elif rsi < 30: # More points for extreme oversold
                 base_score += indicator_point_contribution * 1.0
             elif 30 <= rsi < 40: # Approaching oversold
                 base_score += indicator_point_contribution * 0.9
             elif 60 < rsi <= 70: # Approaching overbought
                 base_score += indicator_point_contribution * 0.6
             else: # Overbought (>70)
                 base_score += indicator_point_contribution * 0.4
        else:
             logger.warning("RSI not calculated or is NaN.")

        macd = indicators.get('MACD', np.nan)
        macd_signal = indicators.get('MACD_signal', np.nan)

        if not pd.isna(macd) and not pd.isna(macd_signal):
             indicators_calculated += 1
             if macd > macd_signal:
                 if macd > 0: # Bullish crossover above zero line
                      base_score += indicator_point_contribution * 1.0
                 else: # Bullish crossover below zero line
                      base_score += indicator_point_contribution * 0.8
             else:
                 if macd < 0: # Bearish crossover below zero line
                      base_score += indicator_point_contribution * 0.2
                 else: # Bearish crossover above zero line
                      base_score += indicator_point_contribution * 0.4
        else:
             logger.warning("MACD or MACD signal is NaN for technical scoring.")

        sma_20 = indicators.get('SMA_20', np.nan)
        sma_50 = indicators.get('SMA_50', np.nan)
        sma_200 = indicators.get('SMA_200', np.nan) # Include 200-day SMA if available

        ma_score_component = 0
        ma_indicators_counted = 0

        if not pd.isna(sma_20) and not pd.isna(sma_50):
             ma_indicators_counted += 1
             if sma_20 > sma_50:
                  ma_score_component += 0.5 # 20 > 50 crossover is bullish
             else:
                  ma_score_component += 0.2 # 20 <= 50 crossover is bearish/neutral

        if not pd.isna(sma_50) and not pd.isna(sma_200):
             ma_indicators_counted += 1
             if sma_50 > sma_200:
                  ma_score_component += 0.5 # 50 > 200 crossover is bullish
             else:
                  ma_score_component += 0.2 # 50 <= 200 crossover is bearish/neutral

        current_price = indicators.get('current_price', np.nan)
        if not pd.isna(current_price):
             if not pd.isna(sma_20) and current_price > sma_20:
                  ma_score_component += 0.3 # Price above 20-day SMA is bullish
             elif not pd.isna(sma_20):
                  ma_score_component += 0.1 # Price below 20-day SMA is bearish/neutral

             if not pd.isna(sma_50) and current_price > sma_50:
                  ma_score_component += 0.4 # Price above 50-day SMA is more bullish
             elif not pd.isna(sma_50):
                  ma_score_component += 0.15 # Price below 50-day SMA

             if not pd.isna(sma_200) and current_price > sma_200:
                  ma_score_component += 0.6 # Price above 200-day SMA is significant bullish signal
             elif not pd.isna(sma_200):
                  ma_score_component += 0.1 # Price below 200-day SMA is significant bearish signal


        if ma_indicators_counted > 0 or (not pd.isna(current_price) and (not pd.isna(sma_20) or not pd.isna(sma_50) or not pd.isna(sma_200))):
             indicators_calculated += 1 # Count MA section if any valid MA comparison/position is made
             # Normalize MA score component to contribute to the total score
             # Max possible ma_score_component (0.5 + 0.5 + 0.3 + 0.4 + 0.6) = 2.3
             # Let's scale this to contribute up to indicator_point_contribution
             max_ma_component = 2.3
             base_score += (ma_score_component / max_ma_component) * indicator_point_contribution if max_ma_component > 0 else 0
        else:
             logger.warning("Insufficient data for Moving Averages analysis.")


        bb_upper = indicators.get('BB_upper', np.nan)
        bb_lower = indicators.get('BB_lower', np.nan)
        bb_middle = indicators.get('BB_middle', np.nan)

        if not pd.isna(current_price) and not pd.isna(bb_upper) and not pd.isna(bb_lower) and not pd.isna(bb_middle):
            indicators_calculated += 1
            if current_price < bb_lower:
                base_score += indicator_point_contribution * 1.0 # Price below lower band (potential buy signal)
            elif bb_lower <= current_price < bb_middle:
                base_score += indicator_point_contribution * 0.8 # Between lower and middle band
            elif bb_middle <= current_price < bb_upper:
                base_score += indicator_point_contribution * 0.6 # Between middle and upper band
            else: # Price above upper band
                base_score += indicator_point_contribution * 0.4 # Potential sell signal
        else:
             logger.warning("Bollinger Bands or current price is NaN for technical scoring.")

        volume_ratio = indicators.get('Volume_ratio', np.nan)
        if not pd.isna(volume_ratio):
             indicators_calculated += 1
             if volume_ratio > 2.0: # Very high volume
                 base_score += indicator_point_contribution * 1.0
             elif volume_ratio > 1.2: # High volume
                 base_score += indicator_point_contribution * 0.8
             elif 0.8 <= volume_ratio <= 1.2: # Normal volume
                 base_score += indicator_point_contribution * 0.5
             else: # Low volume
                 base_score += indicator_point_contribution * 0.3
        else:
             logger.warning("Volume ratio is NaN for technical scoring.")


        achieved_score_from_indicators = base_score

        # Normalize the base score based on how many indicators were successfully calculated
        if indicators_calculated > 0:
            # Assuming each of the num_key_indicators contributes equally if available
            # This approach gives points for each indicator that could be calculated and scored
            # Max points if all indicators could be scored is num_key_indicators * indicator_point_contribution
            # But our scoring within each indicator gives variable points (e.g., 0.4 to 1.0)
            # A simpler approach is to just sum up the points from successfully scored indicators
            # Let's cap the sum at the max possible points from indicators (50)
            final_base_score = min(achieved_score_from_indicators, possible_indicator_points)

            if indicators_calculated < num_key_indicators:
                 logger.warning(f"Only {indicators_calculated}/{num_key_indicators} key technical indicators could be fully calculated/scored. Final base technical score is based on available data.")

        else:
             logger.warning("No key technical indicators calculated. Base technical score is 0.")
             final_base_score = 0


        circuit_risk_adj = circuit_risk if isinstance(circuit_risk, (int, float)) and not pd.isna(circuit_risk) else 0

        final_score = final_base_score + circuit_risk_adj

        return max(0, min(final_score, 50))


    def calculate_value_at_risk(self, price_data: pd.DataFrame, confidence_level: float = 0.95, horizon_days: int = 1) -> Dict: # Modified return type hint
        """
        Calculates Value at Risk (VaR) using the historical method.

        Args:
            price_data: DataFrame with historical price data (must contain 'Close').
            confidence_level: The confidence level for VaR (e.g., 0.95 for 95% VaR).
            horizon_days: The time horizon for VaR (in trading days).

        Returns:
            A dictionary containing the calculated VaR as a percentage loss,
            confidence level, and horizon days, or a dictionary with NaN/None values
            if calculation fails.
        """
        logger.info(f"Calculating {confidence_level*100:.0f}% VaR for {horizon_days}-day horizon.")
        results = {'var_percentage_loss': np.nan, 'confidence_level': confidence_level, 'horizon_days': horizon_days}

        if not isinstance(price_data, pd.DataFrame) or price_data.empty or 'Close' not in price_data.columns:
            logger.warning("Insufficient or invalid price data for VaR calculation.")
            return results # Return dictionary with NaN/None


        try:
            # Calculate daily returns
            returns = price_data['Close'].pct_change().dropna()

            if returns.empty:
                logger.warning("No valid returns data for VaR calculation.")
                return results # Return dictionary with NaN/None

            # Historical VaR: Find the percentile of returns corresponding to the confidence level
            # For downside risk, we look at the lower tail of returns.
            # A 95% confidence level for VaR means we're interested in the 5th percentile of losses.
            # The percentile rank for a confidence level C is 1 - C.
            percentile_rank = (1 - confidence_level) * 100
            logger.debug(f"Calculating {percentile_rank:.2f} percentile of historical returns.")

            # Calculate VaR at the specified percentile
            # The result is a negative percentage loss
            var_percentage = np.percentile(returns, percentile_rank)

            # VaR is typically expressed as a positive loss
            var_percentage_loss = abs(var_percentage)

            # Scale VaR for the desired horizon (assuming returns are independently and identically distributed)
            # This is a simplification; more advanced methods exist for scaling VaR.
            # Scaling by sqrt(horizon) is common but assumes specific return properties.
            scaled_var_percentage_loss = var_percentage_loss * np.sqrt(horizon_days)

            results['var_percentage_loss'] = scaled_var_percentage_loss * 100 # Convert to percentage for display/scoring
            logger.info(f"Calculated {confidence_level*100:.0f}% VaR ({horizon_days}-day horizon): {results['var_percentage_loss']:.2f}%")
            return results

        except Exception as e:
            logger.error(f"Error calculating VaR: {str(e)}. Returning default results.")
            return results # Return dictionary with NaN/None on error


    def calculate_enhanced_dividend_score(self, symbol: str, price_data: pd.DataFrame) -> Dict:
        """
        Calculates an enhanced dividend score considering historical payout trends and stability.

        Args:
            symbol: Stock symbol (e.g., 'RELIANCE.NS').
            price_data: DataFrame with historical price data (must contain 'Close').

        Returns:
            A dictionary containing the enhanced dividend score and potentially other
            relevant dividend metrics (e.g., payout history, yield trend).
        """
        logger.info(f"Calculating enhanced dividend score for {symbol}.")
        enhanced_dividend_score = 0 # Max score for dividend analysis out of 15 (example)
        dividend_metrics = {
            'trailingAnnualDividendYield': np.nan,
            'dividendPayoutRatio': np.nan,
            'payout_history': {},
            'yield_trend': 'N/A',
            'score': enhanced_dividend_score
        }

        if not isinstance(price_data, pd.DataFrame) or price_data.empty or 'Close' not in price_data.columns:
             logger.warning("Insufficient price data for enhanced dividend analysis.")
             return dividend_metrics # Return default metrics if price data is missing


        try:
            ticker = yf.Ticker(symbol)
            info = ticker.info
            dividends = ticker.dividends.sort_index() # Get historical dividend payouts

            # Get current yield and payout ratio from info
            current_yield = info.get('trailingAnnualDividendYield', np.nan) # yfinance provides this as a ratio
            if isinstance(current_yield, (int, float)) and not pd.isna(current_yield):
                 dividend_metrics['trailingAnnualDividendYield'] = current_yield * 100 # Convert to percentage
            else:
                 dividend_metrics['trailingAnnualDividendYield'] = np.nan


            payout_ratio = info.get('payoutRatio', np.nan) # yfinance provides this as a ratio
            if isinstance(payout_ratio, (int, float)) and not pd.isna(payout_ratio):
                 dividend_metrics['dividendPayoutRatio'] = payout_ratio # Keep as ratio
            else:
                 dividend_metrics['dividendPayoutRatio'] = np.nan


            if not dividends.empty:
                 # Analyze payout history and stability
                 dividend_metrics['payout_history'] = dividends.to_dict() # Store the raw history

                 # Check for consistent payouts (e.g., increasing or stable over last 5 years)
                 five_years_ago = datetime.now() - timedelta(days=5*365)
                 recent_dividends = dividends[dividends.index >= five_years_ago]

                 if len(recent_dividends) > 3: # Need at least a few recent payouts
                      # Check for increasing trend
                      is_increasing = all(recent_dividends.iloc[i] >= recent_dividends.iloc[i-1] for i in range(1, len(recent_dividends)))

                      if is_increasing:
                           enhanced_dividend_score += 10 # Significant points for increasing dividends
                           dividend_metrics['yield_trend'] = 'Increasing'
                           logger.debug("Dividend trend: Increasing. +10 pts.")
                      elif len(recent_dividends) > 0 and recent_dividends.min() > 0:
                           enhanced_dividend_score += 5 # Some points for stable (non-zero) dividends
                           dividend_metrics['yield_trend'] = 'Stable'
                           logger.debug("Dividend trend: Stable. +5 pts.")
                      else:
                           dividend_metrics['yield_trend'] = 'Irregular/Decreasing'
                           logger.debug("Dividend trend: Irregular/Decreasing.")
                 elif len(recent_dividends) > 0 and recent_dividends.min() > 0:
                      enhanced_dividend_score += 3 # Some points for recent payouts, even if history is short
                      dividend_metrics['yield_trend'] = 'Recent Payouts'
                      logger.debug("Dividend trend: Recent Payouts. +3 pts.")
                 else:
                      dividend_metrics['yield_trend'] = 'No Recent Payouts'
                      logger.debug("Dividend trend: No Recent Payouts.")


                 # Score based on current yield (Similar to basic analysis but potentially more weight)
                 if not pd.isna(dividend_metrics['trailingAnnualDividendYield']):
                      yield_percentage = dividend_metrics['trailingAnnualDividendYield']
                      if yield_percentage > 3: # > 3%
                           enhanced_dividend_score += 5
                           logger.debug(f"Current Yield ({yield_percentage:.2f}%) > 3%. +5 pts.")
                      elif yield_percentage > 1: # > 1%
                           enhanced_dividend_score += 3
                           logger.debug(f"Current Yield ({yield_percentage:.2f}%) > 1%. +3 pts.")
                      else:
                           enhanced_dividend_score += 1 # Small point for non-zero yield
                           logger.debug(f"Current Yield ({yield_percentage:.2f}) <= 1%. +1 pt.")
                 else:
                      logger.warning("Current dividend yield is NaN.")

                 # Consider Payout Ratio (Lower is generally better, indicates sustainability)
                 if not pd.isna(dividend_metrics['dividendPayoutRatio']):
                      payout_ratio_value = dividend_metrics['dividendPayoutRatio']
                      if payout_ratio_value < 0.5: # Payout ratio < 50%
                           enhanced_dividend_score += 5
                           logger.debug(f"Payout Ratio ({payout_ratio_value:.2f}) < 0.5. +5 pts.")
                      elif payout_ratio_value < 0.8: # Payout ratio < 80%
                           enhanced_dividend_score += 3
                           logger.debug(f"Payout Ratio ({payout_ratio_value:.2f}) < 0.8. +3 pts.")
                      elif payout_ratio_value > 1.0: # Payout ratio > 100% (unsustainable)
                           enhanced_dividend_score -= 5 # Penalty for unsustainable payout
                           logger.debug(f"Payout Ratio ({payout_ratio_value:.2f}) > 1.0. -5 pts.")
                      else:
                           enhanced_dividend_score += 1 # Small point for payout ratio 0.8-1.0
                           logger.debug(f"Payout Ratio ({payout_ratio_value:.2f}) 0.8-1.0. +1 pt.")

                 else:
                      logger.warning("Dividend payout ratio is NaN.")


            else:
                 logger.warning("No historical dividend data found.")

            # Cap the score at a reasonable maximum for this component (e.g., 20 points out of 235 total)
            # Let's allocate up to 20 points for enhanced dividend analysis
            dividend_metrics['score'] = min(enhanced_dividend_score, 20)


        except Exception as e:
            logger.error(f"Error calculating enhanced dividend score for {symbol}: {str(e)}. Returning default metrics.")
            # Ensure the score is set to 0 on error
            dividend_metrics['score'] = 0
            return dividend_metrics # Return dictionary with NaN/None on error


        logger.info(f"Enhanced dividend score calculated: {dividend_metrics['score']}")
        return dividend_metrics

    def assess_global_macro_impact(self) -> Dict:
        """
        Assesses global macro impacts and returns an adjustment to the score.
        NOTE: This is a simplified placeholder. In a real application, this would
        involve fetching and analyzing real global economic data (e.g., GDP growth,
        inflation, interest rates, geopolitical events).
        """
        logger.info("Assessing global macro impact (placeholder).")
        macro_adjustment = 0
        macro_sentiment = self.global_macro_sentiment # Get the current simulated sentiment

        if macro_sentiment == 'positive':
            macro_adjustment = 10 # Positive macro environment adds points
            logger.debug("Global macro sentiment: Positive. +10 pts.")
        elif macro_sentiment == 'negative':
            macro_adjustment = -10 # Negative macro environment deducts points
            logger.debug("Global macro sentiment: Negative. -10 pts.")
        else: # 'neutral'
            macro_adjustment = 0 # Neutral macro environment has no impact
            logger.debug("Global macro sentiment: Neutral. 0 pts.")

        macro_results = {
            'sentiment': macro_sentiment,
            'adjustment': macro_adjustment
        }
        logger.info(f"Global macro impact assessment: {macro_results}")
        return macro_results


    def fetch_news(self, symbol: str) -> List[str]:
        """
        Fetches recent news headlines for the given stock symbol.
        NOTE: This is a placeholder. In a real application, you would integrate
        with a News API (like News API, Alpha Vantage, etc.) here.
        """
        print(f"Fetching news for {symbol} (placeholder)...")
        symbol_str = str(symbol) if symbol is not None else "Unknown Stock"
        return [
            f"{symbol_str} stock price rises on positive market sentiment",
            f"{symbol_str} announces strong quarterly results, beating estimates",
            f"Experts bullish on {symbol_str}'s future outlook",
            f"I am a non-numeric entry that might cause an error",
            f"{symbol_str} faces regulatory challenges in key market",
            f"Competitor's new product launch impacts {symbol_str}'s market share",
            f"Global market trends positively impacting {symbol_str}",
            f"{symbol_str} management provides optimistic guidance",
            f"Increased foreign investment flows into {symbol_str}",
            f"Regulatory approval received for {symbol_str}'s new product",
            f"Production issues reported for {symbol_str}",
            f"Increased competition puts pressure on {symbol_str}'s margins",
            f"Analyst downgrades rating for {symbol_str}",
            f"Supply chain disruptions affect {symbol_str}'s operations",
            f"Geopolitical tensions create uncertainty for {symbol_str}",
            f"{symbol_str} announces stock split",
            f"Dividend declared by {symbol_str}"
        ]


    def analyze_sentiment(self, news_headlines: List[str]) -> Dict:
        """
        Analyzes the sentiment of news headlines using the Gemini API.
        Returns a dictionary with sentiment counts (positive, neutral, negative)
        and a sentiment score.
        """
        if not self.gemini_model:
            print("Gemini API not configured. Skipping sentiment analysis.")
            return {'positive': 0, 'neutral': 0, 'negative': 0, 'score': 0, 'summary': 'Gemini API not configured. Sentiment analysis skipped.'}

        if not isinstance(news_headlines, list) or not news_headlines:
            print("No news headlines to analyze. Skipping sentiment analysis.")
            return {'positive': 0, 'neutral': 0, 'negative': 0, 'score': 12.5, 'summary': 'No news headlines. Sentiment analysis skipped.'}

        logger.info("Analyzing sentiment using Gemini API...")
        prompt = "Analyze the sentiment of the following news headlines for a stock and categorize each as Positive, Neutral, or Negative. Provide a concise summary of the overall sentiment.\n\n"
        for headline in news_headlines:
            if isinstance(headline, str):
                 prompt += f"- {headline}\n"
            else:
                 logger.warning(f"Skipping non-string headline: {headline}")
                 continue

        if len(prompt) > 30000:
             logger.warning("News headlines too long for Gemini API prompt. Truncating.")
             prompt = prompt[:30000] + "\n... (headlines truncated)"

        try:
            response = self.gemini_model.generate_content(prompt)
            sentiment_text = response.text.strip()
            logger.info(f"Gemini API Response: {sentiment_text[:200]}...")

            positive_count = sentiment_text.lower().count('positive')
            neutral_count = sentiment_text.lower().count('neutral')
            negative_count = sentiment_text.lower().count('negative')

            summary_line = "No summary extracted."
            lines = sentiment_text.split('\n')
            for line in lines:
                 lower_line = line.lower()
                 if 'summary' in lower_line or 'overall sentiment' in lower_line:
                     summary_line = line.strip()
                     break
            if summary_line == "No summary extracted." and lines:
                 for line in reversed(lines):
                     if line.strip():
                         summary_line = line.strip()
                         break

            total_headlines = len([h for h in news_headlines if isinstance(h, str)])
            if total_headlines > 0:
                sentiment_score = ((positive_count - negative_count) / total_headlines) * 12.5 + 12.5
            else:
                sentiment_score = 12.5

            return {
                'positive': positive_count,
                'neutral': neutral_count,
                'negative': negative_count,
                'score': max(0, min(sentiment_score, 25)),
                'summary': summary_line
            }

        except Exception as e:
            logger.error(f"Error analyzing sentiment with Gemini API: {e}. Skipping sentiment analysis.")
            return {'positive': 0, 'neutral': 0, 'negative': 0, 'score': 0, 'summary': 'Sentiment analysis failed.'}


    def generate_enhanced_recommendation(self, fundamental_score, technical_score,
                                       liquidity_score, peer_performance, index_membership,
                                       sentiment_analysis_results, valuation_comparison_results,
                                       downside_risk_metrics: Dict, enhanced_dividend_metrics: Dict,
                                       global_macro_results: Dict): # Added global_macro_results
        """Generate comprehensive recommendation including sentiment and valuation comparison"""
        # Get base scores
        fund_score = fundamental_score if isinstance(fundamental_score, (int, float)) and not pd.isna(fundamental_score) else 0
        tech_score = technical_score if isinstance(technical_score, (int, float)) and not pd.isna(technical_score) else 0
        liq_score = liquidity_score if isinstance(liquidity_score, (int, float)) and not pd.isna(liq_score) else 0
        sentiment_score = sentiment_analysis_results.get('score', 0) if isinstance(sentiment_analysis_results.get('score'), (int, float)) and not pd.isna(sentiment_analysis_results.get('score')) else 0
        enhanced_dividend_score = enhanced_dividend_metrics.get('score', 0) if isinstance(enhanced_dividend_metrics.get('score'), (int, float)) and not pd.isna(enhanced_dividend_metrics.get('score')) else 0
        logger.info(f"Enhanced dividend score for recommendation: {enhanced_dividend_score}")


        # Get adjustment scores
        valuation_adjustment = valuation_comparison_results.get('valuation_score_adjustment', 0) if isinstance(valuation_comparison_results.get('valuation_score_adjustment'), (int, float)) and not pd.isna(valuation_comparison_results.get('valuation_score_adjustment')) else 0
        logger.info(f"Valuation comparison adjustment: {valuation_adjustment}")

        var_percentage = downside_risk_metrics.get('var_percentage_loss', np.nan) # Use .get() as it's now a dict
        downside_risk_adjustment = 0 # Adjustment based on VaR

        if isinstance(var_percentage, (int, float)) and not pd.isna(var_percentage):
             # Example scoring: higher VaR means higher penalty
             if var_percentage > 5: # VaR > 5%
                  downside_risk_adjustment -= 15
                  logger.info(f"VaR ({var_percentage:.2f}%) > 5%. Applying downside risk penalty (-15).")
             elif var_percentage > 3: # VaR > 3%
                  downside_risk_adjustment -= 10
                  logger.info(f"VaR ({var_percentage:.2f}%) > 3%. Applying downside risk penalty (-10).")
             elif var_percentage > 1: # VaR > 1%
                  downside_risk_adjustment -= 5
                  logger.info(f"VaR ({var_percentage:.2f}%) > 1%. Applying downside risk penalty (-5).")
             else:
                  logger.info(f"VaR ({var_percentage:.2f}) <= 1%. No downside risk penalty.") # Corrected format string
        else:
             logger.warning("VaR not available. Cannot apply downside risk adjustment.")
             downside_risk_adjustment -= 2 # Small penalty if VaR cannot be calculated

        # Get global macro adjustment
        global_macro_adjustment = global_macro_results.get('adjustment', 0) if isinstance(global_macro_results.get('adjustment'), (int, float)) and not pd.isna(global_macro_results.get('adjustment')) else 0
        logger.info(f"Global macro adjustment: {global_macro_adjustment}")


        peer_analysis_available = isinstance(peer_performance, dict) and \
                                  not (pd.isna(peer_performance.get('stock_return')) and \
                                       pd.isna(peer_performance.get('avg_peer_return')) and \
                                       pd.isna(peer_performance.get('relative_performance')))

        peer_adjustment = 0
        if peer_analysis_available and peer_performance.get('outperformance', False) is True:
            peer_adjustment = 5
            logger.info("Adding bonus for peer outperformance.")
        elif not peer_analysis_available:
             peer_adjustment = -5
             logger.warning("Applying small penalty as peer performance data is incomplete/unavailable.")


        index_adjustment = 0
        if isinstance(index_membership, list) and index_membership:
            index_adjustment = 5
            logger.info("Adding bonus for index membership.")


        # Sum all scores and adjustments
        # Total Score = Fund (50) + Tech (50) + Liq (70) + Sent (25) + Enhanced Dividend (20) + Val Adj (+/-30 max) + Peer Adj (+/-5 max) + Index Adj (+5 max) + Downside Adj (+0/-15 max) + Global Macro Adj (+/-10 max)
        # Total Possible Max Score = 50 + 50 + 70 + 25 + 20 + 30 + 5 + 5 + 0 + 10 = 265
        # Total Possible Min Score = 50 + 0 + 0 + 0 + 0 - 30 - 5 + 0 - 15 - 10 = 0 - 10 = Max 0 if capped at 0

        total_score = fund_score + tech_score + liq_score + sentiment_score + enhanced_dividend_score + \
                      valuation_adjustment + peer_adjustment + index_adjustment + downside_risk_adjustment + global_macro_adjustment


        recommendation = "NEUTRAL"
        confidence = "Low"

        # Adjusting thresholds based on potential max score of 265
        if total_score >= 210: # Increased threshold
            recommendation = "STRONG BUY"
            confidence = "High"
        elif total_score >= 170: # Increased threshold
            recommendation = "BUY"
            confidence = "Moderate to High"
        elif total_score >= 120: # Adjusted threshold
            recommendation = "HOLD"
            confidence = "Moderate"
        elif total_score >= 70: # Adjusted threshold
            recommendation = "SELL"
            confidence = "Moderate"
        else:
            recommendation = "STRONG SELL"
            confidence = "High"

        logger.info(f"Generated enhanced recommendation: {recommendation} (Total Score: {total_score:.2f})")

        return {
            'recommendation': recommendation,
            'confidence': confidence,
            'total_score': total_score,
            'breakdown': {
                'fundamental': fund_score,
                'technical': tech_score,
                'liquidity': liq_score,
                'sentiment': sentiment_score,
                'enhanced_dividend': enhanced_dividend_score, # Include enhanced dividend score in breakdown
                'valuation_comparison': valuation_adjustment, # Include valuation adjustment in breakdown
                'downside_risk': downside_risk_adjustment, # Include downside risk adjustment
                'peer_adjustment': peer_adjustment, # Include peer adjustment in breakdown
                'index_adjustment': index_adjustment, # Include index adjustment in breakdown
                'global_macro_adjustment': global_macro_adjustment # Include global macro adjustment in breakdown
            }
        }

**Reasoning**:
The `EnhancedStockAnalyzer` class has been updated with the `calculate_enhanced_dividend_score` method. Now, I need to integrate this new method into the `analyze_stock_enhanced` function to call the method, store the results in the enhanced report, and update the report display to show the enhanced dividend metrics and their contribution to the total score.



In [ ]:
#@title Part 3: Enhanced Analysis Integration and Report Display (with Enhanced Dividend Analysis)

# Assuming IndianStockAnalyzer and EnhancedStockAnalyzer classes are defined in previous cells

# Ensure logging is configured (assuming this was done in a previous cell)
import logging
logger = logging.getLogger(__name__)

# Integration function to work with your Part 1
def analyze_stock_enhanced(symbol, stock_analyzer_part1, enhanced_analyzer):
    """
    Integrate enhanced features with your Part 1 analyzer

    Args:
        symbol: Stock symbol (e.g., 'RELIANCE.NS')
        stock_analyzer_part1: Instance of your IndianStockAnalyzer from Part 1
        enhanced_analyzer: Instance of EnhancedStockAnalyzer
    """
    if not isinstance(symbol, str) or not symbol:
        logger.error("Invalid stock symbol provided for enhanced analysis.")
        return None
    if not isinstance(stock_analyzer_part1, IndianStockAnalyzer):
        logger.error("Invalid IndianStockAnalyzer (Part 1) instance provided.")
        return None
    if not isinstance(enhanced_analyzer, EnhancedStockAnalyzer):
        logger.error("Invalid EnhancedStockAnalyzer (Part 2) instance provided.")
        return None


    logger.info(f"Starting enhanced analysis for {symbol}")
    # Get basic analysis from Part 1
    stock_data = stock_analyzer_part1.analyze_stock(symbol)

    if not stock_data:
        logger.error(f"Basic analysis failed for {symbol}.")
        return None

    # Get current price - already validated in Part 1 analyze_stock
    current_price = stock_data.current_price
    if pd.isna(current_price) or current_price <= 0:
         logger.warning(f"Current price invalid ({current_price}). Enhanced analysis might be limited.")


    # Enhanced analysis
    logger.info("Starting enhanced analysis features (Part 2).")
    # 1. Circuit breaker analysis
    circuit_limits = enhanced_analyzer.get_circuit_limits(symbol)
    # Pass current_price to assess_circuit_risk (already validated as best as possible in Part 1)
    circuit_risk = enhanced_analyzer.assess_circuit_risk(current_price, circuit_limits)

    # 2. Enhanced liquidity analysis
    # Pass price_data (already validated in Part 1 analyze_stock)
    # CORRECTED: Call get_liquidity_score on enhanced_analyzer and pass price_data from stock_data
    liquidity_score = enhanced_analyzer.get_liquidity_score(stock_data.price_data)

    # 3. Peer analysis (including valuation data)
    # Pass info (already validated in Part 1 analyze_stock)
    # The get_peer_companies method now returns a list of dictionaries with valuation data
    peers_with_valuation = enhanced_analyzer.get_peer_companies(symbol, stock_data.info)
    # Pass the list of peer symbols (extracted from the list of dictionaries) for performance comparison
    peer_symbols_for_performance = [p['symbol'] for p in peers_with_valuation if isinstance(p, dict) and 'symbol' in p]
    peer_performance = enhanced_analyzer.calculate_peer_relative_performance(symbol, peer_symbols_for_performance) # Pass just symbols

    # 4. Valuation Analysis: Calculate peer average valuations and compare
    peer_average_valuation = enhanced_analyzer.calculate_peer_average_valuation(peers_with_valuation)
    valuation_comparison_results = enhanced_analyzer.compare_valuation_to_peers(stock_data.metrics, peer_average_valuation)

    # 5. Downside Risk Assessment: Calculate VaR
    downside_risk_metrics = enhanced_analyzer.calculate_value_at_risk(stock_data.price_data)

    # 6. Enhanced Dividend Analysis
    enhanced_dividend_metrics = enhanced_analyzer.calculate_enhanced_dividend_score(symbol, stock_data.price_data)


    # 7. Index membership
    index_membership = enhanced_analyzer.get_index_membership(symbol)

    # 8. Recalculate technical score with circuit risk
    # Pass indicators (already validated in Part 1 calculate_technical_indicators)
    # Pass circuit_risk (already validated in assess_circuit_risk)
    enhanced_technical_score = enhanced_analyzer.calculate_enhanced_technical_score(
        stock_data.indicators, circuit_risk
    )

    # 9. Fetch and analyze news sentiment
    # Pass symbol (already validated)
    news_headlines = enhanced_analyzer.fetch_news(symbol)
    # Pass news_headlines (already validated)
    sentiment_analysis_results = enhanced_analyzer.analyze_sentiment(news_headlines)


    # 10. Generate enhanced recommendation
    # Pass scores and results (already validated within their respective functions)
    final_recommendation = enhanced_analyzer.generate_enhanced_recommendation(
        stock_data.fundamental_score,
        enhanced_technical_score,
        liquidity_score,
        peer_performance, # Pass potentially incomplete/NaN peer performance
        index_membership,
        sentiment_analysis_results, # Pass sentiment analysis results
        valuation_comparison_results, # Pass valuation comparison results
        downside_risk_metrics, # Pass downside risk metrics
        enhanced_dividend_metrics # Pass enhanced dividend metrics
    )

    # Create enhanced report
    enhanced_report = {
        'basic_analysis': stock_data,
        'enhanced_features': {
            'circuit_limits': circuit_limits,
            'circuit_risk_score': circuit_risk,
            'liquidity_score': liquidity_score, # Ensure correct liquidity score is included
            'peers': peers_with_valuation, # Store the list of peers with their valuation data
            'peer_performance': peer_performance, # Include peer performance, even if NaN
            'peer_average_valuation': peer_average_valuation, # Store peer average valuation
            'valuation_comparison': valuation_comparison_results, # Store valuation comparison results
            'downside_risk': downside_risk_metrics, # Store downside risk metrics (e.g., VaR)
            'enhanced_dividend': enhanced_dividend_metrics, # Store enhanced dividend metrics
            'index_membership': index_membership,
            'dynamic_risk_free_rate': enhanced_analyzer.risk_free_rate,
            'news_headlines': news_headlines, # Include news in report
            'sentiment_analysis': sentiment_analysis_results # Include sentiment analysis results
        },
        'final_recommendation': final_recommendation
    }
    logger.info(f"Enhanced analysis completed for {symbol}")
    return enhanced_report

# Example usage
if __name__ == "__main__":
    # Initialize analyzers
    try:
        stock_analyzer = IndianStockAnalyzer()
        enhanced_analyzer = EnhancedStockAnalyzer()
    except NameError:
        print("Make sure to run the previous cells defining IndianStockAnalyzer and EnhancedStockAnalyzer classes.")
        exit() # Exit if classes are not defined


    # Analyze a stock
    symbol = "RELIANCE.NS" # Example symbol
    result = analyze_stock_enhanced(symbol, stock_analyzer, enhanced_analyzer)

    if result:
        # Display the enhanced report
        enhanced = result['enhanced_features']
        final = result['final_recommendation']
        basic = result['basic_analysis'] # Get basic data for display

        print(f"\n{'='*60}")
        print(f"Enhanced Analysis Report: {basic.company_name} ({basic.symbol})")
        print(f"{'='*60}")

        print(f"\n📊 Current Market Data:")
        print(f"Current Price: ₹{basic.current_price:,.2f}" if not pd.isna(basic.current_price) else "Current Price: N/A")
        market_cap_cr = basic.market_cap / 10000000 if not pd.isna(basic.market_cap) else np.nan
        print(f"Market Cap: ₹{market_cap_cr:,.2f} Cr" if not pd.isna(market_cap_cr) else "Market Cap: N/A")
        print(f"Sector: {basic.info.get('sector', 'N/A')}")

        print(f"\n⚙️ Enhanced Technical Analysis & Risk:")
        # Display enhanced technical indicators and risk
        # Assuming you want to display relevant indicators from basic.indicators and the circuit risk score
        circuit_limits_display = enhanced.get('circuit_limits')
        if circuit_limits_display and not pd.isna(circuit_limits_display.get('lower_circuit')) and not pd.isna(circuit_limits_display.get('upper_circuit')):
             print(f"Circuit Limits: {circuit_limits_display.get('lower_circuit', 'N/A'):.2f} - {circuit_limits_display.get('upper_circuit', 'N/A'):.2f}")
        else:
             print("Circuit Limits: N/A")

        print(f"Circuit Risk Score: {enhanced.get('circuit_risk_score', 'N/A')}")
        # Check if liquidity score is available before formatting
        liquidity_score_display = f"{enhanced.get('liquidity_score', 'N/A')}/70" if isinstance(enhanced.get('liquidity_score'), (int, float)) and not pd.isna(enhanced.get('liquidity_score')) else "Liquidity Score: N/A/70"
        print(liquidity_score_display)


        # Display some key technical indicators from basic analysis
        print("\nKey Technical Indicators (from Basic Analysis):")
        key_tech_indicators = ['RSI', 'MACD', 'MACD_signal', 'SMA_20', 'SMA_50', 'BB_middle', 'Volume_ratio']
        for indicator_name in key_tech_indicators:
            value = basic.indicators.get(indicator_name, np.nan)
            if not pd.isna(value):
                if isinstance(value, float):
                     print(f"  - {indicator_name}: {value:,.2f}")
                else:
                     print(f"  - {indicator_name}: {value}")
            # Add logging for these key technical indicators if they are calculated
            # logger.info(f"Indicator {indicator_name}: {value}") # Removed duplicate logging
            # else: # Removed unnecessary else block
            #     print(f"  - {indicator_name}: N/A")
            #     logger.info(f"Indicator {indicator_name}: N/A")


        # Display Valuation Analysis
        print(f"\n📈 Valuation Analysis:")
        stock_metrics = basic.metrics
        peer_avg_val = enhanced.get('peer_average_valuation', {})
        val_comp_results = enhanced.get('valuation_comparison', {})

        stock_pe = stock_metrics.get('trailingPE', np.nan)
        stock_pb = stock_metrics.get('priceToBook', np.nan)
        stock_ev_ebitda = stock_metrics.get('evToEbitda', np.nan)

        print(f"Stock Valuation:")
        print(f"  - Trailing PE: {stock_pe:.2f}" if not pd.isna(stock_pe) else "  - Trailing PE: N/A")
        print(f"  - Price to Book: {stock_pb:.2f}" if not pd.isna(stock_pb) else "  - Price to Book: N/A")
        print(f"  - EV/EBITDA: {stock_ev_ebitda:.2f}" if not pd.isna(stock_ev_ebitda) else "  - EV/EBITDA: N/A")

        print("\nPeer Average Valuation:")
        avg_pe_display = f"{peer_avg_val.get('avg_pe', np.nan):.2f}" if not pd.isna(peer_avg_val.get('avg_pe', np.nan)) else "N/A"
        avg_pb_display = f"{peer_avg_val.get('avg_pb', np.nan):.2f}" if not pd.isna(peer_avg_val.get('avg_pb', np.nan)) else "N/A"
        avg_ev_ebitda_display = f"{peer_avg_val.get('avg_ev_ebitda', np.nan):.2f}" if not pd.isna(peer_avg_val.get('avg_ev_ebitda', np.nan)) else "N/A"

        print(f"  - Average PE: {avg_pe_display}")
        print(f"  - Average PB: {avg_pb_display}")
        print(f"  - Average EV/EBITDA: {avg_ev_ebitda_display}")

        print("\nValuation Comparison:")
        print(f"  - PE Comparison: {val_comp_results.get('pe_comparison', 'N/A')}")
        print(f"  - PB Comparison: {val_comp_results.get('pb_comparison', 'N/A')}")
        print(f"  - EV/EBITDA Comparison: {val_comp_results.get('ev_ebitda_comparison', 'N/A')}")
        valuation_adj_display = f"{val_comp_results.get('valuation_score_adjustment', 0):+.2f}" if isinstance(val_comp_results.get('valuation_score_adjustment'), (int, float)) and not pd.isna(val_comp_results.get('valuation_score_adjustment')) else "N/A"
        print(f"  - Valuation Score Adjustment: {valuation_adj_display}")


        print(f"\n📉 Downside Risk Analysis:")
        downside_risk = enhanced.get('downside_risk', {})
        var_percentage = downside_risk.get('var_percentage_loss', np.nan)
        var_display = f"{var_percentage:.2f}%" if not pd.isna(var_percentage) else "N/A"
        # Access 'confidence_level' and 'horizon_days' directly from the downside_risk dictionary
        confidence_level_display = downside_risk.get('confidence_level', 0.95)
        horizon_days_display = downside_risk.get('horizon_days', 1)
        print(f"  - {confidence_level_display*100:.0f}% VaR ({horizon_days_display}-day): {var_display}")

        downside_adj_display = f"{final.get('breakdown', {}).get('downside_risk', np.nan):+.2f}" if isinstance(final.get('breakdown', {}).get('downside_risk'), (int, float)) and not pd.isna(final.get('breakdown', {}).get('downside_risk')) else "N/A"
        print(f"  - Downside Risk Score Adjustment: {downside_adj_display}")


        # Display Enhanced Dividend Analysis
        print(f"\n💎 Enhanced Dividend Analysis:")
        enhanced_dividend = enhanced.get('enhanced_dividend', {})
        current_yield_display = f"{enhanced_dividend.get('trailingAnnualDividendYield', np.nan):.2f}%" if isinstance(enhanced_dividend.get('trailingAnnualDividendYield'), (int, float)) and not pd.isna(enhanced_dividend.get('trailingAnnualDividendYield')) else "N/A"
        payout_ratio_display = f"{enhanced_dividend.get('dividendPayoutRatio', np.nan):.2f}" if isinstance(enhanced_dividend.get('dividendPayoutRatio'), (int, float)) and not pd.isna(enhanced_dividend.get('dividendPayoutRatio')) else "N/A"
        yield_trend_display = enhanced_dividend.get('yield_trend', 'N/A')
        enhanced_dividend_score_display = f"{enhanced_dividend.get('score', np.nan):.2f}/20" if isinstance(enhanced_dividend.get('score'), (int, float)) and not pd.isna(enhanced_dividend.get('score')) else "N/A/20"

        print(f"  - Trailing Annual Dividend Yield: {current_yield_display}")
        print(f"  - Dividend Payout Ratio: {payout_ratio_display}")
        print(f"  - Dividend Payout Trend: {yield_trend_display}")
        print(f"  - Enhanced Dividend Score: {enhanced_dividend_score_display}")


        print(f"\n🤝 Peer Analysis (Performance):")
        peer_perf = enhanced.get('peer_performance')
        if isinstance(peer_perf, dict) and \
           not pd.isna(peer_perf.get('stock_return')) and \
           not pd.isna(peer_perf.get('avg_peer_return')) and \
           not pd.isna(peer_perf.get('relative_performance')):

             stock_return_display = f"{peer_perf.get('stock_return', np.nan):.2f}%" if isinstance(peer_perf.get('stock_return'), (int, float)) and not pd.isna(peer_perf.get('stock_return')) else 'N/A'
             avg_peer_return_display = f"{peer_perf.get('avg_peer_return', np.nan):.2f}%" if isinstance(peer_perf.get('avg_peer_return'), (int, float)) and not pd.isna(peer_perf.get('avg_peer_return')) else 'N/A'
             relative_performance_display = f"{peer_perf.get('relative_performance', np.nan):.2f}%" if isinstance(peer_perf.get('relative_performance'), (int, float)) and not pd.isna(peer_perf.get('relative_performance')) else 'N/A'

             print(f"  - {peer_perf.get('days', 'N/A')}-day Stock Return: {stock_return_display}")
             print(f"  - Avg Peer Return: {avg_peer_return_display}")
             print(f"  - Relative Performance: {relative_performance_display}")
        else:
             print("Peer Performance Data Unavailable.")


        if enhanced.get('index_membership'):
            print(f"\n🏛️ Index Membership: {', '.join(enhanced['index_membership'])}")

        # Sentiment Analysis Results
        sentiment_results = enhanced.get('sentiment_analysis', {})
        print(f"\n📰 News Sentiment Analysis:")
        if enhanced.get('news_headlines'):
             # Only print a few headlines to keep the report concise
             print("Sample News Headlines:")
             # Ensure headlines are valid strings before printing
             valid_headlines = [h for h in enhanced['news_headlines'] if isinstance(h, str)]
             for headline in valid_headlines[:5]: # Print up to 5 headlines
                 print(f"- {headline}")
             if len(valid_headlines) > 5:
                  print("  ...")

             print(f"Sentiment Counts: Positive={sentiment_results.get('positive', 0)}, Neutral={sentiment_results.get('neutral', 0)}, Negative={sentiment_results.get('negative', 0)}")
             sentiment_score_display = f"{sentiment_results.get('score', np.nan):.2f}" if isinstance(sentiment_results.get('score'), (int, float)) and not pd.isna(sentiment_results.get('score')) else "N/A"
             print(f"Sentiment Score: {sentiment_score_display}/25")
             print(f"Sentiment Summary: {sentiment_results.get('summary', 'N/A')}")
        else:
             print("No news headlines fetched or sentiment analysis skipped.")


        print(f"\n--- Comprehensive Scores and Recommendation ---")
        print(f"Fundamental Score (from Part 1): {basic.fundamental_score:.2f}/50" if not pd.isna(basic.fundamental_score) else "Fundamental Score: N/A")
        # Add checks for NaN before formatting float for breakdown scores
        enhanced_tech_score_display = f"{final.get('breakdown', {}).get('technical', np.nan):.2f}" if isinstance(final.get('breakdown', {}).get('technical'), (int, float)) and not pd.isna(final.get('breakdown', {}).get('technical')) else "N/A"
        liquidity_score_display_breakdown = f"{final.get('breakdown', {}).get('liquidity', np.nan):.2f}" if isinstance(final.get('breakdown', {}).get('liquidity'), (int, float)) and not pd.isna(final.get('breakdown', {}).get('liquidity')) else "N/A"
        sentiment_score_display_breakdown = f"{sentiment_results.get('score', np.nan):.2f}" if isinstance(sentiment_results.get('score'), (int, float)) and not pd.isna(sentiment_results.get('score')) else "N/A" # Corrected sentiment score breakdown check
        valuation_comp_display_breakdown = f"{final.get('breakdown', {}).get('valuation_comparison', np.nan):.2f}" if isinstance(final.get('breakdown', {}).get('valuation_comparison'), (int, float)) and not pd.isna(final.get('breakdown', {}).get('valuation_comparison')) else "N/A"
        downside_risk_display_breakdown = f"{final.get('breakdown', {}).get('downside_risk', np.nan):+.2f}" if isinstance(final.get('breakdown', {}).get('downside_risk'), (int, float)) and not pd.isna(final.get('breakdown', {}).get('downside_risk')) else "N/A"
        enhanced_dividend_display_breakdown = f"{final.get('breakdown', {}).get('enhanced_dividend', np.nan):.2f}" if isinstance(final.get('breakdown', {}).get('enhanced_dividend'), (int, float)) and not pd.isna(final.get('breakdown', {}).get('enhanced_dividend')) else "N/A"


        print(f"Technical Score (Enhanced): {enhanced_tech_score_display}/50")
        print(f"Liquidity Score: {liquidity_score_display_breakdown}/70")
        print(f"Sentiment Score: {sentiment_score_display_breakdown}/25")
        print(f"Valuation Comparison Adjustment: {valuation_comp_display_breakdown}")
        print(f"Downside Risk Adjustment: {downside_risk_display_breakdown}")
        print(f"Enhanced Dividend Score: {enhanced_dividend_display_breakdown}/20") # Display enhanced dividend score breakdown


        # Display adjustment scores
        peer_adj_display = f"{final.get('breakdown', {}).get('peer_adjustment', np.nan):+.2f}" if isinstance(final.get('breakdown', {}).get('peer_adjustment'), (int, float)) and not pd.isna(final.get('breakdown', {}).get('peer_adjustment')) else "N/A"
        index_adj_display = f"{final.get('breakdown', {}).get('index_adjustment', np.nan):+.2f}" if isinstance(final.get('breakdown', {}).get('index_adjustment'), (int, float)) and not pd.isna(final.get('breakdown', {}).get('index_adjustment')) else "N/A"

        print(f"Peer Performance Adjustment: {peer_adj_display}")
        print(f"Index Membership Adjustment: {index_adj_display}")


        # Updated max score to reflect the addition of Enhanced Dividend Score (20 points)
        # Max Total Score = Fund (50) + Tech (50) + Liq (70) + Sent (25) + Enhanced Dividend (20) + Val Adj (30) + Peer Adj (5) + Index Adj (5) + Downside Adj (0) = 255
        print(f"\nTotal Enhanced Score: {final.get('total_score', 0):.2f}/255" if isinstance(final.get('total_score'), (int, float)) and not pd.isna(final.get('total_score')) else "Total Enhanced Score: N/A")
        print(f"RECOMMENDATION: {final.get('recommendation', 'N/A')} (Confidence: {final.get('confidence', 'N/A')})")


        print(f"\n{'='*60}\n")

In [ ]:
#@title Part 3: Enhanced Analysis Integration and Report Display (Regenerated)

# Assuming IndianStockAnalyzer and EnhancedStockAnalyzer classes are defined in previous cells

# Ensure logging is configured (assuming this was done in a previous cell)
import logging
logger = logging.getLogger(__name__)

# Import numpy and pandas here to ensure they are available globally
import numpy as np
import pandas as pd

# Integration function to work with your Part 1
def analyze_stock_enhanced(symbol, stock_analyzer_part1, enhanced_analyzer):
    """
    Integrate enhanced features with your Part 1 analyzer

    Args:
        symbol: Stock symbol (e.g., 'RELIANCE.NS')
        stock_analyzer_part1: Instance of your IndianStockAnalyzer from Part 1
        enhanced_analyzer: Instance of EnhancedStockAnalyzer
    """
    if not isinstance(symbol, str) or not symbol:
        logger.error("Invalid stock symbol provided for enhanced analysis.")
        return None
    if not isinstance(stock_analyzer_part1, IndianStockAnalyzer):
        logger.error("Invalid IndianStockAnalyzer (Part 1) instance provided.")
        return None
    if not isinstance(enhanced_analyzer, EnhancedStockAnalyzer):
        logger.error("Invalid EnhancedStockAnalyzer (Part 2) instance provided.")
        return None


    logger.info(f"Starting enhanced analysis for {symbol}")
    # Get basic analysis from Part 1
    stock_data = stock_analyzer_part1.analyze_stock(symbol)

    if not stock_data:
        logger.error(f"Basic analysis failed for {symbol}.")
        return None

    # Get current price - already validated in Part 1 analyze_stock
    current_price = stock_data.current_price
    if pd.isna(current_price) or current_price <= 0:
         logger.warning(f"Current price invalid ({current_price}). Enhanced analysis might be limited.")


    # Enhanced analysis
    logger.info("Starting enhanced analysis features (Part 2).")
    # 1. Circuit breaker analysis
    circuit_limits = enhanced_analyzer.get_circuit_limits(symbol)
    # Pass current_price to assess_circuit_risk (already validated as best as possible in Part 1)
    circuit_risk = enhanced_analyzer.assess_circuit_risk(current_price, circuit_limits)

    # 2. Enhanced liquidity analysis
    # Pass price_data (already validated in Part 1 analyze_stock)
    # CORRECTED: Call get_liquidity_score on enhanced_analyzer and pass price_data from stock_data
    liquidity_score = enhanced_analyzer.get_liquidity_score(stock_data.price_data)

    # 3. Peer analysis (including valuation data)
    # Pass info (already validated in Part 1 analyze_stock)
    # The get_peer_companies method now returns a list of dictionaries with valuation data
    peers_with_valuation = enhanced_analyzer.get_peer_companies(symbol, stock_data.info)
    # Pass the list of peer symbols (extracted from the list of dictionaries) for performance comparison
    peer_symbols_for_performance = [p['symbol'] for p in peers_with_valuation if isinstance(p, dict) and 'symbol' in p]
    peer_performance = enhanced_analyzer.calculate_peer_relative_performance(symbol, peer_symbols_for_performance) # Pass just symbols

    # 4. Valuation Analysis: Calculate peer average valuations and compare
    peer_average_valuation = enhanced_analyzer.calculate_peer_average_valuation(peers_with_valuation)
    valuation_comparison_results = enhanced_analyzer.compare_valuation_to_peers(stock_data.metrics, peer_average_valuation)

    # 5. Downside Risk Assessment: Calculate VaR
    downside_risk_metrics = enhanced_analyzer.calculate_value_at_risk(stock_data.price_data)

    # 6. Enhanced Dividend Analysis
    enhanced_dividend_metrics = enhanced_analyzer.calculate_enhanced_dividend_score(symbol, stock_data.price_data)

    # 7. Global Macro Impact Assessment
    global_macro_results = enhanced_analyzer.assess_global_macro_impact()


    # 8. Index membership
    index_membership = enhanced_analyzer.get_index_membership(symbol)

    # 9. Recalculate technical score with circuit risk
    # Pass indicators (already validated in Part 1 calculate_technical_indicators)
    # Pass circuit_risk (already validated in assess_circuit_risk)
    enhanced_technical_score = enhanced_analyzer.calculate_enhanced_technical_score(
        stock_data.indicators, circuit_risk
    )

    # 10. Fetch and analyze news sentiment
    # Pass symbol (already validated)
    news_headlines = enhanced_analyzer.fetch_news(symbol)
    # Pass news_headlines (already validated)
    sentiment_analysis_results = enhanced_analyzer.analyze_sentiment(news_headlines)


    # 11. Generate enhanced recommendation
    # Pass scores and results (already validated within their respective functions)
    final_recommendation = enhanced_analyzer.generate_enhanced_recommendation(
        stock_data.fundamental_score,
        enhanced_technical_score,
        liquidity_score,
        peer_performance, # Pass potentially incomplete/NaN peer performance
        index_membership,
        sentiment_analysis_results, # Pass sentiment analysis results
        valuation_comparison_results, # Pass valuation comparison results
        downside_risk_metrics, # Pass downside risk metrics
        enhanced_dividend_metrics, # Pass enhanced dividend metrics
        global_macro_results # Pass global macro results
    )

    # Create enhanced report
    enhanced_report = {
        'basic_analysis': stock_data,
        'enhanced_features': {
            'circuit_limits': circuit_limits,
            'circuit_risk_score': circuit_risk,
            'liquidity_score': liquidity_score, # Ensure correct liquidity score is included
            'peers': peers_with_valuation, # Store the list of peers with their valuation data
            'peer_performance': peer_performance, # Include peer performance, even if NaN
            'peer_average_valuation': peer_average_valuation, # Store peer average valuation
            'valuation_comparison': valuation_comparison_results, # Store valuation comparison results
            'downside_risk': downside_risk_metrics, # Store downside risk metrics (e.g., VaR)
            'enhanced_dividend': enhanced_dividend_metrics, # Store enhanced dividend metrics
            'global_macro': global_macro_results, # Store global macro results
            'index_membership': index_membership,
            'dynamic_risk_free_rate': enhanced_analyzer.risk_free_rate,
            'news_headlines': news_headlines, # Include news in report
            'sentiment_analysis': sentiment_analysis_results # Include sentiment analysis results
        },
        'final_recommendation': final_recommendation
    }
    logger.info(f"Enhanced analysis completed for {symbol}")
    return enhanced_report

# Example usage (assuming this block will be run directly)
if __name__ == "__main__":
    # Initialize analyzers - ensure these classes are defined in previous cells
    try:
        stock_analyzer = IndianStockAnalyzer()
        enhanced_analyzer = EnhancedStockAnalyzer()
    except NameError:
        print("Make sure to run the previous cells defining IndianStockAnalyzer and EnhancedStockAnalyzer classes.")
        # Attempt to define dummy classes if not found, to allow the rest of the cell to be generated
        # This is a fallback and may not work for complex class dependencies.
        # It's better to ensure the user runs the correct prerequisite cells.
        import numpy as np
        import pandas as pd

        class IndianStockAnalyzer:
            def analyze_stock(self, symbol):
                print(f"Placeholder: Analyzing {symbol} with dummy IndianStockAnalyzer.")
                # Return a dummy object with necessary attributes to prevent immediate errors
                class DummyStockData:
                    def __init__(self):
                        self.current_price = 100.0
                        self.market_cap = 10000000000.0
                        self.info = {'sector': 'Unknown', 'trailingPE': np.nan, 'priceToBook': np.nan, 'enterpriseValue': np.nan, 'ebitda': np.nan}
                        self.metrics = {'trailingPE': np.nan, 'priceToBook': np.nan, 'evToEbitda': np.nan}
                        self.price_data = pd.DataFrame({'Close': [100.0]*100, 'Volume': [1000]*100})
                        self.indicators = {'RSI': np.nan, 'MACD': np.nan, 'MACD_signal': np.nan, 'SMA_20': np.nan, 'SMA_50': np.nan, 'BB_middle': np.nan, 'Volume_ratio': np.nan}
                        self.fundamental_score = np.nan
                        self.technical_score = np.nan # Added technical_score to dummy
                        self.company_name = "Dummy Company"
                        self.symbol = symbol

                return DummyStockData()

        class EnhancedStockAnalyzer:
            def __init__(self):
                self.risk_free_rate = 0.07
                self.gemini_model = None # Assume no Gemini model if not configured
                self.global_macro_sentiment = 'neutral' # Added dummy global macro sentiment

            def get_circuit_limits(self, symbol): return None
            def assess_circuit_risk(self, current_price, circuit_limits): return 0
            def get_liquidity_score(self, price_data): return 30
            def get_peer_companies(self, symbol, stock_info): return []
            def calculate_peer_average_valuation(self, peers_with_valuation): return {'avg_pe': np.nan, 'avg_pb': np.nan, 'avg_ev_ebitda': np.nan}
            def compare_valuation_to_peers(self, stock_valuation, peer_average_valuation): return {'valuation_score_adjustment': 0}
            def calculate_value_at_risk(self, price_data, confidence_level=0.95, horizon_days=1): return {'var_percentage_loss': np.nan, 'confidence_level': confidence_level, 'horizon_days': horizon_days}
            def calculate_enhanced_dividend_score(self, symbol, price_data): return {'score': 0, 'trailingAnnualDividendYield': np.nan, 'dividendPayoutRatio': np.nan, 'yield_trend': 'N/A', 'payout_history': {}}
            def assess_global_macro_impact(self): return {'sentiment': self.global_macro_sentiment, 'adjustment': 0} # Added dummy global macro assessment
            def fetch_news(self, symbol): return []
            def analyze_sentiment(self, news_headlines): return {'positive': 0, 'neutral': 0, 'negative': 0, 'score': 0, 'summary': 'Dummy sentiment analysis.'}
            def get_index_membership(self, symbol): return []
            def calculate_enhanced_technical_score(self, indicators, circuit_risk): return 0
            def calculate_peer_relative_performance(self, symbol, peers, days=30): return {'stock_return': np.nan, 'avg_peer_return': np.nan, 'relative_performance': np.nan, 'outperformance': False, 'days': days} # Added dummy peer relative performance
            def generate_enhanced_recommendation(self, fundamental_score, technical_score, liquidity_score, peer_performance, index_membership, sentiment_analysis_results, valuation_comparison_results, downside_risk_metrics, enhanced_dividend_metrics, global_macro_results): # Added global_macro_results
                 # Simple dummy recommendation based on scores
                 fund_score = fundamental_score or 0
                 tech_score = technical_score or 0
                 liq_score = liquidity_score or 0
                 sentiment_score = sentiment_analysis_results.get('score', 0) or 0
                 enhanced_dividend_score = enhanced_dividend_metrics.get('score', 0) or 0
                 valuation_adjustment = valuation_comparison_results.get('valuation_score_adjustment', 0) or 0
                 downside_risk_adjustment = downside_risk_metrics.get('downside_risk', {}).get('downside_risk_adjustment', 0) if isinstance(downside_risk_metrics, dict) else 0 # Access adjustment from nested dict
                 peer_adjustment = peer_performance.get('peer_adjustment', 0) if isinstance(peer_performance, dict) else 0 # Added check
                 index_adjustment = 5 if index_membership else 0
                 global_macro_adjustment = global_macro_results.get('adjustment', 0) if isinstance(global_macro_results, dict) else 0 # Added check


                 total_score = fund_score + tech_score + liq_score + sentiment_score + enhanced_dividend_score + \
                               valuation_adjustment + peer_adjustment + index_adjustment + downside_risk_adjustment + global_macro_adjustment


                 if total_score > 150: rec = "BUY"
                 elif total_score > 100: rec = "HOLD"
                 else: rec = "SELL"
                 return {'recommendation': rec, 'confidence': 'Dummy', 'total_score': total_score, 'breakdown': {}}


    # Analyze a stock
    symbol = "RELIANCE.NS" # Example symbol
    result = analyze_stock_enhanced(symbol, stock_analyzer, enhanced_analyzer)

    if result:
        # Display the enhanced report
        enhanced = result['enhanced_features']
        final = result['final_recommendation']
        basic = result['basic_analysis'] # Get basic data for display

        print(f"\n{'='*60}")
        print(f"Enhanced Analysis Report: {basic.company_name} ({basic.symbol})")
        print(f"{'='*60}")

        print(f"\n📊 Current Market Data:")
        print(f"Current Price: ₹{basic.current_price:,.2f}" if not pd.isna(basic.current_price) else "Current Price: N/A")
        market_cap_cr = basic.market_cap / 10000000 if not pd.isna(basic.market_cap) else np.nan
        print(f"Market Cap: ₹{market_cap_cr:,.2f} Cr" if not pd.isna(market_cap_cr) else "Market Cap: N/A")
        print(f"Sector: {basic.info.get('sector', 'N/A')}")

        print(f"\n⚙️ Enhanced Technical Analysis & Risk:")
        # Display enhanced technical indicators and risk
        # Assuming you want to display relevant indicators from basic.indicators and the circuit risk score
        circuit_limits_display = enhanced.get('circuit_limits')
        if circuit_limits_display and not pd.isna(circuit_limits_display.get('lower_circuit')) and not pd.isna(circuit_limits_display.get('upper_circuit')):
             print(f"Circuit Limits: {circuit_limits_display.get('lower_circuit', 'N/A'):.2f} - {circuit_limits_display.get('upper_circuit', 'N/A'):.2f}")
        else:
             print("Circuit Limits: N/A")

        print(f"Circuit Risk Score: {enhanced.get('circuit_risk_score', 'N/A')}")
        # Check if liquidity score is available before formatting
        liquidity_score_display = f"{enhanced.get('liquidity_score', 'N/A')}/70" if isinstance(enhanced.get('liquidity_score'), (int, float)) and not pd.isna(enhanced.get('liquidity_score')) else "Liquidity Score: N/A/70"
        print(liquidity_score_display)


        # Display some key technical indicators from basic analysis
        print("\nKey Technical Indicators (from Basic Analysis):")
        key_tech_indicators = ['RSI', 'MACD', 'MACD_signal', 'SMA_20', 'SMA_50', 'BB_middle', 'Volume_ratio']
        for indicator_name in key_tech_indicators:
            value = basic.indicators.get(indicator_name, np.nan)
            if not pd.isna(value):
                if isinstance(value, float):
                     print(f"  - {indicator_name}: {value:,.2f}")
                else:
                     print(f"  - {indicator_name}: {value}")


        # Display Valuation Analysis
        print(f"\n📈 Valuation Analysis:")
        stock_metrics = basic.metrics
        peer_avg_val = enhanced.get('peer_average_valuation', {})
        val_comp_results = enhanced.get('valuation_comparison', {})

        stock_pe = stock_metrics.get('trailingPE', np.nan)
        stock_pb = stock_metrics.get('priceToBook', np.nan)
        stock_ev_ebitda = stock_metrics.get('evToEbitda', np.nan)

        print(f"Stock Valuation:")
        print(f"  - Trailing PE: {stock_pe:.2f}" if not pd.isna(stock_pe) else "  - Trailing PE: N/A")
        print(f"  - Price to Book: {stock_pb:.2f}" if not pd.isna(stock_pb) else "  - Price to Book: N/A")
        print(f"  - EV/EBITDA: {stock_ev_ebitda:.2f}" if not pd.isna(stock_ev_ebitda) else "  - EV/EBITDA: N/A")

        print("\nPeer Average Valuation:")
        avg_pe_display = f"{peer_avg_val.get('avg_pe', np.nan):.2f}" if not pd.isna(peer_avg_val.get('avg_pe', np.nan)) else "N/A"
        avg_pb_display = f"{peer_avg_val.get('avg_pb', np.nan):.2f}" if not pd.isna(peer_avg_val.get('avg_pb', np.nan)) else "N/A"
        avg_ev_ebitda_display = f"{peer_avg_val.get('avg_ev_ebitda', np.nan):.2f}" if not pd.isna(peer_avg_val.get('avg_ev_ebitda', np.nan)) else "N/A"

        print(f"  - Average PE: {avg_pe_display}")
        print(f"  - Average PB: {avg_pb_display}")
        print(f"  - Average EV/EBITDA: {avg_ev_ebitda_display}")

        print("\nValuation Comparison:")
        print(f"  - PE Comparison: {val_comp_results.get('pe_comparison', 'N/A')}")
        print(f"  - PB Comparison: {val_comp_results.get('pb_comparison', 'N/A')}")
        print(f"  - EV/EBITDA Comparison: {val_comp_results.get('ev_ebitda_comparison', 'N/A')}")
        valuation_adj_display = f"{val_comp_results.get('valuation_score_adjustment', 0):+.2f}" if isinstance(val_comp_results.get('valuation_score_adjustment'), (int, float)) and not pd.isna(val_comp_results.get('valuation_score_adjustment')) else "N/A"
        print(f"  - Valuation Score Adjustment: {valuation_adj_display}")


        print(f"\n📉 Downside Risk Analysis:")
        downside_risk = enhanced.get('downside_risk', {})
        var_percentage = downside_risk.get('var_percentage_loss', np.nan)
        var_display = f"{var_percentage:.2f}%" if not pd.isna(var_percentage) else "N/A"
        # Access 'confidence_level' and 'horizon_days' directly from the downside_risk dictionary
        confidence_level_display = downside_risk.get('confidence_level', 0.95)
        horizon_days_display = downside_risk.get('horizon_days', 1)
        print(f"  - {confidence_level_display*100:.0f}% VaR ({horizon_days_display}-day): {var_display}")

        downside_adj_display = f"{final.get('breakdown', {}).get('downside_risk', np.nan):+.2f}" if isinstance(final.get('breakdown', {}).get('downside_risk'), (int, float)) and not pd.isna(final.get('breakdown', {}).get('downside_risk')) else "N/A"
        print(f"  - Downside Risk Score Adjustment: {downside_adj_display}")


        # Display Enhanced Dividend Analysis
        print(f"\n💎 Enhanced Dividend Analysis:")
        enhanced_dividend = enhanced.get('enhanced_dividend', {})
        current_yield_display = f"{enhanced_dividend.get('trailingAnnualDividendYield', np.nan):.2f}%" if isinstance(enhanced_dividend.get('trailingAnnualDividendYield'), (int, float)) and not pd.isna(enhanced_dividend.get('trailingAnnualDividendYield')) else "N/A"
        payout_ratio_display = f"{enhanced_dividend.get('dividendPayoutRatio', np.nan):.2f}" if isinstance(enhanced_dividend.get('dividendPayoutRatio'), (int, float)) and not pd.isna(enhanced_dividend.get('dividendPayoutRatio')) else "N/A"
        yield_trend_display = enhanced_dividend.get('yield_trend', 'N/A')
        enhanced_dividend_score_display = f"{enhanced_dividend.get('score', np.nan):.2f}/20" if isinstance(enhanced_dividend.get('score'), (int, float)) and not pd.isna(enhanced_dividend.get('score')) else "N/A/20"

        print(f"  - Trailing Annual Dividend Yield: {current_yield_display}")
        print(f"  - Dividend Payout Ratio: {payout_ratio_display}")
        print(f"  - Dividend Payout Trend: {yield_trend_display}")
        print(f"  - Enhanced Dividend Score: {enhanced_dividend_score_display}")


        print(f"\n🤝 Peer Analysis (Performance):")
        peer_perf = enhanced.get('peer_performance')
        if isinstance(peer_perf, dict) and \
           not pd.isna(peer_perf.get('stock_return')) and \
           not pd.isna(peer_perf.get('avg_peer_return')) and \
           not pd.isna(peer_perf.get('relative_performance')):

             stock_return_display = f"{peer_perf.get('stock_return', np.nan):.2f}%" if isinstance(peer_perf.get('stock_return'), (int, float)) and not pd.isna(peer_perf.get('stock_return')) else 'N/A'
             avg_peer_return_display = f"{peer_perf.get('avg_peer_return', np.nan):.2f}%" if isinstance(peer_perf.get('avg_peer_return'), (int, float)) and not pd.isna(peer_perf.get('avg_peer_return')) else 'N/A'
             relative_performance_display = f"{peer_perf.get('relative_performance', np.nan):.2f}%" if isinstance(peer_perf.get('relative_performance'), (int, float)) and not pd.isna(peer_perf.get('relative_performance')) else 'N/A'

             print(f"  - {peer_perf.get('days', 'N/A')}-day Stock Return: {stock_return_display}")
             print(f"  - Avg Peer Return: {avg_peer_return_display}")
             print(f"  - Relative Performance: {relative_performance_display}")
        else:
             print("Peer Performance Data Unavailable.")


        if enhanced.get('index_membership'):
            print(f"\n🏛️ Index Membership: {', '.join(enhanced['index_membership'])}")


        # Display Global Macro Analysis
        print(f"\n🌍 Global Macro Analysis:")
        global_macro = enhanced.get('global_macro', {})
        macro_sentiment_display = global_macro.get('sentiment', 'N/A')
        macro_adj_display = f"{global_macro.get('adjustment', np.nan):+.2f}" if isinstance(global_macro.get('adjustment'), (int, float)) and not pd.isna(global_macro.get('adjustment')) else "N/A"

        print(f"  - Global Macro Sentiment: {macro_sentiment_display}")
        print(f"  - Global Macro Adjustment: {macro_adj_display}")


        # Sentiment Analysis Results
        sentiment_results = enhanced.get('sentiment_analysis', {})
        print(f"\n📰 News Sentiment Analysis:")
        if enhanced.get('news_headlines'):
             # Only print a few headlines to keep the report concise
             print("Sample News Headlines:")
             # Ensure headlines are valid strings before printing
             valid_headlines = [h for h in enhanced['news_headlines'] if isinstance(h, str)]
             for headline in valid_headlines[:5]: # Print up to 5 headlines
                 print(f"- {headline}")
             if len(valid_headlines) > 5:
                  print("  ...")

             print(f"Sentiment Counts: Positive={sentiment_results.get('positive', 0)}, Neutral={sentiment_results.get('neutral', 0)}, Negative={sentiment_results.get('negative', 0)}")
             sentiment_score_display = f"{sentiment_results.get('score', np.nan):.2f}" if isinstance(sentiment_results.get('score'), (int, float)) and not pd.isna(sentiment_results.get('score')) else "N/A"
             print(f"Sentiment Score: {sentiment_score_display}/25")
             print(f"Sentiment Summary: {sentiment_results.get('summary', 'N/A')}")
        else:
             print("No news headlines fetched or sentiment analysis skipped.")


        print(f"\n--- Comprehensive Scores and Recommendation ---")
        print(f"Fundamental Score (from Part 1): {basic.fundamental_score:.2f}/50" if not pd.isna(basic.fundamental_score) else "Fundamental Score: N/A")
        # Add checks for NaN before formatting float for breakdown scores
        enhanced_tech_score_display = f"{final.get('breakdown', {}).get('technical', np.nan):.2f}" if isinstance(final.get('breakdown', {}).get('technical'), (int, float)) and not pd.isna(final.get('breakdown', {}).get('technical')) else "N/A"
        liquidity_score_display_breakdown = f"{final.get('breakdown', {}).get('liquidity', np.nan):.2f}" if isinstance(final.get('breakdown', {}).get('liquidity'), (int, float)) and not pd.isna(final.get('breakdown', {}).get('liquidity')) else "N/A"
        sentiment_score_display_breakdown = f"{sentiment_results.get('score', np.nan):.2f}" if isinstance(sentiment_results.get('score'), (int, float)) and not pd.isna(sentiment_results.get('score')) else "N/A" # Corrected sentiment score breakdown check
        valuation_comp_display_breakdown = f"{final.get('breakdown', {}).get('valuation_comparison', np.nan):.2f}" if isinstance(final.get('breakdown', {}).get('valuation_comparison'), (int, float)) and not pd.isna(final.get('breakdown', {}).get('valuation_comparison')) else "N/A"
        downside_risk_display_breakdown = f"{final.get('breakdown', {}).get('downside_risk', np.nan):+.2f}" if isinstance(final.get('breakdown', {}).get('downside_risk'), (int, float)) and not pd.isna(final.get('breakdown', {}).get('downside_risk')) else "N/A"
        enhanced_dividend_display_breakdown = f"{final.get('breakdown', {}).get('enhanced_dividend', np.nan):.2f}" if isinstance(final.get('breakdown', {}).get('enhanced_dividend'), (int, float)) and not pd.isna(final.get('breakdown', {}).get('enhanced_dividend')) else "N/A"
        global_macro_display_breakdown = f"{final.get('breakdown', {}).get('global_macro_adjustment', np.nan):+.2f}" if isinstance(final.get('breakdown', {}).get('global_macro_adjustment'), (int, float)) and not pd.isna(final.get('breakdown', {}).get('global_macro_adjustment')) else "N/A" # Display global macro adjustment breakdown


        print(f"Technical Score (Enhanced): {enhanced_tech_score_display}/50")
        print(f"Liquidity Score: {liquidity_score_display_breakdown}/70")
        print(f"Sentiment Score: {sentiment_score_display_breakdown}/25")
        print(f"Valuation Comparison Adjustment: {valuation_comp_display_breakdown}")
        print(f"Downside Risk Adjustment: {downside_risk_display_breakdown}")
        print(f"Enhanced Dividend Score: {enhanced_dividend_display_breakdown}/20") # Display enhanced dividend score breakdown
        print(f"Global Macro Adjustment: {global_macro_display_breakdown}") # Display global macro adjustment breakdown


        # Display adjustment scores
        peer_adj_display = f"{final.get('breakdown', {}).get('peer_adjustment', np.nan):+.2f}" if isinstance(final.get('breakdown', {}).get('peer_adjustment'), (int, float)) and not pd.isna(final.get('breakdown', {}).get('peer_adjustment')) else "N/A"
        index_adj_display = f"{final.get('breakdown', {}).get('index_adjustment', np.nan):+.2f}" if isinstance(final.get('breakdown', {}).get('index_adjustment'), (int, float)) and not pd.isna(final.get('breakdown', {}).get('index_adjustment')) else "N/A"

        print(f"Peer Performance Adjustment: {peer_adj_display}")
        print(f"Index Membership Adjustment: {index_adj_display}")


        # Updated max score to reflect the addition of Enhanced Dividend Score (20 points) and Global Macro (10 points)
        # Max Total Score = Fund (50) + Tech (50) + Liq (70) + Sent (25) + Enhanced Dividend (20) + Val Adj (30) + Peer Adj (5) + Index Adj (5) + Downside Adj (0) + Global Macro Adj (10) = 265
        print(f"\nTotal Enhanced Score: {final.get('total_score', 0):.2f}/265" if isinstance(final.get('total_score'), (int, float)) and not pd.isna(final.get('total_score')) else "Total Enhanced Score: N/A")
        print(f"RECOMMENDATION: {final.get('recommendation', 'N/A')} (Confidence: {final.get('confidence', 'N/A')})")


        print(f"\n{'='*60}\n")

## Incorporate global macro impacts

### Subtask:
Implement a method within the `EnhancedStockAnalyzer` class to assess global macro impacts on the stock. This method should consider general economic indicators or news and provide an adjustment to the overall score.

## Refine Scoring and Recommendation

### Subtask:
Review the `generate_enhanced_recommendation` method in the `EnhancedStockAnalyzer` class. Adjust the weighting and logic for how the fundamental score, enhanced technical score, liquidity score, sentiment score, enhanced dividend score, valuation comparison adjustment, downside risk adjustment, peer performance adjustment, and index membership adjustment contribute to the total score and the final recommendation.